In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.optimize import minimize_scalar

from src.rent_vs_buy import (
    RentVsBuy,
    rent_vs_buy_breakeven_objective_closure,
    render_plotly_html,
    optimization_per_year,
    breakeven_per_year,
)

## Setup
We can use `RentVsBuy`'s `calculate` function to simulate a financial scenario:

In [2]:
# We will reuse these kwargs later on.
kwargs = dict(
    home_price=1_000_000,  # Value of the home
    financed_fees=0,  # Any fees that are financed (i.e. VA Loan Fee)
    years=20,  # Compute earnings over X-year period
    mortgage_rate=0.06342,  # Mortgage rate
    downpayment=0.2,  # Downpayment amount
    pmi=0.005,  # PMI expressed as a percentage of the total loan value
    length_of_mortgage=30,  # 30-year fixed mortgage
    home_price_growth_rate=0.08,  # Annual appreciation of the home
    rent_growth_rate=0.01,  # How much rent increases annually
    investment_return_rate=0.08,  # How much can we make in the market annually?
    investment_tax_rate=0.15,  # Long term capital gains tax
    inflation_rate=0.02,  # How much does inflation go up each year?
    filing_jointly=True,  # Tax filing status
    property_tax_rate=0.008,  # Annual tax rate on the home
    marginal_tax_rate=0.02,  # Tax bracket based on income
    costs_of_buying_home=0.01,  # How much do we pay at closing to buy?
    costs_of_selling_home=0.06,  # Seller fee
    maintenance_rate=0.005,  # How much we put away each year for maintenance (roofs, appliances, etc.)
    home_owners_insurance_rate=0.004,  # Insurance on the home.
    monthly_utilities=0,  # Utilities for the home (usually a wash with renting utilities)
    monthly_common_fees=350,  # HOA fees, etc.
    monthly_rent=3550,  # How much is the current rent?
    security_deposit=1,  # Proportion of monthly rent paid in the first month
    brokers_fee=0.00,  # Brokers fee (if applicable)
    renters_insurance_rate=0.01,  # Rental insurance
)
# Create our RentVsBuy calculator.
rent_vs_buy = RentVsBuy().calculate(**kwargs)
df = rent_vs_buy.df
df

annual_per  per_inv    home_value  first_month_home_value         ppmt  \
0             0      239  1.000000e+06            1.000000e+06   745.705185   
1             0      238  1.006667e+06            1.000000e+06   749.646237   
2             0      237  1.013378e+06            1.000000e+06   753.608117   
3             0      236  1.020134e+06            1.000000e+06   757.590936   
4             0      235  1.026935e+06            1.000000e+06   761.594804   
..          ...      ...           ...                     ...          ...   
235          19        4  4.765810e+06            4.549220e+06  2573.532960   
236          19        3  4.797582e+06            4.549220e+06  2587.134082   
237          19        2  4.829566e+06            4.549220e+06  2600.807086   
238          19        1  4.861763e+06            4.549220e+06  2614.552351   
239          19        0  4.894175e+06            4.549220e+06  2628.370260   

            ipmt          pmt  maintenance    insurance  property_taxes  ...  \
0    4228.000000  4973.705185   416.666667   333.333333      666.666667  ...   
1    4224.058948  4973.705185   416.666667   333.333333      666.666667  ...   
2    4220.097068  4973.705185   416.666667   333.333333      666.666667  ...   
3    4216.114249  4973.705185   416.666667   333.333333      666.666667  ...   
4    4212.110381  4973.705185   416.666667   333.333333      666.666667  ...   
..           ...          ...          ...          ...             ...  ...   
235  2400.172225  4973.705185  1895.508239  1516.406591     3032.813182  ...   
236  2386.571103  4973.705185  1895.508239  1516.406591     3032.813182  ...   
237  2372.898100  4973.705185  1895.508239  1516.406591     3032.813182  ...   
238  2359.152834  4973.705185  1895.508239  1516.406591     3032.813182  ...   
239  2345.334925  4973.705185  1895.508239  1516.406591     3032.813182  ...   

     total_rent_assets  home_opportunity_cost  rental_opportunity_cost  \
0                  0.0          209604.871852             0.000000e+00   
1                  0.0            3154.871852             0.000000e+00   
2                  0.0            3154.871852             0.000000e+00   
3                  0.0            3154.871852             0.000000e+00   
4                  0.0            3154.871852             0.000000e+00   
..                 ...                    ...                      ...   
235                0.0            7596.642466             0.000000e+00   
236                0.0            7596.642466             0.000000e+00   
237                0.0            7596.642466             0.000000e+00   
238                0.0            7596.642466             0.000000e+00   
239             3550.0               0.000000             4.148234e+06   

     home_opportunity_cost_fv  rental_opportunity_cost_fv  \
0                1.025843e+06               -0.000000e+00   
1                1.533824e+04               -0.000000e+00   
2                1.523666e+04               -0.000000e+00   
3                1.513576e+04               -0.000000e+00   
4                1.503552e+04               -0.000000e+00   
..                        ...                         ...   
235              7.801254e+03               -0.000000e+00   
236              7.749590e+03               -0.000000e+00   
237              7.698269e+03               -0.000000e+00   
238              7.647287e+03               -0.000000e+00   
239             -0.000000e+00                4.148234e+06   

     home_opportunity_cost_fv_post_tax  rental_opportunity_cost_fv_post_tax  \
0                        871966.474075                         0.000000e+00   
1                         13037.503844                         0.000000e+00   
2                         12951.162759                         0.000000e+00   
3                         12865.393470                         0.000000e+00   
4                         12780.192188                         0.000000e+00   

## Calculations
**High level overview**

First, model the cash flows over the life of the home. These cashflows should incorporate any inflation or appreciation that occurs with time.  For this we need:
- Value of the home
- Liabilities of the home
- Assets the home
- Liabilities of renting
- Assets of renting

Next, we can calculate the opportunity cost of either renting or buying. This is the difference in money (i.e. $8000 monthly house payment - $3000 monthly rent payment) that we can invest for the remaining time of the home, appreciated at market rates.

Below, we show the output of these various calculations for the given scenario.

### Home Value
How much will the home be worth when we sell it?

In [3]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Home Value": df.home_value,
                "Initial Home Price": df.home_value[0],
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="bd1f1af5-74c6-4755-bef2-dec6ef440a3a" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("bd1f1af5-74c6-4755-bef2-dec6ef440a3a")) {                    Plotly.newPlot(                        "bd1f1af5-74c6-4755-bef2-dec6ef440a3a",                        [{"hovertemplate":"variable=Home Value\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Home Value","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Home Value","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[1000000.0,1006666.6666666666,1013377.7777777778,1020133.6296296295,1026934.520493827,1033780.7506304524,1040672.6223013218,1047610.4397833305,1054594.509381886,1061625.139444432,1068702.6403740614,1075827.3246432217,1082999.5068075098,1090219.50351956,1097487.6335430234,1104804.2177666435,1112169.5792184211,1119584.043079877,1127047.9367004095,1134561.5896117457,1142125.3335424906,1149739.5024327736,1157404.432448992,1165120.461998652,1172887.9317453096,1180707.1846236116,1188578.5658544356,1196502.4229601317,1204479.1057798658,1212508.9664850647,1220592.3595949654,1228729.641992265,1236921.17293888,1245167.3140918058,1253468.4295190843,1261824.8857158783,1270237.0516206508,1278705.2986314548,1287230.0006223312,1295811.5339598134,1304450.2775195453,1313146.6127030088,1321900.9234543622,1330713.596277391,1339585.0202525735,1348515.5870542573,1357505.6909679524,1366555.7289077386,1375666.10043379,1384837.2077700153,1394069.4558218152,1403363.2521939606,1412719.007208587,1422137.1339233106,1431618.048149466,1441162.1684704623,1450769.9162602655,1460441.7157020005,1470177.9938066802,1479979.180432058,1489845.708301605,1499778.0130236156,1509776.5331104395,1519841.7099978423,1529973.9880644944,1540173.814651591,1550441.6400826015,1560777.9176831522,1571183.1038010397,1581657.6578263799,1592202.042211889,1602816.7224933014,1613502.1673099236,1624258.8484253227,1635087.240748158,1645987.8223531456,1656961.0745021666,1668007.4816655142,1679127.5315432844,1690321.715086906,1701590.5265208187,1712934.4633642908,1724354.0264533858,1735849.719963075,1747422.0514294954,1759071.5317723583,1770798.6753175072,1782603.999819624,1794488.026485088,1806451.2799949886,1818494.2885282882,1830617.5837851434,1842821.7010103776,1855107.1790171135,1867474.5602105607,1879924.3906119643,1892457.2198827106,1905073.6013485952,1917774.0920242523,1930559.252637747,1943429.647655332,1956385.8453063674,1969428.41760841,1982557.9403924658,1995774.9933284153,2009080.1599506047,2022474.0276836087,2035957.1878681658,2049530.235787287,2063193.7706925352,2076948.395830485,2090794.718469355,2104733.349925817,2118764.905591989,2132890.0049626026,2147109.271662353,2161423.333473435,2175832.822363258,2190338.374512346,2204940.630342428,2219640.2345447107,2234437.8361083423,2249334.0883490643,2264329.648938058,2279425.1799309785,2294621.3477971847,2309918.8234491656,2325318.28227216,2340820.4041539743,2356425.8735150006,2372135.3793384335,2387949.61520069,2403869.2793020275,2419895.074497374,2436027.7083273563,2452267.8930495386,2468616.345669869,2485073.7879743343,2501640.94656083,2518318.552871235,2535107.34322371,2552008.058845201,2569021.4459041688,2586148.25554353,2603389.2439138195,2620745.172206579,2638216.8066879553,2655804.918732542,2673510.284857425,2691333.6867564744,2709275.911334851,2727337.7507437496,2745520.0024153744,2763823.4690981433,2782248.958892131,2800797.285284745,2819469.2671866426,2838265.7289678873,2857187.5004943395,2876235.4171643015,2895410.319945397,2914713.055411699,2934144.4757811106,2953705.438952984,2973396.8085460034,2993219.45393631,3013174.2502958854,3033262.078631191,3053483.8258220656,3073840.384660879,3094332.6538919513,3114961.538251231,3135727.948506239,3156632.80149628,3177677.020172922,3198861.5336407414,3220187.2771983463,3241655.192379668,3263266.2269955324,3285021.3351755026,3306921.4774100054,3328967.6205927385,3351160.738063357,3373501.8096504454,3395991.8217147817,3418631.7671928797,3441422.645640832,3464365.4632784375,3487461.233033627,3510710.974587184,3534115.714417765,3557676.4858472166,3581394.329086198,3605270.2912801052,3629305.4265553057,3653500.7960656746,3677857.4680394456,3702376.5178263746,3727059.0279452167,3751906.088131518,3776918.795385728,3802098.2540216325,3827445.5757151097,3852961.8795532105,3878648.292083565,3904505.9473641217,3930535.9870132157,3956739.5602599704,3983117.823995036,4009671.9428216694,4036403.0891071474,4063312.443034528,4090401.1926547578,4117670.5339391227,4145121.67083205,4172755.8153042626,4200574.187406291,4228578.015322332,4256768.535424482,4285146.992327311,4313714.638942826,4342472.736535778,4371422.55477935,4400565.371811211,4429902.474289953,4459435.157451886,4489164.725168231,4519092.490002685,4549219.77326937,4579547.905091166,4610078.224458439,4640812.079288162,4671750.826483416,4702895.831993305,4734248.47087326,4765810.127345748,4797582.194861387,4829566.076160462,4861763.183334865,4894174.93789043],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Initial Home Price\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Initial Home Price","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Initial Home Price","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### House Liabilities
What will we need to pay to upkeep the home?

In [4]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Monthly Fees": df.monthly_common_fees,
                "Insurance": df.insurance,
                "Property Taxes": df.property_taxes,
                "Maintenance": df.maintenance,
                "Closing Costs": df.buying_closing_costs,
                "Mortgage Principal Payment": df.ppmt,
                "Mortgage Interest Payment": df.ipmt,
                "Total Mortgage Payment": df.pmt,
                "Utilities": df.home_monthly_utilities,
                "Downpayment": df.down_fee,
                "PMI": df.pmi,
                "Sellers Fee": df.sellers_fee,
                "Loan Payoff": df.loan_payoff,
            }
        ),
        log_y=True,
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="1fd10f13-0803-4e6d-b119-9e2ca353f6b5" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("1fd10f13-0803-4e6d-b119-9e2ca353f6b5")) {                    Plotly.newPlot(                        "1fd10f13-0803-4e6d-b119-9e2ca353f6b5",                        [{"hovertemplate":"variable=Monthly Fees\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Monthly Fees","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Monthly Fees","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Insurance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Insurance","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Insurance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Property Taxes\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Property Taxes","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Property Taxes","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Maintenance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Maintenance","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Maintenance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Closing Costs\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Closing Costs","line":{"color":"#FFA15A","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Closing Costs","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Mortgage Principal Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Mortgage Principal Payment","line":{"color":"#19d3f3","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Mortgage Principal Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[745.7051850946391,749.6462369978644,753.6081173603989,757.590936260648,761.594804358786,765.6198328998216,769.6661337166979,773.73381923339,777.8230024680379,781.9337970360821,786.0663171534179,790.2206776395733,794.3969939208991,798.59538203377,802.8159586278189,807.0588409691672,811.3241469436898,815.611995060287,819.92250445418,824.2557948902204,828.6119867662146,832.9912011162742,837.3935596141746,841.8191845767351,846.2681989672228,850.7407263987652,855.2368911377816,859.7568181074457,864.3006328911433,868.8684617359731,873.4604315562474,878.0766699370229,882.7173051376399,887.3824660952919,892.0722824286058,896.786884441241,901.5264031255128,906.290970166031,911.0807179433586,915.8957795376887,920.7362887325462,925.6023800184971,930.4941885968951,935.4118503836294,940.355502012907,945.3252808410448,950.3213249502901,955.3437731526524,960.3927649937646,965.4684407567561,970.5709414661551,975.7004088918038,980.8569855527967,986.0408147214439,991.2520404272473,996.4908074609043,1001.7572613783354,1007.0515485047204,1012.373815938568,1017.7242115558024,1023.1028840138752,1028.5099827558888,1033.9456580147535,1039.4100608173608,1044.9033429887809,1050.4256571564765,1055.9771567545486,1061.5579960279974,1067.1683300370041,1072.8083146612498,1078.478106604235,1084.1778633976378,1089.9077434056949,1095.667905829593,1101.4585107119033,1107.2797189410148,1113.1316922556193,1119.0145932491887,1124.9285853745118,1130.8738329482162,1136.8505011553466,1142.8587560539536,1148.8987645796974,1154.9706945505027,1161.074714671202,1167.210994538238,1173.3797046443729,1179.5810163834185,1185.8151020550054,1192.0821348693667,1198.3822889521507,1204.7157393492625,1211.0826620317234,1217.4832339005607,1223.9176327917262,1230.3860374810301,1236.8886276891167,1243.4255840864535,1249.9970882983498,1256.6033229100076,1263.2444714715862,1269.9207185033142,1276.6322495006034,1283.3792509392142,1290.1619102804275,1296.9804159762602,1303.8349574746949,1310.7257252249487,1317.6529106827625,1324.6167063157209,1331.617305608599,1338.6549030687406,1345.7296942314592,1352.8418756654728,1359.9916449783645,1367.179200822075,1374.40474289842,1381.6684719646382,1388.9705898389711,1396.3112994062699,1403.6908046236317,1411.109310526068,1418.5670232321972,1426.0641499499798,1433.6008989824659,1441.1774797335884,1448.7941027139796,1456.4509795468234,1464.1483229737287,1471.8863468606442,1479.6652662038032,1487.4852971356895,1495.3466569310522,1503.2495640129323,1511.1942379587417,1519.1808995063534,1527.2097705602446,1535.2810741976555,1543.3950346747893,1551.5518774330449,1559.7518291052802,1567.9951175221004,1576.2819717182056,1584.6126219387365,1592.9872996456816,1601.4062375243097,1609.8696694896253,1618.3778306928775,1626.930957528089,1635.5292876386247,1644.1730599237949,1652.8625145454935,1661.5978929348648,1670.3794377990262,1679.2073931277941,1688.0820042004743,1697.0035175926746,1705.9721811831514,1714.988244160705,1724.0519570310935,1733.163571624004,1742.323341100036,1751.5315199577494,1760.7883640407276,1770.0941305446813,1779.4490780246088,1788.8534664019699,1798.307556971905,1807.8116124105013,1817.365896782092,1826.9706755465845,1836.6262155668473,1846.3327851161184,1856.0906538854565,1865.9000929912422,1875.7613749826996,1885.6747738494828,1895.6405650292786,1905.659025415459,1915.7304333647785,1925.8550687051106,1936.033212743218,1946.265148272565,1956.5511595811863,1966.8915324595728,1977.2865542086229,1987.7365136476146,1998.24170112224,2008.8024085126722,2019.4189292416627,2030.0915582827047,2040.8205921682274,2051.606328997838,2062.44906844659,2073.349111773333,2084.306761829054,2095.32232306532,2106.3961015427208,2117.528404939374,2128.719542559479,2139.9698253419047,2151.2795658688374,2162.649078374455,2174.078678753664,2185.568684570876,2197.119415068832,2208.731191177472,2220.404335522842,2232.1391724360824,2243.936027962409,2255.795229870188,2267.7171076600516,2279.701992574037,2291.75021760479,2303.8621175048306,2316.0380287958433,2328.27828977803,2340.5832405395063,2352.953222965756,2365.3885807491306,2377.8896593983904,2390.4568062483118,2403.0903704693324,2415.790703077264,2428.5581569430274,2441.393086802471,2454.2958492662233,2467.2668028295916,2480.306307882549,2493.414726719708,2506.5924235504194,2519.8397645088826,2533.1571176643147,2546.544853031171,2560.0033425794404,2573.532960244971,2587.134081939868,2600.8070855629194,2614.5523510101198,2628.370260185205],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Mortgage Interest Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Mortgage Interest Payment","line":{"color":"#FF6692","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Mortgage Interest Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[4228.000000000001,4224.058948096776,4220.097067734241,4216.114248833992,4212.110380735854,4208.085352194818,4204.039051377942,4199.97136586125,4195.882182626602,4191.771388058558,4187.638867941222,4183.484507455067,4179.308191173741,4175.10980306087,4170.889226466821,4166.646344125473,4162.38103815095,4158.093190034353,4153.78268064046,4149.44939020442,4145.093198328425,4140.713983978366,4136.311625480465,4131.886000517905,4127.436986127417,4122.964458695875,4118.468293956858,4113.948366987194,4109.404552203497,4104.836723358667,4100.244753538393,4095.628515157617,4090.987879957,4086.322718999348,4081.632902666034,4076.918300653399,4072.178781969127,4067.414214928609,4062.6244671512814,4057.8094055569513,4052.968896362094,4048.102805076143,4043.210996497745,4038.2933347110106,4033.349683081733,4028.379904253595,4023.38386014435,4018.3614119419876,4013.3124201008754,4008.236744337884,4003.134243628485,3998.004776202836,3992.8481995418433,3987.664370373196,3982.4531446673927,3977.2143776337357,3971.9479237163046,3966.6536365899196,3961.331369156072,3955.9809735388376,3950.602301080765,3945.195202338751,3939.7595270798865,3934.2951242772792,3928.801842105859,3923.2795279381635,3917.7280283400914,3912.1471890666426,3906.536855057636,3900.89687043339,3895.227078490405,3889.527321697002,3883.797441688945,3878.037279265047,3872.2466743827367,3866.425466153625,3860.5734928390207,3854.6905918454513,3848.776599720128,3842.831352146424,3836.8546839392934,3830.8464290406864,3824.8064205149426,3818.7344905441373,3812.630470423438,3806.494190556402,3800.325480450267,3794.1241687112215,3787.8900830396346,3781.6230502252733,3775.3228961424893,3768.9894457453775,3762.6225230629166,3756.2219511940793,3749.787552302914,3743.31914761361,3736.8165574055233,3730.2796010081865,3723.70809679629,3717.1018621846324,3710.460713623054,3703.784466591326,3697.0729355940366,3690.325934155426,3683.5432748142125,3676.7247691183798,3669.870227619945,3662.9794598696913,3656.0522744118775,3649.088478778919,3642.087879486041,3635.0502820258994,3627.9754908631808,3620.863309429167,3613.7135401162755,3606.525984272565,3599.30044219622,3592.0367131300018,3584.734595255669,3577.39388568837,3570.0143804710083,3562.595874568572,3555.1381618624428,3547.6410351446602,3540.104286112174,3532.5277053610516,3524.9110823806604,3517.2542055478166,3509.5568621209113,3501.818838233996,3494.039918890837,3486.2198879589505,3478.358528163588,3470.4556210817077,3462.5109471358983,3454.5242855882866,3446.4954145343954,3438.4241108969845,3430.3101504198507,3422.153307661595,3413.95335598936,3405.7100675725396,3397.4232133764344,3389.0925631559035,3380.7178854489584,3372.2989475703303,3363.8355156050147,3355.3273544017625,3346.774227566551,3338.1758974560153,3329.532125170845,3320.8426705491465,3312.107292159775,3303.325747295614,3294.497791966846,3285.6231808941657,3276.7016675019654,3267.7330039114886,3258.716940933935,3249.6532280635465,3240.541613470636,3231.381843994604,3222.1736651368906,3212.9168210539124,3203.6110545499587,3194.2561070700312,3184.85171869267,3175.397628122735,3165.8935726841387,3156.339288312548,3146.7345095480555,3137.0789695277927,3127.3723999785216,3117.6145312091835,3107.805092103398,3097.9438101119404,3088.030411245157,3078.0646200653614,3068.046159679181,3057.9747517298615,3047.8501163895294,3037.671972351422,3027.440036822075,3017.1540255134537,3006.8136526350672,2996.418630886017,2985.9686714470254,2975.4634839724,2964.902776581968,2954.2862558529773,2943.6136268119353,2932.8845929264126,2922.098856096802,2911.25611664805,2900.356073321307,2889.398423265586,2878.38286202932,2867.3090835519192,2856.176780155266,2844.985642535161,2833.7353597527353,2822.4256192258026,2811.056106720185,2799.626506340976,2788.136500523764,2776.585770025808,2764.973993917168,2753.300849571798,2741.5660126585576,2729.769157132231,2717.909955224452,2705.9880774345884,2694.003192520603,2681.95496748985,2669.8430675898094,2657.6671562987967,2645.42689531661,2633.1219445551337,2620.751962128884,2608.3166043455094,2595.8155256962496,2583.2483788463283,2570.6148146253076,2557.914482017376,2545.1470281516126,2532.312098292169,2519.4093358284167,2506.4383822650484,2493.398877212091,2480.290458374932,2467.1127615442206,2453.8654205857574,2440.5480674303253,2427.160332063469,2413.7018425151996,2400.172224849669,2386.571103154772,2372.8980995317206,2359.1528340845202,2345.334924909435],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Total Mortgage Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Mortgage Payment","line":{"color":"#B6E880","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Total Mortgage Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Utilities\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Utilities","line":{"color":"#FF97FF","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Utilities","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Downpayment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Downpayment","line":{"color":"#FECB52","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Downpayment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=PMI\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"PMI","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"PMI","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Sellers Fee\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Sellers Fee","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Sellers Fee","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,293650.4962734258],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Loan Payoff\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Loan Payoff","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Loan Payoff","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,441143.6117472747],"yaxis":"y","type":"scattergl"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"},"type":"log"},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### House Assets
What money will the home bring in? This includes a tax credit for paying interest on a house (limited to 750K over the lifetime of the loan) and the sale of the house in the last month (shown on the log-scale).

In [5]:
render_plotly_html(px.line(pd.DataFrame({"Total Home Assets": df.total_home_assets}), log_y=True, markers=True))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="66b8df5d-c406-45a4-ba69-c6ba8559dce2" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("66b8df5d-c406-45a4-ba69-c6ba8559dce2")) {                    Plotly.newPlot(                        "66b8df5d-c406-45a4-ba69-c6ba8559dce2",                        [{"hovertemplate":"variable=Total Home Assets\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Home Assets","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Total Home Assets","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.95208893486053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.08610778693706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.13739075383549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.10131760279104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.97298205914564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.74717344170979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.418357105238048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.980653612252112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.427816551370036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.7532089138939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.9497779346481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.010028296925073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.925993594858495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.689205939581747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.290663588111238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7207964659921953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4894174.93789043],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"},"type":"log"},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Rent Liabilities
What will renting cost us? Mostly, this is the rent payment. But also includes some other expenses that do not occur when buying a home.

In [6]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Rent Payment": df.rent,
                "Renters Insurance": df.renters_insurance,
                "Security Deposit": df.security_deposit_cost,
                "Brokers Fee": df.brokers_fee_cost,
            }
        ),
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="17eef566-010b-42bc-a156-7a022a1f180a" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("17eef566-010b-42bc-a156-7a022a1f180a")) {                    Plotly.newPlot(                        "17eef566-010b-42bc-a156-7a022a1f180a",                        [{"hovertemplate":"variable=Rent Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Rent Payment","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Rent Payment","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Renters Insurance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Renters Insurance","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Renters Insurance","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Security Deposit\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Security Deposit","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Security Deposit","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[3550.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Brokers Fee\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Brokers Fee","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Brokers Fee","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Rent Assets
The only positive cash flow in renting is getting back the security deposit in the last month.

In [7]:
render_plotly_html(
    px.line(
        pd.DataFrame({"Rent Assets": df.total_rent_assets}),
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="fba00945-e3ad-4596-841e-cc83f3d0cb09" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("fba00945-e3ad-4596-841e-cc83f3d0cb09")) {                    Plotly.newPlot(                        "fba00945-e3ad-4596-841e-cc83f3d0cb09",                        [{"hovertemplate":"variable=Rent Assets\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Rent Assets","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Rent Assets","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3550.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Opportunity Costs
If we didn't buy the house, how much could we earn in the market after we pay the rent?

```python
opportunity_cost = (total_home_liability - total_home_assets) - (total_rent_liability - total_rent_assets)
```

We take the future value of each month's cash flow (to the last month's period) and then apply a capital gains tax penalty for selling the asset at the end of the period. Then we compute:

```python
buy_vs_rent = rental_cumulative_opportunity - home_cumulative_opportunity
```

In the last month to see the total advantage of owning a home vs. renting.

In [8]:
df[
    [
        "home_opportunity_cost",
        "rental_opportunity_cost",
        "home_opportunity_cost_fv",
        "rental_opportunity_cost_fv",
        "home_opportunity_cost_fv_post_tax",
        "rental_opportunity_cost_fv_post_tax",
        "home_cumulative_opportunity",
        "rental_cumulative_opportunity",
        "buy_vs_rent",
    ]
]

home_opportunity_cost  rental_opportunity_cost  home_opportunity_cost_fv  \
0            209604.871852             0.000000e+00              1.025843e+06   
1              3154.871852             0.000000e+00              1.533824e+04   
2              3154.871852             0.000000e+00              1.523666e+04   
3              3154.871852             0.000000e+00              1.513576e+04   
4              3154.871852             0.000000e+00              1.503552e+04   
..                     ...                      ...                       ...   
235            7596.642466             0.000000e+00              7.801254e+03   
236            7596.642466             0.000000e+00              7.749590e+03   
237            7596.642466             0.000000e+00              7.698269e+03   
238            7596.642466             0.000000e+00              7.647287e+03   
239               0.000000             4.148234e+06             -0.000000e+00   

     rental_opportunity_cost_fv  home_opportunity_cost_fv_post_tax  \
0                 -0.000000e+00                      871966.474075   
1                 -0.000000e+00                       13037.503844   
2                 -0.000000e+00                       12951.162759   
3                 -0.000000e+00                       12865.393470   
4                 -0.000000e+00                       12780.192188   
..                          ...                                ...   
235               -0.000000e+00                        7596.642466   
236               -0.000000e+00                        7596.642466   
237               -0.000000e+00                        7596.642466   
238               -0.000000e+00                        7596.642466   
239                4.148234e+06                           0.000000   

     rental_opportunity_cost_fv_post_tax  home_cumulative_opportunity  \
0                           0.000000e+00                 8.719665e+05   
1                           0.000000e+00                 8.850040e+05   
2                           0.000000e+00                 8.979551e+05   
3                           0.000000e+00                 9.108205e+05   
4                           0.000000e+00                 9.236007e+05   
..                                   ...                          ...   
235                         0.000000e+00                 2.971966e+06   
236                         0.000000e+00                 2.979562e+06   
237                         0.000000e+00                 2.987159e+06   
238                         0.000000e+00                 2.994755e+06   
239                         4.148234e+06                 2.994755e+06   

     rental_cumulative_opportunity   buy_vs_rent  
0                     0.000000e+00 -8.719665e+05  
1                     0.000000e+00 -8.850040e+05  
2                     0.000000e+00 -8.979551e+05  
3                     0.000000e+00 -9.108205e+05  
4                     0.000000e+00 -9.236007e+05  
..                             ...           ...  
235                   0.000000e+00 -2.971966e+06  
236                   0.000000e+00 -2.979562e+06  
237                   0.000000e+00 -2.987159e+06  
238                   0.000000e+00 -2.994755e+06  
239                   4.148234e+06  1.153479e+06  

[240 rows x 9 columns]

Giving a final opportunity of:

In [9]:
print(df.buy_vs_rent.iloc[-1])

1153478.706195381


## Optimize

In addition to visualizing the cost curves, we can also use `scipy.optimize.minimize_scalar` to answer questions. If we only wanted to stay in the house for 5 years, what would be the optimal mortgage rate to breakeven on our investment?

In [10]:
optimize_kwargs = kwargs.copy()
optimize_kwargs["years"] = 5

In [11]:
minimize_scalar(
    fun=rent_vs_buy_breakeven_objective_closure(
        scalar="mortgage_rate", **optimize_kwargs
    ),
    bounds=[0.0001, 0.1],
)

 message: Solution found.
 success: True
  status: 0
     fun: 8689.864875817322
       x: 0.09999339591456094
     nit: 20
    nfev: 20

We can also maximize the house advantage by optimizing the downpayment for a certain amount of years:

In [12]:
optimization_per_year(
    scalar="downpayment",
    bounds=[0.0, 1.0],
    start_year=1,
    end_year=21,
    kwargs=optimize_kwargs,
)

fun status success          message         x nfev nit
1     23103.309099      0    True  Solution found.  0.999994   25  25
2    127799.755015      0    True  Solution found.  0.999994   25  25
3    166019.824956      0    True  Solution found.  0.999994   25  25
4    195736.457907      0    True  Solution found.  0.999994   25  25
5    226646.980796      0    True  Solution found.  0.999994   25  25
6    258721.602081      0    True  Solution found.  0.999994   25  25
7    291917.176596      0    True  Solution found.  0.999994   25  25
8    326175.193242      0    True  Solution found.  0.999994   25  25
9    379172.628531      0    True  Solution found.  0.199254   25  25
10   438689.434209      0    True  Solution found.  0.199254   25  25
11   500626.654147      0    True  Solution found.  0.199257   26  26
12   564957.544662      0    True  Solution found.  0.199256   26  26
13   631632.981999      0    True  Solution found.  0.199257   26  26
14   700581.034804      0    True  Solution found.  0.199256   26  26
15   771700.204818      0    True  Solution found.  0.199255   25  25
16   844857.318232      0    True  Solution found.  0.199256   26  26
17   919884.791461      0    True  Solution found.  0.199257   26  26
18   996573.880403      0    True  Solution found.  0.199255   26  26
19  1074664.208949      0    True  Solution found.  0.199254   26  26
20  1153841.877013      0    True  Solution found.  0.199255   25  25

Or, we can see what home price growth rate we would need to breakeven with renting per year:

In [13]:
breakeven_per_year(
    scalar="home_price_growth_rate",
    bounds=[0.0001, 0.1],
    start_year=1,
    end_year=21,
    kwargs=kwargs,
)

fun status success          message         x nfev nit
1   8762.269232      0    True  Solution found.  0.099993   20  20
2      2.396666      0    True  Solution found.   0.06651   17  17
3      0.627588      0    True  Solution found.  0.057847   15  15
4      2.871965      0    True  Solution found.  0.054638   13  13
5      7.840696      0    True  Solution found.  0.053182   15  15
6      0.727825      0    True  Solution found.  0.052594   14  14
7      14.16125      0    True  Solution found.  0.052506   13  13
8      3.670759      0    True  Solution found.  0.052725   14  14
9      5.841816      0    True  Solution found.  0.053153   14  14
10     5.531079      0    True  Solution found.  0.053726   13  13
11     3.957799      0    True  Solution found.  0.054402   15  15
12    20.046186      0    True  Solution found.  0.055157   14  14
13    34.684645      0    True  Solution found.  0.055966   19  19
14    26.619315      0    True  Solution found.  0.056816   15  15
15    40.044397      0    True  Solution found.  0.057694   14  14
16     0.392994      0    True  Solution found.  0.058597   16  16
17     9.102877      0    True  Solution found.  0.059512   16  16
18    55.321394      0    True  Solution found.  0.060433   14  14
19     19.12799      0    True  Solution found.   0.06136   14  14
20    70.040728      0    True  Solution found.  0.062284   15  15

## Vectorize
Since `numpy-financial` is a vectorized library, we can actually see the breakdown for a range of values. For example, we can visualize a 2-d surface of the total opportunity.
### `home_price` and `downpayment`

In [14]:
_x, _y = np.linspace(100_000, 2_000_000, 100), np.linspace(0.00, 1.0, 100)
x, y = np.meshgrid(_x, _y, indexing="ij")
x = x.flatten()
y = y.flatten()
surface_kwargs = kwargs.copy()
surface_kwargs.update({"home_price": x, "downpayment": y})
render_plotly_html(
    px.imshow(
        RentVsBuy().calculate(**surface_kwargs).value.reshape((100, 100)).T,
        aspect="auto",
        labels=dict(x="Home Price", y="Downpayment", color="Opportunity"),
        x=_x,
        y=_y,
        title="Opportunity Surface",
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="07846f6e-203e-4457-9151-d279cf5d08ca" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("07846f6e-203e-4457-9151-d279cf5d08ca")) {                    Plotly.newPlot(                        "07846f6e-203e-4457-9151-d279cf5d08ca",                        [{"coloraxis":"coloraxis","name":"0","x":[100000.0,119191.9191919192,138383.8383838384,157575.75757575757,176767.67676767678,195959.59595959596,215151.51515151514,234343.43434343435,253535.35353535353,272727.2727272727,291919.1919191919,311111.1111111111,330303.0303030303,349494.9494949495,368686.8686868687,387878.78787878784,407070.70707070705,426262.62626262626,445454.5454545454,464646.4646464646,483838.38383838383,503030.303030303,522222.2222222222,541414.1414141414,560606.0606060605,579797.9797979798,598989.898989899,618181.8181818181,637373.7373737374,656565.6565656565,675757.5757575757,694949.494949495,714141.4141414141,733333.3333333333,752525.2525252525,771717.1717171717,790909.0909090908,810101.0101010101,829292.9292929292,848484.8484848484,867676.7676767677,886868.6868686868,906060.606060606,925252.5252525252,944444.4444444444,963636.3636363635,982828.2828282828,1002020.202020202,1021212.1212121211,1040404.0404040404,1059595.9595959596,1078787.878787879,1097979.797979798,1117171.717171717,1136363.6363636362,1155555.5555555555,1174747.4747474748,1193939.3939393938,1213131.313131313,1232323.2323232323,1251515.1515151514,1270707.0707070706,1289898.98989899,1309090.909090909,1328282.8282828282,1347474.7474747475,1366666.6666666665,1385858.5858585858,1405050.505050505,1424242.424242424,1443434.3434343433,1462626.2626262626,1481818.1818181816,1501010.101010101,1520202.0202020202,1539393.9393939392,1558585.8585858585,1577777.7777777778,1596969.6969696968,1616161.616161616,1635353.5353535353,1654545.4545454544,1673737.3737373736,1692929.292929293,1712121.212121212,1731313.1313131312,1750505.0505050505,1769696.9696969695,1788888.8888888888,1808080.808080808,1827272.727272727,1846464.6464646463,1865656.5656565656,1884848.4848484846,1904040.404040404,1923232.3232323232,1942424.2424242422,1961616.1616161615,1980808.0808080807,2000000.0],"y":[0.0,0.010101010101010102,0.020202020202020204,0.030303030303030304,0.04040404040404041,0.05050505050505051,0.06060606060606061,0.07070707070707072,0.08080808080808081,0.09090909090909091,0.10101010101010102,0.11111111111111112,0.12121212121212122,0.13131313131313133,0.14141414141414144,0.15151515151515152,0.16161616161616163,0.17171717171717174,0.18181818181818182,0.19191919191919193,0.20202020202020204,0.21212121212121213,0.22222222222222224,0.23232323232323235,0.24242424242424243,0.25252525252525254,0.26262626262626265,0.27272727272727276,0.2828282828282829,0.29292929292929293,0.30303030303030304,0.31313131313131315,0.32323232323232326,0.33333333333333337,0.3434343434343435,0.3535353535353536,0.36363636363636365,0.37373737373737376,0.38383838383838387,0.393939393939394,0.4040404040404041,0.4141414141414142,0.42424242424242425,0.43434343434343436,0.4444444444444445,0.4545454545454546,0.4646464646464647,0.4747474747474748,0.48484848484848486,0.494949494949495,0.5050505050505051,0.5151515151515152,0.5252525252525253,0.5353535353535354,0.5454545454545455,0.5555555555555556,0.5656565656565657,0.5757575757575758,0.5858585858585859,0.595959595959596,0.6060606060606061,0.6161616161616162,0.6262626262626263,0.6363636363636365,0.6464646464646465,0.6565656565656566,0.6666666666666667,0.6767676767676768,0.686868686868687,0.696969696969697,0.7070707070707072,0.7171717171717172,0.7272727272727273,0.7373737373737375,0.7474747474747475,0.7575757575757577,0.7676767676767677,0.7777777777777778,0.787878787878788,0.797979797979798,0.8080808080808082,0.8181818181818182,0.8282828282828284,0.8383838383838385,0.8484848484848485,0.8585858585858587,0.8686868686868687,0.8787878787878789,0.888888888888889,0.8989898989898991,0.9090909090909092,0.9191919191919192,0.9292929292929294,0.9393939393939394,0.9494949494949496,0.9595959595959597,0.9696969696969697,0.9797979797979799,0.98989898989899,1.0],"z":[[1677570.0126555362,1665012.4677415271,1652454.922827517,1639897.3779135055,1627339.8329994958,1614782.2880854853,1602224.743171474,1589667.1982574647,1577109.6533434538,1564552.108429443,1551994.5635154329,1539437.0186014227,1526879.4736874125,1514321.9287734018,1501764.383859391,1489206.8389453809,1476649.2940313704,1464099.0618160905,1451558.9953064616,1439026.467972908,1426500.3931898568,1413979.8041776377,1401463.842012509,1388951.7447764138,1376442.837741739,1363936.5244956962,1351432.278917486,1338930.6024347323,1326430.6118902497,1313931.5858511652,1301434.8942221487,1288938.5395265957,1276444.0122990515,1263949.615908466,1251456.9797839199,1238964.3436593693,1226472.9248546558,1213981.966715786,1201491.0085769147,1188993.7991611753,1176501.1630366212,1164008.5269120736,1151498.7400236093,1139004.212796065,1126509.6855685161,1113985.1387672867,1101488.4884036295,1088991.838039972,1076450.0911085722,1063951.0650694864,1051452.0390304038,1038953.0129913213,1026388.6903658588,1013887.0138831134,1001385.3374003549,988883.6609175978,976293.1250159782,963788.4992263014,951283.8734366414,938779.247646973,926274.6218573009,913650.5349251823,901142.6348490575,888634.7347729271,876126.834696807,863618.9346206933,851111.0345445676,838603.1344684372,825933.0006236006,813421.4729757672,800909.9453279227,788398.4176800922,775886.8900322439,763375.3623844106,750863.8347365726,738352.3070887215,725840.779440877,713109.1869999794,700593.6478179367,688078.1086358903,675562.5694538383,663047.0302717872,650531.4910897482,638015.9519077064,625500.4127256591,612984.8735436052,600469.3343615653,587661.9211490676,575141.9532244047,562621.9852997279,550102.0173750557,537582.0494503686,525062.0815256946,512542.11360103637,500022.1456763549,487502.1777516706,474982.2098269984,462462.2419023244,449942.2739776457,437422.30605297815],[1677513.4003917812,1664944.99049786,1652376.5806039397,1639808.170710015,1627239.7608160921,1614671.3509221696,1602102.9410282467,1589534.5311343237,1576966.1212404012,1564397.711346478,1551829.3014525548,1539260.8915586327,1526692.4816647095,1514124.0717707868,1501555.6618768636,1488987.2519829413,1476418.8420890183,1463855.0780044545,1451301.7933870465,1438756.739779772,1426218.1554853122,1413685.0787229172,1401156.6547555807,1388632.1251332783,1376110.817961669,1363592.1391018238,1351076.1137956344,1338562.4584247053,1326050.1494552563,1313539.6673365105,1301030.6746906436,1288522.5442506045,1276016.0346045592,1263510.0058548155,1251005.2500435729,1238501.0932947753,1225997.4553741652,1213494.959661695,1200992.463949224,1188489.9682367505,1175980.309550777,1163476.1528019817,1150955.7471071165,1138449.7183573726,1125943.689607629,1113408.7180900695,1100900.5876500322,1088392.4572099955,1075840.5070803985,1063330.0249616518,1050819.5428429046,1038309.0607241695,1025734.8136683158,1013221.7078780611,1000708.6020878083,988195.4962975467,975682.3905072883,963079.3293037373,950563.3039975436,938047.2786913561,925531.2533851666,913015.2280789744,900378.6902379775,887859.4237189274,875340.1571998876,862820.8906808468,850301.6241617994,837782.3576427521,825263.0911237029,812580.6824966203,800057.8250479987,787534.9675993705,775012.1101507507,762489.2527021281,749966.3952535018,737443.5378048867,724920.6803562641,712181.0893301051,699654.260867835,687127.4324055491,674600.6039432678,662073.775480995,649546.9470187007,637020.1185564203,624493.2900941446,611966.4616318587,599439.6331695747,586912.8047073036,574093.8432237953,561562.6307536336,549031.41828351,536500.2058133688,523968.9933432294,511437.78087310493,498906.56840296276,486375.3559328187,473844.1434626933,461312.93099255674,448781.71852241363,436250.50605227426],[1677458.4750543765,1664879.5239340847,1652300.5728137887,1639721.621693499,1627142.6705732048,1614563.7194529115,1601984.7683326185,1589405.8172123246,1576826.86609203,1564247.9149717374,1551668.9638514444,1539090.0127311507,1526511.0616108573,1513932.110490564,1501353.15937027,1488774.2082499769,1476195.2571296834,1463619.2011629846,1451052.5930103878,1438494.8498302787,1425944.0797779527,1413398.8390189228,1400858.2770029223,1388321.6387479524,1375788.25519295,1363257.9459845256,1350730.4334036738,1338204.742686655,1325680.531342441,1313158.8246009175,1300637.8546917408,1288118.5679288602,1275600.103949754,1263082.897431352,1250566.2148499992,1238050.861230488,1225535.5076109853,1213021.618414999,1200507.9088824666,1187994.1993499412,1175474.0931329522,1162958.7395134429,1150443.3858939377,1137910.8322473322,1125393.6257289257,1112876.4192105243,1100329.2664857055,1087809.9797228328,1075290.692959954,1062726.5357037098,1050204.9212588444,1037683.3068139898,1025161.6923691318,1012575.2477370733,1000051.0363928569,987526.8250486478,975002.6137044313,962390.300259388,949863.1991902185,937336.0981210489,924808.9970518798,912281.895982705,899754.7949135322,887106.1960644126,874575.8868559832,862045.5776475612,849515.2684391364,836984.9592307108,824454.6500222851,811924.3408138594,799230.0544423433,786696.1909464793,774162.3274506126,761628.4639547579,749094.600458893,736560.7369630188,724026.8734671641,711493.0099712983,698741.8136203354,686204.0196313541,673666.2256423747,661128.4316534093,648590.637664427,636052.8436754588,623515.0496864915,610977.2556975111,598439.4617085308,585901.6677195514,573363.8737305738,560533.7771991352,547991.6439370727,535449.5106750187,522907.37741295435,510365.2441508956,497823.11088884063,485280.97762678564,472738.8443647269,460196.7111026887,447654.5778406244,435112.4445785731],[1677476.0630779527,1664900.487436934,1652324.911795911,1639749.3361548905,1627173.76051387,1614598.184872849,1602022.609231827,1589447.0335908048,1576871.4579497841,1564295.8823087623,1551720.3066677414,1539144.7310267193,1526569.1553856984,1513993.5797446773,1501418.0041036555,1488842.4284626343,1476266.8528216132,1463692.4216784227,1451127.4410509022,1438570.6200144417,1426020.8515444295,1413477.3634944079,1400938.5801353818,1388403.7499533892,1375872.7971159266,1363344.7715284768,1350819.1007194077,1338295.2913619387,1325773.8911694102,1313254.0796899446,1300735.2496246097,1288218.5228430259,1275702.0212364965,1263187.5536550798,1250673.0860736605,1238160.2272246832,1225647.593440111,1213135.7086380627,1200624.701991117,1188113.695344171,1175602.6886972198,1163084.4245172418,1150571.79073267,1138042.8778409003,1125528.410259481,1113013.9426780636,1100470.627141925,1087954.1007618592,1075437.5743817808,1062877.4783832477,1050358.6483179098,1037839.8182525737,1025320.9881872432,1012738.8863561377,1000217.4861636027,987696.0859710719,975174.6857785401,962653.2855860111,950042.7766711586,937518.5165446438,924994.256418136,912469.9962916309,899945.7361651259,887421.476038618,874774.7990341932,862247.3638420366,849719.928649867,837192.4934577141,824665.0582655538,812137.6230733888,799448.9665926695,786918.0137551986,774387.060917737,761856.1080802735,749325.1552428165,736794.2024053549,724263.2495678915,711732.2967304243,699201.3438929562,686452.5399639364,673917.6971539026,661382.8543438809,648848.0115338536,636313.1687238198,623778.3259137999,611243.4831037708,598708.6402937444,586173.7974837301,573638.9546736935,561104.1118636727,548276.8864594735,535737.7491111215,523198.6117628021,510659.47441445384,498120.33706612885,485581.1997177927,473042.06236946303,460502.9250211334,447963.78767279163,435424.6503244657],[1677425.5670667505,1664840.3002720682,1652255.0334773806,1639669.766682694,1627084.4998880066,1614499.2330933218,1601913.9662986328,1589328.699503948,1576743.432709262,1564158.1659145756,1551572.8991198887,1538987.632325203,1526402.3655305165,1513817.0987358303,1501231.8319411438,1488646.5651464576,1476061.2983517712,1463476.031557085,1450899.0056637714,1438330.755784445,1425769.5314959716,1413214.3138222578,1400664.1965758875,1388118.3750444348,1375576.1357544984,1363036.8472142206,1350499.9515439984,1337965.4657272135,1325433.301239362,1312902.3183890174,1300373.2060702695,1287845.3281013593,1275318.495471153,1262792.9269571023,1250268.1316797298,1237744.1770313003,1225221.1964487236,1212698.2494076781,1200176.8790133782,1187655.5086190784,1175134.1382247768,1162606.2935358407,1150083.3129532607,1137560.3323706836,1125020.17890603,1112495.3836286562,1099970.5883512823,1087416.0060180551,1074889.1733878553,1062362.3407576499,1049790.9596953387,1037261.8473765901,1024732.7350578369,1012203.6227390915,999610.2537209685,987078.5980471848,974546.9423733973,962015.2866996112,949483.6310258289,936861.9195901798,924327.4337733961,911792.9479566179,899258.4621398384,886723.9763230495,874069.3626696551,861531.7348608263,848994.1070519891,836456.4792431528,823918.8514343128,811381.2236254802,798843.5958166448,786144.0215906026,773602.9127786113,761061.8039666135,748520.6951546129,735979.5863426151,723438.4775306117,710897.3687186111,698356.2599066133,685815.1510946248,673055.7539956709,660510.79573166,647965.8374676537,635420.8792036371,622875.9209396243,610330.962675604,597786.0044115912,585241.0461475737,572696.087883559,560151.1296195462,547606.1713555288,534768.8399151722,522219.631847634,509670.42378010415,497121.2157125473,484572.00764500257,472022.79957745597,459473.59150990844,446924.38344236,434375.1753748115],[1677450.3241908187,1664869.8087633816,1652289.2933359395,1639708.7779084966,1627128.2624810555,1614547.7470536153,1601967.2316261732,1589386.7161987335,1576806.2007712915,1564225.6853438504,1551645.169916409,1539064.6544889687,1526484.1390615278,1513903.6236340865,1501323.1082066454,1488742.5927792042,1476162.0773517634,1463581.5619243218,1451006.0988566312,1438440.552587013,1425882.3586867023,1413330.1935633072,1400783.1556009962,1388240.4438797198,1375701.3480301863,1363165.239057991,1350631.8611533924,1338101.2920613757,1325572.3550300715,1313044.8863959417,1300519.691278508,1287995.1046431803,1275472.4082837529,1262950.156584432,1250429.4761320231,1237908.8682050474,1225389.983345375,1212871.0984857054,1200353.1249654996,1187835.8333447585,1175318.541724014,1162801.2501032692,1150276.674187358,1137757.7893276913,1125222.6716078962,1112701.9911554847,1100181.310703067,1087631.9628923098,1075109.2665328868,1062586.5701734559,1050063.8738140278,1037495.6982023013,1024970.7461510506,1012445.7940997998,999920.8420485533,987330.7019086243,974803.2332744966,962275.7646403695,949748.2960062386,937132.6845046896,924602.4155185446,912072.1465323959,899541.8775462443,887011.6085600983,874481.3395739486,861830.1228740495,849296.7449694397,836763.3670648457,824229.9891602509,811696.6112556532,799163.233351049,786629.8554464513,773933.8792802887,761397.05701466,748860.2347490368,736323.4124834277,723786.5902178017,711249.7679521758,698712.9456865527,686176.1234209267,673639.3011553027,660883.83444844,648343.203251359,635802.5720542548,623261.9408571739,610721.3096600669,598180.6784629906,585640.0472659059,573099.4160688072,560558.7848717188,548018.1536746174,535477.5224775393,522644.61699148826,510099.7807256449,497554.9444598006,485010.1081939461,472465.2719280943,459920.4356622482,447375.59939640295,434830.7631305456],[1677555.332117627,1664994.9697266459,1652434.607335661,1639874.2449446756,1627313.8825536927,1614753.5201627102,1602193.1577717266,1589632.7953807425,1577072.4329897596,1564512.0705987767,1551951.708207793,1539391.3458168092,1526830.983425826,1514270.621034843,1501710.2586438595,1489149.8962528764,1476589.5338618928,1464029.1714709094,1451472.0319629177,1438923.7086103028,1426383.4700644214,1413849.7591605573,1401321.2021515165,1388797.00190963,1376276.451169711,1363759.3171196748,1351245.1971036622,1338733.214267013,1326222.9063614667,1313715.0264362071,1301208.4773658596,1288703.0872513214,1276199.4253138874,1263696.2668944483,1251194.7281402738,1238693.5641820312,1226193.481136415,1213694.0936688643,1201194.8851504293,1188697.0739724548,1176199.2627944811,1163701.4516165121,1151197.156331115,1138697.7688635653,1126198.381396017,1113681.9245996051,1101180.760641362,1088679.5966831162,1076148.890602856,1063645.7321834122,1051142.5737639735,1038595.2849605796,1026089.8948460459,1013584.504731508,1001079.114616971,988510.148379493,976002.2684542327,963494.3885289789,950986.5086037158,938478.6286784583,925881.7482191673,913371.0977328671,900860.4472465748,888349.796760276,875839.1462739808,863328.4957876848,850696.1438576737,838182.4175265199,825668.6911953678,813154.9648642195,800641.2385330694,788127.5122019192,775613.7858707672,762936.882717968,750419.7486679358,737902.6146178953,725385.480567866,712868.3465178227,700351.2124677999,687834.0784177687,675316.9443677254,662799.8103176914,650063.7567132208,637542.8542522667,625021.9517913116,612501.0493303519,599980.1468693977,587459.2444084352,574938.3419474866,562417.4394865269,549896.5370255644,537375.6345646121,524854.732103657,512041.74371070415,499516.6809157524,486991.6181208156,474466.5553258732,461941.4925309252,449416.4297359856,436891.3669410469],[1677590.6498892496,1665037.0656564527,1652483.481423663,1639929.8971908689,1627376.312958076,1614822.7287252825,1602269.14449249,1589715.560259697,1577161.9760269038,1564608.3917941097,1552054.8075613172,1539501.2233285238,1526947.6390957308,1514394.0548629372,1501840.470630144,1489286.886397351,1476733.3021645576,1464179.7179317651,1451627.5271052048,1439084.1620438164,1426548.4120894459,1414019.1984707885,1401495.5611678879,1388976.6470226664,1376461.6989811137,1363950.046362913,1351441.0960637885,1338934.324604246,1326429.9177395275,1313927.6048515297,1301426.3269499994,1288927.1238939043,1276428.7560842773,1263931.7607265597,1251435.5441322431,1238940.5217899038,1226445.7931546366,1213952.5282009388,1201459.263247245,1188967.004192546,1176475.298579073,1163983.5929656005,1151491.8873521276,1138992.9384787572,1126499.67352506,1113990.2983665052,1101495.2760241646,1089000.2536818236,1076476.8303607926,1063979.8350030738,1051482.839645356,1038985.8442876367,1026443.8597159311,1013944.6566598397,1001445.4536037426,988946.2505476521,976382.6202957965,963880.9542011423,951379.2881064885,938877.6220118329,926375.9559171726,913784.491203839,901280.0843391214,888775.6774744093,876271.2706096824,863766.8637449751,851262.4568802482,838635.6609890796,826128.2113530999,813620.7617171425,801113.3120811693,788605.8624452055,776098.4128092295,763590.9631732684,750919.8314398462,738409.0107271187,725898.1900144173,713387.3693016851,700876.5485889809,688365.72787626,675854.9071635511,663344.0864508236,650833.2657381175,638103.3313988652,625588.7827957859,613074.2341926796,600559.685589606,588045.1369865034,575530.5883834362,563016.0397803271,550501.4911772506,537986.9425741499,525472.3939710716,512957.8453679783,500151.48865938466,487632.8244570829,475114.1602547886,462595.496052484,450076.831850186,437558.1676478842],[1677710.2680745327,1665179.6408671965,1652649.0136598619,1640118.386452528,1627587.7592451945,1615057.132037859,1602526.5048305239,1589995.8776231897,1577465.2504158558,1564934.6232085214,1552403.996001186,1539873.3687938515,1527342.7415865173,1514812.1143791822,1502281.4871718476,1489750.8599645128,1477220.232757179,1464689.605549844,1452158.9783425094,1439636.3143690897,1427121.7182028196,1414613.6809452646,1402111.2475654646,1389613.5690495952,1377119.891741654,1364629.547701523,1352141.9459865608,1339656.951913043,1327174.5803955146,1314693.6708312952,1302214.397434508,1289737.005835469,1277260.1687050886,1264785.457592194,1252310.8041634643,1239838.040165834,1227365.3383066617,1214893.8812356058,1202422.9176630308,1189952.210649752,1177482.7894680514,1165013.3682863517,1152543.94710465,1140068.0998001467,1127597.1362275728,1115126.1726549943,1102638.319714977,1090165.6178558087,1077692.9159966363,1065191.0027325503,1052716.3493038202,1040241.6958750952,1027723.426097448,1015246.58896707,1002769.7518366887,990292.9147063037,977753.2891180608,965274.015721275,952794.7423244896,940315.4689277061,927836.1955309175,915269.0959918397,902787.1116159773,890305.127240112,877823.1428642422,865341.1584883798,852859.1741125053,840257.188972285,827772.1948987544,815287.2008252367,802802.2067517154,790317.2126781987,777832.2186046746,765347.2245311616,752862.230457644,740213.1222951431,727724.7937870156,715236.4652788835,702748.1367707597,690259.8082626332,677771.4797545057,665283.1512463745,652794.8227382451,640306.4942301223,627598.9390644664,615106.9231865099,602614.9073085627,590122.8914306089,577630.8755526599,565138.8596746996,552646.8437967533,540154.8279187968,527662.8120408468,515170.79616289306,502678.7802849356,490186.7644069893,477399.23685511574,464903.150112723,452407.0633703368,439910.9766279496],[1677838.4833937075,1665332.4631668166,1652826.4429399313,1640320.4227130448,1627814.4024861571,1615308.382259269,1602802.362032382,1590296.3418054965,1577790.3215786077,1565284.3013517212,1552778.2811248335,1540272.2608979463,1527766.240671059,1515260.220444172,1502754.2002172847,1490248.1799903975,1477742.1597635103,1465236.1395366224,1452730.1193097355,1440228.9274250353,1427736.6205498572,1415251.4096083967,1402771.830106683,1390297.0371751543,1377826.2805558187,1365358.895048744,1352894.708700194,1340433.4033168491,1327974.0389250778,1315516.1826396277,1303060.9518956998,1290606.633515576,1278153.812265897,1265702.2001063437,1253251.538561595,1240801.9041002635,1228353.1726792548,1215904.8797387509,1203457.8675022875,1191010.855265818,1178564.8756904774,1166119.3888955312,1153673.9021005915,1141228.415305649,1128775.7940834821,1116328.7818470118,1103865.8584691668,1091417.1270481516,1078968.3956271457,1066491.6153903706,1054040.9538456239,1041590.2923008772,1029139.6307561309,1016644.5660225963,1004191.7447729143,991738.9235232268,979286.1022735406,966769.7218096531,954314.4910657271,941859.260321802,929404.0295778741,916948.7988339486,904405.0332360314,891947.1213039979,879489.2093719598,867031.2974399393,854573.3855078947,842115.4735758686,829536.972517537,817076.083961457,804615.1954053771,792154.3068492943,779693.4182932219,767232.5297371363,754771.6411810583,742149.5776212132,729685.3912726594,717221.2049241196,704757.0185755575,692292.8322270075,679828.6458784575,667364.4595299112,654900.2731813611,642436.0868327944,629971.9004842583,617288.4554882646,604820.6222904492,592352.7890926087,579884.9558947897,567417.122696951,554949.2894991338,542481.4563013008,530013.6231034761,517545.7899056487,505077.9567078091,492610.1235099882,480142.29031214677,467379.4469407927,454907.58761328086,442435.7282857979],[1677975.4263747286,1665495.688134096,1653015.9498934683,1640536.2116528372,1628056.473412206,1615576.7351715758,1603096.9969309454,1590617.2586903144,1578137.520449684,1565657.7822090543,1553178.0439684237,1540698.3057277936,1528218.5674871632,1515738.8292465333,1503259.0910059027,1490779.3527652721,1478299.6145246418,1465819.8762840114,1453340.1380433813,1440863.3198471856,1428394.7414134475,1415933.2615253474,1403477.9904375605,1391027.7580961795,1378581.8011893951,1366139.6216593632,1353700.4760198859,1341263.8075393573,1328829.1252791542,1316396.960796768,1303966.4841930233,1291536.9711077663,1279109.7836744562,1266682.9454740551,1254257.9508190956,1241833.1208998687,1229410.0349228312,1216986.9489457891,1204565.1952241473,1192143.8093295987,1179722.6898071305,1167302.8124047532,1154882.9350023824,1142463.0576000093,1130036.8798568519,1117615.4939623047,1105194.1080677602,1092756.0891753915,1080333.003198353,1067909.9172213152,1055458.0363397477,1043033.0416847868,1030608.0470298245,1018183.0523748603,1005712.9156846874,993285.7853215178,980858.6549583469,968431.5245951791,955940.1178057687,943510.6047205101,931081.0916352505,918651.5785499886,906222.065464735,893703.3683957327,881271.2039133487,868839.0394309647,856406.8749485826,843974.7104661893,831542.54598381,818989.2701603789,806554.1621275488,794119.0540947132,781683.9460618896,769248.8380290456,756813.7299962174,744378.6219633836,731782.0168616772,719343.6476785168,706905.2784953378,694466.9093121784,682028.5401290143,669590.1709458502,657151.8017626889,644713.4325795295,632275.0633963635,619836.694213206,607179.1154337926,594737.1399218962,582295.1644100109,569853.1888980893,557411.2133862004,544969.2378743021,532527.2623624131,520085.2868505139,507643.31133862864,495201.33582672756,482759.36031483766,470317.3848029496,457580.9900874216,445135.0331806382],[1678208.8132605522,1665773.866442451,1653338.919624355,1640903.9728062549,1628469.0259881564,1616034.0791700578,1603599.1323519598,1591164.1855338602,1578729.2387157618,1566294.2918976636,1553859.3450795643,1541424.398261465,1528989.4514433667,1516554.504625268,1504119.5578071687,1491684.6109890705,1479249.6641709716,1466814.7173528732,1454379.7705347743,1441945.8354633579,1429520.1566724202,1417101.5994213324,1404689.1528640226,1392281.917684356,1379879.0949065248,1367479.9757724076,1355083.9325875237,1342690.4104460892,1330299.592372781,1317911.0121572614,1305523.743483395,1293138.4574930766,1280754.5009486119,1268371.570908225,1255990.1278147448,1243609.3094858453,1231229.6807771018,1218850.7496803044,1206472.3254292523,1194095.0753128934,1181717.8251965404,1169341.566706047,1156965.8081325185,1144590.0495589809,1132214.2909854557,1119831.5746147647,1107454.3244984136,1095077.0743820532,1082682.5076155537,1070303.576518761,1057924.645421959,1045516.2162235035,1033135.3978946102,1020754.579565715,1008330.0400601155,995947.1100197332,983564.1799793374,971181.2499389565,958798.3198985714,946350.4497642354,933965.1637739176,921579.8777835993,909194.5917932885,896809.3058029683,884334.2462811219,871946.3386846604,859558.4310881896,847170.523491729,834782.6158952694,822394.7082988108,809885.2332937634,797494.4152204376,785103.5971471351,772712.7790738251,760321.9610005245,747931.1429271959,735540.3248538887,722987.7609308725,710593.7183493711,698199.6757678725,685805.6331863608,673411.5906048734,661017.5480233543,648623.5054418584,636229.4628603542,623835.4202788388,611441.377697356,598828.2556118835,586430.6472221836,574033.0388325118,561635.4304428166,549237.8220531465,536840.213663457,524442.6052737581,512044.99688408803,499647.3884943798,487249.78010472097,474852.17171501834,462454.56332533434,449763.21617147885],[1678548.1532007589,1666178.3322297698,1653808.5112587817,1641438.690287793,1629068.8693168045,1616699.0483458166,1604329.227374826,1591959.40640384,1579589.5854328512,1567219.764461864,1554849.943490875,1542480.1225198857,1530110.3015488978,1517740.4805779096,1505370.659606921,1493000.838635933,1480631.0176649443,1468261.1966939566,1455891.3757229678,1443521.5547519794,1431158.213477351,1418802.9132878534,1406453.7522280249,1394109.8355076788,1381770.3678679275,1369434.643554305,1357102.0372485938,1344772.6556933292,1332445.8021816232,1320120.7148849233,1307797.398598964,1295476.2640947127,1283155.8728636696,1270837.477570647,1258519.6283004486,1246203.3207221946,1233887.3189705906,1221572.8771786168,1209258.4353866465,1196945.3369091526,1184632.5569955679,1172319.9854694456,1160008.6801493363,1147697.3748292332,1135386.0695091207,1123074.7641890086,1110755.8775140685,1098443.0976004861,1086114.0174669148,1073799.575674951,1061485.133882971,1049142.39947013,1036826.091891876,1024509.7843136224,1012193.4767353651,999832.7811241499,987514.3858311293,975195.9905381091,962877.5952450763,950495.9789124853,938175.2544416934,925854.5299708927,913533.8055001022,901213.0810293062,888804.6448763097,876481.3285903493,864158.0123043777,851834.6960184276,839511.3797324672,827188.0634464994,814864.747160539,802419.4735456454,790093.2798564415,777767.0861672265,765440.8924780199,753114.6987888133,740788.5050996104,728462.3114103982,715974.1963749146,703644.8148196572,691315.4332643868,678986.0517091248,666656.6701538628,654327.2885986026,641997.9070433443,629668.5254880814,617339.1439328203,605009.7623775387,592461.5124517009,580128.6056088042,567795.698765899,555462.7919230126,543129.8850801159,530796.978237222,518464.0713943131,506131.1645514164,493798.2577085076,481465.3508656211,469132.4440227188,456799.5371798277],[1678814.4072519273,1666495.6855432857,1654176.9638346408,1641858.2421260006,1629539.5204173569,1617220.7987087134,1604902.077000072,1592583.3552914304,1580264.6335827855,1567945.9118741439,1555627.1901655004,1543308.4684568585,1530989.7467482162,1518671.025039574,1506352.3033309306,1494033.581622288,1481714.8599136458,1469396.1382050028,1457077.41649636,1444758.6947877177,1432443.946835746,1420136.8940928755,1407836.6808381665,1395542.0559923772,1383251.9169059433,1370965.8378344995,1358683.3289471723,1346403.6177029533,1334126.1817280166,1321850.5858381425,1309577.8092051498,1297306.1606796097,1285035.971143571,1272767.4210467974,1260499.5339163747,1248233.315602745,1235967.4921903631,1223702.9351516026,1211438.9560796851,1199175.4758105418,1186913.1395149701,1174650.8032193924,1162389.349717006,1150128.4710655566,1137867.5924141053,1125606.7137626573,1113339.121741517,1101076.7854459449,1088814.449150369,1076535.1862886264,1064271.207216707,1052007.2281447975,1039714.3175923447,1027448.4941799645,1015182.6707675881,1002916.8473552158,990606.0170470513,978338.1299166298,966070.242786211,953802.3556557852,941470.6641345434,929200.4745985139,916930.2850624751,904660.0955264335,892389.9059904022,880031.5045973007,867758.7530370699,855486.0014768532,843213.2499166336,830940.4983563982,818667.7467961786,806275.5584635232,793999.9625736438,781724.3666837774,769448.7707939092,757173.174904041,744897.579014156,732621.9831242915,720346.3872344242,707908.7677859925,695630.0206722058,683351.273558422,671072.526444627,658793.7793308441,646515.0322170574,634236.2851032652,621957.5379894832,609678.7908757031,597400.0437618978,584902.7204520181,572620.4885711335,560338.2566902544,548056.0248093894,535773.7929285141,523491.56104762666,511209.3291667644,498927.0972858863,486644.8654050054,474362.63352413476,462080.4016432604],[1679194.680754612,1666948.9408293136,1654703.2009040134,1642457.460978714,1630211.7210534161,1617965.9811281161,1605720.2412028178,1593474.5012775194,1581228.7613522194,1568983.0214269205,1556737.2815016205,1544491.5415763217,1532245.8016510233,1520000.0617257245,1507754.3218004247,1495508.5818751256,1483262.8419498266,1471017.1020245273,1458771.3620992284,1446525.6221739296,1434281.868886556,1422045.8342345278,1409816.4491132298,1397592.7616480868,1385373.9254595302,1373159.18904577,1360947.886180394,1348739.4272310347,1336533.2913138645,1324330.1757443259,1312128.7148927834,1299928.434866951,1287730.6627866677,1275533.5071673892,1263338.0107205643,1251143.218169729,1238949.7614442261,1226756.665853017,1214565.0320221586,1202373.3981912984,1190183.0423257947,1177993.0321692298,1165803.1046180995,1153614.5351563017,1141425.9656945113,1129237.3962327144,1117048.8267709156,1104852.9712298084,1092662.961073243,1080457.0598827135,1068265.4260518542,1056073.7922209962,1043882.158390135,1031660.9971107054,1019467.5403852109,1007274.0836597034,995037.0888903751,982841.5924435463,970646.0959967184,958450.5995498952,946255.1031030668,933995.2724585067,921797.5003782157,909599.7282979386,897401.9562176596,885204.1841373816,872917.7596071335,860717.4552936405,848517.1509801475,836316.846666663,824116.5423531728,811916.2380396752,799596.2062542159,787393.090684684,775189.9751151428,762986.8595456108,750783.7439760631,738580.6284065247,726377.5128369955,714174.3972674599,701809.2292113611,689602.9990600413,677396.7689087326,665190.5387574229,652984.3086061124,640778.0784547972,628571.8483034782,616365.6181521574,604159.3880008506,591953.1578495353,579528.7247176832,567319.0503198365,555109.3759219637,542899.7015241161,530690.0271262666,518480.3527284097,506270.67833056394,494061.0039326893,481851.329534838,469641.65513697825],[1679701.8129237592,1667553.4013945577,1655404.9898653561,1643256.578336155,1631108.1668069544,1618959.7552777533,1606811.3437485546,1594662.9322193528,1582514.520690153,1570366.1091609513,1558217.6976317496,1546069.2861025503,1533920.8745733502,1521772.4630441498,1509624.0515149485,1497475.6399857476,1485327.2284565475,1473178.8169273469,1461030.4053981458,1448881.9938689456,1436733.5823397455,1424592.9119104943,1412458.9208516167,1400330.661747668,1388207.2922819876,1376088.0642423765,1363972.3140117063,1351859.5437096297,1339750.0873938776,1327642.6760868628,1315536.8776295274,1303433.3671755248,1291331.3047043858,1279230.2966926133,1267131.1310248822,1255032.3718744074,1242935.3674237258,1230838.6239978592,1218743.1379157742,1206648.1959387306,1194553.6683132867,1182460.330908474,1170366.9935036711,1158274.362261976,1146182.4486025893,1134090.5349432034,1121998.621283819,1109900.3064796194,1097806.969074816,1085713.6316699986,1073603.8341912078,1061508.8922141637,1049413.9502371112,1037290.7291839286,1025193.9857580494,1013097.242332181,1001000.4989063074,988859.6705639567,976760.9114134805,964662.1522630067,952563.3931125365,940464.6339620622,928301.0643512392,916200.0563394642,904099.0483276984,891998.0403159279,879897.0323041519,867706.9908314804,855603.4803774739,843499.9699234758,831396.459469472,819292.949015473,807189.4385614758,794965.9760831529,782859.687446706,770753.3988102535,758647.1101738177,746540.8215373689,734434.5329009155,722328.2442644723,710221.9556280291,697953.6588612907,685844.2922852049,673734.9257091191,661625.5591330379,649516.1925569447,637406.8259808524,625297.4594047815,613188.0928286891,601078.7262525959,588969.3596765297,576642.2443765523,564529.4740744745,552416.7037723986,540303.9334703237,528191.1631682394,516078.39286615327,503965.6225640774,491852.8522619968,479740.0819599191],[1680351.7645422039,1668328.0912024016,1656304.4178626002,1644280.744522796,1632257.0711829937,1620233.397843191,1608209.724503389,1596186.0511635882,1584162.377823785,1572138.7044839829,1560115.0311441808,1548091.3578043801,1536067.6844645776,1524044.0111247757,1512020.3377849732,1499996.6644451716,1487972.9911053698,1475949.3177655677,1463925.6444257651,1451901.9710859635,1439878.2977461615,1427859.178949303,1415847.2522575413,1403841.8065424522,1391841.3135929494,1379845.0512211812,1367852.8343215345,1355863.7595051227,1343877.2713697334,1331892.8798844027,1319910.827974343,1307930.84681336,1295951.8794570034,1283975.0453070418,1271999.0037507028,1260024.391689884,1248050.5454113332,1236077.9250783925,1224105.6354041034,1212134.7950741611,1200163.9547442282,1188194.250622326,1176224.9957625782,1164255.7409028327,1152287.7818651763,1140319.9338014992,1128352.0857378235,1116384.2376741488,1104409.4666040875,1092440.2117443383,1080470.956884595,1068484.7111149249,1056513.8707849933,1044543.0304550501,1032543.3794184062,1020570.7590854578,1008598.1387525215,996625.5184195791,984608.3142909487,972633.7022301266,960659.090169304,948684.4781084768,936646.8549581645,924670.0208081938,912693.186658239,900716.3525082804,888739.5183583153,876762.6842083605,864696.4951685071,852717.1883673063,840737.8815660747,828758.574764871,816779.2679636506,804799.9611624507,792700.5439317767,780718.492021719,768736.4401116734,756754.3882016037,744772.3362915423,732790.2843814902,720808.2324714428,708826.1805613749,696682.2381615769,684697.1449540257,672712.0517464764,660726.958538915,648741.8653313434,636756.7721237894,624771.6789162401,612786.5857086815,600801.4925011126,588816.3992935531,576614.0926598851,564625.6362468842,552637.1798338788,540648.7234208705,528660.2670078753,516671.81059487443,504683.35418186337,492694.897768856],[1681048.1249209333,1669158.096502302,1657268.0680836695,1645378.039665036,1633488.0112464044,1621597.982827771,1609707.9544091397,1597817.9259905084,1585927.8975718766,1574037.869153244,1562147.8407346127,1550257.812315981,1538367.7838973498,1526477.7554787176,1514587.7270600866,1502697.698641454,1490807.6702228226,1478917.6418041904,1467027.613385559,1455137.5849669275,1443247.5565482941,1431360.016732867,1419479.699357007,1407605.5976904104,1395736.8158196588,1383872.557523489,1372012.1162101396,1360154.8658187303,1348300.2525954803,1336448.3394882225,1324598.8792117042,1312750.8438964013,1300904.647479751,1289060.1248302213,1277216.371273842,1265374.555877648,1253532.9976382495,1241693.1839947621,1229853.593486519,1218015.1655089809,1206177.333557922,1194339.7550991056,1182503.4895161847,1170667.2239332707,1158831.4200859936,1146996.5443498031,1135161.6686136103,1123326.792877417,1111491.9171412191,1099649.6304357727,1087813.3648528522,1075961.1820963346,1064123.3501452822,1052285.5181942265,1040447.6862431676,1028580.5548464688,1016740.9643382262,1004901.3738299944,993061.7833217499,981177.1581320581,969335.5998926535,957494.0416532569,945652.4834138528,933747.518138825,921903.7645824505,910060.0110260779,898216.2574696885,886372.5039133094,874528.750356934,862595.3800636474,850749.1836469928,838902.9872303326,827056.7908136882,815210.5943970326,803364.3979803845,791397.9988763463,779549.0904244427,767700.1819725428,755851.2735206503,744002.3650687691,732153.4566168636,720304.5481649525,708455.6397130582,696445.031696273,684593.1185890092,672741.2054817481,660889.2923744824,649037.3792672344,637185.4661599761,625333.5530527113,613481.6399454419,601629.7268381976,589777.8137309458,577709.3035364151,565854.0677442588,553998.8319521071,542143.5961599601,530288.3603678001,518433.1245756345,506577.88878348283],[1681915.6773638288,1670192.1489089823,1658468.6204541363,1646745.0919992942,1635021.563544451,1623298.035089606,1611574.5066347637,1599850.9781799188,1588127.4497250754,1576403.9212702324,1564680.3928153869,1552956.8643605434,1541233.3359056998,1529509.8074508556,1517786.278996012,1506062.7505411678,1494339.222086324,1482615.6936314797,1470892.165176636,1459168.6367217926,1447445.1082669483,1435722.0024755676,1424006.1494582272,1412296.547873582,1400592.306246441,1388892.631955407,1377196.8212709702,1365504.2503471843,1353815.090218493,1342128.4891143455,1330443.773791765,1318760.863840647,1307080.308375283,1295400.612823948,1283722.794903767,1272045.838621197,1260370.189245839,1248695.271552151,1237021.5659112353,1225348.0746700452,1213676.106140498,1202004.1376109417,1190333.0871239007,1178662.6658604364,1166992.2445969707,1155322.8238800736,1143653.775513973,1131984.7271478823,1120315.6787817879,1108640.1382796364,1096969.7170161693,1085299.2957527074,1073612.483785851,1061940.5152563048,1050268.546726747,1038568.5700156111,1026894.8643746944,1015221.1587337716,1003547.4530928563,991830.2536125905,980154.6042372333,968478.9548618766,956803.3054865226,945127.6561111631,933388.2572983634,921710.4393781899,910032.6214579986,898354.8035378158,886676.9856176451,874999.1676974539,863231.5307753077,851551.2997858273,839871.0687963627,828190.8378068823,816510.6068174113,804830.3758279383,793029.916040332,781347.0060892198,769664.0961381001,757981.1861869739,746298.2762358589,734615.3662847523,722932.4563336158,711249.5463825073,699405.2010755977,687719.3231112519,676033.4451469099,664347.5671825735,652661.6892182389,640975.8112538802,629289.9332895437,617604.0553252092,605918.1773608681,594232.2993965261,582546.4214321887,570641.3615961848,558952.2014674926,547263.041338807,535573.881210113,523884.7210814217],[1682979.7600646361,1671460.449501863,1659941.1389390945,1648421.8283763258,1636902.5178135566,1625383.207250785,1613863.8966880175,1602344.5861252465,1590825.2755624773,1579305.9649997079,1567786.6544369378,1556267.3438741686,1544748.0333113987,1533228.7227486295,1521709.4121858599,1510190.10162309,1498670.7910603203,1487151.4804975507,1475632.169934781,1464112.8593720105,1452593.5488092427,1441074.2382464712,1429559.5900869542,1418051.6723778844,1406549.5572100081,1395052.3240228707,1383559.2944997794,1372069.7774062962,1360583.1826603415,1349098.988918689,1337616.8640255504,1326137.5429562747,1314659.4732338912,1303182.925599885,1291708.1583533431,1280233.9939976484,1268761.756529653,1257289.733946622,1245819.3498687306,1234349.2470234265,1222880.1261124332,1211411.7389326626,1199943.3679775586,1188476.5089618224,1177009.649946094,1165542.9404411386,1154077.4373734244,1142611.93430572,1131146.4312380143,1119680.928170308,1108208.4958517137,1096741.6368359877,1085274.7778202542,1073791.0927754599,1062322.7055957043,1050854.3184159282,1039357.4986531003,1027887.395807792,1016417.2929624794,1004947.1901171706,993433.21453691,981961.1919538742,970489.1693708412,959017.1467878157,947545.124204793,936009.0633268254,924534.8989711255,913060.7346154256,901586.5702597396,890112.4059040351,878550.6631415784,867074.1155075673,855597.5678735562,844121.0202395506,832644.4726055395,821167.9249715256,809691.3773375247,798094.6409418415,786615.4474198017,775136.253897747,763657.0603756886,752177.8668536339,740698.6733315857,729219.4798095394,717740.2862874875,706099.9949030103,694617.87000986,683135.7451167284,671653.6202236097,660171.4953304622,648689.3704373306,637207.2455441905,625725.1206510486,614242.995757917,602760.8708647853,591278.7459716443,579578.2581484308,568092.891611496,556607.5250745583,545122.1585376104],[1683850.8302147805,1672498.694731328,1661146.5592478814,1649794.4237644325,1638442.2882809832,1627090.152797534,1615738.017314085,1604385.881830638,1593033.7463471883,1581681.6108637408,1570329.4753802908,1558977.339896842,1547625.2044133923,1536273.0689299447,1524920.9334464953,1513568.797963046,1502216.662479598,1490864.5269961488,1479512.3915127,1468160.2560292506,1456808.1205458022,1445455.9850623533,1434106.3672214225,1422763.5170736364,1411426.4870013008,1400094.4340418153,1388766.6093539598,1377442.3486920018,1366121.0637944287,1354802.4480364844,1343486.8199678299,1332172.8823258786,1320860.362420138,1309550.348442431,1298241.0536186374,1286933.5000146772,1275626.7700036753,1264321.3314002212,1253016.494390945,1241712.9515284807,1230409.4210235616,1219107.5723808352,1207805.7237381,1196504.4985641516,1185204.1589834034,1173903.8194026598,1162604.1172955576,1151305.1167134782,1140006.1161314053,1128707.1155493339,1117408.1149672563,1106101.7819181718,1094801.44233743,1083501.1027566805,1072183.5400252938,1060881.6913825665,1049579.8427398284,1038249.1800041986,1026945.6371417423,1015642.0942792795,1004338.5514168157,992990.8049172498,981685.3663137984,970379.9277103357,959074.4891068777,947769.0505034272,936399.3382918555,925091.784687887,913784.231083936,902476.6774799721,891169.1238760073,879773.8789731497,868463.9718818394,857154.0647905413,845844.1576992413,834534.2506079515,823224.3435166478,811914.4364253515,800484.4467560602,789171.9268503124,777859.4069445776,766546.8870388363,755234.3671330959,743921.8472273517,732609.3273216151,721296.8074158775,709984.2875101259,698508.1857906887,687192.7711559907,675877.3565213084,664561.9418866262,653246.527251929,641931.1126172543,630615.6979825674,619300.283347873,607984.8687131861,596669.4540785132,585136.5764964251,573817.9607384931,562499.3449805602],[1683362.224282104,1671916.3159428872,1660470.4076036753,1649024.4992644568,1637578.5909252428,1626132.6825860285,1614686.774246813,1603240.8659075978,1591794.9575683826,1580349.0492291693,1568903.1408899545,1557457.2325507393,1546011.3242115232,1534565.4158723087,1523119.507533094,1511673.599193879,1500227.690854664,1488781.7825154494,1477335.874176234,1465889.9658370186,1454444.0574978034,1442998.1491585886,1431552.613701158,1420113.8831795673,1408681.0148214707,1397253.1696007692,1385829.6018130637,1374409.649647541,1362993.3474211374,1351579.9137492299,1340168.6617746828,1328759.1596249721,1317352.3654004596,1305946.8133442893,1294542.598860613,1283140.152220841,1271738.261433521,1260338.1791727855,1248938.3766138125,1237540.0104823331,1226142.0796676418,1214744.8644076162,1203348.6063668285,1191952.348326041,1180557.2827940765,1169162.514713219,1157767.7466323725,1146374.0700893337,1134980.6240578024,1123587.1780262776,1112193.7319947476,1100793.9062281116,1089399.1381472568,1078004.3700664141,1066593.509877386,1055197.2518365937,1043800.9937958084,1032404.7357550259,1020979.3250031387,1009581.3941884413,998183.4633737495,986785.5325590465,975343.1151985405,963943.312639568,952543.5100806067,941143.7075216314,929743.904962657,918279.6470267139,906877.7562393891,895475.86545208,884073.974664757,872672.0838774396,861182.448481054,849778.2339973757,838374.0195137039,826969.8050300302,815565.5905463602,804161.3760626912,792757.1615790138,781233.0368482042,769826.2426236849,758419.4483991833,747012.6541746585,735605.8599501466,724199.0657256274,712792.2715011071,701385.4772765907,689978.683052076,678408.8277179189,666999.1754066041,655589.5230952948,644179.8707839698,632770.2184726391,621360.5661613205,609950.9138499908,598541.2615386555,587131.6092273332,575721.9569160277,564095.8226810945,552683.009767076],[1682873.618349429,1671333.9371544456,1659794.2559594673,1648254.5747644845,1636714.8935695025,1625175.2123745217,1613635.5311795403,1602095.84998456,1590556.1687895786,1579016.4875945982,1567476.8063996176,1555937.1252046353,1544397.444009654,1532857.7628146738,1521318.0816196925,1509778.4004247119,1498238.7192297308,1486699.0380347488,1475159.356839768,1463619.6756447868,1452079.9944498066,1440540.3132548255,1429000.6320598437,1417465.2482024585,1405936.1745902258,1394412.5151546374,1382893.4734387863,1371378.3425294533,1359866.4959553613,1348357.379461976,1336850.5035815325,1325346.6135935723,1313844.3683807775,1302343.299837482,1290844.7563700979,1279346.8044270044,1267850.5764563342,1256355.0269453493,1244860.8604308679,1233367.069436186,1221874.7506692545,1210382.431902322,1198891.4889955604,1187400.8215567246,1175910.4066047394,1164421.2100237831,1152932.0134428265,1141443.023465192,1129955.1319842027,1118467.2405032222,1106979.3490222455,1095491.4575412567,1083996.8339570956,1072507.6373761361,1061018.4407951809,1049512.812290636,1038022.1448517917,1026531.4774129479,1015040.8099741037,1003520.694097606,992028.3753306838,980536.0565637564,969043.7377968207,957506.6975688059,946012.5310543105,934518.3645398328,923024.1980253533,911530.0315108718,899971.2813948607,888475.0534241898,876978.8254535189,865482.5974828452,853986.3695121752,842402.4032042027,830903.8813281646,819405.3594521172,807906.8375760671,796408.3157000095,784909.7938239602,773411.2719479268,761793.0783028053,750292.0097595183,738790.9412162257,727289.8726729266,715788.8041296415,704287.7355863424,692786.6670430563,681285.5984997777,669784.5299564721,658120.99429191,646617.1043039318,635113.2143159835,623609.3243280342,612105.4343400607,600601.5443521123,589097.6543641565,577593.7643761886,566089.8743882384,554585.9844002621,543082.0944123166],[1682385.012416752,1670751.5583660037,1659118.1043152546,1647484.6502645118,1635851.1962137632,1624217.7421630153,1612584.2881122692,1600950.8340615209,1589317.3800107753,1577683.9259600274,1566050.4719092813,1554417.0178585318,1542783.5638077853,1531150.1097570378,1519516.6557062909,1507883.2016555436,1496249.7476047967,1484616.2935540495,1472982.8395033022,1461349.3854525546,1449715.9314018087,1438082.477351061,1426449.0233003139,1414817.6430054337,1403192.6171066286,1391573.0538371168,1379958.1601747756,1368347.2318801628,1356739.644489587,1345135.03742055,1333533.6379485135,1321934.2177237878,1310336.371361092,1298741.028499027,1287146.9138795778,1275554.084611154,1263962.8914791448,1252372.326325118,1240783.3442479135,1229194.8137779161,1217607.421670876,1206020.714951709,1194434.3716242965,1182849.2947873995,1171264.2179505029,1159679.9053343446,1148096.2802532883,1136512.6551722274,1124929.6399106067,1113347.3029801664,1101764.9660497317,1090182.6291192956,1078600.292188859,1067010.9046858698,1055427.279604807,1043843.6545237461,1032243.2959077624,1020658.2190708732,1009073.1422339864,997459.9940067763,985873.2872876143,974286.5805684524,962699.8738492923,951113.1671301322,939481.5520280376,927893.0215580389,916304.4910880448,904715.9606180508,893127.4301480502,881474.2413962893,869883.6762422724,858293.1110882489,846702.5459342236,835111.9807801973,823433.7431426216,811840.9138741959,800248.0846057683,788655.2553373426,777062.4260689262,765469.5968004959,753876.7675320711,742283.9382636491,730569.2282577921,718973.8853957253,707378.5425336575,695783.1996715898,684187.856809522,672592.513947458,660997.1710853856,649401.8282233169,637644.6855126014,626046.5578480028,614448.4301834069,602850.3025188195,591252.1748542292,579654.047189638,568055.9195250385,556457.7918604352,544859.6641958375,533261.5365312407],[1681896.4064840758,1670169.179577562,1658441.9526710464,1646714.7257645337,1634987.498858023,1623260.2719515082,1611533.0450449944,1599805.818138483,1588078.591231969,1576351.364325456,1564624.1374189411,1552896.9105124297,1541169.6836059154,1529442.4566994032,1517715.2297928894,1505988.0028863763,1494260.775979863,1482533.5490733492,1470806.3221668361,1459079.0952603233,1447351.8683538102,1435624.6414472973,1423897.4145407844,1412170.1876342702,1400449.0596230319,1388733.5925195995,1377022.8469107656,1365316.3175439117,1353613.6632713801,1341914.0335334595,1330216.9065051696,1318521.821854003,1306829.4565127688,1295138.757160567,1283449.1399797278,1271761.9347282625,1260075.206501954,1248390.3041645736,1236705.8280649702,1225022.9336394602,1213340.0926724938,1201658.998001095,1189977.9033296956,1178297.768018073,1166618.281783123,1154938.7955481815,1143260.5470637456,1131582.4934825818,1119904.4399014157,1108227.3654571152,1096550.5830772193,1084873.800697328,1073197.0183174391,1061520.2359375497,1049836.1184144388,1038158.0648332727,1026480.011252109,1014784.9607288013,1003105.4744938528,991425.9882589024,979718.1992445383,968037.1045731464,956356.0099017397,944674.915230359,932993.8205589429,921267.6785762459,909584.7841507317,897901.8897252223,886218.9952997137,874536.1008742051,862788.5270310296,851103.6246936414,839418.7223562626,827733.8200188885,816048.9176815161,804276.4682962755,792589.331635477,780902.1949746748,769215.0583138848,757527.921653077,745840.7849922888,734153.6483314782,722466.5116706891,710657.8981185015,698968.2809376707,687278.6637568139,675589.0465759849,663899.4293951346,652209.8122143075,640520.1950334683,628830.5778526161,616979.9013800425,605287.5360387927,593595.1706975782,581902.8053563321,570210.4400151037,558518.0746738762,546825.7093326394,535133.3439914035,523440.9786501732],[1681407.8005514,1669586.8007891194,1657765.801026841,1645944.8012645603,1634123.8015022809,1622302.8017400021,1610481.8019777224,1598660.802215443,1586839.8024531633,1575018.802690884,1563197.8029286047,1551376.8031663261,1539555.8034040465,1527734.8036417668,1515913.8038794878,1504092.8041172086,1492271.8043549291,1480450.8045926504,1468629.804830371,1456808.8050680915,1444987.8053058116,1433166.8055435326,1421345.8057812531,1409524.8060189732,1397707.2656026683,1385895.6566755753,1374089.1363020157,1362286.95545694,1350488.4495136968,1338693.0296463703,1326900.1750618273,1315110.5735234388,1303322.822966347,1291536.4858221118,1279752.5943224945,1267969.78484537,1256188.3407921933,1244408.282004028,1232629.0424832935,1220851.0535009936,1209073.7951199738,1197297.2810504753,1185521.7984268442,1173746.3158032163,1161972.3456157525,1150198.4499827493,1138424.8138742014,1126652.3317929362,1114879.8497116668,1103107.4279340617,1091336.2001047148,1079564.9722753707,1067793.744446022,1056022.5166166718,1044244.9572240561,1032472.4751427895,1020699.9930615206,1008927.5109802568,997137.8067537262,985363.9111207221,973590.0154877212,961787.6285778433,950012.1459542122,938236.6633305782,926461.180706949,914642.3355944501,902865.0772134233,891087.818832404,879310.5604513837,867533.3020703644,855756.0436893413,843914.1382990442,832134.8987783138,820355.6592575749,808576.4197368436,796797.1802161084,785017.9406953733,773149.1346120043,761367.6905588424,749586.2465056619,737804.8024524879,726023.3583993064,714241.9143461362,702460.4702929566,690558.0193416821,678774.1278420631,666990.2363424487,655206.3448428372,643422.4533432228,631638.561843601,619854.6703439895,608070.7788443714,596286.887344759,584340.0388763212,572553.4358584518,560766.832840587,548980.2298227111,537193.6268048417,525407.0237869741,513620.4207691066],[1680919.194618724,1669004.4220006783,1657089.6493826346,1645174.8767645874,1633260.104146541,1621345.3315284955,1609430.5589104507,1597515.7862924056,1585601.0136743593,1573686.2410563135,1561771.4684382672,1549856.6958202226,1537941.9232021777,1526027.1505841324,1514112.377966086,1502197.6053480408,1490282.8327299955,1478368.0601119504,1466453.2874939046,1454538.5148758595,1442623.742257814,1430708.9696397693,1418794.1970217214,1406879.4244036772,1394965.8087288819,1383058.1756024507,1371155.6858468074,1359257.5933699708,1347363.2357560112,1335472.5715797336,1323584.9614263012,1311699.64492401,1299816.1894199243,1287935.3487254202,1276056.048665259,1264177.8828468693,1252301.883516925,1240426.2598434868,1228552.6831393996,1216679.1733625326,1204807.5510259983,1192935.9286894673,1181065.6935239937,1169195.822948128,1157326.409448374,1145458.104417317,1133589.7993862615,1121722.1701032892,1109855.259521918,1097988.3489405462,1086121.81713221,1074256.1438534074,1062390.470574609,1050524.7972958032,1038659.1240170063,1026786.8854523152,1014919.9748709421,1003053.0642895717,991170.139013601,979301.8339825473,967433.528951495,955565.2239204347,943668.2820066698,931798.4114308041,919928.5408549402,908058.6702790782,896145.3702761149,884273.7479395857,872402.1256030481,860530.5032665152,848658.8809299832,836787.2585934484,824851.0752003631,812977.4984962754,801103.9217921877,789230.3450880991,777356.768384005,765483.1916799098,753520.3228037888,741644.5713582346,729768.8199126916,717893.06846713,706017.3170215907,694141.5655760188,682265.8141304906,670269.5919272993,658391.4261089172,646513.2602905137,634635.0944721382,622756.9286537329,610878.7628353545,599000.5970169492,587122.4311985718,575244.2653801665,563204.0663605714,551323.2256660517,539442.3849715628,527561.5442770412,515680.7035825448,503799.86288803536],[1680430.5886860476,1668422.0432122387,1656413.4977384275,1644404.9522646128,1632396.4067907992,1620387.8613169894,1608379.3158431787,1596370.770369367,1584362.224895554,1572353.6794217436,1560345.1339479308,1548336.5884741198,1536328.0430003086,1524319.4975264962,1512310.9520526847,1500302.406578874,1488293.8611050623,1476285.31563125,1464276.7701574382,1452268.2246836275,1440259.6792098153,1428251.1337360037,1416242.588262192,1404234.0427883798,1392225.4973145677,1380221.4814928626,1368223.0018864076,1356229.2622161596,1344239.5554472555,1332253.2547136557,1320269.805117894,1308288.7163245755,1296310.7204740131,1284334.4775023607,1272359.5030080283,1260387.062870862,1248415.426241654,1236445.3674037284,1224476.323795496,1212508.409908054,1200541.3069320256,1188575.3206399796,1176609.5886211437,1164645.330093042,1152681.0715649407,1140717.7588518853,1128755.0444227788,1116792.3299936694,1104830.6693321685,1092869.330250693,1080907.9911692198,1068947.3154314472,1056987.1967031928,1045027.0779749407,1033066.9592466797,1021106.8405184168,1009139.956680363,997178.6175988824,985217.2785174064,973239.7568443627,961277.042415251,949314.3279861417,937351.613557036,925360.1595310355,913395.9010029342,901431.6424748311,889467.3839467373,877459.6770467628,865493.6907547181,853527.7044626698,841561.7181706261,829595.7318785675,817629.7455865201,805599.3377349572,793631.4238475142,781663.509960073,769695.59607263,757727.6821851861,745759.7682977449,733702.8962108204,721732.8373728944,709762.7785349619,697792.7196970377,685822.6608591173,673852.6020211903,661882.5431832643,649792.6158753838,637820.1757382192,625847.7356010498,613875.2954638833,601902.8553267214,589930.415189553,577957.9750523912,565985.5349152079,554013.094778047,542040.6546408823,529904.5401203968,517929.46174925193,505954.3833781127,493979.3050069604],[1679941.9827533728,1667839.6644237956,1655737.3460942134,1643635.027764637,1631532.7094350609,1619430.391105484,1607328.0727759034,1595225.7544463272,1583123.43611675,1571021.117787172,1558918.799457594,1546816.4811280165,1534714.162798439,1522611.8444688623,1510509.526139283,1498407.207809705,1486304.8894801287,1474202.571150551,1462100.2528209721,1449997.9344913955,1437895.616161816,1425793.29783224,1413690.9795026626,1401588.6611730831,1389486.342843506,1377386.172942316,1365291.5972237536,1353201.822623263,1341116.1445373225,1329033.9378475791,1316955.0674545087,1304879.2796338787,1292805.5947184896,1280733.6062793017,1268664.2902315245,1256596.2428948572,1244529.5284389094,1232464.6027360861,1220400.2365057832,1208337.7133807936,1196275.4623099985,1184214.7125904863,1172154.3623429304,1160094.8372379541,1148036.1907576225,1135977.5442772834,1123920.2894592928,1111863.165632132,1099806.079142422,1087750.3115608417,1075694.543979254,1063638.776397679,1051583.92283177,1039529.3586540595,1027474.7944763498,1015420.2302986402,1003365.6661209352,991304.1709081968,979248.4033266213,967192.6357450364,955120.5558790173,943063.4320518523,931006.3082246901,918949.1843975326,906863.2611509282,894804.614670597,882745.9681902574,870687.3217099262,858585.2559063751,846524.9056588188,834464.555411254,822404.2051637014,810343.8549161386,798283.5046685832,786158.9259028612,774096.6748320591,762034.423761256,749972.1726904707,737909.9216196612,725847.6705488637,713696.8548330963,701632.4886027919,689568.1223724904,677503.7561421786,665439.3899118984,653375.0236815764,641310.6574512897,629246.2912209835,617060.3767299717,604993.6622740161,592926.9478180679,580860.2333621327,568793.5189061826,556726.8044502586,544660.0899943067,532593.375538379,520526.66108241584,508297.37922145706,496228.0631736759,484158.7471259022],[1679453.3768206968,1667257.2856353514,1655061.1944500068,1642865.1032646648,1630669.0120793206,1618472.9208939765,1606276.8297086323,1594080.7385232889,1581884.6473379452,1569688.556152601,1557492.4649672573,1545296.3737819125,1533100.2825965695,1520904.1914112256,1508708.1002258821,1496512.009040538,1484315.9178551943,1472119.8266698504,1459923.7354845074,1447727.644299163,1435531.5531138193,1423335.4619284759,1411139.3707431313,1398943.279557789,1386747.1883724458,1374551.097187101,1362360.2906760152,1350174.652739126,1337993.4315795521,1325815.9601457254,1313641.6474800408,1301469.9709041594,1289300.4689629688,1277134.0788002242,1264969.1813318275,1252805.627607416,1240644.4341441318,1228483.8380684396,1216325.1644457644,1204167.0168535472,1192010.4285993953,1179854.1045410023,1167699.3903379277,1155544.676134856,1143391.309950302,1131238.275517736,1119085.5344958096,1106934.001270594,1094782.468045381,1082631.2928709849,1070481.096789301,1058330.9007076109,1046180.7046259325,1034031.6393331904,1021882.629706033,1009733.6200788678,997584.6104516955,985435.6008245302,973279.5281358254,961129.3320541428,948979.1359724561,936812.5361175621,924661.002892347,912509.4696671311,900357.9364419188,888177.5868663527,876024.5524337869,863871.5180012127,851718.4835686376,839522.1068549696,827367.3926519062,815212.6784488251,803057.9642457562,790903.2500426872,778748.5358396051,766529.8397040423,754373.2514498923,742216.6631957358,730060.0749415746,717903.4866874292,705746.898433269,693502.1986706192,681343.525047943,669184.851425265,657026.1778025879,644867.5041799173,632708.8305572337,620550.1569345593,608391.4833118739,596112.0290841479,583951.0403094385,571790.0515347151,559629.0627600038,547468.0739852898,535307.0852105748,523146.0964358561,510985.1076611504,498824.1188864298,486501.742969241,474338.1892448338],[1678964.7708880198,1666674.9068469072,1654385.0428057993,1642095.1787646895,1629805.314723579,1617515.4506824696,1605225.58664136,1592935.7226002496,1580645.8585591402,1568355.9945180311,1556066.1304769209,1543776.2664358104,1531486.4023947003,1519196.5383535908,1506906.6743124807,1494616.810271371,1482326.946230261,1470037.0821891506,1457747.2181480424,1445457.354106932,1433167.4900658214,1420877.6260247119,1408587.7619836011,1396297.8979424932,1384008.0339013827,1371718.1698602727,1359431.130448711,1347149.324486944,1334872.002848969,1322598.5004128304,1310328.2275055705,1298061.150895718,1285796.988156199,1273534.7558886139,1261274.1685697858,1249016.2810310824,1236759.3398493482,1224504.0767558576,1212250.0923857354,1199997.1113706874,1187745.3948887833,1175494.4694512784,1163244.4183329288,1150995.3401743467,1138746.4291429794,1126499.00675818,1114251.5843733712,1102004.8369090576,1089758.8942857916,1077512.951662518,1065267.6495993412,1053023.0250175544,1040778.4004357671,1028533.9200123232,1016290.4649357004,1004047.0098590837,991803.5547824684,979560.0997058512,967316.6446292261,955066.0283632483,942821.4037814578,930576.7791996626,918315.6975600142,906069.7549367314,893823.81231347,881577.8696901985,869303.1366773043,857055.7142925095,844808.2919077035,832560.8695228938,820313.4471380934,808021.1517339638,795772.0735753682,783522.9954167772,771273.9172582002,759024.8390996158,746712.0791385258,734461.1537010055,722210.2282635001,709959.3028259948,697708.3773884801,685457.451950985,673118.9277233947,660865.9467083337,648612.9656932997,636359.9846782265,624107.0036631916,611854.0226481315,599601.0416330723,587348.0606180243,574975.1328007923,562719.8697073087,550464.6066138279,538209.3435203247,525954.0804268457,513698.81733334344,501443.55423985794,489188.29114636686,476933.02805285994,464677.76495937444],[1678476.164955344,1666092.5280584656,1653708.8911615927,1641325.2542647137,1628941.6173678404,1616557.9804709642,1604174.3435740883,1591790.7066772108,1579407.0697803355,1567023.4328834594,1554639.7959865837,1542256.1590897066,1529872.5221928307,1517488.885295955,1505105.248399079,1492721.6115022027,1480337.9746053272,1467954.3377084516,1455570.7008115747,1443187.0639147,1430803.4270178229,1418419.7901209465,1406036.1532240715,1393652.5163271963,1381268.879430321,1368885.2425334419,1356501.9702214035,1344123.9962347622,1331750.6092337319,1319381.6443921905,1307016.169552351,1294653.6278128538,1282293.527005162,1269935.700565609,1257580.6485020828,1245226.9344547542,1232874.9053770644,1220524.708142295,1208175.1707300306,1195827.7319182986,1183480.4435108611,1171135.0985573144,1158789.8359364541,1146446.00421383,1134102.5620997269,1121759.737998622,1109417.9276615847,1097076.1173245446,1084735.3205261924,1072394.968504867,1060054.6164835491,1047715.1493274923,1035376.0962456022,1023037.0431637038,1010698.3001653766,998360.3996393085,986022.4991132244,973684.5985871572,961346.6980610862,949008.7975350069,936663.671590466,924324.618508569,911985.56542667,899630.0402063401,887289.6881850231,874949.3361636968,862608.984142378,850239.9105837895,837898.1002467582,825556.2899097167,813214.4795726743,800872.6692356439,788486.1829049764,776142.7407908821,763799.2986767804,751455.856562689,739112.4144485891,726705.6442062836,714360.381585421,702015.1189645594,689669.8563436847,677324.5937228315,664979.331101968,652634.0684811119,640199.7535839919,627852.4651765684,615505.1767691327,603157.8883616943,590810.599954267,578463.3115468416,566116.0231394172,553768.7347319899,541300.1504676379,528950.6130553642,516601.0756431045,504251.5382308252,491902.0008185599,479552.4634062955,467202.92599403765,454853.3885817714],[1677987.559022668,1665510.149270024,1653032.7395173842,1640555.3297647408,1628077.9200120985,1615600.5102594579,1603123.1005068137,1590645.690754172,1578168.28100153,1565690.8712488879,1553213.4614962474,1540736.0517436033,1528258.6419909617,1515781.23223832,1503303.8224856774,1490826.4127330356,1478349.002980393,1465871.5932277516,1453394.1834751093,1440916.773722467,1428439.3639698238,1415961.9542171834,1403484.544464541,1391007.1347118996,1378529.7249592573,1366052.3152066155,1353574.9054539716,1341100.6812480618,1328631.3573920217,1316166.2265239204,1303704.660996494,1291246.104729991,1278790.8219033517,1266338.0869932568,1253887.128434385,1241438.1468033758,1228991.3211240964,1216545.3395287427,1204101.5277976915,1191658.3524658955,1179216.7566660945,1166775.7276633517,1154336.1278591338,1141896.6682533203,1129458.8621837045,1117021.0561140939,1104584.2709497907,1092148.0726605211,1079711.874371253,1067276.9853472258,1054842.22392785,1042407.4625084717,1029973.7920554322,1017540.3104734374,1005106.8288914454,992673.7894195188,980241.4434439959,967809.0974684684,955376.7514929371,942944.4055174133,930512.0595418802,918072.4578174753,905638.9762354717,893205.4946534662,880755.5640565678,868320.8026371971,855886.0412178142,843451.2797984304,830987.9085858176,818551.7102965433,806115.5120072635,793679.3137179939,781243.115428729,768762.4861649834,756324.6800953755,743886.8740257667,731449.0679561533,719011.261886538,706510.5349073485,694070.9351031166,681631.3352989042,669191.7354946984,656752.1356904712,644312.5358862607,631872.9360820455,619344.9456748776,606903.3498750878,594461.7540752795,582020.1582754683,569578.5624756524,557136.9666758608,544695.3708760543,532253.7750762543,519691.88259040564,507248.0708593605,494804.2591283219,482360.4473972786,469916.6356662335,457472.82393519673,445029.01220415905],[1677498.9530899911,1664927.7704815825,1652356.587873171,1639785.405264768,1627214.2226563583,1614643.0400479506,1602071.8574395417,1589500.6748311333,1576929.492222725,1564358.3096143173,1551787.127005907,1539215.9443975007,1526644.761789093,1514073.579180684,1501502.3965722756,1488931.2139638672,1476360.0313554604,1463788.8487470513,1451217.6661386425,1438646.4835302355,1426075.3009218266,1413504.1183134196,1400932.9357050112,1388361.7530966029,1375790.570488193,1363219.387879785,1350648.205271374,1338077.669343998,1325512.1055503122,1312950.8713702213,1300393.90065408,1287840.1836671263,1275289.1957120972,1262740.473420909,1250194.342138726,1237650.4295795865,1225107.736871127,1212567.232440167,1200027.8848653645,1187489.798815548,1174953.0698213195,1162417.1666291421,1149882.419781813,1137348.4827942452,1124815.162267676,1112282.9922425533,1099750.8222174277,1087220.0279964996,1074689.4417549944,1062159.0021895827,1049629.8313721563,1037100.6605547178,1024571.4897372928,1012043.5777831711,999515.6677010637,986987.7576189684,974460.387774758,961933.5963497814,949406.8049247959,936880.0134998141,924353.2220748356,911826.4306498617,899292.3870442603,886764.4769621529,874236.5668800585,861692.2691106889,849163.0982932625,836633.9274758212,824104.7566584041,811547.1306833625,799016.5444418518,786485.9582003495,773955.371958836,761424.785717342,748850.0615139706,736317.8914888334,723785.7214637185,711253.5514385868,698721.3814134663,686189.211388344,673592.8142541191,661058.8772665393,648524.9402789706,635991.0032913974,623457.0663038353,610923.1293162648,598301.5229810197,585765.6197888413,573229.7165966658,560693.8134044977,548157.9102123193,535622.0070201242,523086.103827944,510550.2006357601,498014.29744358733,485356.980025813,472818.8939759936,460280.80792617146,447742.7218763614,435204.63582654856],[1677010.3471573158,1664345.39169314,1651680.436228965,1639015.4807647895,1626350.5253006178,1613685.5698364442,1601020.614372269,1588355.658908095,1575690.703443921,1563025.7479797464,1550360.792515571,1537695.8370513988,1525030.8815872238,1512365.9261230486,1499700.970658875,1487036.0151946996,1474371.0597305251,1461706.1042663516,1449041.148802177,1436376.1933380037,1423711.237873829,1411046.282409655,1398381.3269454807,1385716.3714813057,1373051.4160171312,1360386.4605529574,1347721.5050887845,1335056.5496246102,1322394.8253403716,1309737.695938982,1297084.4936481249,1284434.626288309,1271787.6086918386,1259144.080477417,1246502.5761450601,1233862.7123557958,1221225.3364126072,1208589.4105105656,1195954.6684029615,1183321.8815644407,1170689.5211958461,1158059.1723919953,1145428.9618074382,1132800.4375335728,1120172.1633626493,1107544.9283710136,1094918.3943903823,1082291.9833324766,1069667.0091387369,1057042.0349449967,1044417.438816457,1031793.858600969,1019170.2783854879,1006546.8450928945,993924.5065106936,981302.1679284866,968679.8293462833,956058.0952310963,943436.8583566546,930815.6214822168,918194.3846077854,905573.1477333512,892951.9108589105,880323.4592708508,867701.1206886545,855078.7821064442,842440.1553686969,829816.5751532158,817192.9949377375,804569.414722247,791917.5768764447,779292.6026827013,766667.6284889635,754042.6542952266,741417.6801014785,728748.9089519121,716122.3749712761,703495.8409906458,690869.3070100043,678242.773029373,665616.239048739,652926.0190384043,640297.7448674683,627669.4706965499,615041.1965256296,602412.9223547019,589784.6481837649,577156.3740128484,564439.2749178717,551809.0643333066,539178.8537487583,526548.6431642091,513918.43257964496,501288.2219950883,488658.0114105437,476027.80082597677,463277.340554703,450644.9801861197,438012.61981753353,425380.25944892876],[1676521.7412246394,1663763.0129046985,1651004.2845847583,1638245.556264818,1625486.8279448773,1612728.0996249365,1599969.3713049972,1587210.642985057,1574451.9146651153,1561693.1863451744,1548934.4580252338,1536175.7297052937,1523417.0013853554,1510658.273065414,1497899.5447454732,1485140.8164255328,1472382.0881055922,1459623.3597856518,1446864.6314657116,1434105.9031457705,1421347.1748258304,1408588.4465058893,1395829.7181859494,1383070.989866009,1370312.261546068,1357553.5332261291,1344794.8049061892,1332036.076586248,1319277.961759547,1306524.5205077443,1293775.4391485988,1281030.2003652218,1268288.0024900606,1255548.3511777672,1242810.8101513959,1230076.3740265793,1217343.4220327334,1204611.8489334537,1191882.6087967302,1179153.9643133301,1166427.3296259684,1153701.178154842,1140976.6601779182,1128252.3922728999,1115529.7809186224,1102807.1695643421,1090085.9665633244,1077365.0686271787,1064644.576522477,1051925.214376505,1039205.8522305223,1026487.0566472183,1013769.0670336778,1001051.077420142,988333.3453203146,975616.5782380099,962899.8111556973,950183.0440733903,937466.9117885167,924751.2294646278,912035.5471407278,899319.8648168445,886604.1824929565,873888.5001690593,861165.6744972263,848448.9074149178,835732.1403326131,822999.2228306038,810281.2332170615,797563.2436035322,784845.253990002,772099.2471650513,759379.8850190826,746660.5228731055,733941.1607271209,721221.798581156,708459.0284788469,695738.1305426881,683017.2326065339,670296.3346703965,657575.436734247,644854.5387980891,632070.549455977,619347.9381016977,606625.3267474268,593902.7153931465,581180.1040388532,568457.4926845813,555734.8813303057,542924.3152621519,530199.7972851917,517475.2793082716,504750.761331344,492026.2433544211,479301.7253774721,466577.20740054734,453852.68942362536,441128.171446681,428282.51775867864,415555.8830713099],[1676033.135291963,1663180.6341162566,1650328.132940551,1637475.6317648462,1624623.1305891355,1611770.6294134303,1598918.1282377227,1586065.6270620166,1573213.1258863108,1560360.6247106038,1547508.1235348978,1534655.6223591915,1521803.1211834853,1508950.6200077778,1496098.1188320713,1483245.6176563657,1470393.1164806585,1457540.6153049523,1444688.1141292462,1431835.6129535376,1418983.1117778316,1406130.6106021258,1393278.1094264193,1380425.6082507123,1367573.1070750067,1354720.6058992993,1341868.1047235948,1329015.6035478886,1316163.1023721818,1303313.5627256427,1290468.340127833,1277626.804221573,1264788.3962882801,1251953.2810458844,1239120.9266179071,1226290.455611602,1213461.8670654693,1200635.7373974905,1187810.5491904803,1174987.3317296165,1162165.138056079,1149344.2290499373,1136524.3585483911,1123705.5331790857,1110887.3984745983,1098070.4499369608,1085253.5387362745,1072438.276844611,1059623.014952946,1046808.3938080124,1033994.643709797,1021180.8936115964,1008367.8556818715,995555.4566702889,982743.0576586933,969930.9885475268,957119.7929651155,944308.5973826987,931497.4018002916,918686.8374470277,905876.709673686,893066.581900333,880256.4541269839,867446.3263536431,854636.1985803079,841819.0327234035,829007.8371409802,816196.6415585764,803369.4714964041,790557.0724848174,777744.6734732175,764932.2744616373,752119.8754500374,739278.3914509956,726464.6413527802,713650.8912545694,700837.1411563614,688023.391058146,675165.1582030831,662349.8963114247,649534.6344197579,636719.3725280883,623904.1106364299,611088.8487447677,598209.456969209,585392.5084315706,572575.5598939322,559758.6113562919,546941.6628186787,534124.7142810347,521307.7657433916,508401.91545233876,495583.0900830496,482764.26471372973,469945.4393444229,457126.61397511326,444307.78860580735,431488.9632365117,418670.13786719553,405731.5066937022],[1675544.5293592853,1662598.2553278138,1649651.9812963433,1636705.7072648667,1623759.4332333952,1610813.1592019247,1597866.885170451,1584920.6111389794,1571974.3371075063,1559028.063076034,1546081.7890445606,1533135.5150130892,1520189.2409816144,1507242.9669501432,1494296.6929186704,1481350.4188871975,1468404.1448557242,1455457.8708242513,1442511.5967927794,1429565.3227613056,1416619.0487298341,1403672.7746983613,1390726.5006668894,1377780.226635416,1364833.952603944,1351887.6785724717,1338941.4045409986,1325995.1305095246,1313048.8564780538,1300102.8474680574,1287161.460338217,1274224.3125688618,1261290.5435256062,1248359.6260211433,1235431.0942098633,1222505.2958412603,1209581.7962291762,1196659.6258615218,1183739.6584500722,1170821.034446735,1157903.579309225,1144987.7631612816,1132072.579836362,1119158.9241571892,1106245.7913955026,1093333.7303095828,1080422.4445885918,1067511.4850620474,1054601.8592148712,1041692.2333676931,1028783.4351890786,1015875.2971386313,1002967.1590881948,990059.83592042,977153.0275107799,964246.2191011319,951339.7747745318,938434.150692015,925528.5266095009,912622.9025269831,899717.8722066302,886813.2989838235,873908.7257610327,861004.1525382288,848099.5793154249,835195.0060926313,822283.5339493733,809377.909866849,796472.285784333,783566.661701818,770644.092956448,757737.2845468009,744830.4761371594,731923.6677275104,718988.1219784357,706079.9839279912,693171.8458775384,680263.7078270996,667355.5697766598,654403.4579524305,641493.8321052613,628584.2062580967,615674.5804109089,602764.9545637416,589855.3287165677,576882.3014700087,563971.015749013,551059.730028036,538148.4443070507,525237.1585860495,512325.8728650622,499414.5871440815,486415.4188347319,473502.28607304953,460589.1533113709,447676.0205496857,434762.88778800797,421849.75502632465,408936.6222646432,396023.48950295243],[1675055.9234266109,1662015.8765393742,1648975.8296521315,1635935.7827648944,1622895.7358776578,1609855.6889904176,1596815.6421031791,1583775.5952159383,1570735.5483287007,1557695.5014414636,1544655.4545542237,1531615.4076669847,1518575.360779746,1505535.313892507,1492495.2670052685,1479455.220118029,1466415.1732307922,1453375.1263435527,1440335.0794563137,1427295.032569074,1414254.9856818365,1401214.9387945973,1388174.8919073585,1375134.8450201182,1362094.7981328804,1349054.7512456437,1336014.7043584036,1322974.6574711646,1309934.6105839275,1296894.5636966894,1283856.8934038694,1270823.289958565,1257793.158244272,1244766.3045542643,1231742.7581080857,1218721.3603248245,1205701.7253928785,1192684.8320041252,1179669.3497580127,1166655.18519047,1153643.044349716,1140631.3515355675,1127621.8396289297,1114612.3819852406,1101604.874981233,1088597.4348271713,1075591.387777858,1062585.7648735126,1049580.7034767927,1036576.7136741057,1023572.7238714187,1010569.70066567,997567.1746629942,984564.6486603217,971562.9973628605,958561.7795551717,945560.5617474681,932559.704001327,919559.6514187083,906559.5988360904,893559.5462534651,880560.016067314,867560.9973950731,854561.9787228238,841562.9600505643,828563.9413783057,815564.922706048,802559.1781751243,789559.1255925,776559.0730098682,763559.0204272699,750542.2946319813,737541.0768242814,724539.8590165898,711538.6412088871,698509.0766013954,685506.5505987201,672504.0245960467,659501.4985933751,646498.9725906895,633453.029790787,620449.0399880838,607445.0501854047,594441.0603827201,581437.070580028,568433.0807773387,555429.0909746457,542360.848699756,529355.2257954031,516349.60289108846,503343.9799867403,490338.3570823837,477332.73417803925,464327.11127368826,451232.8672783142,438225.4271242656,425217.98697020207,412210.5468161432,399203.10666207504,386195.66650802456],[1674567.3174939337,1661433.4977509328,1648299.6780079242,1635165.8582649208,1622032.038521917,1608898.2187789108,1595764.3990359064,1582630.5792929013,1569496.7595498962,1556362.939806893,1543229.120063887,1530095.3003208812,1516961.4805778759,1503827.6608348722,1490693.841091867,1477560.021348862,1464426.2016058564,1451292.3818628527,1438158.5621198474,1425024.7423768416,1411890.922633838,1398757.1028908328,1385623.2831478282,1372489.4634048236,1359355.6436618178,1346221.8239188127,1333088.004175809,1319954.184432802,1306820.364689798,1293686.544946796,1280552.72520379,1267423.2359861648,1254297.4916310105,1241174.93013524,1228055.0542603845,1214937.7925491054,1201823.420223,1188710.6087465067,1175599.5629364434,1162490.6793423267,1149382.5093901968,1136276.5788994175,1123171.0994214965,1110067.3674590276,1096963.9585669572,1083862.2110205214,1070760.4634740935,1057660.3708794243,1044560.4107917221,1031461.1939805108,1018362.8402223159,1005264.4864641139,992167.1902377969,979070.2762828916,965973.3623279752,952877.3400091915,939781.7128034397,926686.0855976916,913590.7762279231,900496.2951451913,887401.8140624771,874307.3329797452,861213.269029106,848119.8049074076,835026.3407856971,821932.8766639847,808839.4125422724,795745.948420573,782645.9654006697,769551.4843179425,756457.0032352144,743362.5221524946,730251.6775113996,717156.0503056515,704060.4230999015,690964.7958941469,677869.1686883867,664744.3413649974,651647.4274100903,638550.5134551711,625453.5995002547,612356.6855453514,599215.5199598828,586117.1662016809,573018.8124434957,559920.4586852733,546822.104927077,533723.7511688843,520562.0072837919,507462.0471960893,494362.08710840065,481262.12702068035,468162.16693299916,455062.206845304,441962.2467575902,428862.2866698988,415673.0861523906,402571.33860595804,389469.5910595264,376367.843513092],[1674078.7115612582,1660851.1189624907,1647623.5263637186,1634395.9337649462,1621168.3411661745,1607940.7485674052,1594713.155968632,1581485.5633698618,1568257.970771092,1555030.3781723212,1541802.7855735505,1528575.1929747798,1515347.6003760071,1502120.007777236,1488892.4151784652,1475664.822579695,1462437.2299809237,1449209.6373821534,1435982.0447833806,1422754.4521846103,1409526.8595858389,1396299.2669870688,1383071.6743882995,1369844.0817895285,1356616.4891907566,1343388.8965919856,1330161.3039932156,1316933.7113944418,1303706.1187956748,1290478.5261969026,1277250.93359813,1264024.8173750609,1250802.545431186,1237583.8461243995,1224368.7237508341,1211156.0398784443,1197945.3815981746,1184737.2739043888,1171531.187257512,1158326.1734941779,1145123.6146144574,1131921.8062632708,1118721.638095852,1105522.3529328196,1092324.3466515802,1079126.987213877,1065930.932275061,1052734.9768853378,1039540.6796142762,1026346.3823432238,1013152.9565732148,999960.2388594989,986767.5211457787,973575.903905455,960384.6019983045,947193.3000911712,934002.863859429,920812.8272556183,907622.7906518094,894432.991454293,881244.0818714704,868055.1722886413,854866.2627058169,841677.631091998,828489.7215208253,815301.8119496685,802113.9023785004,788925.9928073315,775738.0832361802,762543.8956260094,749354.9860431748,736166.0764603382,722977.1668775193,709772.2415947188,696582.204990916,683392.1683871122,670202.1317833094,657012.0951794926,643793.3562267944,630602.0543196481,617410.7524124896,604219.4505053572,591028.1485982127,577793.2720206715,564600.5543069448,551407.8365932312,538215.1188795185,525022.4011657992,511829.6834520791,498636.96573836636,485380.1942300545,472185.8969590096,458991.59968795814,445797.30241690576,432603.0051458441,419408.70787478983,406214.410603744,392932.1303957766,379736.07545696665,366540.0205181511],[1673590.105628584,1660268.7401740442,1646947.37471951,1633626.0092649707,1620304.6438104347,1606983.278355898,1593661.9129013596,1580340.5474468241,1567019.1819922864,1553697.8165377507,1540376.451083213,1527055.0856286753,1513733.7201741382,1500412.354719601,1487090.9892650642,1473769.6238105271,1460448.2583559905,1447126.8929014537,1433805.5274469166,1420484.161992378,1407162.7965378421,1393841.4310833048,1380520.0656287686,1367198.7001742325,1353877.3347196938,1340555.9692651578,1327234.6038106175,1313913.238356087,1300591.8729015451,1287270.507447007,1273949.1419924719,1260627.776537934,1247309.604927076,1233994.9763350429,1220683.3601941268,1207374.5390910546,1194068.9278505724,1180765.132171162,1167462.9213690017,1154163.460925886,1140864.9116102257,1127568.155574414,1114272.6277830624,1100978.22193901,1087685.057806592,1074392.7772065825,1061101.4010760342,1047811.0387448575,1034520.9484368348,1021232.3139824267,1007943.6795280138,994655.9912548824,981368.909585658,968081.8279164289,954795.8416686505,941510.1518092677,928224.461949884,914939.5689135436,901655.1229116851,888370.6769098248,875086.3496804722,861803.0115975467,848519.6735146018,835236.3354316726,821953.1022559572,808670.7472353419,795388.3922147136,782106.0371940956,768823.6821734961,755541.327152878,742258.9721322525,728969.6307682032,715686.2926852629,702402.9546023356,689119.6165194092,675819.5408800766,662535.0948782004,649250.648876342,635966.2028744789,622653.5951841278,609367.9053247543,596082.2154653678,582796.5256059887,569510.8357466031,556225.145887224,542895.2145011714,529608.1328319507,516321.0511627281,503033.96949349623,489746.88782428484,476459.806155039,463109.6668973183,449821.0324428994,436532.3979884973,423243.76353409886,409955.12907968275,396666.49462527875,383377.86017086357,370089.2257164642,356712.1975232428],[1673101.4996959064,1659686.3613856018,1646271.223075302,1632856.0847649982,1619440.946454695,1606025.8081443924,1592610.669834088,1579195.5315237849,1565780.3932134828,1552365.2549031787,1538950.1165928738,1525534.9782825713,1512119.8399722693,1498704.7016619667,1485289.5633516626,1471874.4250413598,1458459.2867310564,1445044.148420753,1431629.0101104514,1418213.8718001465,1404798.7334898445,1391383.5951795413,1377968.4568692364,1364553.3185589337,1351138.1802486319,1337723.0419383284,1324307.9036280245,1310892.7653177246,1297477.6270074188,1284062.4886971149,1270647.350386813,1257232.2120765108,1243817.3346392068,1230406.6891255707,1216999.5600598445,1203595.1863966535,1190193.1263569528,1176793.7181767868,1163396.6765273763,1150000.9328256445,1136607.4297516188,1123215.255414097,1109824.302222197,1096434.9739495902,1083046.3304289002,1069659.214042821,1056272.6591240247,1042887.200153457,1029502.5304298946,1016118.2456216267,1002735.2739838618,989352.3023460992,975970.2980255196,962588.8524007858,949207.406776051,935827.0035273698,922446.9257157515,909066.8479041392,895687.4551715627,882308.5997716542,868929.7443717364,855550.8889718186,842173.0843233988,828795.3177403631,815417.5511573227,802039.7845742889,788662.8820509389,775286.0815808689,761909.281110798,748532.4806407355,735155.6801706553,721778.8797005899,708395.418493025,695017.6519099837,681639.885326949,668262.1187439049,654868.0579731064,641489.2025731923,628110.3471732847,614731.4917733753,601352.6363734622,587944.9804253764,574564.9026137656,561184.82480215,547804.7469905335,534424.6691789189,521001.14678438473,507619.7011596523,494238.2555349069,480856.80991016794,467475.3642854206,454093.91866068076,440712.47303594183,427267.49356009625,413884.52192234155,400501.5502845794,387118.5786468191,373735.6070090635,360352.6353712948,346969.6637335392],[1672612.893763231,1659103.982597162,1645595.0714310908,1632086.160265026,1618577.249098955,1605068.3379328852,1591559.4267668168,1578050.5156007453,1564541.604434677,1551032.693268607,1537523.7821025369,1524014.8709364694,1510505.9597703996,1496997.0486043321,1483488.13743826,1469979.2262721914,1456470.315106124,1442961.4039400537,1429452.492773985,1415943.5816079145,1402434.6704418454,1388925.759275777,1375416.8481097072,1361907.936943636,1348399.0257775702,1334890.1146115006,1321381.2034454318,1307872.292279363,1294363.3811132922,1280854.4699472215,1267345.5587811526,1253836.6476150844,1240327.7364490132,1226820.567160941,1213316.766680964,1199816.296605593,1186319.0343238045,1172823.8678601943,1159330.4316857648,1145840.014666807,1132350.7492300747,1118863.0937347384,1105377.204905787,1091892.0545577952,1078408.8447628357,1064925.9635655093,1051444.821641489,1037963.9924039282,1024484.3844461176,1011005.40733018,997526.8684397154,984049.5596185937,970572.2507974762,957095.8768851431,943620.0673048864,930144.2577246334,916669.3894816162,903194.9237177735,889720.4579539159,876246.5226334641,862773.2578354878,849299.9930375293,835826.7282395652,822354.3000490488,808882.1049659029,795409.9098827718,781937.7147996221,768466.1259676367,754994.8800481148,741523.6341285836,728052.3882090691,714581.1422895407,701109.8963700151,687632.3492176402,674160.1541344989,660687.9590513492,647215.7639682125,633743.5688850665,620254.4914720878,606781.2266741116,593307.9618761605,579834.6970781842,566333.279621535,552858.8138576876,539384.3480938468,525909.882329979,512435.4165661372,498960.9508022843,485442.54157631565,471966.73199604824,458490.92241581064,445015.11283556465,431539.3032553084,418063.4936750671,404587.684094795,391047.9714894779,377570.66266834456,364093.3538472438,350616.04502610676,337138.7362050079],[1672124.2878305544,1658521.603808719,1644918.9197868826,1631316.2357650478,1617713.5517432124,1604110.867721378,1590508.1836995417,1576905.4996777074,1563302.8156558722,1549700.1316340365,1536097.4476122004,1522494.7635903652,1508892.0795685311,1495289.3955466952,1481686.7115248595,1468084.0275030239,1454481.3434811886,1440878.659459354,1427275.9754375184,1413673.2914156825,1400070.6073938468,1386467.9233720128,1372865.2393501769,1359262.5553283405,1345659.8713065074,1332057.1872846717,1318454.5032628372,1304851.8192410017,1291249.1352191668,1277646.451197329,1264043.767175495,1250441.0831536592,1236838.3991318224,1223235.715109988,1209636.0963291563,1196039.5796371792,1182445.6924917232,1168854.903975585,1155266.511717528,1141679.6667167922,1128095.146589438,1114512.260791866,1100930.452875217,1087350.760085295,1073771.4125812156,1060194.240222814,1046617.121348849,1033041.8806026191,1019466.7770462828,1005892.953954028,992319.6694457168,978746.8168910914,965175.1708866195,951603.5248821429,938032.7278337348,924462.5542979753,910892.3807622055,897322.999531405,883754.1458153212,870185.2920992412,856616.7712992625,843049.0971032353,829481.4229072221,815913.7487111995,802346.6587744886,788780.0351912463,775213.4116080003,761646.7880247524,748080.478985427,734514.7876164457,720949.0962474653,707383.4048784804,693817.7135095047,680252.0221405281,666686.330771545,653113.799358787,639547.1757755512,625980.5521923061,612413.9286090601,598830.9615748832,585263.2873788476,571695.6131828297,558127.9389868118,544560.2647907827,530963.9491971498,517395.09548106976,503826.24176497664,490257.38804889936,476688.5343328165,463119.68061673176,449506.4805461969,435936.3070104299,422366.13347466383,408795.9599389024,395225.7864031382,381655.6128673693,368085.4393316079,354451.1006854195,340879.45468094293,327307.8086764794],[1671635.6818978786,1657939.2250202761,1644242.7681426757,1630546.3112650742,1616849.8543874728,1603153.397509873,1589456.9406322695,1575760.4837546686,1562064.0268770673,1548367.5699994648,1534671.1131218642,1520974.6562442626,1507278.1993666617,1493581.7424890588,1479885.2856114577,1466188.8287338573,1452492.371856256,1438795.9149786541,1425099.4581010519,1411403.001223451,1397706.5443458492,1384010.0874682479,1370313.630590646,1356617.173713047,1342920.7168354425,1329224.2599578418,1315527.803080241,1301831.3462026387,1288134.8893250376,1274438.432447435,1260741.9755698363,1247045.5186922352,1233349.061814631,1219652.604937029,1205956.148059427,1192263.9221438468,1178574.545414344,1164887.5730058076,1151202.7953519868,1137521.057928924,1123840.6955310334,1110162.07229529,1096485.4675110318,1082809.5071730637,1069135.9209526242,1055462.5168801178,1041791.1603295179,1028119.8037789161,1014450.390926192,1000781.0130510726,987112.908677185,973445.3167764992,959778.0909757572,946112.1077879295,932446.1246000985,918780.8508712961,905116.3133800183,891451.7758887438,877787.8336767284,864124.5920083979,850461.3503400935,836798.2011689506,823136.1175748706,809474.0339807961,795811.9503867244,782150.1604997264,768489.1084163748,754828.0563330231,741167.0042496789,727505.9521663236,713845.8042858746,700185.6674674358,686525.5306489952,672865.3938305667,659205.2570121344,645545.1201936966,631878.5875828872,618217.535499528,604556.4834161829,590895.431332822,577234.3792494889,563556.5292874556,549894.4456933923,536232.3620993057,522570.2785052443,508908.19491116516,495217.0669638226,481553.8252954958,467890.5836271858,454227.34195885435,440564.1002905518,426857.5011852952,413192.96369403973,399528.4262027377,385863.88871147484,372199.35122018866,358534.8137289034,344870.27623762656,331142.8643357763,317476.88114795275],[1671147.0759652015,1657356.8462318357,1643566.616498468,1629776.3867651024,1615986.1570317324,1602195.9272983652,1588405.6975649968,1574615.4678316293,1560825.2380982616,1547035.0083648937,1533244.778631527,1519454.54889816,1505664.3191647928,1491874.0894314244,1478083.8596980562,1464293.6299646897,1450503.4002313223,1436713.170497954,1422922.9407645855,1409132.711031219,1395342.4812978525,1381552.2515644846,1367762.0218311157,1353971.7920977497,1340181.5623643799,1326391.3326310148,1312601.1028976436,1298810.8731642796,1285020.643430912,1271230.4136975443,1257440.1839641777,1243649.9542308082,1229859.7244974421,1216069.494764073,1202279.265030707,1188490.0968256998,1174704.044605021,1160921.6640124423,1147141.4752445272,1133363.1099664196,1119587.5064944122,1105813.5181285054,1092040.7917843563,1078270.3693875694,1064500.429324042,1050732.687507785,1036965.1993101873,1023199.6050829459,1009434.0108557148,995670.3526122156,981906.7004183242,968144.248615596,954382.34932253,940620.6906937114,926860.370322532,913100.0499513489,899340.2459978461,885581.3445510464,871822.4431042597,858063.8919175658,844306.2622970091,830548.6326764589,816791.003055905,803034.3192504048,789277.8262582766,775521.3332661502,761764.840274022,748009.3246413022,734253.8440578412,720498.3634743923,706742.8828909434,692987.9300563894,679233.3477885025,665478.7655206015,651724.1832527202,637969.6009848285,624215.0187169313,610460.4364490556,596699.0382233104,582943.5576398559,569188.0770564014,555432.5964729469,541660.9523999691,527904.4594078548,514147.9664157238,500391.4734235918,486634.98043146916,472850.262542109,459092.63292156346,445335.0033010077,431577.3736804528,417819.74405989423,404062.11443934124,390260.8924665749,376501.9910197975,362743.08957299776,348984.18812620547,335225.2866794225,321466.3852326274,307707.4837858472],[1670658.470032526,1656774.467443392,1642890.4648542614,1629006.4622651234,1615122.4596759905,1601238.4570868588,1587354.4544977252,1573470.4519085907,1559586.4493194572,1545702.446730324,1531818.4441411905,1517934.441552055,1504050.4389629234,1490166.4363737903,1476282.4337846546,1462398.4311955222,1448514.4286063872,1434630.4260172541,1420746.423428121,1406862.4208389872,1392978.4182498534,1379094.4156607203,1365210.4130715854,1351326.4104824527,1337442.4078933178,1323558.4053041865,1309674.4027150515,1295790.400125916,1281906.3975367856,1268022.3949476518,1254138.3923585159,1240254.3897693814,1226370.3871802506,1212486.384591116,1198602.3820019825,1184718.3794128513,1170836.3676101165,1156957.0998649728,1143080.9354314837,1129207.3992078644,1115335.50970894,1101465.857413969,1087598.1345091732,1073731.3050566353,1059866.9664214766,1046002.9987231083,1032141.0597507618,1018279.4063869794,1004419.5744831087,990559.7425792394,976701.7656606878,962843.8391480218,948986.9737692568,935130.7670838181,921274.6160449623,907419.9584904285,893565.3009358831,879710.913213361,865857.6478110561,852004.3824087502,838151.1742539518,824299.1566811493,810447.1391083775,796595.1215355797,782743.7021298315,768892.7997396467,755041.8973494675,741190.9949592892,727340.6838660156,713490.7747824639,699640.8656988963,685790.9566153381,671941.164927993,658092.137210655,644243.1094933143,630394.0817759708,616545.0540586188,602696.0263412837,588846.9986239364,574991.683946874,561141.7748633213,547291.865779764,533441.9566962048,519592.0476126345,505725.6543261986,491874.7519360082,478023.8495458374,464172.94715564325,450322.0447654752,436442.6646431321,422590.6470703343,408738.6294975607,394886.611924774,381034.59435199667,367182.5767791914,353286.82792582177,339433.56252352055,325580.2971212212,311727.0317189153,297873.76631660573],[1670169.8640998497,1656192.0886549521,1642214.3132100478,1628236.5377651507,1614258.7623202533,1600280.986875353,1586303.2114304518,1572325.4359855514,1558347.6605406532,1544369.885095753,1530392.1096508533,1516414.3342059534,1502436.5587610528,1488458.7833161536,1474481.007871253,1460503.2324263542,1446525.4569814547,1432547.6815365546,1418569.9060916556,1404592.1306467555,1390614.3552018548,1376636.5797569552,1362658.8043120562,1348681.0288671565,1334703.2534222538,1320725.4779773557,1306747.7025324577,1292769.9270875538,1278792.1516426585,1264814.3761977612,1250836.6007528573,1236858.8253079583,1222881.0498630577,1208903.2744181564,1194925.4989732567,1180947.7235283577,1166969.948083459,1152994.9350410048,1139022.4476944925,1125052.3596714768,1111085.3448958267,1097119.8118055305,1083156.110687601,1069194.54467306,1055233.6762146591,1041275.2586127543,1027317.1738099963,1013361.0376815591,999405.1381105054,985451.0685299877,971496.9989494844,957544.6300716139,943592.4292401704,929641.0841381578,915690.5700603398,901740.0559825334,887790.8722916082,873841.8775537256,859892.8828158369,845945.2231600415,831997.5938022183,818049.9644444035,804103.2751608444,790156.8696358111,776210.4641107917,762264.2662131414,748318.9544249093,734373.6426366763,720428.3308484433,706483.1860905215,692538.8485068558,678594.5109231928,664650.1733395401,650705.8357558772,636762.0357339038,622818.5625670953,608875.0894003017,594931.6162335034,580988.1430667043,567044.6698998986,553101.196733105,539151.1350865746,525206.7975029163,511262.45991924964,497318.1223355988,483373.78475193586,469412.71866020374,455467.40687195957,441522.0950837163,427576.7832954861,413631.4715072531,399657.51493523177,385711.10941020213,371764.7038851874,357818.29836016614,343871.8928351486,329925.48731013294,315935.3075630022,301987.678205193,288040.0488473736],[1669681.2581671732,1655609.7098665105,1641538.161565842,1627466.6132651775,1613395.064964509,1599323.5166638456,1585251.968363181,1571180.4200625133,1557108.8717618475,1543037.3234611838,1528965.7751605166,1514894.2268598513,1500822.678559184,1486751.1302585178,1472679.5819578515,1458608.0336571857,1444536.48535652,1430464.9370558546,1416393.3887551893,1402321.840454523,1388250.2921538574,1374178.743853192,1360107.1955525258,1346035.6472518598,1331964.0989511937,1317892.5506505282,1303821.0023498612,1289749.4540491956,1275677.9057485294,1261606.3574478657,1247534.8091471985,1233463.2608465329,1219391.7125458673,1205320.1642451999,1191248.6159445343,1177177.0676438664,1163105.5193432062,1149033.9710425418,1134965.7991183745,1120899.9483449492,1106836.0465129907,1092775.3123084134,1078716.016256208,1064658.2663153103,1050602.7486201627,1036547.8159846202,1022495.2459614025,1008442.9545847042,994392.6213001837,980342.3944807472,966294.0872236029,952245.7799664633,938198.9458449786,924152.4706947659,910106.5797223174,896061.7582521299,882016.9367819298,867973.1117260838,853929.7798048425,839886.4478835976,825844.0705979783,811802.0772846555,797760.0839713262,783718.6177360332,769677.824258795,755637.0307815261,741596.2373042749,727556.2903140672,713516.5691277683,699476.8479414834,685437.1267551966,671398.0652310569,657359.2991472911,643320.533063516,629281.766979753,615243.0433582375,601205.1247419808,587167.2061257325,573129.2875094749,559091.368893221,545053.4502769662,531015.5316607263,516977.61304446496,502932.87222584244,488894.1061420962,474855.3400583258,460816.57397455256,446761.8665882889,432722.14540197793,418682.42421570513,404642.70302941557,390602.9818431111,376563.26065684203,362494.81341838464,348454.01994113065,334413.2264638813,320372.4329866078,306331.6395093752,292290.846032111,278206.33137814794],[1669192.6522344982,1655027.3310780688,1640862.0099216336,1626696.6887652026,1612531.3676087721,1598366.046452338,1584200.7252959064,1570035.4041394764,1555870.0829830426,1541704.7618266118,1527539.4406701801,1513374.1195137454,1499208.7983573151,1485043.4772008825,1470878.1560444497,1456712.8348880182,1442547.5137315874,1428382.1925751544,1414216.8714187222,1400051.5502622915,1385886.2291058598,1371720.9079494292,1357555.5867929948,1343390.2656365621,1329224.9444801325,1315059.6233237002,1300894.3021672661,1286728.9810108363,1272563.6598544035,1258398.3386979715,1244233.017541539,1230067.6963851058,1215902.375228675,1201737.0540722432,1187571.7329158115,1173406.4117593816,1159241.0906029437,1145075.7694465136,1130910.4482900868,1116748.9598422311,1102589.6018163473,1088432.0247878903,1074277.3014456257,1060124.1230609547,1045972.3242971031,1031822.7463504705,1017673.6509119547,1003526.9284674148,989380.3410472353,975235.8106066249,961091.280166015,946948.6305639381,932806.0856301589,918664.7129808059,904523.9635118167,890383.4605217148,876244.3316591559,862105.2027965859,847966.6767938491,833829.0076892506,819691.3385846568,805554.1901249187,791417.8328560675,777281.4755872302,763145.1844067769,749010.0029772939,734874.8215478035,720739.6401183167,706604.8074071072,692470.6768227704,678336.546238427,664202.415654079,650068.4249550384,635935.230371167,621802.0357872937,607668.8412034279,593535.6466195583,579402.7960179513,565270.4319522474,551138.0678865369,537005.7038208293,522873.3397551235,508740.97568941396,494608.61162371654,480476.24755800795,466336.89536472876,452203.70078085363,438070.5061969822,423937.31161311176,409788.06513591204,395653.9345515752,381519.80396723375,367385.67338289786,353251.5427985452,339117.41221419536,324954.56009259634,310819.37866310775,296684.19723362755,282549.0158041427,268413.8343746457],[1668704.0463018212,1654444.952289628,1640185.858277429,1625926.7642652285,1611667.6702530317,1597408.5762408338,1583149.4822286346,1568890.3882164357,1554631.2942042388,1540372.20019204,1526113.1061798404,1511854.0121676428,1497594.918155445,1483335.824143249,1469076.7301310473,1454817.6361188511,1440558.5421066526,1426299.4480944546,1412040.354082256,1397781.2600700594,1383522.1660578602,1369263.0720456634,1355003.9780334665,1340744.8840212657,1326485.7900090693,1312226.6959968698,1297967.6019846748,1283708.507972476,1269449.4139602752,1255190.319948076,1240931.225935882,1226672.1319236816,1212413.0379114822,1198153.9438992867,1183894.8498870856,1169635.7558748843,1155376.661862691,1141117.567850498,1126858.4738382925,1112599.7278170413,1098344.4172125608,1084091.4081086977,1069840.0247874097,1055591.3123074602,1041344.1322197774,1027098.2846329631,1012854.5378639991,998611.2959778938,984370.3061307967,970129.4312649509,955890.6056008926,941651.8780041886,927414.698551009,913177.915940586,898941.931479082,884706.907691319,870471.8839035397,856238.2902814317,842004.8540264918,827771.5674949028,813539.5612069517,799307.5549190035,785075.5817408087,770844.8605164569,756614.1392921042,742383.4180677421,728153.4057913283,713923.8364096088,699694.2670278791,685464.697646155,671235.9657216668,657007.4257392557,642778.8857568614,628550.3457744587,614322.3045948436,600094.6815108694,585867.0584268915,571639.4353429182,557411.8122589542,543184.7668798538,528957.9573646914,514731.14784953557,500504.3383343704,486277.5288192034,472050.71930405125,457823.90978887957,443590.82758713234,429363.2045031823,415135.58141920157,400907.9583352171,386680.33525125775,372436.62609132566,358208.08610892855,343979.546126551,329751.00614414737,315522.46616173536,301293.9261793569,287036.75495789014,272807.18557616416,258577.61619442515],[1668215.4403691455,1653862.5735011827,1639509.7066332204,1625156.839765255,1610803.9728972896,1596451.106029324,1582098.2391613615,1567745.3722933982,1553392.5054254332,1539039.63855747,1524686.7716895042,1510333.9048215414,1495981.0379535765,1481628.171085612,1467275.3042176466,1452922.437349685,1438569.5704817194,1424216.7036137553,1409863.83674579,1395510.9698778274,1381158.1030098628,1366805.2361418968,1352452.3692739347,1338099.5024059694,1323746.6355380067,1309393.7686700416,1295040.9018020798,1280688.0349341142,1266335.168066149,1251982.3011981845,1237629.4343302213,1223276.5674622562,1208923.7005942944,1194570.8337263288,1180217.9668583658,1165865.099990401,1151512.23312244,1137159.366254468,1122806.4993865085,1108453.6325185387,1094101.3335106121,1079752.17122937,1065405.367221986,1051060.0465115565,1036717.3448939226,1022376.0437326883,1008036.147322908,993698.1231607455,979360.7781662443,965025.3789515574,950690.3516643881,936357.006282982,922024.0815301877,907692.2911848067,893361.2708977489,879030.6013398021,864701.3032332584,850372.0051267194,836043.6341189565,821715.8904716335,807388.1468243189,793061.4403579524,778735.096886646,764408.753415334,750083.1602658136,735758.0750859547,721432.9899060866,707108.0327009028,692784.0753669376,678460.1180329816,664136.1606990173,649812.4358244371,635489.4864439806,621166.5370635362,606843.5876830928,592520.638302628,578198.4702342199,563876.4186501475,549554.3670660714,535232.3154819962,520910.2638979135,506588.9559439346,492267.70097932406,477946.44601470977,463625.19105010293,449303.9360854812,434982.6811208483,420661.42615624797,406333.8512252867,392011.7996412134,377689.74805713724,363367.6964730434,349045.6448889738,334707.54945453256,320384.60007408913,306061.65069363173,291738.7013131827,277415.75193272904,263092.8025522847,248741.39801423997],[1667726.8344364683,1653280.194712738,1638833.5549890113,1624386.9152652794,1609940.2755415516,1595493.6358178204,1581046.9960940897,1566600.356370358,1552153.716646629,1537707.0769228982,1523260.437199167,1508813.7974754376,1494367.157751707,1479920.5180279766,1465473.8783042452,1451027.2385805182,1436580.598856787,1422133.9591330555,1407687.3194093248,1393240.679685596,1378794.0399618654,1364347.4002381347,1349900.7605144044,1335454.120790673,1321007.4810669438,1306560.8413432136,1292114.2016194838,1277667.5618957537,1263220.9221720207,1248774.2824482913,1234327.6427245582,1219881.003000832,1205434.3632771014,1190987.723553369,1176541.08382964,1162094.4441059153,1147647.8043821775,1133201.1646584515,1118754.524934717,1104307.8852109923,1089861.2454872583,1075415.2358506601,1060972.221892661,1046531.4791562171,1032092.0899603274,1017655.3992050062,1003219.8575996626,988785.9123669234,974353.5022407002,959922.054137812,945492.1469296864,931062.9672212056,916635.0126528889,902207.8907440142,887781.4084653179,873356.1505016326,858930.8925379375,844507.1501376564,830083.5777123319,815660.363171733,801238.3121320447,786816.2610923462,772394.6451422228,757973.9644875675,743553.2838329114,729132.7321041487,714713.2829687772,700293.8338333899,685874.3846980212,671455.538419799,657037.1931336084,642618.8478474142,628200.502561219,613782.7283526091,599365.3695740998,584948.0107955988,570530.652017084,556113.4019573824,541696.9218731932,527280.4417890226,512863.9617048381,498447.48162065074,484031.06362427026,469615.3632102115,455199.6627961444,440783.96238206234,426368.2619679924,411952.56155393086,397536.8611398572,383121.16072577983,368699.16086300183,354282.68077883776,339866.2006946476,325449.7206104584,311033.2405262813,296600.8352255272,282183.47644701693,267766.1176685067,253348.75889001414,238931.40011151694],[1667238.228503793,1652697.8159242957,1638157.4033448007,1623616.990765304,1609076.5781858081,1594536.1656063108,1579995.7530268151,1565455.3404473194,1550914.9278678242,1536374.5152883267,1521834.1027088305,1507293.6901293334,1492753.2775498382,1478212.8649703423,1463672.4523908438,1449132.0398113476,1434591.6272318517,1420051.2146523572,1405510.80207286,1390970.389493364,1376429.9769138682,1361889.564334372,1347349.1517548752,1332808.7391753765,1318268.32659588,1303727.9140163844,1289187.5014368892,1274647.0888573916,1260106.6762778945,1245566.2636983984,1231025.8511189027,1216485.4385394074,1201945.025959911,1187404.613380413,1172864.2008009173,1158323.7882214193,1143783.37564193,1129242.963062426,1114702.5504829343,1100162.1379034352,1085621.725323941,1071081.312744439,1056541.4348371918,1042004.5692024291,1027469.7439113958,1012936.1551337219,998405.4752407135,983875.5738207204,969347.5797650176,954820.6751038707,940295.1238925843,925770.6100651822,911247.2779353871,896724.6247106213,882203.3056456572,867682.050392569,853162.5547522362,838643.0591119071,824124.4484045133,809606.6016604193,795088.7549163206,780572.1190076796,766055.7605756167,751539.4021435585,737024.1577217905,722509.1398837725,707994.1220457572,693479.7629405735,678965.9498496819,664452.1367587792,649938.3236678988,635425.4923297651,620912.7590913456,606400.0258529121,591887.2926144917,577375.3832885567,562863.6151120048,548351.8469354408,533840.078758873,519328.56809603795,504817.65951176174,490306.7509274669,475795.842343173,461284.9337588921,446774.134542183,432263.9886786593,417753.84281513,403243.6969516054,388733.55108808074,374223.4052245524,359713.2593610333,345203.1134975068,330686.7565003205,316175.8479160238,301664.9393317355,287154.0307474658,272643.12216317374,258132.21357887518,243604.7152277343,229092.94705118798],[1666749.622571118,1652115.4371358554,1637481.2517005922,1622847.0662653316,1608212.880830069,1593578.695394805,1578944.5099595431,1564310.3245242815,1549676.1390890186,1535041.9536537556,1520407.7682184936,1505773.582783231,1491139.397347969,1476505.211912707,1461871.0264774435,1447236.8410421808,1432602.6556069187,1417968.4701716555,1403334.2847363935,1388700.099301132,1374065.9138658703,1359431.7284306057,1344797.542995344,1330163.3575600826,1315529.1721248173,1300894.9866895557,1286260.8012542923,1271626.6158190293,1256992.4303837682,1242358.2449485057,1227724.0595132436,1213089.87407798,1198455.688642717,1183821.5032074568,1169187.3177721933,1154553.1323369313,1139918.946901665,1125284.7614664086,1110650.5760311396,1096016.3905958813,1081382.2051606183,1066748.0197253604,1052113.8342900989,1037479.930470197,1022849.2131586722,1008220.1614875025,993592.6083377916,978967.5730010448,964343.1923958571,949721.1495171925,935099.6417502565,920479.9874305739,905860.7683580532,891243.2838069373,876625.842456182,862010.3262351183,847394.8100140588,832780.4836495705,818166.7503326265,803553.1980338031,788941.0769709358,774328.9559080601,759717.311098584,745106.6452741483,730495.9794496968,715885.676589299,701276.3215679331,686666.9665465634,672057.6115251975,657449.3379549384,642841.1609085333,628232.9838621216,613625.0156214535,599017.8944307975,584410.7732401378,569803.6520494744,555196.5782068977,540590.4006322864,525984.2230576789,511378.04548308253,496771.86790845916,482166.0202342747,467560.6831498854,452955.3460654989,438350.0089811245,423744.6718967203,409139.4236622611,394534.8323492799,379930.24103631545,365325.6497233361,350721.05841035023,336116.46709736437,321511.8757843999,306907.284471428,292302.69315843564,277691.30105280224,263085.96396841947,248480.62688403204,233875.28979964275,219269.95271525532],[1666261.016638441,1651533.0583474133,1636805.1000563854,1622077.141765359,1607349.183474328,1592621.225183298,1577893.2668922725,1563165.3086012418,1548437.350310214,1533709.3920191864,1518981.4337281568,1504253.4754371285,1489525.517146099,1474797.5588550712,1460069.600564042,1445341.642273013,1430613.6839819853,1415885.7256909562,1401157.7673999267,1386429.8091089001,1371701.8508178708,1356973.89252684,1342245.9342358129,1327517.9759447852,1312790.0176537563,1298062.0593627281,1283334.1010717007,1268606.1427806686,1253878.1844896416,1239150.2261986148,1224422.2679075864,1209694.3096165552,1194966.351325526,1180238.3930345005,1165510.434743469,1150782.4764524363,1136054.5181614165,1121326.5598703832,1106598.6015793625,1091870.643288326,1077142.6849972955,1062414.7267062683,1047686.7684152354,1032958.8101242133,1018230.851833187,1003506.1537614055,988782.7318845675,974061.2551453244,959341.6924860012,944623.0343176033,929906.6216234453,915190.5299218153,900476.6447517788,885762.8873237157,871050.9848358585,856339.3478634306,841628.9525124049,826919.2391352635,812209.9371936414,797501.9662000397,782793.9952064464,768087.0036439048,753380.6082622604,738674.2128806217,723968.9151879111,709263.9419710906,694558.968754272,679854.8288853811,665151.1366806505,650447.4444759274,635743.9981491761,621041.4571472555,606338.9161453312,591636.3751434125,576934.253865785,562232.7447228897,547531.2355799843,532829.7264371039,518128.3673564764,503427.7803838169,488727.1934111556,474026.60643848684,459326.01946581155,444625.75837210845,429925.99278761726,415226.22720311955,400526.46161862,385826.6960341232,371126.93098454457,356427.8942220984,341728.85745966714,327029.82069724705,312330.78393479344,297631.747172378,282932.7104099393,268233.6736474931,253534.63688507117,238829.04018918425,224129.27460468933,209429.50902019627],[1665772.4107057643,1650950.6795589721,1636128.948412179,1621307.2172653805,1606485.4861185886,1591663.7549717922,1576842.0238249993,1562020.2926782044,1547198.5615314078,1532376.8303846144,1517555.0992378194,1502733.368091024,1487911.636944231,1473089.9057974345,1458268.1746506407,1443446.4435038457,1428624.7123570514,1413802.9812102567,1398981.2500634613,1384159.5189166686,1369337.78776987,1354516.0566230768,1339694.3254762846,1324872.5943294885,1310050.8631826935,1295229.1320358994,1280407.4008891054,1265585.66974231,1250763.938595514,1235942.2074487207,1221120.4763019257,1206298.7451551293,1191477.0140083341,1176655.282861542,1161833.5517147477,1147011.8205679506,1132190.0894211512,1117368.3582743648,1102546.627127566,1087724.8959807688,1072903.1648339806,1058081.433687185,1043259.7025403907,1028437.9713935899,1013616.2402467988,998794.5091000008,983975.3910106113,969157.4551025657,954342.0547737977,939527.8336955756,924715.1863895021,909903.9960837695,895093.8555591791,880285.0958561953,865477.2353197434,850670.3810221548,835864.5916284136,821059.1844775164,806255.2739442829,791451.363411041,776648.7067141877,761846.498043959,747044.381679331,732243.7119789114,717443.0422784919,702642.5703169247,687843.2897077389,673044.0090985354,658244.728489344,643446.6324480381,628648.603059954,613850.5736718727,599053.0254749497,584256.1205175165,569459.2155600823,554662.3106026454,539865.9403011808,525070.0432060445,510274.14611092117,495478.24901578855,480682.5189138409,465887.52254312485,451092.5261724163,436297.5298017012,421502.5334309777,406707.7825095188,391913.5884249117,377119.3943403112,362325.2002557097,347531.00617113337,332736.81208652817,317943.1742971074,303149.69208521396,288356.209873328,273562.7276614299,258769.24544955138,243975.76323766913,229182.2810257813,214388.79881389067,199589.06532512605],[1665283.80477309,1650368.3007705305,1635452.796767969,1620537.2927654083,1605621.7887628463,1590706.2847602856,1575790.7807577245,1560875.2767551667,1545959.7727526037,1531044.2687500427,1516128.7647474827,1501213.2607449214,1486297.756742361,1471382.2527398001,1456466.748737239,1441551.2447346793,1426635.7407321185,1411720.236729558,1396804.7327269944,1381889.2287244366,1366973.7247218727,1352058.2207193142,1337142.7167167536,1322227.2127141932,1307311.7087116314,1292396.20470907,1277480.7007065082,1262565.19670395,1247649.692701387,1232734.1886988285,1217818.6846962657,1202903.1806937072,1187987.6766911442,1173072.1726885843,1158156.668686022,1143241.164683463,1128325.6606809022,1113410.1566783371,1098494.6526757777,1083579.1486732215,1068663.644670655,1053748.1406680932,1038832.6366655319,1023917.1326629757,1009001.6286604125,994086.124657853,979170.6206552885,964256.9249063004,949344.6525645042,934435.0072232136,919525.996629782,904619.3601860153,889713.2728981753,874809.0835506236,859905.3407438309,845003.3770989832,830101.4723658133,815201.5305507574,800301.5887357099,785402.9144411143,770504.8067517951,755606.9718750874,740710.5255281972,725814.0791813377,710918.2670580102,696023.323038809,681128.3790196218,666234.0226596277,651340.4346580543,636446.8466564901,621553.4538487159,606661.0872772764,591768.72070585,576876.3541344162,561984.5969786746,547093.328065726,532202.0591527829,517310.7902398277,502420.07492763083,487529.7898802692,472639.5048328936,457749.2197855385,442859.03287898935,427969.6271102205,413080.2213414544,398190.8155726688,383301.4098039018,368412.0926465159,353523.47006179765,338634.8474771092,323746.2248924142,308857.60230769124,293968.9797229981,279080.6725742845,264192.74491293356,249304.81725160033,234416.88959025778,219528.96192890778,204641.03426757827,189753.10660622828],[1664795.198840416,1649785.9219820886,1634776.6451237586,1619767.3682654356,1604758.0914071063,1589748.8145487802,1574739.5376904523,1559730.2608321258,1544720.983973799,1529711.707115473,1514702.4302571472,1499693.1533988179,1484683.8765404923,1469674.5996821653,1454665.3228238388,1439656.0459655107,1424646.7691071827,1409637.4922488562,1394628.21539053,1379618.9385322046,1364609.6616738753,1349600.3848155483,1334591.1079572213,1319581.8310988958,1304572.5542405695,1289563.2773822425,1274554.0005239134,1259544.7236655897,1244535.4468072616,1229526.1699489343,1214516.8930906076,1199507.6162322797,1184498.339373952,1169489.0625156267,1154479.785657297,1139470.508798968,1124461.2319406494,1109451.955082322,1094442.6782239904,1079433.4013656653,1064424.1245073355,1049414.8476490136,1034405.5707906862,1019396.2939323564,1004387.0170740322,989377.7402157001,974368.4633573713,959359.1864990462,944350.7554484764,929344.6314299055,914340.1124935737,899336.8649857421,884335.5557071622,869334.9301257059,854336.2138961456,839337.9757256303,824341.3126614261,809345.2459081728,794350.1646304661,779356.0703941016,764362.1606258005,749369.8557803798,734377.5509349629,719386.0776591776,704395.3936356688,689404.7096121479,674415.0551615823,659425.8368236199,644436.6184856454,629448.2454328192,614460.350038873,599472.4546449296,584484.8971278928,569498.1933730962,554511.4896183088,539524.7858635271,524538.7126603518,509553.07979188766,494567.4469234282,479581.8140549669,464596.65774514247,449611.98474555183,434627.31174594536,419642.63874635845,404657.96574676316,389674.0940851085,374690.27891827933,359706.4637514474,344722.64858462755,329738.83341780026,314755.63769826666,299772.58661346696,284789.53552867007,269806.48444385827,254823.43335905485,239840.38905364554,224858.015942852,209875.64283205476,194893.2697212603,179910.89661045372],[1664306.592907739,1649203.5431936472,1634100.4934795527,1618997.4437654603,1603894.3940513674,1588791.3443372743,1573688.2946231808,1558585.2449090863,1543482.1951949936,1528379.1454809024,1513276.0957668058,1498173.0460527143,1483069.9963386224,1467966.9466245286,1452863.8969104374,1437760.8471963424,1422657.7974822498,1407554.747768157,1392451.6980540631,1377348.6483399724,1362245.5986258779,1347142.5489117827,1332039.4991976914,1316936.4494835993,1301833.3997695067,1286730.3500554124,1271627.3003413195,1256524.250627229,1241421.200913132,1226318.1511990419,1211215.1014849504,1196112.0517708536,1181009.0020567614,1165905.9523426662,1150802.9026285782,1135699.8529144828,1120596.8032003874,1105493.753486292,1090390.703772205,1075287.6540581142,1060184.6043440173,1045081.5546299238,1029978.5049158293,1014875.455201732,999772.4054876459,984669.3557735495,969566.3060594574,954463.2563453699,939360.2066312744,924257.1569171799,909156.9069417943,894057.3705848777,878960.1994875213,863863.7729529184,848769.1582337879,833675.2465957394,818582.9596022684,803491.0420070924,788400.8721454525,773310.7022838145,758222.2472098684,743134.0005521951,728046.5104967728,712960.008495274,697873.5064937649,682788.2327367263,667703.3110365886,652618.3893364379,637534.7459900677,622451.2533333199,607367.7606765768,592285.2386365077,577203.035850198,562120.833063867,547038.9916736521,531957.9507355029,516876.90979735367,501795.8688592268,486715.3725199625,471635.3756959997,456555.3788720323,441475.38204805646,426395.68875373434,411316.62780188676,396237.5668500615,381158.5058982223,366079.44494639523,351000.9234677674,335922.6989028873,320844.47433801927,305766.24977312423,290688.0252082553,275610.09133433737,260532.61174942367,245455.13216449972,230377.6525795944,215300.17299469654,200222.69340977818,185145.5051749386,170068.68661469035],[1663817.9869750643,1648621.1644052044,1633424.3418353442,1618227.519265484,1603030.6966956244,1587833.8741257656,1572637.0515559088,1557440.228986049,1542243.4064161903,1527046.5838463309,1511849.76127647,1496652.938706612,1481456.1161367516,1466259.2935668926,1451062.4709970355,1435865.648427175,1420668.8258573164,1405472.0032874572,1390275.180717597,1375078.3581477401,1359881.5355778805,1344684.7130080205,1329487.890438163,1314291.0678683023,1299094.2452984427,1283897.4227285846,1268700.600158726,1253503.7775888664,1238306.9550190067,1223110.1324491461,1207913.3098792888,1192716.48730943,1177519.6647395708,1162322.842169711,1147126.0195998517,1131929.1970299962,1116732.374460136,1101535.5518902764,1086338.729320414,1071141.906750558,1055945.084180696,1040748.2616108395,1025551.439040977,1010354.616471122,995157.7939012591,979960.9713313975,964764.1487615434,949567.3261916842,934370.5036218222,919173.6810519649,903976.8584821098,888781.4791001612,873587.651728082,858395.6868102355,843204.5921233473,828015.4080664795,812826.8042096561,797639.8458329849,782453.5100165494,767268.2921659509,752084.0191959096,736900.119183016,721717.7201040201,706535.3210250102,691354.0717433933,676173.3725858014,660992.6734282235,645813.437107739,630634.2777309557,615455.1183541715,600277.3395434413,585099.5725679183,569921.8055923972,554745.0022706967,539568.4920919975,524391.9819133068,509215.737485975,494040.3593644947,478864.9812429929,463689.6031214893,448514.57655754406,433340.2157780528,418165.8549985811,402991.4942190964,387817.1679533692,372643.7190492973,357470.2701452328,342296.82124117017,327123.3723371038,311950.1152582178,296777.4812952867,281604.8473323425,266432.2133694133,251259.57940648217,236086.94544354267,220914.92288494948,205743.01479993667,190571.1067149248,175399.1986299008,160227.29054488055],[1663329.3810423864,1648038.785616762,1632748.190191138,1617457.5947655109,1602166.999339886,1586876.4039142597,1571585.8084886353,1556295.2130630114,1541004.6176373847,1525714.0222117594,1510423.4267861336,1495132.8313605087,1479842.235934884,1464551.6405092594,1449261.045083634,1433970.4496580071,1418679.8542323825,1403389.2588067576,1388098.6633811307,1372808.0679555093,1357517.4725298837,1342226.8771042572,1326936.2816786314,1311645.6862530056,1296355.0908273824,1281064.4954017566,1265773.89997613,1250483.3045505062,1235192.7091248804,1219902.113699252,1204611.5182736316,1189320.9228480037,1174030.327422378,1158739.7319967537,1143449.1365711265,1128158.5411455007,1112867.945719875,1097577.3502942473,1082286.754868622,1066996.1594430008,1051705.564017375,1036414.968591752,1021124.3731661271,1005833.7777404999,990543.1823148839,975252.5868892488,959961.9914636249,944671.396038,929380.8006123728,914090.2051867475,898799.6097611259,883509.0143355015,868218.4189098743,852930.668856164,837643.3269538986,822358.3086774843,807073.6796237938,791791.0866292883,776508.6344177686,761228.2497783713,745947.8651389759,730669.1298912168,715390.7538127992,700113.1348132119,684836.5833129035,669560.031812571,654284.8443656815,639009.9480519891,623735.0878256205,608461.6907722019,593188.293718785,577914.8966653617,562642.835821717,547370.7945274264,532098.7532331357,516827.53633538727,501556.71876432374,486285.90119323786,471015.1345649045,455745.4192600604,440475.7039552117,425205.98865034897,409936.32477306575,394667.60003805626,379398.8753030794,364130.15056806523,348861.42583307717,333593.258487761,318325.4216314675,303057.58477515634,287789.74791887496,272521.9110625526,257254.6260954691,241987.5827344805,226720.53937349003,211453.49601251166,196186.4526515277,180919.52002007328,165653.1834349651,150386.846849842],[1662840.7751097116,1647456.4068283197,1632072.0385469277,1616687.6702655354,1601303.3019841444,1585918.9337027532,1570534.5654213612,1555150.19713997,1539765.8288585797,1524381.4605771888,1508997.0922957966,1493612.7240144052,1478228.355733013,1462843.9874516227,1447459.619170232,1432075.2508888408,1416690.88260745,1401306.514326057,1385922.1460446673,1370537.7777632764,1355153.4094818851,1339769.0412004949,1324384.6729190995,1309000.3046377082,1293615.936356317,1278231.568074926,1262847.199793535,1247462.8315121431,1232078.4632307522,1216694.09494936,1201309.7266679693,1185925.358386577,1170540.990105186,1155156.621823796,1139772.2535424037,1124387.885261016,1109003.5169796236,1093619.1486982312,1078234.7804168435,1062850.4121354497,1047466.0438540569,1032081.6755726705,1016697.3072912768,1001312.9390098834,985928.5707284925,970544.2024471024,955159.8341657114,939775.4658843172,924391.0976029267,909006.7293215333,893622.3610401461,878237.9927587495,862853.6244773543,847469.2561959676,832085.9826307236,816704.451905123,801324.1780525539,785945.2361866096,770567.3907735469,755190.8086413369,739814.8918942753,724440.458431921,709066.0343697397,693693.5551829161,678321.0759960879,662949.5407581506,647578.8368365113,632208.1329148784,616838.8283635881,601469.7348938249,586100.6414240552,570732.9937301045,555365.359000056,539997.7242699964,524631.2348249005,509264.91921183467,493898.6035987688,478532.84083058126,463167.71586712915,447802.59090368263,432437.4659402231,417073.13042221125,401709.0779340053,386345.02544579655,370980.97295759246,355617.5284760222,340254.43978550844,324891.3510949956,309528.26240450237,294165.24611730315,278803.02130877227,263440.79650024325,248078.5716917012,232716.34688316286,217354.13330344763,201992.68054440338,186631.227785361,171269.7750263177,155908.32226727903,140546.8695082292],[1662352.1691770349,1646874.0280398754,1631395.8869027179,1615917.7457655605,1600439.6046284037,1584961.4634912463,1569483.322354091,1554005.1812169333,1538527.040079772,1523048.8989426168,1507570.7578054592,1492092.616668303,1476614.4755311445,1461136.3343939884,1445658.193256832,1430180.0521196732,1414701.9109825145,1399223.7698453574,1383745.6287082022,1368267.4875710434,1352789.3464338852,1337311.205296729,1321833.0641595686,1306354.9230224118,1290876.7818852558,1275398.6407480978,1259920.499610939,1244442.3584737824,1228964.217336625,1213486.0761994685,1198007.9350623116,1182529.7939251545,1167051.6527879974,1151573.5116508412,1136095.3705136827,1120617.2293765247,1105139.0882393625,1089660.9471022035,1074182.8059650483,1058704.6648278902,1043226.5236907359,1027748.3825535779,1012270.2414164236,996792.100279267,981313.959142107,965835.818004949,950357.6768677868,934879.5357306339,919401.3945934735,903923.2534563169,888445.1123191598,872966.971182005,857488.830044847,842010.6889076941,826532.5477705318,811054.4066333808,795578.2106494112,780102.828247034,764629.3347931057,749156.4693376208,733685.1453725463,718214.5741026364,702744.9540789444,687276.4717936171,671808.1503362712,656341.568041049,640874.9857458407,625409.3370178249,609944.4806748759,594479.6243319223,579016.0237371614,563552.7331113042,548089.4424854359,532627.3459814703,517165.4735747874,501703.601168097,486242.53655297216,470781.94662112277,455321.3566892948,439860.91575625446,424401.4834004333,408942.0510445973,393482.61868879106,378023.4928509332,362565.1031793868,347106.7135078097,331648.3238362605,316190.00109191425,300732.54844591115,285275.0957998745,269817.643153863,254360.19050782733,238903.06917712558,223446.4564163573,207989.84365558624,192533.2308948217,177076.6181340497,161620.14076210558,146164.27860499732,130708.41644791886],[1661863.5632443589,1646291.649251433,1630719.7352585108,1615147.8212655885,1599575.9072726662,1584003.9932797397,1568432.0792868184,1552860.1652938933,1537288.251300971,1521716.3373080464,1506144.4233151237,1490572.5093221995,1475000.595329275,1459428.6813363521,1443856.7673434298,1428284.8533505038,1412712.939357582,1397141.0253646586,1381569.1113717349,1365997.1973788121,1350425.2833858856,1334853.369392963,1319281.4554000404,1303709.5414071183,1288137.6274141923,1272565.7134212705,1256993.7994283477,1241421.885435422,1225849.9714424976,1210278.057449575,1194706.1434566535,1179134.2294637284,1163562.3154708035,1147990.401477879,1132418.4874849543,1116846.573492032,1101274.6594991111,1085702.7455061902,1070130.8315132656,1054558.9175203443,1038987.0035274187,1023415.0895344955,1007843.1755415667,992271.2615486402,976699.3475557189,961127.4335627975,945555.5195698729,929983.6055769515,914411.691584026,898839.7775911023,883267.8635981772,867695.9496052563,852124.0356123345,836552.1216194117,820980.2076264909,805408.2936335569,789836.37964065,774264.4656477207,758695.0319610536,743125.7978819227,727558.7971755369,711992.0081305392,696426.9434207827,680862.3830131898,665298.8209717525,649736.2898634663,634173.8538692119,618613.1684656041,603052.4830619935,587492.5235922448,571933.5148279788,556374.5060637146,540816.4304863578,525258.9427044187,509701.4549224684,494144.74752629735,478588.63744296227,463032.5273596309,447476.7410059413,431921.876755327,416367.01250471827,400812.1482540993,385258.02136424,369704.281616034,354150.54186783545,338596.80211963877,323043.779691346,307491.05283643026,291938.32598151267,276385.5991265746,260833.20128422882,245281.38468269072,229729.56808115635,214177.75147963222,198625.93487809505,183074.56452354882,167523.5638105385,151972.5630975403,136421.56238454022,120870.56167153269],[1661374.957311682,1645709.2704629914,1630043.5836143035,1614377.8967656156,1598712.2099169225,1583046.5230682339,1567380.836219544,1551715.1493708561,1536049.462522165,1520383.775673476,1504718.0888247872,1489052.4019760957,1473386.7151274062,1457721.0282787168,1442055.3414300275,1426389.6545813372,1410723.9677326472,1395058.280883959,1379392.5940352695,1363726.9071865797,1348061.2203378885,1332395.5334891998,1316729.8466405093,1301064.159791822,1285398.4729431302,1269732.7860944404,1254067.0992457522,1238401.4123970624,1222735.725548371,1207070.038699683,1191404.351850993,1175738.6650023027,1160072.978153611,1144407.291304924,1128741.6044562324,1113075.9176075482,1097410.230758857,1081744.5439101597,1066078.8570614732,1050413.1702127857,1034747.4833640922,1019081.796515408,1003416.1096667103,987750.4228180279,972084.7359693395,956419.0491206441,940753.3622719599,925087.6754232673,909421.988574578,893756.301725891,878090.6148771993,862424.9280285137,846759.2411798234,831093.5543311252,815427.8674824364,799762.1806337498,784096.4937850544,768430.8069363758,752765.1200876785,737099.5323215369,721436.4465656485,705773.3608097602,690112.5651998539,674451.852565323,658792.7849182757,643134.2353729885,627476.4925727248,611819.9126414806,596163.3327102428,580508.3564565843,564853.5679445956,549199.1004814003,533545.9392958106,517892.77811023686,502240.04861120787,486588.3636731813,470936.67873514444,455285.1983645782,439634.85060461145,423984.50284465216,408334.1550846677,392684.70961442497,377035.5710450364,361386.43247564044,345737.3297585454,330089.2826179527,314441.23547739163,298793.1883368036,283145.1411962267,267498.0434316145,251850.9793933276,236203.91535507515,220556.8513167994,204910.21774347685,189264.03718643263,173617.85662937444,157971.67607233673,142325.4955152925,126679.73215655982,111034.34349132888],[1660886.3513790062,1645126.8916745507,1629367.4319700967,1613607.9722656384,1597848.512561184,1582089.052856727,1566329.5931522711,1550570.1334478152,1534810.6737433616,1519051.2140389043,1503291.7543344502,1487532.294629994,1471772.8349255372,1456013.3752210815,1440253.9155166256,1424494.4558121702,1408734.996107715,1392975.5364032593,1377216.0766988029,1361456.6169943481,1345697.1572898915,1329937.6975854374,1314178.2378809787,1298418.7781765258,1282659.3184720697,1266899.858767613,1251140.399063156,1235380.9393587024,1219621.4796542434,1203862.019949792,1188102.5602453353,1172343.1005408785,1156583.6408364233,1140824.1811319687,1125064.721427516,1109305.2617230495,1093545.8020185963,1077786.3423141432,1062026.8826096905,1046267.4229052309,1030507.9632007717,1014748.5034963181,998989.0437918655,983229.5840874072,967470.1243829527,951710.6646785,935951.2049740362,920191.7452695873,904432.2855651276,888672.8258606754,872913.3661562265,857153.9064517603,841394.4467473021,825634.9870428499,809875.5273383968,794116.0676339334,778356.6079294784,762597.1482250253,746837.6885205712,731078.2288161153,715318.7691116603,699559.3918958567,683802.4544632034,668045.5170305679,652290.6388660632,636536.0026419954,620782.6698650001,605030.1311820447,589277.9688818567,573527.3401276572,557776.7113734372,542027.1320140054,526278.2403936116,510529.3487732215,494781.7540783975,479034.44047151506,463287.12686461397,447540.99601758085,431795.1139234593,416049.2318293229,400304.11305969674,384559.5276230974,368814.942186499,353070.44519843254,337327.0323102707,321583.61942210514,305840.20653393306,290097.05405037757,274354.6995174233,258612.34498447366,242869.99045151845,227127.685293369,211386.2840717528,195644.8828501273,179903.48162850272,164162.0804068679,148421.05046964623,132680.5059570819,116939.96144452319,101199.41693195887],[1660397.745446334,1644544.512886109,1628691.2803258887,1612838.047765665,1596984.8152054427,1581131.5826452216,1565278.3500849998,1549425.1175247775,1533571.8849645555,1517718.6524043325,1501865.419844112,1486012.1872838894,1470158.9547236688,1454305.7221634462,1438452.4896032244,1422599.257043003,1406746.0244827813,1390892.7919225586,1375039.5593623365,1359186.326802116,1343333.0942418934,1327479.8616816718,1311626.6291214502,1295773.3965612277,1279920.1640010057,1264066.9314407841,1248213.698880561,1232360.4663203412,1216507.2337601169,1200654.0011998976,1184800.7686396737,1168947.5360794538,1153094.3035192331,1137241.0709590074,1121387.838398783,1105534.6058385675,1089681.373278346,1073828.1407181174,1057974.9081578972,1042121.6755976779,1026268.4430374596,1010415.2104772362,994561.9779170114,978708.7453567917,962855.5127965668,947002.2802363513,931149.0476761241,915295.815115903,899442.5825556796,883589.3499954599,867736.1174352351,851882.8848750135,836029.652314797,820176.4197545755,804323.1871943492,788469.9546341365,772616.7220739052,756763.4895136775,740910.2569534723,725057.0243932381,709203.7918330161,693350.5592727968,677497.326712573,661644.094152349,645793.0556537304,629942.2665443141,614093.0181741416,598244.4583605584,582396.5982609885,566550.070440351,550703.5426196977,534858.5723219691,519013.8947448032,503169.4951378405,487326.50040905643,471483.5056802705,455640.95917572547,439799.49314751197,423958.02711930405,408116.8397376314,392276.76048740465,376436.6812372068,360596.6019869847,344757.60169498995,328918.77858177386,313079.9554685317,297241.3963003913,281403.7090934431,265566.0218864912,249728.3346795682,233890.93398797233,218054.27206264157,202217.61013732478,186380.94821199868,170544.28628667258,154708.50161177386,138872.76320677064,123037.02480180189,107201.286396835,91365.69946272299],[1659909.1395136567,1643962.1340976665,1628015.1286816774,1612068.1232656925,1596121.1178497015,1580174.1124337143,1564227.1070177278,1548280.101601738,1532333.0961857499,1516386.0907697608,1500439.085353775,1484492.0799377873,1468545.074521799,1452598.0691058105,1436651.063689823,1420704.0582738356,1404757.0528578456,1388810.0474418586,1372863.0420258697,1356916.0366098813,1340969.0311938955,1325022.0257779062,1309075.020361919,1293128.014945932,1277181.0095299445,1261234.0041139547,1245286.9986979668,1229339.9932819805,1213392.98786599,1197445.9824500056,1181498.977034015,1165551.971618028,1149604.9662020397,1133657.9607860474,1117710.9553700648,1101763.9499540776,1085816.9445380834,1069869.9391221032,1053922.933706116,1037975.9282901194,1022028.9228741359,1006081.9174581505,990134.9120421619,974187.9066261677,958240.901210187,942293.8957941979,926346.890378207,910399.8849622142,894452.8795462348,878505.8741302504,862558.8687142516,846611.8632982676,830664.8578822785,814717.8524662852,798770.8470503055,782823.841634321,766876.836218331,750929.8308023484,734982.8253863407,719035.81997036,703088.8145543765,687141.8091383874,671194.803722404,655247.7983064102,639300.7928904286,623353.7874744292,607408.2501371959,591463.6093510352,575519.7031241115,559577.2197210044,543634.736317886,527694.0531478953,511753.536189558,495813.60922444053,479874.88282431755,463936.15642418154,447998.3479909329,432061.250153753,416124.15231658705,400187.9361382611,384252.3176887352,368316.6992392037,352381.61842702515,336447.3420207156,320513.06561442465,304578.7892081188,288645.6642704904,272712.60348060913,256779.54269074276,240846.70148968883,224914.73996397853,208982.77843826637,193050.81691253744,177118.96957131103,161188.00025360845,145257.03093591426,129326.0616182182,113395.09230051376,97464.69605164975,81534.6204633303],[1659420.5335809803,1643379.755309225,1627338.9770374694,1611298.198765714,1595257.4204939643,1579216.6422222103,1563175.8639504528,1547135.0856787008,1531094.3074069442,1515053.5291351918,1499012.750863437,1482971.972591684,1466931.1943199295,1450890.4160481761,1434849.6377764218,1418808.8595046652,1402768.0812329138,1386727.3029611588,1370686.524689405,1354645.74641765,1338604.9681458976,1322564.189874143,1306523.4116023874,1290482.633330634,1274441.855058879,1258401.0767871272,1242360.2985153743,1226319.5202436184,1210278.7419718632,1194237.9637001099,1178197.1854283575,1162156.4071566039,1146115.6288848491,1130074.8506130935,1114034.0723413434,1097993.2940695845,1081952.515797833,1065911.7375260754,1049870.9592543235,1033830.1809825711,1017789.4027108168,1001748.6244390612,985707.846167306,969667.0678955531,953626.2896237997,937585.5113520403,921544.733080287,905503.9548085365,889463.1765367831,873422.3982650293,857381.6199932694,841340.8417215231,825300.0634497674,809259.2851780117,793218.5069062607,777177.7286345009,761136.9503627466,745096.1720909942,729055.3938192436,713014.6155474801,696973.8372757407,680933.0590039752,664892.2807322182,648851.5024604835,632810.7241887199,616769.9459169563,600729.1676452002,584688.3893734561,568648.0379136298,552609.5454507126,536571.0529877916,520534.28672333155,504497.87973068655,488462.0793046877,472427.5732086655,456393.0671126433,440359.6756119691,424326.900388889,408294.12516580615,392262.58219365496,376231.38124809135,360200.18030252773,344169.9985729102,328140.2277020719,312110.4568312485,296081.21417987626,280052.7406174764,264024.2670551054,247995.79349270556,231968.30078615993,215941.00231966376,199913.70385314617,183886.40538663045,167860.12492185459,151833.889077371,135807.65323286224,119781.41738837957,103755.88409032486,87730.60738022253,71705.33067015931],[1658931.9276483045,1642797.3765207827,1626662.8253932623,1610528.274265742,1594393.7231382204,1578259.172010703,1562124.6208831812,1545990.06975566,1529855.518628139,1513720.9675006217,1497586.4163731001,1481451.8652455807,1465317.3141180612,1449182.76299054,1433048.2118630197,1416913.660735499,1400779.1096079794,1384644.5584804588,1368510.007352938,1352375.4562254192,1336240.9050978997,1320106.3539703805,1303971.802842859,1287837.251715338,1271702.7005878175,1255568.1494602982,1239433.5983327774,1223299.0472052577,1207164.4960777354,1191029.944950215,1174895.3938226956,1158760.8426951752,1142626.291567653,1126491.7404401386,1110357.189312613,1094222.6381850974,1078088.087057571,1061953.5359300561,1045818.9848025409,1029684.4336750163,1013549.8825474987,997415.3314199713,981280.7802924565,965146.2291649347,949011.6780374143,932877.1269098949,916742.5757823698,900608.0246548494,884473.4735273393,868338.9223998087,852204.3712722939,836069.8201447725,819935.2690172526,803800.7178897331,787666.1667622151,771531.6156346835,755397.0645071641,739262.513379652,723127.9622521382,706993.4111246094,690858.8599970834,674724.3088695705,658589.7577420482,642455.2066145353,626320.6554870084,610186.1043594852,594051.5532319685,577917.0021044556,561782.4509769278,545647.8998494092,529513.3487218833,513380.0748433443,497247.73070366494,481115.65936754644,464985.328785385,448854.99820321053,432725.6536770193,416597.15844329353,400468.6632095929,384341.4490617532,368214.6250156928,352087.8009696463,335962.1977460347,319836.89369207155,303711.5896381093,287587.1464796681,271463.2231875118,255339.29989536386,239215.62699620426,223092.95627772622,206970.28555923328,190847.61484075617,174725.51124201715,158603.9750988772,142482.43895573728,126360.90281256288,110239.86396710202,94119.35380382277,77998.84364054166,61878.33347729221],[1658443.3217156287,1642214.9977323408,1625986.6737490566,1609758.3497657687,1593530.0257824822,1577301.7017991967,1561073.3778159088,1544845.0538326234,1528616.7298493362,1512388.4058660497,1496160.0818827646,1479931.7578994776,1463703.4339161925,1447475.1099329044,1431246.785949618,1415018.4619663323,1398790.1379830453,1382561.8139997595,1366333.4900164737,1350105.1660331876,1333876.842049902,1317648.5180666142,1301420.194083328,1285191.8701000414,1268963.5461167544,1252735.222133468,1236506.8981501823,1220278.5741668951,1204050.2501836084,1187821.9262003212,1171593.6022170386,1155365.2782337535,1139136.954250466,1122908.6302671814,1106680.3062838935,1090451.9823006056,1074223.6583173177,1057995.334334041,1041767.0103507456,1025538.6863674661,1009310.3623841736,993082.0384008931,976853.7144176001,960625.3904343182,944397.0664510257,928168.7424677406,911940.4184844629,895712.0945011713,879483.7705178862,863255.4465345959,847027.1225513173,830798.7985680238,814570.4745847415,798342.1506014499,782113.8266181601,765885.5026348904,749657.1786515927,733428.8546683034,717200.5306850243,700972.2067017434,684743.8827184476,668515.5587351527,652287.2347518727,636058.9107685955,619830.5867853062,603602.2628020104,587373.9388187323,571145.614835456,554917.2908521565,538688.9668688793,522460.6428855844,506232.3189022979,490003.9949190095,473775.6709357314,457549.0017124992,441322.8058960531,425097.08348195255,408872.8293102598,392648.57513854466,376425.293223219,360202.80885183066,343980.3244804293,327758.9295737343,311538.05670472886,295317.18383572716,279097.1946480498,262877.787485688,246658.3803233467,230439.3798585441,214221.30414507538,198003.22843160015,181785.15271814354,165567.98900142778,149351.12112685293,133134.25325228367,116917.38537771441,100701.52181825787,84485.74799847789,68269.97417869978,52054.200358895585],[1657954.7157829523,1641632.6189439,1625310.5221048477,1608988.4252657928,1592666.3284267401,1576344.2315876894,1560022.1347486372,1543700.0379095841,1527377.941070532,1511055.8442314814,1494733.747392429,1478411.6505533736,1462089.553714322,1445767.4568752693,1429445.3600362164,1413123.2631971645,1396801.1663581128,1380479.06951906,1364156.9726800073,1347834.875840955,1331512.7790019028,1315190.6821628485,1298868.5853237968,1282546.4884847444,1266224.391645693,1249902.2948066415,1233580.1979675887,1217258.1011285342,1200936.004289483,1184613.9074504294,1168291.8106113812,1151969.713772324,1135647.6169332778,1119325.5200942196,1103003.423255174,1086681.3264161176,1070359.22957707,1054037.132738011,1037715.0358989658,1021392.9390599076,1005070.8422208559,988748.7453818037,972426.6485427539,956104.5517036999,939782.4548646472,923460.3580255979,907138.2611865439,890816.1643474875,874494.0675084367,858171.9706693804,841849.8738303347,825527.7769912756,809205.6801522304,792883.5833131731,776561.4864741173,760239.3896350674,743917.2927960269,727595.1959569575,711273.0991178993,694951.0022788625,678628.9054398155,662306.8086007433,645984.7117616953,629662.6149226502,613340.518083591,597018.4212445328,580696.3244054997,564374.2275664378,548052.1307273814,531730.0338883344,515407.93704928644,499085.84021022543,482763.743371184,466441.64653212857,450119.5496930769,433797.4528540233,417475.3560149707,401154.81852108426,384834.7710278742,368514.9660591865,352196.7882979596,335878.61053672805,319560.9979432924,303244.5244342098,286928.05092513654,270612.1171479821,254297.1954559991,237982.27376403287,221667.57289971504,205354.06262895092,189040.55235819798,172727.0420874562,156414.47057474498,140102.2424399564,123790.01430517156,107477.83878858946,91166.77375792712,74855.7087272685,58544.64369660616,42233.642477855086],[1657466.1098502763,1641050.2401554578,1624634.3704606348,1608218.5007658168,1591802.6310709994,1575386.7613761823,1558970.891681362,1542555.0219865446,1526139.1522917273,1509723.2825969094,1493307.412902087,1476891.5432072703,1460475.6735124525,1444059.8038176342,1427643.934122815,1411228.064427998,1394812.194733177,1378396.3250383595,1361980.4553435417,1345564.5856487232,1329148.7159539042,1312732.8462590862,1296316.9765642679,1279901.1068694484,1263485.2371746302,1247069.3674798133,1230653.4977849962,1214237.6280901765,1197821.758395354,1181405.8887005378,1164990.019005722,1148574.1493108994,1132158.2796160826,1115742.4099212675,1099326.5402264409,1082910.6705316254,1066494.8008368057,1050078.9311419912,1033663.0614471715,1017247.191752357,1000831.3220575377,984415.4523627148,967999.5826678951,951583.7129730792,935167.8432782604,918751.9735834445,902336.1038886262,885920.2341938051,869504.364498985,853088.4948041663,836672.6251093522,820256.7554145302,803840.8857197096,787425.0160248983,771009.1463300763,754593.2766352538,738177.4069404397,721761.53724562,705345.667550805,688929.7978559807,672513.9281611564,656098.0584663404,639682.1887715356,623266.3190767113,606850.4493818823,590434.5796870701,574018.7099922439,557602.8402974391,541186.9706026213,524771.1009077951,508355.23121298105,491939.3615181623,475523.49182333704,459107.6221285267,442691.7524337033,426275.88273888547,409860.0130440695,393444.1433492331,377028.2736544339,360612.4039596189,344197.5252691135,327783.626099146,311369.7269291729,294957.2057484947,278545.10439774115,262133.00304697547,245722.3051904766,229311.8425437035,212901.37989693973,196492.04126951005,180083.07075456623,163674.10023963638,147265.71912185382,130858.10574272648,114450.49236357585,98042.87898442522,81636.3419204168,65229.96136431396,48823.58080821112,32417.243452453986],[1656977.5039175996,1640467.8613670107,1623958.2188164277,1607448.5762658457,1590938.9337152592,1574429.291164676,1557919.648614091,1541410.0060635041,1524900.363512922,1508390.7209623384,1491881.0784117507,1475371.4358611673,1458861.7933105822,1442352.1507599987,1425842.508209414,1409332.8656588292,1392823.2231082455,1376313.58055766,1359803.9380070749,1343294.2954564912,1326784.6529059054,1310275.0103553222,1293765.3678047373,1277255.7252541522,1260746.0827035683,1244236.4401529834,1227726.7976023997,1211217.1550518167,1194707.5125012281,1178197.8699506428,1161688.2274000589,1145178.5848494759,1128668.9422988882,1112159.2997483108,1095649.6571977194,1079140.0146471323,1062630.3720965548,1046120.7295459686,1029611.0869953879,1013101.4444448012,996591.8018942121,980082.1593436287,963572.516793048,947062.874242459,930553.231691875,914043.5891412892,897533.9465907072,881024.3040401195,864514.661489536,848005.018938953,831495.3763883659,814985.7338377857,798476.091287198,781966.4487366118,765456.8061860306,748947.163635456,732437.5210848553,715927.8785342751,699418.2359836996,682908.5934331054,666398.950882527,649889.3083319422,633379.6657813406,616870.023230765,600360.3806801904,583850.7381295906,567341.095579017,550831.4530284293,534321.810477851,517812.1679272689,501302.52537667844,484792.8828260908,468283.2402755087,451773.597724922,435263.95517434645,418754.3126237476,402244.6700731646,385735.02752258163,369225.38497201167,352715.7424214035,336206.09987083357,319696.4573202608,303186.8147696508,286677.1722190846,270169.37110984325,253661.62026311923,237154.08166184835,220648.05672156904,204142.0317812711,187636.15112061612,171131.69933615252,154627.24755168986,138122.79576722998,121619.57480733842,105116.55546944495,88613.5361315608,72111.04047687538,55609.32398931682,39107.6075018011,22605.891014257446],[1656488.8979849233,1639885.482578569,1623282.0671722202,1606678.6517658709,1590075.236359518,1573471.8209531694,1556868.4055468184,1540264.9901404672,1523661.5747341162,1507058.1593277678,1490454.7439214136,1473851.3285150651,1457247.913108714,1440644.4977023646,1424041.0822960127,1407437.6668896617,1390834.2514833102,1374230.8360769595,1357627.4206706109,1341024.0052642599,1324420.5898579084,1307817.1744515589,1291213.7590452055,1274610.3436388571,1258006.9282325066,1241403.5128261556,1224800.0974198037,1208196.6820134562,1191593.2666071008,1174989.8512007496,1158386.4357944024,1141783.0203880533,1125179.6049816976,1108576.1895753504,1091972.7741690003,1075369.3587626466,1058765.9433562956,1042162.5279499479,1025559.1125435946,1008955.6971372413,992352.2817308903,975748.8663245435,959145.4509181925,942542.0355118429,925938.6201054938,909335.2046991405,892731.7892927905,876128.3738864376,859524.9584800871,842921.5430737385,826318.1276673847,809714.7122610351,793111.2968546911,776507.8814483369,759904.4660419887,743301.0506356359,726697.6352292793,710094.2198229302,693490.8044165745,676887.389010231,660283.9736038791,643680.5581975402,627077.1427911837,610473.7273848327,593870.311978464,577266.8965721317,560663.4811657742,544060.0657594213,527456.6503530778,510853.23494671937,494249.8195403684,477646.4041340314,461042.98872766364,444439.5733213164,427836.1579149645,411232.7425086256,394629.3271022737,378025.9116959227,361422.4962895671,344819.080883218,328215.66547686514,311612.2500705011,295008.8346641697,278405.41925782803,261802.00385146774,245198.5884451149,228595.17303876393,211992.00605998654,195390.40353649203,178788.80101301614,162187.46750823595,145587.5189784132,128987.57044860069,112387.62191878445,95789.18038032018,79190.73945816234,62592.298536023125,45994.71776146628,29397.649600625038,12800.58143977262],[1656000.2920522473,1639303.1037901284,1622605.9155280136,1605908.7272658944,1589211.539003781,1572514.3507416616,1555817.1624795478,1539119.9742174293,1522422.785955311,1505725.5976931965,1489028.409431077,1472331.2211689618,1455634.032906844,1438936.8446447277,1422239.6563826115,1405542.468120494,1388845.2798583778,1372148.0915962607,1355450.9033341436,1338753.7150720283,1322056.5268099103,1305359.3385477914,1288662.1502856775,1271964.962023561,1255267.773761445,1238570.585499327,1221873.3972372075,1205176.2089750944,1188479.0207129759,1171781.832450857,1155084.6441887436,1138387.4559266288,1121690.2676645084,1104993.079402389,1088295.8911402705,1071598.7028781623,1054901.514616041,1038204.3263539211,1021507.1380918045,1004809.9498296948,988112.7615675735,971415.5733054541,954718.3850433403,938021.1967812232,921324.0085191047,904626.8202569876,887929.6319948751,871232.4437327506,854535.2554706419,837838.0672085257,821140.8789464063,804443.6906842864,787746.5024221684,771049.3141600611,754352.12589793,737654.9376358213,720957.7493737154,704260.5611115806,687563.3728494672,670866.1845873604,654168.9963252349,637471.8080631224,620774.6198010053,604077.4315388901,587380.2432767767,570683.0550146587,553985.8667525388,537288.6784904189,520591.4902282916,503894.3019661857,487197.113704063,470499.9254419459,453802.7371798372,437105.548917721,420408.36065560766,403711.17239348684,387013.9841313707,370316.7958692508,353619.60760712996,336922.41934501193,320225.231082893,303528.04282079265,286830.85455866717,270133.6662965454,253436.4780344423,236739.28977231216,220042.10151020065,203344.913248091,186647.72498596087,169950.53672385216,153253.34846173227,136556.16019961052,119860.62254908495,103165.16834883206,86469.71414859407,69775.59251904115,53081.72039969079,36387.84828032553,19694.748322984204,3002.3182631470263],[1655511.6861195713,1638720.725001686,1621929.7638838075,1605138.8027659215,1588347.8416480399,1571556.8805301576,1554765.9194122716,1537974.9582943895,1521183.9971765066,1504393.0360586257,1487602.0749407394,1470811.1138228602,1454020.1527049749,1437229.1915870928,1420438.2304692094,1403647.2693513262,1386856.3082334443,1370065.3471155607,1353274.3859976784,1336483.4248797966,1319692.4637619113,1302901.5026440285,1286110.5415261465,1269319.5804082649,1252528.6192903821,1235737.6581724973,1218946.6970546145,1202155.7359367316,1185364.7748188502,1168573.8137009637,1151782.8525830796,1134991.8914651996,1118200.9303473206,1101409.9692294374,1084619.008111551,1067828.0469936668,1051037.0858757873,1034246.1247579074,1017455.163640019,1000664.2025221381,983873.2414042517,967082.2802863694,950291.3191684922,933500.3580506081,916709.3969327188,899918.4358148389,883127.474696957,866336.5135790706,849545.5524611971,832754.5913433079,815963.6302254265,799172.6691075414,782381.7079896592,765590.7468717704,748799.7857538881,732008.824636003,715217.8635181254,698426.9024002533,681635.9412823664,664844.9801644823,648054.0190466037,631263.0579287065,614472.0968108298,597681.1356929466,580890.1745750625,564099.2134571867,547308.2523393026,530517.2912214054,513726.3301035296,496935.36898565385,480144.4078677725,463353.44674987905,446562.48563200235,429771.52451411635,412980.56339623965,396189.6022783555,379398.64116046764,362607.6800425844,345816.7189247012,329025.75780681707,312234.79668893013,295443.835571059,278652.87445316557,261861.91333529446,245070.95221741218,228279.99109951593,211489.02998163924,194698.0688637523,177907.1077458663,161116.14662798494,144325.18551009614,127534.22439222131,110743.26327434275,93952.30215647165,77161.34103858098,60370.37992068194,43580.02814759128,26790.722270576283,10001.41639356874,-6787.722656559199],[1655023.080186896,1638138.3462132458,1621253.6122395948,1604368.878265949,1587484.1442922996,1570599.410318649,1553714.676344999,1536829.9423713512,1519945.2083977014,1503060.474424054,1486175.740450404,1469291.0064767557,1452406.272503105,1435521.5385294564,1418636.8045558075,1401752.070582161,1384867.3366085088,1367982.6026348618,1351097.8686612123,1334213.1346875639,1317328.4007139138,1300443.6667402636,1283558.9327666152,1266674.1987929677,1249789.4648193172,1232904.730845669,1216019.9968720202,1199135.2628983692,1182250.5289247222,1165365.794951072,1148481.0609774236,1131596.327003776,1114711.5930301263,1097826.8590564816,1080942.12508283,1064057.3911091816,1047172.6571355285,1030287.9231618778,1013403.1891882289,996518.4552145838,979633.7212409358,962748.9872672898,945864.2532936349,928979.519319986,912094.7853463348,895210.0513726887,878325.3173990366,861440.5834253887,844555.8494517342,827671.1154780928,810786.3815044407,793901.6475307955,777016.9135571411,760132.1795834927,743247.4456098452,726362.7116362043,709477.97766254,692593.243688901,675708.5097152432,658823.7757416042,641939.0417679474,625054.3077943008,608169.5738206571,591284.8398469985,574400.1058733603,557515.3718997138,540630.637926056,523745.903952403,506861.16997876484,489976.43600511085,473091.702031455,456206.9680578178,439322.2340841703,422437.5001105191,405552.76613686886,388668.0321632158,371783.2981895646,354898.5642159255,338013.8302422613,321129.09626862314,304244.3622949766,287359.62832131516,270474.8943476649,253590.16037401836,236705.42640038393,219820.69242671598,202935.95845308248,186051.2244794229,169166.49050578196,152281.756532141,135397.0225584805,118512.28858483583,101627.55461118091,84742.82063754741,67858.08666389249,50973.352690238506,34088.618716588244,17203.88474295847,319.15076928399503,-16565.58320436068],[1654534.4742542193,1637555.967424803,1620577.4605953873,1603598.95376597,1586620.446936558,1569641.9401071444,1552663.4332777285,1535684.9264483117,1518706.4196188976,1501727.9127894822,1484749.4059600662,1467770.8991306506,1450792.3923012384,1433813.8854718215,1416835.3786424058,1399856.8718129932,1382878.3649835761,1365899.8581541618,1348921.3513247452,1331942.8444953302,1314964.3376659157,1297985.830836502,1281007.3240070855,1264028.8171776708,1247050.3103482546,1230071.803518841,1213093.2966894247,1196114.789860011,1179136.283030596,1162157.7762011783,1145179.2693717633,1128200.762542347,1111222.2557129315,1094243.7488835156,1077265.2420541025,1060286.7352246935,1043308.2283952776,1026329.7215658631,1009351.2147364467,992372.7079070313,975394.201077613,958415.6942481971,941437.1874187822,924458.6805893653,907480.1737599522,890501.6669305395,873523.1601011213,856544.6532717082,839566.1464422904,822587.6396128726,805609.1327834628,788630.6259540487,771652.1191246384,754673.6122952197,737695.1054657977,720716.5986363823,703738.0918069705,686759.5849775542,669781.0781481406,652802.571318714,635824.0644893032,618845.5576598924,601867.0508304816,584888.5440010661,567910.0371716414,550931.5303422399,533953.023512817,516974.51668340154,499996.00985399075,483017.50302457716,466038.99619515333,449060.48936574254,432081.98253632244,415103.4757069051,398124.96887748875,381146.46204808913,364167.9552186774,347189.4483892666,330210.94155984744,313232.43473043013,296253.9279010091,279275.42107159086,262296.91424218286,245318.4074127674,228339.90058334917,211361.3937539328,194382.88692452107,177404.38009510655,160425.87326569576,143447.36643627286,126468.85960685555,109490.35277744476,92511.84594803676,75533.33911860641,58554.832289192826,41576.325459796935,24597.818630378693,7619.311800967902,-9359.195028452203,-26337.701857864857],[1654045.8683215436,1636973.588636361,1619901.30895118,1602829.0292659993,1585756.7495808166,1568684.469895634,1551612.190210457,1534539.910525275,1517467.6308400927,1500395.3511549109,1483323.0714697305,1466250.791784548,1449178.5120993657,1432106.2324141862,1415033.9527290044,1397961.6730438222,1380889.3933586415,1363817.1136734607,1346744.8339882796,1329672.5543030978,1312600.2746179174,1295527.994932736,1278455.7152475575,1261383.4355623738,1244311.155877193,1227238.8761920112,1210166.5965068291,1193094.3168216506,1176022.037136468,1158949.757451287,1141877.477766106,1124805.1980809234,1107732.9183957423,1090660.6387105621,1073588.3590253792,1056516.0793401967,1039443.7996550137,1022371.5199698354,1005299.2402846552,988226.9605994704,971154.6809142898,954082.4012291157,937010.1215439318,919937.8418587502,902865.5621735663,885793.2824883847,868721.002803205,851648.7231180212,834576.4434328438,817504.1637476604,800431.8840624811,783359.604377301,766287.3246921161,749215.0450069346,732142.7653217651,715070.4856365705,697998.2059513945,680925.9262662055,663853.646581036,646781.3668958396,629709.0872106785,612636.8075254792,595564.5278403107,578492.2481551189,561419.9684699522,544347.6887847641,527275.4090995751,510203.1294144001,493130.8497292213,476058.5700440388,458986.2903588619,441914.01067366265,424841.730988496,407769.45130330883,390697.17161812726,373624.89193295036,356552.6122477744,339480.33256259095,322408.05287740193,305335.77319222596,288263.4935070509,271191.21382185444,254118.93413668312,237046.65445149038,219974.37476633023,202902.09508113656,185829.81539596058,168757.5357107725,151685.25602559838,134612.97634041123,117540.69665523339,100468.41697004251,83396.13728487678,66323.85759968497,49251.57791450247,32179.29822931625,15107.018544137478,-1965.26114102453,-19037.540826205164,-36109.82051139884],[1653557.2623888678,1636391.2098479196,1619225.1573069724,1602059.1047660257,1584893.0522250785,1567726.9996841282,1550560.9471431812,1533394.8946022335,1516228.8420612859,1499062.78952034,1481896.7369793924,1464730.684438445,1447564.6318974993,1430398.5793565523,1413232.5268156026,1396066.474274656,1378900.4217337088,1361734.3691927618,1344568.3166518142,1327402.2641108665,1310236.2115699186,1293070.1590289737,1275904.1064880244,1258738.053947077,1241572.0014061308,1224405.9488651832,1207239.8963242362,1190073.8437832897,1172907.791242342,1155741.7387013924,1138575.686160447,1121409.6336195003,1104243.5810785526,1087077.5285376073,1069911.4759966577,1052745.4234557115,1035579.3709147666,1018413.3183738175,1001247.2658328698,984081.213291923,966915.1607509749,949749.1082100295,932583.0556690795,915417.0031281314,898250.9505871814,881084.8980462328,863918.845505286,846752.7929643393,829586.7404233953,812420.6878824467,795254.6353414962,778088.5828005518,760922.530259599,743756.477718656,726590.4251776999,709424.3726367624,692258.320095812,675092.2675548662,657926.215013924,640760.1624729717,623594.109932024,606428.057391081,589262.0048501352,572095.95230918,554929.8997682445,537763.8472272921,520597.7946863435,503431.7421453912,486265.6896044435,469099.63706350606,451933.58452254906,434767.53198160883,417601.47944066674,400435.42689972,383269.37435876764,366103.32181782555,348937.2692768751,331771.2167359274,314605.16419498343,297439.1116540311,280273.0591130825,263107.0065721385,245940.95403118804,228774.90149023756,211608.8489492992,194442.79640834592,177276.74386740662,160110.69132645335,142944.6387855122,125778.58624456171,108612.53370360844,91446.48116265517,74280.42862172145,57114.37608077377,39948.32353982143,22782.270998883992,5616.218457920477,-11549.834083011374,-28715.886623978615,-45881.93916491233],[1653068.656456191,1635808.8310594787,1618549.005662765,1601289.1802660506,1584029.354869335,1566769.529472623,1549509.7040759097,1532249.878679197,1514990.0532824807,1497730.2278857692,1480470.4024890559,1463210.5770923442,1445950.7516956306,1428690.9262989163,1411431.1009022014,1394171.2755054892,1376911.4501087754,1359651.6247120616,1342391.7993153487,1325131.9739186342,1307872.1485219216,1290612.3231252087,1273352.4977284954,1256092.6723317814,1238832.8469350683,1221573.0215383552,1204313.1961416425,1187053.3707449283,1169793.5453482154,1152533.719951501,1135273.89455479,1118014.069158075,1100754.243761363,1083494.418364645,1066234.592967933,1048974.7675712146,1031714.9421745031,1014455.1167777888,997195.2913810769,979935.4659843645,962675.6405876521,945415.8151909397,928155.989794225,910896.1643975149,893636.3390008016,876376.513604084,859116.6882073693,841856.8628106606,824597.0374139422,807337.212017233,790077.3866205197,772817.5612238054,755557.7358270846,738297.910430382,721038.085033657,703778.2596369581,686518.4342402294,669258.6088435343,651998.7834468065,634738.9580501048,617479.1326533817,600219.3072566679,582959.4818599569,565699.6564632496,548439.8310665293,531180.0056698238,513920.18027310725,496660.3548763897,479400.5294796787,462140.7040829584,444880.87868623994,427621.0532895243,410361.22789281886,393101.4024961125,375841.5770993866,358581.7517026821,341321.92630596645,324062.1009092564,306802.275512537,289542.45011583343,272282.6247191252,255022.79932239745,237762.97392569575,220503.1485289745,203243.32313226722,185983.49773554318,168723.6723388387,151463.84694212023,134204.02154542133,116944.19614869263,99684.37075198162,82424.54535526969,65164.719958549365,47904.89456184395,30645.069165123627,13385.243768399581,-3874.581628292799,-21134.40702500567,-38394.232421724126,-55654.05781846307],[1652580.0505235142,1635226.4522710368,1617872.8540185536,1600519.2557660753,1583165.657513597,1565812.0592611148,1548458.4610086372,1531104.862756158,1513751.264503678,1496397.6662511965,1479044.0679987199,1461690.4697462413,1444336.8714937603,1426983.2732412792,1409629.6749887997,1392276.076736322,1374922.478483841,1357568.8802313635,1340215.2819788824,1322861.6837264022,1305508.0854739242,1288154.4872214447,1270800.8889689646,1253447.2907164844,1236093.6924640066,1218740.0942115246,1201386.495959044,1184032.897706565,1166679.299454087,1149325.7012016084,1131972.1029491303,1114618.504696649,1097264.90644417,1079911.3081916869,1062557.7099392074,1045204.1116867289,1027850.5134342499,1010496.9151817728,993143.3169292947,975789.7186768125,958436.120424333,941082.5221718485,923728.9239193713,906375.3256668923,889021.7274144138,871668.1291619376,854314.5309094512,836960.9326569708,819607.3344044979,802253.7361520189,784900.1378995399,767546.5396470577,750192.9413945749,732839.343142096,715485.7448896207,698132.1466371398,680778.5483846627,663424.9501321809,646071.3518797047,628717.7536272276,611364.1553747449,594010.5571222724,576656.9588697888,559303.3606173005,541949.7623648271,524596.1641123444,507242.56585986447,489888.9676073836,472535.3693548972,455181.77110242285,437828.1728499355,420474.5745974714,403120.9763449868,385767.37809251156,368413.7798400307,351060.18158753496,333706.5833350625,316352.9850825779,298999.3868301045,281645.7885776209,264292.19032515027,246938.59207267407,229584.99382019602,212231.39556771703,194877.79731523525,177524.19906275813,160170.6008102782,142817.00255780295,125463.4043053044,108109.80605282076,90756.20780034829,73402.6095478693,56049.01129539404,38695.41304292157,21341.814790455624,3988.2165379561484,-13365.381714504212,-30718.979967001826,-48072.57821948826,-65426.176471948624],[1652091.4445908389,1634644.0734825947,1617196.702374347,1599749.331266102,1582301.9601578559,1564854.5890496084,1547407.2179413647,1529959.8468331196,1512512.4757248727,1495065.1046166273,1477617.7335083825,1460170.362400136,1442722.9912918902,1425275.6201836462,1407828.2490753978,1390380.8779671548,1372933.506858908,1355486.135750662,1338038.7646424165,1320591.3935341705,1303144.022425926,1285696.6513176805,1268249.2802094338,1250801.9091011893,1233354.5379929435,1215907.1668846963,1198459.7957764533,1181012.4246682045,1163565.0535599617,1146117.6824517166,1128670.3113434692,1111222.9402352227,1093775.5691269757,1076328.1980187315,1058880.8269104878,1041433.4558022446,1023986.0846939976,1006538.7135857446,989091.3424775028,971643.9713692591,954196.6002610154,936749.229152766,919301.8580445219,901854.4869362707,884407.1158280293,866959.7447197833,849512.3736115363,832065.0025032917,814617.6313950489,797170.2602868024,779722.8891785559,762275.5180703104,744828.1469620587,727380.775853809,709933.404745575,692486.0336373216,675038.6625290867,657591.2914208332,640143.9203125862,622696.5492043421,605249.1780960923,587801.8069878593,570354.435879603,552907.0647713616,535459.6936631175,518012.32255486865,500564.95144662354,483117.58033836074,465670.20923013054,448222.8381218873,430775.4670136487,413328.0959053943,395880.7247971473,378433.3536888985,360985.9825806599,343538.6114724139,326091.24036417063,308643.8692559209,291196.4981476711,273749.12703944184,256301.75593118556,238854.38482294139,221407.01371469814,203959.64260643348,186512.27149820887,169064.9003899591,151617.52928173076,134170.15817346983,116722.78706522193,99275.41595698055,81828.044848728,64380.67374049593,46933.30263223499,29485.931523988023,12038.560415742919,-5408.810692479834,-22856.181800743565,-40303.552908999845,-57750.92401723191,-75198.29512548633],[1651602.8386581624,1634061.69469415,1616520.5507301376,1598979.4067661255,1581438.262802115,1563897.118838102,1546355.9748740904,1528814.8309100792,1511273.6869460694,1493732.5429820553,1476191.3990180455,1458650.2550540308,1441109.1110900221,1423567.967126011,1406026.8231619964,1388485.679197985,1370944.535233974,1353403.3912699628,1335862.2473059492,1318321.1033419392,1300779.959377926,1283238.815413916,1265697.6714499043,1248156.527485891,1230615.3835218786,1213074.2395578665,1195533.095593858,1177991.951629843,1160450.8076658328,1142909.663701823,1125368.5197378104,1107827.375773795,1090286.2318097851,1072745.0878457772,1055203.9438817636,1037662.7999177473,1020121.6559537388,1002580.5119897267,985039.3680257201,967498.224061701,949957.0800976884,932415.9361336818,914874.7921696738,897333.6482056584,879792.5042416425,862251.3602776327,844710.2163136182,827169.072349607,809627.9283855995,792086.7844215874,774545.6404575687,757004.496493563,739463.3525295537,721922.2085655453,704381.064601534,686839.9206375182,669298.776673507,651757.6327094892,634216.4887454743,616675.3447814696,599134.200817462,581593.0568534434,564051.912889435,546510.7689254228,528969.6249614051,511428.48099739477,493887.3370333826,476346.1930693686,458805.0491053546,441263.9051413508,423722.7611773396,406181.6172133228,388640.47324931435,371099.3292853022,353558.18532130495,336017.0413572863,318475.8973932713,300934.7534292508,283393.6094652461,265852.46550122276,248311.3215372134,230770.17757320497,213229.03360920027,195687.88964518812,178146.74568118993,160605.60171717033,143064.45775315445,125523.3137891423,107982.16982512455,90441.02586111333,72899.88189710397,55358.737933089025,37817.59396908432,20276.4500050731,2735.3060410562903,-14805.837922966108,-32346.981886960566,-49888.1258509811,-67429.26981501095,-84970.41377900355],[1651114.2327254869,1633479.3159057065,1615844.3990859315,1598209.48226615,1580574.5654463754,1562939.6486265957,1545304.7318068189,1527669.814987041,1510034.8981672633,1492399.981347484,1474765.0645277095,1457130.1477079284,1439495.2308881518,1421860.3140683738,1404225.3972485957,1386590.4804288165,1368955.5636090403,1351320.6467892628,1333685.729969484,1316050.813149706,1298415.8963299294,1280780.9795101515,1263146.0626903719,1245511.145870595,1227876.229050819,1210241.312231041,1192606.3954112614,1174971.4785914873,1157336.5617717071,1139701.6449519256,1122066.7281321534,1104431.8113123719,1086796.8944925983,1069161.9776728172,1051527.0608530384,1033892.1440332662,1016257.2272134884,998622.3103937004,980987.3935739286,963352.4767541518,945717.5599343712,928082.6431145924,910447.7262948165,892812.8094750387,875177.8926552646,857542.9758354793,839908.0590157071,822273.1421959256,804638.2253761552,787003.3085563718,769368.3917365898,751733.4749168083,734098.5580970412,716463.6412772592,698828.7244574893,681193.8076377045,663558.8908179253,645923.9739981517,628289.0571783688,610654.1403585887,593019.2235388234,575384.3067190414,557749.3898992566,540114.4730794914,522479.5562597038,504844.63943991903,487209.72262015566,469574.8058003681,451939.8889805898,434304.9721608106,416670.0553410286,399035.1385212578,381400.2217014814,363765.3048817003,346130.3880619239,328495.47124214657,310860.5544223711,293225.63760257885,275590.72078280896,257955.8039630372,240320.88714325428,222685.97032348998,205051.0535037052,187416.13668391947,169781.21986415423,152146.30304436292,134511.38622458372,116876.4694048129,99241.55258503184,81606.63576525543,63971.71894548461,46336.80212569516,28701.885305928066,11066.968486133963,-6567.948333630338,-24202.86515340209,-41837.781973203644,-59472.69879297353,-77107.61561273783,-94742.5324325189],[1650625.62679281,1632896.9371172644,1615168.2474417244,1597439.5577661796,1579710.8680906338,1561982.1784150908,1544253.488739547,1526524.799064004,1508796.1093884578,1491067.4197129128,1473338.7300373719,1455610.040361827,1437881.350686281,1420152.661010738,1402423.9713351952,1384695.28165965,1366966.5919841072,1349237.902308562,1331509.2126330195,1313780.522957475,1296051.8332819308,1278323.1436063878,1260594.4539308422,1242865.7642553,1225137.0745797555,1207408.384904211,1189679.6952286651,1171951.0055531254,1154222.31587758,1136493.6262020352,1118764.9365264876,1101036.246850947,1083307.5571754035,1065578.8674998572,1047850.1778243165,1030121.4881487754,1012392.7984732226,994664.1087976848,976935.4191221441,959206.7294465946,941478.0397710525,923749.3500955063,906020.6604199642,888291.9707444217,870563.2810688759,852834.5913933306,835105.9017177895,817377.2120422437,799648.5223667021,781919.8326911535,764191.1430156128,746462.4533400694,728733.7636645334,711005.0739889638,693276.3843134223,675547.6946378937,657819.0049623502,640090.3152868142,622361.625611267,604632.9359357115,586904.2462601624,569175.5565846246,551446.8669090876,533718.1772335423,515989.48755799886,498260.79788245354,480532.1082068933,462803.4185313545,445074.72885582875,427346.03918028716,409617.349504726,391888.6598291863,374159.97015364934,356431.2804781003,338702.590802555,320973.901127005,303245.21145146806,285516.5217759302,267787.8321003886,250059.1424248293,232330.45274929702,214601.7630737396,196873.0733982073,179144.38372266572,161415.6940471232,143687.00437157042,125958.31469602324,108229.62502047606,90500.93534493726,72772.24566940591,55043.555993859656,37314.866318299435,19586.17664275877,1857.4869672264904,-15871.202708330005,-33599.89238388464,-51328.58205940947,-69057.27173496969,-86785.96141049266,-104514.65108604729],[1650137.0208601342,1632314.5583288218,1614492.0957975117,1596669.633266206,1578847.1707348945,1561024.7082035835,1543202.2456722753,1525379.783140963,1507557.3206096534,1489734.8580783408,1471912.3955470356,1454089.9330157246,1436267.4704844125,1418445.0079531015,1400622.5454217927,1382800.0828904829,1364977.620359172,1347155.1578278611,1329332.695296554,1311510.2327652427,1293687.770233932,1275865.3077026233,1258042.8451713128,1240220.3826400016,1222397.9201086918,1204575.4575773817,1186752.9950460684,1168930.532514764,1151108.069983452,1133285.6074521416,1115463.1449208297,1097640.6823895248,1079818.2198582129,1061995.757326902,1044173.2947955956,1026350.8322642795,1008528.369732975,990705.9072016557,972883.4446703508,955060.9821390468,937238.5196077288,919416.0570764234,901593.5945451125,883771.1320137982,865948.6694824891,848126.2069511823,830303.7444198714,812481.281888559,794658.8193572508,776836.3568259347,759013.8942946317,741191.4317633156,723368.9692320125,705546.5067007048,687724.044169385,669901.5816380698,652079.1191067724,634256.6565754572,616434.1940441411,598611.731512852,580789.2689815387,562966.8064502096,545144.3439189084,527321.8813876007,509499.41885629017,491676.95632498246,473854.4937936729,456032.03126235306,438209.5687310556,420387.1061997339,402564.6436684262,384742.18113712594,366919.7186058061,349097.2560745068,331274.7935431842,313452.33101187926,295629.8684805585,277807.4059492536,259984.94341795146,242162.48088663258,224340.0183553258,206517.5558240246,188695.09329270292,170872.63076139428,153050.16823009588,135227.70569877326,117405.24316746369,99582.78063616529,81760.31810484827,63937.855573528446,46115.393042235635,28292.930510914885,10470.467979608104,-7351.994551692158,-25174.45708301477,-42996.919614316896,-60819.38214563206,-78641.84467695281,-96464.30720825866,-114286.7697395552],[1649648.4149274565,1631732.179540381,1613815.9441533028,1595899.7087662288,1577983.4733791528,1560067.237992079,1542151.002605,1524234.7672179225,1506318.5318308463,1488402.2964437732,1470486.0610566982,1452569.8256696216,1434653.5902825466,1416737.3548954686,1398821.1195083917,1380904.8841213153,1362988.6487342387,1345072.4133471637,1327156.1779600878,1309239.942573011,1291323.7071859336,1273407.4717988588,1255491.2364117817,1237575.001024706,1219658.7656376269,1201742.530250555,1183826.2948634792,1165910.059476403,1147993.824089326,1130077.5887022493,1112161.3533151713,1094245.1179280952,1076328.88254102,1058412.647153947,1040496.4117668639,1022580.1763797929,1004663.940992712,986747.705605641,968831.4702185625,950915.2348314868,932998.9994444158,915082.7640573331,897166.528670263,879250.2932831822,861334.0578961088,843417.8225090308,825501.5871219523,807585.3517348771,789669.1163478056,771752.8809607234,753836.6455736523,735920.4101865795,718004.174799487,700087.9394124234,682171.7040253505,664255.4686382748,646339.2332511954,628422.9978641123,610506.7624770375,592590.5270899711,574674.2917028842,556758.0563158048,538841.8209287338,520925.5855416665,503009.3501545908,485093.1147675123,467176.87938042823,449260.64399335627,431344.40860627964,413428.17321919743,395511.9378321301,377595.70244504977,359679.4670579778,341763.23167089466,323846.9962838162,305930.7608967526,288014.52550967317,270098.2901225835,252182.05473551527,234265.81934844796,216349.5839613611,198433.34857427888,180517.1131872069,162600.87780014146,144684.64241307415,126768.40702598356,108852.17163889855,90935.93625182007,73019.70086475648,55103.46547769103,37187.23009059485,19270.994703533128,1354.7593164574355,-16561.47607062012,-34477.71145769581,-52393.94684476592,-70310.18223184347,-88226.41761894152,-106142.6530060079,-124058.88839309663],[1649159.8089947817,1631149.800751939,1613139.7925090971,1595129.7842662546,1577119.7760234135,1559109.7677805698,1541099.7595377276,1523089.7512948858,1505079.7430520437,1487069.7348092026,1469059.7265663606,1451049.7183235171,1433039.7100806746,1415029.7018378319,1397019.6935949903,1379009.6853521478,1360999.6771093057,1342989.6688664625,1324979.660623621,1306969.65238078,1288959.6441379366,1270949.6358950958,1252939.6276522523,1234929.6194094077,1216919.6111665685,1198909.6029237243,1180899.594680883,1162889.5864380416,1144879.5781951994,1126869.5699523566,1108859.561709513,1090849.553466672,1072839.5452238303,1054829.536980988,1036819.5287381453,1018809.520495302,1000799.5122524602,982789.5040096114,964779.4957667757,946769.4875239339,928759.4792810911,910749.4710382489,892739.4627954066,874729.4545525671,856719.4463097248,838709.4380668821,820699.4298240356,802689.4215811943,784679.4133383567,766669.4050955055,748659.3968526712,730649.3886098294,712639.3803669838,694629.372124128,676619.3638813021,658609.3556384612,640599.3473956082,622589.3391527766,604579.3309099376,586569.3226670921,568559.31442424,550549.3061814066,532539.2979385685,514529.2896957202,496519.28145286813,478509.2732100291,460499.26496719755,442489.256724352,424479.24848149903,406469.24023867026,388459.23199582193,370449.2237529801,352439.2155101346,334429.20726730116,316419.1990244482,298409.1907816101,280399.18253877107,262389.17429592926,244379.16605308466,226369.15781024564,208359.14956740662,190349.14132456202,172339.13308170997,154329.1248388728,136319.11659603938,118309.10835319012,100299.10011034645,82289.0918674944,64279.08362467028,46269.07538181916,28259.067138976417,10249.058896133676,-7760.949346698821,-25770.95758955367,-43780.9658323843,-61790.9740752466,-79800.98231807724,-97810.99056091718,-115820.99880375527,-133831.00704658963],[1648671.2030621055,1630567.421963499,1612463.6408648917,1594359.8597662821,1576256.078667671,1558152.2975690635,1540048.516470458,1521944.7353718483,1503840.9542732397,1485737.1731746336,1467633.3920760187,1449529.610977411,1431425.8298788043,1413322.048780197,1395218.2676815873,1377114.4865829803,1359010.7054843712,1340906.9243857644,1322803.1432871548,1304699.3621885474,1286595.5810899392,1268491.79999133,1250388.0188927217,1232284.2377941143,1214180.4566955054,1196076.6755968952,1177972.8944982868,1159869.1133996798,1141765.332301071,1123661.551202464,1105557.7701038579,1087453.989005242,1069350.2079066401,1051246.4268080276,1033142.6457094247,1015038.8646108122,996935.0835122098,978831.3024135968,960727.5213149837,942623.7402163809,924519.9591177665,906416.178019166,888312.3969205515,870208.6158219432,852104.8347233357,834001.0536247264,815897.2725261208,797793.4914275119,779689.7103289077,761585.9292302979,743482.14813169,725378.367033069,707274.5859344704,689170.804835869,671067.0237372713,652963.242638641,634859.4615400368,616755.6804414336,598651.8993428089,580548.1182442103,562444.3371456098,544340.5560469897,526236.7749483865,508132.9938497711,490029.2127511753,471925.4316525571,453821.6505539464,435717.8694553459,417614.08835672867,399510.30725813564,381406.5261595128,363302.74506091606,345198.96396229975,327095.18286369834,308991.4017650839,290887.6206664769,272783.83956787735,254680.05846925732,236576.27737065312,218472.49627204426,200368.71517344099,182264.93407483306,164161.15297621023,146057.37187761068,127953.59077900182,109849.80968038552,91746.0285817841,73642.24748317804,55538.46638458688,37434.68528595101,19330.904187343083,1227.123088747263,-16876.658009858802,-34980.439108477905,-53084.22020707652,-71188.00130568258,-89291.78240429424,-107395.56350289844,-125499.34460151754,-143603.12570012175],[1648182.5971294285,1629985.043175056,1611787.4892206818,1593589.9352663069,1575392.3813119289,1557194.8273575567,1538997.2734031836,1520799.7194488093,1502602.165494435,1484404.6115400614,1466207.057585684,1448009.5036313091,1429811.9496769367,1411614.3957225622,1393416.8417681872,1375219.2878138137,1357021.7338594368,1338824.1799050642,1320626.6259506887,1302429.071996316,1284231.5180419409,1266033.9640875666,1247836.4101331912,1229638.8561788178,1211441.302224442,1193243.748270066,1175046.1943156966,1156848.6403613188,1138651.0864069448,1120453.5324525712,1102255.9784981976,1084058.424543819,1065860.8705894467,1047663.3166350727,1029465.7626806949,1011268.2087263251,993070.6547719482,974873.1008175742,956675.5468632011,938477.9929088228,920280.4389544544,902082.8850000771,883885.3310457016,865687.777091328,847490.2231369494,829292.669182573,811095.1152282013,792897.5612738263,774700.0073194532,756502.453365068,738304.8994107153,720107.3454563385,701909.791501957,683712.2375475783,665514.6835931968,647317.1296388479,629119.5756844673,610922.0217300821,592724.4677757081,574526.9138213247,556329.3598669618,538131.8059125906,519934.2519582156,501736.6980038313,483539.144049461,465341.5900950851,447144.0361407213,428946.4821863454,410748.9282319583,392551.3742775833,374353.8203232186,356156.2663688343,337958.7124144677,319761.15846009087,301563.6045057243,283366.05055134837,265168.496596965,246970.94264258258,228773.3886882197,210575.83473385777,192378.28077947255,174180.7268250892,155983.17287071608,137785.61891635042,119588.0649619773,101390.51100759581,83192.95705322642,64995.403098838404,46797.84914446622,28600.295190107077,10402.741235725582,-7794.812718642876,-25992.36667301692,-44189.92062741332,-62387.474581781775,-80585.02853615582,-98782.58249052614,-116980.13644487783,-135177.6903992761,-153375.24435363337],[1647693.9911967549,1629402.6643866133,1611111.337576471,1592820.010766331,1574528.683956192,1556237.3571460517,1537946.0303359113,1519654.7035257714,1501363.37671563,1483072.0499054904,1464780.7230953455,1446489.3962852068,1428198.0694750692,1409906.7426649262,1391615.4158547854,1373324.0890446426,1355032.7622345046,1336741.4354243623,1318450.1086142238,1300158.781804083,1281867.4549939425,1263576.1281838012,1245284.8013736617,1226993.4745635213,1208702.1477533798,1190410.8209432405,1172119.4941330976,1153828.1673229565,1135536.8405128168,1117245.5137026794,1098954.1868925346,1080662.8600823954,1062371.5332722533,1044080.2064621183,1025788.8796519735,1007497.552841831,989206.2260316955,970914.8992215507,952623.5724114068,934332.2456012745,916040.9187911302,897749.5919809905,879458.2651708522,861166.9383607064,842875.6115505677,824584.2847404238,806292.9579302841,788001.6311201449,769710.3043100033,751418.9774998706,733127.6506897276,714836.323879581,696544.9970694454,678253.6702592922,659962.3434491539,641671.0166390222,623379.689828882,605088.3630187353,586797.0362086035,568505.7093984578,550214.3825883139,531923.055778184,513631.72896804195,495340.4021578962,477049.0753477579,458757.7485376168,440466.4217274729,422175.09491734114,403883.76810718514,385592.4412970524,367301.1144869141,349009.7876767712,330718.46086662635,312427.13405649364,294135.8072463479,275844.4804362105,257553.153626075,239261.8268159218,220970.5000057984,202679.17319563963,184387.84638550598,166096.51957537234,147805.19276521914,129513.8659550855,111222.53914494906,92931.21233480051,74639.88552466873,56348.55871452391,38057.23190438282,19765.905094244517,1474.5782841173932,-16816.748526040465,-35108.07533616759,-53399.40214632824,-71690.72895645723,-89982.05576660112,-108273.38257674687,-126564.70938686095,-144856.03619701602,-163147.36300719157],[1647205.3852640782,1628820.2855981744,1610435.1859322626,1592050.0862663602,1573664.986600452,1555279.8869345454,1536894.7872686367,1518509.6876027319,1500124.5879368237,1481739.4882709188,1463354.3886050102,1444969.2889391058,1426584.1892731984,1408199.0896072914,1389813.989941383,1371428.890275477,1353043.7906095705,1334658.6909436649,1316273.5912777593,1297888.491611852,1279503.3919459446,1261118.2922800367,1242733.1926141311,1224348.0929482253,1205962.993282318,1187577.8936164118,1169192.7939505037,1150807.694284595,1132422.5946186897,1114037.4949527862,1095652.3952868772,1077267.2956209723,1058882.1959550628,1040497.0962891583,1022111.996623253,1003726.8969573448,985341.7972914325,966956.697625536,948571.5979596269,930186.4982937151,911801.3986278106,893416.2989619067,875031.1992959972,856646.0996300876,838260.9999641813,819875.9002982774,801490.8006323688,783105.7009664644,764720.6013005534,746335.5016346471,727950.4019687399,709565.3023028346,691180.2026369385,672795.1029710341,654410.0033051223,636024.9036392067,617639.8039732985,599254.7043073876,580869.6046414897,562484.5049755899,544099.4053096836,525714.3056437746,507329.20597786363,488944.10631195083,470559.0066460511,452173.90698013734,433788.8073142357,415403.70764833223,397018.6079824269,378633.508316515,360248.40865060873,341863.30898470804,323478.20931878965,305093.10965288524,286708.00998697523,268322.9103210736,249937.81065516733,231552.71098926477,213167.61132335756,194782.5116574457,176397.411991545,158012.3123256294,139627.21265973896,121242.1129938243,102857.0133279236,84471.91366201732,66086.81399610825,47701.714330192655,29316.614664286375,10931.514998380095,-7453.584667517804,-25838.68433343433,-44223.783999344334,-62608.883665245026,-80993.98333114572,-99379.08299704269,-117764.18266295455,-136149.28232887015,-154534.38199477643,-172919.48166067153],[1646716.7793314033,1628237.9068097305,1609759.0342880564,1591280.1617663833,1572801.289244711,1554322.4167230385,1535843.544201366,1517364.6716796923,1498885.7991580176,1480406.926636348,1461928.0541146721,1443449.1815930025,1424970.309071327,1406491.4365496559,1388012.5640279825,1369533.6915063104,1351054.8189846366,1332575.9464629649,1314097.0739412892,1295618.2014196194,1277139.3288979472,1258660.4563762732,1240181.5838546008,1221702.7113329275,1203223.838811255,1184744.9662895831,1166266.093767907,1147787.2212462388,1129308.3487245636,1110829.4762028884,1092350.6036812188,1073871.7311595473,1055392.8586378735,1036913.9861161988,1018435.1135945227,999956.2410728578,981477.3685511821,962998.4960295069,944519.6235078322,926040.7509861626,907561.8784644911,889083.0059428192,870604.1334211417,852125.2608994707,833646.3883777973,815167.5158561259,796688.6433344516,778209.7708127806,759730.8982911091,741252.0257694395,722773.1532477513,704294.2807260891,685815.4082044158,667336.5356827453,648857.6631610785,630378.7906393912,611899.9181177272,593421.0455960631,574942.1730743786,556463.3005527109,537984.428031032,519505.5555093661,501026.6829876993,482547.81046601105,464068.93794434424,445590.06542266905,427111.1929009985,408632.3203793345,390153.44785765,371674.57533597853,353195.7028143024,334716.8302926319,316237.9577709669,297759.08524928987,279280.2127276156,260801.34020593576,242322.4676842671,223843.5951626012,205364.7226409223,186885.85011924896,168406.97759757098,149928.10507591534,131449.23255423456,112970.36003255378,94491.48751089163,76012.61498919968,57533.74246754497,39054.869945880026,20575.99742419552,2097.124902524054,-16381.747619157657,-34860.620140822604,-53339.49266249314,-71818.36518414877,-90297.23770584352,-108776.11022752151,-127254.98274918087,-145733.85527086444,-164212.72779252566,-182691.60031421855],[1646228.1733987243,1627655.5280212876,1609082.8826438494,1590510.2372664073,1571937.5918889714,1553364.946511532,1534792.3011340946,1516219.6557566521,1497647.0103792152,1479074.3650017774,1460501.7196243356,1441929.0742468967,1423356.4288694605,1404783.7834920192,1386211.1381145814,1367638.4927371435,1349065.8473597053,1330493.201982264,1311920.5566048264,1293347.9112273876,1274775.2658499484,1256202.6204725082,1237629.975095069,1219057.3297176315,1200484.6843401934,1181912.0389627544,1163339.3935853122,1144766.7482078774,1126194.102830436,1107621.4574529957,1089048.81207556,1070476.1666981196,1051903.5213206839,1033330.8759432435,1014758.2305658017,996185.5851883623,977612.9398109228,959040.2944334913,940467.6490560514,921895.0036786105,903322.3583011683,884749.7129237293,866177.0675462908,847604.4221688514,829031.7767914142,810459.1314139757,791886.4860365358,773313.8406590996,754741.1952816611,736168.549904218,717595.9045267766,699023.2591493372,680450.6137718894,661877.9683944592,643305.3230170384,624732.6776395952,606160.0322621511,587587.3868847145,569014.7415072732,550442.0961298309,531869.4507523887,513296.8053749474,494724.1599975135,476151.5146200685,457578.8692426402,439006.2238651961,420433.578487752,401860.9331103284,383288.28773288336,364715.6423554411,346142.99697800074,327570.3516005641,308997.7062231209,290425.0608456936,271852.41546825226,253279.77009081002,234707.12471337337,216134.4793359274,197561.83395849075,178989.18858104944,160416.54320360348,141843.89782617614,123271.25244873948,104698.60707130097,86125.96169386711,67553.31631642208,48980.6709389817,30408.02556154877,11835.380184096284,-6737.265193337575,-25309.910570777953,-43882.555948199704,-62455.201325649396,-81027.84670308232,-99600.49208053015,-118173.13745796122,-136745.78283540905,-155318.42821283452,-173891.07359026745,-192463.71896771714],[1645739.5674660502,1627073.149232843,1608406.7309996416,1589740.312766435,1571073.8945332295,1552407.4763000256,1533741.0580668189,1515074.6398336156,1496408.2216004126,1477741.8033672064,1459075.3851339982,1440408.966900793,1421742.5486675915,1403076.1304343857,1384409.7122011792,1365743.2939679762,1347076.8757347686,1328410.4575015625,1309744.0392683612,1291077.6210351554,1272411.2028019505,1253744.7845687447,1235078.3663355415,1216411.9481023345,1197745.5298691308,1179079.1116359243,1160412.6934027178,1141746.2751695178,1123079.8569363104,1104413.438703103,1085747.0204698965,1067080.6022366951,1048414.1840034905,1029747.7657702891,1011081.3475370822,992414.9293038761,973748.5110706692,955082.0928374622,936415.6746042566,917749.2563710501,899082.8381378502,880416.4199046423,861750.0016714428,843083.5834382339,824417.1652050312,805750.7469718195,787084.328738621,768417.9105054149,749751.4922722094,731085.074039001,712418.6558057973,693752.2375726011,675085.8193393806,656419.4011061918,637752.9828729779,619086.564639777,600420.1464065686,581753.7281733667,563087.3099401733,544420.8917069556,525754.4734737538,507088.055240538,488421.6370073417,469755.21877413243,451088.80054092593,432422.38230772223,413755.96407451946,395089.5458413102,376423.12760810833,357756.7093749065,339090.29114169814,320423.8729084963,301757.4546752935,283091.03644208424,264424.6182088796,245758.1999756666,227091.78174246196,208425.3635092657,189758.94527605362,171092.52704285737,152426.1088096602,133759.69057644252,115093.2723432444,96426.85411003511,77760.4358768398,59094.01764361281,40427.59941041935,21761.181177202612,3094.7629440128803,-15571.655289191753,-34238.07352239266,-52904.491755619645,-71570.90998879261,-90237.32822200842,-108903.74645521119,-127570.16468844563,-146236.58292162046,-164903.00115483813,-183569.41938803717,-202235.83762124926],[1645250.9615333735,1626490.7704444013,1607730.5793554292,1588970.3882664624,1570210.1971774898,1551450.00608852,1532689.8149995469,1513929.6239105747,1495169.4328216058,1476409.2417326346,1457649.0506436632,1438888.8595546898,1420128.668465718,1401368.4773767493,1382608.286287778,1363848.0951988061,1345087.9041098366,1326327.7130208656,1307567.5219318948,1288807.330842924,1270047.1397539526,1251286.94866498,1232526.7575760102,1213766.566487038,1195006.3753980668,1176246.1843090965,1157485.9932201281,1138725.8021311564,1119965.6110421834,1101205.4199532103,1082445.228864239,1063685.0377752674,1044924.8466862943,1026164.6555973236,1007404.4645083556,988644.2734193848,969884.0823304183,951123.8912414461,932363.700152474,913603.5090635056,894843.3179745311,876083.126885558,857322.9357965868,838562.7447076165,819802.5536186425,801042.3625296741,782282.1714407057,763521.9803517321,744761.7892627614,726001.5981737878,707241.4070848152,688481.2159958435,669721.0249068756,650960.8338178992,632200.6427289331,613440.4516399615,594680.2605509814,575920.0694620153,557159.8783730445,538399.6872840729,519639.49619510025,500879.3051061379,482119.1140171671,463358.92292819545,444598.7318392191,425838.54075025115,407078.3496612832,388318.15857230686,369557.96748333424,350797.77639436815,332037.5853053881,313277.3942164229,294517.2031274559,275757.0120384898,256996.82094951533,238236.62986053992,219476.4387715701,200716.247682604,181956.05659362115,163195.86550465878,144435.6744156871,125675.48332671169,106915.29223775398,88155.10114877205,69394.91005980596,50634.718970832415,31874.52788186632,13114.336792885326,-5645.854296080768,-24406.04538505338,-43166.236474025995,-61926.42756299302,-80686.61865197122,-99446.80974094197,-118207.00082993135,-136967.19191888534,-155727.3830078654,-174487.57409682497,-193247.76518579386,-212007.95627475716]],"type":"heatmap","xaxis":"x","yaxis":"y","hovertemplate":"Home Price: %{x}\u003cbr\u003eDownpayment: %{y}\u003cbr\u003eOpportunity: %{z}\u003cextra\u003e\u003c\u002fextra\u003e"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Home Price"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"autorange":"reversed","title":{"text":"Downpayment"}},"coloraxis":{"colorbar":{"title":{"text":"Opportunity"}},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"title":{"text":"Opportunity Surface"}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### `home_price_growth_rate` and `investment_return_rate`

In [15]:
_x, _y = np.linspace(0.02, 0.1, 100), np.linspace(0.02, 0.1, 100)
x, y = np.meshgrid(_x, _y, indexing="ij")
x = x.flatten()
y = y.flatten()
surface_kwargs = kwargs.copy()
surface_kwargs.update({"home_price_growth_rate": x, "investment_return_rate": y})
render_plotly_html(
    px.imshow(
        RentVsBuy().calculate(**surface_kwargs).value.reshape((100, 100)).T,
        aspect="auto",
        labels=dict(x="Home Price Growth", y="Investment Growth", color="Opportunity"),
        x=_x,
        y=_y,
        title="Opportunity Surface",
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="c7b512dc-1d3a-409f-a885-bbc1ad284f38" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("c7b512dc-1d3a-409f-a885-bbc1ad284f38")) {                    Plotly.newPlot(                        "c7b512dc-1d3a-409f-a885-bbc1ad284f38",                        [{"coloraxis":"coloraxis","name":"0","x":[0.02,0.02080808080808081,0.021616161616161617,0.022424242424242423,0.023232323232323233,0.024040404040404043,0.02484848484848485,0.025656565656565655,0.026464646464646465,0.027272727272727275,0.02808080808080808,0.028888888888888888,0.029696969696969697,0.030505050505050507,0.031313131313131314,0.03212121212121212,0.03292929292929293,0.03373737373737374,0.034545454545454546,0.03535353535353535,0.03616161616161616,0.03696969696969697,0.03777777777777778,0.038585858585858585,0.0393939393939394,0.0402020202020202,0.04101010101010101,0.041818181818181824,0.04262626262626262,0.043434343434343436,0.04424242424242424,0.04505050505050505,0.04585858585858586,0.04666666666666666,0.047474747474747475,0.04828282828282829,0.04909090909090909,0.0498989898989899,0.05070707070707071,0.051515151515151514,0.05232323232323233,0.053131313131313126,0.05393939393939394,0.05474747474747475,0.05555555555555555,0.056363636363636366,0.05717171717171718,0.05797979797979798,0.05878787878787879,0.05959595959595959,0.060404040404040404,0.06121212121212122,0.06202020202020202,0.06282828282828283,0.06363636363636364,0.06444444444444444,0.06525252525252526,0.06606060606060606,0.06686868686868687,0.06767676767676768,0.06848484848484848,0.0692929292929293,0.07010101010101011,0.07090909090909091,0.07171717171717172,0.07252525252525252,0.07333333333333333,0.07414141414141415,0.07494949494949495,0.07575757575757576,0.07656565656565657,0.07737373737373737,0.07818181818181819,0.07898989898989899,0.0797979797979798,0.08060606060606061,0.08141414141414141,0.08222222222222222,0.08303030303030304,0.08383838383838384,0.08464646464646465,0.08545454545454546,0.08626262626262626,0.08707070707070708,0.08787878787878788,0.08868686868686869,0.0894949494949495,0.0903030303030303,0.09111111111111111,0.09191919191919193,0.09272727272727273,0.09353535353535354,0.09434343434343435,0.09515151515151515,0.09595959595959597,0.09676767676767677,0.09757575757575758,0.09838383838383839,0.09919191919191919,0.1],"y":[0.02,0.02080808080808081,0.021616161616161617,0.022424242424242423,0.023232323232323233,0.024040404040404043,0.02484848484848485,0.025656565656565655,0.026464646464646465,0.027272727272727275,0.02808080808080808,0.028888888888888888,0.029696969696969697,0.030505050505050507,0.031313131313131314,0.03212121212121212,0.03292929292929293,0.03373737373737374,0.034545454545454546,0.03535353535353535,0.03616161616161616,0.03696969696969697,0.03777777777777778,0.038585858585858585,0.0393939393939394,0.0402020202020202,0.04101010101010101,0.041818181818181824,0.04262626262626262,0.043434343434343436,0.04424242424242424,0.04505050505050505,0.04585858585858586,0.04666666666666666,0.047474747474747475,0.04828282828282829,0.04909090909090909,0.0498989898989899,0.05070707070707071,0.051515151515151514,0.05232323232323233,0.053131313131313126,0.05393939393939394,0.05474747474747475,0.05555555555555555,0.056363636363636366,0.05717171717171718,0.05797979797979798,0.05878787878787879,0.05959595959595959,0.060404040404040404,0.06121212121212122,0.06202020202020202,0.06282828282828283,0.06363636363636364,0.06444444444444444,0.06525252525252526,0.06606060606060606,0.06686868686868687,0.06767676767676768,0.06848484848484848,0.0692929292929293,0.07010101010101011,0.07090909090909091,0.07171717171717172,0.07252525252525252,0.07333333333333333,0.07414141414141415,0.07494949494949495,0.07575757575757576,0.07656565656565657,0.07737373737373737,0.07818181818181819,0.07898989898989899,0.0797979797979798,0.08060606060606061,0.08141414141414141,0.08222222222222222,0.08303030303030304,0.08383838383838384,0.08464646464646465,0.08545454545454546,0.08626262626262626,0.08707070707070708,0.08787878787878788,0.08868686868686869,0.0894949494949495,0.0903030303030303,0.09111111111111111,0.09191919191919193,0.09272727272727273,0.09353535353535354,0.09434343434343435,0.09515151515151515,0.09595959595959597,0.09676767676767677,0.09757575757575758,0.09838383838383839,0.09919191919191919,0.1],"z":[[-125546.27668532706,-106368.492240789,-86862.39774120797,-67022.56601090136,-46843.48158944596,-26319.539309679996,-5445.0428533048835,15785.796716659097,37378.86244679056,59340.13300836948,81675.68423595838,104391.69069130882,127494.42725182814,150990.2707245706,174885.70148582617,199187.3051473801,223901.77424852597,249035.909975881,274596.6239103081,300590.93980173836,327025.9953722821,353909.04414853477,381247.45732221566,409048.7256414287,437320.4613315989,466070.4000471898,495306.40285440884,525036.4582460604,555268.6841875187,586011.330196349,617272.7794545477,649061.5509546062,681386.3016796093,714255.828818711,747679.0720167537,781665.1156609138,816223.1912031376,851362.6795197788,887093.1133086705,923424.1795249877,960365.7218547203,997927.7432287277,1036120.4083762569,1074954.0464192117,1114439.153507638,1154586.3954977824,1195406.6106715186,1236910.8125003562,1279110.1924527592,1322016.1228463345,1365640.159745235,1409994.0459043665,1455089.71375905,1500939.2884636512,1547555.09097784,1594949.6412020954,1643135.6611629033,1692126.0782494484,1741934.028500119,1792572.859942925,1844056.1359881922,1896397.6388753813,1949611.3731744946,2003711.5693440493,2058712.6873437983,2114629.4203066677,2171476.698268148,2229269.69195512,2288023.8166346555,2347754.7360249897,2408478.366266628,2470210.879958485,2532968.7102571735,2596768.55504155,2661627.381143217,2727562.4286452276,2794591.2152469316,2862731.5407002443,2932001.4913152307,3002419.44453747,3074004.073597815,3146774.3522370635,3220749.559503343,3295949.2846278725,3372393.43197698,3450102.2260825727,3529096.2167537836,3609396.2842670316,3691023.644641013,3773999.8549940567,3858346.818986679,3944086.7923501213,4031242.3885040213,4119836.5842602104,4209892.725619841,4301434.533660928,4394486.110519501,4489071.945465215,4585216.921074817,4682946.319500266],[-135672.04553100653,-116505.12971093913,-97009.96513974585,-77181.12503918714,-57013.09434905683,-36500.26830520318,-15636.950995154679,5582.6461094066035,27164.405643542996,49114.30586413713,71438.42218846106,94142.92875805474,117234.10002717073,140718.31237675133,164602.04575398774,188891.8853385616,213594.52323463582,238716.76019065012,264265.50734621217,290247.78800691036,316670.73944739066,343541.61474364717,370867.78463363345,398656.73940850305,426916.0908335224,455653.57409976213,484877.04980678204,514594.50597745646,544814.0601039294,575543.9612272053,606792.5920493682,638568.4710796161,670880.2548143461,703736.7399525899,737146.8656456203,771119.7157835646,805664.5213178173,840790.6626206527,876507.6718822734,912825.235546635,949753.1967849098,987301.5580094999,1025480.4834275395,1064300.3016351303,1103771.5082528042,1143904.7686035514,1184710.92043222,1226200.9766694992,1268386.128239208,1311277.7469104552,1354887.3881950143,1399226.7942915051,1444307.8970750214,1490142.821135739,1536743.8868651355,1584123.613591479,1632294.722764986,1681270.1411944847,1731063.004333895,1781686.6596226115,1833154.6698781706,1885480.8167430342,1938679.1041859682,1992763.7620599905,2047749.2497150437,2103650.2596699186,2160481.721341609,2218258.804834094,2276996.9247871214,2336711.7442871407,2397419.178838373,2459135.4003989277,2521876.8414800465,2585660.1993106203,2650502.440067664,2716420.803174969,2783432.8056679377,2851556.2466298016,2920809.2116971714,2991210.077637377,3062777.5169981807,3135530.5028324146,3209488.313495338,3284670.5375203486,3361097.078570978,3438788.1604713094,3517764.332317602,3598046.473668308,3679655.7998190233,3762613.867159806,3846942.5786176967,3932664.1891852175,4019801.311537994,4108376.9217385272,4198414.365033277,4289937.361740156,4382970.013229657,4477536.808000405,4573662.627852604,4671372.754156092],[-146011.258934631,-126855.71994587581,-107371.99787912669,-87554.66639312205,-67398.21086776734,-46897.02698219335,-26045.419270382263,-4837.599653688259,16732.31405004859,38670.29964155215,60982.432078652084,83674.88504014118,106753.9325141944,130225.95041231625,154097.41820888245,178374.92060735147,203065.1492322269,228174.90434883465,253711.0966101801,279680.74883174314,306090.99779451685,332949.09607728804,360262.4139172039,388038.44110099645,416284.78888586373,445009.19195113797,474219.5103809375,503923.7316789685,534129.9728144482,564846.482301671,596081.6423121968,627843.9708208551,660142.1237858054,692984.89736394,726381.2301604648,760340.2055154741,794871.053826344,829983.1549073115,865686.040386487,901989.3961416562,938903.0647737074,976437.0481206176,1014601.5098109194,1053406.7778579078,1092863.3472950687,1132981.8828540773,1173773.221684165,1215248.3761160672,1257418.5364692823,1300295.073904196,1343889.543319427,1388213.6862959682,1433279.434086789,1479098.9106554007,1525684.435762046,1573048.528099157,1621203.9084764689,1670163.5030576545,1719940.4466467705,1770548.0860285952,1821999.9833612705,1874309.9196230427,1927491.898113609,1981560.1480120197,2036529.1279893254,2092413.5298804562,2149228.2824135358,2206988.5549986353,2265709.761576508,2325407.5645294935,2386097.878652536,2447796.8751892773,2510520.9859312484,2574286.9073823513,2639111.604989294,2705012.3174402025,2772006.5610294156,2840112.1340936525,2909347.1215175404,2979729.8993108915,3051279.1392583894,3124013.8136441754,3197953.2000491684,3273116.8862267286,3349524.775054613,3427197.0895653493,3506154.378057811,3586417.519287197,3668007.727739932,3750946.558990944,3835255.915146136,3920958.0503708366,4008075.5765073854,4096631.468778848,4186649.071586058,4278152.104395059,4371164.667718193,4465711.249189593,4561816.729738589,4659506.389857717],[-156557.4900702678,-137413.79696904856,-117941.99049484276,-98136.64478005853,-77992.24568174256,-57503.189359677024,-36663.78083202173,-15468.2325078703,6089.337303617969,28014.905908227898,50314.54776521679,72994.4360511289,96060.8442481982,119520.14775826177,143378.82554227416,167643.46178648784,192320.7475943889,217417.48270644643,242940.57724693534,268897.0534987019,295294.0477061805,322138.8119076316,349438.71579568484,377201.2486085363,405434.0210507908,434144.7672451015,463341.3467147895,493031.7463986201,523224.08269668533,553926.6035499431,585147.6905523685,616895.8610969302,649179.7705556152,682008.2144947986,715390.1309247888,749334.6025863716,783850.8592731676,818948.2801911796,854636.3963557654,890924.893027409,927823.6121851106,965342.555040335,1003491.8845904535,1042281.9282128967,1081723.1803005624,1121826.3049397718,1162602.1386295937,1204061.693045766,1246216.1578479097,1289076.903531614,1332655.4843257526,1376963.6411365739,1422013.3045372781,1467816.597806533,1514385.8400146286,1561733.5491588933,1609872.4453487748,1658815.45404242,1708575.7093330945,1759166.5572894735,1810601.5593482414,1862894.4957607938,1916059.3690945366,1970110.4077907575,2025062.0697772158,2080929.0461399893,2137726.264852741,2195468.8945654295,2254172.348453002,2313852.288126246,2374524.6276027784,2436205.5373430746,2498911.448349628,2562659.056331387,2627465.325934127,2693347.4950390384,2760323.0791274575,2828409.8757169964,2897625.9688670174,2967989.733755864,3039519.8413305017,3112235.2630310496,3186155.275588054,3261299.465898106,3337687.7359757395,3415340.30798376,3494277.7293447694,3574520.877931092,3656090.967339621,3739009.5522490516,3823298.533862292,3908980.1654348653,3996077.0578924417,4084612.1855344973,4174608.8918313053,4266090.895311322,4359082.295542217,4453607.579206318,4549691.626273948,4647359.716271392],[-167307.36636437895,-148175.95956240164,-128716.51284184109,-108923.60084273235,-88791.70993353997,-68315.23678928567,-47488.48694721679,-26305.67333939788,-4760.914801866515,17151.76544033899,39438.4413116856,62105.28544991976,85158.57079444476,108604.67220019945,132450.06807714468,156701.34205640806,181365.1846821853,206448.39513146388,231957.8829608066,257900.66988109075,284283.8915604786,311114.7994566378,338400.7626772409,366149.2698711294,394367.93114912114,423064.4800356212,452246.77545119426,481922.8037273183,512100.68065222516,542788.6535504065,573995.1033947214,605728.546952344,637997.6389647557,670811.174363059,704178.0905175109,738107.4695240059,772608.5405264099,807690.6820760637,843363.4245287203,879636.4524802745,916519.6072401109,954022.8893450231,992156.461112597,1030930.649235351,1070355.9474160736,1110443.0190457604,1151202.699922884,1192646.0010172853,1234784.1112773342,1277628.4004819805,1321190.4221380223,1365481.9164241592,1410514.8131805225,1456301.234947144,1502853.500050073,1550184.1257367488,1598305.8313610274,1647231.5416197253,1696974.389838993,1747547.7213145534,1798965.0967042658,1851240.295474771,1904387.3194027506,1958420.3961327442,2013353.982789701,2069202.7696507731,2125981.6838745303,2183705.8932896303,2242390.810243453,2302052.0955129103,2362705.6622753832,2424367.680144699,2487054.579270238,2550783.054501321,2615570.069617538,2681432.8616272653,2748388.945132363,2816456.1167642423,2885652.459689287,2955996.348186029,3027506.452294744,3100201.742541918,3174101.4947374696,3249225.294850315,3325593.0439602043,3403224.9632879975,3482141.599307134,3562363.8289334932,3643912.8648001957,3726810.2606147667,3811077.9166014907,3896738.08502976,3983813.375831541,4072326.7623049766,4162301.5869113104,4253761.567162219,4346730.801600778,4441233.775876837,4537295.368920314,4634940.859209072],[-178257.45636795717,-159138.75699913513,-139692.09480542142,-119912.04496822145,-99793.09440147865,-79329.64032983943,-58515.988844350446,-37346.353435087716,-15814.85350034223,6084.487168318359,28357.741924519418,51011.08283081744,74050.78224704997,97483.21444417187,121314.85724371206,145552.29368387721,170202.2137114245,195271.41590134217,220766.8092036005,246695.41471785237,273064.3674963745,299880.9183762348,327152.4358397664,354886.4079056962,383090.4440499041,411772.27715699887,440939.7655028589,470600.8947693324,500763.78009005566,531436.6681299042,562627.9391970693,594346.1093889605,626599.8327721618,659397.9035977,692749.2585505352,726662.9790360115,761148.2935021373,796214.5797990509,831871.3675759183,868128.3407166132,904995.3398130273,942482.3646789447,980599.5769033753,1019357.3024446554,1058766.0342657415,1098836.4350120986,1139579.3397309245,1181005.7586349973,1223126.879909787,1265954.072565462,1309498.889334109,1353773.0696137382,1398788.5424577657,1444557.429613436,1491092.048607881,1538404.9158834603,1586508.7499827216,1635416.4747849023,1685141.2227922534,1735696.3384702231,1787095.3816399712,1839352.1309249643,1892480.587252177,1946494.9774098669,2001409.7576600623,2057239.617410307,2113999.4829428224,2171704.521203122,2230370.143648587,2290012.0101592317,2350646.0330085694,2412288.380899546,2474955.483063586,2538664.0334249223,2603430.9948308636,2669273.603350265,2736209.372638151,2804256.0983717395,2873431.8627557894,2943755.039099731,3015244.296467186,3087918.6044003842,3161797.2377173197,3236899.7813872527,3313246.1354824873,3390856.520208581,3469751.481015752,3549951.8937886767,3631478.9701212207,3714354.2626735475,3798599.6706143967,3884237.4451493723,3971290.1951383417,4059780.8927989686,4149732.8795035696,4241169.871666381,4334115.96672445,4428595.649212949,4524633.79693839,4622255.687246434],[-189404.47401256207,-170298.89201946824,-150865.42797309963,-131098.6576319189,-110993.06849139638,-90543.05836220155,-69742.93392595276,-48586.909267864656,-27069.104385985993,-5183.54367629136,17075.845605750335,39715.23490887252,62740.895974327344,86159.20244967635,109976.63152821781,134199.76561503857,158835.29401885928,183890.0146716684,209370.83587542805,235284.7780767486,261638.97566977236,288440.6788283102,315697.2553662681,343416.19262873917,371605.0994127139,400271.7079186174,429423.8757328051,459069.58784222184,489216.9586801722,519874.23420572956,551049.7940157701,582752.1534908258,614989.965974998,647772.024991178,681107.2664904972,715004.7711387211,749473.7666384878,784523.6300887335,820163.8903815262,856404.2306377154,893254.4906801784,930724.6695476549,968824.9280480319,1007565.5913523857,1046957.1516302298,1087010.2707273627,1127735.782885022,1169144.6975036943,1211248.2019501708,1254057.6644095017,1297584.6367821628,1341840.8576280028,1386838.2551556833,1432588.9502610366,1479105.2596130772,1526399.698789278,1574484.985460486,1623374.0426273562,1673080.0019066322,1723616.2068712618,1774996.216442859,1827233.808338241,1880342.9825705665,1934337.9650070593,1989233.2109814275,2045043.4089655627,2101783.4842986222,2159468.6029755943,2218114.1754958173,2277735.860773667,2338349.570109376,2399971.4712248947,2462617.9923628774,2526305.8264509453,2591051.935331893,2656873.554062088,2723788.195276037,2791813.6536223334,2860968.0102689434,2931269.6374802734,3002737.203266607,3075389.676108472,3149246.3297537128,3224326.748092929,3300650.830111185,3378238.7949181385,3457111.186859387,3537288.8807061883,3618793.0869301204,3701645.3570601107,3785867.589124652,3871482.0331800054,3958511.2969275247,4046978.3514171196,4136906.5368440375,4228319.568436038,4321241.542434221,4415696.942168261,4511710.644229529,4609307.924738845],[-200745.65105654253,-181653.56788370514,-162233.68680234498,-142480.58418047975,-122388.74812792602,-101952.57707452867,-81166.37832594337,-60024.36659633997,-38520.66251766123,-16649.291124770883,5595.819682941306,28220.840705220355,51232.04302921309,74635.79964321782,98438.58707607398,122646.9870631632,147267.6882382182,172307.48785291007,197773.29352351115,223672.12500552018,250011.11599651352,276797.51596821565,304038.6920269055,331742.1308044391,359915.4403789332,388566.35222623753,417702.72320239944,447332.53755826317,477463.9089851987,508105.0826944632,539264.437529179,570950.4881101456,603171.8870157006,635937.4269968846,669256.0432268512,703136.8155872023,737588.9709901791,772621.8857380301,808245.0879198043,844468.2598469302,881301.2405264273,918754.0281746781,956836.7827706551,995559.8286499162,1034933.6571397965,1074968.9292371897,1115676.478327668,1157067.3129492095,1199152.6195992199,1241943.7655864097,1285452.3019279225,1329689.966293207,1374668.6859934076,1420400.5810196556,1466897.9671290345,1514173.3589797786,1562239.4733161412,1611109.232204752,1660795.7663208582,1711312.4182883531,1762672.7460721948,1814890.526424866,1867979.7583874539,1921954.6668473103,1976829.7061504035,2032619.563772948,2089339.164050429,2147003.6719660945,2205628.4969994063,2265229.2970366627,2325821.982341746,2387422.7195919156,2450047.9359767265,2513714.3233622173,2578438.842521048,2644238.7274308284,2711131.4896386107,2779134.9226967627,2848267.106668187,2918546.412703301,2989991.5076894164,3062621.358975014,3136455.2391667524,3211512.731004824,3287813.732314581,3365378.461036605,3444227.4603379527,3524381.6038018074,3605862.1007020613,3688690.5013602595,3772888.7025877303,3858478.953213716,3945483.8597026113,4033926.3918573298,4123829.888616019,4215218.063939145,4308115.012790227,4402545.217210992,4498533.5524944095,4596105.293452349],[-212281.08447655966,-193202.87023967097,-173796.94548639574,-154057.88722546946,-133980.18421252898,-113558.2355283713,-92786.3491347949,-71658.74040733348,-50169.53064456396,-28312.745553298853,-6082.313710234361,16527.93500166596,39524.270981391426,62913.06652358663,86700.79745790223,110894.04481494497,135499.49651803263,160523.94910274725,185974.30946354452,211857.59662832925,238180.9435612578,264951.59899476497,292176.92928991164,319864.4203273738,348021.6794280568,376656.43730452727,405776.55004342995,435390.00112002785,465504.90344388597,496129.5014381688,527272.1731515825,558941.4324041202,591145.9309668757,623894.4607771514,657195.9561878138,691059.4962535768,725494.3070531308,760509.7640484625,796115.3944815821,832320.8798100597,869136.0581801601,906570.9269405727,944635.645195574,983340.5363989628,1022696.0909891895,1062712.9690670923,1103402.0031149322,1144774.2007600826,1186840.7475819776,1229613.0099639252,1273102.5379901782,1317321.068389743,1362280.527525705,1407993.0344334508,1454470.9039065738,1501726.6496320115,1549772.9873748324,1598622.8382145474,1648289.3318312648,1698785.8098456631,1750125.829211334,1802323.1656611813,1855391.8172084193,1909346.007704177,1964200.1904497663,2019969.0518682492,2076667.5152333833,2134310.7444580486,2192914.147942617,2252493.382485508,2313064.3572538598,2374643.2378192423,2437246.450256506,2500890.68530789,2565592.9026130857,2631370.3350074785,2698240.492886577,2766221.1686417907,2835330.4411655804,2905586.6804283555,2977008.5521277627,3049615.022412896,3123425.362681207,3198459.1544537796,3274736.2943268865,3352276.9990019533,3431101.81039673,3511231.6008348563,3592687.5783203514,3675491.291894472,3759664.63707776,3845229.86139807,3932209.5700077293,4020626.731386813,4110504.683139758,4201867.137882364,4294738.189222432,4389142.317834815,4485104.397634374,4582649.702043536],[-224008.7721317811,-204944.7727151023,-185553.1531605752,-165828.49114608264,-145765.27610177558,-125357.90778838983,-104600.69485313445,-83487.85336246248,-62013.50531143369,-40171.67710895208,-17956.29803945939,4638.80130063137,27619.890591111965,50993.341401626356,74765.62883097585,98943.33317301795,123533.14160838048,148541.849923949,173976.3642594337,199843.70288189943,226150.99798850296,252905.49753847858,280114.5671134321,307785.6918082668,335926.47815175983,364544.65605792706,393648.0808083913,423244.7350668695,453342.7309247905,483950.31198055623,515075.8554514039,546727.8743191136,578915.0195097527,611646.0821087342,644929.9956101077,678775.8382027855,713192.8350926132,748190.360861632,783777.941864741,819965.2586651728,856762.1485075697,894178.6078316511,932224.7948253099,970911.0320184887,1010247.8089182456,1050245.78468642,1090915.7908586129,1132268.8341078006,1174316.0990512215,1217068.9511021285,1260538.939366782,1304737.7995881988,1349677.457135405,1395370.0300415924,1441827.832089953,1489063.3759487474,1537089.3763560161,1585918.7533558297,1635564.6355843598,1686040.3636097864,1737359.4933245678,1789535.799391763,1842583.2787459579,1896516.1541507898,1951348.8778111457,2007096.1350446483,2063772.8480105312,2121394.179497974,2179975.536774408,2239532.5754959863,2300081.2036781744,2361637.5857313825,2424218.1465597358,2487839.575725097,2552518.8316770466,2618273.146051051,2685120.028032791,2753077.268793866,2822162.945996859,2892395.42837214,2963793.380367073,3036375.7668701364,3110161.8580077407,3185171.234019428,3261423.7902093185,3338939.741975991,3417739.62992357,3497844.3250511894,3579275.03402739,3662053.3045468866,3746201.0307725137,3831740.458863168,3918694.1925908616,4007085.199043898,4096936.814423397,4188272.74993019,4281117.097745354,4375494.337105176,4471429.340473987,4568947.379811628],[-235927.898871599,-216878.45162876055,-197501.47759508726,-177791.5551466957,-157743.17441764835,-137350.73587830737,-116608.54889128497,-95510.83024425618,-74051.70265937364,-52225.19327855483,-30025.23212522082,-7445.650540790521,15519.820403447142,38877.55151993525,62634.017144093756,86795.796800168,111369.57689288654,136362.1524269029,161780.4287533022,187631.42334407545,213922.26759480266,240660.20865658415,267852.6112962775,295506.959787369,323630.8598304866,352232.0405047124,381318.3562498947,410897.788881063,440978.44963400904,471568.58124446473,502676.5600599204,534310.8981852648,566480.2456624662,599193.3926855747,632459.2718499473,666286.9604384073,700685.682743262,735664.8124254865,771233.8749113313,807402.549827734,844180.6734753156,881578.2413420014,919605.4106560363,958272.5029797952,997590.006844782,1037568.5804292301,1078219.0542770098,1119552.4340611673,1161579.90339075,1204312.8266624815,1247762.751957683,1291941.4139859504,1336860.7370743393,1382532.8382054323,1428970.0301031175,1476184.8243675667,1524189.9346598843,1572998.2799382643,1622622.9877439972,1673077.3975412974,1724375.0641094907,1776529.7609892574,1829555.4839834813,1883466.4547146894,1938277.1242371753,1994002.1767083972,2050656.5331177681,2108255.355074917,2166814.048657899,2226348.268323588,2286873.920878173,2348407.1695127194,2410964.437901831,2474562.4143676013,2539218.0561094927,2604948.5935024265,2671771.5344610047,2739704.6688751434,2808766.0731150256,2878974.114607823,2950347.456486808,3022905.0623153737,3096666.200883765,3171650.451084169,3247877.7068620734,3325368.182246057,3404142.416458751,3484221.2791062295,3565625.975452292,3648378.051775132,3732499.400809186,3818012.2672729585,3904939.2534859823,3993303.3250718866,4083127.8167548124,4174436.438246205,4267253.280225234,4361602.820413653,4457509.92974852,4554999.878649553],[-248038.36558956024,-229003.80372708163,-209641.81142244197,-189946.96778035385,-169913.7636694688,-149536.60030078562,-128809.78778364835,-107727.54365861486,-86283.99140692875,-64473.15893585165,-42288.977040464524,-19725.27784021315,3224.2068091700785,26565.84692919557,50306.116057828534,74451.59291526093,99008.96309552388,123985.02078586258,149386.670513198,175220.92891852488,201494.92655955208,228215.90974256094,255391.24238259182,283028.4078942486,311135.0111121542,339718.7802421951,368787.5688437759,398349.3578441723,428412.257584044,458984.50989654986,490074.4902191006,521690.7097389393,553841.8175727467,586536.6029815862,619783.9976200522,653593.0778223716,687973.0669243457,722933.3376224695,758483.4143704665,794632.9758146296,831391.8572667539,868770.053217672,906777.7198902112,945425.1778329278,984722.914555023,1024681.587203878,1065312.0252838873,1106625.2334199199,1148632.3941640668,1191344.8708472368,1234774.2104760113,1278932.146676232,1323830.6026821055,1369481.694374209,1415897.7333651627,1463091.2301345458,1511074.897213453,1559861.65242058,1609464.6221481357,1659897.1447015905,1711172.773691775,1763305.2814810458,1816308.6626840427,1870197.137725041,1924985.1564499848,1980687.4017977994,2037318.7935290975,2094894.4920143336,2153429.9020819417,2212940.6769286236,2273442.7220897586,2334952.1994748614,2397485.5314661656,2461059.4050824647,2525690.776208908,2591396.873894975,2658195.204718624,2726103.5572218,2795140.0064153103,2865322.918355431,2936670.954792924,3009203.0778969545,3082938.555051712,3157896.963731407,3234098.196451516,3311562.46579846,3390310.3095404813,3470362.595816899,3551740.528412299,3634465.6521130856,3718559.858149219,3804045.3897219333,3890944.8476205794,3979281.1959255785,4069077.7678047237,4160358.271399853,4253146.795807157,4347467.817151918,4443346.20476111,4540807.227430631],[-260340.027672305,-241320.66941624123,-221973.97989071335,-202294.53895736195,-182276.83824805787,-161915.27974336874,-141204.1743286047,-120137.74032675405,-98710.10200800141,-76915.28807512322,-54747.23012534203,-32199.76108693541,-9266.613631211454,14058.581440957496,37782.29683828936,61911.110444803024,86451.7070113679,111410.87987503014,136795.53270538687,162612.68127895193,188869.45528170583,215573.10014091176,242730.97888522502,270350.57403543894,298439.4895248718,327005.4526505461,356056.316055377,385600.0597424593,415644.793120513,446198.7570829247,477270.3261194376,508868.0104616571,541000.4582625951,573676.4578115281,606904.9397830914,640694.9795232986,675055.7993714081,709996.7710189796,745527.4179063218,781657.4176577723,818396.6045545416,855754.9720481818,893742.6753134606,932370.0338420176,971647.5340772138,1011585.8320915711,1052195.7563055356,1093488.3102508564,1135474.675377248,1178166.2139039144,1221574.4717163073,1265711.1813096406,1310588.2647779074,1356217.8368517996,1402612.2079842975,1449783.8874854934,1497745.58670707,1546510.2222782883,1596090.919391815,1646501.015143384,1697754.0619237984,1749863.8308650097,1802844.3153407776,1856709.7345239343,1911474.5369983255,1967153.4044300194,2023761.25529592,2081313.2486718374,2139824.788080507,2199311.5254017925,2259789.3648429993,2321274.466974222,2383783.2528268304,2447332.4080572184,2511938.887176474,2577619.917848279,2644393.0052529243,2712275.936522739,2781286.7852468346,2851443.9160476215,2922765.989229707,2995271.9655037103,3068981.1107827635,3143913.0010574088,3220087.5273467167,3297524.900727867,3376245.657446895,3456270.6641078396,3537621.1229468123,3620318.5771884355,3704384.9164874614,3789842.3824563595,3876713.574282049,3965021.454428726,4054789.3544340413,4146040.980795649,4238800.420951387,4333092.14935389,4428941.033643065,4526372.340913184],[-272832.8626968025,-253829.01189766463,-234497.93165486644,-214834.20261336525,-194832.31719490665,-174486.6781765623,-153791.5972468322,-132741.2935386293,-111329.89213887439,-89551.42257393803,-67399.8172715588,-44868.90999751631,-21952.434267669218,1355.9782654575538,25062.7994517521,49174.60630916804,73698.08271521423,98640.02112621046,124007.32432359876,149807.00718824216,176046.19850291032,202732.14278402342,229872.2021417017,257473.8581704381,285544.7138694131,314092.49559361325,343125.05503594177,372650.37124143494,402676.5526526426,433211.8391885904,464264.60435640183,495843.3573967293,527956.7454632125,560613.5558372883,593822.7181771831,627593.3068038675,661934.5430228489,696855.7974831385,732366.5925736371,768476.6048583188,805195.6675490106,842533.7730187683,880501.0753546744,919107.8929514177,958364.7111460469,998282.1848953369,1038871.141494469,1080142.5833403142,1122107.690738019,1164777.8247524416,1208164.5301048192,1252279.5381161938,1297134.769696345,1342742.3383816152,1389114.5534204117,1436263.922907936,1484203.1569705708,1532945.171001773,1582503.088947812,1632890.24664733,1684120.1952232542,1736206.704528768,1789163.7666478925,1843005.5994526313,1897746.6502148244,1953401.5992772542,2009985.3637821593,2067513.1014591916,2126000.2144733523,2185462.353335073,2245915.420870431,2307375.576256389,2369859.2391191768,2433383.093697926,2497964.093074245,2563619.4634699877,2630366.7086111754,2698223.614163305,2767208.2522359816,2837338.985959322,2908634.474132737,2981113.6759486273,3054795.855788773,3129700.5880991053,3205847.7623407133,3283257.588019292,3361950.599795781,3441947.6626753774,3523269.977281493,3605939.085212053,3689976.874480996,3775405.585045739,3862247.8144237734,3950526.5233953614,4040265.041799588,4131487.0744207716,4224216.706968523,4318478.412152204,4414297.05585328,4511697.90339228],[-285519.4785297585,-266531.4420091902,-247216.28054870444,-227568.57560319698,-207582.8204111975,-187253.41857346613,-166574.68260916462,-145540.83248888934,-124145.99414434168,-102384.19795382791,-80249.37720424356,-57735.366527805105,-34835.90031415946,-11544.611097036395,12144.972084771376,36239.4253529869,60745.431681272574,85669.78261441621,111019.38001458207,136801.23783552833,163022.48392503336,189690.3618565651,216812.23278925195,244395.5773584938,272447.99759620265,300977.218881842,329991.0919244706,359497.59477688116,389504.8348808994,420021.05114628305,451054.61606225837,482614.0378428721,514707.9626063814,547345.1765899716,580534.608398682,614285.3312912732,648606.5655019365,683507.6805991847,718998.1978821561,755087.7928157048,791786.2975031061,829103.7031993463,867050.1628638324,905635.9937538803,944871.6800593967,984767.8755801385,1025335.4064443142,1066585.2738717776,1108528.6569805266,1151176.91563803,1194541.5933578254,1238634.4202428397,1283467.3159742288,1329052.3928491084,1375401.9588659632,1422528.520859275,1470444.7876838143,1519163.673450439,1568698.3008117238,1619062.00430143,1670268.333726308,1722331.0576119765,1775264.1667033888,1829081.877521882,1883798.635976925,1939429.1210371128,1995988.2484585806,2053491.174572851,2111953.300134655,2171390.27423191,2231817.99825582,2293252.629936008,2355710.5874387817,2419208.5535306474,2483763.4798077815,2549392.590993676,2616113.389302942,2683943.6588765047,2752901.4702861137,2823005.1851106216,2894273.4605846466,2966725.2543221544,3040379.8291127095,3115256.757796131,3191375.9282133654,3268757.548235817,3347422.15087585,3427390.5994756706,3508684.092981158,3591324.171298028,3675332.7207331993,3760731.979522123,3847544.543445239,3935793.3715305477,4025501.7918495033,4116693.5074033057,4209392.6021027975,4303623.546842793,4399411.205674284,4496780.842071251],[-298397.8453934025,-279425.90176429914,-260126.9400496782,-240495.54253728956,-220526.20330560568,-200213.32680247352,-179551.2264013479,-158534.12293439615,-137156.1432022287,-115411.318459464,-93293.58287678589,-70796.77197773173,-47914.62105087959,-24640.763536566636,-968.7293879331555,23108.056594581343,47596.276454594685,72502.72079889197,97834.29054361326,123597.99868839234,149800.97211871087,176450.45343749458,203553.80282501644,231118.49992941786,259152.1457868917,287662.46477263,316657.306582802,346144.64824860403,376132.59618149046,406629.3882519873,437643.39590115217,469183.1262858417,501257.2244580239,533874.4755794124,567043.8071703047,600774.2913953725,635075.1473852855,669955.7435955135,705425.6002025439,741494.3915388898,778171.9485657052,815468.2613859812,853393.4817971778,891957.9258846093,931172.0766560442,971046.5867188678,1011592.2809985906,1052820.1595019577,1094741.4001233308,1137367.3614959503,1180709.5858884107,1224779.8021478981,1269589.9286889217,1315152.0765309604,1361478.5523837546,1408581.861781848,1456474.7122687662,1505170.0166327239,1554680.8961921348,1605020.684134997,1656202.9289105686,1708241.397675155,1761150.079792465,1814943.190390547,1869635.1739734258,1925240.7080919964,1981774.7070723162,2039252.3258033374,2097688.96358463,2157100.268036225,2217502.1390686138,2278910.732917754,2341342.466243217,2404814.0202915957,2469342.345125836,2534944.663922771,2601638.4773368086,2669441.5679349843,2738372.0047013615,2808448.1476131817,2879688.6522893966,2952112.4747141185,3025738.8760327445,3100587.4274264844,3176678.0150631014,3254030.8451261045,3332666.4489251105,3412605.6880846075,3493869.7598176273,3576480.20228179,3660458.900020519,3745828.0894902265,3832610.3646766385,3920828.6827972103,4010506.3700968884,4101667.127734239,4194335.0377612095,4288534.56919731,4384290.584201651,4481628.344339618],[-311470.57712491136,-292515.00764048647,-273232.5292964807,-253617.72523662436,-233665.09040268965,-213369.0301132081,-192723.85861977492,-171723.79764021817,-150362.97486839443,-128635.4224598282,-106535.07549383095,-84055.77041036868,-61191.24342231336,-37935.128902226454,-14280.957743490813,9777.844305138336,34247.95833080239,59136.173975585494,84449.39118262054,110194.62197016319,136378.99223385495,163009.74357825867,190094.23517666943,217639.9456615732,245654.475044735,274145.54666808364,303121.00918559264,332588.8385772626,362557.14019426866,393034.15083767846,424028.24086984666,455547.9163595892,487601.82126141083,520198.73963005166,553347.5978692491,587057.4670174303,621337.5650692505,656197.2593343067,691646.0688332715,727693.6667327986,764349.882818056,801624.7060058292,839528.2868970493,878070.9403701075,917263.1482153307,957115.5618120711,997639.0048471054,1038844.4760776544,1080743.1521376625,1123346.3903889577,1166665.7318176348,1210712.9039771783,1255499.8239771088,1301038.6015205139,1347341.541989238,1394421.1495783187,1442290.1304800662,1490961.39611965,1540448.066440518,1590763.4732436503,1641921.1635791492,1693934.9031918962,1746818.6800218064,1800586.7077606427,1855253.4294635474,1910833.521219816,1967341.8958810696,2024793.7068488759,2083204.3519223481,2142589.477207872,2202964.981088981,2264347.0182612417,2326752.0038302764,2390196.6174750496,2454697.807677068,2520272.796017796,2586939.081542204,2654714.4451937056,2723616.9543184144,2793664.9672411727,2864877.137913935,2937272.420639108,3010870.0748655205,3085689.6700628176,3161751.090672069,3239074.5411348147,3317680.551003295,3397589.9801290627,3478824.023936517,3561404.218778804,3645352.447378882,3730690.944356564,3817442.3018446844,3905629.4751913785,3995275.7887556814,4086404.941793506,4179041.014437248,4273208.4737698175,4368932.179996516,4466237.392711546],[-324737.5943776311,-305798.67138903437,-286532.95109424705,-266935.01751636073,-246999.36648408195,-226720.40421051532,-206092.44584953832,-185109.71402904205,-163766.33736079582,-142056.34892616002,-119973.68473827653,-97512.18217900558,-74665.57841125573,-51427.508765838575,-27791.505102674942,-3750.994145426899,20700.704209702788,45570.3786129395,70864.92800695566,96591.36340087256,122756.80967245251,149368.5073995369,176433.81471979222,203960.20922107622,231955.28986145277,260426.77891998715,289382.5239785451,318830.49993569404,348778.8110517557,379235.6930274565,410209.51511521055,441708.7822642233,473742.13729963196,506318.3631369467,539446.3850307392,573135.2728602355,607394.243450759,642232.662932365,677660.0491358633,713686.0740276421,750320.5661821035,787573.513294664,825455.064734187,863975.5341362071,903145.4020372995,942975.3185520656,983476.1060914453,1024658.7621256143,1066534.461990182,1109114.5617372305,1152410.601031602,1196434.3060939119,1241197.5926890946,1286712.5691638126,1332991.5395315553,1380047.0066069476,1427891.6751897319,1476538.45530024,1526000.4654646884,1576291.0360543323,1627423.7126769319,1679412.259622294,1732270.6633624092,1786013.1361081505,1840654.11942068,1896208.2878821031,1952690.5528235226,2010116.0661125316,2068500.2240006907,2127858.671033126,2188207.304018285,2249562.276062684,2311940.0006688097,2375357.15589827,2439830.688600863,2505377.8187118666,2572016.0436154353,2639763.1425794256,2708637.181259494,2778656.516275,2849839.7998572504,2922205.98457269,2995774.3281187355,3070564.398198031,3146596.0774688926,3223889.5685742376,3302465.3992516357,3382344.427521794,3463547.8469619346,3546097.192061533,3630014.3436632473,3715321.534489792,3802041.3547599553,3890196.75789072,3979811.0662926966,4070907.9772559544,4163511.5689294487,4257646.306394897,4353337.0478384895,4450609.050817232],[-338202.0959644271,-319280.10232631664,-300031.42541662056,-280450.65016340336,-260532.27330819226,-240270.70198484464,-219660.25227598194,-198695.14774631895,-177369.5179525877,-155677.39692933555,-133612.72165118298,-111169.33046984789,-88340.96152653452,-65121.2511388869,-41503.73216226045,-17481.83232443058,6951.127465517726,31801.93483608216,57077.48769928818,82784.79602459748,108930.98364101979,135523.29006847437,162569.0723774454,190075.80707926163,218051.09204602824,246502.64846133394,275438.32280196785,304866.08885171753,334794.04974634876,365230.44005215494,396183.627877153,427662.11701609613,459674.5491295124,492229.70595807116,525336.5115711542,559004.034652364,593241.490820878,628058.2449899705,663463.8137629458,699467.86786786,736080.2346298576,773310.9004840676,811170.0135279442,849667.886114367,888814.9974859315,928621.9964518116,969099.7041059704,1010259.1165899541,1052111.4078989765,1094667.9327328529,1137940.2293921704,1181940.0227212089,1226679.2270963357,1272169.9494633498,1318424.492422424,1365455.3573623297,1413275.2476442775,1461897.0718372855,1511333.9470033506,1561599.2020364855,1612706.381054059,1664669.2468422186,1717501.7843559012,1771218.204275412,1825832.946617698,1881360.6844068829,1937816.3274021796,1995215.0258852732,2053572.1745076485,2112903.416200064,2173224.6461421754,2234552.015797143,2296901.9370094053,2360291.0861676806,2424736.4084339123,2490255.122040401,2556864.7226530807,2624582.987806171,2693427.981406172,2763418.05830758,2834571.868961015,2906908.3641362498,2980446.7997178948,3055206.7415795033,3131208.070533868,3208470.987361783,3287016.0179219563,3366864.0183393266,3448036.1802782826,3530554.0362982494,3614439.465294441,3699714.698024585,3786402.3227247717,3874525.2908114027,3964106.9226764594,4055170.9135731636,4147741.3395952373,4241842.663750601,4337499.74213291,4434737.830187724],[-351863.37127669854,-332958.56754844345,-313727.19678409805,-294163.84483889176,-274263.0093896069,-254019.0985135301,-233426.42924495763,-212479.22610859224,-191171.61962955957,-169497.64481927105,-147451.23963776627,-125026.24343081447,-102216.39534238796,-79015.33270168351,-55416.5893844855,-31413.594147964148,-6999.668939623982,17831.9728214785,43088.22799061448,68776.10547119193,94902.72801679373,121475.33406244311,148501.27958417404,175988.03998922743,203943.21203588252,232374.51578408387,261289.7965770564,290697.0270550307,320604.309200105,351019.8764147167,381952.0956327412,413409.46946440544,445400.63837523316,477934.3829003086,511019.62589275464,544665.4348091455,578881.0240307467,613675.7572219465,649059.1497260858,685040.8710000745,721630.7470866237,758838.7631270329,796675.0659134388,835149.9664817865,874273.9427460302,914057.6421748749,954511.8845098836,995647.6645281559,1037476.1548482974,1080008.708781248,1123256.863226333,1167232.3416140606,1211947.0568944183,1257413.1145740733,1303642.815801222,1350648.6604996738,1398443.350552581,1447039.7930376818,1496451.103512343,1546690.6093524816,1597771.8531437656,1649708.5961269243,1702514.8216976186,1756204.7389628887,1810792.7863523078,1866293.6352883715,1922722.1939142873,1980093.6108812136,2038423.2791954554,2097726.8401278006,2158020.1871829974,2219319.470134234,2281641.099120761,2345001.7488107444,2409418.362630088,2474908.1570593882,2541488.62599709,2609177.5451939814,2677992.9767570253,2747953.273724943,2819077.0847161794,2891383.3586517717,2964891.3495508875,3039620.621404771,3115591.0531268707,3192822.84358145,3271336.5166933425,3351152.9266360793,3432293.263104977,3514779.0566725302,3598632.184229007,3683874.874509004,3770529.7137071164,3858619.651179714,3948168.0052400436,4039198.469043699,4131735.1165677123,4225802.408684062,4321425.199331038,4418628.741779214],[-365723.4189366719,-346836.0628679114,-327622.2581548904,-308076.59160160937,-288193.56184196554,-267967.57791875256,-247392.95784026268,-226463.9271137626,-205174.617255572,-183519.06427703053,-161491.2071469028,-139084.88622857002,-116293.8416925848,-93111.71190377465,-69532.03178269416,-45548.23114049644,-21153.63298798632,3658.548183097737,28895.208145851037,54563.35471194913,80670.10953357676,107222.7099346118,134228.5107701032,161694.98631637357,189629.7321907645,218040.4673021615,246935.03583251848,276321.40925048664,306207.68835618324,336602.1053595601,367513.0259914079,398948.95164817036,430918.52157079056,463430.51505886763,496493.85371904215,530117.6037502934,564310.9782650908,599083.339647705,634444.2019499433,670403.2333256474,706970.2585028308,744155.2612963493,781968.3871600307,820419.945779558,859520.4137065336,899280.4370351231,939710.8341200335,980822.5983390964,1022626.900899102,1065135.09368652,1108358.71216342,1152309.478310164,1196999.3036135712,1242440.2921039967,1288644.743440065,1335625.1560426117,1383394.2302783057,1431964.871694753,1481350.1943054167,1531563.5239284015,1582618.4015775386,1634528.5869075384,1687308.061713753,1740971.033488466,1795531.9390319313,1851005.4481226061,1907406.4672448165,1964750.1433758398,2023051.867832975,2082327.2801827192,2142592.27221012,2203862.9919530926,2266155.847799893,2329487.512651846,2393874.9281519805,2459335.308981873,2525886.1472246093,2593545.216799161,2662330.577964034,2732260.5818927116,2803353.875321443,2875629.405271974,2949106.4238468977,3023804.4931034553,3099743.490003491,3176943.6114418944,3255425.3793561803,3335209.645914469,3416317.5987883676,3498770.7665082174,3582591.0239035003,3667800.597629217,3754422.07178138,3842478.3935986077,3931992.8792570466,4022989.219755653,4115491.4868951016,4209524.139351094,4305112.028845561,4402280.406412402],[-379785.90981890913,-360916.268274546,-341720.2987337577,-322192.58897210064,-302327.6386035938,-282119.8576598163,-261563.5651465787,-240652.98757748236,-219382.25748408958,-197745.41190196038,-175736.3908331527,-153349.03568348754,-130577.08767520008,-107414.1862341296,-83853.86735124653,-59889.561917628394,-35514.594033586094,-10722.179290048545,14494.576977117453,40142.681460764725,66229.25468430505,92761.53283278923,119746.86961230193,147192.73813996022,175106.73286356707,203496.57151203952,232370.09707683674,261735.27982547856,291600.21934623225,321973.14662636607,352862.42616305966,384276.55810810486,416224.18044666364,448714.0712113066,481755.1507302965,515356.48391278205,549527.2825698357,584276.9077726624,619614.8722482133,655550.842813588,692094.6428480428,729256.2548055646,767045.8227668717,805473.6550321928,844550.2267552032,884286.1826195559,924692.3395567506,965779.6895085764,1007559.4022328388,1050042.8281539571,1093241.5012587563,1137167.1420390394,1181831.6604796173,1227247.1590952687,1273425.9360153435,1320380.4881175722,1368123.5142115587,1416667.9182737453,1466026.81273221,1516213.5218053125,1567241.5848926287,1619124.7600199862,1671877.027339073,1725512.592683601,1780045.891180182,1835491.5909184415,1891864.5966785112,1949180.0537179976,2007453.3516188827,2066700.128196553,2126936.273469012,2188177.9336910374,2250441.515451499,2313743.6898359386,2378101.3966550454,2443531.8487413432,2510052.536311986,2577681.231402978,2646435.9923726646,2716335.168477005,2787397.4045171943,2859641.6455622218,2933087.1417440753,3007753.4531313367,3083660.4546789927,3160828.3412566846,3239277.6327581257,3319029.1792888977,3400104.16643915,3482524.1206386695,3566310.914597101,3651486.7728301454,3738074.277274859,3826096.3729910683,3915576.373956094,4006537.968949841,4099005.227533512,4193002.606122737,4288554.954158541,4385687.520372927],[-394050.9592394647,-375199.28202455805,-356021.399499765,-336511.90043230576,-316665.2854366539,-296475.96555371815,-275938.2608075957,-255046.39873921522,-233794.51291659125,-212176.64142094832,-190186.7253093084,-167818.60705184448,-145066.02894461364,-121922.63149684062,-98381.95179253654,-74437.42182556028,-50082.36680884636,-25310.00345586799,-113.43823506124318,25514.334403738612,51580.43383104727,78092.09506870992,105056.67064919858,132481.63250551792,160374.57389075332,188743.2113284003,217595.38659369713,246939.0687270325,276782.35607853415,307133.478386221,338000.7988867988,369392.81646026904,401318.1678085639,433785.6296694875,466804.12106489134,500382.70558574516,534530.5937130766,569257.1451760605,604571.8713475203,640484.437678219,677004.6661687468,714142.5378819744,751908.1954949517,790311.9458915438,829364.2627962718,869075.7894506976,909457.3413311937,950519.9089112396,992274.6604670491,1034732.9449280074,1077906.2947723693,1121806.4289696782,1166445.255968679,1211834.8767341445,1257987.5878313382,1304915.8845596958,1352632.4641361812,1401150.2289301222,1450482.2897478528,1500641.96917123,1551642.8049484345,1603498.5534388474,1656223.1931125287,1709830.9281062272,1764336.191834115,1819753.6506577523,1876098.207613428,1933385.006198981,1991629.4342205434,2050847.1277014222,2111053.97485115,2172266.120099484,2234499.9681935823,2297772.188360396,2362099.718535007,2427499.769657132,2493989.830033781,2561587.6697732736,2630311.345288595,2700179.203872476,2771209.88834486,2843422.3417752795,2916835.812277877,2991469.857884847,3067344.351496049,3144479.4859070824,3222895.7789185233,3302614.078523534,3383655.5681803785,3466041.772167312,3549794.5610225946,3634936.1570705175,3721489.1400365,3809476.452748304,3898921.406930563,3989847.689089693,4082279.366492378,4176240.893238507,4271757.11643194,4368853.282445887],[-408520.55281486805,-389687.0860348671,-370527.53861452045,-351036.50033210835,-331208.4728221715,-311037.8681547572,-290519.00739227515,-269646.11912322533,-248413.33797252947,-226814.70308775338,-204844.1566017759,-182495.54207023792,-159762.6028843678,-136638.98065834446,-113118.21359102591,-89193.73480109149,-64858.87063636421,-40106.83895537956,-14930.747381896013,10676.408468571957,36721.74679039675,63212.50141872582,90156.02368867677,117559.78432514053,145431.3753622251,173778.51209349278,202609.03505317448,231930.91202948033,261752.24010908464,292081.247755161,322926.2969180825,354295.8851799101,386198.6479329262,418643.3605934577,451638.9408499424,485194.45094788517,519319.100010684,554022.2463976033,589313.4000991555,625202.2251712803,661698.5422071053,698812.3308492841,736553.7323417682,774933.0521223291,813960.7624562657,853647.5051126734,894004.0940820742,935041.518338595,976770.9446454651,1019203.7204053246,1062351.3765557795,1106225.6305117048,1150838.3891530125,1196201.751861336,1242328.0136043795,1289229.6680694353,1336919.4108466152,1385410.142663541,1434714.9726698468,1484847.2217755446,1535820.426041689,1587648.340125108,1640344.940777718,1693924.4304024035,1748401.2406635897,1803790.0361570532,1860105.71813712,1917363.4283033197,1975578.5526469748,2034766.7253599262,2094943.832803419,2156126.017541948,2218329.682440259,2281571.4948255876,2345868.39071582,2411237.5791158387,2477696.546379997,2545263.0606459826,2613955.1763379825,2683791.238741598,2754789.8886511307,2826970.0670917816,2900351.0201144973,2974952.303669212,3050793.788554296,3127895.66544444,3206278.4499996966,3285962.988052906,3366970.4608820197,3449322.3905647947,3533040.645418615,3618147.4455263214,3704665.3683510944,3792617.354437493,3882026.7132057687,3972917.1288365554,4065312.666249171,4159237.7771743396,4254717.306324735,4351776.497660156],[-423196.8286688959,-404381.81556788424,-385240.84843338944,-365768.5180735589,-345959.32716199884,-325807.6888171118,-305307.92515902873,-284454.2658434324,-263240.84657197935,-241661.7075786125,-219710.7920923161,-197381.94477466634,-174668.91013273527,-151565.33090656856,-128064.74643099122,-104160.59097084543,-79846.19203042914,-55114.768635140965,-29959.42958610435,-4373.171686784364,21651.122058534995,48120.68427554914,75042.86507902946,102425.13396250829,130275.08171762899,158600.42238427978,187408.99523168616,216708.76677163295,246507.8328028242,276814.4204888209,307636.8904686186,338983.73900103546,370863.60014312016,403285.2479638688,436257.59879217646,469789.71350167086,503890.79983142647,538570.2147438107,573837.466819762,609702.218692821,646174.289520795,683263.6574979585,720980.4624067042,759335.0082099359,798337.7656846493,837999.375098076,878330.6489251778,919342.5746107008,961046.3173745289,1003453.2230618752,1046574.8210386871,1090422.8271338148,1135009.1466266506,1180345.8772836667,1226445.312442618,1273319.9441459377,1320982.4663237992,1369445.7780286798,1418722.9867197112,1468827.4116008985,1519772.5870116337,1571572.2658712706,1624240.423178306,1677791.2595660791,1732239.2049132132,1787598.9220132527,1843885.3103017136,1901113.5096425654,1959298.904174677,2018457.126220359,2078604.060254105,2139755.8469362473,2201928.8872098103,2265139.84646257,2329405.658755047,2394743.5311166737,2461170.9479081007,2528705.675254876,2597365.7655504416,2667169.5620308565,2738135.703421912,2810283.128661136,2883631.0816924646,2958199.1163392877,3034007.101253714,3111075.2249442614,3189424.0008847043,3269074.272701302,3350047.2194449347,3432364.36094559,3516047.563251989,3601119.0441572024,3687601.378813346,3775517.5054323957,3864890.7310802983,3955744.737561456,4048103.5873968084,4141991.7298963354,4237434.007329392,4334455.661189656],[-438081.9621973408,-419285.6431523921,-400163.49857226654,-380710.12031319423,-360920.01210624806,-340787.5881367759,-320307.17160142283,-299472.9932420526,-278279.18985624146,-256719.80278370064,-234788.77636912419,-212479.95639982447,-189787.08851873945,-166703.81661200197,-143223.6811708454,-119340.1176269413,-95046.45466193277,-70335.91248918301,-45201.601108495845,-19636.518532847753,6366.451013050508,32814.53892343654,59715.09407048253,87075.5846938903,114903.60032012477,143206.8537124258,171993.18285182305,201270.55295021553,231047.0584946426,261330.92532509216,292130.5127449762,323454.3156653985,355310.96678344905,387709.23879580153,420658.0466465452,454166.44981190306,488243.65462082066,522899.0166126988,558142.0429325306,593982.3947648301,630429.8898051574,667494.5047721944,705186.3779592696,743515.8118266191,782493.2756348422,822129.4081209023,862435.0202155046,903421.0978049992,945098.8045366071,987479.4846684728,1030574.6659649247,1074396.06263852,1118955.5783375197,1164265.3091823054,1210337.546849421,1257184.7817048302,1304819.7059868306,1353255.2170404743,1402504.4206017433,1452580.6341356635,1503497.3902266095,1555268.440022771,1607907.7567351523,1661429.5391931187,1715848.215454684,1771178.446475983,1827435.1298381558,1884633.4035336787,1942788.6498126222,2001916.4990910473,2062032.8339195624,2123153.793016821,2185295.7753661936,2248475.4443776514,2312709.732115589,2378015.8435948156,2444411.261142672,2511913.7488325387,2580541.356986641,2650312.426750596,2721245.594740331,2793359.7977639055,2866674.2776159914,2941208.585950726,3016982.589230776,3094016.4737548414,3172330.750766298,3251946.261640232,3332884.1831553676,3415166.032848343,3498813.674453143,3583849.323426502,3670295.5525623853,3758175.2976925913,3847511.863480659,3938328.9293061164,4030650.5552423536,4124501.188128908,4219905.66774157,4316889.233057107],[-453178.1648085718,-434400.777326306,-415297.6946525733,-395863.5097092936,-376092.727302804,-355979.7627033787,-335518.9402023533,-314704.4916461194,-293530.5549467404,-271991.17256844183,-250080.2899905725,-227791.75414532213,-205119.311830848,-182056.60809891485,-158597.18461692566,-134734.47800334264,-110461.81813733233,-85772.42644060403,-60659.41413225187,-35115.78045557905,-9134.410876763519,17291.92474570847,44170.57401993219,71509.00391005632,99314.8026554226,127595.68172045494,156359.47777556814,185614.1547101338,215367.80567662907,245628.65516834683,276405.06112973765,307705.51710056653,339538.65439408645,371913.24431052804,404838.20038478216,438322.5806710257,472375.59006317356,507006.5826524808,542225.0641225665,578040.6941832032,614463.2890417196,651502.8239149491,689169.4355806145,727473.424969461,766425.259798584,806035.5772472834,846315.1866743127,887275.0723796394,928926.3964095409,971280.5014064994,1014348.9135043428,1058143.3452701233,1102675.698691461,1147958.0682128142,1194002.7438193662,1240822.2141701395,1288429.169780763,1336836.506257718,1386057.3275823987,1436104.9494490253,1486992.902654834,1538734.9365443555,1591345.0225082615,1644837.3575387627,1699226.3678397187,1754526.7124959466,1810753.2871999433,1867921.2280380465,1926045.9153365237,1985142.9775697961,2045228.2953288215,2106318.0053544147,2168428.5046337508,2231576.454562057,2295778.78517027,2361052.6994208326,2427415.6775696296,2494885.481599319,2563480.1597219324,2633218.050953253,2704117.7897595223,2776198.3107790784,2849478.8536166423,2923978.967715975,2999718.517308731,3076717.68644177,3154996.9840855952,3234577.2493211823,3315479.6566116903,3397725.721156551,3481337.304330661,3566336.6192095866,3652746.236183837,3740589.088659227,3829888.478850576,3920668.0836657397,4012951.960683251,4106764.554224373,4202130.701522983,4299075.638990063],[-468487.6940928523,-449729.47483965557,-430645.6925828849,-411230.94132796477,-391479.72697449883,-371386.4658958884,-350945.483496543,-330151.01274598157,-308997.19268952636,-287478.0669349164,-265587.58211533935,-243319.58632726828,-220667.82754367287,-197625.95200177422,-174187.50256516458,-150345.9170593461,-126094.52658145875,-101426.5537822533,-76335.11112100491,-50813.19909246988,-24853.704425655305,1550.6017466501798,28407.06574664265,55723.15324087115,83506.45115925814,111764.66964488756,140505.6440347829,169737.33687276556,199467.83995348774,229705.37640002393,260458.30277408706,291735.11122007295,323544.43164311326,355895.0339224236,388795.8301588907,422255.87695955415,456284.37775793043,490890.68517149333,526084.3033965724,561874.8906420018,598272.2616003994,635286.3899599744,672927.4109557876,711205.6239617402,750131.4951237657,789715.6600355508,829968.9264556335,870902.2770690431,912526.8722922215,954854.0531228066,997895.344034607,1041662.4559193382,1086167.289073824,1131421.9362361054,1177438.6856691856,1224230.0242939878,1271808.6408719637,1320187.4292391883,1369379.4915902675,1419398.1418160764,1470256.9088938008,1521969.5403310517,1574550.005664548,1628012.5000153407,1682371.4476987594,1737641.5058935618,1793837.568368474,1850974.769268175,1909068.4869592283,1968134.3479381092,2028188.230799423,2089246.270269048,2151324.8613004424,2214440.6632361733,2278610.604035369,2343851.8845693385,2410181.982983322,2477618.6591296233,2546179.9590700213,2615884.219649953,2686750.0731450226,2758796.4519824497,2832042.5935351327,2906508.044994128,2982212.6683172886,3059176.6452563675,3137420.4824652467,3216965.0166865615,3297831.420023189,3380041.205292129,3463616.23146348,3548578.7091854187,3634951.2063982505,3722756.6540345605,3812018.351812672,3902759.97412045,3995005.5759927277,4088779.599183103,4184106.8783336068,4281012.647238945],[-484012.8573738588,-465274.04382778273,-446209.80134541716,-426814.7250343567,-407083.32190635044,-387010.0094570103,-366589.1142231168,-345814.8703168214,-324681.41793644195,-303182.80185316387,-281312.9698741841,-259065.77128064516,-236434.95524094766,-213414.16919861105,-189996.95723449555,-166176.7584024507,-141946.90503914212,-117300.62104613986,-92231.02014493989,-66731.10410402925,-40793.76093775593,-14411.763075996423,12422.23449452105,39715.696119457716,67476.20739577198,95711.4771213932,124429.33927587792,153637.75503315474,183344.81480543152,213558.74032066623,244287.88673266373,275540.7447649967,307325.9428889265,339652.24953663326,372528.5753486734,405963.9754583235,439967.6518117897,474548.95552554843,509717.3892811036,545482.6097585196,581854.43010755,618842.822459317,656457.9204774238,694710.0219498209,733609.5914218312,773167.2628717546,813393.8424278114,854300.3111296257,895897.8277330205,938197.7315596433,981211.5453917813,1024950.9784139667,1069427.9292000069,1114654.48874895,1160642.9435686693,1207405.7788086645,1254955.6814425113,1303305.5435017985,1352468.4653598648,1402457.759069384,1453286.9517522326,1504969.7890434403,1557520.2385897066,1610952.4936044489,1665280.9764775776,1720520.342444477,1776685.483312361,1833791.5312460898,1891853.8626139106,1950888.1018953172,2010910.125649074,2071936.0665461773,2133982.317465974,2197065.53565751,2261202.64696678,2326410.8501321627,2392707.6211459776,2460110.717687406,2528638.1836247337,2598308.3535892977,2669139.8576218197,2741151.6258936357,2814362.8935005683,2888793.205335172,2964462.421035183,3041390.7200104017,3119598.6065507075,3199106.9150124565,3279936.8150897725,3362109.8171681836,3445647.7777633863,3530572.9050459974,3616907.764455362,3704675.284399503,3793898.7620483134,3884601.8692171546,3976808.6583440118,4070543.5685610855,4165831.431864187,4262697.479376754],[-499755.98679378955,-481036.8131666413,-461992.34649464546,-442617.18300547625,-422905.83084112743,-402852.70863772184,-382452.1440829204,-361698.37245021085,-340585.53510981356,-319107.67801548215,-297258.75016775215,-275032.60205198056,-252422.98405175377,-229423.54483686597,-206027.82972564548,-182229.27902070037,-158021.22631887067,-133396.8967934195,-108349.40544916107,-82871.75534964679,-56956.83581614867,-30597.42059743381,-3786.1660112098325,23484.39094499848,51221.834512641886,79433.87212170195,108128.33637122181,137313.187041929,166996.5131400309,197186.5349745385,227891.60626726644,259120.21629659832,290880.99207530054,323182.70056361286,356034.2509165774,389444.6967682552,423423.23855178733,457979.2258566194,493122.1598231108,528861.6955759327,565207.6446950596,602169.9777273063,639758.8267373024,677984.4878992026,716857.4241295946,756388.267762925,796587.823268312,837467.0700108886,879037.1650564233,921309.446020812,964295.4339647379,1008006.836335111,1052455.5499519533,1097653.6640442011,1143613.4633331276,1190347.4311649813,1237868.2526932824,1286188.818112587,1335322.2259420548,1385281.7863628678,1436081.0246079394,1487733.6844056747,1540253.7314783253,1593655.3570968616,1647952.9816905714,1703161.2585158485,1759295.0773823934,1816369.5684388233,1874400.1060182417,1933402.312545908,1993392.062507073,2054385.4864797576,2116398.975230667,2179449.183876349,2243553.0361102605,2308727.7284979858,2374990.734838608,2442359.8105974263,2510852.9974079775,2580488.6276458073,2651285.329074572,2723262.029567064,2796437.961898874,2870832.6686204234,2946466.0070051914,3023358.1540763834,3101529.6117147207,3181001.211844624,3261794.1217052666,3343929.849203988,3427430.2483548312,3512317.5248030564,3598614.241438722,3686343.3240963807,3775528.0673480574,3866192.1403865726,3958359.593002488,4052054.861655422,4147302.77564321,4244128.563365663],[-515721.3599448933,-497022.0722586905,-477997.62938267866,-458642.6286842573,-438951.5794555353,-418918.90149323037,-398538.9236561905,-377805.88239977136,-356713.92028686614,-335257.0844747897,-313429.32517864276,-291224.49410946085,-268636.34288772685,-245658.5214314554,-222284.5763186249,-198507.9491230538,-174321.9747244462,-149719.87959069805,-124694.78003314836,-99239.68043387751,-73347.4714448019,-47010.92815756891,-20222.708245130256,7024.6499272736255,34738.72922078031,62927.235672333045,91598.00047509186,120758.98199093784,150418.26779412315,180584.0767484887,211264.7611173184,242468.80870698672,274204.84504462755,306481.63559111,339308.08798820805,372693.25434270385,406646.3335463065,441176.6736327219,476293.7741721482,512007.2887045101,548327.0272103213,585262.9586220675,622825.2133750436,661024.0859989193,699870.0377504737,739373.6992889079,779545.8733924816,820397.5377197016,861939.847613808,904184.1389520878,947141.9310403864,990824.9295543907,1035245.0295263445,1080414.3183806946,1126345.079017349,1173049.7929441421,1220541.1434589766,1268832.0188833957,1317935.5158460017,1367864.942619697,1418633.8225112064,1470255.8973046744,1522745.1307598308,1576115.7121666924,1630382.0599549704,1685558.82536269,1741660.8961621772,1798703.4004455127,1856701.7104698936,1915671.446565118,1975628.4811012216,2036588.942521059,2098569.219436033,2161585.9647870474,2225656.100071362,2290796.8196376157,2357025.595046998,2424360.1795057496,2492818.612366974,2562419.2237041984,2633180.6389572676,2705121.783653139,2778261.888199341,2852620.4927557735,2928217.4521827204,3005072.9410672793,3083207.4588309443,3162641.8349155337,3243397.2340540355,3325495.161623766,3408957.4690846796,3493806.35950365,3580064.39316779,3667754.493283917,3756899.951771277,3847524.4351446307,3939651.9904909236,4033307.0515403715,4128514.4448353443,4225299.395993875],[-531910.1284266848,-513230.95943303534,-494226.7748834024,-474892.17330213205,-455221.66514884925,-435209.67139846133,-414850.5220987813,-394138.4549050196,-373067.6135909043,-351632.0465356887,-329825.7051876164,-307642.4425021666,-285076.0113556958,-262120.06293362705,-238768.14509299165,-215013.700698419,-190850.06593229994,-166270.4685772017,-141268.0262712536,-115835.74473553104,-89966.51597330626,-63653.11644004518,-36888.205185099505,-9664.321962817805,18026.114685995504,46190.80938337045,74837.59189448948,103974.41914002085,133609.3772396769,163750.68358927662,194406.68897047848,225585.8796942928,257296.87977861287,289548.4531610571,322349.50594600826,355709.08868856286,389636.398714317,424140.78247631015,459231.73794933944,494918.9170630921,531212.1281728342,568121.3385706556,605656.6770361573,643828.4364278617,682647.0763158067,722123.2256566926,762267.6855103667,803091.4318008644,844605.6181207167,886821.5785801241,929750.8307012997,973405.0783595834,1017796.2147699995,1062936.3255227294,1108837.6916662105,1155512.792839425,1202974.3104538622,1251235.13092694,1300308.3489652248,1350207.2709015086,1400945.4180841427,1452536.5303204674,1504994.5693747855,1558333.7225228706,1612568.406161215,1667713.2694754351,1723783.1981660814,1780793.3182338937,1838758.9998249724,1897695.8611380595,1957619.7723919966,2018546.8598580814,2080493.5099556085,2143476.3734126203,2207512.369492541,2272618.690289012,2338812.805086826,2406112.464794244,2474535.7064445727,2544100.8577695154,2614826.5418448313,2686731.6818109457,2759835.505666134,2834157.5511381375,2909717.6706319298,2986536.0362559147,3064633.1449292754,3144029.8235676745,3224747.2343538515,3306806.880090547,3390230.60963857,3475040.623440817,3561259.4791353755,3648910.0972547107,3738015.767018144,3828600.1522146873,3920687.2971794503,4014301.632864467,4109467.983007314,4206211.570394351],[-548324.5408443965,-529665.7183736084,-510682.02168237895,-491368.05046817183,-471718.31637446943,-451727.2415708923,-431389.15731091495,-410698.3024664812,-389648.8220392149,-368234.7656475466,-346450.08599028224,-324288.6372849692,-301744.17368164146,-278810.34765112586,-255480.70834769378,-231748.6999451723,-207607.6599472277,-183050.8174698865,-158071.29149704357,-132662.08910798468,-106816.10367673077,-80526.11304217228,-53784.7776498911,-26584.638663386693,1081.88395433384,29222.49338992522,57845.017959871795,86957.41312272241,116567.76352248364,146684.28506557457,177315.3270304117,208469.3742107849,240155.04909326485,272381.11406988604,305156.4736850613,338490.17691938207,372391.4195092672,406869.54630374955,441934.0536586691,477594.59186962666,513860.9676425399,550743.1466047454,588251.2558555198,626395.5865573399,665186.5965683316,704634.913117242,744751.3355197764,785546.8379394838,827032.5721918934,869219.8705935341,912120.2488560947,955745.4090273988,1000107.2424777667,1045217.8329353447,1091089.4595690027,1137734.6001204753,1185165.9340861342,1233396.3459502333,1282438.9284679594,1332306.9860023127,1383014.0379132784,1434573.8220010444,1487000.2980038167,1540307.6511521249,1594510.295777871,1649622.878982562,1705660.2843629138,1762637.635795911,1820570.3012837858,1879473.8968610873,1939364.2905619484,2000257.6064522315,2062170.2287248708,2125118.80586039,2189120.254853364,2254191.7655070117,2320350.804793925,2387615.121288161,2456002.749666617,2525532.015282134,2596221.538808965,2668090.2409631154,2741157.347295341,2815442.3930625087,2890965.228175126,2967746.022223318,3045805.2695839284,3125163.7946059857,3205842.7568810424,3287863.656595868,3371248.3399702664,3456019.0047808364,3542198.2059738194,3629808.8613640424,3718874.257427127,3809418.055182072,3901464.2961673797,3995037.4085116317,4090162.213101829,4186863.929846355],[-564967.1007951547,-546328.8494337691,-527365.8668336506,-508072.75388078054,-488444.0234180627,-468474.09882556414,-448157.3125783289,-427487.9047810859,-406460.0216796007,-385067.71414788696,-363304.93615190336,-341165.54318802315,-318643.290696898,-295731.8324518716,-272424.7189217708,-248715.395607105,-224597.20135047846,-200063.36661925004,-175107.01176113728,-149721.1452318721,-123898.66179467714,-97632.3406905036,-70914.84378000256,-43738.71365486877,-16096.371719555696,12019.883757756092,40617.87962520216,69705.56985932356,99291.03760834504,129382.49726897618,159988.29659587052,191116.9188448575,222776.98495020554,254977.25573718036,287726.6341687951,321034.1676294897,354909.0502446089,389360.62523705745,424398.3873213469,460031.98513638205,496271.2237158902,533126.0669993486,570606.6403823523,608723.2333077379,647486.301897839,686906.4716293388,726994.5400494239,767761.4795364933,809218.4401041674,851376.7522501005,894247.9298500095,937843.6730984447,982175.871495012,1027256.6068795004,1073098.1565146246,1119712.9962179868,1167113.8035436566,1215313.4610152512,1264325.0594087895,1314161.9010894082,1364837.5034003197,1416365.6021058734,1468760.1548891454,1522035.3449060882,1576205.5843943588,1631285.5183413643,1687290.0282096711,1744234.2357218626,1802133.5067053034,1861003.4549990369,1920859.946420802,1981719.102799008,2043597.3060678425,2106511.202427589,2170477.706570869,2235514.005977015,2301637.565272588,2368866.1306632375,2437217.734434856,2506710.6995264683,2577363.6441754457,2649195.4866376435,2722225.4499801425,2796473.066952386,2871958.1849334715,2948700.9709578836,3026721.916822344,3106041.8442710433,3186681.9102657335,3268663.612338158,3352008.7940276195,3436739.650404495,3522878.733682814,3610448.958918933,3699473.6098035006,3789976.344543752,3881981.2018393883,3975512.606952854,4070595.377877417,4167254.731599856],[-581842.3842012933,-563224.9413528143,-544282.9120453092,-525010.8983705764,-505403.41438844043,-485454.8847070816,-465159.64304097043,-444511.9307456985,-423505.8953294193,-402135.5889401918,-380394.9668297777,-358277.8857922333,-335778.1025778912,-312889.2722818919,-289604.9467070815,-265918.5727003489,-241823.4904631537,-217312.93183429958,-192380.0185456709,-167017.76045001787,-141219.05372054433,-114976.67902129097,-88283.2996492181,-61131.45964569552,-33513.58187834127,-5421.966092121089,23151.213070553727,52213.9080817888,81774.20057133306,111840.30340325949,142420.5627852711,173523.46041179937,205157.61564111803,237331.78770775744,270054.87796911807,303335.9321889868,337184.1428568866,371608.8515445611,406619.5512998793,442225.88907947065,478437.668218995,515264.85094390786,552717.5609196881,590806.0858427833,629540.8800727122,668932.5673067479,708991.9432959191,749729.9786055472,791157.8214190688,833286.8003867115,876128.427519321,919694.4011289801,963996.6088150381,1009047.1304990731,1054858.2415074566,1101442.4157031176,1148812.3286669597,1196980.8609307506,1245961.1012597838,1295766.3499894093,1346410.1224138057,1397906.152228843,1450268.3950294852,1503511.0318637297,1557648.472841256,1612695.3608012598,1668666.575037652,1725577.2350837323,1783442.7045567376,1842278.595064507,1902100.770172336,1962925.3494346933,2024768.7124901363,2087647.5032214127,2151578.6339814644,2216579.2898875945,2282666.9331817618,2349859.307662194,2418174.4431843436,2487630.660233515,2558246.5745698647,2630041.1019482967,2703033.462910964,2777243.187658166,2852690.1209953865,2929394.4273588015,3007376.5959218657,3086657.4457802866,3167258.13122188,3249200.1470787292,3332505.3341645026,3417195.8847976965,3503294.34841396,3590823.6372644906,3679807.0322077456,3770268.1885914393,3862231.142228165,3955720.3154653884,4050760.5233532405,4147376.979906932],[-598952.9774535086,-580356.5772349538,-561435.7370786654,-542185.0602987813,-522599.06218872836,-502672.1686016824,-482398.7145086017,-461772.94253316964,-440789.00146335457,-419440.94473886583,-397722.72891507065,-375628.2121017012,-353151.1523769554,-330285.2061761413,-307023.9266547144,-283360.7620247153,-259289.05386544135,-234802.0354063243,-209892.82978281425,-184554.4482642773,-158779.7884537161,-132561.6324582845,-105892.64503151062,-78765.37168489536,-51172.2367699102,-23105.541529222857,5442.537883074256,34479.9524132777,64014.78215106507,94055.23840603512,124609.66581686353,155686.54449427035,187294.49219797808,219442.26654897933,252138.76727600605,285393.0384988964,319214.2710477803,353611.80481943465,388595.1311710016,424173.8953525035,460357.89897691924,497157.10253081727,534581.6279244018,572641.7610823051,611347.9545755363,650710.8302959732,690741.1821722123,731449.9789299285,772848.3668955348,814947.6728446789,857759.4068958932,901295.2654510424,945567.1341811586,990587.0910612561,1036367.4094526768,1082920.5612346856,1130259.2199857126,1178396.2642160284,1227344.780650237,1277118.0675636106,1327729.6381706567,1379193.2240677965,1431522.778730561,1484732.4810673394,1538836.739027815,1593850.1932705988,1649787.7208882289,1706664.4391916224,1764495.7095544206,1823297.1413194467,1883084.5957653269,1943874.1901380096,2005682.3017454455,2068525.57211748,2132420.9112316174,2197385.5018069814,2263436.8036643527,2330592.5581575856,2398870.7926743,2468289.8252082807,2538868.2690042527,2610625.037277515,2683579.348006211,2757750.7288019694,2833159.021856715,2909824.388967891,2987767.3166448213,3067008.6212934116,3147569.454485755,3229471.30831202,3312736.020817505,3397385.7815256286,3483443.137049952,3570930.996792337,3659872.6387343523,3750291.7153190114,3842212.2594261,3935658.6904418976,4030655.8204266676,4127228.860376781],[-616299.9285763283,-597724.7892814704,-578825.358093174,-559596.2395615651,-540031.9502274714,-520126.9172029947,-499875.47672965075,-479271.8727134436,-458310.25523652066,-436984.6790447603,-415289.10201179585,-393217.3835778523,-370763.2831639529,-347920.4585607124,-324682.4642914601,-301042.749948832,-276994.6585055506,-252531.4245974454,-227646.1727794467,-202331.9157536202,-176581.55256902333,-150387.86679234775,-123743.52465026709,-96641.07314119069,-69072.93811737979,-41031.42233631085,-12508.703481059521,16503.16785139474,46012.270192997996,76026.81328103668,106555.14016713924,137605.72936009173,169187.19700266374,201308.29908371344,233977.9336845132,267205.14326195675,300999.1169676066,335369.1930038673,370324.86101757595,405875.76453233464,442031.70341843995,478802.6364033518,516198.6836215765,554230.1292052853,592907.4239161054,632241.187819438,672242.2130001397,712921.4663227131,754290.0922347887,796359.4156154369,839140.9446686588,882646.3738636384,926887.5869204137,971876.6598444753,1017625.8640089624,1064147.669286059,1111454.7472280343,1159559.9742997428,1208476.4351609217,1258217.4260023143,1308796.4579340676,1360227.2604281872,1412523.7848155473,1465700.2078394261,1519770.9352637385,1574750.6055404744,1630654.0935344675,1687496.5143076514,1745293.226963176,1804059.8385516724,1863812.2080376307,1924566.4503307075,1986338.9403801933,2049146.3173346578,2113005.48876751,2177933.634970704,2243948.2133145696,2311066.9626789754,2379307.907953782,2448689.364611022,2519229.943349411,2590948.5548137375,2663864.4143869085,2737997.047060314,2813366.2923803907,2889992.3094736105,2967895.5821525482,3047096.9241003576,3127617.484140082,3209478.751586305,3292702.561681913,3377311.101120826,3463326.913659743,3550772.9058160186,3639672.352658791,3730048.9036904443,3821926.588821637,3915329.8244407335,4010283.419581015,4106812.582182505],[-633888.1132509708,-615334.4672925504,-596456.6791761698,-577249.3547036961,-557707.0116792893,-537824.0784900603,-517594.8926643566,-497013.6994069237,-476074.65011070506,-454771.8008445273,-433099.11081725475,-411050.44081674586,-388619.55162418354,-365800.10240298137,-342585.6490620503,-318969.64259250555,-294945.4273785667,-270506.2394807243,-245645.20489185164,-220355.33776537539,-194629.53861526633,-168460.59248681087,-141841.16709909774,-114763.81095690769,-87220.95143297082,-59204.89281946933,-30707.814348554006,-1721.768180812709,27761.322637449717,57749.66625043377,88251.60410174611,119275.61307703634,150830.30768067949,182924.44224774162,215566.9131901299,248766.7612796626,282533.17396695353,316875.487737424,351803.19050474325,387325.92404299043,423453.4864564203,460195.83468977246,497563.0870779855,535565.5259366494,574213.6001936169,613517.9280631694,653489.2997615021,694138.680266751,735477.2121223004,777516.2182849084,820267.2050180233,863741.8648318409,907952.0794688072,952909.9229380165,998627.664597238,1045117.77228409,1092392.915496916,1140465.9686270824,1189350.014241071,1239058.3464164112,1289604.4741298798,1341002.124699742,1393265.247282573,1446408.0164265651,1500444.8356795784,1555390.3412563298,1611259.4057629942,1668067.1419812043,1725828.9067119826,1784560.3046817519,1844277.1925085126,1904995.6827329292,1966732.1479125533,2029503.2247812536,2093325.8184745286,2158217.106822999,2224194.5447119833,2291275.8685124246,2359479.100581115,2428822.5538326087,2499324.8363835094,2571004.856271596,2643881.8262476167,2717975.2686454044,2793305.0203281846,2869891.2377132885,2947754.4018779774,3026915.3237436395,3107395.1493448415,3189215.365180698,3272397.803651368,3356964.6485805004,3442938.440826714,3530342.0839811713,3619198.8501584455,3709532.385877662,3801366.7180372654,3894726.2599841426,3989635.817680528,4086120.59596555],[-651721.0140866702,-633189.0965990605,-614333.1884008868,-595147.8965617301,-575627.7401652063,-555767.1488897507,-535560.4615670124,-515001.9247171404,-494085.69106069347,-472805.8180064601,-451156.26611572574,-429130.8975413486,-406723.47444223124,-383927.65737232775,-360737.0036440622,-337144.9656651474,-313144.8892496247,-288730.011901153,-263893.4610692584,-238628.25237764,-212927.28782429057,-186783.3539524076,-160189.11999301706,-133137.13597700885,-105619.83081751992,-77629.51036159601,-49158.35541084129,-20198.41971003497,9258.372095440282,39220.22653567302,69695.48342497298,100692.61800436443,132220.24311810895,164287.1114255092,196902.11764691328,230074.3008466349,263812.8467516531,298127.0901074931,333026.5170714748,368520.76764473435,404619.6381418088,441333.08370078565,478671.2208328452,516644.33001253544,555262.8583092245,594537.4220610745,634478.809590366,675097.9839633307,716406.0857932968,758414.4360886116,801134.5391457919,844578.0854893855,888756.9548572875,933683.2192349497,979369.145937206,1025827.2007392768,1073070.0510574216,1121110.569181052,1169961.8355546244,1219637.1421133673,1270149.9956712776,1321514.1213631534,1373743.4661412106,1426852.2023281776,1480854.731225125,1535765.6867784627,1591599.9393042903,1648372.5992722062,1706099.021149002,1764794.807304452,1824475.811977273,1885158.1453059572,1946858.1774227936,2009592.5426130432,2073378.1435400262,2138232.155538371,2204172.030973348,2271215.503671554,2339380.5934208906,2408685.610542232,2479149.1605334436,2550790.148788282,2623627.7853879053,2697681.589970752,2772971.3966785534,2849517.359180831,2927339.9557804004,3006459.994597337,3086898.6188377324,3168677.3121447936,3251817.9040350644,3336342.575420586,3422273.864220082,3509634.671056264,3598448.265046383,3688738.2896831003,3780528.7688089404,3873844.112685125,3968709.1241581608,4065149.0049210954],[-669798.9086344328,-651288.9317247005,-632455.1169515526,-613292.0726641765,-593794.3192376537,-573956.2876538755,-553772.3180600409,-533236.6583040659,-512343.4624466221,-491086.78924907,-469460.6006378485,-447458.760143687,-425075.0313161856,-402303.07611296535,-379136.45326321013,-355568.6166046022,-331592.9133945191,-307202.58259343053,-282390.75312129664,-257150.44208600116,-231474.55298361438,-205355.87386944052,-178787.07550077583,-151760.70944909472,-124269.20618256275,-96304.87311785715,-67859.89264094969,-38926.32009587437,-9496.081742308335,20439.02732036612,50887.345261646435,81857.34566224646,113357.63969048299,145396.97831391264,177984.25454517687,211128.5057247593,244838.9158395403,279124.8178784943,313995.6962257917,349461.1890926403,385531.090986717,422215.35522212577,459524.0964687932,497467.59334255755,536056.2910364387,575300.8039944512,615211.9186267424,655800.596069244,697077.9749866317,739055.374420075,781744.2966801862,825156.4302867074,869303.6529536233,914198.0346231994,959851.8405475868,1006277.5344196477,1053487.7815534081,1101495.4521159618,1150313.624409171,1199955.5882051922,1250434.84813424,1301765.1271264483,1353960.3699082437,1407034.7465552536,1461002.6560998985,1515878.7301981824,1571677.836853822,1628415.0842018332,1686105.8243520036,1744765.657294446,1804410.43486534,1865056.2647775305,1926719.514714305,1989416.81648835,2053165.0702665797,2117981.4488631627,2183883.402098591,2250888.661230143,2319015.243451584,2388281.456464608,2458705.903122585,2530307.486149211,2603105.4129297533,2677119.200380677,2752368.67989538,2828874.002368429,2906655.6433007927,2985734.4079835294,3066131.436766271,3147868.210408041,3230966.5555131948,3315448.6500532907,3401337.0289779836,3488654.5899120593,3577424.598945665,3667670.6965149,3759416.9033759497,3852687.6266736407,3947507.666107705,4043902.22019372],[-688129.5579483795,-669641.7695661805,-650830.2980158698,-631689.7529438374,-612214.656034432,-592399.4395909696,-572238.4450943752,-551725.9217387065,-530856.0249433103,-509622.81484088604,-488020.2547419998,-466042.20957442047,-443682.4442978373,-420934.6222931256,-397792.3037260263,-374248.9438842216,-350297.89148864965,-325932.3869770486,-301145.56076048873,-275930.43145194114,-250279.90406667534,-224186.7681934603,-197643.69613745832,-170643.241032538,-143177.83492397354,-115239.78682036093,-86821.28071458917,-57914.373572738376,-28510.99329181481,1397.0633760292549,31818.132931932807,62760.68727250816,94233.33586605662,126244.82796406024,158804.05484685977,191920.05210625636,225602.00196388457,259859.23562676413,294701.2356802253,330137.6385195777,366178.23681936716,402832.98204318737,440111.9869928616,478025.52839838155,516584.0495489973,555798.1629668348,595678.6531218644,636236.4791913745,677482.7778627218,719428.8661809138,762086.2444413642,805466.5991293823,849581.8059051028,894443.932637299,940065.2424848394,986458.1970272625,1033635.4594450577,1081609.8977513611,1130394.5880734385,1180002.8179880055,1230448.0899088015,1281744.1245282046,1333904.8643134143,1386944.4770591287,1440877.359494931,1495718.1409518395,1551481.6870862325,1608183.1036631921,1665837.7403997516,1724461.194870226,1784069.3164717057,1844678.2104544293,1906304.2420153222,1968964.0404566994,2032674.503410872,2097452.8011329,2163316.380859437,2230282.9712389275,2298370.586831056,2367597.532677932,2437982.408947581,2509544.1156523274,2582301.8574397797,2656275.148462175,2731483.817321863,2807948.0120952623,2885688.2054378036,2964725.1997673544,3045080.13253241,3126774.4815626377,3209830.0705045094,3294269.074342904,3380114.02501169,3467387.817090466,3556113.713594496,3646315.351854984,3738016.749492924,3831242.3104873165,3926016.831341134,4022365.5073419455],[-706712.1882695588,-688246.8042495373,-669457.8931749245,-650340.0660013297,-630887.8457344872,-611095.6660113973,-590957.8696590778,-570468.7072302194,-549622.33551546,-528412.8160315701,-506834.1134861107,-484880.094216882,-462544.52460676595,-439821.0694731502,-416703.2904317032,-393184.64423361956,-369258.4810770347,-344918.04289073823,-320156.46159081464,-294966.7573093772,-269341.8365950859,-243274.4905845034,-216757.3931451235,-189783.09898784105,-162344.04174977867,-134432.53204637463,-106040.75549248606,-77160.77069144393,-47784.50719294627,-17903.763417418813,12489.795452250633,43404.63961274875,74849.37681638217,106832.7545824193,139363.66244260315,172451.134223575,206104.35036510578,240332.64027545997,275145.4847241577,310552.51827347535,346563.5317475314,383188.4747419094,420437.45817268034,458320.75686615356,496848.81218978553,536032.2347256201,575881.8069850716,616408.4861682002,657623.4069663072,699537.8844092858,742163.4167582125,785511.688444634,829594.5730552941,874424.1363657713,920012.6394216996,966372.5416691785,1013516.5041348264,1061457.3926572714,1110208.2811684366,1159782.4550286445,1210193.414413966,1261454.8777576392,1313580.7852460255,1366585.3023710619,1420482.823537436,1475287.9757289046,1531015.622231989,1587680.8664191016,1645299.0555915476,1703885.7848846319,1763456.9012329096,1824028.5074003262,1885616.9660735037,1948238.904020214,2011911.2163137305,2076651.0706253285,2142475.9115828853,2209403.4652008144,2277451.7433792725,2346639.048475067,2416983.9779448938,2488505.4290634603,2561222.6037141844,2635155.0132582877,2710322.483479989,2786745.159610204,2864443.5114311804,2943438.3384596696,3023750.7752148174,3105402.296568407,3188414.723180235,3272810.227019382,3358611.336974515,3445840.9445503345,3534522.309657184,3624679.0664910404,3716335.2295070305,3809515.199487356,3904243.7697069207,4000546.1321936515],[-725552.298412011,-707109.5517448277,-688343.4357613699,-669248.5627410809,-649819.4570258384,-630050.5536012431,-609936.1966554702,-589470.6381150591,-568648.0361573151,-547462.4536986265,-525907.8568592358,-503978.11340283393,-481666.9911515303,-458968.1563753977,-435875.17215640564,-412381.49672577064,-388480.48177554016,-364165.3707424151,-339429.29706455045,-314265.2824104198,-288666.2348794821,-262624.9471736718,-236134.09474059357,-209186.23388611246,-181773.79985734797,-153889.10489490815,-125524.33625414199,-96671.5541943456,-67322.68993682112,-37469.543589381734,-7103.782038252102,23783.0631938132,55199.598123416305,87154.56851767446,119656.86213960289,152715.5110303387,186339.69382722303,220538.73811898823,255322.12283834978,290699.480693331,326680.60063617793,363275.4303727944,400494.07891160995,438346.81915310724,476844.0905205924,515996.5016334285,555814.833021625,596310.0398849461,637493.254895292,679375.7910438753,721969.1445336062,765284.9977181859,809335.2220866398,854131.8812967457,899687.2342560582,946013.7382521066,993124.0521322491,1041031.0395349495,1089747.7721708482,1139287.5331576562,1189663.8204072965,1240890.3500670826,1292981.060015472,1345950.1134142985,1399811.9023156932,1454581.0513282153,1510272.4213402893,1566901.113303105,1624482.4720733915,1683032.090318276,1742565.812480302,1803099.7388073257,1864650.229445559,1927233.908597779,1990867.6687474563,2055568.6749509815,2121354.369196017,2188242.4748311774,2256251.0010649646,2325398.2475364152,2395702.8089580843,2467183.57983388,2539859.7592494977,2613750.855741252,2688876.6922409637,2765257.4110994036,2842913.479190631,2921865.693094886,3002135.1843661824,3083743.4248822606,3166712.2322796774,3251063.7754747523,3336820.5802735966,3424005.535068237,3512641.8966259225,3602753.295968797,3694363.744347109,3787497.639306809,3882179.7708548666,3978435.3277192484],[-744652.909656686,-726233.0304769232,-707489.9412918178,-688418.2557185121,-669012.4994490632,-649267.1088318164,-629176.4294304154,-608734.7145596968,-587936.1237982346,-566774.7214767674,-545244.4751431283,-523339.2540019385,-501052.827329732,-478378.8628646317,-455310.9251704116,-431842.4739739862,-407966.8624771419,-383677.33564048726,-358967.028440404,-333828.9640980428,-308256.0522801562,-282241.08727071877,-255776.74611425493,-228855.58672861988,-201470.04598810477,-173612.43777584075,-145274.95100520225,-116449.64760917309,-87128.46049854928,-57303.19148661685,-26965.509181201458,3893.0531570347957,35281.099790334934,67207.374714843,99680.76390578668,132710.29759957618,166305.15261167916,200474.65469166706,235228.28091560001,270575.66211721,306526.5853566055,343090.9964295586,380279.00241614785,418100.8742701607,456567.0494496261,495688.1345898914,535474.9082180089,575938.3235116522,617089.5111013174,658939.7819172884,701500.630081838,744783.7358481099,788800.9685844593,833564.3898076748,879086.2562637916,925379.0230580973,972455.3468347504,1020328.0890078736,1069010.319042387,1118515.3177886978,1168856.5808696141,1220047.822121319,1272102.9770888782,1325036.2065782573,1378861.9002630203,1433594.6803502152,1489249.4053035902,1545841.1736262538,1603385.3277032045,1661897.4577059592,1721393.405557326,1781889.268961044,1843401.4054945954,1905946.4367671446,1969541.2526434287,2034203.0155357295,2099949.1647619377,2166797.420975012,2234765.7906616195,2303872.5707125315,2374136.3530653245,2445576.0294219553,2518210.79603891,2592060.158595761,2667143.937139764,2743482.2711090394,2821095.624436589,2900004.790732932,2980230.898553376,3061795.4167476925,3144720.1598948906,3229027.2938238806,3314739.3412231673,3401879.1873366963,3490470.085752819,3580535.6642836714,3672099.9309380474,3765187.2799886833,3859822.4981372426,3956030.7707739654],[-764016.7932013292,-745620.0064047802,-726900.1704032664,-707851.9001643927,-688469.7227432508,-668748.0758639411,-648681.3064787013,-628263.6693039299,-607489.3253328521,-586352.3403240915,-564846.6832667142,-542966.2248200763,-520704.73572906107,-498055.88521391223,-475013.239334411,-451570.2593275318,-427720.29991928395,-403456.6076088189,-378772.3189255323,-353660.45865819464,-328113.9380559197,-302125.5529999519,-275687.982147136,-248793.78504284122,-221435.40020423126,-193605.14317281428,-165295.20453600027,-136497.64791662712,-107204.40793132759,-77407.28811536194,-47097.95881482423,-16267.955045108916,15091.325684661046,46988.625582468,79432.82881841552,112432.96380665526,145998.20552308555,180137.87786022061,214861.45601941808,250178.56894185394,286099.0017770801,322632.6983921046,359789.7639198662,397580.4673484387,436015.24415138736,475104.69896064955,514859.6082807286,555290.9232474328,596409.772429853,638227.4646771592,680755.4920105767,724005.5325620859,767989.4535585532,812719.3143547352,858207.369513913,904466.0719376446,951508.0760452158,999346.2410044856,1047993.6340125201,1097463.5336300563,1147769.4331682003,1198925.044129164,1250944.2997015615,1303841.358312184,1357630.6072324894,1412326.666244225,1467944.391362432,1524498.8786178506,1582005.4678992205,1640479.7468576604,1699937.5548711913,1760394.9870741447,1821868.3984496882,1884374.4079875536,1947929.902907637,2012552.0429517287,2078258.2647413178,2145066.2862067632,2212994.1110856994,2282060.0334931365,2352282.6425638963,2423680.8271699194,2496273.780710113,2570081.0059786523,2645122.3201092733,2721417.8595981766,2798988.0854077414,2877853.788148884,2958036.093348018,3039556.466796433,3122436.719984764,3206699.0156233464,3292365.8732516114,3379460.174933594,3468005.171046613,3558024.4861603007,3649542.125009165,3742582.478559557,3837170.330174297,3933330.8618720267],[-783649.0599519501,-765275.6064175054,-746579.2661923276,-727554.6556105037,-708196.3031063668,-688498.6477961617,-668456.0380372673,-648062.7299643799,-627312.8860023043,-606200.5733546463,-584719.7624689993,-562864.3254769274,-540628.0346093457,-518004.5605864648,-494987.47098214785,-471570.228561671,-447746.18959372607,-423508.60213467595,-398850.60428578756,-373765.2224225139,-348245.3693956204,-322283.8427031059,-295873.32263384666,-269006.37038065004,-241675.42612369382,-213872.8070832251,-185590.70554128918,-156821.1868313963,-127556.18729706272,-97787.51221679989,-67506.83369649295,-36705.688528030645,-5375.476013893727,26492.544243491488,58907.25458662212,91877.68158463365,125412.99835087522,159522.5268975529,194215.74052780634,229502.26626649336,265391.8873285621,301894.5456279537,339020.3443258833,376779.5504198638,415182.5973738618,454240.08779098885,493962.7961275114,534361.6714513693,575447.8402439756,617232.6092468067,659727.4683531662,702944.0935467023,746894.3498852919,791590.2945338837,837044.1798448749,883268.456487692,930275.7766279813,978078.9971582762,1026691.1829783847,1076125.6103296392,1126395.7701813644,1177515.371671395,1229498.3456011298,1282358.8479870888,1336111.2636671462,1390770.209965941,1446350.5404176158,1502867.3485479974,1560335.9717166466,1618771.995020986,1678191.2552605774,1738609.8449662724,1800044.116492482,1862510.6861746619,1926026.4385526204,1990608.5306620058,2056274.3963918532,2123041.7509134533,2190928.595178514,2259953.220488973,2330134.2131391615,2401490.4591328185,2474041.1489726626,2547805.7825284144,2622804.173980776,2699056.4568440625,2776583.089069612,2855404.8582278327,2935542.886774885,3017018.637401728,3099853.918468294,3184070.8895235322,3269692.0669144765,3356740.329481449,3445238.9243463967,3535211.472791614,3626681.976231966,3719674.82228153,3814214.7909178655,3910327.0607410534],[-803551.4722986151,-785201.5775421298,-766528.960094482,-747528.2376690783,-728193.9400924685,-708520.5078861215,-688502.2908257989,-668133.5464778615,-647408.4387121825,-626321.0361909957,-604865.3108341964,-583035.1362594808,-560824.2861978351,-538226.4328836652,-515235.1454192416,-491843.888112647,-468046.01878990466,-443834.78707937384,-419203.3326691361,-394144.68353643245,-368651.75414892496,-342717.3436367931,-316334.1339364969,-289494.6879040259,-262191.44739845674,-234416.73133484018,-206162.73370606918,-177421.5215727263,-148185.0330217632,-118445.0750916833,-88193.32166506792,-57421.31132737687,-26120.445191715844,5718.015311653726,38102.95067997929,71043.3856196052,104548.4913633191,138627.58802484744,173290.14699074207,208545.79335098062,244404.30836715433,280875.63198118773,317969.8653634223,355697.2735014539,394068.2878300969,433093.5089038806,472783.7091108544,513149.83543091966,554203.0122374156,595954.5441435203,638415.9188937931,681598.8103024887,725515.0812372472,770176.7866516991,815596.1766656493,861785.6996944214,908758.0056278608,956525.9490607376,1005102.5925729482,1054501.2100635143,1104735.2901368407,1155818.5395429889,1207764.8866725047,1260588.4851077348,1314303.717228829,1368925.1978789,1424467.7780865305,1480946.5488476967,1538376.84496758,1596774.2489644326,1656154.5950336116,1716533.973076465,1777928.7327923556,1840355.4878358562,1903831.1200398314,1968372.783706626,2033997.9099653475,2100724.211200481,2168569.6855497533,2237552.621473672,2307691.6023974083,2379005.5114275375,2451513.536141295,2525235.173454341,2600190.2345644156,2676398.8499736954,2753881.4745918936,2832658.892917997,2912752.2243066556,2994182.9283169126,3076972.810146062,3161144.026149371,3246719.0894488394,3333720.8756280765,3422172.628520308,3512097.966086761,3603520.886388571,3696465.773653065,3790957.4044377077,3887020.953888782],[-823729.4967031209,-805403.4021519365,-786754.7505955045,-767778.1611420632,-748468.1650260523,-728819.2041900218,-708825.629844131,-688481.701002585,-667781.5829966976,-646719.3459638646,-625288.9633130049,-603484.3101648269,-581299.1617674893,-558727.1918868308,-535761.9711710035,-512396.9654885207,-488625.53424056084,-464440.9286455093,-439836.28999647824,-414804.64789090236,-389338.91843197285,-363431.9024008459,-337076.2834006024,-310264.6259696279,-282989.3736653479,-255242.8471172601,-227017.24204897066,-198304.62726821075,-169096.94262567908,-139385.99694038485,-109163.4658923531,-78420.88988159248,-47149.671853031265,-15341.075086218305,17013.779049190925,49923.91337482142,83398.49722075579,117446.84877984412,152078.43749940395,187302.88651164365,223129.97510165325,259569.64121592813,296631.98401028384,334327.2664384828,372665.91788199334,411658.53682230366,451315.8935545306,491648.9329455297,532668.7772352917,574386.7288831284,616814.2734590156,659963.0825816891,703845.0169021178,748472.129135903,793856.667143238,840011.0770580356,886948.0064666988,934680.3076383197,983221.0408046255,1032583.4774937774,1082781.1039163684,1133827.624405488,1185736.9649112932,1238523.2765520886,1292200.939220103,1346784.5652463818,1402289.0031230608,1458729.3412850308,1516120.9119515126,1574479.295029644,1633820.3220782727,1694160.0803365814,1755514.9168158402,1817901.4424563367,1881336.536350194,1945837.3500322797,2011421.311837228,2078106.131327752,2145909.8037922545,2214850.6148140603,2284947.1449130285,2356218.2742619882,2428683.187475727,2502361.3784784144,2577272.6554470123,2653437.1458332893,2730875.30146663,2809607.903735461,2889656.0688533243,2971041.253207302,3053785.2587915487,3137910.2387266853,3223438.70286824,3310393.523501197,3398797.94112766,3488675.570344925,3580050.4058170104,3672946.828340626,3767389.6110087684,3863403.9254690306],[-844185.6162641598,-825883.5532345906,-807259.1004152867,-788306.8783226194,-769021.4196123545,-749397.1676616834,-729428.4751288756,-709109.6024898372,-688434.7165513062,-667397.8889399767,-645993.0945680633,-624214.2100737256,-602055.0122368569,-579509.1763694636,-556570.2746804387,-533231.7746137662,-509487.03716096305,-485329.3151457738,-460751.7514818397,-435747.3774024602,-410309.11066215555,-384429.75370906014,-358101.9918290174,-331318.3912591296,-304071.3972716576,-276353.3322272047,-248156.3935969225,-219472.65195269813,-190294.0489261623,-160612.39513419103,-130419.3680718008,-99706.50997127709,-68465.22562729195,-36686.78018677258,-4362.29690457345,28517.24513772549,61961.013348824345,95978.32398182177,130578.64452573331,165771.59613571642,201566.95610088645,237974.66035264288,275004.8060123641,312667.65397988143,350973.63156301226,389933.3351496749,429557.53292130167,469857.16761074006,510843.35930344835,552527.408283452,594920.7979245123,638035.1976279633,681882.4658060097,726474.6529138759,771824.0045295712,817942.9644827717,864844.1780333887,912540.4951015068,961044.973547142,1010370.8825037745,1060531.7057641256,1111541.1452199803,1163413.124356519,1216161.7918031686,1269801.5249391012,1324346.933557916,1379812.863589636,1436214.4008821337,1493566.8750424404,1551885.8633400844,1611187.1946706176,1671486.9535839753,1732801.4843759625,1795147.3952449174,1858541.5625142492,1923001.134923085,1988543.5379829947,2055186.4784060381,2122947.9486020864,2191846.2312477664,2261899.9039277732,2333127.8438510327,2405549.2326393668,2479183.5611946457,2554050.6346418737,2630170.5773509005,2707563.8380389395,2786251.194951639,2866253.7611288936,2947592.9897529054,3030290.679581406,3114368.9804667407,3199850.398963952,3286757.80402502,3375114.432786168,3464943.896445597,3556270.1862346944,3649117.679483626,3743511.1457845853,3839475.75324977],[-864923.9399816503,-846646.1431362136,-828046.1252741065,-809118.5083334422,-789857.8264050487,-770258.5243146596,-750314.9561826827,-730021.3839608927,-709371.9759457794,-688360.8052677955,-666981.848357076,-645228.9833839973,-623095.9886751012,-600576.5411036378,-577664.2144544735,-554352.4777624495,-530634.6936249633,-506504.11648683064,-481953.8908980957,-456977.0497439485,-431566.51244645147,-405715.0831370803,-379415.44880098803,-352660.17739070975,-325441.7159102205,-297752.38846829673,-269584.3943008871,-240929.80576145486,-211780.56628015917,-182128.48828954995,-151965.25111760618,-121282.39884705492,-90071.33814066835,-58323.33603130677,-26029.517677770462,6819.1359162156,40231.79021882173,74217.7595240525,108786.50934306625,143947.65883422154,179710.9832706824,216086.4165485648,253084.05373444315,290714.15365359373,328987.14151935466,367913.61160503025,407504.3299570901,447770.2371528775,488722.45110158017,530372.2698899934,572731.1746734469,615810.8326134523,659623.0998607725,704180.0245873462,749493.8500658241,795577.0177982473,842442.1706943591,890102.1563013424,938570.0300833103,987859.0587546346,1037982.723665467,1088954.724241329,1140788.9814771959,1193499.6414880943,1247101.0791143826,1301607.9015861377,1357034.9522449384,1413397.3143250067,1470710.3147942717,1528989.5282574464,1588250.780919277,1648510.1546126585,1709783.9908898212,1772088.8951787446,1835441.7410054053,1899859.6742841569,1965360.1176741575,2031960.775007144,2099679.6357844616,2168534.9797457056,2238545.38150977,2309729.715290672,2382107.159685959,2455697.202543511,2530519.64590436,2606594.611024067,2683942.5434749303,2762584.218326755,2842540.7454123166,2923833.574675122,3006484.501602288,3090515.6727433074,3175949.5913177924,3262809.1229093643,3351117.501252643,3440898.334110614,3532175.6092454996,3624973.700483989,3719317.373880141,3815231.793972991],[-885949.3984839276,-867696.1123913539,-849120.7757306014,-830218.0118766703,-810982.3563707623,-791408.2555026175,-771490.0648704064,-751222.0479175798,-730598.3744463362,-709613.1191070271,-688260.2598640064,-666533.6764363435,-644427.1487138828,-621934.3551479569,-599048.871116407,-575764.1672621204,-552073.6078057594,-527970.4488307603,-503447.83654132276,-478498.805492471,-453116.2767919323,-427293.0562728485,-401021.83263820363,-374295.17557468545,-347105.5338369068,-319445.2333009457,-291306.474986871,-262681.33304926264,-233561.7527365666,-203939.5483169579,-173806.4009715633,-143153.85665395018,-111973.32391560916,-80256.071696172,-47993.22707945481,-15175.77301262482,18205.454010433983,52159.766302868025,86696.62737598037,121825.65436908719,157556.6205169293,193899.4576576131,230864.25877992297,268461.28061136603,306700.94624731666,345593.84782273206,385150.7492251231,425382.5888520675,466300.48241195874,507915.7257695608,550239.7978367149,593284.3635097635,637061.2766524004,681582.5831273757,726860.5238757986,772907.5380455842,819736.2661695541,867359.5533949011,915790.4527624841,965042.2285398925,1015128.3596067335,1066062.5428939727,1117858.6968777878,1170530.965129889,1224093.7199225705,1278561.5658928333,1333949.343763914,1390272.1341261622,1447545.26127784,1505784.2971279412,1565005.0651591602,1625223.6444557454,1686456.3737944127,1748719.855800468,1812030.961169789,1876406.8329589167,1941864.8909411943,2008422.836034249,2076098.6547967433,2144910.623996705,2214877.3152522794,2286017.599747264,2358350.653019191,2431895.9598258403,2506673.3190877293,2582702.8489091923,2660004.9916802463,2738600.519257068,2818510.538227103,2899756.4952565185,2982360.182522739,3066343.7432328765,3151729.6772311367,3238540.8466923665,3326800.4819086907,3416532.187166531,3507759.9467171277,3600508.1308413954,3694801.5020124842,3790665.2211530027],[-907263.0245561879,-889034.4698132764,-870484.036278298,-851606.3487734423,-832395.9443007101,-812847.2706243473,-792954.6848308889,-772712.4518661592,-752114.7430488649,-731155.6345601398,-709829.1059095629,-688129.0383759795,-666049.2134237527,-643583.3110935688,-620724.908367662,-597467.4775084734,-573804.384371537,-549728.8866906366,-525234.1323359509,-500313.1575442718,-474958.8851210396,-449164.1226132305,-422921.5604539213,-396223.7700763375,-369063.20199823193,-341432.1838755973,-313322.9185253545,-284727.48191604996,-255637.82112739142,-226045.7522762695,-195942.95841016364,-165320.98736682837,-134171.24959992152,-102485.01596943964,-70253.41549793328,-37467.43308984488,-4117.907215089537,29804.472444518236,64309.16738619562,99405.79271584423,135104.11961528147,171414.0778499588,208345.75831609126,245909.41562850494,284115.4707495528,322974.51366063347,362497.3060749639,402694.7841948634,443578.0615122956,485158.43165420834,527447.3712730156,570456.5429838062,614197.7983469716,658683.1808996731,703924.9292349298,749935.4801298319,796727.4717233977,844313.7467458108,892707.3557974868,941921.5606818721,991969.837790512,1042865.8815421266,1094623.607876232,1147257.1578031965,1200780.9010090479,1255209.4395193313,1310557.6114203832,1366840.4946399415,1424073.4107876993,1482271.929057823,1541451.8701916803,1601629.3105054,1662820.5859805276,1725042.2964198454,1788311.3096690993,1852644.7659067758,1918060.0819999785,1984574.9559315983,2052207.3712967788,2120975.6018709503,2190898.2162502734,2261994.0825668797,2334282.373276638,2407782.5700253583,2482514.4685909688,2558498.1839042753,2635754.155150506,2714303.1509494744,2794166.274620341,2875364.969528749,2957921.0245190226,3041856.5794322575,3127194.130713372,3213956.5371043384,3302167.0254304386,3391849.196476948,3483027.0309592863,3575724.8955875365,3669967.5492286095,3765780.1491631214],[-928873.9210559989,-910670.3629291764,-892145.0997091081,-873292.7576835706,-854107.8753339265,-834584.9019177218,-814718.1960288554,-794502.0241347214,-773930.5590899568,-752997.8786261166,-731697.9638178423,-710024.6975238444,-687971.8628032797,-665533.14130673,-642702.1116415635,-619472.2477107511,-595836.9170259221,-571789.3789926553,-547322.7831688032,-522430.16749485955,-497104.4564961635,-471338.45945594227,-445124.8685600583,-418456.25701120123,-391325.07711346564,-363723.65832620393,-335644.2052869119,-307078.7958020882,-278019.37880695215,-248457.77229164005,-218385.66119479365,-187794.59526342922,-156675.9868787583,-125021.10884678969,-92821.09215474222,-60066.9236905938,-26749.443926815176,7140.655432975851,41612.833844761364,76676.70435358421,112342.03606056212,148618.75663042488,185516.95483841887,223046.88315794803,261218.9603893049,300043.7743309494,339532.0844920804,379694.8248496973,420543.1066489364,462088.2212481778,504341.64300931245,547315.0322347065,591020.2381496336,635469.3019334809,680674.4597986243,726648.1461183969,773402.9966047104,820951.8515370479,869307.7590412996,918483.9784223023,968493.9835486594,1019351.4662915841,1071070.34001825,1123664.7431415915,1177149.0427248543,1231537.8381452067,1286845.9648147197,1343088.4979607342,1400280.7564661335,1458438.3067715946,1517576.966838026,1577712.81017387,1638862.1699254555,1701041.643032547,1764268.094449778,1828558.6614361359,1893930.7579105399,1960402.0788787296,2027990.6049294076,2096714.60680199,2166592.6500267303,2237643.5996396514,2309886.6249699956,2383341.2045061146,2458027.1308373325,2533964.51567437,2611173.79495057,2689675.734001711,2769491.432830465,2850642.3314531674,2933150.2153316843,3017037.220891156,3102325.841126703,3189038.931296291,3277199.714706652,3366831.788589578,3457959.1300717285,3550606.1022387776,3644797.4602971985,3740558.357830767],[-950780.3678173181,-932602.0230228234,-914102.148070681,-895275.3707231738,-876116.2309500355,-856619.1795111774,-836778.5765170171,-816588.6899657419,-796043.6942572142,-775137.6686827985,-753864.5958916601,-732218.3603318986,-710192.746667088,-687781.438167403,-664978.0150751187,-641775.9529436072,-618168.6209505177,-594149.2801832631,-569711.0818974744,-544847.0657475414,-519550.15798897995,-493813.1696516415,-467628.7946846187,-440989.60807062057,-413888.0639106999,-386316.4934782982,-358267.10324229184,-329731.9728580401,-300703.0531272574,-271172.1639244186,-241130.9920905172,-210571.08929309784,-179483.86985229375,-147860.60853159637,-115692.43829446868,-82970.34802406793,-49685.180207203375,-15827.628581149736,18611.764256884344,53642.60927899927,89274.67349346308,125517.88245199667,162382.32279650634,199878.24484649627,238016.06522760028,276806.36954264296,316259.91508400114,356387.6335904421,397200.6340472433,438710.2055310807,480927.82010005694,523865.1357304733,567533.9992989893,611946.4496136298,657114.7204924067,703051.2438910645,749768.6530804583,797279.7858753055,845597.6879127673,894735.6159847532,944707.0414225026,995525.6535351411,1047205.3631028095,1099760.3059261898,1153204.8464308134,1207553.5813303739,1262821.3433474754,1319023.204993729,1376174.4824097282,1434290.7392670503,1493387.7907304047,1553481.7074846625,1614588.819824919,1676725.7218117565,1739909.2754923962,1804156.6151898787,1869485.151858361,1935912.5775096891,2003456.869709251,2072136.296143415,2141969.419259354,2212975.1009796453,2285172.5074894344,2358581.114101982,2433220.710200216,2509111.4042568253,2586273.6289351564,2664728.1462687063,2744496.052925218,2825598.7855531373,2908058.1262131203,2991896.207895436,3077135.520126324,3163798.9146604915,3251909.6112666796,3341491.2036036164,3432567.6651894324,3525163.3554654405,3619303.025957555,3715011.8265324086],[-972991.244145399,-954838.3711446862,-936364.1447131959,-917563.1941041412,-898430.0607922318,-878959.1970565307,-859144.96454093,-838981.6327915585,-818463.37777084,-797584.2803474534,-776338.3247627916,-754719.3970722437,-732721.28356189,-710337.6691397657,-687562.135701546,-664388.1604696712,-640809.1143067069,-616818.2600009714,-592408.7505251735,-567573.6272670974,-542305.8182321531,-516598.13621673686,-490443.27695332374,-463833.81722499174,-436762.2129503535,-409220.7972377357,-381201.77840844006,-352697.2379879416,-323699.1286659646,-294199.2722230458,-264189.3574244953,-233660.93788061407,-202605.42987290258,-171014.1101450408,-138878.11365968618,-106188.43131839158,-72935.90764573682,-39111.238436394604,-4704.96836474305,30292.5114441514,65890.96588117117,102100.31836026022,138930.6533649303,176392.219035442,214495.42979716416,253250.86903138924,292669.2917874763,332761.627539495,373538.98298608745,415012.64489515685,457194.08299366385,500094.952904148,543727.0991266817,588102.5580696347,633233.5611280431,679132.5378111382,725812.1189194582,773285.1397733688,821564.6434914018,870663.8843222912,920596.3310293122,971375.6703285887,1023015.8103819452,1075530.8843461908,1128935.253977138,1183243.513292679,1238470.4922932438,1294631.2607416231,1351741.1320026838,1409815.666945062,1468870.677903071,1528922.232703406,1589986.6587549378,1652080.547203693,1715220.7571536857,1779424.4199558254,1844708.9435628974,1911092.0169558409,1978591.6146392939,2047226.001208717,2117013.7359898933,2187973.677753248,2260124.9895006376,2333487.143330629,2408079.925379672,2483923.4408419044,2561038.119069691,2639444.7187527707,2719164.3331820476,2800218.3955956534,2882628.6846101396,2966417.3297375157,3051606.816991245,3138219.9945783885,3226280.078684808,3315810.659350741,3406835.7064398485,3499379.575702615,3593467.014937399,3689123.170246142],[-995508.2969447011,-977381.1343262655,-958932.7965028303,-940157.9142309042,-921051.0305026396,-901606.5991288528,-881818.9832996235,-861682.4541218742,-841191.1891335482,-820339.2707937388,-799120.6849492881,-777529.3192762276,-755558.9616966178,-733203.2987699853,-710455.9140591389,-687310.2864694807,-663759.7885624976,-639797.6848415865,-615417.1300108198,-590611.1672058173,-565372.7261964623,-539694.6215604232,-513569.55082841986,-486990.09259892534,-459948.70462325914,-432437.72185996943,-404449.3544982404,-375975.6859492862,-347008.6708066156,-317540.1327727614,-287561.7625534469,-257065.1157179703,-226041.61052562157,-194482.52571685286,-162378.9982702483,-129722.02112267166,-96502.44085357012,-62710.95533224731,-28338.11132768262,6625.697920275386,42190.23516737204,78365.42167179985,115161.33974053292,152588.23531628354,190656.52060563676,229376.77674966957,268759.7565358388,308816.38715443667,349557.77299821004,390995.198506834,433140.1310564508,476004.2238959996,519599.3191288924,563937.4507435802,609030.8476916575,654891.9370151535,701533.3470234061,748967.9105213247,797208.6680874834,846268.8714059116,896161.9866501587,946901.6979213608,998501.9107408244,1050976.7555990377,1104340.5915594446,1158608.0099212276,1213793.8379395097,1269913.1426048838,1326981.2344828718,1385013.6716153286,1444026.2634820445,1504035.0750271948,1565056.4307488212,1627106.9188535265,1690203.3954770043,1754362.9889726462,1819603.1042662328,1885941.4272818854,1953395.9294373034,2021984.8722105855,2091726.811779417,2162640.6037350385,2234745.4078687523,2308060.693036838,2382606.2421014253,2458402.1569499355,2535468.8635953036,2613827.1173547744,2693498.008113351,2774502.965669504,2856863.765166021,2940602.5326066767,3025741.7504618727,3112304.2633604137,3200313.2838743473,3289792.398394174,3380765.5730974884,3473257.1600120375,3567291.903176352,3662894.9448951026],[-1018334.4162022129,-1000233.1919502117,-981810.9720526774,-963062.3887794233,-943981.9866501616,-924564.2210176433,-904803.4566284395,-884693.9661606222,-864229.9287381333,-843405.4284210273,-822214.4526722592,-800650.8907992393,-778708.5323708532,-756381.065609046,-733662.0757548022,-710545.0434076034,-687023.3428391044,-663090.2402790925,-638738.8921744318,-613962.3434201165,-588753.5255621285,-563105.2549711443,-537010.2309879691,-510461.03403841937,-483450.12371857837,-455969.8368493628,-428012.3855001095,-399569.85498016,-370634.20179928397,-341197.25159462076,-311250.6970250085,-280786.0956315864,-249794.8676643665,-218268.29387363326,-186197.51326707914,-153573.5208301698,-120387.16521063028,-86629.1463659408,-52290.01317333663,-17360.161001222674,18170.17075708788,54310.9011900085,91072.11041382654,128464.0421594074,166497.10640013823,205181.88202232216,244529.11953684734,284549.74383540684,325254.8569899071,366655.7410966973,408763.86116589047,451590.86805745214,495148.60146264406,539449.0929343388,584504.5689649102,630327.4541132529,676930.3741814201,724326.1594426446,772527.8479191614,821548.688713768,871402.1453936188,922101.8994280207,973661.8536807764,1026096.1359589165,1079419.1026162049,1133645.34221566,1188789.6792494627,1244867.1779182367,1301893.1459702002,1359883.1386022987,1418852.9624215257,1478818.6794710816,1539796.611319542,1601803.343215289,1664855.7283067228,1728970.891930567,1794166.2359662494,1860459.4432615698,1927868.4821276027,1996411.6109052016,2066107.38260388,2136974.6496154577,2209032.5685002306,2282300.6048515583,2356798.538236421,2432546.4672145033,2509564.8144380795,2587874.3318304955,2667496.1058492153,2748451.5628312323,2830762.4744235356,2914450.9630994163,2999539.5077637704,3086050.9494444863,3174008.497076921,3263435.7333787256,3354356.620818137,3446795.507676614,3540777.1342091192,3636326.6388990427],[-1041479.4071053674,-1023404.3973090368,-1005008.5734015311,-986286.5691846792,-967232.9307246568,-947842.1149352822,-928108.4881389346,-908026.3246043918,-887589.805061341,-866793.0151907874,-845629.9440919808,-824094.482724139,-802180.4223236195,-779881.4527956594,-757191.1610805229,-734103.0294931175,-710610.4340368316,-686706.6426896674,-662384.8136633479,-637637.9936345357,-612459.115947864,-586840.998789828,-560776.3433343761,-534257.7318579794,-507277.62582506356,-479828.3639427626,-451902.1601846658,-423491.10178262275,-394587.14718731283,-365182.12399540516,-335267.72684406955,-304835.51527177077,-273876.9115450233,-242383.19845000468,-210345.51704988233,-177754.86440537777,-144602.0912594609,-110877.89968504407,-76572.84069519071,-41677.31181474868,-6181.554614278022,29924.347796330694,66650.47331383312,104007.06342808483,142004.5258497768,180653.43718060758,219964.54562471807,259948.7737446865,300617.2212607162,341981.1678946186,384052.0762588992,426841.59479262726,470361.5607426078,514624.0031934888,559641.1461453675,605425.4116405924,651989.4229401373,699346.0077513666,747508.2015056107,796489.2506894483,846302.6162282652,896961.9769237633,948481.2329460424,1000874.5093820668,1054156.1598388935,1108340.7701059044,1163443.1618744694,1219478.3965168972,1276461.7789253453,1334408.8614126318,1393335.447673284,1453257.5968093807,1514191.6274194638,1576154.1217526197,1639161.9299284066,1703232.174224868,1768382.2534325798,1834629.8472800078,1901992.9209280931,1970489.7295364523,2040138.8229019018,2110959.0501717934,2182969.5646298626,2256189.82856046,2330639.618188776,2406339.02869959,2483308.4793367973,2561568.718581546,2641140.829414927,2722046.234663031,2804306.7024270524,2887944.351599222,2972981.6574677457,3059441.4574078233,3147346.9566657366,3236721.7342333044,3327589.748815766,3419975.3448940567,3513903.258884671,3609398.6253942004],[-1064941.8901700207,-1046893.3226665565,-1028524.1238782093,-1009828.9291481219,-990802.2860982994,-971438.6532130691,-951732.3984001409,-931677.7975285992,-911269.0329435656,-890500.1919567522,-869365.2653135487,-847858.1456349085,-825972.6258346704,-803702.3975114501,-781041.049314976,-757982.0652858368,-734518.8231695055,-710644.5927026316,-686352.5338723187,-661635.6951475048,-636487.0116821819,-610899.3034894441,-584865.2735872685,-558377.5061137488,-531428.4644127155,-504010.48908867245,-476115.79603072326,-447736.4744045576,-418864.4846132181,-389491.65622443706,-359609.68586534867,-329210.1350834798,-298284.42817367846,-266823.8499699519,-234819.54360297485,-202262.50822086865,-169143.5966740772,-135453.5131632944,-101182.81084986497,-66321.8894276265,-30860.992657064926,5209.794140875805,41900.54663052689,79221.50404775003,117183.07182744099,155795.82427347545,195070.50726992218,235018.04103681166,275649.5229290826,316976.2302803369,359009.6232916992,401761.34796741,445243.2390958159,489467.32327917404,534445.8220110419,580191.1548028355,626715.942359928,674033.0098091774,722155.3899762356,771096.3267165795,820869.2782987864,871487.9208418066,922966.1518067517,975318.0935451007,1028558.0969016468,1082700.7448764644,1137760.8563442682,1193753.4898331105,1250693.9473629738,1308597.778346315,1367480.7835488124,1427359.0191149092,1488248.8006564379,1550166.7074063653,1613129.58643847,1677154.5569549892,1742259.0146403923,1808460.6360863997,1875777.383286269,1944227.5082006496,2013829.5573957944,2084602.3767565647,2156565.1162719387,2229737.234898923,2304138.5055024032,2379789.0198735804,2456709.193829143,2534919.7723890278,2614441.8350388003,2695296.801074301,2777506.435031411,2861092.852201636,2946078.524236632,3032486.2848388758,3120339.335545361,3209661.251601687,3300475.987929534,3392807.88518856,3486681.675935851,3582122.490880082],[-1088728.285847835,-1070706.4064949253,-1052364.080202763,-1033695.943868111,-1014696.5466812487,-995360.3487095623,-975681.7194587742,-955654.9364110716,-935274.1835399235,-914533.5498007997,-893427.0275984018,-871948.5112287102,-850091.7952964681,-827850.5731072358,-805218.4350338739,-782188.866856484,-758755.2480765688,-734910.8502035129,-710648.8350140157,-685962.2527836713,-660844.0404903067,-635287.0199882763,-609283.896154355,-582827.255003195,-555909.5617731127,-528523.1589811917,-500660.26444742153,-472312.96928685624,-443473.2358705669,-414132.8957531885,-384283.64756778814,-353917.05488709686,-323024.5440505962,-291597.4019565778,-259626.7738198454,-227103.6608926966,-194018.91815000772,-160363.25193736586,-126127.2175816861,-91301.2169632488,-55875.49605009705,-19840.142392034642,16814.917424866464,54099.92036431143,92025.26956705842,130601.53702064045,169839.46627034806,209749.97517476277,250344.15870447643,291633.29178561317,333628.8321884363,376342.4234627206,419785.8979184679,463971.2796554975,508910.7876405823,554616.8388337209,601102.0513640293,648379.2477569771,696461.4582114513,745361.923930522,795094.1005044598,845671.6613477166,897108.501190465,949418.7396265268,1002616.7247160347,1056717.036647126,1111734.4914550097,1167684.144800385,1224581.2958077434,1282441.490965621,1341280.5280870427,1401114.460334783,1461959.6003096527,1523832.524204,1586750.0760210063,1650729.3718620762,1715787.804280274,1781943.0467050225,1849213.057936069,1917616.0867090155,1987170.6763332034,2057895.6694043805,2129810.2125898753,2202933.761492167,2277286.0855883947,2352887.273248439,2429757.7368337554,2507918.2178747733,2587389.792332903,2668193.875944847,2750352.229651982,2833886.9651155625,2918820.5503208954,3005175.815267615,3092975.957753007,3182244.549245682,3273005.540852703,3365283.26938105,3459102.4634966794,3554488.2499783062],[-1112845.797520616,-1094850.8763471916,-1076535.6944111516,-1057894.8901793757,-1038923.0144270832,-1019614.5288215836,-999963.804483624,-979965.1205257087,-959612.6625670451,-938900.5212244324,-917822.6905796428,-896373.0666216405,-874545.4456642072,-852333.5227381785,-829730.8899580899,-806731.0348622713,-783327.3387271939,-759513.0748540938,-735281.4068286174,-710625.3867525321,-685537.9534472737,-660011.9306284005,-634040.025051682,-607614.8246287087,-580728.7965128964,-553374.2851547489,-525543.5103262046,-497228.5651129605,-468421.41387564107,-439113.89017752185,-409297.6946796053,-378964.3930020286,-348105.41355138924,-316712.0453129597,-284775.4356086247,-252286.58781806938,-219236.35906405188,-185615.4578607676,-151414.44172465848,-116623.71474669455,-81233.52512695594,-45233.96266888315,-8614.956233969424,28633.728844021913,66522.4933861238,105061.90703767305,144262.71097902162,184135.8206808772,224692.32870276365,265943.50753630185,307900.8124935804,350575.88464228576,393980.5537861665,438126.8414943749,483026.96417834004,528693.3362178095,575138.5731364563,622375.4948288668,670417.1288373466,719276.7136824019,768967.702245492,819503.7652057642,870898.7945312741,923166.9070266769,976322.4479355686,1030379.9946019291,1085354.3601888856,1141260.5974568417,1198114.0026014852,1255930.1191537455,1314724.7419399414,1374513.9211067292,1435313.9662090847,1497141.4503635024,1560013.2144669755,1623946.3714840608,1688958.3107999968,1755066.7026450871,1822289.502588336,1890644.9561026366,1960151.6032023462,2030828.2831555866,2102694.139269093,2175768.6237514312,2250071.502652166,2325622.8608795935,2402443.107299187,2480552.979910683,2559973.551109673,2640726.2330316002,2722832.782980716,2806315.308944944,2891196.2751996447,2977498.507997548,3065245.201351631,3154459.9229084565,3245166.6199148567,3337389.6252790177,3431153.6637291415,3526483.858066773],[-1137294.7563075111,-1119327.028075357,-1101039.2265829793,-1082425.9918768923,-1063481.8763268464,-1044201.3432097274,-1024578.7652710711,-1004608.4232635142,-984284.5044619231,-963601.1011544203,-942552.2091099096,-921131.726020518,-899333.4499192911,-877151.0775726892,-854578.2028473162,-831608.3150502474,-808234.797243508,-784450.9245308705,-760249.8623176811,-735624.6645427567,-710568.2718820863,-685073.509923476,-659133.0873128118,-632739.5938698605,-605885.4986744572,-578563.1481219472,-550764.7639477204,-522482.44121973915,-493708.14629988535,-464433.71477188496,-434650.8493365715,-404351.1176735135,-373525.9502685168,-342166.6382060549,-310264.3309274155,-277810.03395207785,-244794.6065632142,-211208.75945625524,-177043.05234991992,-142287.89155870164,-106933.5275276876,-70970.05232700007,-34387.397106758785,2824.6704885242507,40676.54894676525,79178.80555756344,118342.17912271479,158177.58271129522,198696.10645788908,239909.0204056278,281827.77739432454,324464.01599533716,367829.5634918199,411936.4389077765,456796.85608470114,502423.2268073559,548828.1639791285,596024.4848487349,644025.2142867898,692843.5881159967,742493.0564936465,792987.2873480534,844340.1698695421,896565.8180578095,949678.574324016,1003693.0131519302,1058623.944816412,1114486.4191612415,1171295.7294368302,1229067.4161998942,1287817.2712732772,1347561.3417706192,1408315.9341840171,1470097.618536903,1532923.2326027318,1596809.88619171,1661774.9655036037,1727836.1375517817,1795011.3546565385,1863318.8590099467,1932777.1873130864,2003405.1754880114,2075221.9634622629,2148247.0000317334,2222500.0477994755,2298001.188193094,2374770.826562833,2452829.697358281,2532198.869389622,2612899.7511712383,2694954.096350308,2778384.009221288,2863211.9503293056,2949460.742159663,3037153.574920388,3126314.0124151227,3216965.9980094316,3309133.860691489,3402842.3212303105,3498116.498428701],[-1162081.7903246684,-1144141.5079002073,-1125881.3412715206,-1107295.9320772956,-1088379.8342944826,-1069127.5128223205,-1049533.3420440305,-1029591.6043654236,-1009296.488730167,-988642.0891109772,-967622.4029773229,-946231.3297379918,-924462.6691589898,-902310.1197561736,-879767.2771621624,-856827.6324668268,-833484.5705319499,-809731.3682781721,-785561.1929450186,-760967.1003229658,-735942.032957312,-710478.8183230099,-684570.1669711072,-658208.6706447427,-631386.8003654962,-604096.9044890439,-576331.2067298729,-548081.8041539923,-519340.665140487,-490099.627309632,-460350.39541836083,-430084.53922208026,-399293.49130238267,-367968.5448596887,-336100.8514715845,-303681.4188144244,-270701.1083490369,-237150.6329694849,-203020.55461434787,-168301.2818394052,-132983.0673526693,-97056.00550906779,-60510.02976561664,-23334.910096046515,14479.749635782093,52944.51434489945,92070.12043552333,131867.47855588933,172347.67639623722,213521.98153160885,255401.8443097463,297998.9007857032,341324.9757018322,385392.08551660506,430212.44148098724,475798.45276395604,522162.72962758644,569318.0866535255,617277.5460192482,666054.3408280136,715661.918491058,766113.9441638007,817424.3042365015,869607.109881362,922676.7006543553,976647.6481560403,1031534.7597497809,1087353.0823392645,1144117.906205941,1201844.7689083545,1260549.459241704,1320248.0212621745,1380956.7583743045,1442692.2374835405,1505471.2932146061,1569311.0321979215,1634228.8374220743,1700242.3726575435,1767369.5869496786,1835628.7191832345,1905038.3027192396,1975617.1701066494,2047384.4578664768,2120359.6113543413,2194562.3896989054,2270012.8708189023,2346731.4565208843,2424738.8776755286,2504056.199478569,2584704.826793965,2666706.509582179,2750083.3484142255,2834857.800074718,2921052.6832509628,3008691.184315119,3097796.8631966845,3188393.659348419,3280505.897806597,3374158.2953488417,3469375.9667466637],[-1187214.0934497658,-1169301.5320686586,-1151069.2773791268,-1132511.9726284076,-1113624.1734171775,-1094400.346283754,-1074834.8672659395,-1054922.0204397694,-1034655.9964349773,-1014030.8909262929,-993040.7031013521,-971679.3341034164,-949940.585449422,-927818.1574228059,-905305.6474405632,-882396.5483939203,-859084.2469631699,-835362.0219047822,-811223.0423116165,-786660.3658451668,-761666.9369396011,-736235.5849767902,-710359.0224329352,-684029.8429947279,-657240.5196459193,-629983.4027231329,-602250.7179408104,-574034.564384073,-545326.91247051,-516119.6018784675,-486404.33944273414,-456172.69701654185,-425416.1092995028,-394125.8716304437,-362293.1377459604,-329908.91750222025,-296964.07456090255,-263449.3240381719,-229355.2301161578,-194672.20361587824,-159390.49953248538,-123500.21453019883,-86991.28439765936,-49853.48146284232,-12076.411966664717,26350.486605789047,65437.94823531527,105196.88112319261,145638.3704889114,186773.68141274992,228614.2617233917,271171.7449323111,314457.95321349613,358484.90043200785,403264.7952201036,448810.0441024909,495133.254671148,542247.2388115176,590165.0159785091,638899.8165261727,688465.08508963,738874.4840209587,790141.8968796218,842281.4319792404,895307.4259891547,949234.4475949402,1004077.3012163318,1059851.03078445,1116570.9235789198,1174252.5141269173,1232911.5881624129,1292564.186650191,1353226.6098729074,1414915.4215833098,1477647.4532223078,1541439.8082050434,1606309.8662730474,1672275.2879176293,1739354.0188725158,1807564.294678046,1876924.6453176988,1947453.899929395,2019171.1915892772,2092095.9621739085,2166247.967298347,2241647.2813328113,2318314.3025000426,2396269.758051292,2475534.709526813,2556130.558098695,2638079.049998739,2721402.282032128,2806122.707180076,2892263.140288531,2979846.763849931,3068897.1338752853,3159438.185859694,3251494.2408421803,3345090.0115631092,3440250.6087163077],[-1212692.42639578,-1194807.8279601121,-1176603.728472224,-1158074.7727976376,-1139215.518170562,-1120020.4327782963,-1100483.8943231918,-1080600.188561578,-1060363.5078193806,-1039767.9494835844,-1018807.5144703027,-997476.1056676537,-975767.5263540011,-953675.4785909245,-931193.5615904548,-908315.270055929,-885033.9924969794,-861343.009516834,-837235.4920727082,-812704.4997082169,-787742.978757615,-762343.7605209851,-736499.5594110664,-710202.9710695988,-683446.47045409,-656222.4098937975,-628523.0171148779,-600340.3932334683,-571666.5107176306,-542493.2113158661,-512812.2039529965,-482615.06259237416,-451893.22406401206,-420637.9858576348,-388840.5038814491,-356491.7901841528,-323582.71064108657,-290103.98260342726,-256046.17250989424,-221399.69345988845,-186154.80274896557,-150301.59936398827,-113830.02143872483,-76729.84366897168,-38990.67468642676,-601.9543901160359,38447.04876339296,78167.24051434314,118569.70359729929,159665.70058345422,201466.6767680184,243984.26310431957,287230.27918329975,331216.7362618088,375955.8403384662,421459.99527864065,467741.80598901166,514814.08164351666,562689.838959035,611382.305524827,660904.9231841583,711271.3514699326,762495.4710947955,814591.3874976756,867573.4344449951,921456.177690933,976254.418695041,1031983.1983991689,1088657.8010642729,1146293.7581691765,1204906.8523694738,1264513.1215212657,1325128.8627678761,1386770.6366917752,1449455.2715323018,1513199.8674714314,1578021.8009855757,1643938.7292686673,1710968.5947244302,1779129.629530259,1848440.3602733798,1918919.612661826,1990586.516307848,2063460.509589753,2137561.344589635,2212909.09210966,2289524.1467690794,2367427.2321798126,2446639.406206599,2527182.0663094227,2609076.9549710173,2692346.1652101553,2777012.146183873,2863097.7088758075,2950626.031877545,3039620.667260283,3130105.546539937,3222104.986736582,3315643.69653143,3410746.782518527],[-1238522.3419398316,-1220665.9554709112,-1202490.2616425478,-1183989.9069492365,-1165159.4502695175,-1145993.3614505248,-1126486.0198701418,-1106631.71297606,-1086424.6348015512,-1065858.8844571144,-1044928.4645987034,-1023627.2798707595,-1001949.1353246204,-979887.734811676,-957436.6793507545,-934589.4654691622,-911339.4835178447,-887680.0159588705,-863604.2356259886,-839105.2039572282,-814175.8691993158,-788809.064583069,-762997.5064703806,-736733.7924707688,-710010.3995283295,-682819.6819779242,-655153.8695705028,-627005.0654663849,-598365.2441974275,-569226.2495957087,-539579.7926896266,-509417.44956631213,-478730.6591999703,-447510.7212451403,-415748.7937956669,-383435.89110693755,-350562.88128224295,-317120.48392218887,-283099.2677366333,-248489.64811802516,-213281.8846771354,-177466.07873841142,-141032.1707958202,-103969.93792820163,-66268.99117336096,-27918.772859772667,11091.446103002876,50772.5689784172,91135.67600019835,132192.02721439535,173953.06536653265,216430.4188354551,259635.90461256402,303581.5313298479,348279.50233547995,393742.21881854534,439982.2829833324,487012.50127498293,534845.8876549499,583495.6669301088,632975.2781341425,683298.3779628566,734478.8442640426,786530.7795836693,839468.5147668589,893306.6126178326,948059.871617205,1003743.3296986409,1060372.2680853466,1117962.215188521,1176528.9505659537,1236088.508945426,1296657.1843111394,1358251.5340552982,1420888.3831954994,1484584.8286601827,1549358.24364011,1615226.2820110884,1682206.8828259124,1750318.2748779026,1819578.9813367217,1890007.8244589847,1961623.9303713292,2034446.7339318865,2108495.983667654,2183791.746790417,2260354.414293377,2338204.706126381,2417363.6764556705,2497852.71900595,2579693.5724874865,2662908.3261090033,2747519.425179531,2833549.6767963283,2921022.2556258147,3009960.7097748606,3100388.966755455,3192331.339543716,3285812.5327364854,3380857.6488025626],[-1264713.6166639584,-1246885.7338778297,-1228738.7394263512,-1210267.2814512947,-1191465.920494053,-1172329.1280803278,-1152851.2852824468,-1133026.681258574,-1112849.5117686759,-1092313.8776662934,-1071413.783366963,-1050143.135291408,-1028495.7402841293,-1006465.3040067044,-984045.4293053856,-961229.6145522806,-938011.2519607176,-914383.6258728902,-890339.9110206037,-865873.1707580637,-840976.3552664684,-815642.2997295891,-789863.7224809544,-763633.2231205849,-736943.2806021373,-709786.2512892841,-682154.3669812472,-654039.7329062703,-625434.3256839861,-596329.9912543027,-566718.4427737142,-536591.2584779281,-505939.8795103971,-474755.60771581344,-443029.60339927603,-410752.8830487621,-377916.3170216973,-344510.6271945869,-310526.3845751928,-275954.0068760845,-240783.75605060672,-205005.7357884487,-168609.88897175575,-131585.99509068253,-93923.66761775967,-55612.35133980401,-16641.31964860717,23000.32821292663,63323.66994779697,104339.96304668719,146060.64767487627,188497.34960582713,231661.88320014533,275566.2544333623,320222.66397126066,365643.51029433403,411841.39287180174,458829.1153869848,506619.6890124874,555226.3357390263,604662.49175653,654941.810889177,706078.1680849441,758085.6629615328,810978.6234070281,864771.6092395447,919479.4159242385,975117.0783496303,1031699.874663814,1089243.330172561,1147763.22129763,1207275.5795998443,1267796.6958651743,1329343.1242560134,1391931.6865282455,1455579.4763163333,1520303.8634844813,1586122.498548993,1653053.3171699154,1721114.544714172,1790324.7008910598,1860702.6044624913,1932267.378025706,2005038.4528743797,2079035.5739356247,2154278.8047855473,2230788.532745475,2308585.4740568376,2387690.6791405096,2468125.53793849,2549911.785340622,2633071.5066970726,2717627.143419794,2803601.4986700322,2891017.7431388474,2979899.420917993,3070270.455464142,3162155.1556575256,3255578.221958033,3350564.752656056],[-1291266.1604209216,-1273467.0305834967,-1255348.9861707008,-1236906.676981566,-1218134.6652303594,-1199027.4241314854,-1179579.3364619163,-1159784.6931005858,-1139637.69154453,-1119132.4344007971,-1098262.927855062,-1077023.0801149665,-1055406.699828888,-1033407.4944794278,-1011019.0687511486,-988234.9228719717,-965048.4509286862,-941452.9391547942,-917441.5641914334,-893007.3913203485,-868143.3726686758,-842842.3453847019,-817097.0297852436,-790900.0274725768,-764243.8194217675,-737120.7640372643,-709523.0951785978,-681442.9201540998,-652872.2176834175,-623802.8358266661,-594226.4898808897,-564134.7602429441,-533519.0902381986,-502370.7839142792,-470681.00380043336,-438440.7686302215,-405640.95102831395,-372272.2751603408,-338325.3143452876,-303790.48862929223,-268658.0623218226,-232918.14149150578,-196560.67142243544,-159575.43403000245,-121952.04523547506,-83679.95229816437,-44748.43110635644,-5146.583423700649,35136.66590755433,76112.57180810673,117792.57184822345,160188.28918109555,203311.53552152496,247174.31417344417,291788.8231049734,337167.45807259995,383322.8157949508,430267.69717784366,478015.1105892365,526578.2751877476,575970.6243034769,626205.8088727472,677297.7009273688,729260.3971402775,782108.2224258771,835855.7335993978,890517.7230936051,946109.2227347936,1002645.5075787217,1060142.0998083558,1118614.772691904,1178079.5546055413,1238552.7331191842,1300050.8591474341,1362590.751166307,1426189.4994979887,1490864.4706616607,1556633.311795523,1623513.9551480585,1691524.6226408565,1760683.8305037334,1831010.393984608,1902523.4321318846,1975242.3726551565,2049186.956861856,2124377.2446724265,2200833.6197161977,2278576.7945058476,2357627.815696387,2438008.0694264346,2519739.286744539,2602843.549121268,2687343.294050263,2773261.3207353307,2860620.795870572,2949445.2595108063,3039758.631035424,3131585.215206556,3224949.708324769,3319877.2044794564],[-1318185.2008039085,-1300415.0756594124,-1282326.2344370726,-1263913.3286031988,-1245170.9220550694,-1226093.4897059652,-1206675.4160477936,-1186910.993690634,-1166794.4218790284,-1146319.8049840813,-1125481.150972219,-1104272.3698487284,-1082687.2720766652,-1060719.5669705367,-1038362.8610642098,-1015610.6564524879,-992456.3491067868,-968893.2271631842,-944914.4691834955,-920513.1423884553,-895682.2008626817,-870414.483730614,-844702.7133040994,-818539.4931995065,-791917.3064252406,-764828.513438558,-737265.3501714526,-709219.9260245534,-680684.22182989,-651650.0877802181,-622109.2413256872,-592053.2650369387,-561473.6044340422,-530361.5657804175,-498708.3138424379,-466504.8696123352,-433742.10799522186,-400410.75545916543,-366501.3876478048,-332004.42695438163,-296910.14005813235,-261208.63542038715,-224889.86074108072,-187943.60037484113,-150359.47270578938,-112126.92747998517,-73235.24309656536,-33673.523854416795,6569.302844368853,47504.48933575163,89143.47058014013,131497.86709545506,174579.48793558404,218400.33371760882,262972.5996966115,308308.678889574,354421.16524887737,401322.8568870714,449026.75935155526,497546.0889527905,546894.276144865,597084.9689599178,648132.0364971347,700049.572468061,752851.8987966767,806553.5692784307,861169.3732966399,916714.3395982021,973203.7401291798,1030653.093932251,1089078.1711044312,1148494.996819451,1208919.8554132232,1270369.294534442,1332860.1293609971,1396409.4468843723,1461034.6102601471,1526753.2632296532,1593583.3346109209,1661543.042861117,1730650.9007113385,1800925.7198761078,1872386.615835396,1945053.0126949698,2018944.6481226552,2094081.5783631396,2170484.183333472,2248173.1717971237,2327169.586622607,2407494.8101243456,2489170.5694885906,2572218.942285132,2656662.3620679234,2742523.624061773,2829825.890942068,2918592.6987047605,3008847.962629812,3100615.983338923,3193921.452950809,3288789.461331132],[-1345479.5937908897,-1327738.758008366,-1309679.4064828008,-1291296.1923635933,-1272583.681247002,-1253536.3497613308,-1234148.5841297403,-1214414.678709904,-1194328.834510521,-1173885.1576835604,-1153077.6579931912,-1131900.2472594862,-1110346.737777522,-1088410.8407112195,-1066086.1644614453,-1043366.2130077428,-1020244.3842242253,-996713.9681677723,-972768.1453393281,-948399.9849172339,-923602.4429623892,-898368.3605944116,-872690.4621393706,-846561.3532471375,-819973.5189790723,-792919.3218650345,-765390.9999294986,-737380.6646856344,-708880.2990982675,-679881.755513445,-650376.7535552909,-620356.8779893266,-589813.5765516004,-558738.1577428663,-527121.7885883774,-494955.4923610147,-462230.1462685405,-428936.4791039049,-395065.06885807216,-360606.3402943085,-325550.5624847999,-289887.84630696056,-253608.1419002344,-216701.23608237598,-179156.74972453015,-140964.13508389937,-102112.67309515178,-62591.4706173297,-22389.457637583837,18504.61556986021,60102.18132980494,102414.85749890516,145454.45044384012,189232.9580682777,233762.57288741274,279055.68515159097,325124.8860195009,371982.9707827261,419642.94214006653,468118.0135255349,517421.61248856597,567567.3841282004,618569.194581748,670441.1345698182,723197.5229960862,776852.9106060304,831422.0837029968,886920.067923638,943362.1320731258,1000763.7920223577,1059140.8146652686,1118509.2219409454,1178885.2949187434,1240285.5779485484,1302726.8828768427,1366226.2933307942,1430801.1690683737,1496469.150399698,1563248.1626775912,1631156.4208596935,1700212.4341428746,1770435.0106723816,1841843.2623234647,1914456.6095613977,1988294.7863773163,2063377.8453026707,2139726.162504326,2217360.442958257,2296301.7257077317,2376571.389203823,2458191.1567309042,2541183.1019179593,2625569.654338795,2711373.605198316,2798618.1131118266,2887326.7099746005,2977523.306924893,3069232.200401227,3162478.078297243,3257286.026211185],[-1373153.2123787617,-1355441.940216857,-1337412.3543180116,-1319059.1095281797,-1300376.773156188,-1281359.8235590926,-1262002.6487051158,-1242299.5447135472,-1222244.7143713394,-1201832.2656255795,-1181056.2100525529,-1159910.461301659,-1138388.8335146506,-1116485.0397196717,-1094192.6901995263,-1071505.290833608,-1048416.2414139579,-1024918.8339336484,-1001006.2508482698,-976671.563309463,-951907.7293702613,-926707.5921614321,-901063.8780394306,-874969.1947039182,-848416.0292856793,-821396.7464038066,-793903.5861920302,-765928.6622930146,-737463.9598215015,-708501.3332941057,-679032.5045263781,-649049.0604963377,-618542.4511738312,-587503.9873148948,-555924.838221811,-523796.0294664423,-491108.44057771284,-457852.8026921351,-424019.696166893,-389599.54815432243,-354582.6301388093,-318959.0554333008,-282718.7766363714,-245851.58304873947,-208347.09804860223,-170194.7764245239,-131383.9016671353,-91903.58321626438,-51742.75366494944,-10890.1659186529,30665.60969057679,72936.1883366569,115933.3736773734,159669.16088109463,204155.73970099213,249405.49759843294,295431.0229158718,342245.10810111463,389860.75298135774,438291.16809088085,487549.77805100894,537650.2250039387,588606.3721011458,640432.3070480986,693142.3457037164,746751.0357387969,801273.1603517774,856723.7420438388,913118.0464538359,970471.5862551341,1028800.1251126179,1088119.6817044918,1148446.5338070309,1209797.222444526,1272188.5561050023,1335637.615023973,1400161.7555341842,1465778.6144866375,1532506.1137407715,1600362.4647262292,1669366.173076883,1739536.043339639,1810891.1837556828,1883451.0111201098,1957235.255717413,2032263.9663355285,2108557.515360551,2186136.603950028,2265022.267290782,2345235.8799389773,2426799.161245237,2509734.18086553,2594063.36436097,2679809.498883687,2766995.7389556803,2855645.6123379744,2945783.025993209,3037432.2721424545,3130618.034419649,3225365.394120633],[-1401209.4894141671,-1383528.040293701,-1365528.4808866493,-1347205.4677457786,-1328553.569902829,-1309567.2674540335,-1290240.9501232237,-1270568.9158018802,-1250545.3690659245,-1230164.4196683264,-1209420.0810083759,-1188306.2685757445,-1166816.7983699257,-1144945.3852944167,-1122685.6415252124,-1100031.074852878,-1076975.0869987924,-1053510.9719037542,-1029631.9139896177,-1005330.9863930272,-980601.1491709563,-955435.2474772388,-929826.0097107121,-903766.0456329456,-877247.844456346,-850263.7729015586,-822806.0732239794,-794866.8612082768,-766438.1241317492,-737511.7186942981,-708079.3689157418,-678132.6639995375,-647663.0561623727,-616661.858428746,-585120.2423912424,-553029.2359340885,-520379.7209208617,-487162.4308452669,-453367.94844440743,-418986.70327356784,-384008.96924326755,-348424.8621160849,-312224.33696392365,-275397.1855847854,-237933.03387834178,-199821.33917910838,-161051.38754835865,-121612.2910215226,-81492.98481245479,-40682.22447293624,831.4169929418713,43059.552061029244,86013.98166477168,129706.69822162436,174149.888707045,219355.9377775453,265337.4309433601,312107.1577923768,359678.115263917,408063.5109761418,457276.76660568453,507331.5213212264,558241.6352715627,610021.1931300354,662684.5076936875,716246.1235413468,770720.8207491315,826123.6186651713,882469.779744261,939774.8134443518,998054.4801832656,1057324.795360105,1117602.0334397242,1178902.7321022958,1241243.696458737,1304642.0033340477,1369115.0056167664,1434680.3366796058,1501355.9148693043,1569159.948067984,1638110.9383268682,1708227.6865746626,1779529.2973984862,1852035.1839030702,1925765.0726458756,2000739.0086507895,2076977.3605024042,2154500.8255189527,2233330.4350096984,2313487.5596146234,2394993.9147291337,2477871.566014532,2562142.9349973993,2647830.804755047,2734958.3256939407,2823549.021418435,2913626.7946929093,3005215.933498179,3098341.1171854343,3193027.422724892],[-1429653.9695656868,-1412002.6054945984,-1394033.336043608,-1375740.8194826422,-1357119.6265767803,-1338164.239171934,-1318869.0487581387,-1299228.3550097512,-1279236.3643024215,-1258887.1882058766,-1238174.8419533486,-1217093.242885831,-1195636.2088716717,-1173797.4567009674,-1151570.6004542008,-1128949.1498445135,-1105926.508534152,-1082495.9724232221,-1058650.7279115447,-1034383.8501325569,-1009688.3011590724,-984556.9281799721,-958982.461648579,-932957.5134005563,-906474.5747422299,-879526.0145081931,-852104.0770880131,-824200.8804209637,-795808.4139596096,-766918.5365999592,-737522.9745790241,-707613.319338737,-677181.0253557749,-646217.4079363458,-614713.6409766946,-582660.7546869074,-550049.6332788086,-516871.0126169843,-483115.47783230804,-448773.4608969181,-413835.23816156154,-378290.92785260733,-342130.4875295856,-305343.7115022051,-267920.2282061572,-229849.49753657356,-191120.80814014282,-151723.27466282854,-111645.83495435165,-70877.24722796539,-29406.087175111752,12779.254966557492,55690.577390585095,99339.86974783335,143739.3162201927,188901.29864337156,234838.39967926685,281563.40603961004,329089.3117594817,377429.3215243821,426596.85404961,476605.5455135121,527469.2530452535,579202.0582689638,631818.2709025429,685332.4324154868,739759.3197440142,795113.9490655358,851411.5796329263,908667.717670735,966898.1203315421,1026118.7997170426,1086346.026962188,1147596.3363844235,1209886.5296987216,1273233.680300631,1337655.1376153585,1403168.5315180393,1469791.776823226,1537543.0778459148,1606440.9330348661,1676504.1396806356,1747751.798696096,1820203.3194752932,1893878.4248281764,1968797.155993882,2044979.877734648,2122447.2835083315,2201220.400725412,2281320.596088281,2362769.5810155147,2445589.417151958,2529802.521967671,2615431.674442952,2702500.0208463687,2791031.0806030305,2881048.7522562817,2972577.319523699,3065641.4574505733,3160266.2386580775],[-1458497.1074468642,-1440876.1344060334,-1422937.4629287892,-1404675.7530203834,-1386085.5771976053,-1367161.4190746227,-1347897.6719264223,-1328288.6372292025,-1308328.5231774962,-1288011.4431771527,-1267331.4143149124,-1246282.355802859,-1224858.0873981894,-1203052.3277977786,-1180858.6930070177,-1158270.6946822528,-1135281.7384474333,-1111885.1221830535,-1088074.034288178,-1063841.551914571,-1039180.6391726292,-1014084.1453082685,-988544.8028515026,-962555.2257345587,-936107.9073803758,-909195.2187604222,-881809.4064216369,-853942.5904813344,-825586.7625910279,-796733.7838667817,-767375.382786999,-737503.1530565536,-707108.551436827,-676182.8955407431,-644717.3615934795,-612702.9821564956,-580130.6438156869,-546991.0848326341,-513274.8927583359,-478972.50200847164,-444074.19140094845,-408570.08165318985,-372450.13283989485,-335704.141810318,-298321.739564341,-260292.38858614117,-221605.38013661932,-182249.83150134282,-142214.683195299,-101488.69612294715,-60060.44869312411,-17918.333887360524,24949.44371725805,68554.87097394979,112910.12923964811,158027.59749751212,203919.85552901123,250599.6871372927,298080.0834203642,346374.2460979228,395495.59089038894,445457.7509518671,496274.5803576256,547960.1576479096,600528.789426446,653995.01401794,708373.605182915,763679.5758918109,819928.1821589721,877134.9269384961,935315.5640802667,994486.1023507188,1054662.8095166069,1115862.2164939055,1178101.1215625228,1241396.5946489526,1305765.981675034,1371226.908977833,1437797.2877987954,1505495.3188443976,1574339.496919062,1644348.6156328549,1715541.772181613,1787938.3722054036,1861558.1347228824,1936421.0971441604,2012547.6203643451,2089958.3939356487,2168674.44132399,2248717.125247835,2330108.153102116,2412869.5824678596,2497023.8267106875,2582593.6606654827,2669602.226413893,2758073.0391522422,2848029.9931527367,2939497.3678190294,3032499.833839245,3127062.4594336986],[-1487740.3346141893,-1470150.0245714248,-1452242.2245811792,-1434011.5963954092,-1415452.714294089,-1396560.0636711933,-1377328.0395983395,-1357750.9453653304,-1337822.9909974649,-1317538.291748665,-1296890.8665712716,-1275874.6365606312,-1254483.423375076,-1232710.9476306613,-1210550.8272701812,-1187996.5759058057,-1165041.6011359054,-1141679.2028341887,-1117902.5714119177,-1093704.7860522422,-1069078.8129162944,-1044017.5033203352,-1018513.5918844759,-992559.6946510102,-966148.3071731345,-939271.8025729728,-911922.4295687007,-884092.3104696814,-855773.4391404947,-826957.6789315357,-797636.7605769597,-767802.2800590945,-737445.6964386636,-706558.3296500645,-675131.3582623086,-643155.8172032558,-610622.5954480437,-577522.4336705159,-543845.9218572569,-509583.49688304216,-474725.44004865736,-439261.87457842706,-403182.7630782011,-366477.904952927,-329136.93378294073,-291149.31465790747,-252504.3414695002,-213191.13415961014,-173198.635925367,-132515.61037945375,-91130.63866529148,-49032.11652560718,-6208.251325602643,37352.940972652286,81663.63888332015,126736.21851905715,172583.25676277233,219217.5344907255,266652.03984541725,314899.9715620931,363974.7423475045,413889.98231256194,464659.54245949024,516297.4982253327,568818.1530801514,622236.042184215,676565.9361024904,731822.8445784166,788022.0203676028,845178.9631332918,903309.4234021045,962429.4065844268,1022555.1770578632,1083703.2623157706,1145890.4571816213,1209133.8280913294,1273450.71744161,1338858.7480095294,1405375.8274412672,1473020.1528124218,1541810.2152605373,1611764.8046924197,1682903.0145638888,1755244.2467378373,1828808.2164182225,1903614.9571625204,1979684.8259748626,2057038.5084777465,2135697.024168226,2215681.7317563593,2297014.334588687,2379716.886157414,2463811.7956985645,2549321.8338761176,2636270.1385591524,2724680.220689272,2814575.970241398,2905981.6622788366,2998921.9631058923,3093421.936515114],[-1517388.906093961,-1499829.5290614674,-1481952.872064733,-1463753.5986129954,-1445226.284760063,-1426365.4176904818,-1407165.3942833378,-1387620.5196529606,-1367725.0056664278,-1347472.9694368553,-1326858.4317933794,-1305875.3157259023,-1284517.4448052754,-1262778.5415782232,-1240652.2259365122,-1218132.0134598042,-1195211.313732665,-1171883.428633875,-1148141.5505988726,-1123978.7608542356,-1099388.0276240252,-1074362.2043070893,-1048894.0276260006,-1022976.1157456066,-996600.9663619355,-969760.9547604334,-942448.3318432986,-914655.2221248494,-886373.6216957348,-857595.396153762,-828312.2785020866,-798515.8670138088,-768197.6230624388,-737348.8689174089,-705960.7855052177,-674024.4101339318,-641530.6341817975,-608470.2007489775,-574833.7022717516,-540611.578098265,-505794.11202654196,-470371.42980230646,-434333.4965771972,-397670.11432658555,-360370.91922612814,-322425.3789859754,-283822.7901437008,-244552.27531276504,-204602.78038782394,-163963.07170528965,-122621.73315881379,-80567.1632681014,-37787.57220237842,5729.021244939882,49994.79272908205,95022.1154758567,140823.56345316302,187411.91459375992,234800.15406789584,283001.47760948585,332029.2948945728,381897.2329736869,432619.1397586847,484209.08756591985,536681.3767141877,590050.53918151,644331.3423193726,699538.7926261364,755688.1395804188,812794.8795362194,870874.7596783168,929943.7820423376,990018.2075978415,1051114.5603965246,1113249.631786229,1176440.4846928804,1240704.457968533,1306059.1708105342,1372522.5272499179,1440112.72071133,1508848.238645194,1578747.8672346552,1649830.696174906,1722116.1235309145,1795623.8606709703,1870373.937278775,1946386.706446162,2023682.849844418,2102283.382980032,2182209.6605327567,2263483.3817786486,2346126.596098853,2430161.708577297,2515611.4856844037,2602499.061053809,2690847.941349333,2780682.0122253154,2872025.544381273,2964903.199713998,3059340.0375643633],[-1547448.7031193143,-1529920.5318102203,-1512075.292026978,-1493907.6490467545,-1475412.180707939,-1456583.3759964781,-1437415.6336098297,-1417903.2604978706,-1398040.470380548,-1377821.382241363,-1357240.0187975161,-1336290.304944855,-1314966.0661782196,-1293261.0269865475,-1271168.8092222407,-1248682.9304442245,-1225796.8022350906,-1202503.7284906546,-1178796.9036825707,-1154669.4110930033,-1130114.2210211891,-1105124.1889609036,-1079692.0537496584,-1053810.4356874006,-1027471.8346256542,-1000668.6280259478,-973393.0689874005,-945637.2842422454,-917393.2721203051,-888652.9004800068,-859407.9046068033,-829649.8850779925,-799370.3055934203,-768560.4907711912,-737211.6239090583,-705314.7447091723,-672860.7469669082,-639840.3762228324,-606244.2273771758,-572062.7422657884,-537286.2071984308,-501904.75045680255,-465908.33975306805,-429286.77964785835,-392029.70892713405,-354126.5979366214,-315566.7458750182,-276339.2780427197,-236433.14304739004,-195837.10996479122,-154539.76545455027,-112529.51082924102,-69794.55907813087,-26322.93184213573,17897.543658729177,62879.237747007515,108634.72145899711,155176.76976726577,202518.3648529225,250672.6994315558,299653.18013135483,349473.4309251611,400147.29661701806,451688.84638501797,504112.377378908,557432.4183765813,611663.7334979768,666821.3259781878,722920.4420004464,779976.5745909936,838005.4675740851,897023.1195917358,957045.7881864468,1018089.9939490925,1080172.524732511,1143310.4399331608,1207521.0748387459,1272822.04504704,1339231.2509539141,1406766.8823128575,1475447.422866818,1545291.6550547066,1616318.6647903812,1688547.8463199567,1761998.907154975,1836691.8730840776,1912647.0932653449,1989885.2453971868,2068427.3409736995,2148294.7306222976,2229509.1095263287,2312092.522933455,2396067.3717528805,2481456.4182386636,2568282.791765986,2656569.994697652,2746341.9083440206,2837622.799017149,2930437.3241825076,3024810.5387052405],[-1577930.074164697,-1560433.4215442892,-1542619.9139748905,-1524484.2185193817,-1506020.914818694,-1487224.4936783027,-1468089.3556323925,-1448609.8094849044,-1428780.070827372,-1408594.2605325964,-1388046.403224951,-1367130.4257255404,-1345840.155472685,-1324169.3189172654,-1302111.5398922425,-1279660.3379559068,-1256809.1267092302,-1233551.2120856128,-1209879.7906136897,-1185787.947652199,-1161268.655596737,-1136314.7720574646,-1110919.0380084629,-1085074.0759066592,-1058772.3877811905,-1032006.353292024,-1004768.2277577629,-977050.1401513871,-948844.091064946,-920141.950640793,-890935.4564702385,-861216.2114585852,-830975.6816560552,-800205.1940537379,-768895.9343452367,-737038.9446515911,-704625.1212104172,-671645.2120280468,-638089.8144942955,-603949.3729586415,-569214.1762687494,-533874.3552687955,-497919.8802581909,-461340.5584098785,-424126.03114745626,-386265.77147985715,-347749.08129480993,-308565.08860787563,-268702.74476824794,-228150.82161988737,-186897.90861753654,-144932.4098961004,-102242.54129466088,-58816.32733175438,-14641.598133054562,30294.013689956628,76003.07621088019,122498.36141064065,169792.84844946628,217899.72699248232,266832.40058755456,316604.49009702215,367229.83718391974,418722.5078545371,471096.79605566384,524367.2273307913,578548.5625336501,633655.801601008,689704.1873853183,746709.2095492352,804686.6085202959,863652.3795102965,923622.7765976894,984614.3168750782,1046643.7846625191,1109728.23578873,1173885.001938411,1239131.6950706672,1305486.2119066813,1372966.7384888413,1441591.7548121917,1511380.0395305748,1582350.674735196,1654523.0508115683,1727916.8713722508,1802552.1582681355,1878449.2566803712,1955628.8402908426,2034111.9165371135,2113919.8319496284,2195074.2775739105,2277597.2944784784,2361511.279351701,2446838.990184605,2533603.5520467213,2621828.462952152,2711537.5998190083,2802755.2245231355,2895505.9900493072,2989814.9467370864],[-1608835.4757362632,-1591370.626683107,-1573589.1378301405,-1555485.6780380192,-1537054.828762374,-1518291.0826404968,-1499188.8420556707,-1479742.4176783843,-1459946.0269843708,-1439793.7927484296,-1419279.7415149363,-1398397.8020431616,-1377141.8037279968,-1355505.4749953954,-1333482.441672162,-1311066.2253293584,-1288250.2415998527,-1265027.7984682654,-1241392.0945339531,-1217336.2172461164,-1192853.1411107245,-1167935.7258684372,-1142576.7146441608,-1116768.7320661882,-1090504.2823557707,-1063775.747385958,-1036575.3847095894,-1008895.325555305,-980727.5727924162,-952063.9988623699,-922896.3436776358,-893216.2124869619,-863015.0737065324,-832284.256716154,-801014.9496211202,-769198.1969773881,-736824.89748094,-703885.8016201982,-670371.5092909997,-636272.4673730214,-601578.9672686057,-566281.1424012752,-530368.965674757,-493832.2468915065,-456660.6301301066,-418843.5910802148,-380370.4343362842,-341230.2906468753,-301412.11412071297,-260904.67938811285,-219696.57871726993,-177776.21908392292,-135131.81919565052,-91751.40646742098,-47622.81394960638,-2733.6772069055587,42928.568852239754,89376.69319756282,136623.67194867972,184682.69170035003,233567.15290002292,283290.67327926075,333867.0913396464,385310.4698949866,437635.0996682849,490855.50294759125,544986.4372991966,600042.899340116,656040.1285703955,712993.6112672719,770919.0844395286,829832.539846506,889750.2280801553,950688.6627121773,1012664.6245069443,1075695.1657023826,1139797.6143568675,1204989.578767341,1271288.9519565431,1338713.916231804,1407282.9478160911,1477014.821553755,1547928.6156887226,1620043.7167210113,1693379.8243390946,1767956.9564307667,1843795.4541746723,1920915.9872103278,1999339.5588927106,2079087.5116289617,2160181.532300203,2242643.657769027,2326496.2804758437,2411762.1541213538,2498464.3994418303,2586626.5100748125,2676272.358518058,2767426.2021827907,2860112.6895444565,2954356.8663880755],[-1640170.2837050236,-1622737.5192798143,-1604988.3317501177,-1586917.3917852575,-1568519.2826665514,-1549788.4988741914,-1530719.4446517143,-1511306.4325474289,-1491543.6819326053,-1471425.3174954911,-1450945.367712003,-1430097.763291232,-1408876.3355963246,-1387274.815040178,-1365286.8294553503,-1342905.9024376853,-1320125.4516640808,-1296938.7871825926,-1273339.1096756405,-1249319.5086952925,-1224872.9608704352,-1199992.3280848481,-1174670.355627003,-1148899.6703094065,-1122672.7785583746,-1095982.0644730835,-1068819.7878538042,-1041178.082198103,-1013048.9526659257,-984424.2740113311,-955295.7884815517,-925655.1036825115,-895493.6904101996,-864802.8804470953,-833573.8643242533,-801797.6890467447,-769465.2557832422,-736567.3175186722,-703094.4766694438,-669037.182660141,-634385.7294625598,-599130.2530944995,-563260.72907905,-526766.9698633617,-489638.6221963172,-451865.1644637268,-413435.9039823469,-374339.9742494095,-334566.33214902785,-294103.75511382753,-252940.8382415753,-211065.99136506533,-168467.43607675284,-125133.20270456234,-81051.12724021962,-36208.848218438216,9406.196453351062,55806.7727166065,103005.85363308853,151016.62270980934,199852.4772762875,249527.03191561298,300054.1219500811,351447.80698304623,403722.37449557055,456892.3435019711,510972.46826264635,565977.7420562422,621923.4010116365,678824.9280017,736698.0565973707,795558.7750862734,855423.3305544206,916308.2330330187,978230.259710995,1041206.4592155186,1105254.1559585715,1170390.9545546146,1236634.7443074952,1304003.7037688447,1372516.3053687336,1442191.3201210215,1513047.822401166,1585105.194802334,1658383.133067357,1732901.65109919,1808681.0860520294,1885742.1035009213,1964105.7026959122,2043793.221898349,2124826.34380224,2207227.1010413268,2291017.8817849876,2376221.4354202384,2462860.8783266037,2550959.6997413356,2640541.7677178965,2731631.3351787566,2824253.046065663,2918431.941584541],[-1671940.7365651992,-1654540.340648627,-1636823.739882057,-1618785.6067544478,-1600420.5263838912,-1581722.9951046107,-1562687.4190316608,-1543308.1126025075,-1523579.2970953868,-1503495.0991235299,-1483049.5491060077,-1462236.5797134468,-1441050.024289088,-1419483.6152446403,-1397530.982430448,-1375185.651479266,-1352441.0421242006,-1329290.4664890564,-1305727.1273517332,-1281744.116379693,-1257334.4123373271,-1232490.8792642874,-1207206.2646254224,-1181473.1974303925,-1155284.1863236122,-1128631.6176435577,-1101507.7534511702,-1073904.7295263144,-1045814.5533330676,-1017229.1019516774,-988140.119977864,-958539.2173885456,-928417.8673734455,-897767.4041317757,-866579.0206345397,-834843.7663501999,-802552.5449345326,-769696.1118835069,-736265.0721487622,-702249.8777145483,-667640.8251370457,-632428.0530433524,-596601.5395910637,-560151.0998873264,-523066.3833667226,-485336.87112676445,-446951.873222176,-407900.5259146928,-368171.7888797363,-327754.4423683374,-286637.08432401903,-244808.12745301006,-202255.79624912236,-158968.1239698748,-114932.94956509722,-70137.91455645254,-24570.459867423866,21782.177397987805,68932.96722631296,116895.09001934063,165681.93997097714,215307.12849806948,265784.48772564204,317128.07402851153,369352.1716275378,422471.2962448625,476500.19881642936,531453.8692637896,587347.540325705,644196.6914516794,702017.0527555388,760824.6090338072,820635.6038470147,881466.5436661183,943334.2020847071,1006255.6240991512,1070248.1304547521,1135329.3220630833,1201517.0844885097,1268829.592506181,1337285.3147323108,1406903.018329125,1477701.7737822933,1549700.9597566202,1622920.2680275957,1697379.708491461,1773099.6142558646,1850100.6468091384,1928403.801273916,2008030.4117430951,2089002.1567007136,2171341.0645285957,2255069.5191018214,2340210.2654702733,2426786.4156330484,2514821.454403222,2604339.2453658837,2695364.036930441,2787920.4684804357,2882033.576617945],[-1704155.3796826932,-1686787.6594561338,-1669103.954504089,-1651098.9391505388,-1632767.2003658372,-1614103.2363539457,-1595101.4551172599,-1575756.1729983843,-1556061.613198677,-1536011.904272658,-1515601.07859903,-1494823.0708265987,-1473671.7162954805,-1452140.74943321,-1430223.8021250393,-1407914.4020579343,-1385205.9710387331,-1362091.8232846458,-1338565.1636868909,-1314619.0860463523,-1290246.571281148,-1265440.4856051772,-1240193.578678265,-1214498.4817259794,-1188347.705629815,-1161733.6389866744,-1134648.5461375255,-1107084.5651640813,-1079033.7058542997,-1050487.847634567,-1021438.7374692312,-991877.9877265645,-961797.0740106106,-931187.3329580808,-900039.9600009064,-868346.0070921783,-836096.3803961934,-803281.8379416082,-769892.987237175,-735920.2828489221,-701354.0239397013,-666184.3517684969,-630401.2471502214,-593994.5278750286,-556953.8460864727,-519268.68561729323,-480928.3592839376,-441922.0061366507,-402238.5886664316,-361866.8899672176,-320795.5108530717,-279012.8669286808,-236507.18561450252,-193266.50312321773,-149278.6613886035,-104531.30494534504,-59011.87775935978,-12707.620006798767,34394.435196744744,82307.46512058936,131044.86079553142,180620.2304443526,231047.4029667345,282340.43148052273,334513.59691769443,387581.41167922225,441558.6233472689,496460.2184566329,552301.426326077,609097.7229514085,666864.8349587843,725618.7436227109,785375.6889469652,846152.1738106878,907964.9681802047,970831.1133887912,1034767.9264824586,1099793.0046369522,1165924.22964379,1233179.7724679704,1301578.0978777888,1371137.9691494303,1441878.4528439776,1513818.92366269,1586979.0693781697,1661378.895843991,1737038.7320849523,1813979.2354658926,1892221.3969449387,1971786.5464089843,2052696.3580941614,2134972.856092004,2218638.4199445248,2303715.7903252435,2390228.0748132,2478198.753757228,2567651.6862335093,2658611.1160974405,2751101.678132936,2845148.4042963716],[-1736821.1608904223,-1719486.431243096,-1701835.9389483589,-1683864.3601816618,-1665566.2837822814,-1646936.2098407068,-1627968.548263645,-1608657.6173160027,-1588997.642139583,-1568982.7532476953,-1548606.9849963665,-1527864.2740304207,-1506748.457704929,-1485253.272481466,-1463372.352298619,-1441099.2269161844,-1418427.3202335224,-1395349.9485802727,-1371860.318980147,-1347951.527386864,-1323616.5568918607,-1298848.275903077,-1273639.4362953294,-1247982.6715303229,-1221870.4947470336,-1195295.296821445,-1168249.3443954324,-1140724.777873605,-1112713.6093891005,-1084207.7207359727,-1055198.8612689446,-1025678.645769618,-995638.5522785624,-965069.9198924331,-933963.9465267833,-902311.6866422202,-870104.04893474,-837331.7939891049,-803985.5318948212,-770055.7198236021,-735532.6595691536,-700406.4950467525,-664667.2097533019,-628304.6241869065,-591308.3932253174,-553668.0034619682,-515372.7705007782,-476411.83620655024,-436774.16591214295,-396448.5455810125,-355423.5789246247,-313687.684473298,-271229.0926015987,-228035.8425051272,-184095.77912963415,-139396.55005122907,-93925.60230691079,-47670.17917394079,-617.3168993494473,47246.15862409491,95933.63523416314,145458.71792931762,195835.23235329427,247077.2283360646,299198.9834894482,352215.00686164666,406140.04264908796,460989.07396756625,516777.32668313663,573520.2733049304,631233.6369380662,689933.3953012708,749635.7848074725,810357.3047095151,872114.7213115706,934925.0722485813,998805.6708316449,1063774.1104645892,1129848.2691296935,1197046.313944939,1265386.7057934697,1334888.2040277594,1405569.8712462056,1477451.0781480297,1550551.508464009,1624891.1639656764,1700490.3695551585,1777369.7784335623,1855550.3773537986,1935053.491955583,2015900.7921855184,2098114.297802772,2181716.3839737,2266729.786952434,2353177.609854429,2441083.328520247,2530470.7974726553,2621364.2559680175,2713788.334145096,2807768.0592685207],[-1769941.8715277845,-1752640.4250526433,-1735023.4396081185,-1717085.5932323686,-1698821.4766449146,-1680225.5918341817,-1661292.350622717,-1642016.0732093023,-1622390.9866879093,-1602411.2235424079,-1582070.8201180475,-1561363.7150676663,-1540283.7477733723,-1518824.6567429525,-1496980.0779806327,-1474743.5433314207,-1452108.4787996914,-1429068.2028400882,-1405615.9246215404,-1381744.7422633737,-1357447.6410432726,-1332717.491576236,-1307547.04796518,-1281928.9459211722,-1255855.7008540237,-1229319.7059322584,-1202313.2301121878,-1174828.416135027,-1146857.278492872,-1118391.7013613451,-1089423.4364995775,-1059944.1011166368,-1029945.1757038604,-999418.001832231,-968353.7799154173,-936743.566936174,-904578.2741369242,-871848.6646733948,-838545.3512308486,-804658.7936017876,-770179.2962260186,-735097.0056904792,-699401.9081895519,-663083.8269449384,-626132.4195843427,-588537.1754777827,-550287.4130326975,-511372.27694458,-471780.73540447606,-431501.57726178225,-390523.409141975,-348834.65251769405,-306423.54073449597,-263278.1159878783,-219386.2262527533,-174735.5221639201,-129313.45384691749,-83107.26769770915,-36104.00311250193,11709.510835972149,60346.65877439873,109821.04245870467,160146.4842583444,211337.03069689963,263406.95604735287,316370.7659863061,370243.2013054737,425039.2416824498,480774.1095112944,537463.2737949649,595122.45409788,653767.6245631669,713415.0179929142,774081.1299934797,835782.7231865632,898536.8314882391,962360.7644539955,1027272.111694877,1093288.747362859,1160428.8347076713,1228710.8307058807,1298153.4907646244,1368775.8734978423,1440597.345580725,1513637.586680042,1587916.5944629046,1663454.6896862197,1740272.521364592,1818391.0720227365,1897831.6630300577,1978615.9600202464,2060765.9783965289,2144304.0889258403,2229253.023418941,2315635.880503497,2403476.1314873947,2492797.6263153707,2583624.599619914,2675981.676869582,2769893.880611974],[-1803524.1293288297,-1786256.2615633993,-1768673.0801200955,-1750769.264911103,-1732539.4085475723,-1713978.0149273744,-1695079.4978004752,-1675838.1793112422,-1656248.2885175613,-1636303.9598857833,-1615999.231762321,-1595328.0448201194,-1574284.240480481,-1552861.5593096975,-1531053.6393899892,-1508854.0146640593,-1486256.113253878,-1463253.255751772,-1439838.6534846711,-1416005.4067504124,-1391746.5030259162,-1367054.8151463398,-1341923.09945593,-1316343.993928437,-1290310.0162579378,-1263813.56191905,-1236846.9021961747,-1209402.1821809104,-1181471.4187382085,-1153046.498439244,-1124119.1754616802,-1094681.0694562825,-1064723.6633795383,-1034238.301291211,-1003216.1861176991,-971648.3773786086,-939525.7888776409,-906839.1863564576,-873579.1851111641,-839736.2475702683,-805300.6808350342,-770262.6341795325,-734612.096511242,-698338.8937911922,-661432.6864129007,-623882.9665389955,-585679.0553965583,-546810.1005280586,-507265.0729991463,-467032.76456170995,-426101.7847719132,-384460.5580615769,-342097.3207642082,-299000.1180923367,-255156.80106729642,-210555.02339999191,-165182.2383220601,-119025.69536588667,-72072.43709470937,-24309.295779248234,24277.109977920074,73700.37867188407,123974.32937932294,175113.0052987882,227130.67734562932,280041.8478058912,333861.25404743943,388603.8722903924,444284.92143736593,500919.8669655118,558524.4248787593,617114.5657246984,676706.5186744179,737316.7756674914,798962.095622682,861659.5087165344,925426.3207280738,990280.1174545661,1056238.7691964763,1123320.4353139023,1191543.5688552577,1260926.9212606149,1331489.5471374895,1403250.809114921,1476230.382773323,1550448.2616529036,1625924.7623425922,1702680.529647598,1780736.5418413347,1860114.115999564,1940834.9134195368,2022920.9451248422,2106394.5774590364,2191278.537765377,2277595.9201593767,2365370.1913916958,2454625.196804244,2545385.1663806345,2637674.720894005,2731518.878149477],[-1837574.650427017,-1820340.6598854312,-1802791.582584606,-1784922.100322145,-1766726.807612334,-1748200.2102740526,-1729336.7239963713,-1710130.6728809986,-1690576.2879616185,-1670667.7056990082,-1650398.966452821,-1629764.0129282565,-1608756.6885980836,-1587370.7360994618,-1565599.795605052,-1543437.4031678375,-1520876.9890400334,-1497911.875964435,-1474535.2774388513,-1450740.2959526076,-1426519.9211949504,-1401867.0282344145,-1376774.3756698712,-1351234.603751166,-1325240.232470196,-1298783.6596212818,-1271857.1588307105,-1244452.877554304,-1216562.8350438464,-1188178.9202802056,-1159292.8898737687,-1129896.3659313545,-1099980.833889034,-1069537.6403099606,-1038557.9906479262,-1007032.9469742402,-974953.4256688175,-942310.1950743285,-909093.8731129216,-875294.9248644449,-840903.6601070585,-805910.2308175568,-770304.6286322414,-734076.6822672901,-697216.0548980557,-659712.2414958575,-621554.5661236886,-582732.1791873858,-543234.0546437185,-503048.9871637481,-462165.58925113175,-420572.28831381863,-378257.32369035995,-335208.74362756824,-291414.40221058065,-246861.95624392573,-201538.86208304856,-155432.37241456425,-108529.53298675176,-60817.17928644689,-12281.933163766284,37089.800597062334,87311.83776135044,138398.21818384668,190363.2094033528,243221.3102962575,296987.25478732074,351676.015619779,407302.80818520347,463883.09441520646,521432.5867332951,579967.2520714151,639503.3159494102,700057.2666196306,761645.8592772563,824286.1203385247,887995.3517850274,952791.1355790854,1018691.3381483126,1085714.1149416561,1153877.9150576652,1223201.48594744,1293703.87819,1365404.4503459204,1438322.8738868255,1512479.1382033029,1587893.5556934816,1664586.7669300907,1742579.7459119651,1821893.805397755,1902550.6023246213,1984572.143312599,2067980.7902577957,2152799.2660116525,2239050.6601530337,2326758.4348506103,2415946.43081855,2506638.8733664127,2598860.3785465267,2692635.9593959367],[-1872103.2945338008,-1854903.5108507234,-1837388.8693772028,-1819554.0538136573,-1801393.6605950836,-1782902.1974791617,-1764074.0821119875,-1744903.640570772,-1725385.1058833045,-1705512.6165232991,-1685280.2148823629,-1664681.845716844,-1643711.3545700668,-1622362.486169338,-1600628.882797288,-1578504.082636839,-1555981.518090378,-1533054.5140712599,-1509716.2862685036,-1485959.939383458,-1461778.4653384371,-1437164.7414563193,-1412111.5286117855,-1386611.4693522004,-1360657.0859888892,-1334240.7786577682,-1307354.823349078,-1279991.3699052013,-1252142.4399873426,-1223799.9250088232,-1194955.5840358282,-1165601.0416545128,-1135727.7858040682,-1105327.165574822,-1074390.3889720098,-1042908.5206429418,-1010872.479568305,-978273.0367165946,-945100.8126610867,-911346.2751583322,-876999.7366890172,-842051.3519585594,-806491.1153582535,-770308.8583859303,-733494.2470254744,-696036.7790839775,-657925.7814876479,-619150.4075333294,-579699.6340968395,-539562.2587966272,-498726.89711239794,-457181.9794570501,-414915.74820335675,-371916.25466188276,-328171.35601135856,-283668.71218009293,-238395.78267774917,-192339.82337599015,-145487.8832392376,-97826.8010019171,-49343.201793498825,-23.493709666654468,50146.13567099534,101179.7228265605,153091.5318860514,205896.05828259746,259608.03246390168,314242.4236619598,369814.44372256333,426339.55099667935,483833.454291909,542312.1168886316,601791.760619178,662288.8700119625,723820.1965014837,786402.7627061498,850053.8667721716,914791.086788578,980632.2852713335,1047595.6137190112,1115699.5172406323,1184962.739258213,1255404.32628171,1327043.6327622393,1399900.3260211414,1473994.3912574574,1549346.1366360835,1625976.1984543665,1703905.5463932073,1783155.4888502713,1863747.678358254,1945704.1170887663,2029047.1624451084,2113799.5327409925,2199984.3129722304,2287624.960678577,2376745.311898966,2467369.5872208965,2559522.3979273275,2653228.7522381153],[-1907115.6267796475,-1889950.3701419677,-1872470.486564883,-1854670.661666478,-1836545.4938174686,-1818089.4927294462,-1799297.0780208453,-1780162.5777597895,-1760680.2269837651,-1740844.1661951488,-1720648.4398333721,-1700086.9947219712,-1679153.6784910036,-1657842.237974284,-1636146.3175808887,-1614059.4576403482,-1591575.0927220332,-1568686.5499268675,-1545387.0471521972,-1521669.6913287167,-1497527.4766292993,-1472953.2826488086,-1447939.8725555593,-1422479.8912124399,-1396565.8632683775,-1370190.1912192286,-1343345.153437725,-1316022.9021715312,-1288215.461510138,-1259914.725318442,-1231112.4551376696,-1201800.2780527798,-1171969.6845257417,-1141612.0261938856,-1110718.5136339606,-1079280.2140894812,-1047288.0491623776,-1014732.7924676014,-981605.0672504054,-947895.3439651029,-913593.9378161626,-878691.0062591098,-843176.5464619296,-807040.3927259878,-770272.2138658618,-732861.510546736,-694797.6125806295,-656069.676178176,-616666.6811572779,-576577.4281070367,-535790.535506675,-494294.43679780187,-452077.3774113762,-409127.411745911,-365432.40009820927,-320980.0055450401,-275757.69077526825,-229752.71487079747,-182952.1300376542,-135342.77828358952,-86911.28804341378,-37644.07075051544,12472.682646109723,63453.00522107119,115311.15766654629,168061.63194506709,221719.15499964403,276298.692523133,331815.4527873723,388284.8905341765,445722.71092646476,504144.8735640114,563567.5965622156,624007.3606959186,685480.9136089282,748005.2740915432,811597.7364240037,876275.8747910811,942057.5477658836,1008960.9028650159,1077004.381176082,1146206.7220597821,1216586.967924436,1288164.4690788044,1360958.8886606796,1434990.2076439597,1510278.729926352,1586845.0874955333,1664710.2456797785,1743895.5084807426,1824422.5239912504,1906313.2898987355,1989590.1590775093,2074275.8452670518,2160393.4288431797,2247966.3626794373,2337018.4781018137,2427573.9909376577,2519657.5076620304,2613294.031638691],[-1942616.982689092,-1925486.5607979782,-1908041.7444882211,-1890277.2213072595,-1872187.5915732556,-1853767.366963593,-1835010.9690809625,-1815912.727996446,-1796466.8807693683,-1776667.56994301,-1756508.8420169672,-1735984.6458944033,-1715088.8313046556,-1693815.1472006699,-1672157.2401307349,-1650108.6525838508,-1627662.8213093122,-1604813.0756086812,-1581552.635600806,-1557874.6104590122,-1533771.996620118,-1509237.6759644735,-1484264.4139676553,-1458844.8578217735,-1432971.5345272191,-1406636.848953726,-1379833.0818705994,-1352552.387944962,-1324786.7937089058,-1296528.195493307,-1267768.3573289937,-1238498.9088143758,-1208711.342948995,-1178397.0139321084,-1147547.1349269722,-1116152.7757885135,-1084204.8607551432,-1051694.166103705,-1018611.3177669784,-984946.7889126921,-950690.897484919,-915833.8037052178,-880365.5075343284,-844275.846093378,-807554.4910439518,-770190.9459258192,-732174.5434534326,-693494.4427679647,-654139.6266463008,-614098.8986652391,-573360.8803207092,-531914.0081003644,-489746.53051081067,-446846.5050561549,-403201.79516903264,-358800.0670926091,-313628.78671302367,-267675.21634064894,-220926.41144152405,-173369.21731524635,-124990.26572067058,-75775.97144775558,-25712.528834934812,25214.091769763734,77018.14760231366,129714.12713513477,183316.75378687866,237840.98969139904,293302.0395265571,349715.35440484993,407096.6358241243,465461.8396830186,524827.1803592918,585209.1348532937,646624.4469971,709090.1317316275,772623.4794496875,837242.0604102211,902963.7292216136,969806.6293965331,1037789.1979789739,1106930.1702459245,1177248.5844815169,1248763.7868293514,1321495.4362207232,1395463.509381217,1470688.3059179867,1547190.453485454,1624990.91303549,1704110.9841497568,1784572.310457022,1866396.8851361172,1949607.0565077579,2034225.533712362,2120275.39248069,2207780.080994809,2296763.4258422772,2387249.6380645554,2479263.319302899,2572829.4680387764],[-1978614.4870338873,-1961519.2106989464,-1944109.7741500596,-1926380.8668757244,-1908327.0911534126,-1889942.9606382127,-1871222.8989291124,-1852161.2381122888,-1832752.2172811255,-1812989.9810321305,-1792868.5779374994,-1772381.9589925425,-1751523.9760385116,-1730288.3801602412,-1708668.8200580429,-1686658.84039334,-1664251.8801084184,-1641441.2707186139,-1618220.2345775648,-1594581.8831145929,-1570519.2150439539,-1546025.1145450717,-1521092.3494144941,-1495713.5691873936,-1469881.303229552,-1443587.9587986378,-1416825.8190746822,-1389587.0411585653,-1361863.6540393948,-1333647.5565285352,-1304930.5151610686,-1275704.162063622,-1245959.9927882017,-1215689.3641109904,-1184883.4917969147,-1153533.4483275567,-1121630.160593221,-1089164.4075481568,-1056126.8178283125,-1022507.8673306219,-988297.8767546723,-953487.0091041108,-918065.2671486107,-882022.4908453682,-845348.3547194516,-808032.3652018057,-770063.8579260209,-731431.9949807203,-692125.7621187661,-652133.9659218164,-611445.2309198263,-570047.9966639248,-527930.514753974,-485080.84581742436,-441486.85644063493,-397136.2160512237,-352016.3937508003,-306114.6550965747,-259418.05883309804,-211913.45357048837,-163587.4744104785,-114426.53951858776,-64416.84664183948,-13544.369570215698,38205.14545657905,90846.18340120837,144393.46413813718,198861.94622224662,254266.83071726048,310623.5650860192,367947.8471408081,426255.6290584509,485563.12145820167,545886.7975448961,607243.397317653,669649.9318465996,733123.6876155571,797682.2309358367,863343.4124291651,930125.3715821304,998046.5413727523,1067125.6529718009,1137381.740516474,1208834.145962311,1281502.524010966,1355406.8471164056,1430567.4105716795,1507004.8376742369,1584740.0849756165,1663794.4476133361,1744189.564727739,1825947.4249644987,1909090.3720659665,1993641.1105484609,2079622.711472509,2167058.6183032375,2255972.652864118,2346389.0213848557,2438332.3206467694,2531827.544222703],[-2015115.3706155894,-1998055.5537888021,-1980681.8126509679,-1962988.8386434931,-1944971.236015088,-1926623.5204105375,-1907940.1174372768,-1888915.3612087753,-1869543.4928647876,-1849818.6590674792,-1829734.910474147,-1809286.2001848903,-1788466.3821655929,-1767269.2096457758,-1745688.3334907538,-1723717.300547378,-1701349.5519640804,-1678578.4214832305,-1655397.1337066104,-1631798.802333011,-1607776.4283676988,-1583322.8983029008,-1558430.9822699132,-1533093.332160895,-1507302.4797210528,-1481050.8346101653,-1454330.6824332497,-1427134.1827393193,-1399453.3669890019,-1371280.1364888023,-1342606.2602927787,-1313423.3730706745,-1283722.972941923,-1253496.419274781,-1222734.9304510814,-1191429.5815944276,-1159571.3022625237,-1127150.874102656,-1094158.9284696989,-1060585.9440056938,-1026422.2441817522,-991657.9947997788,-956283.2014546813,-920287.7069561514,-883661.1887093056,-846393.1560529671,-808472.9475567429,-769889.7282736646,-730632.4869497279,-690690.0331887319,-650050.9945720597,-608703.8137319041,-566636.7453791033,-523837.85328230914,-480295.00719965156,-435995.87976135174,-390927.943302826,-345078.46664660564,-298434.5118343858,-250982.93080561608,-202710.3620238686,-153603.22704933863,-103647.72705690842,-52829.83929797774,-1135.3135075457394,51450.33174752258,104941.81277755974,159354.08453834476,214702.3444591784,271002.0363327628,328268.8542652107,386518.7466906756,445767.9204489463,506032.84492813516,567330.256273054,629677.1616615043,693090.8436465818,757588.86457005,823189.0710448804,889909.598509267,957768.8758527911,1026785.6301173079,1096978.8912701928,1168367.9970558533,1240972.5979230353,1314812.6620305846,1389908.4803337865,1466280.6717492254,1543950.1884040046,1622938.320967183,1703266.7040661834,1784957.3217887827,1868032.513273995,1952514.97838892,2038427.7834984334,2125794.36732514,2214638.54690258,2304984.523622642,2396856.8893804336,2490280.632813676],[-2052128.6867333832,-2035104.6623818916,-2017766.9515654906,-2000110.2476937561,-1982129.1570019978,-1963818.1971402995,-1945171.7957401893,-1926184.28895829,-1906849.9199967352,-1887162.8375994705,-1867117.0945252206,-1846706.6459953033,-1825925.3481168647,-1804766.956280912,-1783225.1235346466,-1761293.3989274849,-1738965.2258312586,-1716233.9402328052,-1693092.7689996378,-1669534.8281178002,-1645553.120901515,-1621140.5361738915,-1596289.846419309,-1570993.705905372,-1545244.6487753529,-1519035.0871099506,-1492357.308958214,-1465203.4763364852,-1437565.6231962938,-1409435.6533588506,-1380805.3384169894,-1351666.315603496,-1322010.0856254273,-1291828.0104634413,-1261111.311136843,-1229851.0654320251,-1198038.205595064,-1165663.5159874407,-1132717.6307043494,-1099191.0311545278,-1065074.0436024345,-1030356.8366702325,-995029.4188003261,-959081.6356773628,-922503.167609198,-885283.5268654614,-847412.0549749047,-808877.9199783592,-769670.1136385542,-729777.448605265,-689188.5555354194,-647891.8801665935,-605875.6803451674,-563128.0230058278,-519636.78110352205,-475389.63049642276,-430374.04677933315,-384577.30206593033,-337986.46172117535,-290588.381040202,-242369.70187505567,-193316.8492075042,-143416.0276674945,-92653.21799527388,-41014.17344886856,11515.58384730434,64950.766611124855,119306.32617068989,174597.45629198942,230839.5970684234,288048.43887040764,346239.9263596656,405430.26256641885,465635.91303166933,526873.610015193,589160.3567714421,652513.4318914246,716950.3937156796,782489.0848164279,849147.6365511157,916944.4736882173,985898.3191076657,1056028.1985736564,1127353.4455857677,1199893.7063057688,1273668.9445629856,1348699.4469401725,1425005.8279378954,1502609.035223327,1581530.3549612295,1661791.4172298103,1743414.2015223433,1826421.0423374963,1910834.6348557249,1996678.0407084702,2083974.6938376082,2172748.406448148,2263023.375055173,2354824.186628143,2448175.8248298024],[-2089662.9696855028,-2072675.084023811,-2055373.7518472727,-2037753.6685510646,-2019809.442374757,-2001535.5929915213,-1982926.5500749943,-1963976.6518431103,-1944680.1435787412,-1925031.1761261735,-1905023.8043642652,-1884651.9856545038,-1863909.5782643931,-1842790.3397657333,-1821287.9254071524,-1799395.8864603152,-1777107.6685403623,-1754416.6098987085,-1731315.939688945,-1707798.7762048643,-1683858.1250904005,-1659486.8775205098,-1634677.808353796,-1609423.5742547035,-1583716.711786194,-1557549.6354717228,-1530914.6358264002,-1503803.8773562005,-1476209.3965260785,-1448123.0996947046,-1419536.7610166627,-1390442.020311024,-1360830.380895936,-1330693.2073881766,-1300021.7234684986,-1268807.0096102804,-1237040.0007724431,-1204711.4840554039,-1171812.096319654,-1138332.321765874,-1104262.4894773844,-1069592.770922414,-1034313.1774168885,-998413.5575467665,-961883.594549248,-924712.8036516542,-886890.5293690688,-848405.9427575837,-809248.0386244259,-769405.6326934136,-728867.3587253541,-687621.6655918607,-645656.8143038349,-602960.8749912581,-559521.7238354911,-515327.0399525557,-470364.302226861,-424620.78609378496,-378083.5602724198,-330739.48344480665,-282575.20088299084,-233577.14102222864,-183731.5119797252,-133024.29801717494,-81441.25594857009,-28967.91148916725,24410.444452843163,78708.7595438296,133942.22385209752,190126.27373697003,247276.59579806542,305409.1308891326,364540.07819490926,424685.8993730019,485863.32276145136,548089.3476542518,611381.2486428199,675756.5800285684,741233.1803045995,807829.1767088906,875562.9898496089,944453.3384051379,1014519.2438964732,1085780.03553783,1158255.3551630904,1231965.1622306579,1306929.7389088804,1383169.6952400208,1460705.9743885305,1539559.8579715784,1619752.9714744492,1701307.2897516578,1784245.1426168163,1868589.2205185122,1954362.580309019,2041588.6511032484,2130291.240230934,2220494.5392830516,2312223.1302556167,2405501.9917880567],[-2127722.9825794618,-2110771.559249661,-2093506.9310978297,-2075923.7955168523,-2058016.7627627584,-2039780.354544089,-2021209.002588994,-2002297.0471892275,-1983038.735720981,-1963428.2211415928,-1943459.5604628634,-1923126.713199346,-1902423.539791934,-1881343.8000063312,-1859881.1513058117,-1838029.1471976407,-1815781.2355537121,-1793130.7569035667,-1770070.9427005358,-1746594.9135599572,-1722695.6774693492,-1698366.127969533,-1673599.0423074586,-1648387.0795586498,-1622722.778720101,-1596598.556772499,-1570006.706711632,-1542939.3955477974,-1515388.662274224,-1487346.4158019875,-1458804.432862508,-1429754.355876362,-1400187.6907881047,-1370095.8048661049,-1339469.924468138,-1308301.1327703134,-1276580.3674602294,-1244298.4183932163,-1211445.9252111884,-1178013.3749230187,-1143991.0994472797,-1109369.2731147725,-1074137.9101316156,-1038286.8620018815,-1001805.8149090474,-964684.2870552177,-926911.6259590304,-888477.0057092081,-849369.424174997,-809577.7001719195,-769090.4705824791,-727896.1874302877,-685983.1149088866,-643339.3263618187,-599952.701215284,-555810.9218617161,-510901.47049387917,-465211.6258877455,-418728.46013560146,-371438.83532559266,-323329.4001691588,-274386.58657457447,-224596.6061660326,-173945.44674654212,-122418.8687060955,-70002.40137101617,-16681.339295976795,37559.26150313113,92734.58737616986,148860.07092748722,205951.39496476902,264024.49651254434,323095.57088861195,383181.0758455121,444297.7357777348,506462.5459968075,569692.777072344,634005.9792442219,699419.9869038528,765952.9231469356,833623.2043983713,902449.5451118159,972450.9625416603,1043646.7815932161,1116056.6397486455,1189700.4920714549,1264598.6162914098,1340771.6179680289,1418240.4357384294,1497026.3466473543,1577150.971562108,1658636.2806731923,1741504.5990836653,1825778.6124845338,1911481.3729229672,1998636.304660758,2087267.2101259641,2177398.2759588393,2269054.0791551033,2362259.5933037745],[-2166316.3965482395,-2149401.76247436,-2132174.16702886,-2114628.3096145433,-2096758.802516167,-2078560.1694900522,-2060026.8443313309,-2041153.1694181592,-2021933.3942327606,-2002361.6738582917,-1982432.0674524147,-1962138.536695741,-1941474.9442156649,-1920435.0519850121,-1899012.5196950233,-1877200.9031019995,-1854993.6523481198,-1832384.1102547336,-1809365.5105886383,-1785930.9763006212,-1762073.5177357702,-1737786.0308148724,-1713061.2951874498,-1687891.9723544505,-1662270.6037613612,-1636189.608860674,-1609641.283143542,-1582617.7961394303,-1555111.1893847494,-1527113.3743580696,-1498616.1303828345,-1469611.1024964242,-1440089.79928526,-1410043.5906848712,-1379463.705745756,-1348341.2303625736,-1316667.1049675718,-1284432.1221870917,-1251626.9244606895,-1218242.0016217981,-1184267.68844074,-1149694.1621275316,-1114511.4397952766,-1078709.3758830419,-1042277.6595376763,-1005205.811953262,-967483.1836693222,-929098.9518247237,-890042.1173683633,-850301.5022252486,-809865.746417522,-768723.3051388967,-726862.4457837599,-684271.2449276438,-640937.5852601202,-596849.1524687591,-551993.4320735298,-506357.7062100321,-459929.0503629041,-412694.3300457918,-364640.19742901623,-315753.0879135425,-266019.2166503966,-215424.57500392757,-163954.92696031975,-111595.8054773002,-58332.50877656182,-4150.096576963551,50966.61373203853,107033.05097873881,164064.89415768255,222078.07644273108,281088.7892631367,341113.4864446046,402168.8884160924,464271.98648446705,527440.0471752035,591690.6166441008,657041.5251581324,723510.8916477896,791117.1283315443,859878.9454150111,929815.3558624377,1000945.6802464649,1073289.5516736312,1146866.9207883775,1221698.0608575041,1297803.5729332361,1375204.391100559,1453921.7878068024,1533977.3792760684,1615393.1310093915,1698191.3633736242,1782394.7572763148,1868026.3599334313,1955109.590727291,2043668.2471577246,2133726.5108874715,2225308.953884905,2318440.5446613873],[-2205450.996980545,-2188573.482404431,-2171383.2516797087,-2153875.006231368,-2136043.360385284,-2117882.83995803,-2099387.8808242856,-2080552.8274613165,-2061371.93147013,-2041839.3500726142,-2021949.1445852735,-2001695.2788678699,-1981071.6177475045,-1960071.9254174454,-1938689.8638103076,-1916918.9909449646,-1894752.7592475205,-1872184.5138448656,-1849207.4908311649,-1825814.815506609,-1801999.500587953,-1777754.4443901542,-1753072.428979643,-1727946.118297221,-1702368.0562514311,-1676330.664781262,-1649826.2418880204,-1622846.9596352684,-1595384.8621176502,-1567431.8633963964,-1538979.7454022793,-1510020.1558049107,-1480544.605848107,-1450544.4681502618,-1420010.9744703912,-1388935.2134376606,-1357308.1282450315,-1325120.5143060691,-1292363.0168743148,-1259026.128624239,-1225100.1871945295,-1190575.3726911778,-1155441.705151158,-1119689.0419655899,-1083307.0752618536,-1046285.3292433289,-1008613.1574879135,-970279.7402022146,-931274.0814325521,-891585.0062313345,-851201.1577783888,-810110.994455744,-768302.7868770366,-725764.6148682646,-682484.364401075,-638449.7244770038,-593648.1839622282,-548067.0283711189,-501693.3366000145,-454513.97760748584,-406515.6070424351,-357684.66381831,-308007.3666329649,-257469.71043222584,-206057.4628187837,-153756.16040229192,-100551.10509210825,-46427.360330919735,8630.252731467132,64637.15912611177,121609.03401301056,179561.80669381283,238511.6646874561,298475.05787087884,359468.70268535754,421509.58641082793,484614.97150608804,548802.4000201477,614089.6980726612,680494.980405814,748036.6550083691,816733.4278143891,886604.3074742942,957668.6102041481,1029945.9647107683,1103456.317195191,1178219.9364367644,1254257.4189556506,1331589.6942597725,1410238.0301738568,1490224.0382534242,1571569.679284452,1654297.2688717432,1738429.4831133317,1823989.3643677062,1911000.3271112018,1999486.1638886826,2089471.0513583883,2180979.556434155,2274036.6425222177],[-2245134.6852415404,-2228294.6237591277,-2211142.0931388726,-2193671.7968402957,-2175878.351242867,-2157756.2842359515,-2139300.0337864906,-2120503.946483597,-2101362.27605996,-2081869.1818891668,-2062018.72745964,-2041804.878823513,-2021221.5030208726,-2000262.3664788285,-1978921.1333849228,-1957191.3640341822,-1935066.5131504158,-1912539.9281798874,-1889604.8475580916,-1866254.398948691,-1842481.597454319,-1818279.3437984216,-1793640.4224787357,-1768557.4998904883,-1743023.1224199373,-1717029.7145073477,-1690569.5766790968,-1663634.8835478644,-1636217.681781687,-1608309.8880397533,-1579903.2868755343,-1550989.528606432,-1521560.1271493384,-1491606.4578212565,-1461119.7551056796,-1430091.1103823124,-1398511.4696210385,-1366371.6310389473,-1333662.2427200517,-1300373.8001964367,-1266496.6439918894,-1232020.9571252586,-1196936.762574391,-1161233.9206996271,-1124902.1266261782,-1087930.907584154,-1050309.6202074303,-1012027.4477880844,-973073.3974877442,-933436.2975042514,-893104.7941933321,-852067.3491436578,-810312.2362065874,-767827.5384772718,-724601.145228222,-680620.7487939075,-635873.8414058387,-590347.7119764541,-544029.4428332285,-496905.90639931103,-448963.76182192285,-400189.4515469768,-350569.1978392536,-300088.9992463505,-248734.62700794404,-196491.6214062241,-143345.288059053,-89280.69415391935,-34282.664622139186,21664.22174874693,78575.63626266224,136467.5043273149,195356.00952945603,255257.59777514124,316188.9814957185,378167.14392168866,441209.3434225363,505333.11791763315,570556.2893562457,636896.9682690063,704373.5583915301,773004.7613626611,842809.5814951458,913807.3306244188,986017.63303329,1059460.4304549354,1134155.9871565318,1210124.8951013023,1287388.0791949653,1365966.8026143261,1445882.6722207735,1527157.6440594243,1609814.0289470484,1693874.4981459402,1779362.0891306382,1866300.2114448198,1954712.6526514613,2044623.584377143,2136057.568453793,2229039.5631548814],[-2285378.006938401,-2268575.7588713644,-2251461.2908199662,-2234029.308295638,-2216274.429749002,-2198191.185159958,-2179774.01460558,-2161017.2668049177,-2141915.1976405936,-2122461.968656338,-2102651.6455311207,-2082478.196528238,-2061935.490919771,-2041017.297385852,-2019717.282388312,-1998029.0085179296,-1975945.9328159606,-1953461.4050680103,-1930568.6660710808,-1907260.8458726804,-1883530.961981878,-1859371.9175513827,-1834776.4995312595,-1809737.376792332,-1784247.0982200021,-1758298.0907774556,-1731882.6575379872,-1704992.9756854693,-1677621.0944836556,-1649758.933212176,-1621398.2790700118,-1592530.7850453085,-1563147.967751252,-1533241.2052269676,-1502801.734704083,-1471820.650336823,-1440288.9008961595,-1408197.2874271765,-1375536.460869019,-1342296.9196363646,-1308469.0071632997,-1274042.9094069903,-1239008.6523118983,-1203356.0992335677,-1167074.9483213043,-1130154.7298585172,-1092584.8035618598,-1054354.355835971,-1015452.3969851625,-975867.7583803902,-935589.089581206,-894604.8554111817,-852903.3329879497,-810472.608704634,-767300.5751637593,-723374.9280621889,-678683.1630265042,-633212.5723972982,-586950.2419635747,-539883.0476437495,-491997.65211441973,-443280.50138530275,-393717.821319825,-343295.6140993796,-291999.6546329898,-239815.48690810427,-186728.42028414411,-132723.52572682826,-77785.6319827442,-21899.321692174766,34951.07255825028,92781.47224743571,151608.05699467612,211447.2686995552,272315.81574748224,334230.6772831436,397209.1075499221,461268.6403003847,526427.0932758404,592702.5727573871,660113.478189087,728678.506875793,798416.6587533229,869347.2412368469,941489.8741450766,1014864.4947028235,1089491.3626241335,1165391.0652738875,1242584.522913746,1321092.994030287,1400938.0807479834,1482141.7343278802,1564726.2607549778,1648714.3264116524,1734128.9638438202,1820993.5776173347,1909331.9502676162,1999168.2483434468,2090527.0285481364,2183433.243975241],[-2326188.348137528,-2309424.270285241,-2292348.2236590157,-2274954.9158382476,-2257238.967360823,-2239194.9103135066,-2220817.186899958,-2202100.147985791,-2183038.0516203437,-2163625.0615344048,-2143855.245614604,-2123722.5743526584,-2103220.919270058,-2082344.0513175668,-2061085.6392490503,-2039439.2479689738,-2017398.336854132,-1994956.2580477714,-1972106.2547268171,-1948841.45934126,-1925154.891825427,-1901039.4577802739,-1876487.9466273522,-1851493.0297324515,-1826047.258499664,-1800143.0624348114,-1773772.7471780598,-1746928.4925045744,-1719602.3502941255,-1691786.2424673322,-1663471.9588893657,-1634651.1552400426,-1605315.3508499824,-1575455.9265017132,-1545064.122196581,-1514131.0348850284,-1482647.6161611076,-1450604.6699200573,-1417992.8499785573,-1384802.6576564861,-1351024.4393210658,-1316648.383890843,-1281664.5203001867,-1246062.714923372,-1209832.6689575594,-1172963.9157634275,-1135445.818164628,-1097267.5657028696,-1058418.171849844,-1018886.4711745731,-978661.1164656584,-937730.5758070191,-896083.1296082875,-853706.8675865387,-810589.6857005376,-766719.2830360304,-722083.1586414715,-676668.6083126073,-630462.7213272867,-583452.3771267827,-535624.2419449165,-486964.7653833744,-437460.1769326292,-387096.48243660526,-335859.46050267154,-283734.6588528445,-230707.39061769983,-176762.73057107953,-121885.5113050756,-66060.31934319716,-9271.491193474736,48496.89066304872,107261.00184677215,167037.28021900868,227842.4300871105,289693.42647752585,352607.519474769,416602.23863146827,481695.3974475325,547905.0979207302,615249.7351694265,683748.002129972,753418.8943264093,824281.7147184219,896356.0786250364,969661.9187267539,1044219.490148155,1120049.375619072,1197172.4907200364,1275610.089209904,1355383.768438376,1436515.4748441447,1519027.5095418086,1602942.5339947306,1688283.5757807344,1775074.0344479713,1863337.6874639834,1953098.6962590124,2044381.612366599,2137211.383658774],[-2367572.2001804803,-2350846.63647362,-2333809.357037993,-2316455.0715335836,-2298778.402598319,-2280773.884438674,-2262435.96139786,-2243758.986501119,-2224737.2199776643,-2205364.827758643,-2185635.8799517127,-2165544.3492905134,-2145084.1095595444,-2124248.933993914,-2103032.4936533477,-2081428.355769941,-2059429.9820700795,-2037030.727068796,-2014223.8363372555,-1991002.4447423148,-1967359.5746580658,-1943288.1341483511,-1918780.915120966,-1893830.591451546,-1868429.7170778613,-1842570.7240635187,-1816245.920630751,-1789447.4891614201,-1762167.4841668184,-1734397.8302241634,-1706130.3198805628,-1677356.6115234145,-1648068.227216757,-1618256.5505026923,-1587912.8241685408,-1557028.1479774094,-1525593.4763629632,-1493599.6160872828,-1461037.2238614187,-1427896.8039274118,-1394168.7056027376,-1359843.1207845723,-1324910.0814145813,-1289359.4569033124,-1253180.9515134576,-1216364.1017008182,-1178898.2734140307,-1140772.6593499691,-1101976.2761660246,-1062497.9616477098,-1022326.3718312699,-981449.978079733,-939857.0641136151,-897535.7229930162,-854473.8540521865,-810659.1597851207,-766079.1426816876,-720721.1020125239,-674572.1305642053,-627619.1113208979,-579848.7140938756,-531247.3920971821,-481801.37846889533,-431496.68273621006,-380319.0872257636,-328254.1434152229,-275287.16822758457,-221403.240266277,-166587.19599050656,-110823.6258288417,-54096.87023272645,3608.9843346779235,62310.1094724834,122022.93898116006,182764.1730672503,244550.78261598805,307400.0135298688,371329.3911382342,436356.72467603907,502500.1118339617,569777.9433807218,638208.9078599783,707811.9963596398,778606.5073593166,850612.0516535644,923848.5573535315,998336.2749691433,1074095.7825697246,1151147.9910290055,1229514.1493522525,1309215.850088275,1390275.0348270894,1472713.9997863057,1556555.4014835078,1641822.2625013832,1728537.9773431104,1816726.31838087,1906411.441898616,1997617.8942321455,2090370.61800374]],"type":"heatmap","xaxis":"x","yaxis":"y","hovertemplate":"Home Price Growth: %{x}\u003cbr\u003eInvestment Growth: %{y}\u003cbr\u003eOpportunity: %{z}\u003cextra\u003e\u003c\u002fextra\u003e"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Home Price Growth"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"autorange":"reversed","title":{"text":"Investment Growth"}},"coloraxis":{"colorbar":{"title":{"text":"Opportunity"}},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"title":{"text":"Opportunity Surface"}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

## Simulate
We can also simulate market conditions.

In [16]:
n = 10_000

In [17]:
simulated_investment_rate = np.random.beta(10, 5, n) * (0.1 - 0.03) + 0.03
render_plotly_html(px.histogram(simulated_investment_rate, title="Investment Rate"))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="1a7b1286-3e0c-4f8c-865a-b587cd3a9e3e" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("1a7b1286-3e0c-4f8c-865a-b587cd3a9e3e")) {                    Plotly.newPlot(                        "1a7b1286-3e0c-4f8c-865a-b587cd3a9e3e",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[0.07579253587650053,0.08278500750666953,0.07839649400655815,0.08850894791598687,0.07742135738583653,0.08697776353236036,0.0838830620921951,0.06522071391058541,0.07880786605330051,0.07763351247693023,0.06740389886941127,0.08264861000108006,0.07628860924669689,0.08566976035524637,0.07156193760655838,0.07648309278416568,0.0741398093397902,0.0913266670561555,0.07740417487280185,0.06839579086254025,0.07008351462189097,0.07353868973669064,0.06818374379235921,0.0768023165539117,0.08667322420149101,0.0667246425846655,0.07821820895561718,0.07348336745739378,0.07757355856329999,0.07956044324204598,0.07440829118127398,0.07386276303143346,0.08415382142592773,0.07171942753975649,0.06737452229472804,0.0709465907408291,0.06216093150367573,0.05980144199152392,0.07747037037984433,0.0783885196666802,0.07029550481680708,0.08140017601437685,0.08758121088280055,0.06489603211280404,0.0710874563971158,0.07734409552154606,0.08131667086241703,0.08533125777612255,0.07600198137980912,0.082843889539084,0.08337567317283481,0.0821214044961418,0.0764716096222125,0.07908767136083325,0.06471565450663633,0.08000059091232319,0.07887916625856149,0.07807567971814772,0.08277807530282072,0.07211743752389602,0.07232324415613814,0.08059295870079983,0.08638293131462998,0.07733566990912283,0.09050059808985762,0.07620485449096208,0.08322055550810445,0.077740006931894,0.08771063897486076,0.08543583982133982,0.08117049795949209,0.07943363821669491,0.08368054235447844,0.07342778621275795,0.08583802911888658,0.0768983978984675,0.08233174475414222,0.07960963148092674,0.0855982169301846,0.0752756466650226,0.08703455495121909,0.0723062678184816,0.07552829925629023,0.06077759459695388,0.0702755126042092,0.08658115364747321,0.07236629749145929,0.06537085777587506,0.06986907806845788,0.08743146924993414,0.0689388469317525,0.0890095283799187,0.0710148528418679,0.07349659426676088,0.06752659368282868,0.08386201723573095,0.06660398198751677,0.05987539816408752,0.0917498767938784,0.07786060454520978,0.07358711977497925,0.07891050306428535,0.06354659471062428,0.0817548951155938,0.07002174416685615,0.08363425029575741,0.06907626693856746,0.08527101660381842,0.0776934007057781,0.075039225325963,0.07489964265315216,0.0692195329817204,0.0672766348565232,0.07862522748119234,0.0814047115776295,0.06185985717205719,0.09061740879029023,0.09161060217377362,0.08088954115380995,0.08031901760194976,0.07600706379856298,0.06959678035770533,0.08636670283094154,0.0660341747691918,0.08752186640378318,0.09664259075477913,0.08426822661586768,0.07209666202378565,0.07890876507509866,0.08613378342357098,0.07649344934114884,0.08152785174585113,0.08411056561828217,0.06980088432162188,0.08026545826587564,0.07450176080303396,0.07952677141450679,0.07055642292444843,0.06811533730538766,0.06725174071196663,0.06280834010733209,0.06915265427968476,0.08807172407826871,0.09025790088402522,0.08803851072209905,0.08775654401491939,0.07807439443086236,0.08633539018157808,0.05997351170989562,0.08099590699002529,0.07790392942729724,0.07636677732157868,0.07330347060075806,0.08781841631188414,0.07511312086873027,0.07843314764102069,0.08087827122770683,0.0761156855722672,0.07332174401250158,0.055063982688948146,0.07823189780121051,0.08877135867654326,0.07448785387842441,0.06811590758943185,0.07001984014318141,0.083191977871821,0.06653160747989981,0.08702961582718069,0.0722082353404691,0.08129350484221434,0.07969362607480948,0.0717798682096698,0.07261827047695418,0.07543930407076577,0.06640917803802393,0.07090644355817183,0.06774394808416102,0.06671303322943978,0.08129677964177617,0.06401881214778524,0.08732719045610726,0.06782837070951982,0.07629559137340014,0.05311271215440717,0.06874429339897192,0.07123709909873648,0.08347131405137598,0.07607665389146177,0.08475652731344333,0.07621667776682345,0.07906893714329538,0.06754039911181121,0.06310120068173211,0.08076795220152963,0.06794138623898066,0.0794567356879877,0.06276705011172516,0.07864790834509755,0.07481373271825079,0.0755590423259635,0.07077810353111953,0.06847007266431354,0.08154039078575062,0.08937843656186946,0.07846861861768217,0.07840991786269993,0.0876752867926579,0.07176447654090506,0.08259591068581575,0.07431834218163937,0.08026551676974195,0.07365496038052773,0.07675923977330007,0.0836794527026602,0.0671974001341783,0.08221299624025154,0.06965650418415616,0.0832488299941763,0.07595617042781833,0.08292233894488264,0.06974897481558494,0.06440374999085521,0.08219444150710281,0.06468294273649194,0.08642348401863692,0.06287580297758952,0.06724240121188714,0.07685384184045924,0.06517712417432389,0.07639263169371863,0.08227446213645384,0.06127124079951723,0.07574572756713176,0.08214136859686343,0.07537200876086572,0.06768066558865499,0.075221088928517,0.06033747301108532,0.06596625372739445,0.07746374414669585,0.08765310187766823,0.07994002534456261,0.062529692428511,0.06548007185554125,0.08027741275001116,0.07598459181289814,0.07911115596783108,0.06327168198617907,0.06914417966618461,0.0634669144676395,0.08841431975017969,0.07789484829524128,0.07939473480466522,0.08732014469453567,0.0705976604127238,0.07790787978885837,0.08164437047065537,0.0880756484144231,0.08095851890131417,0.07478719937635976,0.06973269431373114,0.07376864557880636,0.07064584459950526,0.07446321278999743,0.07239197999587067,0.08452547438377797,0.07014992715841786,0.08541892560415729,0.0770975975769902,0.08287448533047749,0.0787037559973022,0.08006859362547884,0.07722619600289093,0.07219758266246246,0.07931339479777064,0.08518641661569798,0.07829569728317085,0.08309178463836042,0.0724114877897396,0.08522963764621921,0.07365134802965093,0.09059180737493899,0.08384222469307007,0.0625037910208601,0.06633528581145501,0.05854182347928555,0.07831346364268081,0.0767574645003287,0.080905055125565,0.07654458398640543,0.06598493343157306,0.0703287148708105,0.08291962625072528,0.08549476332032738,0.06439371422462084,0.07640293313498511,0.0873967333785787,0.06558559922123107,0.06805204069656542,0.07884492144071256,0.07560975772246274,0.06893027026360177,0.08494691501144278,0.06754846986341417,0.09119933692232546,0.07509915349328512,0.059106891436573675,0.0667035064936411,0.06856634679725007,0.06550895360477248,0.07432748400437639,0.07095054076788451,0.07482933406857416,0.09007649932628478,0.07960857346407321,0.08424098469797532,0.07211151467226613,0.08872281010451052,0.07902809652372407,0.06505925996199033,0.07804699145713778,0.07613220696068795,0.0795702216495087,0.06419560167763233,0.08391389149999762,0.07689580035347097,0.082205984460552,0.07614098837924503,0.06639721540548951,0.0746512471308308,0.07645493467624903,0.0871301983268548,0.08316527888489775,0.06222535112359038,0.07612975506636194,0.0701044436523897,0.06749608443206119,0.07518454658962068,0.08203150327120834,0.06857500009090187,0.08286528858840773,0.06574502742331165,0.06258144509405934,0.08048353136338227,0.0841126044472891,0.07984061643808779,0.08303580811869503,0.08476202155996962,0.06637338630722922,0.09364783816616917,0.07353313858267232,0.088673886376857,0.07378067182338968,0.07613749417496506,0.07602596630832598,0.07315553254776483,0.07190507115428506,0.06689678363488002,0.07482893553656511,0.07617166321058245,0.07496317613921995,0.07170788623205329,0.07383217192394576,0.07550826797026304,0.06665771715005378,0.07149428254255863,0.08960497813541743,0.07399652172248583,0.07375381810024989,0.0807047329411325,0.07490860617091971,0.07803065654528865,0.08060334279691653,0.071717837280317,0.08319989774997061,0.09493114953731831,0.07876760094732439,0.09006426830141523,0.0869230855199622,0.07012899657417435,0.09098013871522925,0.08032319683661121,0.07537477481192445,0.07637659500879436,0.07133567475689072,0.062224308133415354,0.07216332467376649,0.07224484438282727,0.0843439027669075,0.08964337878932695,0.06939461938139868,0.07452985964690298,0.09035367600983069,0.058904126565319806,0.0766848374508651,0.0888274001274441,0.06254573622696756,0.06637945107114149,0.07174025557896764,0.06717389039897058,0.08027287057233934,0.0807259004872872,0.07416875032825579,0.0644065185806329,0.07780153380852109,0.08041490107671492,0.06883016727398655,0.08795814918241641,0.08730118433052131,0.0670905602145161,0.07965554265595554,0.08025059610892439,0.06993224658900817,0.07060194376594667,0.05971507364073005,0.07247248110526636,0.07927698239800077,0.06560359637801844,0.08013883358892443,0.07650076855565466,0.0827841113244652,0.07380478345671498,0.06729211586064945,0.06350676830830472,0.08097797478280602,0.08736571957693895,0.0802474682281957,0.07667172094177181,0.08481789726009338,0.08443373563720827,0.06476604944463668,0.0781726985702981,0.05715412063963199,0.07170755656289793,0.0726326266429029,0.07101502157420758,0.05686315068278221,0.08136950815240229,0.08581046505428797,0.08127475489983393,0.08818352196759455,0.08379540555410017,0.09344546973655418,0.07137978730179145,0.08020711889245848,0.08506171966806142,0.08630550922994762,0.07568740391010337,0.06771784371325038,0.08032310236335004,0.08542956930814623,0.07296892034512885,0.08682643136947168,0.08907814473045186,0.08174579886807955,0.07692910741067857,0.07157174771292157,0.06687327537680376,0.08433066187826707,0.07996213336265638,0.07563209905596885,0.0713434216416789,0.0676211125830243,0.07834202747702929,0.08337694211979739,0.08466933768602855,0.07383506556452955,0.09148606499539051,0.07162807774825536,0.06609084693481568,0.07819070145815486,0.0667749486812956,0.07165783823068975,0.0857491097233047,0.06103407156293554,0.08978853177188376,0.08871163911986299,0.07843061843660618,0.07554379991924676,0.09110463840629177,0.07292487837583043,0.07564034037892386,0.06869598703996399,0.065488691517578,0.09043249911753298,0.061108128556171676,0.08331719583129527,0.08503068133000873,0.08815458665769157,0.06830811121045482,0.08961050443192625,0.08476830808306786,0.0858118114967401,0.0744578773770602,0.07571845525933624,0.07541195428126732,0.07217703830235866,0.08086231175235786,0.08333309851178647,0.0792007214770494,0.08230630607700695,0.06923239890886485,0.08559178691598075,0.07913750121082863,0.07394711517063651,0.06608822793421067,0.08667273329179484,0.0835714253478973,0.08772870009509848,0.0785416086328253,0.078421503747902,0.07083741673295152,0.07502505503714427,0.0701722657963284,0.07322794047225964,0.07893121638451779,0.06822682153790185,0.08544354513411512,0.06912989041493953,0.05673236772306647,0.07384293020748947,0.05843245225063415,0.08934027680097548,0.07813056618347698,0.08208757049549706,0.06345725521004884,0.07275074793242783,0.06989778704317662,0.07755850717410971,0.06515161556398094,0.07046237202708083,0.07389870141980787,0.09070843439938153,0.09229815119237064,0.05951294686650921,0.0620498656044163,0.08559299833964677,0.08026848911668033,0.07158362634841062,0.08703833642920716,0.06600548115988747,0.07595196461189654,0.0731671386188231,0.0710869740378117,0.08904477935965607,0.08152900757781911,0.09181346868378176,0.07684611356657074,0.07645705704778631,0.07362153286605866,0.08697505872239263,0.05901129906850393,0.07755577029260752,0.07454943572177836,0.08795731531625614,0.08584064306445582,0.08443493633100316,0.07379415816683851,0.07382450648238104,0.07265626133954259,0.0684714696745801,0.0965012067101325,0.07550518385724239,0.07511365651818011,0.08512883026238438,0.07447203188931781,0.08066168833723628,0.08123899931718075,0.0754943116915607,0.08633780120176235,0.08304577607978136,0.08550407711085539,0.07216046360899268,0.08065447774724274,0.07272694601375822,0.07104618122805587,0.07883090147249329,0.07296528071255051,0.07115564853585933,0.07448863879859186,0.08417612152007084,0.07056761294838002,0.07892599460074312,0.0789154041038044,0.08328790357005163,0.08051453554431884,0.0724369673767887,0.0586341587017963,0.08142257009407654,0.07430502878011286,0.07772151716521569,0.074756353721828,0.07776128109004557,0.08611735350732089,0.06748445536745074,0.061969131327213815,0.07283124473938227,0.0846511147626803,0.07252056023877525,0.07090029999781147,0.08543656212563872,0.0731766901636529,0.07674753869853064,0.08007932070229308,0.08626030221289638,0.07195766600151002,0.06821669148665974,0.07383185067243236,0.0858499501492407,0.07625733075957868,0.07035940111594688,0.08264278025848742,0.06927977309822946,0.07055426489824826,0.06846449892452758,0.07278309766556176,0.09447715473298379,0.07136354238214054,0.08818808195339442,0.09016068600896143,0.0884057217728732,0.058349513374139364,0.0839335309626522,0.07464389324139051,0.07674305997696534,0.07144544824921222,0.07214315111949124,0.08289505651647913,0.06578593610077435,0.07863116576230626,0.07056854901659554,0.09266740929683477,0.08708550471076129,0.0785808771231376,0.08494686373007614,0.0859194816606095,0.08060325835950877,0.09221042406653215,0.0774751565190307,0.07763002088086608,0.07746767197505461,0.07786935718767465,0.0790916631819748,0.06855315086217231,0.06675273347266941,0.08421078262476903,0.06886834418807597,0.0775354195292531,0.08025154471028241,0.07199962402406082,0.0658594193430524,0.07013257706477172,0.06781429395003646,0.074041143378899,0.07923936599672943,0.07229121008417701,0.08558210784313403,0.07259149454696398,0.08903440168896393,0.07527256444310738,0.06313373131478515,0.07267432848733658,0.07896511284711025,0.0855390953941249,0.07561361039996875,0.08689829290097438,0.08370803203528948,0.08450728230027452,0.0857283080355199,0.06844107528238079,0.06969861508106955,0.08932377135630379,0.08838156894240501,0.08120757124594148,0.07042334113068946,0.08480680980047586,0.08354261427851147,0.08691836967194021,0.07582885338395802,0.08213117298433817,0.06586729613214985,0.06309062842071872,0.07853981057741083,0.08198910760162939,0.08271372457792363,0.08630149265459866,0.08742337077931842,0.08212866715717686,0.07970580803545949,0.06673382196824157,0.08963675450896727,0.07586701275458649,0.09028610356574679,0.07728366073628973,0.06870660793729078,0.08070721668057548,0.09193101637501627,0.06500784819671804,0.07563021101358001,0.07998744468512631,0.08412916153719274,0.08240580250658461,0.0870304931767932,0.08411039286432945,0.07135172124422814,0.08766464638544183,0.06873567959585931,0.08024814559977217,0.08341759676791524,0.09459040100105551,0.05935319032389235,0.08991072255312617,0.08804931658063214,0.07538239386551347,0.07635068339543168,0.07700424185128364,0.07334153067563565,0.06554709356075453,0.07236110214074784,0.06950198566870637,0.057842926833958114,0.08434117438741207,0.055517656511906384,0.08319909917288973,0.07998145435562284,0.0758140220512796,0.07905912700977881,0.08192020060029907,0.08144913353349968,0.06836165809224237,0.08187940937139854,0.06440810693674251,0.07422637560638708,0.06590226361147977,0.0883327530160024,0.08010143944821535,0.07294681545693762,0.05755629506679483,0.06518222020819073,0.09170696496316198,0.08426359538004666,0.07899024237693786,0.07853582076389762,0.06559409916362102,0.054185771729226374,0.07801910741937144,0.07409532199950468,0.07069641625709233,0.0870389737982252,0.08158023167342007,0.0680569969634592,0.06208204031191364,0.07923115954901194,0.08041719304558771,0.06829446297081833,0.07636337946636967,0.08415471990099671,0.07157004365170966,0.0876460302419026,0.08619314705050908,0.059638012108444396,0.07792698306100039,0.07170446717514872,0.08943020514335759,0.08157081164067329,0.07689602822120423,0.08732832124485548,0.07630263651840531,0.08253254744838567,0.07763595811945391,0.08437304263332086,0.08082924644222574,0.0700244814087614,0.07096495771324611,0.08045131254227926,0.08395553334522216,0.0899566034887346,0.06907459959259848,0.07802167193179518,0.06928228854951311,0.07053290656786765,0.08433303719360832,0.07339604345798897,0.0771189077063684,0.08307035388677235,0.07484986920190198,0.08952563139109493,0.07407971447567381,0.08668283267559754,0.08713497862668636,0.07689991765144207,0.07072769655648903,0.06953635957447957,0.07606220837591654,0.09130664279352704,0.07051922890427591,0.0837844716887837,0.08107670375618867,0.0860829618215653,0.08581989569609864,0.08440381933016394,0.08811615042828475,0.07259279649033266,0.06536182243352602,0.06587933894513992,0.08013002809860328,0.07878051139663164,0.06489487982010678,0.08629904778891107,0.08529776807345009,0.07045586438649232,0.06769397754334272,0.07049798562709794,0.0758991529042371,0.056496439397307426,0.0841154588546143,0.06064597693502138,0.05131768241145277,0.07466332690589722,0.08013099264599242,0.07039798035317874,0.08221530189337806,0.08517175332042279,0.08214769820466368,0.07550083071602126,0.053188422722563466,0.06802476397059057,0.07819378032979304,0.08533009316050422,0.08002098475990013,0.0769331479869306,0.07537091415702021,0.0753599474881434,0.07966007592991195,0.08787577184325795,0.07200720260278246,0.06742913535257888,0.06909712550966803,0.06711234264157337,0.07753007693749717,0.09325316062629385,0.07610957123627216,0.07214941831436106,0.08219764590419298,0.07029430306000203,0.07112617487887443,0.07757629395289067,0.06574960930712978,0.0736956212985048,0.08808415612688605,0.07275771659676403,0.06417573963276496,0.08114079532782864,0.07905073444000282,0.07401256723530397,0.07581889188703209,0.06733969939361335,0.08782172865497856,0.0793940634129735,0.06860389618860752,0.07202924348649087,0.07538441425338815,0.07714441128532645,0.08119537810241659,0.08437981865851604,0.08115103222052159,0.07552682791570162,0.07814695000204178,0.08252436755385603,0.06366090931906798,0.08121740973978948,0.07166912543117321,0.06780928735121636,0.06584598787832185,0.07052790656880778,0.05900004305658248,0.08916921673943491,0.07999597361734906,0.07587437822170996,0.07266677889242004,0.08426806872735662,0.0865824444288661,0.06494258139061376,0.057172815341153815,0.07125264918383799,0.07905923917753016,0.08814237469482816,0.07411721274953117,0.06434036967912295,0.06414171969693475,0.08387872515397107,0.07253046809605235,0.08470934338670488,0.0743479631814927,0.07049055146564046,0.06821257053638945,0.08395512916806247,0.07475593998455268,0.055598021002827264,0.07468903990113719,0.07365635496848039,0.08373961738019954,0.07545894106014708,0.05925779990023302,0.07549818184217447,0.07831291340447183,0.07300218756021888,0.06790873124985117,0.06748513365521587,0.047486267033494886,0.08306264875290142,0.07523680763732107,0.07431875516033282,0.07263038708689015,0.07522981134225992,0.07606536630337243,0.08086469615320743,0.08816064101760647,0.05775490911856286,0.07754089082773707,0.07721903399647669,0.07736438081889112,0.08859692306106731,0.062041962877873155,0.08406373194166727,0.06022025558046548,0.0819403374561116,0.07674836839813927,0.07575334298368477,0.08017703591543192,0.08179839585445597,0.07501064298739465,0.07837314421933131,0.06880586449235021,0.07363932292404443,0.05742328760505168,0.08597562750431287,0.07890116963702809,0.09010146429424976,0.08659150616715255,0.07960561892345613,0.074749345921125,0.06456103208966395,0.06527581450223016,0.0815622894677912,0.0716727899232012,0.07436969061859316,0.0776659250018058,0.08406977663900386,0.08207132311476725,0.07657254806792305,0.08187770930793972,0.06846437723599787,0.08608964959634477,0.07198829874032936,0.06944196660934057,0.08405858642128188,0.054532911227555436,0.0697226921916672,0.09180086394667286,0.06444424929036414,0.06615730646455464,0.06891552229342429,0.07602727953585511,0.07793039441696048,0.07344484089379283,0.08510845291125321,0.08307468833231005,0.07977829202010203,0.06736225886638843,0.056209564790680525,0.0922732374858789,0.07789625708382995,0.07378799346535309,0.0846107601008773,0.09058500822313609,0.0723197784934847,0.07224245534059581,0.0634940010281077,0.07243691620936762,0.07710624194902785,0.07686258048043917,0.07603257028012184,0.07709821995897853,0.07456786615382782,0.08337786347031387,0.07196173457910299,0.0693050875797654,0.08124089990731409,0.06799379916499429,0.0765521671797051,0.07873105990255318,0.06069621282136585,0.07228150556690205,0.07556125518639858,0.08220929481931455,0.08608230071383384,0.08224204009754957,0.07996696224938588,0.06998746237637235,0.07926993654385348,0.08010763584385838,0.0797700848222047,0.08567019117718219,0.08209577416663025,0.07024942820438121,0.050198584942311636,0.08220743245160114,0.07924068353234079,0.08013222936896869,0.08785409973967578,0.0795741299768834,0.06792229860515234,0.0828115984538747,0.07796748128097385,0.06719446829116071,0.08052054878902931,0.08030145929612448,0.08360138160475314,0.07911582886382776,0.0855085604694988,0.08229739412187573,0.07649829734561497,0.07585565461786178,0.07557741033668111,0.0628826502113634,0.07482550822171902,0.08987985045145414,0.07355118144173176,0.08915871813680094,0.07209694032554095,0.06784010799345863,0.07827371216608758,0.08173866559793902,0.07993769943099813,0.0863996883744553,0.07162734090789555,0.07892057050019075,0.06981473716259484,0.0772861234764338,0.07186013545797341,0.08866556565024175,0.08616487922190266,0.08395214655805233,0.06805175160157476,0.06551709346545231,0.07865252594178324,0.08389985623588489,0.07696961730945473,0.07818910812899349,0.066674202263732,0.0834393389445971,0.05992103577054189,0.06894698045684358,0.06819328344890667,0.07566916363957195,0.07914749393078434,0.09190806755942574,0.07538756637277211,0.06322484123638746,0.08488752915116435,0.07228133070025325,0.06516966031054529,0.06645325891073484,0.07431755797290315,0.06957557264634429,0.07045249908650675,0.07377285106183235,0.0769008014999022,0.08434541423778977,0.08315148650598086,0.0695534539543273,0.07972634782685822,0.07796858764286396,0.07475160841741257,0.0856037777195148,0.07704742034402112,0.08507856667563206,0.06439082676741972,0.07520526190920894,0.07460054295752891,0.06088357332683235,0.08544378719630658,0.07680162972377241,0.08667479049420163,0.07352202244865477,0.07394358979628049,0.07695025053962304,0.0788921808085803,0.08641690154577444,0.07961267621096738,0.06976073131111464,0.0762110873002517,0.07303410814788239,0.06038648622532608,0.09114624737377292,0.07469599284103295,0.08671824621069986,0.0592863796955789,0.08197298463101454,0.06654178742011138,0.08420738206124266,0.08536046090922494,0.07010148983546917,0.08304529522417702,0.06908164018956872,0.06657928592955537,0.08504770346190434,0.07409189049630951,0.08966120502517178,0.07059844778737215,0.0708144118454213,0.07014261346779363,0.059096514775138775,0.08122898100171713,0.08111798181457236,0.07816828236464915,0.07499554060059976,0.08207084273498236,0.0709003124646536,0.06742564789558565,0.08149896413773586,0.07142046365463706,0.07362862821772931,0.08470134124913523,0.08663892446785174,0.07541061875726585,0.07519631491189926,0.0822805724866513,0.0773184545405615,0.0762345236433549,0.07914702682820549,0.08720759181094472,0.0813686960070554,0.06341209985014812,0.07402495788523196,0.07554866285012075,0.08126036238250228,0.07477532232459867,0.06798891819403782,0.07955379207861557,0.07560400403876175,0.06533151122895459,0.07708633754729066,0.06370642249809702,0.08315035343474562,0.088098522822073,0.06666288262453549,0.07881782567501672,0.07691241980150972,0.08490246217118316,0.06972877533914389,0.09499173075640087,0.07231343389050715,0.08768706337636047,0.08452442358692189,0.06939493454260423,0.06634371358138673,0.08208574985830155,0.08103741971586932,0.07481322172788987,0.07724345598311058,0.08271997705882861,0.08982732494428312,0.08317239102656812,0.08490279997986336,0.07064527429097225,0.06482974045075071,0.06997472931572025,0.07902497309824075,0.07381844555814068,0.06563368014715462,0.06002651766253784,0.08018183631147202,0.08316419655649188,0.07222046571376162,0.09646154392118968,0.08251660627107485,0.082929208144217,0.06072470021496551,0.06628505443303903,0.06682251241214304,0.06480889895868,0.07958925301848424,0.08794482058806319,0.06848690400041435,0.07635232331294016,0.07018006968536519,0.06238340751072981,0.07002083147891831,0.07415086531199605,0.0891699173617822,0.07866145061091884,0.08273487979650905,0.0851609068270632,0.08036870902961482,0.07169419911223872,0.06837488407517645,0.0648136773206553,0.07255817705187642,0.0856629104707901,0.08290908138097075,0.07221700501036873,0.08158363670047947,0.08821110073794848,0.08644608318733785,0.0768069410080081,0.08589475457438067,0.044606014131996614,0.06786299014951058,0.07966213418764914,0.07872697727514175,0.07322002439366584,0.08624394933319873,0.07895981608780483,0.0801327538879103,0.06639430176612887,0.07430620194909845,0.0838024135821138,0.0856976384377689,0.0682291434217293,0.07407650123088705,0.06691750427549414,0.08882408034145667,0.06302190032752086,0.08262969485733768,0.07450131073270158,0.07550632686234005,0.09305519932469936,0.06399098214007534,0.08004441984317381,0.07606464833294997,0.08280918795017131,0.08019220192296814,0.07839416578121258,0.07981562104994433,0.08524820719707416,0.05907483933244878,0.08746068694114459,0.07336896301013442,0.06622578150178818,0.08722020824895049,0.07680337681656646,0.07858916695359532,0.07972301887393496,0.08492000812266975,0.0846077893068383,0.06616385891596198,0.08088282267888489,0.08352471785905974,0.06392337939302889,0.071319621047607,0.07573679039728431,0.07892571702378245,0.0701721760050583,0.07490015926783414,0.08066895299653318,0.07094758020379291,0.06721778009808047,0.06822204184081052,0.08912119383650105,0.06862789677029243,0.08214100719788038,0.07177554197442157,0.08822144054534864,0.07423726976638512,0.08240591360401243,0.0757762919008222,0.07090908441788096,0.06263796522147982,0.08070859113297552,0.06918272794646003,0.07880560600337666,0.07900791906472693,0.0749354590575379,0.08446018741988773,0.08254963582786713,0.07788618064156264,0.06684194693645316,0.07451120981478984,0.06593591653920794,0.06802432925883808,0.08585630008778404,0.07642407268422416,0.07973396995532567,0.08071859148757998,0.07876518624806239,0.07603630345110268,0.07772289673292646,0.08211529149851568,0.07564923918376991,0.08759276703382482,0.06355912857502022,0.08403838717666803,0.07291869937997264,0.07903138706097093,0.07475806213954965,0.07524576263919161,0.07087062794766733,0.07079341258741498,0.08216887875542711,0.07973561483078002,0.0777950890862306,0.08539679603055546,0.07443365226218457,0.06347145020276831,0.09052788230850412,0.07882102759261397,0.06535120308887243,0.07555281676562636,0.07765885579578359,0.07404011076168082,0.07022622734698143,0.07594843260940719,0.069755058909537,0.08004818438289077,0.07697341579612224,0.07345450433320563,0.06646990337308478,0.07679882552845563,0.07043853528117246,0.0669301059146595,0.0776096927032234,0.07412297053114802,0.07135701046424311,0.06563162463593407,0.08162593170117347,0.08581768815824621,0.08052064275717091,0.0842215270339374,0.0718029529023616,0.08478083642596163,0.06945408900086765,0.06736246829323822,0.08452680685562278,0.09050074352392234,0.0805220270983745,0.07950179651147822,0.09442151076222478,0.07354032647564285,0.08006945194168974,0.06466114617768814,0.07795997577210984,0.07606827085434988,0.0790319569062794,0.08316743980478386,0.07167942890758974,0.07905606998124912,0.07274893575954294,0.06771465020753177,0.08134707615296127,0.09168224731240537,0.061294125222534836,0.07137925839037365,0.07314878928140015,0.0760679973327627,0.07587527277962469,0.07173730686824845,0.054928303930699335,0.08227300210049109,0.0828625396421046,0.06838996931344632,0.073299428208995,0.09529352642283159,0.06962150632134517,0.08245524401446382,0.07881004128458852,0.07157096769137011,0.07455622895268338,0.07751137475843828,0.0769423450332882,0.0668474481161423,0.08820977395549828,0.08197425836684918,0.057065673797946276,0.08849766057244239,0.07638986003566237,0.08402102305078374,0.0830550703394966,0.08273490063184633,0.06799908893597231,0.06622335067978072,0.08570801196716804,0.07488610432069798,0.07123420064750108,0.07866035348344719,0.07074523574954783,0.06367702057447001,0.0742756442742895,0.0817867823170198,0.08402056316380876,0.08032250422178402,0.07811916785521014,0.08470559091178964,0.07796843549533576,0.06406451398614754,0.04926171853050686,0.0768995409387124,0.06794102106950471,0.07421283753346937,0.07495951727626403,0.07850291195711714,0.07795625624647434,0.07855528129726622,0.07454691481189171,0.07864353142158073,0.0820134127665506,0.07643695133044384,0.06935463736198495,0.08452907906988451,0.08192637873066926,0.05589857137449417,0.07656085022786926,0.07987458164422284,0.08878430018269837,0.07645707437653988,0.09124468938177092,0.08606436470907611,0.06638246787792085,0.07218032319229956,0.07560553093638162,0.07625650057987256,0.07785943308254809,0.08496827143818078,0.07303337636436302,0.07242553266444891,0.08830981544190525,0.08618272717649891,0.06216369100362884,0.06597209598379161,0.08458726718956107,0.0707915514068862,0.08038609700360573,0.08889134147186706,0.07330642362305662,0.06636344799769478,0.08684515960817255,0.07568129122404153,0.058183282239079544,0.07322459816002286,0.06579045362209142,0.07562323312763,0.06128847321524735,0.08045309644698388,0.07918561061945942,0.08623995614431684,0.08565507005269896,0.0752321708960294,0.08244187805439086,0.08486434044461814,0.08961424555094237,0.0903747341225268,0.06788453018171932,0.07512989332078251,0.08041264760275066,0.0807420173227073,0.07135981358306623,0.08482392397975064,0.08781589869725848,0.08001411671427044,0.06520005870086043,0.08279218317715464,0.07751764801605349,0.08291740227460694,0.08109988659066297,0.06933562163446726,0.06286684775601133,0.05835931686206592,0.0760289307851636,0.05549219954971468,0.07640703361553591,0.07034249322045236,0.08883229655400218,0.08679619423924345,0.0815181253353704,0.08471122993534522,0.08680334805605286,0.08236203287898262,0.08309457052861496,0.07131096772134045,0.09174232514502567,0.08776858870269943,0.0859495867238528,0.08462309455867793,0.0649392364623034,0.07721229620511821,0.07810187384961075,0.08462308661884232,0.07079065304729473,0.08658512704141448,0.09090544894015784,0.0809641537259459,0.07439027133476067,0.07907664167528436,0.07271902939647813,0.06423105156398913,0.08812841340526277,0.057228606506890156,0.07707779312610559,0.08184530424752172,0.07815567982916641,0.07406235618841436,0.08079040026247844,0.08760620434415972,0.06544964681582185,0.08292233027981029,0.07146445180682554,0.06700129745960651,0.0723240179093183,0.06707606129172161,0.0780841383276983,0.09055108847208462,0.07501972135577248,0.08239919634934462,0.07426844177800077,0.08219308731046898,0.08570931100217521,0.0796158638855072,0.09483689459072636,0.07850627967972773,0.06867388861596863,0.0673858850018085,0.07326510042485679,0.08730617884714886,0.0749936327970814,0.08399349607815881,0.07168788337951273,0.06413287638916348,0.07991368463165355,0.08828618281212083,0.07298161221513202,0.05532076031496088,0.06966467587244898,0.0729731944564931,0.0798407378117284,0.08641665813300946,0.0691991062281443,0.07456983805734643,0.07157950799481214,0.07235511048426496,0.0890081429660283,0.06640807335634517,0.07237739158099137,0.0774387540571082,0.0742619161721575,0.06017526731552479,0.08665358854957864,0.08731846461614362,0.0745239907666441,0.08372042431470275,0.08549999609111095,0.07524196729887482,0.07644811727795187,0.08091061468665305,0.07961974445856214,0.0796499385978566,0.07604071906568982,0.07693227575905837,0.091120229839951,0.07746443279137846,0.08430669989805518,0.06258502469047031,0.08273153801970531,0.08654664867779535,0.07598839135325104,0.07204701866176538,0.07862925972974524,0.06869470709938401,0.0792285823649567,0.0814910574103565,0.08099615494435632,0.07611032041610463,0.06855756780642021,0.07146038945796762,0.06807392220075857,0.07729663821733798,0.08394062801529487,0.08682300141602403,0.08157035150218199,0.06962182390018702,0.07423714141160687,0.061493826341356805,0.07442065112065535,0.08086559354955292,0.07786937447037491,0.08081986982960632,0.0681344212256615,0.08938819607419404,0.06918768756452831,0.0642002828626989,0.06647909587146805,0.08068605497271497,0.07454199148865545,0.05674319521230238,0.07012169265858287,0.07448015515464103,0.06999497764793045,0.07404709587269345,0.075141030624715,0.08604074923205017,0.08027985555305404,0.07995631325403262,0.07403257379738062,0.08280206655566169,0.08491757484423731,0.07490180013705691,0.084089796380213,0.0773284706587197,0.06767052678069649,0.07705459485187593,0.07730508245212486,0.07850171362607718,0.0871744747585669,0.07340157876945493,0.08754759631411424,0.061943664209998806,0.06752045376462354,0.08734171497884922,0.05248715166695714,0.06271767860683058,0.08845514100660634,0.08767557254155824,0.07237020971965649,0.0775077574760932,0.06980966780565546,0.07693808735133993,0.07690796919140194,0.07122852986990919,0.08233659907456672,0.071686555062122,0.07844894374289971,0.0753789288868092,0.0837365115755744,0.09061339236841202,0.08817546217243782,0.08692309304413998,0.07996122299070518,0.08290179959312699,0.07642773275436239,0.08278007168821731,0.08036755574435975,0.06829668974875619,0.08585182430674203,0.07532437904689138,0.06756094923265049,0.08036309200264105,0.06384702938953654,0.07848768895339828,0.06414492532035489,0.06748889022776478,0.07896060153114903,0.09050554567140165,0.08675482955707452,0.08554614912661243,0.09129688776906575,0.07849100179775709,0.06911559841186377,0.08794592747257451,0.07419752036481073,0.08499294487482911,0.07182969594013885,0.07749151334729859,0.06699556341566779,0.07233555663049573,0.08314040217468249,0.07552468090313996,0.0767935865301654,0.08748353078760548,0.085113599121834,0.08194836551766811,0.07589073178480839,0.07985433504100908,0.07794753703739908,0.06862076223688718,0.0798923573458265,0.06764071783308186,0.07940483517178604,0.07967989818714222,0.052092039575374546,0.06734515783889133,0.07613979645173401,0.08260070354277688,0.08633584652544021,0.07756672928152668,0.08233366711364455,0.08521565294923855,0.08214233086553316,0.07371920477769386,0.08082160030081284,0.08613466008019563,0.08134345628138029,0.06687185797827916,0.06991425959727943,0.08240531673800564,0.07932711727063813,0.06736004201807826,0.07672195177682878,0.06357900478157821,0.07261565374126622,0.08718153043977062,0.07865936009101504,0.08855104526718338,0.06935194199839187,0.06864884358496698,0.07473914194291506,0.07973480753244427,0.06247632897083763,0.0837319097573993,0.0770449815834433,0.08547863604187966,0.07175694226205083,0.06651343920066849,0.07902818677023092,0.07318436519545707,0.07564606074072713,0.08239214534132797,0.07030638148371103,0.0869973384179692,0.08129994717201569,0.08179524108447248,0.07426786841216951,0.07331600876159422,0.06668036558638886,0.0853547717943818,0.0681580789802437,0.08928932385325267,0.07772857826238169,0.08402191841864734,0.07340251208556542,0.07476304007357876,0.06443418786622095,0.07011946217393472,0.08834505515998828,0.08363269634944417,0.0904194087809867,0.07523544607503663,0.07612785029821938,0.07208906418870832,0.0710391387155124,0.07406366110007882,0.07783584528175347,0.08101047228868663,0.07062778358818154,0.07666247383199383,0.08272759998832105,0.06601011219850067,0.0888373445941457,0.08548257158448838,0.07813464505228127,0.0648473957301621,0.07061378227532641,0.07590663619808447,0.07581001445385284,0.06983260191518689,0.07527976684338017,0.07367637377148574,0.09270093669842748,0.07434205813271681,0.07939400376129707,0.0684057404351114,0.09077868476296935,0.07259483576519438,0.0820852718053266,0.0821104635227771,0.06792848333294901,0.08224972190252144,0.06565266236008588,0.07591060640788178,0.06341259397818039,0.08149779626697073,0.08077596923191285,0.08842713555511585,0.07116968564914543,0.06728016948461815,0.08202976340279083,0.08236009470481062,0.05221369950705866,0.08335586660385833,0.08862043059064101,0.06946955153216167,0.06543838924949102,0.08493660337230431,0.08742933595027692,0.08237401623843241,0.06024773321120758,0.06445512379154858,0.0599547388149423,0.09266104096807938,0.07764906329082337,0.08500362309354687,0.0860437488601469,0.06408210992317881,0.07267250251744405,0.07800156671832688,0.07161741361333124,0.08819739108157036,0.06649191371358906,0.08459620488082815,0.06379161855088138,0.061127667066284766,0.061136109775569394,0.07579980611387738,0.07872665606706529,0.07367173684044083,0.0652409847863632,0.07651347985570825,0.08121368444898042,0.07749686496493774,0.08127562673291758,0.06898224226988856,0.0782136149341931,0.0777601664434954,0.07779956157545828,0.08086662099629809,0.07135631002526807,0.09168441298261623,0.07516648488163055,0.058234618896974184,0.07037540673218175,0.07996694284120309,0.06843416230852079,0.09132629547055618,0.07721630266487015,0.0652391916804414,0.07744172036917485,0.06905963952609123,0.07147828629599778,0.06251304275506527,0.08956267542882973,0.0771772972887288,0.07404139534490439,0.07389444875805112,0.06673566895063554,0.06798669607747185,0.0821812198785074,0.08950327524729254,0.07033355983793747,0.08268707650334309,0.0730437182633421,0.08412593746358271,0.07106125073316516,0.09217262647561966,0.08492716688102596,0.07179337260110878,0.07244171458431473,0.07024318418713335,0.07776214332160086,0.08369255915022775,0.07702812976398694,0.08091228548533672,0.06895656892992827,0.07219717103278017,0.07582060039040339,0.0773342586172826,0.07934468118626234,0.08836594125930958,0.06889948982862307,0.06661109020876896,0.08236112922940067,0.07588780066056802,0.08265032457757564,0.06202237121886817,0.07162869443382389,0.07045818135478932,0.09060414583277776,0.08122294754839945,0.08360320441070601,0.08127681820341517,0.07708582914922081,0.06749048660964908,0.07258619242337243,0.08345015951826391,0.07178517058362702,0.06453655962675625,0.07852916745278982,0.0660093394409593,0.07468008113027948,0.07599675809116563,0.08489667441659662,0.08534631792075592,0.07425617062659345,0.07099774058679312,0.07229231241364877,0.08110129325284107,0.08526266143803232,0.09208963665466334,0.07972747097625882,0.07964451730658458,0.07740510075179041,0.07676754816620306,0.06503104319468606,0.06869891699290612,0.08288537704283414,0.08851414747317196,0.07131066099971728,0.08430689143482431,0.07584735140814769,0.05497444383565321,0.07769841548868606,0.07152808907786999,0.062055451296397716,0.07362723645109806,0.07874810304161409,0.08546106683766323,0.07038248802194844,0.08204576578988712,0.06710086886883845,0.08958413794543699,0.06410839461463533,0.09332542412231333,0.0734784985486143,0.07192554134328139,0.08184812154354977,0.08038626981294397,0.06475902858887064,0.07537311430323201,0.07034096708781941,0.07001452633785424,0.08557520666767054,0.07204039720820367,0.08124834906858719,0.07867342600063129,0.0793300749401353,0.07190075090492327,0.07008560032985435,0.061225187549382845,0.09132530435881012,0.07568336365335382,0.06701151631215586,0.08079906057985892,0.08623763652847058,0.07526522209469022,0.09263844102568757,0.06905293531362552,0.05669443021869974,0.07356496770921986,0.06705892725533155,0.06692431622646616,0.07566268823955388,0.07263878373644662,0.08697989294653233,0.06539839092435207,0.08780582087953151,0.07069573707224047,0.06495089320299124,0.07263088130407802,0.08087802757601188,0.08450937130050876,0.07354143435133301,0.08989245156583606,0.07156887062101572,0.07508681026844874,0.08649727107374144,0.06802343533969553,0.0774969111009392,0.07867644420725294,0.06551053412866271,0.07630561981344253,0.08177344645933873,0.0848893458576456,0.07994161303008329,0.07128645324181501,0.07220904751334602,0.06208600879212755,0.07833557949232614,0.07559554965366819,0.08154835775279837,0.07449197967648681,0.07483284036924816,0.0642575225681537,0.059417507511984824,0.07257778158050604,0.08139940982825586,0.08028137623852702,0.0863988603544263,0.08471515738109431,0.07518406404805199,0.0850503229447003,0.07924435108454818,0.07609477026777427,0.07199703946576012,0.08529696926854541,0.08234373893687369,0.07361885941173793,0.08813477067224015,0.06742165499870639,0.07441895098636582,0.07744036023569016,0.0616062327500548,0.06860929372995714,0.06584886373328555,0.07448796759953283,0.085541568803505,0.07290939406895244,0.08337814483898509,0.07024303916923186,0.07597042891734804,0.07944279486003743,0.08592813640866781,0.07804953552104449,0.08216819657664788,0.08763081021591304,0.07257446777662718,0.07303825702734701,0.0772865533901443,0.07926843137164112,0.09112583367527759,0.0775614157816277,0.06485812207112092,0.058152759838024204,0.08313024906212538,0.08247856410436022,0.06621144929149218,0.08999984919241705,0.07390580843852859,0.08421159885010618,0.07565933881917467,0.07672683822318253,0.07351130106199324,0.06981914448378514,0.08214984314807627,0.08441445276970899,0.072270344373247,0.08358256734616518,0.06891765463534676,0.07675462068808186,0.08017116586134519,0.07913556671259077,0.05641871728520586,0.07142921228487174,0.0650375435907472,0.07161005307183713,0.08195707044027022,0.07626525590771377,0.07084137945862873,0.07842299997218954,0.08616188576103753,0.05914683858046736,0.07708626764212523,0.08460953294973225,0.06117308486221501,0.08556131548218446,0.07624135383429048,0.06102269631573848,0.07834308361937962,0.060850416855654524,0.07941641653202262,0.08070323717150493,0.07362452777786294,0.07413268720184801,0.08629012865461931,0.05587904519665923,0.08779474162247826,0.08228207930608283,0.08354440013649048,0.09012557865687515,0.07729848673381129,0.07460220406140601,0.0713198537384681,0.0730706394395293,0.08137407065131175,0.06768894546819479,0.0782114406191895,0.07696868538998232,0.06909835420130514,0.07739885102543012,0.0808717054395856,0.08768814394346808,0.06749091853336989,0.06848887883650275,0.07214514454992606,0.07180332053543094,0.06426275419126601,0.08119071838232847,0.08613417795442121,0.08905990268703165,0.08787451647449734,0.0749261166573568,0.09006377001911678,0.08749776063901117,0.0782624500358222,0.0788565558763768,0.06576355967580401,0.08094501021820587,0.06721008600661185,0.060389208564733676,0.07692731610780222,0.06437192301809874,0.07398613710110431,0.08563346560197646,0.09040571815283117,0.06828181452241416,0.07030758744989894,0.08763961673357867,0.08431114501210529,0.08030105528151835,0.0890275158477778,0.0749269384749447,0.06571093038479678,0.08172309857348728,0.08540978359145482,0.06705476549668553,0.07566820462182325,0.064589295593137,0.07482736479256835,0.069818427955561,0.07524124901076146,0.07573422091915388,0.06345441941700536,0.07782528549827535,0.08021017690635156,0.0684201762758176,0.07437865857090323,0.06820829494034886,0.08173821748120509,0.06742758533060209,0.0731695264250593,0.08069390181447728,0.07606900396064659,0.07905984985852724,0.06428902007475504,0.08444380771108984,0.08649150701682795,0.06663775534603135,0.07944342169069624,0.07511196105550012,0.07701576818331679,0.07807278859503762,0.09232629000677153,0.07409926616036729,0.08251518027563787,0.08722356374566445,0.08180927702817997,0.07874534564732263,0.08519078837858185,0.061910798881813375,0.07502149362132476,0.08910710256752637,0.07081930050307421,0.07703735358444092,0.06499457440714895,0.060512709384568014,0.08982365374239884,0.07322109003910085,0.07536318843497175,0.07712613827699802,0.07476950196274362,0.056011080524837956,0.06662295620653066,0.07771099157237542,0.07474030074269641,0.08991019703827438,0.07033731581665054,0.07092178501115005,0.08526568954376275,0.07610307065727664,0.07294805337446028,0.058982859881119294,0.08031841665204989,0.06978679867882703,0.07266221317260371,0.0855190884397643,0.06375289600397174,0.09277391773044798,0.07878401688065383,0.07892056667873194,0.07162134342446784,0.07892270570180154,0.06742807541663055,0.06629156704688674,0.06346357708151579,0.0738099204272373,0.06386136440769692,0.06840589229984559,0.0812747576009844,0.08169648364997331,0.08998412046198555,0.07530057375349151,0.08667210949926182,0.07193726969677863,0.09190789362041615,0.07559467140324522,0.08564289037448204,0.0812943497569055,0.07760359813552038,0.0725129280690143,0.05613445340491216,0.08789046004313665,0.09153681864688815,0.05892930523570343,0.07387671760116325,0.08662602744852771,0.07561536374120756,0.08094362909386868,0.07972425043492949,0.0715017229490123,0.07156102978680808,0.07248780146166076,0.07138745386814618,0.06483415041068219,0.07310690254490199,0.0722153372568007,0.06462195773025466,0.059730005374563286,0.08983846954968386,0.05373469286626391,0.0724980524635579,0.06074069518965695,0.08379012390950569,0.07898662991732489,0.08691684853508819,0.08903096583946538,0.06849107486920122,0.08134943510630059,0.07251641353943694,0.07538032415920634,0.06500868763810955,0.0793526173506513,0.07586641394155587,0.0723485456188886,0.078787927016888,0.06738513380284361,0.07756452237563641,0.08246589327679493,0.07087194358603717,0.07969458026565442,0.0718834476668973,0.07048794218719846,0.08562857987219427,0.06915354013163787,0.08171367606607793,0.060515208548044416,0.08289001559339508,0.07438833692774072,0.07570859574068962,0.09006537914449023,0.061978246110543915,0.08345253137070659,0.07628107292692163,0.0910100555591088,0.07342313339306197,0.07613975110459545,0.08282374328851627,0.06280669126196942,0.07744297428246914,0.08472602461765055,0.06400850641966516,0.07202306765702404,0.08500380281303592,0.08746504031662021,0.0852568870890944,0.08697039032346685,0.0680119932962984,0.080538073837613,0.07261132435176634,0.08752186464268236,0.08422211440172793,0.07961390486489739,0.07623623047182117,0.07347351349341269,0.08030203955696423,0.0713817888553274,0.08405715913179224,0.09351677406663016,0.08180218465057207,0.08076984047986249,0.08776800622691894,0.06568137628157526,0.08479676602036779,0.07499942009329219,0.08566608178327276,0.0652198337001721,0.07235031567089129,0.07700508375458784,0.07925253646993456,0.08058028361741185,0.07201611640100701,0.08500838048505711,0.08431015515596249,0.06919363643348866,0.08624264805231757,0.07178732862030418,0.07049910360541399,0.07697220278140379,0.08945950638020654,0.07734001584481273,0.07360512259163032,0.07988694383628667,0.08863591694002418,0.05567228782185561,0.06917761945710987,0.076071840202864,0.0710265266415062,0.06414278215179703,0.08508829180123817,0.07769958067521396,0.08269858987179285,0.07488441199274445,0.0798433114476797,0.09086954733871352,0.08123157977908901,0.09017286932346477,0.0843228411457245,0.07131252078197857,0.0837208593972159,0.08458592839096636,0.06158753393552687,0.08745804334754106,0.0642566877661997,0.07070558563182619,0.06299479788492811,0.07782497347846255,0.07703526524796997,0.0688799582037417,0.08642985362146094,0.06525881690382543,0.08818006354075408,0.08205973724051219,0.08713170193187335,0.08786139093957113,0.09336987034060852,0.07374352066096315,0.07871365854738493,0.08112460110655574,0.07973020453496453,0.08349738370790027,0.08418775533740533,0.06653829409605376,0.08436368986682274,0.09226283168920407,0.07633074724494937,0.08136280513793354,0.07344128128665783,0.06742849178781093,0.08144313369848133,0.08370740278434011,0.08218527522315887,0.0793168971030129,0.052579112866716224,0.0725367357122865,0.06705179408903275,0.07463110805545045,0.08617878393534248,0.08227283076484995,0.06581201639352396,0.0857520066972389,0.07147887244297715,0.06584617154772153,0.06619472150913863,0.08203355310724764,0.08161965812706805,0.08024482964437046,0.056386439067815305,0.08302905029001677,0.07890947777588678,0.06892937531172597,0.07258010306013528,0.06846143458524698,0.063953699102289,0.08527083747409782,0.0719135945221141,0.08812395277472197,0.06583990549354893,0.0739317564535663,0.08764649780039407,0.0752435144713782,0.07993308130115051,0.07238056158528514,0.08182469834060502,0.06147085828682757,0.07226285997704965,0.0763433156624374,0.07212895766257109,0.07168068697648591,0.07797094011925254,0.06261928615437433,0.080335108653406,0.07373933087395484,0.06828638914765049,0.08498149210079489,0.06969714269433469,0.0602837745279942,0.07076333539739058,0.0617411648898079,0.06088990041596336,0.08076006037642067,0.0784521755735402,0.06238411882268015,0.0754992971834728,0.06738452874445298,0.082477901664397,0.08847115730134705,0.06123350697306571,0.08288335020803747,0.08073421927488295,0.07092943912546844,0.0653907200708764,0.08637320264613285,0.09175484452853508,0.06623186383829517,0.08496902042615873,0.08896525842652322,0.0782910482710411,0.07375860660423667,0.08404264212492435,0.07835737976624035,0.09054817818895042,0.07116207177661452,0.08683852639710993,0.08007718593015462,0.07179796269563145,0.07018033414527197,0.06617735997157649,0.07897055319264906,0.07028176036548352,0.06790178986128477,0.06550647918612626,0.06516255017888523,0.07655709594363697,0.0794305279301691,0.08734026918632268,0.08306651093170132,0.0710016238481096,0.07787836129811485,0.08510344540597972,0.08834318020288345,0.07881974345977853,0.07516241141656377,0.07797222186016782,0.06565824285074788,0.08129015067025613,0.0923667531053822,0.06847537686929128,0.06878760039126858,0.07296235341251947,0.08237149554536123,0.07377034318683841,0.07930789026889398,0.07003239793923569,0.06914522389237178,0.07785046604603685,0.08122427652582485,0.06093698729316546,0.077001435087436,0.0653489316896401,0.08741175173288612,0.07456477159013297,0.08496797351795138,0.0650088217992867,0.08289992846527089,0.08264348266708554,0.07569834819447488,0.0760429563031042,0.08224091022815566,0.0740848575145891,0.06799894263625941,0.058190021850833495,0.0805375558656042,0.06787480841669052,0.08000731490533951,0.0786143860078265,0.06934459194701177,0.08829834151602556,0.08283650796389243,0.07962264907740921,0.07434031744062249,0.08590295235217943,0.08126227663076785,0.06948323917993618,0.08526701037223205,0.07687856401880665,0.07882684009287122,0.07352284042900294,0.06851553187542894,0.07966838645871659,0.061808372802921244,0.08377145687922359,0.08862785671112863,0.06144146880927592,0.07812750533019612,0.08372878233553539,0.07790199991424261,0.06995931906478643,0.07676089808438498,0.08047098672717273,0.0687004652861538,0.05858175741973347,0.07153662332431633,0.0821513003653315,0.06086330399778868,0.06832873335936457,0.07349854888524499,0.08492470164088721,0.06539547795607156,0.07131764080284686,0.0758750119021758,0.05695963071292351,0.0780802390777198,0.08210798420514274,0.062352437377091764,0.07669217521327279,0.07821551986280863,0.07777938697634157,0.07612579835736061,0.07836003734406721,0.07903898517132145,0.08167871571925388,0.07532724428970822,0.08260305041706373,0.06778987809805632,0.0803249368795392,0.07096057875563555,0.0763207481406876,0.07798738240711553,0.08001551815276908,0.07642484338981767,0.06862092761043424,0.07608643886726009,0.08561217503747273,0.07756012346976698,0.0844892593686428,0.0791975003152484,0.07883930891714765,0.0906046309998095,0.0815719958952389,0.09099707190117692,0.07907157289734504,0.08268321227658545,0.08806419634305354,0.06432999028730321,0.06933957179552372,0.058992963394843764,0.07472508354942045,0.06671639900699936,0.08192138361571753,0.06387140838298216,0.08248692907405791,0.08678865602586451,0.088813510334997,0.06338209990004025,0.07778591426165546,0.07943723314364648,0.07363440103084257,0.055746201116955815,0.07627603054559068,0.07718410410983692,0.08554728633134717,0.07595386468756209,0.07548323171747509,0.08401523832619767,0.0677913361258177,0.07661627673585464,0.08781212579401124,0.06009397868909061,0.07151041557079707,0.07600561232817332,0.06542223994397445,0.0914367697971919,0.0692539444009795,0.0875103239981408,0.08621050801749364,0.08489527061203489,0.08594808840985027,0.06174663873394111,0.07868336528515929,0.07814617313675874,0.06406646747380598,0.06353917220737498,0.07894332914035909,0.06567986356318878,0.07090721365938793,0.07986083000406546,0.06502839208211479,0.07092323802735512,0.0592558463804886,0.07497633847327487,0.08552054677080807,0.0916934329081956,0.07952509150247938,0.0669727217705835,0.08695892887979889,0.08111163268086408,0.08755106543000266,0.059921735614175346,0.08549023741265542,0.07370929615987146,0.07429659524457659,0.07473783275536251,0.07784197506448898,0.06818019578902741,0.08642632737367346,0.07712743666659719,0.08007854806994089,0.08167011997471585,0.0593831672845231,0.07574850861096807,0.06246708879860098,0.091096761639296,0.0706435723422923,0.06264382946764789,0.0815241885967332,0.08593127079030266,0.07371764818971563,0.08814741569992118,0.07855281180014868,0.07299157989537094,0.07011347848019055,0.05455976184922498,0.06378525235606555,0.07646442936980248,0.08255930600536218,0.06817277538658713,0.0721082636362339,0.07587416531733593,0.08062748189990965,0.07323728423051855,0.055502152413941894,0.06924791612694278,0.08201678789478307,0.08032313896098503,0.08774115349426369,0.08201857349642677,0.0790414939080488,0.07259305895040333,0.06662659008128008,0.07355698512276122,0.0907299189021432,0.06775258417298145,0.07822185848551227,0.07997886950188915,0.08310340437263145,0.08278632411358111,0.07921569361801012,0.0737815712201921,0.07870715244524164,0.08054137383060635,0.07949325800108104,0.07009028298309097,0.07823517159186655,0.0718712327826734,0.09031085483393159,0.07266926113460899,0.06935882047993855,0.0881120193776681,0.07968555396477693,0.07429310589850041,0.08375593805552317,0.08775324249202271,0.06678770696252964,0.07240759246340232,0.06940186822766217,0.07392419133213361,0.07999706426419864,0.08590213269355909,0.054616017939817386,0.08583633621562473,0.09218243556341267,0.06484625985869569,0.06475565123396568,0.08825628443926989,0.0825102292929841,0.07754507319841142,0.07188010019408675,0.07521581982167952,0.0717691490183558,0.07687568856833323,0.05012817397487744,0.05992138605246541,0.07147159986661517,0.08313610633029796,0.0637901328102723,0.07987541310530949,0.07719325908786556,0.07377541540898731,0.07482185129052614,0.08733009252737223,0.08181813542789723,0.08712542677037147,0.07775205102840366,0.07187964728244087,0.08550626882557896,0.09310886180602564,0.07547577218728707,0.07683046887954964,0.0749159188692097,0.0852246149550956,0.0908116943154214,0.08928368642802974,0.07230261741145905,0.07986107367829492,0.0816151806741074,0.07090186523282062,0.07512084876966682,0.08831833626695823,0.08386494303167225,0.0800817623109853,0.06788654681196397,0.07477608650898424,0.07963024512962666,0.08092858227588012,0.06735320844980454,0.0795199432907524,0.07237717834468858,0.07609938229805249,0.06592618263760733,0.0874632213755335,0.07078783485670834,0.07501798647560443,0.08019414808186795,0.06658882729733545,0.07071917753085782,0.06713846446472402,0.08008895679655517,0.07844510186371359,0.06922758630183454,0.08321068523814193,0.08636727055330376,0.08196567291435773,0.07955506220809541,0.0660473053163955,0.06846728859273823,0.07354193876600851,0.07899567260659901,0.07315671401168086,0.08132310623792072,0.06795374163991698,0.07708611248006453,0.08092664737884403,0.07271545725293621,0.07762511664748681,0.08024555719886838,0.078185820013534,0.0850164949004889,0.06635680557057269,0.08351476065882954,0.08088031300713003,0.07905357937330289,0.07395521481320072,0.08239138292221786,0.0645188907215844,0.07872949203941859,0.0892172563930134,0.0790906905824604,0.06558168079247778,0.07781107287341203,0.0639427107473397,0.07898741059338538,0.08020458918297815,0.08459737144030205,0.07994278871100391,0.07153106293306001,0.07772101277095933,0.07664530588564562,0.09205985180886292,0.07823473479637191,0.07573699787485519,0.074114318945659,0.08232516996946136,0.0835596204971062,0.06339380313972542,0.08598099982357298,0.08392004901853459,0.08635410239732727,0.07495390793610807,0.0744793456326294,0.07616622310444385,0.07772494654287049,0.08503445762498096,0.07499642420667987,0.07172883726929871,0.08558760635341289,0.07653370814619703,0.09519328076803553,0.0863974541864117,0.09280762888357628,0.07430325885999742,0.058699720523158444,0.08305025290566953,0.07955418537121878,0.0693556137673584,0.08113306397745419,0.07709805128729409,0.08269228881686416,0.0817622524241871,0.06160578300860477,0.06737035239182666,0.08598437910118092,0.07724984887867292,0.07897576585831739,0.06942835770505956,0.08200481633196949,0.07152905968412857,0.06784245742260481,0.0713482051648579,0.09282233691735652,0.07079463023790053,0.079251264039724,0.0788563528909107,0.0810839927761208,0.08603779842140391,0.07968502318390691,0.0915064241000943,0.0742684219869558,0.08124389266939369,0.08624736454700016,0.07608291336975885,0.06927861807255425,0.08336050845009432,0.0750521458841619,0.06106981831860144,0.07392404281548456,0.06364853473243728,0.06517869733895892,0.07853175036287846,0.07152437395958464,0.09024313210652801,0.09066341361720204,0.08500892659519056,0.06711093043415636,0.0747042961226283,0.06523522112569827,0.08475898954727536,0.08695821789809546,0.08533401473030075,0.07197496332127629,0.07938632037615541,0.08226399118024463,0.08215313470055566,0.08198160835182852,0.06935992799397322,0.08892960431293667,0.08654684347125391,0.07563566978612951,0.07558623398771225,0.08762181807949354,0.07920591864078948,0.07927326411787847,0.08986420790166476,0.07727536163916596,0.08554234419766937,0.06263240932873884,0.08895168853295055,0.07332535308177188,0.05961443843688287,0.0772951685108597,0.08740006797101207,0.07583505097634634,0.08081054382922213,0.06836513576322077,0.08537554844660147,0.06611930129093063,0.07310825480318986,0.06853281230886588,0.0736116191415501,0.058602505864700234,0.06781746826650828,0.06594226912265005,0.08926088035601966,0.07626875428716781,0.08263307644846365,0.07581989935386794,0.07371957270027224,0.07932124141495656,0.06964411510267013,0.08339672290504659,0.078908421691674,0.07701525877548995,0.08999921586280926,0.0743071772800003,0.06959379269054392,0.08067839907295477,0.07249569293178612,0.05337616057300297,0.0804385015542188,0.09324352507437572,0.07555451495827056,0.09197414292114672,0.06612699448696696,0.06938043041183686,0.06861644528091917,0.09110159183711258,0.08523943078223475,0.07774200213604715,0.08237115951084628,0.08387286703400106,0.0841287177896397,0.08765176391299984,0.08107795515028066,0.0897242486852522,0.07335970555336971,0.06959977669411141,0.08274115967846721,0.0620239269783715,0.08415563458615109,0.08574158292492809,0.07068384732117527,0.08096994944266411,0.09190461359003936,0.06451644940723453,0.07289837375845745,0.08499231224728471,0.07815112927924264,0.07900970966889587,0.0870039212292063,0.07637996430986577,0.07594646463521751,0.06486782910817537,0.0772761531657756,0.09199481447855551,0.08770044008825535,0.07987326067479592,0.08446071189729065,0.07103932150487,0.0610778355377099,0.07399347481451575,0.07805266731698726,0.08385661589946755,0.07148223858715069,0.07990898765851598,0.08026953624448735,0.07244392662175365,0.07761239440256402,0.0529380607411706,0.07125959395592485,0.08284963901013981,0.07870428934249701,0.08696420959235074,0.0732293141693091,0.07432130123843625,0.0806113401492072,0.06637626828810539,0.07633989013231626,0.06816407494413064,0.0863157119256891,0.07464096656947501,0.08349434151039162,0.07582005520100146,0.08308213529173093,0.07017690497713662,0.09080122363342688,0.07419385746775563,0.07848015257269327,0.09365971542202683,0.06877067399040745,0.07191115360563406,0.07524910937051085,0.07556771801312505,0.06343535371051714,0.08013081337585994,0.0742926156757047,0.07569473485830852,0.07827210011939363,0.0626168692702054,0.08175839438630392,0.091384313798203,0.06884760914848895,0.07941393382478432,0.06050325205367767,0.06825587504735617,0.0738829554504064,0.08790309563612186,0.07262199054796124,0.08286287395320834,0.08051863843027204,0.08708178987312583,0.07208254873479167,0.08653924034504548,0.07960933466967923,0.07026894280617314,0.07824669479613808,0.06597046918268626,0.0759799698589748,0.06868383785745857,0.08064381765803456,0.07934274008787554,0.0697666410335597,0.07188594165358073,0.08272952982319885,0.07276206459796583,0.08050410093761641,0.07352868392484395,0.06484737253852937,0.08378735605303035,0.07894647681311576,0.08376049656229163,0.07190931128841346,0.07823240217595356,0.07172361129447391,0.07400161663153909,0.0706164935759507,0.08399683966811461,0.0856954101668358,0.08247570392819745,0.08460177397889032,0.08438180435768947,0.0797476802417523,0.07161883438577574,0.08353873727553415,0.08111111633747889,0.08042420364230092,0.07714088148380116,0.09317418487836883,0.06768414930836658,0.07754293149740413,0.08339928471021832,0.08690294521656522,0.0819997848939199,0.07513853586592953,0.07560886354483387,0.07906626186492875,0.06948120963804208,0.0704757281293015,0.0635291942432005,0.07482912343456471,0.06412968178365244,0.07601544957300399,0.09147664520702511,0.0822167321555293,0.08436115087891069,0.07264776977890819,0.07648477626906544,0.08453871474503954,0.06670161902204008,0.0650388175785887,0.08128397087807626,0.08774736722756853,0.06203776693709151,0.07930533950956759,0.08763695265058037,0.08629559842052256,0.07733759271115409,0.07492667765044975,0.07966761521441448,0.08631971261135046,0.06699155990885827,0.08058756942747158,0.07664279971950588,0.06263830027621112,0.06235823723891965,0.07862392519529302,0.08063418689656401,0.08488163302287642,0.07869836068120167,0.0772904788650019,0.07294962776945892,0.08360999622641996,0.08440098538490118,0.07689660326499159,0.06716945146998768,0.08042635693039839,0.07768984315975677,0.06743421371480135,0.06184282985364305,0.0680833211879912,0.0775979152160797,0.08675091790595646,0.06711777809773228,0.06360845596135403,0.06951898245253932,0.08129856040392691,0.07408093003121338,0.07997410058048596,0.08911058767286481,0.08443409668316185,0.07323463156380855,0.06808293826753356,0.06278448592300034,0.07671292074070898,0.08762225711878982,0.06745813206285853,0.08012013602498883,0.07982990302392902,0.07856205405527597,0.0638626976368998,0.07343387054949241,0.08872879300939898,0.08619795680894357,0.06958424223326677,0.09317611730817388,0.07602825035028575,0.0696472378390514,0.08089219720634527,0.0705076983940327,0.07865733853091439,0.06656568989980079,0.060513435270412676,0.07817510505482278,0.08315006123768005,0.0744425990718405,0.0666078353822835,0.0642043588452475,0.08995067976521132,0.08121747048273299,0.07649674248807277,0.08297528457018222,0.06082719282099039,0.06761458739863918,0.06794765651619518,0.08784142809155901,0.0591792729186678,0.0732990517339718,0.07498751278578193,0.06741079192656294,0.05932709740497692,0.0833575887068992,0.08471573818087844,0.06513467208823906,0.07589332601108834,0.07311825441883901,0.080822103319694,0.08401293740799835,0.08211440727141464,0.08430774462021007,0.049631314608663796,0.07965907447400974,0.08900284981605465,0.07499012499515179,0.08354247962356093,0.08204550733058387,0.0777892025385796,0.06800179349350233,0.0818518765445552,0.07828238951699303,0.06438478008349031,0.07686427969079104,0.06381907134334161,0.06383396004248956,0.07544588052395625,0.08188267528286097,0.08310224819345052,0.08059838881736331,0.07483261503736596,0.09292493249332774,0.06626854493034195,0.07428994835751193,0.06864655771098452,0.0804319026176214,0.07113154756926843,0.08319911846303565,0.06852413367231872,0.0874725036891616,0.06335527690154621,0.08034192104697463,0.07922074570836013,0.08448836392745576,0.07764486307263464,0.06555976075811232,0.07288125014588545,0.07164586569728319,0.08227687634476653,0.08978535052503875,0.08665498443459482,0.07736177250966765,0.08283390196018461,0.08306769676816977,0.07902013459918152,0.07479451236970361,0.07656511507615044,0.07785399057182057,0.08399461608960951,0.07565885807436826,0.06932535794159642,0.06960696006519185,0.08955061630627337,0.056044638931531485,0.08475357432644157,0.07303921741021395,0.0913070800902875,0.0784244416609946,0.07818906789640852,0.08067276867438422,0.06722768968523357,0.07352715784665584,0.07581661612022397,0.07723926140667356,0.07338373070565402,0.0702009688891002,0.08766605728893923,0.08297966451747699,0.0682042800382414,0.0808007933480567,0.07946023930400617,0.08191831084209304,0.07232896169590362,0.07143797460794536,0.07925553338640882,0.07589123152215174,0.0881594462167479,0.08045326181926514,0.08604235430838325,0.0735004016627128,0.06983299555487334,0.08630811422272355,0.08139603560017455,0.07944659558169498,0.0825123024658928,0.0777567020048233,0.07617280908165883,0.08364620755313396,0.08222950020860686,0.08660137522696913,0.0705585851584792,0.08292729878938469,0.0625271900092669,0.0778491904054424,0.08499589795792106,0.07748654314984474,0.07714196802982415,0.09315927534089066,0.0832930881519973,0.07951577334370102,0.08898623133112582,0.0864930817925981,0.08644325707700107,0.07515937028220218,0.08378155134996829,0.07991990064401908,0.0717154432520782,0.08526653586714004,0.09235338734684288,0.08930720044834867,0.07099706160858575,0.07383127917212068,0.08062876982082612,0.07907573726445707,0.0812803551426386,0.08815341139513966,0.08752905861800697,0.06626896005496853,0.07807017029855995,0.06931845015018401,0.07508905588899123,0.06880093263613948,0.08901582032279999,0.06701090562891962,0.07688583992245976,0.06876197320353791,0.07794955439170541,0.0752837194244169,0.07059861432520517,0.08347640624873383,0.08660575802133033,0.08635567774532851,0.07827478439944544,0.0760810647244872,0.07633317656212629,0.08920319229544815,0.08025550443186776,0.06569712142791596,0.06734867639904679,0.08055705309203579,0.08584212749911212,0.07481823234785742,0.08035580660458379,0.06694314932783943,0.08894754720663389,0.08539502873316904,0.07930773309176475,0.07020685299051721,0.08206267272715365,0.06486929419450418,0.08194798276761991,0.06321547915331417,0.08063580712636785,0.07469506355464989,0.09245295285412891,0.0749608021361109,0.07085859992319737,0.08058916142382491,0.07323144234223936,0.06162746654342195,0.07838595929531292,0.06167951238336484,0.0804682598412404,0.07692703881249509,0.07092826621755566,0.07129312176199817,0.07741035481664696,0.07616949945032396,0.08013418036749606,0.0734309563784715,0.08879821826452095,0.07310534130029236,0.08212544376530238,0.08998539353547023,0.07448763731753374,0.07420383754583768,0.08933390621005156,0.0835240153289285,0.08424954199288905,0.0787461083765438,0.08270215464513858,0.07210384253213445,0.07394936511760536,0.0869663313363387,0.07855049569480937,0.08618283642579126,0.08514392313105476,0.09097712725806109,0.08326706851429584,0.07589763109770511,0.07956526349680648,0.07361288520185746,0.0831315888698886,0.07322296644782833,0.059878800736472955,0.08555817527089363,0.08518598763665844,0.0757755801016336,0.07302856388209164,0.08589082324115004,0.08338798668660297,0.08011684080336266,0.07606225447864282,0.08896073044149046,0.05419708249566579,0.08787320333466171,0.08604156380884395,0.07383402021201997,0.08820243841589723,0.07406343667190804,0.07793278415302346,0.08502583060764564,0.07774802966271732,0.06819188815086871,0.07429500371292339,0.0708296068554474,0.07656472764770256,0.08852951306731019,0.07905406954853336,0.07725811152859695,0.06442046287211431,0.06380637279950112,0.08178421100867828,0.09188522795468851,0.07350446153567711,0.08843731519816835,0.07593259181331594,0.07630752966572041,0.06930007367312332,0.07574814413247452,0.09004117152339973,0.07492611875660415,0.07900501277312358,0.07669728633496951,0.06803293133255178,0.0797025820303574,0.07775889745874182,0.058374865556237915,0.07735895844166463,0.08876506472165697,0.07568672735553955,0.08682568215024983,0.06466138426249786,0.06867191931669547,0.07963393772108102,0.07811160470272996,0.06101617736237801,0.08222978169330825,0.07956435870492853,0.07726333789209747,0.07781903582373845,0.08130204817202867,0.09141154651085809,0.08953426699799764,0.08473881152002276,0.08462538159861654,0.09035392378133744,0.0811158993723088,0.08948081734166395,0.08245773862367146,0.07757346894562797,0.08037052870198177,0.06964449810515068,0.06496942182444043,0.08760003810830601,0.07791467331915616,0.07797043702828477,0.06611386440094313,0.07809383208119308,0.08349439098611405,0.07071572271490875,0.0732546937454504,0.06315733148364128,0.08117225998785875,0.08535646019153509,0.0763619564659326,0.08060633227286473,0.08136452560674212,0.08585780734280979,0.09450891305186832,0.06159696595692786,0.06351399406290101,0.08000185898050277,0.07676075153916738,0.08280127045371223,0.07917548591820134,0.08458138085782749,0.07525323851909899,0.09270047454389742,0.08049036202359597,0.07960665581756513,0.0747324366757334,0.0804795104430757,0.08117253145485306,0.07640953741325149,0.06983316327388793,0.07210877201220459,0.07160772225561299,0.08217379750737333,0.06847608345870942,0.0725342644257505,0.08658279290362494,0.08684175514209794,0.08184795698732295,0.0807619122772248,0.08967024456721728,0.06613770339207563,0.08305616585983475,0.07676782556336967,0.07343725179897717,0.07592484308641312,0.06578031028614656,0.06899561236012394,0.07217409165446129,0.08321854418505845,0.0612310226975428,0.0748991988596796,0.0764981540998271,0.075355196442988,0.0809518079966709,0.08940722733432495,0.07971487422366541,0.07000534059054339,0.08466749563968587,0.07574946733894489,0.08082334312163646,0.07481608236476109,0.06631748974714304,0.07662172325984892,0.08563454937550508,0.062178364250728854,0.0712244791995536,0.08447973177938835,0.08148959477683987,0.07721751381889046,0.07999600353618941,0.0821817829117921,0.08135895628743196,0.0811016477568055,0.0899117457256091,0.096572419547055,0.06831313502081884,0.067219514722312,0.07857898774177412,0.07288210340595166,0.07615632605480496,0.06288425944094694,0.08701993287985041,0.07652677813879777,0.08620181279943886,0.08616737573078935,0.07749085841298761,0.08331339837114451,0.07430015189527076,0.07229008096159763,0.0752502377379616,0.08287510114119942,0.07333178151755715,0.07869872515660963,0.06848851647737715,0.07646954796383867,0.06334285541195464,0.08408944525194247,0.07938150302696634,0.07329834148749734,0.08531041900330354,0.07298133796658501,0.06695376543727066,0.062928462452091,0.09035611691434238,0.07281667679559203,0.08821140858651774,0.06680806836151457,0.09068521703601827,0.09018262712522518,0.06459283114149766,0.07557596724475787,0.07328575004234927,0.08884765838115133,0.07738477743736685,0.07422769374396812,0.07118875584253116,0.08694577233628215,0.07468167304279119,0.07932772215692106,0.07455337991547994,0.07810854768707785,0.066986984074171,0.07475782555804597,0.06924695671198733,0.08461359410406702,0.0848212598778002,0.09458743390666606,0.08696827391454219,0.08536504025737796,0.07894881777543286,0.060648156473768776,0.07783086905326637,0.08098909273499921,0.07695446902258685,0.0668558805359622,0.0753522968958977,0.07701911192471156,0.08894191846679009,0.08041071918716468,0.07385651487659416,0.09184108186366444,0.08517245325912892,0.08218285485146706,0.07574796372716038,0.0760581147604657,0.07918612338084852,0.07702454337610361,0.07382638152100335,0.0689318933097836,0.07813205116668157,0.0668071243610257,0.07373555846364001,0.07310810929275594,0.05473160756845091,0.08547276751853995,0.07356491152259167,0.08166288402769938,0.05960151729124937,0.07933769223653278,0.07982890383964766,0.06665736353572135,0.07197061533820129,0.06013931263049996,0.08081782414389829,0.08133925328155006,0.07049551621775405,0.08487257276951461,0.08076100212124951,0.07117633703515541,0.07691636961207471,0.0832120275155697,0.08938144856609515,0.06181772519025694,0.0711097630200482,0.06813409229557335,0.071312429873372,0.08053659385061937,0.06183510622302293,0.07704696418983523,0.07261502410530425,0.08963907712003921,0.07307065553387096,0.07147498248265266,0.08255658688840635,0.08527406756690245,0.07870523733135654,0.08797036649326856,0.053419244293520074,0.0635802335498328,0.08807839959101041,0.06647040867296451,0.04813885702489573,0.06163790655310994,0.06853877810597106,0.07474253241124204,0.08072617138424079,0.08115295581227194,0.07753947139906756,0.06662100839244087,0.07040346505096037,0.06908471622970544,0.07335829536148311,0.08795982550327608,0.07647336467204674,0.08110209870684165,0.07512586694315924,0.0830812494224672,0.07609040042326287,0.07456385498031817,0.07896865567419858,0.08289166397314135,0.07361691713754373,0.08149451365569992,0.06547438947969103,0.07975810523805382,0.08399196088170108,0.08036718498873646,0.07759421780604042,0.06896954033225192,0.08258676272978815,0.07010839458563409,0.06533353893506486,0.07536900299103394,0.07360145835155903,0.06484680506933688,0.0766864028033765,0.08775135805123127,0.0545455894212772,0.07236903459317442,0.06927953469286738,0.09076727134765553,0.08909116130917943,0.07049927246525424,0.07675137208546051,0.06891636008311974,0.08069367487264631,0.07564289260744028,0.07418116016143941,0.08125270485359404,0.07649697187643213,0.07352647649826591,0.07832948657076905,0.07267602580309729,0.0645117515521923,0.08153081653588043,0.0757521786341118,0.07404401099330774,0.08278176678741335,0.0555704274417796,0.08091133440079282,0.08150540034377607,0.07881348814997619,0.08308015209872684,0.07192563169916848,0.07187213842744263,0.08443507326244148,0.07966600713448138,0.07002549321584617,0.06599353091089931,0.07608681073600199,0.07317378995710766,0.06425452868861178,0.08011527130550122,0.06870175192309022,0.07644326819020053,0.06828283025582216,0.0701520532615323,0.08137354253721896,0.0737776782431158,0.0663021372283454,0.05924737084780246,0.07858738065901942,0.07519050772384007,0.09194758237516201,0.05325738204413615,0.08756386709562057,0.07566443991396885,0.08181965910252931,0.06344159013754454,0.08446837180510453,0.08000887429274006,0.08272331669108807,0.08775747607931825,0.0715961995223578,0.07641805521383316,0.07452113883669484,0.06477377222118078,0.06808985438490545,0.07287089510135367,0.07559539466633726,0.07327668208998028,0.07054020965885908,0.0853275628714176,0.07920503101882417,0.07626892511090597,0.0809681648941204,0.0805660588277203,0.06379995228338602,0.06712814056431221,0.08820661729176929,0.08824079756599368,0.07358348501850395,0.07500056654333931,0.0726814195425387,0.0743078390627167,0.07097645762540539,0.07820014968481262,0.07952201781914878,0.07540236712880498,0.072090814970339,0.07230831711747174,0.08113444953591448,0.07616715639411309,0.08690808627813987,0.06481580818302182,0.07226722543779035,0.06335080249626127,0.07452625680345157,0.07451424137729472,0.07430064753125566,0.07234196410539959,0.08384657130888393,0.08063542010299149,0.07928502586676728,0.07786416276145326,0.08442919384887648,0.08375363980074922,0.057532287970027296,0.07626800147778637,0.06964645988512244,0.07469343900096047,0.0776239250407044,0.05391865507739838,0.08726691410899522,0.08466315979407409,0.08058405043226025,0.08530563095873803,0.08068333930392652,0.057291134338602105,0.07339447998238285,0.06770896100140517,0.0749725575602537,0.07027032324921344,0.08152494401880224,0.09193516952759875,0.0707580941301397,0.06390843734735308,0.06753618319598687,0.08439912665641028,0.07404885131293149,0.08347311547129171,0.08552084296548168,0.07502339228147614,0.07076972741753179,0.08304126751956667,0.08302057623727384,0.06973936840640824,0.08583865625749193,0.06869003801636092,0.06769095112528375,0.07961744509885546,0.07618572813904825,0.0701061484824056,0.06758105818722787,0.0802981050506452,0.07843262344169452,0.07977067464743938,0.07247657375921826,0.08758871672237051,0.08685190646857208,0.086137309691731,0.07787958194362349,0.08456207138973332,0.06741660476862148,0.08638709923906124,0.08117353378130793,0.0846416683626069,0.08536007696599195,0.08323610344192942,0.08183379970697002,0.08612926171810026,0.08724213244564372,0.08290847562586853,0.08715958594580352,0.06825355364472924,0.07573128828718209,0.07977659693639139,0.09101199387674234,0.06703411917182064,0.07061579144785901,0.079603872388085,0.09004627715468079,0.06991897783152276,0.08029831368470233,0.08950941470483725,0.08521658394637702,0.06631354272168305,0.07914291265786738,0.07187216458886284,0.09100991870092995,0.09036021623398124,0.07599501392484237,0.07534185367827016,0.07495156189868477,0.07277945044571332,0.07396246770088667,0.07630885552221003,0.09075406299656698,0.07479301019090953,0.08512316432905731,0.09289847870871394,0.07456273956165405,0.08798026951049256,0.06771580644586866,0.09391723293366178,0.07137048777087865,0.06368206962639195,0.07517505983188053,0.0735833139455986,0.08163555383825531,0.08319003612431933,0.08203703058729517,0.07972482215004491,0.07618766986995293,0.09151410185101222,0.07439355569444525,0.050637500944710374,0.07708717730647482,0.07016833299976485,0.0816176135740937,0.08413602023387695,0.08226045819596728,0.07537220204209738,0.07437733957136444,0.07713863329222391,0.06031321981043922,0.07873102565124736,0.08826081493974845,0.08673769916539548,0.057552687890105,0.08311585572599539,0.080484732096633,0.07553792525270825,0.0917737576310915,0.07001621564491896,0.06331182430267923,0.07973628753986753,0.07337072841829344,0.07183477333641547,0.07810364221180396,0.08637609002455614,0.08036666299961295,0.08065635440429628,0.06739721515963026,0.06707696177277396,0.061056607455905826,0.07470583974664935,0.06885326116143956,0.07080432959808858,0.06863383675037632,0.07669215888411537,0.09102798882255186,0.08269151504680522,0.07633213579478158,0.08709268754441943,0.07650231325525106,0.08191735389083402,0.0703058303095698,0.09452464222574496,0.07558868150262449,0.06702791611691791,0.07247413488800407,0.07501568674212587,0.07771198938191204,0.06379838578639419,0.07637240406542903,0.07644326190542947,0.07810560346281656,0.07491449314954665,0.06327694618666689,0.07359012057069975,0.06708151360142381,0.08120531976666082,0.06812109234603561,0.05410377803876011,0.07820413595618289,0.08165626817410464,0.07562644002497165,0.08976649811239142,0.08341795653023486,0.06353345120699525,0.07682030030630255,0.08084308786834829,0.08183951828369634,0.0789561914609657,0.06890339371138621,0.0667495247690772,0.07187985118456833,0.08085714825519552,0.0839515152783198,0.07766465042065575,0.07848086268184647,0.07227734945023753,0.08825036485061455,0.07117407371049875,0.0702412896579978,0.08020636323654368,0.09280640793950568,0.08637802206334803,0.07230025867231386,0.08350279739287592,0.07381747587708543,0.0786932553587581,0.06677290262653682,0.0730343926842712,0.07728638812412747,0.0834928502051405,0.06398652876950522,0.06921856348858929,0.07160186475652947,0.08793958753101784,0.07630308769717797,0.08310406866152534,0.08402048424940783,0.06847800453568334,0.07350049936548583,0.06267262555913244,0.09096791937589413,0.08153169981569264,0.07366250486727907,0.07019411779409285,0.07570956873499572,0.08689848006457151,0.06695008752877839,0.07041167572214264,0.07434648364559782,0.08010125406430504,0.08764783820368258,0.07515059304700358,0.07350073929535184,0.08315857355153841,0.08634976776447478,0.07668285643003328,0.08736136748511006,0.08625745656472332,0.07333242612092596,0.07651267813804895,0.0696214972283194,0.07331568244251507,0.06959956814451906,0.07906717216458344,0.08025088441025177,0.060141057132631875,0.06774353207461217,0.06010592546601036,0.09006128521985107,0.07915365896650375,0.07816263197135966,0.07362522587054117,0.07951698892890988,0.07948215144785044,0.08093199151466152,0.09113690533147992,0.07984702580328298,0.08100522909383004,0.07530583675013225,0.08473414979491078,0.06436926020097178,0.06672554501777936,0.06418546226761107,0.08060230139995879,0.08801542109074262,0.07662133591368177,0.07527442463369187,0.08265185059216193,0.07185145932199899,0.08636015225181135,0.07601831568483429,0.06923584141882586,0.08447916846548612,0.06794311521222504,0.07134480690013639,0.08013279196148584,0.07334654030178214,0.06980275496012968,0.07827116689348484,0.08583213345727551,0.08791647045404058,0.06681191065334091,0.06618398283120294,0.06785904274313187,0.06735214654792927,0.0672158060382623,0.08338446428723602,0.07745602267487026,0.07955543916981483,0.089089320974111,0.07103377870568052,0.08849060537983218,0.0766494337103663,0.06807742333201373,0.0729243554559185,0.095600573678921,0.0817026099247734,0.07643978526162816,0.07899784027934709,0.07610256024301308,0.09431545896731724,0.07678207872072629,0.06886954483857809,0.07178429815271556,0.06654999377803997,0.07244378582591245,0.07019110136208515,0.07708088882613244,0.06767828265732827,0.07024370384729,0.08920831996364424,0.08311809055657265,0.06891957358431619,0.07478402241444203,0.07483616731452587,0.08612832943439358,0.06392007865408447,0.06727637389656477,0.0788145416344552,0.09139983312024325,0.08439449187531961,0.0795439154144254,0.062072891883367745,0.07620886095264509,0.08072541117007788,0.0838885451155681,0.07286282296533195,0.08500191684057479,0.07215897912898034,0.06816679220095544,0.08298479793000034,0.0825731689871927,0.0888833428439727,0.08294522586822681,0.06703518514616524,0.06917591816105229,0.07535983861618904,0.06862941419595989,0.08528202380259926,0.08704196237899402,0.07636695545680283,0.07848469552407478,0.06781335026540877,0.08875336192504835,0.07571587546135229,0.0751512107923057,0.07818771595724189,0.09381965009010589,0.06954363524054574,0.06422609516919221,0.0592347724886869,0.08568529762516268,0.08828285177120049,0.07146216100757796,0.0831410922812603,0.07566040481209807,0.07040282059872409,0.09274981960032952,0.07710508137717598,0.08045007672197757,0.07567418545832866,0.08401390245242867,0.08861499885921759,0.06485664511864955,0.07345388712547163,0.08004339808563538,0.06844456426378734,0.08909409578873892,0.07934211968709229,0.07509124890919641,0.07679841265297793,0.06380493548742465,0.07181454694122968,0.0728694416447582,0.08303455329860839,0.07488603732390187,0.07980172428096238,0.07460359377052983,0.07009299096797444,0.07735554764015587,0.08543208988010059,0.07704994120251632,0.0687349544162956,0.06957041971737812,0.07785036139410417,0.07143881103452024,0.062434947942436904,0.0818626633053454,0.0853526057229961,0.0873615810916211,0.08795415680969211,0.07715296184960341,0.08678866384756877,0.06862438755674871,0.07784563516755633,0.08447662051575011,0.0696725462162453,0.09258447982349,0.07169003595283148,0.0762483019761129,0.08554643857588279,0.07715935014543343,0.07097384304433074,0.0813065014460537,0.08761822314241538,0.06882991543164162,0.0858270512843455,0.0690613799799859,0.07950202775701184,0.07295945173076615,0.0878027292267539,0.0806818555572988,0.08014147826390952,0.07986186508484568,0.07762618039689405,0.08664411619432216,0.08650179784367246,0.06955255333786539,0.07281812780924504,0.0827256701726974,0.08024333337571646,0.0885715102214951,0.07005038262037741,0.06384634041630595,0.08289799786727381,0.06450248379177198,0.08457670832652955,0.07805312466103301,0.0688935395125475,0.08509704628148776,0.07898161852132847,0.06980239125906551,0.08945008057213599,0.08292725261158379,0.0733766378036077,0.07420120038399128,0.062059082948917585,0.07363709053461884,0.08030328300140994,0.07312586130186594,0.08379295200795839,0.06501806730618165,0.0950630003735192,0.07835655042700379,0.0875235338935285,0.06792559636742959,0.07253796071095081,0.08160187214483008,0.06992293153476341,0.07612483862402002,0.081437791126557,0.08228292294534872,0.0899569863204541,0.0726692562786874,0.0788842264304204,0.07944135699725145,0.07967716468930916,0.08080987052046726,0.07147688884535885,0.0822487077554375,0.07439146162515911,0.08155276514207566,0.07034588093117,0.08394100336942215,0.0883777637323315,0.07306401767597004,0.07825862719973067,0.08313378441086852,0.07694620287321556,0.06954279384273472,0.06895399754407894,0.06830770184159124,0.0728724151005826,0.0810799486041962,0.08894556280008875,0.09049874428246676,0.08941326968716522,0.07177296572020758,0.07204839066113952,0.07789268576126412,0.07174767226128084,0.0635064337489496,0.09176290040816082,0.08825355666355268,0.09001859288816959,0.0782596777145311,0.07798689712791015,0.08468852798692614,0.06838122561472047,0.07750992536496543,0.08066613394621287,0.07507204146122778,0.06949720008005421,0.07481792216194455,0.07999751567631383,0.07120947505225787,0.07703016778910472,0.07906783663615595,0.0851722784301181,0.06874663353497287,0.08082514036292962,0.07405110123068373,0.0647283292283253,0.08426689128055243,0.07922023356356492,0.08394352846506936,0.07660347276573695,0.06864671128611538,0.08116981857406916,0.08777185915710939,0.08298530308865358,0.07525346543721759,0.07581933257527611,0.08523137929742879,0.06540557535314778,0.0859202739598011,0.08673480987868154,0.07035451296462647,0.067064079680486,0.0817577870341466,0.08376144922533718,0.06633996507764503,0.07080135857881034,0.0732325268899418,0.07482844010781377,0.078492250502635,0.07919599274387418,0.08798167777947483,0.08108368185383673,0.07445175039782685,0.08809131095743437,0.08011945110325497,0.07430124984456059,0.06844116524619068,0.06019926529665599,0.08458865855066111,0.0609627722305772,0.07716310638081562,0.07304836039672696,0.07780524552963125,0.07208229583046936,0.06541380513176501,0.06498800797754906,0.06725342607839745,0.0765677128623562,0.0677384162125241,0.08230354019165852,0.07389612935962962,0.0827387878412084,0.06737277636307967,0.07699734382808843,0.08565014899014736,0.08313381672839196,0.07706840587266357,0.07130870243957467,0.08865043787353352,0.07886441135248307,0.07611058666323563,0.08223703243786236,0.05242033176303201,0.08737499619039246,0.0892543820066591,0.07682148875040268,0.06407338389738697,0.08156354837117948,0.0794539894161603,0.07176711505835678,0.08833395092107041,0.07842500329343662,0.0772143665310091,0.07044785519592309,0.08725419035658877,0.07531632129203711,0.07710612254574309,0.08054540624539928,0.0848630448039007,0.06595486306746046,0.06426979116733264,0.0733509238747993,0.08310715142262311,0.07069012660012527,0.07566228422777942,0.07653541515095458,0.07842661404384868,0.07277756160095677,0.07015768462746495,0.08847731729846182,0.06943553399754389,0.08929955954487635,0.0662784122956028,0.07865221775397138,0.07187610764823243,0.0860456902046547,0.07246543664854976,0.07335117815889622,0.07641470455147108,0.06133532045259267,0.06217847571321909,0.0916904110262932,0.076023416176945,0.07722080964654328,0.09626149731015103,0.08573551987267472,0.08673225926047848,0.061957163384646095,0.0842787044278118,0.07739176158939198,0.07740854587964077,0.07926529399530408,0.08292672949003244,0.06957452670664008,0.0781090315277754,0.058109932166475295,0.06378094069731866,0.08371471113599602,0.0820766953055365,0.08355956361543442,0.07449444994602951,0.07465494910589218,0.08310860649826021,0.06632405862193781,0.08205846458275061,0.08892789004709295,0.06889038260983316,0.0845233138047329,0.07548978184906001,0.07663615438821386,0.07633026841769489,0.07778670888717981,0.0731084901902194,0.07265210118537213,0.06239172668438531,0.07450174289078383,0.0838719183732563,0.06994877684412405,0.07466745002853689,0.07505868058556253,0.07182794873282453,0.08002267047561384,0.06873718122288236,0.0866207944283928,0.0732312326484624,0.07133460985677087,0.07258105604413864,0.07829425949646635,0.059010856997862994,0.07574333483721697,0.0838407962110754,0.07209907668868898,0.07890434226986166,0.075981515076007,0.08082008535587701,0.08399086186736976,0.0702243967412185,0.07626471285226255,0.06768982955798258,0.06848432207141826,0.07934821515037699,0.07533898503971591,0.08868904843179092,0.08087075757998542,0.06991192679244584,0.08392498253743588,0.07958492988418242,0.07127025080793623,0.07377458761070602,0.06294264823216462,0.07981458466557659,0.08763891466559628,0.0789631975324529,0.09047851711755027,0.07431692035171258,0.06881131654523534,0.08500564286265043,0.07618631784818064,0.07568053369910294,0.06950788577112588,0.07056511583953268,0.07635507008286202,0.06272735561937753,0.0630264991252742,0.07494744042391308,0.0764046380050088,0.07949798577664162,0.09625551046156011,0.07563301509008225,0.07986670334581536,0.06578565927302918,0.08246663075061825,0.08454651290303478,0.07496676772640593,0.08516048530489928,0.07892162046223394,0.07035060830597126,0.07058416761715351,0.08501142950079439,0.07562359876081327,0.07429367447028126,0.0808584868309575,0.08662614115806663,0.07454203061597356,0.08196726500433672,0.06933622901107725,0.07403155520386588,0.08494706354447315,0.07288314738208733,0.05656049121660386,0.07270759363400275,0.08413179858137837,0.050978058144989326,0.07730400469258755,0.07778355290944103,0.06373176725056379,0.08433451079961486,0.08541944016081715,0.09072150082789768,0.0709610640215986,0.09111798845280569,0.08285913251980839,0.0736116312344299,0.07585664771561221,0.07638265858655331,0.06931312573323736,0.08352351511776476,0.0796326002909018,0.09293782131709584,0.08764585602255001,0.07485890083006608,0.08564997723668498,0.061902477883112816,0.07378787979622714,0.08916555080535123,0.07081247897129239,0.08358213931846499,0.08730092575869801,0.09022908988889247,0.07586540663738414,0.07018479488897464,0.08242176136085552,0.07635400126915516,0.0777142705492645,0.06451049232415093,0.07090508554641159,0.08147322188238412,0.06164648386433952,0.08080236518365173,0.07512470618951339,0.07661172938827415,0.08855476690947336,0.07762186686836281,0.0663270574647366,0.06144357159414229,0.08071796467160805,0.07732578924839673,0.07002998667146916,0.07557717459098649,0.09139789378921942,0.07964326223555235,0.08590311915306473,0.08514315152797483,0.06915402922209649,0.06463753902528335,0.08492841580171695,0.08229163592704924,0.07004491049430833,0.07658238467045918,0.08438871545447604,0.07168991626593525,0.07680805654430611,0.07141724399368012,0.08050953821884463,0.08507382973393865,0.09494184104766326,0.06530894793108172,0.07126216302121775,0.08541225657622611,0.07618841206456663,0.07429045132764704,0.08517245532647863,0.06803061923715999,0.08853338874475604,0.06986287055784313,0.08471690843982682,0.06632104964533925,0.07215687019470156,0.07561888618889692,0.06916032009749998,0.0738833015815678,0.07701712923869726,0.06015162755136155,0.06209063818032803,0.08488785493763494,0.0843937186898824,0.07410638093857583,0.09095188182051017,0.08317035058502378,0.0890638805377788,0.08923801510150323,0.08580133658697542,0.07385414663390874,0.07977274017912227,0.08124857754152717,0.0869357202192538,0.07243118110055038,0.08980373878298605,0.07683647618894739,0.0902420796668105,0.09134323550014313,0.07756878812282028,0.06936212369445066,0.08413508471788925,0.08880195440994196,0.0864177225517688,0.07003415977294787,0.07429383691240532,0.07646236047784954,0.07736884562480759,0.07175250662224716,0.07091703451567942,0.0880848791107976,0.07802635842514029,0.08078031568305577,0.08797444351973571,0.08323872599433174,0.0844729051614088,0.07427514234062776,0.06408977472392918,0.08436074680754249,0.08463734116532795,0.07105758953105708,0.0816004993184287,0.09144074064124343,0.08347166444278767,0.07555849658436732,0.06945198372640488,0.08636871368652396,0.07930899304197592,0.07811113569687456,0.08076589241105502,0.08448313506871134,0.06729232959483378,0.0728091895327669,0.07233016226969953,0.0793609927230606,0.06497392807526248,0.08460518644459426,0.07205447196443904,0.08150168913791321,0.08838903164637038,0.07190205229335762,0.08529708004602202,0.07312144068044035,0.08071579895500598,0.0749165298900149,0.07271638226231575,0.06438006255132131,0.08418730438932234,0.06573095218930297,0.06278367112128289,0.07982996302618935,0.0890965192036291,0.08050286490625545,0.07943045133661364,0.0660754098650575,0.07514803072376451,0.07799523561281686,0.060264158154053955,0.07393428507390945,0.05986692195141248,0.0854941902849129,0.07031231574034018,0.06420752737543271,0.08350944240610716,0.09076717011113333,0.08448166593813167,0.07586451234106781,0.07781078082412629,0.07743613273575675,0.0803200423003595,0.0759368184344761,0.08325914502046203,0.08326552701553876,0.07524488632099152,0.09263789289297421,0.07453124443844433,0.08333998599776421,0.06801502200645386,0.08032204202411714,0.08658488283494087,0.06592480107840876,0.07691998579853218,0.08658359354814432,0.07333933391455638,0.08906614273532615,0.07771456191976503,0.059237579237035876,0.09303600855099502,0.08240545072429512,0.0877386402664753,0.08305135062811678,0.06775504170259136,0.06687454507091135,0.07716627379943941,0.07801097293919745,0.06978934244125298,0.07732838041811263,0.06385702198812479,0.0754069587012368,0.08806575344605116,0.07656539649204516,0.05394217294262517,0.08015904844477117,0.08276709371243718,0.08285946957144134,0.08087039073101188,0.07175429014673614,0.07380112838171471,0.08857131976427857,0.06348097546373882,0.08887881198887129,0.07499861145268577,0.08329560316243383,0.0779186791370576,0.08172926455818058,0.08068307703195324,0.0671799658055868,0.08527554362444863,0.06477492638630832,0.09235812113671012,0.08421242286418519,0.08645005767981964,0.08354134004721987,0.08655777581312783,0.07539046352867457,0.06819480654780287,0.09347875453634215,0.07686226139188285,0.06817614467317887,0.08402732973880644,0.07479374403209113,0.06708527841101003,0.07125458157895331,0.07389099344110547,0.08350381258871838,0.07540384816311915,0.09539445362241111,0.08866420668580272,0.07982992180962874,0.07981509662317926,0.07468049533112173,0.08194282111863578,0.0662647195690973,0.07728309086555128,0.06808743528627567,0.07141818598160755,0.0708692580112266,0.08813662121950436,0.07455469190004493,0.0939885548287277,0.06477819488568959,0.07986067283972954,0.07353173918122442,0.075834863722467,0.06488215864554712,0.06898040416587595,0.07289185332769064,0.07028452584932049,0.07581043415550039,0.07498072517616741,0.07733234785312676,0.07855224105009087,0.06643321927732873,0.06609168356582372,0.07263763488756923,0.06569350921354472,0.07964819629803979,0.08708576743158858,0.08462691773669107,0.055813358100765514,0.0729337721212184,0.07128704253673336,0.08548115763617327,0.07906458987263831,0.07157353046874271,0.0927772406513713,0.07441039792481519,0.07867712409519535,0.08694177398473218,0.0632282040859405,0.07742067886063311,0.07065163669202106,0.07178029681119943,0.08354833406445078,0.05996796285979667,0.08213108750397866,0.09417766572379219,0.08243441909637632,0.06389800068786486,0.07429707470927639,0.07180364725657963,0.06199430194801752,0.07784763635034635,0.08601809838248288,0.07491294442744742,0.08081037399876395,0.0756243619714562,0.07680745824686065,0.07779233716965975,0.07788194194508596,0.07444355585080192,0.06271258313971759,0.06727262185315727,0.08361228008902388,0.08137683631207301,0.06922075322416574,0.07268990020984475,0.08477898180034778,0.08051718340699743,0.08318049476022309,0.07741178551289299,0.08436251449936547,0.06464152675174838,0.060093125338869666,0.07290638747945771,0.06351086451966395,0.0887135560851233,0.07399093519746233,0.0721088605404912,0.0795331974127034,0.07711768093653762,0.06851738824772108,0.06674454206628307,0.08090546979609134,0.05916970824816585,0.07083272534334714,0.08045221958636153,0.07320652188755539,0.08288169293196856,0.0798765931394537,0.08109025892383268,0.08714871395269258,0.08006893861806133,0.07031675406858685,0.06629191530317316,0.07371167306795832,0.07681935121566905,0.07590773060311282,0.08157455836007005,0.07969751433935829,0.09281559023176582,0.07449415152112335,0.06093088697921499,0.083763196920389,0.07539915498372318,0.05849229810206506,0.05887101203272789,0.09220723205914205,0.06685952243087737,0.06285611489289462,0.07277178374416096,0.07535630186281406,0.07768177943780889,0.09297121584786726,0.0858412335415068,0.07602723179367538,0.08748264999959454,0.07405218963386626,0.08599466540704273,0.07304311365509784,0.0853817365000212,0.07718095370204969,0.08075558535099064,0.08514942010906648,0.06473035184567505,0.08055774603661904,0.08928455638566626,0.07679306394585964,0.06024981798476048,0.076116313459885,0.07018921357928998,0.08442421276964929,0.07378330677415607,0.06866423273467548,0.0801504473721355,0.07815074280420364,0.07621875641901493,0.08401573100800475,0.06957429754994338,0.08150465370022605,0.06408436855640827,0.0840815417441757,0.08491395214460094,0.08429447100292577,0.09046371233249507,0.08957168666273926,0.06847619486920423,0.09132606669013713,0.08351345323810469,0.07085246561793605,0.07220229351580595,0.08463686747116467,0.07770456082211158,0.07112507838869156,0.06838086031887639,0.07599752067912827,0.08315869356865449,0.06494963980967687,0.06520345606923789,0.07608258170278351,0.08624500821667919,0.07929384400656783,0.06891585461869704,0.08288418211275113,0.08370673332012099,0.08052393262001027,0.07924678232320101,0.07883853602242796,0.08514603562989584,0.07623449618997885,0.08425649205084842,0.09383740788018857,0.08466120682719347,0.08260777578942285,0.08062167129892175,0.07713409290004819,0.07823344862067985,0.07008142671488465,0.08887088320654896,0.08339455749810529,0.08356841774842574,0.0738344982073951,0.08571106752898555,0.08657255038479936,0.07598735381173116,0.08000183804209715,0.09098484612328875,0.07358852778303066,0.08318255210971767,0.06538564453405973,0.06161305908496475,0.07025538269049558,0.07890994740022605,0.07261421485911773,0.07215852194156343,0.07101289505637098,0.08944782079716705,0.06588964650396723,0.08192950096787054,0.06489764018965857,0.07972161959053331,0.08541540867989496,0.08029806763439558,0.06513372632813089,0.0866003055871742,0.07407719883647734,0.07643742889300048,0.09368142707853512,0.06537939927477944,0.07979163761109753,0.06978594997455242,0.07997923922686645,0.0729996284488574,0.0867867045449205,0.0861895742085943,0.07911639937397702,0.07983121007408471,0.06663392456726197,0.06838729604024502,0.07961986233377069,0.09190658394826304,0.07372731006633063,0.06371329051395785,0.08554620030530166,0.07824852278110403,0.07857773220143316,0.06512916957870918,0.060410993690548496,0.07825435418789325,0.08894526774780431,0.07444890435967164,0.07192310025312171,0.07099425037283948,0.07552457700706446,0.07735790233542894,0.07003674907821791,0.0683523565946815,0.05299582804099275,0.05773948770191478,0.06417580886415902,0.08374179621251449,0.07923452564002874,0.05611587432607706,0.07437252294313701,0.06543014903825871,0.07953843891155227,0.07397547867483473,0.07480435440225786,0.07821219243996841,0.08732074480330133,0.0838831650804421,0.07911470826923718,0.07568990296418615,0.06097104896838271,0.08575965892699769,0.06337396499570142,0.060822199698972894,0.08038504864490484,0.08112808011788557,0.08619818986894298,0.07992245711169794,0.09219868138650238,0.056805785860848626,0.06953101409892458,0.08009949265586702,0.06718758438711868,0.07746991199687427,0.08498543729141739,0.08332336175383459,0.06861790645901017,0.07389805961057147,0.07462675290322153,0.0849150198968182,0.08225252313139915,0.0835583276760006,0.07042029087795884,0.07629139836339809,0.0762574715842751,0.08740873670624119,0.08288101124247643,0.09156864420102523,0.0694486527763166,0.0803868199047533,0.06204223468508713,0.06252102711790877,0.07577904897510958,0.07851762896193228,0.08558341848525733,0.07606325388183871,0.06394928048640054,0.07406727363592247,0.06959183723785117,0.09135112184341018,0.08770742070855056,0.08002695892189723,0.08354318535877801,0.0894637118740425,0.08248110605154578,0.07830000849247384,0.08245087849829778,0.08514595815447931,0.07782144342112937,0.07934739318846086,0.07533394380796915,0.08874561547100293,0.0767124783705487,0.07178420066978376,0.08496082949892356,0.0836608663378024,0.07942732546353612,0.08723611016769892,0.07225219274393221,0.06303588886495645,0.058488898628710165,0.07540607366534718,0.07318670802856639,0.06892539197327288,0.09037374426186978,0.07525700484571743,0.08691901180110062,0.06692525677221897,0.08785121851714514,0.07273014181167642,0.06201167432111763,0.08025487081444946,0.08510288792213047,0.06808572392244237,0.078394372642205,0.07599139499764013,0.08368052763073394,0.08036072144159079,0.07383869107741514,0.08340493228895031,0.08509936533293619,0.08707194789226341,0.0781759953484476,0.07913724510417827,0.08002460907601944,0.08454784641139065,0.08542847075792946,0.07238679350201654,0.07798571623841569,0.08149553107558644,0.07148840899986905,0.07183685478335998,0.07256072431945398,0.07759703712115268,0.0832124399874889,0.0774752834207017,0.07804864955123633,0.07752900479528108,0.07738231128267267,0.07450800561630894,0.09412070863537438,0.06656653599780836,0.09090139157515592,0.06669528362662239,0.06547755692277415,0.07846718106684672,0.07576078596455854,0.08452317621876845,0.06571104448449655,0.07435157332636896,0.07724418577287666,0.0854177998141948,0.07985198214771644,0.08380900165914461,0.08351762308428246,0.0731777184614753,0.0916369946720767,0.06480027285132628,0.07199124116404496,0.062019340692728714,0.07178916002034369,0.0640997614144309,0.075705915029615,0.07756532313282266,0.08425036760155397,0.07703967158115238,0.07742788862296762,0.07427118172241376,0.07839814401229808,0.09216424043091595,0.07523589034306474,0.07234332190317,0.0782599390497053,0.06595668702536638,0.06797171803733817,0.0613476957791899,0.07934068806109376,0.06188650315407557,0.08663801840887839,0.0744423640570886,0.0848366271209745,0.0808296757579292,0.07772039904269248,0.06754787530432899,0.07870656862136813,0.08111563698916453,0.08485099662226937,0.08524093227588679,0.08610855940998226,0.07967911894937123,0.07734315724097188,0.06941600448401428,0.0744584020865754,0.08218761127782115,0.09447328590265215,0.08363903342687133,0.0706440785250675,0.06108429500865669,0.07562141052534138,0.08128076222262536,0.08180647376960487,0.0725341067868158,0.06405121514282232,0.08069355461242278,0.06869393893536364,0.07981511212347359,0.06366183265378525,0.07128844976057885,0.068454612009309,0.05689105144586991,0.08517775945031462,0.084200442316508,0.06167312106407657,0.0757303398846623,0.08381408044925742,0.0842030851808276,0.08460448692480585,0.07930027395946326,0.06899581826411763,0.08122500403342525,0.0835252135231352,0.0914606482401534,0.08278186808090428,0.08083203029866545,0.07346368261350453,0.06834193460288426,0.070999118917423,0.07370664250085493,0.0765329651922477,0.07869279587410521,0.08254825990445824,0.08330118935780931,0.06622023385324098,0.08693303445439875,0.07687042776266331,0.06620256262448636,0.06906451143530312,0.07036436696291633,0.07774149330795585,0.06058210533423521,0.07425959054008446,0.07606101764899131,0.07997964444616913,0.07346489440789783,0.08355269204664019,0.08034442874642847,0.07122866176410526,0.08095560415825914,0.08480722658817501,0.0858893801064145,0.07161175306143831,0.07973293630490827,0.07711930279102383,0.07950165052169385,0.07073374510522498,0.07811717422569503,0.07404911794564999,0.06713279720632044,0.07743941376988064,0.08240580758705057,0.08827293774294029,0.0881333310442246,0.07227428639871726,0.06637099261297622,0.08543613594090939,0.08604097755814995,0.066838583118267,0.07953401157633283,0.07813611278847969,0.0810990923827031,0.08109291372040286,0.08400628592330768,0.0798577857350227,0.08885692281248725,0.08228904666475689,0.0846032082621498,0.0783532312593991,0.07137910773758359,0.06350509327177167,0.07810518896450444,0.07411828055261363,0.06252722831533775,0.07551475527289775,0.075349131857661,0.08029533272542752,0.08332583403014712,0.07828628806967702,0.08419661061651657,0.06569183785851801,0.07467291384079226,0.08077069904037168,0.08991759309498576,0.07224645400239689,0.07525413616793539,0.07603254882165059,0.0884270412950526,0.07013737904862616,0.07160747557123506,0.07851442000591746,0.07385145603937297,0.07852783549877132,0.07160771464268516,0.08156594350874263,0.07641028661012247,0.08380571112424004,0.0795457926066869,0.07141183193133305,0.08460031344313468,0.0690139761048095,0.07465190526643675,0.08481255934954698,0.08468245591474806,0.07841475059487593,0.05680123985511957,0.06929506474397198,0.08826226627862371,0.08858690713434782,0.07421009885124068,0.06846414538290047,0.082353183061082,0.0645722399602284,0.09077286298443782,0.07401220333558586,0.08653586890746948,0.08593488550791718,0.06577852447220073,0.07552608604772346,0.06897921861116724,0.08240141125291711,0.08397873258023206,0.07669025459441543,0.08311266353401212,0.08572761076063909,0.07925110450527821,0.06703863671899651,0.07241062227986102,0.07184057820504568,0.0889844531591435,0.08768016065846104,0.08936151999071978,0.07095047519510368,0.06884598152385112,0.0755965791886219,0.08369971345096189,0.09047374181261747,0.08571386611228483,0.08106434961463282,0.07359599153844565,0.07742063528758548,0.07571463787043033,0.0672718945674739,0.07553755662551533,0.08686484275679951,0.0710541895740518,0.0815592713967333,0.08542357527222019,0.07077071757819228,0.08259754173596254,0.07349701291547332,0.08872502382151075,0.08347428548935495,0.07531751354964349,0.06661606413592011,0.07332877947993119,0.06904427224317473,0.07470996532578103,0.07317020222487673,0.07586014363063416,0.0817129459223412,0.07217780038438351,0.08463127898365168,0.06529723600688103,0.0766749787054189,0.06471600860075336,0.06796037565152509,0.07287216220545684,0.07121024082691049,0.07282973308689092,0.0783623301749845,0.09042900218446737,0.08769551847823999,0.08985304925003304,0.07502542454756031,0.08917180989626783,0.08047653696294467,0.06298356795174345,0.08664182445810927,0.060398973005627216,0.0790801200593346,0.07121266589742885,0.0740158260652682,0.07405637682364485,0.08336678790982596,0.0775832130778206,0.07229279606887223,0.08891946248320964,0.08532470238696874,0.07881309978611029,0.07810272614802755,0.07257680599410371,0.08009306176165745,0.08743902516341481,0.07847812705658194,0.060674203030956705,0.07590722623702997,0.0940515036524264,0.0826715250849195,0.05391363765158795,0.06708303938170343,0.07293105316347503,0.07936861740361185,0.07867097117076441,0.07284971941020225,0.08178268326737137,0.07683807664599385,0.06852236793342965,0.06858214733677587,0.08124691142508597,0.08560211651218248,0.08420615664563777,0.07445880296278762,0.06851758935464275,0.08560445408698739,0.07832124295998713,0.08518442710457158,0.07397292778938691,0.06756218889098636,0.06475827732771064,0.08286385015252246,0.07074219590922454,0.07286371712514052,0.07326905491060742,0.0794211054459564,0.07915002293257989,0.0754539633459102,0.08690293269779542,0.05454334529861396,0.08647991021131865,0.07921695058538197,0.07095820884785964,0.07982509011613212,0.09081909278715633,0.08014093839730341,0.09020562580010602,0.08233656440415876,0.0749862029096231,0.07368028003970153,0.08748840381117483,0.08743291148656082,0.08884022288995294,0.07156645475713867,0.08607518480656867,0.08121214299802917,0.07080187339886201,0.07663523288897778,0.08773956237260289,0.06756390377278115,0.07858075147194342,0.0866891831158291,0.06570672945443351,0.07867544718200856,0.08221418329363787,0.07264027394629147,0.08569321605560778,0.09183010672588326,0.08633121513288253,0.09448109448308474,0.08647980883241341,0.07753810092847624,0.08197344436634879,0.06572397091486507,0.07567378921964074,0.0663940228947639,0.07795236871224354,0.08441627031193855,0.07483314873125646,0.08244085495245115,0.07883358060963508,0.07453809969805558,0.08821401236777689,0.05367645115665515,0.07371730486938378,0.07301861541700116,0.08662337965417338,0.07450506165546655,0.06903522391855546,0.08114972094887515,0.08715072992731451,0.07241945838956007,0.08084634240991606,0.06957134424185081,0.08778730706296872,0.08030843671998975,0.08162719249999653,0.07535672425643875,0.07816884986941561,0.06323840834177957,0.07246164223023172,0.07903079261599616,0.08388584022653064,0.057636849006368965,0.07804561453755823,0.07513965534283244,0.07096723679947156,0.07734190223423136,0.07541894368101593,0.058360169617947225,0.08276343071808756,0.07443513949683389,0.061402836857681525,0.07723461039088837,0.07610142844451956,0.06053416480624088,0.07018706622526075,0.08452245269167383,0.07921099357615177,0.08382886599222814,0.0674555100789312,0.06270130738742544,0.06403439522953638,0.07262509516017225,0.0930423230728822,0.08126673687964886,0.06101254769192446,0.08741095279609598,0.07969925517357354,0.0836362000403164,0.08597842837137315,0.07120385226671694,0.06996337281497227,0.08182018497743646,0.06661343671151645,0.08403422145620408,0.08329578565912438,0.06132916527683317,0.07362665203634755,0.0599053562968453,0.08538766944447695,0.07138992257646155,0.0803793001843898,0.08981395636339985,0.08595179619205004,0.08317553306407667,0.08686311156387856,0.07416928296481812,0.07179419503354678,0.0732987393273917,0.07542584754851878,0.07387485833268,0.08329628721522209,0.06896327343579306,0.08157825947464863,0.07896956311883553,0.07381717062811177,0.07994035108416042,0.0740445987352348,0.07457094351669624,0.07199308085503694,0.07640619151179526,0.07908979964531143,0.07824291842294204,0.0883737439237228,0.06572835544242861,0.09561820423538575,0.08488429603996896,0.07232420999510299,0.09199078299373703,0.07458356583961698,0.08647256049248453,0.07653933594207477,0.08241554371032535,0.06517818886527398,0.08964432842678874,0.07989051293891039,0.07816817627109815,0.09045007782019637,0.08098581964584764,0.07866309935863634,0.07815723416498473,0.07634550029351916,0.0784982839235119,0.07786942105516445,0.07631585115776071,0.08200798534620068,0.06839526897774978,0.06867339933643343,0.08578977727374738,0.06712661359147234,0.07061180759864341,0.07548763017259175,0.08407682117501267,0.08644748126483867,0.08507753106151211,0.07167510761241845,0.06036181616473325,0.051112516927896326,0.07259084488437663,0.0707777794751897,0.07750427168833907,0.07333136552727723,0.08525759403279762,0.08108978845149417,0.06664281083628565,0.0887148773163095,0.0759629497845016,0.053445798166859154,0.06791901595424824,0.06700761064117125,0.0714480022795318,0.09169632262611405,0.0755864992393126,0.06993163825497076,0.09184462234708317,0.07708372377615791,0.06196456838671842,0.0767282312521874,0.08708539585908961,0.08819950193673598,0.05860942927866364,0.07128284726431894,0.07632315053594754,0.07424723379415818,0.07579381590226908,0.06506973557201955,0.0796662935760449,0.0690023447018829,0.06041652406021455,0.07634889049419893,0.07641067640280289,0.07991744076192378,0.06548452247340472,0.06733385653560586,0.09169529014556585,0.0744191943122137,0.07567141705382324,0.06955601501460959,0.07118534447004349,0.08113206620091244,0.06780597206085409,0.07583628776993422,0.06986956224208088,0.07256503439830189,0.09109984457213233,0.07117805424406487,0.0787890386348042,0.051808078070468486,0.08774355548220698,0.08464299349391563,0.06571250451913863,0.08497658009058698,0.06560710892535812,0.06968928842070958,0.06040638532577414,0.07000139081427205,0.07974084541795456,0.08312538845103441,0.06295385499926151,0.07861783191029265,0.08041962781995057,0.07151056533147299,0.09022482903230017,0.08063475537494019,0.08137264384348496,0.06868287523253372,0.08160377457449966,0.06145680606907465,0.07000921623834142,0.07371509618034455,0.08417431300989534,0.0708739451598393,0.05439833561882816,0.08294956259209751,0.07281424283766574,0.07940012683660885,0.07119792996739296,0.08038388450616868,0.06534139902610753,0.06882076448647494,0.06560222981074992,0.08115821570475171,0.07687030400361936,0.07989195045822603,0.08060294923118147,0.07707558077409978,0.07453415641211938,0.09348421163988911,0.07619007040206753,0.07939804050061951,0.07656394103427369,0.0779271480640556,0.08430183784320842,0.07370689961296538,0.07709366832920089,0.08604340037846843,0.0829049145652617,0.06665302449312904,0.09142024777595387,0.07534606068735081,0.06534143486428981,0.08060825020229324,0.08457986835265226,0.0749601624157605,0.08195336329002129,0.07086028801767165,0.0737075868340353,0.07293894177615487,0.07159066955228717,0.08034835578028923,0.0777046082229523,0.06754806964153509,0.0643302552522389,0.0759974669193701,0.07514834402894881,0.07847885767465074,0.06983757434976288,0.071702284380387,0.08058060474726986,0.07712610197662145,0.08568320320407238,0.08216215496964463,0.07967916680271458,0.07897144396920741,0.07347231932576373,0.05615073410978356,0.06832399360250936,0.06361275372487843,0.07078860490724982,0.0709497362606685,0.07255194599572487,0.07242416166871025,0.07743819485435391,0.0662266928149342,0.07035637154429195,0.08814188166738393,0.08112018986956261,0.06742601827981043,0.08144792945671601,0.06690281411968466,0.08813227995934347,0.055820781810731385,0.06479341448413077,0.07395506417109853,0.07387488280340074,0.06702754118959073,0.08057103117483386,0.07911480232202311,0.07297791181879054,0.08404791633716893,0.07824524759496182,0.08888198041705767,0.07657770882792006,0.05899314439310411,0.07881904708711181,0.07821300065381995,0.08345576917250763,0.08354732777962726,0.07013405201188187,0.07578414798655247,0.07403668942500406,0.07916531090624457,0.06752268136826824,0.08098535854588867,0.0779740078226475,0.09385345602639462,0.07096624411337085,0.08525026669533643,0.07958069459918465,0.07208139962556376,0.07338359586873774,0.08341702977802845,0.07110996212465778,0.08291564596108375,0.07417806007501251,0.08986313990794428,0.06532717909535396,0.07685258072604523,0.07602256808606672,0.07659337883555195,0.07235874947333218,0.07696512181965126,0.07264485235631968,0.08121526753334998,0.07169181220359465,0.08472253581273992,0.06814041929302851,0.057586348872367375,0.08834400142872376,0.07082564585224982,0.07093045694492825,0.0918569579906497,0.07922000625931457,0.07194057753365322,0.07644369018459113,0.07619199166050158,0.0877106423363598,0.078872269795316,0.06842106774303222,0.09280214396229085,0.07430713330394545,0.0865997549703032,0.08924639291570718,0.06666370065334133,0.06415247982383494,0.0907578169190569,0.08503311051890125,0.07670102414389239,0.08532641286857431,0.0665841262709516,0.06934538020342748,0.08269744015702894,0.09396081750987836,0.05819265788552129,0.08199117777359025,0.0711059943766495,0.07762493985032456,0.07770739699832577,0.06987207042374818,0.06614004852058328,0.07596503327816884,0.06022363168834648,0.07535446755384331,0.08167024295380734,0.07104906057779437,0.07299031653372157,0.06548387155420965,0.08331387315540095,0.08389986544199987,0.0761653485392107,0.08554528003442494,0.069868113275563,0.09044887861261589,0.07134975745678195,0.07900564374952072,0.07228858604878816,0.09582153806149203,0.07211706523867285,0.09218648337391902,0.07557405534525885,0.08144292163321264,0.07794355650920227,0.0657073306346395,0.08149667182584172,0.08043408804834765,0.07624266042473724,0.0848334077375835,0.07039961656199001,0.06577593899522927,0.08384263710672757,0.06618894384611218,0.06341030531385555,0.07225654356367366,0.07067073052789094,0.07729026952948531,0.07129733902144991,0.08256671695807537,0.066355936807492,0.07683710753781044,0.0716011860085712,0.08216886119119604,0.08347078923675097,0.06914749592783598,0.07975423145508892,0.0752469861834322,0.052710255638449444,0.07431714206025543,0.07555509821470272,0.09407985148815075,0.08328684881153733,0.0866049312705802,0.06246671097545475,0.06643725229224968,0.06603937557196596,0.0882862998948471,0.07320596048968046,0.08454818864817253,0.06542238727922695,0.08945116093858749,0.07563724209143516,0.07769765926896283,0.07228628389477905,0.07699181211597206,0.08289179712124373,0.08147543513117456,0.05915787302143954,0.066406710144128,0.06658493526005313,0.09072247780085153,0.06299352769673659,0.08031678155499042,0.07849447454389524,0.08553481159554832,0.07743480540518732,0.0744824646729053,0.09428625674428172,0.08037092583201595,0.0881318836352993,0.08658645046685717,0.06918909559488992,0.08316249723053198,0.0800898901316067,0.07181954808311986,0.06918975686795897,0.07977087855362663,0.0772655896303976,0.08287533304611407,0.07988796540611717,0.08097550400722199,0.07541096040289871,0.07116680009786402,0.06363520978997453,0.07319138790393603,0.08011604960059554,0.07470416405714858,0.08871807281913771,0.07446636494705502,0.08746992643897067,0.07262782332969356,0.09651958381030404,0.08741399467306459,0.06513446258987687,0.06155250124998938,0.07748305165964192,0.061822870771294364,0.05773869084237565,0.06694021321290428,0.07653090395916486,0.07022815527920499,0.07369344288556148,0.06705745296104274,0.08383134793082854,0.0727856197051687,0.08170601455581986,0.07511004862083756,0.07560909981048286,0.08280824469306758,0.07123649168987076,0.07816880720260944,0.06127851214147454,0.0737647314540826,0.07169446157663034,0.07466510698367006,0.07564260955151234,0.08297995070192221,0.07636413408149967,0.08310746426779081,0.07036850624873506,0.08406373593838837,0.0639985813518835,0.07961154485960523,0.08936496329811253,0.0756735650733551,0.06978632115761235,0.06515349271010676,0.06709964105936288,0.0754387323553502,0.0723122396795885,0.07804490305378725,0.07471322240000029,0.08383742247517663,0.08039933956325401,0.08621532514663077,0.06895564167360094,0.07828111332329299,0.07262916996847754,0.08117957366045075,0.07128977502305142,0.07387429841797308,0.07326537448946573,0.0815530926669879,0.08345806641499104,0.07017832498036063,0.08909932042921492,0.08448629052013223,0.06867370431626812,0.08583879525206643,0.07616657560156787,0.0650326363810472,0.08204769939373804,0.07172730525482268,0.08989076320646236,0.07556003760017424,0.07522612405438228,0.07685427181818033,0.049002386441969206,0.08597743343107675,0.06615450367655576,0.08659791431304803,0.08586760063644802,0.09470261078255503,0.07343992414917513,0.06880096397783181,0.07912877305755864,0.0727306956591825,0.08417445995117542,0.08061829379454283,0.08665598595023083,0.08492602511069985,0.06056618670829825,0.07386551117991147,0.0839140095471422,0.08409261132124138,0.07188765879418543,0.07452841668617255,0.068313608128616,0.08163141437256667,0.07919010702978754,0.07989210312747727,0.07781577104717663,0.0835883868149673,0.07904834412540326,0.06465314203167921,0.08460652329713525,0.08125645220561512,0.07493858372508247,0.07073761456829,0.0667578131606344,0.07769166835767458,0.07292417746450333,0.08118122789343055,0.08081302740239996,0.07060068855967803,0.07044149718836934,0.08595427640055919,0.0709353154809816,0.0744153440777224,0.09031945702658603,0.07876913761327232,0.08863000034097072,0.09287220221460549,0.055234453421883226,0.08215052382389205,0.0839887697037871,0.07846747258951861,0.08263619002694925,0.06417193844491771,0.05746499213054078,0.08645112847506134,0.08435681702925066,0.08962279614882482,0.08245636640854503,0.07237551824444488,0.07714772738913533,0.07227266307384822,0.07006424462539756,0.07422904276933713,0.0816676183621237,0.08327907405572502,0.07109575114647673,0.07172213547533876,0.06403909953332565,0.08432399581519322,0.07472482163562148,0.07486714589811472,0.07206068612829356,0.0743696045356606,0.0904682461717081,0.08510830555800795,0.08573507243279535,0.07223320841931999,0.08430665201227552,0.04991794652103678,0.0778199761404843,0.08283637260155964,0.08482009899616608,0.08140595082436272,0.07894403043858646,0.0630403916299391,0.08113106595327087,0.08636583295753336,0.07439523900590776,0.07561086008839613,0.08142253751698816,0.07020196087223658,0.07677350415085873,0.08239554358318579,0.09521671094649198,0.08825921732332974,0.08074918436049856,0.07588841158794335,0.06364291426970703,0.08764833006424744,0.07932097062770099,0.06698036673311678,0.08522739730326653,0.07773583445262691,0.07461210336323243,0.06942938871091517,0.07955135417569725,0.0794625280148859,0.08574039992081942,0.0721587187752763,0.08490624042993372,0.07276756177688459,0.08762557446186953,0.05851212881207522,0.07806347011465993,0.08412252843099816,0.07631950689640028,0.07040799404329554,0.08925329467845286,0.08964426289630327,0.0760212273488437,0.07509383099069075,0.09269945897993721,0.07137844459445286,0.06771259172321098,0.08012592606543921,0.08736255870922664,0.07382668587924221,0.07351637059542446,0.08621343185290314,0.06646299314245566,0.07892210555062645,0.07150089035311853,0.08111620862180213,0.0834241445121133,0.07800011045829558,0.07671501532952399,0.08254587167648372,0.06410743050888201,0.07756857558781995,0.07404347461813798,0.05626352037908121,0.06732916949869007,0.08247859866171736,0.08445742087001988,0.07160606051110413,0.07386581445938366,0.07241049269631117,0.07286402785010068,0.08968373775803856,0.0907281558861085,0.07874629090567567,0.073258102450428,0.07693800816383223,0.0862490735300071,0.08866026825825632,0.05715590923921689,0.08874559315287024,0.0869238035072128,0.06592639190147227,0.08784506868781705,0.07916609479094239,0.07156204450864004,0.07618959136247941,0.07211009553392447,0.07624631853940583,0.07501725124450606,0.06202048925460049,0.07342630154770038,0.08182966376258827,0.0721027173081456,0.058978186646058375,0.07579411202050443,0.08686481305337003,0.07715921265359671,0.07573288862929686,0.0932685119192287,0.07449700639235857,0.08455389014038943,0.0890477134607714,0.08083412331967296,0.07044036808189649,0.08239186248458727,0.0868449138717644,0.07327238241018494,0.08965292185588947,0.07874235573509555,0.08157034099389533,0.08025632844385404,0.06443465651883198,0.0793091567993918,0.09278639811370304,0.07765688941107102,0.08562980905357595,0.06642024560186172,0.08654478108283513,0.08792183277041044,0.07548322206264776,0.08888790541696553,0.07817826376291506,0.0743898810157779,0.08498986103565698,0.07047889168843233,0.0580057998230222,0.07964375889953382,0.07797371562998698,0.07853614607216522,0.08428178213641502,0.07609452705387243,0.07356781227481549,0.07400582075650974,0.07634166249714609,0.07944281244127005,0.07373576365768911,0.06619913077960612,0.06405871132703331,0.08967160294929538,0.07090326789951128,0.09496935189733201,0.0690283297723433,0.058397052264761315,0.08230790188154619,0.07979820470755339,0.07640216468916125,0.07795826190768607,0.07998039087846773,0.08230012438367314,0.08494443198756765,0.06537359147211792,0.07160757872493956,0.06956500669056445,0.07438645095379,0.07690110132491894,0.07914551589788447,0.08028475170454509,0.06680490543537235,0.07129234937333995,0.08402672508122186,0.07905939589746949,0.07781272803144806,0.0771612465741988,0.07898354620859563,0.08013781964601294,0.07626094608669981,0.07937507566642871,0.07040854689839744,0.08167206611180121,0.07931547579257534,0.0657465529264776,0.06331714698830462,0.06338030770484801,0.08487933709891865,0.08091352714354762,0.06585922444049017,0.077271649211232,0.07904060841540603,0.0794583102104032,0.08368126788031874,0.07439268420841705,0.06596182910480075,0.07487336972372814,0.07452063500409288,0.08337898593340909,0.07000303440316741,0.08743836324071533,0.07738532800979984,0.07292994981956433,0.07806012267221243,0.07987105606745978,0.07931729370494991,0.059617216253007047,0.07200992449189197,0.08795595518022548,0.0666798225800484,0.08085807166898967,0.07173797486073749,0.08122152674499325,0.06372618279345851,0.07742583638609221,0.08820546747381693,0.07791506807951304,0.08505060096928907,0.07921560401141207,0.08012116715872597,0.08857578421484971,0.07540402866806818,0.08435738020117908,0.0739401336138188,0.06640922015012757,0.07054201901737367,0.0721451165469281,0.07789136058347762,0.07004533554204148,0.0800500665862948,0.0569468714096778,0.07489828379318744,0.06329463426176457,0.07535252303899306,0.07229573356173544,0.07329000309856798,0.058664550264082016,0.07348756372360979,0.06954503044225734,0.06852430871648976,0.08612424153554399,0.0672118753616093,0.07228348842648494,0.07838553208539048,0.0690717628585879,0.08672552236545283,0.07887804787179672,0.07788673901039145,0.07151376106687013,0.06765363674759076,0.09013836027041786,0.07536798193886385,0.06874248583392369,0.07558295552564434,0.06672556104711351,0.08141303218387297,0.0741327698160815,0.06821430003416715,0.08303193437533374,0.07772656122897673,0.08223803667067098,0.0727288653968651,0.0785059628845293,0.0719263613316555,0.09086697998173021,0.0775717162787593,0.08370034706513595,0.06009671533452896,0.0726727039005906,0.07152234170933165,0.07258024421934742,0.08147478295480896,0.07610036510709026,0.07759127035349014,0.07924257495348302,0.05855980788058113,0.07235360739397556,0.05881075500913313,0.07976709986809918,0.07350452161890655,0.08763370775949642,0.08429280264063636,0.07777358183587509,0.07143352977173642,0.07128246584627108,0.06488151796640984,0.0820500305963261,0.07675883698585148,0.08482715772538757,0.07501019023486802,0.06692411591921117,0.07750677739569946,0.07585816663266913,0.06577522521617732,0.0719023952402314,0.07440906912582004,0.08779621598493134,0.06950402073037246,0.07771912529980438,0.0635533407781431,0.07056259519055733,0.08043841101283211,0.06497505049564517,0.0839128217040635,0.07741356338287303,0.07203331697316886,0.055686916583036754,0.07325257644953999,0.08127434223269955,0.05946826764181619,0.0696301202211321,0.07218434448142436,0.0907899355856879,0.07692165479791266,0.06505542414101023,0.06929265479267274,0.07930915926805084,0.06454604947479722,0.08765944121872775,0.08606151388837349,0.08838944661416417,0.08318323516036942,0.07572008260583363,0.08794565430691598,0.07026915395683186,0.08942521993243571,0.0724148595155823,0.07108441739785334,0.07202156761772915,0.055565329309563,0.08031608359626677,0.08351502449946645,0.0711386578522518,0.07893218791653092,0.06942957970850197,0.07199065556615583,0.07506199173239354,0.0595745827915877,0.0690000160187061,0.08012781579887157,0.0799583268271318,0.07910060793030566,0.06516599483180904,0.08037597447383554,0.07318383229950098,0.06774753187506612,0.07213974414911298,0.07350394042967004,0.06516210490539914,0.08190925459581058,0.07114926282163361,0.08297026817681402,0.07005367822592451,0.0891210247250857,0.07498531357421509,0.06628133028740207,0.0800619856212153,0.0782135436227532,0.08893846799519683,0.07982182086258935,0.07833413767561367,0.07268472072103696,0.07465795816660001,0.07059895345394271,0.06280573113633028,0.0800037567709315,0.07975187000822706,0.08408679342252892,0.07613204368868787,0.08551975415208848,0.07692937338127008,0.0873386964426966,0.06808039673827085,0.07739355938796945,0.05429094171340737,0.07048169409111657,0.07134765014064766,0.08096403562250877,0.08964583692078426,0.07903773395500921,0.08002324296422325,0.07474099675789489,0.08551719302147792,0.0629509217112332,0.07934789596130926,0.08948666892394586,0.07443672490740191,0.058828748277225056,0.08389170781119798,0.08093236354077302,0.06958186197685969,0.08554715270356594,0.07519454405723003,0.07446809635976415,0.08103858891793278,0.08077968536222588,0.06671258752284927,0.08447098154368879,0.07525012086852131,0.07849246687788275,0.08442015936618608,0.06986077294443777,0.07491965465016409,0.08212910720933225,0.0786237287498515,0.0755810225929388,0.07248089215080204,0.07213608914021323,0.07800012732589369,0.0699010233322983,0.0696438250262598,0.07585013173756537,0.08023318119777381,0.08151620247513044,0.06431768999113782,0.0710025432646672,0.06390320831317095,0.0731378472131293,0.08146733230098249,0.05615770460963092,0.08203820817069118,0.08318141082124535,0.07321932117823662,0.08102756249314033,0.08009565387045858,0.07887344669111379,0.07934433950297282,0.06477196417083864,0.07555404720618056,0.07812006396624156,0.09094607443467041,0.07360620659689096,0.06367632283290789,0.08478223701821894,0.07309036027780491,0.0892571619977691,0.07629824474538388,0.06504808701737719,0.06943729893925124,0.0700774455422567,0.08382590446475716,0.07038045004378543,0.07246096311516337,0.08062487759041573,0.08960987654669278,0.07528876959951053,0.08128714392189063,0.08681908264400878,0.06396627799933358,0.08124536077051836,0.08617570306047898,0.06521125726338317,0.0833083100150349,0.07571455048722045,0.07523775452176207,0.06545560001647716,0.07366268470888127,0.05498389149348706,0.07045097883168125,0.06839525069232821,0.07575269140242746,0.08975549276446051,0.08045181076165632,0.08013411244970267,0.08560718619655029,0.08287073475971746,0.06557981187155906,0.06927845289531888,0.0794216054713765,0.08359342723812119,0.0933527892016409,0.07223323627241891,0.0899680925239119,0.0913917958970468,0.08962649754339455,0.062226299111205946,0.07403697808542455,0.08307006209561675,0.07473975381954369,0.07872845233630815,0.07868966867085067,0.059027906311300525,0.08420916680405756,0.07239118498506647,0.06356597206375998,0.07821923874491951,0.07630035215893201,0.07607259220067411,0.07894960453408897,0.07013950727945933,0.0628914105262991,0.06417420997819376,0.08483778366874384,0.09042564139595355,0.07924742721084324,0.0868484794747719,0.08321808144208176,0.08470380336237986,0.07883247624670187,0.08678076925199828,0.08113510865393989,0.08625827147384067,0.07989239087666825,0.09010427445289847,0.07168660274256679,0.08074659788920793,0.07322385759253,0.07497460230734998,0.08067638718447182,0.07382971077573328,0.07264208503981251,0.082128521594131,0.07523292422552887,0.07272282539299593,0.08847357543997315,0.06952230634335374,0.07003235301569835,0.08046604139775781,0.07383836563662605,0.08091598944099995,0.08009749339812719,0.06696387855956476,0.06577511305340135,0.0780423989034717,0.08106883199467867,0.08871728308343249,0.08091134673405788,0.07476641541386073,0.06248311914518181,0.06790793081521851,0.08456085236670172,0.07477909530963142,0.05466402116986757,0.07481805350383328,0.08936986713354189,0.08611660180842276,0.07214191881866278,0.08072263245356809,0.05744600553587276,0.06959232124010484,0.07598365648486438,0.07797480921743011,0.09107205996464583,0.08832091149190668,0.07413069476842934,0.07034035913629193,0.07797035994136878,0.06691167624187994,0.07357335345438684,0.06332202184430363,0.0705449363890053,0.07603576319052283,0.06938478817478615,0.06931191253021407,0.08882600070070162,0.07123658039547337,0.06911528307122833,0.0866173063826952,0.07730362734169527,0.08162706833302377,0.06226777368113164,0.08277545900395222,0.06296741588650046,0.0674298485842888,0.08114610603376043,0.07605912537759166,0.08792021140527283,0.08220092392450826,0.07680693770617311,0.07980462214343714,0.07444735272371615,0.07492249790792199,0.07291601549198587,0.08197374123198993,0.08203471822812801,0.07811366370503374,0.05885436447910061,0.08747617067431737,0.06885834080254041,0.07560113550604877,0.0782651077345907,0.07321377208665195,0.08777732283077724,0.055558920708221674,0.06011744176877615,0.08221072124114989,0.0785083277503664,0.06967290510107271,0.07046113847582226,0.07314595505831098,0.08486058050071853,0.07347878211603678,0.06634006930543199,0.07468084523861844,0.07787123896055037,0.08220972504661614,0.0806382447896383,0.07908849891790962,0.0759489582735231,0.06935905832751091,0.07379538187627853,0.06968617447017311,0.09083050393718148,0.07000060231129943,0.07664600494598985,0.08665117098230292,0.08485032157271652,0.09282592877843114,0.07568969575827919,0.08067943978808334,0.07415096390425813,0.047463471851631314,0.07530105436807193,0.09298698230762062,0.08303333331153678,0.080924731555335,0.07295796124102957,0.08085074419607163,0.08636566621743841,0.08679040720307157,0.08695105518465868,0.05692287952046682,0.07052914448175449,0.08465310358308203,0.08635895445527783,0.07485604152911232,0.06504242025053766,0.08901142789972455,0.07180044559203405,0.06300421172956946,0.06699464116774029,0.08758983645167082,0.07140976020061773,0.08160912231667575,0.05825960576410137,0.07547489364514878,0.07306023476761891,0.06730500373887427,0.0767294701007031,0.075459583522972,0.07492430957664079,0.07672959796359013,0.08162695134936382,0.05362758465064901,0.07711333213237738,0.08921361020458721,0.08093007620752236,0.08492334995585518,0.0839524513962418,0.0796974058707644,0.08003224953556243,0.07173789758090657,0.0623185154224708,0.08458165017297017,0.07904408269453088,0.08651734902977462,0.0880026608464822,0.0878112435617043,0.0833920638330186,0.07268126873850436,0.07855245682305331,0.0774225551917683,0.08930807469931692,0.07843329810789293,0.0736184345297543,0.0611518444171998,0.07002294510156026,0.07260254748913678,0.06602270059022221,0.08844832387416006,0.08060833375265161,0.07374659326965197,0.06826733468139526,0.08070131394561816,0.0754029814594975,0.0514686799829342,0.07438414899512401,0.06506077560332396,0.07559257278306222,0.07774966021825527,0.060642016728195296,0.0719158386723408,0.07517714906884088,0.08389251963381053,0.08485251583222675,0.0758466256522942,0.07285037004083667,0.07740725635004311,0.07067739518518357,0.07427011879178566,0.07170991653501857,0.0887169928983209,0.08186110234766739,0.0779314712166434,0.08362985006099435,0.07658672127529541,0.0764902155224098,0.08107445995820524,0.08525235671550052,0.0827856975960502,0.08709353952410764,0.06584933619484162,0.08390550692173757,0.08145163238238076,0.07606016961357936,0.08714132363124495,0.07933936427929483,0.062457509981733904,0.08179070521684477,0.0761475608032259,0.07660075959696455,0.08852407718972109,0.07013428715824457,0.07841763854333379,0.07570059944737519,0.068019042399181,0.07756581394446573,0.062055899341992564,0.07780041072919791,0.08245703804755095,0.08013339917436357,0.08790031574070947,0.08153092975543602,0.06027882937705876,0.0837437787515586,0.06340365950235005,0.07370088547944409,0.0743749083641837,0.07933638620539982,0.07841041852508385,0.07555838137455834,0.08436768384324456,0.07838787963281738,0.08402353711704033,0.07416052153765565,0.08766102553145622,0.08297613694969097,0.08290846204847183,0.08243621334377135,0.08019507854483385,0.0823996514206678,0.06965683450282752,0.08179037260924985,0.07260126629906635,0.06272169276012432,0.0703794872880348,0.08062917865274928,0.07631774904878659,0.07507319225776732,0.07596673152835218,0.06657980529840951,0.07585300809356846,0.08272253724491399,0.0722636647297266,0.07582861541974076,0.07591787986738036,0.09091232707889449,0.08370897085004113,0.07829936146219771,0.08493622797994235,0.0724646263923871,0.07343963506200121,0.06887735008145845,0.07666361704452232,0.06268517973820761,0.083232633538916,0.08277352411563554,0.08181199570677186,0.08151603355320067,0.07108277590394008,0.07612491732521709,0.0818591349771717,0.09362665048586956,0.06819012241216235,0.08409357002746212,0.08996519771649475,0.0901896240019969,0.07984310273842221,0.07861999946414125,0.06868781760108125,0.0836990922252216,0.06859785039170149,0.08204841841343938,0.0686703790648648,0.08411389399590594,0.08148085755667708,0.08673006808215053,0.08835025152445755,0.09198856278150438,0.06754477319327551,0.06642077714090275,0.08113224387570156,0.06819356649830847,0.08564494191329083,0.0784397058406194,0.0833085991658083,0.08695937439000898,0.0770347877644654,0.0793162547221552,0.07286185897540347,0.06289624802703712,0.07321787618145073,0.07406388272181355,0.08881886384416099,0.08098167800627544,0.08529676719114272,0.07983823292681606,0.07182277951286084,0.07777441089699995,0.07296956357294901,0.09165506035659612,0.0870309453863139,0.07233341409528869,0.0771263737939647,0.07494153167406804,0.07623762117843,0.07395545666979433,0.07551937354293842,0.06169992457250519,0.07539144121079347,0.0704566260245387,0.07050228453826066,0.0569470920912779,0.08382788707282943,0.07389185809560062,0.06897520261817686,0.06689655392147832,0.0889096744046744,0.06940115772364669,0.07830636749926584,0.07307073862808083,0.07014184704575877,0.0698159259731653,0.0841242462424395,0.06317224748494377,0.08823549030092132,0.07726094487362055,0.07769056194936269,0.06657299018986523,0.08720677178680661,0.09076054815937667,0.07515504573659001,0.09009292483001308,0.07863643141011326,0.0841394022019887,0.06294070634083837,0.08463502816262136,0.07757477143561471,0.07926060052177072,0.09033641048928881,0.09231939083640558,0.07931837085499843,0.07627049287026419,0.08236924011990841,0.08077561412312231,0.07867340570067735,0.08889125880411322,0.07344018937843846,0.06539517343712967,0.07513085209738912,0.07963428117723956,0.07082556040003221,0.08413976423882387,0.0878979985343109,0.08614520065827921,0.06668602714853228,0.06872929906022301,0.08714301114729371,0.07379239767180543,0.07796393814932762,0.07535982703534577,0.07356577562945715,0.0722608514033515,0.08550682664999187,0.07439029844013642,0.06174271519736768,0.08670851039324338,0.07821749757756638,0.08194648703605757,0.08991723742964805,0.06903209049920253,0.08422064234314294,0.06415458030025684,0.07046355512989091,0.0784403611394677,0.08918982994586291,0.07127683274940258,0.08832959654125186,0.07866773237566743,0.08183709930086266,0.07868771978768108,0.07754812910362699,0.07276356845069148,0.08854182258496401,0.08084946091774087,0.07888829698720917,0.07872034940367689,0.07390125064490166,0.08311405580467426,0.06791920606349491,0.07590945330546904,0.08004825743791202,0.07051263960340437,0.08961969529864788,0.07452849497929809,0.06375204947192828,0.07364553140490768,0.088523177420449,0.08255876074100169,0.07479859738997155,0.07612210085198573,0.07023288517634442,0.07281319280173881,0.08524135944633654,0.08615710514466449,0.0767725122630534,0.08733464985898765,0.07649194707055308,0.07827538459385361,0.07900058938215637,0.075003235134245,0.0766596694728264,0.06992325015844289,0.07316590018973934,0.07155365424958832,0.06513811855810278,0.07811575959825884,0.07254496307564715,0.06356182470565769,0.08449117210624135,0.08693672778949377,0.06960608431627388,0.07708864523274755,0.06898271917850823,0.08556067638815955,0.07893690899373033,0.08329882080917075,0.07401438008637712,0.08987364922572412,0.08224087964731897,0.08135399731637236,0.07689012802057259,0.08321382034900304,0.07491521216740861,0.08053357701607661,0.08357968405802126,0.08014431126301166,0.0683774378650849,0.08921868049330847,0.0742434522014008,0.08728545739350393,0.09174265231800635,0.07592966267105883,0.07956352899124398,0.08221785734876375,0.06848420898175003,0.08870080889418172,0.06853665786523164,0.06488461074457202,0.08661088335639891,0.0870772981732619,0.08621380444688916,0.07129179714355,0.07777238196520522,0.07512257096556368,0.08829991169882495,0.08913824328998599,0.07932913269457587,0.08128793308545493,0.07196596394060994,0.08869834831840219,0.07122129195444711,0.08379949528908495,0.07928785450760825,0.08962603132548957,0.053105864469317796,0.08900408373314739,0.09354508631186029,0.07113484405730253,0.08749266953570747,0.06391269065173419,0.07618991324616439,0.07110178759764552,0.08525428119024941,0.07831638557915682,0.08354875624086755,0.07757603561298129,0.06668961577153726,0.07884424446678909,0.0810575445803389,0.07372205045539788,0.07147925576418462,0.06588811138693615,0.07261000918354193,0.06564986015664709,0.0765860660499213,0.07277586885464646,0.09034689931961942,0.07230561749095474,0.06807924270379134,0.069790713351182,0.07008791745033022,0.0700820619522677,0.06881914803905967,0.07043239353270295,0.07983609022018723,0.08721861340531294,0.08026542453330945,0.08509267632159456,0.07086439168674519,0.05569724948370543,0.07131239471710117,0.08101423918268273,0.067289920806589,0.07627119905203904,0.08177853367770302,0.083691485139181,0.0895029789487897,0.09571141735380832,0.07129716523738674,0.08639103699840203,0.07043931749199808,0.08567643243358135,0.08490934239298845,0.06914652950456038,0.0729646907131693,0.07519565170876935,0.07675206441175952,0.07651274605841712,0.07958520256677856,0.067743232451839,0.08544911118983199,0.08107757452202644,0.08598403945839228,0.07308571542733386,0.07283698705512204,0.07718736682757392,0.0702785256653479,0.06416020806927777,0.0657864130614423,0.06953865639771033,0.07548977787709388,0.08857794355845983,0.09508917120170386,0.07002036906582716,0.08257499931153697,0.06746263593708117,0.06465770595394757,0.07036311487332175,0.06487798195989675,0.06537233134257664,0.06934385969831823,0.07963701291129703,0.08150853130791938,0.08398976539659857,0.07610591423824356,0.0921522044182643,0.08504399716633781,0.060468437146592585,0.07900290785382363,0.07353591081385218,0.06898402639057852,0.08801597437402298,0.05518859419799446,0.07112971894389464,0.06964596921179583,0.07965441988291685,0.08271164517398472,0.07775275740912095,0.08879018202021888,0.08703181304023846,0.0723828993982896,0.06603277550503192,0.07983134508263155,0.0749516904283,0.07631122231391568,0.08517661941229726,0.08438859130043048,0.06874320671866595,0.07401572373780081,0.07818373249480981,0.07524273238801513,0.06853607288694047,0.0815517906244218,0.061692412668748274,0.08067825862030273,0.07451368598821731,0.05885743550507369,0.0779556889589349,0.0791703648230268,0.06781371448109222,0.06874044614464883,0.08781377139943242,0.08225700155313073,0.0840042896954166,0.07718231025593933,0.0727367894146704,0.08303422748106717,0.0797410349343102,0.07589749713503569,0.07271216077683351,0.07470404023357695,0.07646566885565732,0.08000861874166582,0.0771350936601371,0.07318525392646347,0.08827751441905735,0.07555713587482402,0.0797265756268913,0.06947713113650894,0.06708529603849248,0.0842729199477252,0.07736813433955492,0.062787332174155,0.08865133705721713,0.07877327418728017,0.06816969211535027,0.0813553575513196,0.07136862824909136,0.07254402859445047,0.08186165568293645,0.06965977084308181,0.06626648519607468,0.07056393410432754,0.09211140729426917,0.05821848390456509,0.0913345385415057,0.08052834591486897,0.07535652887367955,0.07954199378073212,0.07626928496994109,0.08279088789776241,0.07898055458101276,0.08192438694136338,0.0940019685788449,0.07322399908457584,0.07843396018168744,0.07512514396411549,0.07370001508963826,0.06653833174804394,0.07438975002267734,0.07016380324344608,0.08054429788014283,0.07299075748413428,0.07821753569986195,0.0870448809373344,0.08489236675838238,0.08713893030549315,0.09069014673323247,0.07375026978752416,0.08287129480386056,0.0752046583941382,0.0906870769961948,0.07967559171878666,0.07025339744309647,0.07975596714757754,0.07992651742546769,0.0881855940155967,0.08146925467769245,0.08977241213288496,0.0803306080626396,0.07673949252252008,0.06317178652395977,0.0738257773642686,0.0820435228801194,0.08386396394358901,0.07806375787194547,0.08289240838433376,0.09059077809530698,0.08154771981674327,0.08148158540674223,0.08537944226740568,0.07882730829375866,0.07166906825509507,0.06620344650763355,0.07447004293713683,0.06831915914740139,0.0754805399256809,0.07774762545167554,0.08369683644171928,0.07416227708854785,0.07266157569803805,0.07532078639695025,0.0840708082630352,0.08903121685185794,0.06178581626273119,0.0682894114123544,0.07376251967268703,0.06875088258269864,0.06635285406280501,0.07192637240739039,0.08168531959389039,0.08753510739332751,0.06673506257989634,0.0857133978387101,0.07674208363607656,0.07571368719419685,0.08271626621524936,0.08901137659915254,0.08234604996034588,0.07493734876387505,0.06319899544805202,0.07222424078819363,0.07843908146106418,0.07435217218756748,0.06209981545687549,0.08357354923126337,0.07550712450936774,0.058539205414954096,0.06992491529535438,0.07459171769279832,0.07599084076239954,0.07907074921011058,0.07917642559066707,0.07932978547110217,0.07181020607368016,0.07171310099289974,0.07107911349729557,0.07689873356123969,0.0740994434862699,0.08660554280039431,0.07078615184846568,0.07392472294377117,0.09082919339535249,0.06734815459631716,0.08745154325137755,0.08012611125521002,0.07727456345825684,0.06929185754630302,0.06855219988524319,0.06955148773756428,0.08920790967196102,0.08235427947209256,0.08321418500958558,0.0825122145719565,0.08618112063016777,0.06563951715926164,0.06925382777574957,0.06531701752177525,0.08525446762736244,0.08929637737524093,0.06891502718965542,0.07758729334341297,0.0830968879163937,0.08109156464387539,0.06557432493744025,0.07372753195015117,0.07980288489714674,0.08849343593991413,0.06964975900912833,0.07681748975739316,0.07871089598074027,0.08722013788507887,0.06872522363988301,0.07745727913188027,0.07773121020650765,0.07756154131536043,0.07973363823391716,0.08651135906414782,0.0672951202803693,0.06639114976931197,0.0781074811813536,0.07292434274139789,0.07354643246108902,0.07437323913513305,0.07233477417425449,0.0719944358834454,0.0657322206467558,0.07671170774106995,0.08181488745230897,0.07926479204514997,0.0754620922300305,0.08838314818446366,0.09109535354914305,0.07575920401793251,0.08823435293884556,0.07850644364988965,0.0728072570954269,0.06461603925768733,0.0745462855100031,0.06569683203994899,0.06653270713718848,0.0588552904510001,0.07874996881873109,0.06662379093468668,0.06134264143667352,0.0691444878721598,0.0798457018565168,0.07597046600474641,0.06543983399311919,0.07178943122104087,0.07155471572603196,0.07225985423078074,0.08860982489876676,0.0750999839963726,0.08906322191070522,0.07711155169013853,0.0731891527681803,0.0770991069055926,0.0858045654265789,0.07508535757169436,0.07835060639407956,0.08545501871645506,0.08671202109646758,0.07083370086712432,0.07879142822430055,0.08558534873717716,0.06902512699696534,0.07159142762826645,0.08010575132515671,0.08046604476140365,0.08285172513978233,0.0807080933077505,0.07657867274818425,0.08466658304162533,0.07078364878275088,0.07531138891634387,0.0796469703563927,0.07634434774275109,0.07072414263659782,0.07411723455680648,0.07551822028482588,0.07940102752827993,0.08040095666842381,0.06559751652469598,0.09336539615288497,0.08034507918705816,0.08078760777216373,0.08804265708659939,0.0681133860793876,0.07168125986733688,0.06547416280125037,0.07931801888932113,0.07342621789356106,0.07647109215494394,0.08650938302296993,0.07058480762744827,0.0792532084523587,0.07388207471897046,0.0828325582145426,0.07027876145511655,0.08418621481779935,0.07286098530769798,0.07850819378836614,0.0674629075560742,0.08311611173421071,0.0789996041148083,0.07506805841428461,0.07495000410858252,0.07633918498820962,0.053882166909855166,0.07165669268300381,0.08550404565367,0.08596653686543963,0.08988077091949266,0.07152816938518297,0.08252864001833697,0.08013213297185372,0.08597588003426879,0.07604004287515413,0.0753877961355986,0.06810489770020588,0.0830606232656452,0.06685080198267933,0.0757526641499719,0.08739563305319335,0.06632008062572946,0.07770187411424315,0.0891020729961423,0.062404007898195626,0.07285804743504937,0.07078240696554247,0.06012353305990216,0.060127048018864954,0.07927784595034659,0.058585058069037216,0.0925192354170271,0.060724136949801166,0.07476254173136522,0.07244752438801663,0.0807422791592553,0.0895718113822668,0.07818765687106005,0.07217157586762005,0.07198848814346426,0.08812216672065094,0.08907793067943146,0.06389461546506989,0.0746796702490329,0.07421387851953004,0.07515279623142276,0.07963085152408952,0.07332604474540268,0.08863020048770365,0.05971168747561331,0.08099262870226259,0.08061527110074995,0.0710844986029823,0.07661163094595944,0.08777686854965548,0.08388774934686039,0.07334244329085512,0.07557557110274557,0.06505762560605591,0.08063705138711696,0.06194699242954421,0.0816131826032235,0.07243068972979669,0.07745536420807567,0.06379663862532509,0.07828808247502117,0.08751567329093399,0.07796664005264486,0.09155667869978475,0.0708434209319609,0.07131021572304899,0.0851970392297918,0.08395985047011975,0.07130184044841155,0.07677414735226695,0.08334220618219859,0.05842471270863498,0.09148330137960195,0.07807194478993607,0.07596601034483635,0.08824112885915149,0.08176789619567452,0.06169282534063823,0.06362482725993157,0.07597648026975193,0.07962678290878952,0.08590484988071145,0.08483858723467536,0.08451685582809618,0.08073983610428256,0.08286031822129944,0.06601770363404089,0.08882779096877808,0.07995663557038223,0.09132701939648138,0.08677743026994739,0.08485763337284555,0.08358515465559321,0.08334655767880039,0.09039181294605259,0.081795037236371,0.06235618344187292,0.08548846625711629,0.08843042742557833,0.08796939623276168,0.05387094270340094,0.0867216794780139,0.08277419988421596,0.06443483359193405,0.06748978252554542,0.06723041153217867,0.07842038013714614,0.09001464474143567,0.07966030017167913,0.08503035182109975,0.06477825216204999,0.07354641946382706,0.08230805935252788,0.07384297779009856,0.09246813610520271,0.09179998826512706,0.07420985800947272,0.04915462618578253,0.07863282700523519,0.08073718101739961,0.08075708971434448,0.08450161050932507,0.08642206271018213,0.05940574847497279,0.07190029100485051,0.06636971237913403,0.07991469605344763,0.08176888748795749,0.07325565213484181,0.08055687293844992,0.06928397761783135,0.0642853353511289,0.08178845881035621,0.07148128164571613,0.08406320157665678,0.08296491298262125,0.07595024617096158,0.08476158119931858,0.08603584164564759,0.07222438034237069,0.06499930802452022,0.06752632167416153,0.0848024258025574,0.0664372185631173,0.07936285928684098,0.09406791138840835,0.08831497298166543,0.07256451528055295,0.06992075213099236,0.0717522172748522,0.06938141050347531,0.09419708657319427,0.07019186034716246,0.08283668242849512,0.07584911980066866,0.08837642070545942,0.08160202656266274,0.07164593038640449,0.08608504052667716,0.07044509611854818,0.08110216147947874,0.08132100651147263,0.0700674967191328,0.07850861298751377,0.08015024770460971,0.06590843228924792,0.07102349558566709,0.07841842331658355,0.06487486306922857,0.08728713065773294,0.07339217985907472,0.07515091703252164,0.0758172551505093,0.07043965447278919,0.07216474734869971,0.0885587962763616,0.06911880024464687,0.07893622548583429,0.08993248126853753,0.07151813785352099,0.07409903353899851,0.06454113187379293,0.0820561883024484,0.06695014784392467,0.08106543070934397,0.08173508888513181,0.07365066134184496,0.06608869408258977,0.07524635077551856,0.08074944313333268,0.05742424120854757,0.07874480302948575,0.05736385796302758,0.06973716842924028,0.08952878385901863,0.07037720962859308,0.08190173983002239,0.0875908936992893,0.07590368086661656,0.08363147090477768,0.07638461910136268,0.08038024311876288,0.08165316328312294,0.078524883691961,0.07824302117925121,0.07282110803021266,0.08905314088492906,0.08835389852877065,0.07412407344068442,0.0805666903603659,0.07542220085765027,0.08023255980532015,0.08775008004522891,0.07131183749926245,0.07236835731107429,0.07189648987153581,0.08069847280209627,0.07891783245608579,0.0867850590405144,0.07497978180019392,0.05525025931335342,0.06934254949119473,0.07432793185624396,0.08930543515787157,0.07598679672678206,0.08044366685692428,0.0837079820749444,0.07127739402242453,0.08310173405179937,0.08241851607483246,0.05940001516873937,0.06827655461902934,0.07589875208862334,0.07029596489754428,0.07045370687088073,0.06701364014717806,0.06338501935082923,0.07336021485253838,0.07860092275564952,0.09234889523556979,0.06974043998085974,0.07934263739697474,0.05400162851170122,0.079811350796717,0.08703720610472243,0.07519532639683996,0.0703765199382734,0.07214353745559605,0.06940375100330892,0.06320983177856655,0.08699626757259016,0.0829530184605593,0.07985013153071902,0.07239180578330348,0.08308404345957489,0.07584563614817924,0.0847480611460964,0.06899072900124516,0.06936285283044252,0.07468169729015635,0.07232043848685876,0.07982269662991695,0.06486010185630411,0.07127337812188417,0.07800892300886647,0.08175661419524373,0.07656778878108246,0.06624071617339242,0.081393855708292,0.07608310394466339,0.06906556932985729,0.07608238148688506,0.08055022708731423,0.08298491381809424,0.08196433934053896,0.0762129567391853,0.06859266584411655,0.0763781260960471,0.0871798506165393,0.06936879023169595,0.07375306088759809,0.0662686332141365,0.07310672439227339,0.06559035016859546,0.07352608103869904,0.06703209387972733,0.09642711155473585,0.07262221241391675,0.062220351307629845,0.0863516337346999,0.08960346912005795,0.08652067745541506,0.06456693875799711,0.05345176471048502,0.06622523005129471,0.09190293225771187,0.08214465783564344,0.08004948270284856,0.07000833974146561,0.08660205098747353,0.09142622531039896,0.09178525755974937,0.07540861290035444,0.08351727746292195,0.06223903676015995,0.0549417798345987,0.08194225574473932,0.060323796968519874,0.07536436966273344,0.08703394143542724,0.07216499235818068,0.06306228764971433,0.07783805524436767,0.07315934682313999,0.08100032270098179,0.06785335241874924,0.08182819778851712,0.0898020097850532,0.06032353053197671,0.07724474087287482,0.07775974902649856,0.07815395286470263,0.07778638545393665,0.08936931632919577,0.08206658903042156,0.08206071982502203,0.07088282243434482,0.07280257461428828,0.09394691168811851,0.07505780323629652,0.0709608491288366,0.06606045139443803,0.06898236562501797,0.06537923957747657,0.0812172365264614,0.08527807707406047,0.07630271622695328,0.07313510841341606,0.07512956958545211,0.08076869214459753,0.07694525440316287,0.08494428118965708,0.07335347585775981,0.08501420636872487,0.08135885248014132,0.07180730157396484,0.08414534369286583,0.0787582409632275,0.08641506349555983,0.0833150452364106,0.08343959989924743,0.07621831709198959,0.0823384868500403,0.08366970975439092,0.07950301687648587,0.08994777416015509,0.08425146555565058,0.06582509950291754,0.08357923870061819,0.07095655439060197,0.07648504906427647,0.05807711488391143,0.07520110858641564,0.07796529917601756,0.0769607368603244,0.06674171462713394,0.07642953008679258,0.08609668039850094,0.06397232130210057,0.08800821204834489,0.07703180396933584,0.06967712442354204,0.06672893708526761,0.07984455437403654,0.07911642694562748,0.08641807219246642,0.07564720265008293,0.07563937694213675,0.07600167726780749,0.09430484001475191,0.09145218127423241,0.07563619124137816,0.08215669871446293,0.06360107031913459,0.07752088186092107,0.08420915431102471,0.05740921797218576,0.0853254025533444,0.06887505659864374,0.08255511574919706,0.08602821999945917,0.07944083554374687,0.08246105184346393,0.07549697140737194,0.08101751167911223,0.08175659174192217,0.06353133546867687,0.07260046985064443,0.09190335895113058,0.08293793882964065,0.07364567513907919,0.07929636949328836,0.07145594030162891,0.07892776256843781,0.07421980635278086,0.08270001875353701,0.07828348382358369,0.07217912002971974,0.07785540522715739,0.08661798761135701,0.0766641324069472,0.06883810026021553,0.06338830395176479,0.0843857188708883,0.08592994636610463,0.07173911182161147,0.08803807405594519,0.07442056597077681,0.06871202284384142,0.07782567955031319,0.07777555925188852,0.07236078394720705,0.07314017261056883,0.0675078123384846,0.07759904806414643,0.07469408013038625,0.08352962545289364,0.07663898706723246,0.07484035925561475,0.08342573524613475,0.07866868302450934,0.06657642698322527,0.06551914263597569,0.07596964623888591,0.07845266295832305,0.07351136396039912,0.07578668467258456,0.07863632126808345,0.07295279333403565,0.08631118382937983,0.09234736654441643,0.06650358794844657,0.07503537342533607,0.06937918184495331,0.07604426370802218,0.07919267818901793,0.08443957113484679,0.06705967587405144,0.07919720590307178,0.057459462392126584,0.07385468675020843,0.07394671985224954,0.08055075062608702,0.07239666198771252,0.08675348900821594,0.07974872860160903,0.08157953093338069,0.08153760411171099,0.07426871725948633,0.06610730402434017,0.0729073319443137,0.0801548187768776,0.07310253484084411,0.06648531990855457,0.08803482317629041,0.07650619242265154,0.06794551842312438,0.07078582196268399,0.07965443431877346,0.08916691287165661,0.07225912009413418,0.07078656026395636,0.07436582634754267,0.06980362910601268,0.08399606148943785,0.08806282400308243,0.06187380393404846,0.0649421176019217,0.06357400679804392,0.08902611754716608,0.08503330978018889,0.08107626120573824,0.07731436630905697,0.07487331667202934,0.0749654080968716,0.06013803114491348,0.07895498809672272,0.08703705196481651,0.05778894763350163,0.07441391549411837,0.08132492677638134,0.061970011109916796,0.07622008430077912,0.07750376042482875,0.08597308230995324,0.07960279848266363,0.07291197755127388,0.07976416137573988,0.07366837491469752,0.08648865711940866,0.07251118745288013,0.08431043680019353,0.07972357668661986,0.08118198196394404,0.07251777031160064,0.07711130091674281,0.06970074848653533,0.08125649057248055,0.06816170080036099,0.07748903237987025,0.0669856896483389,0.08132258651221211,0.06876218254471457,0.08145515994731484,0.07244194697292512,0.07435098262767512,0.08535936490289331,0.07450929833736791,0.06209378880613314,0.08036760816420893,0.0661969148374411,0.08221568778628524,0.07757232490894712,0.08160145581089517,0.08190995738450728,0.07489311474240001,0.0811073151238533,0.06787244573930533,0.07742887020534805,0.07237092057203678,0.08066994695564308,0.07130885144705276,0.08666633123836937,0.07526872352461217,0.07234017277646704,0.08143986539633213,0.07814950921100938,0.08209969614193639,0.0853964834518142,0.08628174761100017,0.08778398910396983,0.06037538290628527,0.07730124325100826,0.09216939727835788,0.07133089788317304,0.08124061929510887,0.0758178235055123,0.07322144357744462,0.05889346238731957,0.07742194263266641,0.09011809454900024,0.07068706798214475,0.07041407763505644,0.0660816085292692,0.07727328442654785,0.07958209188059505,0.08730996098918925,0.07742248644745225,0.09331380109817614,0.07557940776443217,0.079420582816943,0.07182739036984331,0.06963279228650009,0.07700906184118889,0.0783021548550476,0.08896650458478259,0.0741945588508487,0.0742244045391757,0.08987299081070918,0.0712067971555714,0.06446340825546931,0.08559899922039182,0.06385388654090213,0.07458274321622561,0.08290121662655547,0.0766085111362466,0.08358594240062231,0.09240311296607873,0.09413213845763056,0.08635165601555905,0.08118911694681365,0.08500470416218457,0.06065645846200826,0.08488726861033527,0.0800009921043699,0.08600392522907052,0.08674726028211933,0.08257718724047454,0.06279186479285358,0.08899961727285173,0.06326121934205925,0.07615338694102083,0.08456450529580833,0.0668507908451843,0.07030743299816708,0.05795041388108081,0.07437011129204738,0.06345747769538485,0.07116692080771424,0.07695705320998111,0.0811211535997827,0.06719353436779874,0.08082245801369348,0.07032086023761704,0.06949466385148577,0.08362463451020138,0.06343001551083999,0.07144574367115045,0.08811842446766116,0.07226791857356694,0.08598243031922834,0.09096996604344562,0.07493959466022296,0.07778238208086498,0.08239487830828304,0.06730020139258451,0.06378794580967184,0.07380925873642352,0.07496294489029237,0.08533006044740166,0.0795951768135452,0.07791798722880694,0.0717959588191187,0.08344373440477709,0.059690514263502026,0.07768038534680566,0.07325720700135217,0.08400364578859199,0.07442988114078021,0.08696847216670442,0.07724978478835322,0.0646710876988511,0.08108944729468807,0.04739004294943018,0.08496539905881653,0.06575715936072493,0.07227301221170565,0.05531408856494415,0.07393294236388058,0.060411952960880526,0.06592320627592478,0.08009035125824718,0.05967994792192703,0.07293947444545473,0.08637499282438738,0.07760122127813296,0.07761106547101763,0.08142165722360495,0.07998778833994655,0.08464054544964988,0.06351916056389001,0.07716465733859826,0.07824681201711525,0.07732213181720923,0.0843614915796923,0.05044095552556545,0.08534248346691917,0.06151496790239658,0.06178178334965737,0.07221930763903299,0.08553347831384353,0.0753874957199989,0.07728580070120297,0.07720603734639503,0.07194805701094016,0.07503620360639598,0.06150415525922474,0.07009701981475785,0.08784642853188289,0.06664165353514295,0.0711741424830098,0.07361683101440941,0.07468991347636364,0.07227293430652136,0.08171853623782196,0.07030341865212371,0.07998981820848108,0.076672988598876,0.07333580116605565,0.06728832756969719,0.07338256765597054,0.07979343394648813,0.08395805097868014,0.09234391736496553,0.0788663708692526,0.07361065762929961,0.08781445844201854,0.07787228714800097,0.07617776333327085,0.07648386114825145,0.07811187371222389,0.0907625519876617,0.07560551406276234,0.07913437159918953,0.06786565806528279,0.08224804861714155,0.07543366381666741,0.08037992030033295,0.07872195891011774,0.08099789780126371,0.08622148678564781,0.07666773822420858,0.06897804022541401,0.07689390625010378,0.0791458616625122,0.08374010817415345,0.085879564411181,0.0859853283012324,0.08926554576993435,0.07849235035085766,0.08441732095293274,0.0743142037010055,0.07384607448761449,0.07281312974892033,0.08424457241455954,0.06499309279171878,0.08326892535830341,0.074307226819149,0.06391296390907507,0.07855254266436243,0.07524129357216322,0.07571115327251146,0.08888862475553377,0.07264057398559437,0.0777512555509784,0.07112217564357837,0.07806805378695089,0.08455823491765946,0.07402978085780723,0.0798813431602615,0.08549584916797148,0.07023273208461467,0.07978742504890873,0.06725493243852479,0.08765927226680696,0.07623329680504323,0.07409704883206347,0.0689449043928429,0.09406594311018646,0.08230720631677861,0.08075460659926366,0.055033857076680875,0.06339742767682226,0.06707259784201544,0.07735055107211172,0.08376512161642137,0.08654876519217303,0.07557501375627537,0.07594783507143016,0.06966039342537719,0.07346087240577377,0.07595108283298597,0.0858710624253753,0.07376312767970686,0.08183540325253746,0.07544831061075537,0.08091747011849972,0.08615211473871018,0.07762199141668084,0.08205245409944362,0.06946883837427856,0.07107432122316754,0.06349661183549593,0.08315978171582378,0.06739023125630811,0.07880231288641668,0.0881597228964496,0.07562406686608239,0.07646616915136129,0.09055959347145698,0.06862799527498419,0.08574957107165743,0.08091825936273614,0.06574919062585113,0.08005302650882445,0.07684683427829171,0.07584607629045576,0.07424819273293405,0.06473852165390082,0.08374329275766927,0.06805729544219682,0.09007816418692274,0.07235873214387716,0.0736064715546155,0.0757187273291943,0.08817683331580473,0.07119806914529431,0.09261069923900445,0.0687438339204603,0.07190182058581937,0.0839444979813069,0.06871549786658881,0.060021578637585615,0.08160270684536809,0.08856251116089255,0.09268528649408536,0.07733777401331002,0.06989961000491762,0.07655575716283772,0.0823170424579463,0.07288335091635062,0.06370153011962548,0.08522856866555253,0.07590952619896879,0.0815611762557999,0.07331133469742028,0.0741994803112058,0.0761133897689866,0.08019838806356379,0.078861299950034,0.08212624056500414,0.06832125656108681,0.0761413861573651,0.06725754584954763,0.07696737071185933,0.07395969473986508,0.07814522538970967,0.07429359527852442,0.07533971053237562,0.08153693356678687,0.06906842030991002,0.07699036457181296,0.06955936727588008,0.08738158712919786,0.07451571708733286,0.05953606043511345,0.08016249488923355,0.09228254462027831,0.07266500851436347,0.07958697736930292,0.08087243072617287,0.0697882953955693,0.07437975453246307,0.07389266871477479,0.07177994168533838,0.08485860136718625,0.09109446937520957,0.06462442818344588,0.07679116861798277,0.08011569996531576,0.0833659939840182,0.06812825555411836,0.08142572485536004,0.0803875054341355,0.06809262194633181,0.06781600069217295,0.08942391587504173,0.07065371457824232,0.07471829517729431,0.07674097770715532,0.0744908128615864,0.07138052153494451,0.07485433947180975,0.07705866442704627,0.08519077783695955,0.08004675121121768,0.09044379033618691,0.09024254310041496,0.06892168872604373,0.0687321097525892,0.07036591222644224,0.08285583291339219,0.07255308958630141,0.07945936617053567,0.0918562671687633,0.07937197524739631,0.08129895352321509,0.07814827011742576,0.08106721431340883,0.07167833055338765,0.07783588539439754,0.0721571143679147,0.075493257015524,0.06267537105666698,0.06990466844561438,0.08710179240937593,0.08509518332081148,0.07895270302256527,0.0805858580929246,0.0677771804978903,0.06166884291270264,0.08016306999475148,0.06519116718308335,0.08991577961333302,0.07903337674806571,0.07316088990547283,0.07730725020248838,0.06324681161885944,0.0733620612567496,0.08512513789164913,0.08753619402854873,0.06754249048719085,0.07278397895957499,0.07109052568001045,0.05843192126276693,0.07327316707831047,0.09231031643374957,0.08647056498135315,0.06353058388422805,0.077623364420485,0.07370339171027201,0.07961491158780729,0.0817323417427484,0.0663716944263866,0.07526630878948573,0.08167398861545525,0.08233394020769644,0.08276022044930606,0.07821602149649373,0.08817427364446581,0.07510796342222294,0.07453056312772777,0.08141590613948346,0.07747897349540059,0.0835846765563752,0.06845008599356826,0.0912115724594184,0.07582794146003677,0.07516198621881987,0.08750925393070454,0.09305739373928525,0.06439857913956543,0.07308120962734921,0.07590649196902063,0.0751736058732135,0.0895546774543376,0.08465761305915093,0.06591438198802807,0.07205197747801668,0.07879774842177582,0.06859748015396092,0.08389732996689306,0.0694378698350789,0.07508803839475081,0.07345509805951037,0.07291787345723139,0.0832334136731468,0.07293662835740017,0.07726547640859299,0.08483782763300912,0.08207409721257083,0.08801171513199879,0.07793394374188523,0.07303589392247875,0.07062912952344533,0.08336867030533995,0.09031088833987277,0.07596920085344964,0.07037839463972696,0.0742614958336589,0.08076154748245563,0.09622144613581864,0.07457712203721822,0.08459189197414341,0.059506155473709355,0.06812249907771657,0.0724533860447468,0.062402877716892846,0.06062462297577717,0.06942460727325914,0.0797534151349427,0.08752159355485309,0.08239252931945601,0.05936809934069945,0.07244153677597434,0.07841375095353667,0.07963972315307594,0.07555322778244963,0.07176206314449492,0.07052603723530218,0.06985819287260067,0.07992981390319265,0.08935958163199478,0.07829603847536662,0.08851771345174736,0.07994106637375784,0.07843069282024143,0.08694636089416521,0.07216041098453937,0.07325019959779609,0.07159522552779833,0.08317021356947067,0.07965577533804744,0.07895457137442403,0.0799631960195182,0.09149751775622653,0.08129447564492667,0.07557301239775846,0.07505598985269812,0.07464344429474506,0.06766159720093692,0.07716706165803042,0.07494879717654954,0.08893003290540083,0.08646475900387074,0.0767935341793316,0.08534765077328448,0.07594634761512595,0.06152637277649434,0.06904176330824707,0.08572986984761284,0.0795071965891974,0.08149581480668089,0.07187571893553811,0.07840706302321404,0.08056557310042796,0.06804132100541191,0.08232864427733863,0.0813746953632716,0.07447781399985157,0.07997856501219999,0.08068861837396754,0.08815825951435927,0.08148248323218282,0.07521966826528231,0.08698095960041528,0.07672740919306523,0.09012659895951333,0.08139925506860078,0.08776850777508279,0.08491911191762913,0.08929831536489835,0.05676555451842645,0.07932038518553186,0.07568479261205234,0.08714298591886203,0.08061254880677322,0.08780237560732314,0.06753973266100666,0.0586685499387915,0.07351680324716336,0.07075477665342805,0.08238581087647595,0.07427219687571199,0.0696295135876309,0.07461034614728292,0.07209797889013755,0.0660846079316548,0.07925210584458135,0.08728741265353052,0.07523397556622748,0.075759582561266,0.09320450599463449,0.06943300349241883,0.09136435599055967,0.08502814137122475,0.08479679113469318,0.07505916491964598,0.07304228232797096,0.08058967657170807,0.08066823516314214,0.07326500429188779,0.07789242630567604,0.06870071610497441,0.08157577957786029,0.07881985788034418,0.08056754341982322,0.05507689220379937,0.07808520861631701,0.07913668248881905,0.07852512631024872,0.07606480744543204,0.0822328275130054,0.08636157122577415,0.08054453442913811,0.08462954218342222,0.06613492902629167,0.0817329219957152,0.08685102126528559,0.08041338113938928,0.08959014925288475,0.06663276529156414,0.08535712305412542,0.07493177599867701,0.06239654080286151,0.0655661445181453,0.07172283943666408,0.056456616233533305,0.0844686904874209,0.09055974434116995,0.07395898332547854,0.08282179643526552,0.07680293997464406,0.08462557005561441,0.06789953107120283,0.07079738992976778,0.08376480732467888,0.0678014223953256,0.06587661337512696,0.08802608724348665,0.07017763644426363,0.06712368467088267,0.09020993673697954,0.07624716533316775,0.06724977691098838,0.07814065185848032,0.07552588047938016,0.07730588692618237,0.0791559694605547,0.08396913264323647,0.08354129373472843,0.07857248424539279,0.07932387989221719,0.07985933527487363,0.08755119139986386,0.08824236331196852,0.06312896033067328,0.08508211886425823,0.08444450280464975,0.07345087716186653,0.08018445972841257,0.09080248805017149,0.08964416989833837,0.07869961259853964,0.07616495206759671,0.08702498378732532,0.08217930816814786,0.08192514707919182,0.07983528979185926,0.0813438997415151,0.08592242503676475,0.07684600502674196,0.08319210499370902,0.07624644143994239,0.0674168378163692,0.09274398269793502,0.06918112637092633,0.06830352897477909,0.07543240703567947,0.06625605981594196,0.07390110340739373,0.06595422266449281,0.08119667425079184,0.08839586015464013,0.08268974255003916,0.06987200131441436,0.08083204262660129,0.07095913891301656,0.08057478042793192,0.06986064059630895,0.07769868655171948,0.08443749252929797,0.08045555333344277,0.08204362277676862,0.09520410715312537,0.05992078870195244,0.05797080275863267,0.07691725906726213,0.06368700207443931,0.07922093447070747,0.07737971946547552,0.0759025509198043,0.08115693263858506,0.08095409916556295,0.06093428311477092,0.06335597106386695,0.07854079338277584,0.0811713309405428,0.06956997071903281,0.07054167713194481,0.0729868347677215,0.07827856121946353,0.0865950848795194,0.07647152011722573,0.075963274043948,0.07077908701401356,0.08371162530264792,0.07188096768161574,0.0789031616771737,0.07501768953787163,0.07612148261502336,0.08268963301497692,0.08425057793258997,0.07557454459022167,0.08420555180530695,0.07843493058753367,0.09479960785322918,0.08141518810272097,0.07662873464028352,0.07793535274194052,0.08760697843942553,0.08583260046961869,0.07232355895097248,0.0666783216431082,0.08019136854069397,0.07825285748533164,0.08815024642592935,0.08706049806824318,0.07051952358429447,0.07939660384500433,0.0807995108206862,0.07032341499069868,0.07373882116179084,0.06667590332366499,0.0703474978569966,0.0664140709490991,0.07703152624523332,0.056776980093290685,0.08270011864660648,0.0690619930703168,0.07042445000410641,0.05404351859863287,0.055611651543899174,0.0684230322772854,0.0693937620177852,0.07090680515159777,0.08946094197768367,0.06565267396408093,0.07934430875010161,0.06386669768730285,0.07631263508614639,0.086919013898583,0.06993876831652991,0.09133065330552992,0.07816466290224089,0.06621676619260927,0.07655292824806877,0.08110651220389163,0.07001650867460382,0.08078365846292941,0.0784111513928192,0.06452796929869734,0.07700164187163494,0.08151384883878679,0.08449985917141917,0.08068989083603886,0.07897788586536611,0.08004774227751679,0.08648610435160711,0.07830876111547883,0.07190105822049801,0.08589535420372393,0.08987815210932368,0.06528987420945531,0.08227404483422124,0.06395040514515646,0.07675601419750433,0.08425470176079584,0.08460807977739182,0.08073709143688117,0.06933468760283779,0.07354775588993406,0.07404005819751291,0.07659814241150363,0.08365049828615408,0.0764936309919656,0.09192024025252449,0.08701895316961938,0.06847994704212529,0.07648050651408397,0.07272226409168622,0.07108741524717246,0.06371744060557757,0.08925513325243947,0.08104411863919689,0.06521894061630593,0.07899253631840555,0.07863613290463224,0.0837891175252034,0.07735813570008644,0.08586094170739081,0.08537070240860521,0.07659396060172532,0.06037832911766493,0.07757624605577694,0.08427219019031089,0.08020902272882882,0.07479478989587524,0.07188170826097807,0.06803378686826433,0.06971649504794034,0.0824307259008295,0.07851978813683849,0.08109466181414471,0.07453592182766805,0.08922669735567386,0.061984153406077803,0.08554716510154772,0.07477927867108677,0.05860282867701966,0.08089556883875908,0.07320676082251538,0.07969043574424937,0.0861540042897371,0.05860760906697002,0.06264562632249927,0.08552758576097577,0.08687098451560707,0.08209519278499196,0.08180827372240773,0.054346884114007024,0.08109007128880272,0.08439363239286829,0.08813394900069149,0.07819877734358609,0.07579790401980621,0.07457927200381706,0.09231638165870827,0.07411660380120852,0.08298933091783678,0.07011661306766745,0.08114661621892864,0.08336047804740901,0.06963489770799747,0.08515562763839558,0.07878293876505238,0.07389050810484654,0.08868178711199927,0.06684494556614742,0.06953403631862196,0.06286487270215156,0.07638787043005663,0.08562854084508675,0.0727520934896975,0.07901061316634553,0.0822394589279771,0.06477678776498708,0.0716812614084362,0.08018294646468696,0.06894539417016271,0.06732111119805273,0.04879647806220364,0.08384984123057637,0.07255795667995951,0.07140229204708762,0.056038603214710665,0.07938932188503216,0.07884331242692863,0.07567967569091677,0.07568295474124481,0.08142515642447844,0.07312490937067133,0.05851467524746716,0.07037326950416925,0.0816479374384225,0.08090921918663938,0.07165661019503472,0.0764190869236531,0.07453287004485894,0.06717827478542199,0.08978881866300734,0.07968096728997331,0.08761261834887205,0.08596936917652953,0.07040382559409647,0.061831150044862795,0.08930475997938589,0.058756760015815215,0.07230109925271927,0.0835127426612115,0.07431312076297002,0.07288122498948081,0.0723858008828525,0.07670630858353136,0.07836495723266744,0.07035838339610563,0.07268077132394155,0.08427947981296519,0.07659375529971646,0.07140396257394854,0.08511659245054463,0.07471460565193955,0.06548823557520658,0.07900984672444664,0.08081119809240156,0.08701893466364026,0.0709049443868004,0.0657487053747899,0.07713679162721321,0.09421596849049972,0.08072270143808433,0.08777901866434663,0.05530851524511167,0.072138978307236,0.07580107197900532,0.06365277271541905,0.08170299372148632,0.07474322023256397,0.08462207340534852,0.07489108829137589,0.0601770370794803,0.07286541738950075,0.0811000034022614,0.07259303654360588,0.07633741485282824,0.06285188306204065,0.08230627358640144,0.07096187101616835,0.08159488666847468,0.0764712899148711,0.06561793166880017,0.08160850198184863,0.0669259597027137,0.08257332665701693,0.07978752911984487,0.08355378706693967,0.07227915354505017,0.07195646758145428,0.08728985072062051,0.07925120139322517,0.057290939952252956,0.07229408791372464,0.08690221303977014,0.08013300746277048,0.07798529866928829,0.05942256415871246,0.07225314159228918,0.07640320143911092,0.06715037371142314,0.08700675131432277,0.09253541288848785,0.08316975330809656,0.06432984234576834,0.0884175046889485,0.07224309332155054,0.07005035508609544,0.0696704404671381,0.08345356637332356,0.07092527411630206,0.08200393808206133,0.07464888904658135,0.07091584370568575,0.08058722761319165,0.0730756425881704,0.06528007485587951,0.07726875346064467,0.07054244100834269,0.06714664995693315,0.071562849386584,0.07242378495921614,0.06736411472976313,0.08778399915978377,0.07424350774895372,0.0748749978297256,0.07762569704033906,0.07213587246039734,0.07734104608908354,0.07401319332302198,0.07703741376812127,0.06669765396746129,0.07918658933353273,0.0705540412627375,0.08288794624884945,0.07491870103994255,0.07610543872716805,0.06781351275354214,0.08068496549150517,0.07970636826534513,0.08008722674256163,0.08661329946485681,0.09135864917708189,0.08837789272047686,0.06524630169657675,0.07443711259249294,0.08436912407805613,0.08763828411625356,0.07917432432425967,0.06590603333462589,0.07426459500363682,0.08498415388464445,0.05669053972483978,0.07802552753018255,0.07880152433926599,0.07203399665173088,0.08471983717687966,0.0827845952150374,0.06937476635650355,0.06861486697663123,0.07067423984742252,0.08737155927157075,0.08828172830491746,0.07623730918878305,0.08050348600832184,0.07643103010276925,0.08448557248147515,0.08028396052411232,0.09021687440780118,0.08440678469675242,0.068219897789248,0.08741989961435705,0.08775579496933474,0.0760864832648328,0.08565403163032623,0.06449604961639066,0.07724653010368909,0.08273244126254664,0.087200271230436,0.0744974177131153,0.07570307361141376,0.07362665350363987,0.0848113973804108,0.09295845924106473,0.07523829134112726,0.0771367452740196,0.07248827219839939,0.08047488992364811,0.0763579934004179,0.06759344763962163,0.08824800987053996,0.081037271444495,0.08278556531693146,0.07150160252129563,0.0735667806577788,0.09059800541544198,0.08830724859471181,0.07901746182984778,0.0758040014976895,0.07645581495505197,0.06554626407166017,0.07818248622683616,0.06837457119545182,0.09153134995819412,0.06481905099357284,0.08827640112368541,0.08033134804636893,0.07881052988508541,0.08294035789587736,0.08431772424846458,0.06938350591602022,0.07074599280567749,0.06077281957593754,0.07309840444068354,0.06636055095972665,0.0691726483865536,0.0683452975210246,0.0667559410665982,0.07137358999120003,0.07897860594319009,0.08225200584122977,0.07069390004667071,0.07689275566039402,0.07937067778297723,0.08018877817832554,0.06031853399140565,0.07481330717313678,0.08914931670126186,0.07150419705963869,0.0760319511889774,0.08591946867821859,0.08094601176546116,0.08095594637010911,0.07485256054232686,0.07760075289940335,0.06725913110825515,0.07053206535680505,0.07584839623848671,0.08640619401358761,0.08525727290624172,0.05797295632520276,0.06106798972201131,0.058080682825200895,0.07344074087301086,0.07528502641298415,0.06767449623052833,0.06686359413977024,0.07629249980384945,0.07583432101037246,0.0803556624176302,0.07933426300646022,0.06242770900182463,0.0628683260199084,0.07598860295694937,0.08365433559405375,0.07784335244297691,0.06174520484883018,0.07605091224598834,0.062442838175730854,0.08553943935879973,0.07723710272956102,0.08535900847100078,0.07261303281333181,0.07124427488636459,0.07019606278176335,0.06865422819517379,0.07577726115698552,0.06734271077223011,0.07083983103483024,0.0660047570533388,0.06995949340332758,0.08594532932024217,0.07006069515877145,0.06409357513370592,0.07365180692350094,0.06947115821254174,0.0753521209546803,0.0692561144029309,0.0676141440436719,0.07239100034040885,0.0846168520967139,0.06382281096072016,0.07530397867783058,0.07383700543407438,0.08983388634891365,0.07303078437499308,0.049116374582674024,0.0828143821684407,0.08372791218900469,0.08490690298619213,0.06798520742694447,0.0819876870573306,0.08211878056625471,0.08518434542588169,0.0640022535476268,0.07696706674692297,0.07349527473839203,0.0849646240669053,0.059194952214136703,0.06596017943010651,0.08891182740653966,0.06980939387324214,0.08256125346733267,0.0639871707394722,0.07505902672674947,0.07382332303665921,0.0887075845440819,0.08282601619490254,0.07030247225599867,0.0777255803356563,0.07404968743862592,0.08597763164624911,0.07918310442593075,0.0706836143075123,0.08098576708734193,0.0790567517954773,0.07806169758796916,0.08593598367397448,0.06727273346807291,0.08087000741651873,0.08147281546613658,0.07280920039105879,0.08542731970821246,0.0793872787421111,0.08721659191391473,0.06027892440383851,0.07593350723349804,0.08042407597960496,0.0803285850616483,0.08080038272489734,0.08612419870388748,0.0843307398839572,0.07851836858166433,0.0846799588922798,0.07337147758500862,0.07814266145742581,0.08464203403176554,0.08312971085490864,0.07776597310037758,0.08668363840054297,0.08203144260641002,0.06820870225125344,0.08339939071453462,0.07361925206457207,0.07165127641596586,0.08078723209373033,0.08457307483673064,0.07521838192456218,0.08282205725161249,0.06186986109876227,0.07316694689336165,0.07304985875503547,0.06850656374130537,0.07910570821072657,0.08415336670874524,0.07936888671543707,0.06661530969550089,0.06260241002314824,0.06333589986449598,0.07695082736545467,0.06480927136365466,0.07943157860820024,0.0855211133265758,0.06566844936657251,0.0720707084486944,0.08238951964034058,0.08040973705785126,0.07081257044571723,0.07532551938648824,0.08284301903231509,0.0647209475518971,0.07561588664832468,0.08222616000064586,0.0733230887541017,0.08061362980746858,0.06947249946762289,0.0717088362680944,0.07344756820945059,0.08055064241607601,0.0823862564828444,0.07380272977089997,0.07571197348746142,0.07302473802742178,0.07978898256921294,0.07450557888806612,0.0682942755260679,0.07748072916455492,0.05840306125710331,0.07564076742708009,0.08965603641331985,0.07638860766562489,0.06765208032576465,0.09057762983960373,0.06191698707720196,0.08045610956688588,0.0717727075896118,0.08850835961981893,0.07990065822922199,0.056927634839412926,0.08486217043966604,0.04926548194124741,0.07801564962819085,0.08253508987162816,0.08849868871743719,0.0795278420702176,0.07470559397741663,0.07357004554971823,0.05939335585992803,0.07103377835292132,0.07464888978818301,0.06915080830439715,0.07447826367911446,0.0853078029807898,0.07062047407890573,0.05945656104607911,0.0729642391699935,0.06949827912202206,0.08319504630431851,0.07697813827217584,0.08007722439397388,0.07438577595230672,0.05472264044648059,0.06906061229526822,0.08550812711704373,0.08703704265913847,0.08625233321756368,0.07487717882289531,0.08159594709595863,0.06555987138411398,0.07687495844147632,0.06891698803376745,0.06775167653262662,0.06142145122452555,0.07093672346416,0.07940084062597785,0.06508712413315236,0.07820937065638744,0.09089666066234293,0.06006513862527146,0.08237750399562849,0.06428074924917479,0.07195913189026573,0.07772256404707506,0.08202015564558741,0.07531936784162847,0.06170322333081051,0.08274326045455763,0.07165988538562082,0.08072932218079851,0.07231783766781907,0.08490918380149093,0.08385004142397624,0.07044388819880004,0.07528131919534203,0.07869827666110271,0.08615862172348746,0.08305554489634756,0.07523231469473518,0.07026684531716401,0.06756197275742756,0.07671154191719927,0.08484552749377419,0.07860867354007978,0.07385829471113173,0.07658202896153823,0.07976630547119667,0.07043335720733022,0.08015365561836871,0.08648564334407882,0.057320705434963745,0.07297865871441987,0.06717550872042072,0.0696086483679547,0.061284709943467235,0.06020362177125341,0.07665985495473471,0.08005964998851253,0.0705315499110839,0.08373123787248656,0.07120819986855179,0.07455920647597142,0.07908982506106237,0.06979453868172311,0.05998435534854524,0.07341860385673715,0.06380286425682391,0.08347814360334171,0.06678511507690883,0.07528397959279101,0.07358989661519807,0.06639747972612428,0.08001932464322353,0.08157014644461752,0.07888680033858134,0.07709475500712502,0.07227221169790451,0.05945473950016274,0.07059660332863893,0.08049742257637141,0.0832609086246621,0.0679027142098958,0.08242750208322559,0.07031161301888772,0.08784427542911932,0.08468526290861231,0.07799061675375887,0.07738670140230687,0.06422531116097283,0.08255875828397666,0.07544980817656183,0.07826122678474265,0.0785367959191475,0.0657913892514917,0.0835583198544865,0.06903751746447193,0.08977618119670522,0.0852206272995562,0.07513566300183072,0.07931939833254245,0.07793542116086444,0.06441352142399448,0.07360629016453918,0.08660382988436907,0.0659687406429644,0.08691753027675247,0.08148566423432677,0.07983441204740824,0.06257159777030909,0.07778687632008598,0.06913847891853868,0.08204936840350943,0.08262681102180579,0.09560492274002526,0.07648184733338281,0.07775363494275474,0.08205555205944617,0.074328103284161,0.060041288460433245,0.08602881523726286,0.08305437103135821,0.06997842894710046,0.08266146224002188,0.07180833991700264,0.05923641853939332,0.08433674794765676,0.06985126031046504,0.0887698872753297,0.08361740224820803,0.07013486185038692,0.07360184833692099,0.069570644055829,0.08941408191461317,0.06691747107691358,0.07806689548302598,0.08094644628164023,0.06130159707292472,0.07426745746147242,0.07251295884758407,0.08009278816920326,0.08087391209804153,0.07048978825508587,0.07088708609506353,0.07447567269884932,0.0818128212674656,0.07663148843656874,0.0639046212648298,0.07804383073514082,0.08486775027663393,0.07934625146944135,0.07943059146553774,0.07715510357200328,0.07727431183357805,0.08000739541162252,0.06657153893181522,0.07164816621869058,0.08170215712683884,0.07006425337243079,0.0803269396339403,0.07673820068592918,0.07733649120079274,0.089622238746661,0.06645450884318366,0.07113629007232214,0.07919276439225303,0.0779578999291769,0.08317828288949144,0.0867319364534235,0.0863857617053019,0.06724917326980712,0.07511403030871158,0.091761085236584,0.08407636337092822,0.08853469107447363,0.05964628367735601,0.08775174730037083,0.08145402826015143,0.07562665074494972,0.08581506870824851,0.07739933575233299,0.0738144005396527,0.07981736846267995,0.08992202980650468,0.09103860870183896,0.08680015451372378,0.08187837111347297,0.07722758443170719,0.07579470372822802,0.06006775774329999,0.08112774396828693,0.08019427378975064,0.06458330997075937,0.07914902121113584,0.07647983778723871,0.07661895461896559,0.07485791266596492,0.07110184168939421,0.08590551532458068,0.07071550446376759,0.08269203715677828,0.08495465204688507,0.09046155428733396,0.08559340886795958,0.09031004534650727,0.07979138259498918,0.0792211890789797,0.08342819309585285,0.06281692808281039,0.07511335684160163,0.07687116997287494,0.07788489861843748,0.08502244423533047,0.0932482239342201,0.06265889343408682,0.08232384306090332,0.07033676694972926,0.08033384488969664,0.08374565157720765,0.07316898382006073,0.085914177651778,0.07642583452939707,0.07945956575829821,0.05680106910404745,0.07702508287445173,0.08625470956524028,0.08218257700561615,0.0606741789372857,0.08586515456885127,0.07851318554730682,0.06878464093586542,0.07158503928999721,0.08417519510985434,0.08121268651212958,0.07686342332967133,0.06856202035139457,0.07122404177489743,0.0912201704862283,0.07165711067382002,0.09035512860129433,0.0689665191489632,0.08890002254817303,0.0756437679122946,0.06148882252016151,0.07231863693895861,0.06702191422741782,0.07778051927078508,0.07862581011384537,0.06955670929899396,0.07339560231961959,0.0742260100056116,0.0742864378949222,0.06729301464534551,0.08128492454534886,0.08165356410339591,0.07840764704087486,0.06515672682438331,0.06592891739752293,0.0906762484828435,0.08737553471876983,0.08672619199932136,0.07942993990969588,0.06601826495172522,0.06800224947686545,0.06506365038696772,0.0662244825650791,0.0757786961596908,0.06635636478425061,0.07862500302450025,0.08145025610400036,0.06116390375409373,0.08014746018024359,0.08693997251493485,0.0793645057655353,0.07880981713140346,0.06880317078934968,0.07155650806723506,0.06822646299247057,0.06194976259135375,0.06474338019591885,0.07050934821551592,0.06881821413230041,0.08550744313641113,0.0885229809031027,0.07244658114770483,0.07244977522781088,0.08195071446538235,0.06661244367289021,0.07787960068885214,0.07227686413174123,0.08700915148281758,0.0974800728449574,0.08898369364769586,0.060582074478982104,0.07160041044380205,0.07866618589752548,0.05985838534951303,0.06261164945951349,0.06479701395297383,0.08555226838354099,0.09096169433763587,0.0770894679246983,0.09003670763622937,0.07711264815049668,0.07727875567102904,0.08586159564266671,0.06236381887174082,0.07234259896884276,0.08362907925444554,0.0913044401798514,0.07706106282570588,0.08608021360277077,0.07582584342025322,0.06996403457044076,0.06460882403869703,0.07104633673856979,0.08777557037519154,0.07560889729271383,0.07720281836288898,0.08559484410278592,0.08485470494105882,0.08112651786675616,0.07705404189945628,0.0661888624217671,0.07483026195682183,0.07147500756627617,0.08113111676637047,0.07653427748997577,0.08804781309323487,0.07730789433842691,0.09041218683122128,0.08221364526893321,0.07948639153924046,0.0775602040304274,0.07597659735275572,0.0869662818564148,0.07734559485288676,0.08415989051838627,0.07709752352264132,0.07044519545939248,0.07338254284462009,0.06857734106125553,0.06374956026555725,0.08937992399282445,0.08231611263117028,0.08232180146527446,0.08343887736056674,0.07081135731213765,0.08512954944395151,0.06803558124435422,0.07685940147295389,0.07392638267090765,0.09101986237327964,0.06187183857932456,0.06581486147270321,0.0843306886169263,0.08920941627690569,0.07754259380481299,0.07605267229488638,0.06238935069787669,0.05910861787415958,0.07247749499309519,0.07693016730627378,0.0724311285709206,0.06299608129084552,0.07035110188543488,0.07653427438777662,0.0729412961974596,0.058241220373824745,0.06600380531740183,0.08570842790648492,0.06836902041954963,0.08055424175242519,0.06060276657574207,0.0911226473167219,0.08113748811051999,0.06421182363969986,0.08382100261259628,0.06442365750339413,0.06346227743503127,0.08826079809758708,0.08924681174612657,0.08233162185049268,0.07284541540115287,0.09231439776751664,0.07771584669013218,0.08293217531995381,0.07390558006339903,0.05603617619503026,0.07198435459463871,0.08697981335126934,0.0559415081609259,0.0839778204890286,0.08752504567647232,0.06477056891521654,0.07862737040697373,0.07352320531154273,0.09117865952131596,0.07180000956458413,0.07408618411587481,0.07117010189914877,0.08177742940005245,0.0920430975178555,0.0762799599709115,0.07688847453428403,0.06250064924609638,0.08393186486813822,0.08320208059937517,0.08843113913323482,0.0892457883228934,0.08435248176903862,0.07623094482779946,0.08441155493569316,0.06942843414715125,0.08808796814234446,0.07767938436543528,0.07296926259336364,0.08504292311918014,0.061495151337341555,0.07245539845386295,0.07857875385536603,0.062447791035069054,0.08626570654920528,0.06060358381431481,0.08685150752805451,0.08000538774786876,0.08838438193494672,0.0630920135095393,0.07602705105272814,0.06733695344875394,0.0865038543664208,0.08390428574204459,0.0784977664464829,0.08370854068913115,0.07576747681248025,0.06670691986999616,0.08123917193653779,0.0914760381210144,0.07965441232799342,0.06952755159771128,0.07814289882214724,0.0753863414424772,0.06860076341218645,0.09036024064341916,0.0836978634119967,0.07282514902079093,0.07082231984692494,0.07396030153363042,0.07567853919649911,0.06622733308451287,0.06803919130369868,0.08698286986869722,0.08733865721540507,0.09426946363413403,0.07955574031128396,0.08582566913094357,0.06130815690242045,0.08161057619760421,0.0805819680589791,0.07637066258416311,0.07765584388167705,0.08440937684083206,0.08648952962257137,0.0785288772097023,0.06709928714318242,0.07561520817607875,0.09156286607646033,0.08806284367013319,0.06459606684373839,0.06957871493985898,0.05893940237796037,0.08485276005127362,0.08971885712110693,0.06438215501674945,0.08420135796866897,0.06919512470492212,0.07608465976747436,0.06645298681331162,0.08361571178211098,0.0914620471420933,0.06760967546997379,0.08236405779219168,0.08785229796323082,0.07239208883304671,0.08294034071761923,0.07953821636225103,0.07193086832154477,0.08880500138150935,0.07599641800345161,0.06915913675030128,0.0675767319689127,0.061299603562172685,0.07093835873608667,0.07808004027985027,0.07889694925506804,0.0663920052388835,0.08664639150197966,0.06468549986731853,0.07697419715144233,0.08299405288371753,0.07454739524465609,0.07766025029578438,0.06644137750644138,0.07263857193659368,0.07582271456413629,0.07872915596736552,0.08924070178227708,0.07273587769715553,0.08538480522192349,0.0719983731875994,0.060752765990311236,0.05952873709445569,0.07605762429620092,0.06125094057096851,0.07248467582484217,0.07900665096004356,0.0752583251751936,0.07158315981873453,0.07495888424682101,0.07526085661996232,0.07187189570895129,0.08401613059373526,0.08596273328241952,0.07588281381691003,0.08879416225643394,0.08808654715210332,0.08187569610142029,0.07735411111320678,0.07640141857421168,0.07811291657123702,0.0818693251631896,0.08277336418412852,0.07907208188356539,0.07445641506484901,0.09074197402065758,0.06055416095736618,0.07387253364846827,0.0856326883061479,0.06743056242267409,0.08127177492593422,0.07909589064194081,0.07610606143141202,0.0701943733663972,0.07780058809966595,0.06544274039439293,0.07785954655635999,0.07235802272917821,0.08041995383128651,0.07958272963753997,0.08106626871869835,0.07661515414630579,0.06581867002517,0.08762264471478656,0.08236373579662311,0.07296006621718917,0.08818464124051031,0.07939296867669046,0.06649989549619426,0.07808860082055372,0.09192637609184282,0.0832953552637099,0.07836849947153224,0.06434585197820325,0.05396976447000347,0.07825144537338187,0.08018035864128081,0.05933717104321891,0.06340222965359893,0.08512533409292916,0.08286628514229408,0.07251234834293727,0.0724269355648792,0.0734428747113319,0.0669708171917783,0.07579366724106916,0.06848990255152954,0.08088441708656235,0.06879031590402779,0.07935601701402854,0.08527653822382386,0.06121478775309381,0.07593926476528265,0.06793600251184087,0.07985996398565856,0.06800075607999484,0.08718375750019872,0.08511556750766477,0.07556758905611305,0.07550512099379313,0.08319594625696251,0.07815460429206973,0.07373557727176847,0.06519690330085381,0.0717676542469161,0.0716858859611317,0.07857130225845611,0.06793703679922856,0.08319447564987237,0.08044546587278767,0.06305522339687589,0.09031804697445187,0.0738895807876219,0.07970242200770691,0.07903352685202641,0.06337915447115444,0.06755778655714262,0.06835737275940382,0.0769529929124991,0.08843637751637654,0.08970026827141608,0.08247233940795208,0.08063980180008432,0.07889362788905058,0.0847471968574404,0.08324438975358651,0.07056480659128392,0.07540309440912213,0.07683774350118684,0.08005295079506575,0.08515161565953627,0.09133154190980908,0.07405996708341547,0.08384308165902675,0.0674990854880177,0.09040898269493552,0.08790560629412958,0.08433088709260583,0.08406249591198575,0.0661987502468995,0.06755758008271613,0.08243259440516093,0.09002370799968029,0.07966441229244167,0.07643669054639532,0.07239835014290744,0.08058991330228406,0.0739501082343065,0.0840019736407491,0.07825964712267801,0.07911350447017262,0.08680769014935553,0.06269278102388573,0.09369779315486519,0.07958322784222507,0.08139969760119646,0.06773063846937874,0.06885678807067397,0.05954888687917918,0.07879477654493384,0.08396410203955124,0.08525167540496909,0.08551756104008512,0.07991990299102039,0.07277105397180697,0.08301726537971159,0.07152765368668836,0.07109058290509473,0.09147691940914268,0.07529799523438306,0.08636690765951752,0.07762926550440646,0.08584354437583734,0.07814049727713929,0.07137777497554315,0.0771833625848827,0.08499542338897913,0.08301260301266603,0.07280928425527967,0.08616429681665608,0.08309376499632021,0.07342366534653308,0.07815116654343893,0.055346739756100875,0.07414687984368691,0.074980855861804,0.07352954563966355,0.08547841582890578,0.06799161757356453,0.08268768809297306,0.0840645518565453,0.06601176075659601,0.08856346182107727,0.08209722855206814,0.07159337176658362,0.08799384422158395,0.08695051617939527,0.07097802906848891,0.0741095004028342,0.08569652841169284,0.08243494686281208,0.0794108011532833,0.07199034544755031,0.08143441278088728,0.07851571062005003,0.06968802562580934,0.07111513238106307,0.07434662542975051,0.08138134812572903,0.08189146321889815,0.06076440741008044,0.06892674898185962,0.06922534478805628,0.06549374960432883,0.0779504918783052,0.07876438177893764,0.08322921159444178,0.062110529744483375,0.08013992817960104,0.05715142272611047,0.07993999447607021,0.08035171959415482,0.0757586965942085,0.09028978126531217,0.08093924604273571,0.08241978948511204,0.09147388632479114,0.07626042877444009,0.08155605983367467,0.07825131516767789,0.09520244479071227,0.07075376484078615,0.0700498261309284,0.0862793509049255,0.07703462151584445,0.07604789887371442,0.0594673757211798,0.08410902973266954,0.08462239627003311,0.08797148570631878,0.08454480123005884,0.07577111109138855,0.07073726987033338,0.08566399781835773,0.06925326101876947,0.07746423750717067,0.0722369074308361,0.08393849765221653,0.07705392497722452,0.07706326921242312,0.06963484137287487,0.08012842341087202,0.07763342451114805,0.06557875986753357,0.08705178785435622,0.08202734023789054,0.0611693609749083,0.06766832923230837,0.07035335120385902,0.08732529315676447,0.07581445443061213,0.07565849137861463,0.07768021142555318,0.06436793873488389,0.07379734124061318,0.07169506252515927,0.07647996888030645,0.07798727437098031,0.07379454124160975,0.07061114707967,0.08404861465259914,0.07821357482871877,0.07646368305633164,0.07825601644218509,0.08339240651413224,0.06005522468026424,0.06718600040936593,0.08405602665468748,0.08252781263250476,0.07434065522443908,0.07498050149913893,0.07841720748704967,0.08176814320809156,0.08897487105417962,0.0688565000112786,0.08597315121569021,0.06524033026054632,0.08150872388332397,0.05893707101037357,0.0782253044964566,0.08382077947070513,0.06395387832268692,0.0787920399663532,0.07789865544601512,0.07575193020776173,0.07019017456650667,0.05621809680575593,0.0712172705258422,0.06841685953556352,0.06266229082465777,0.07586524864624869,0.07247705713490427,0.07801691868613925,0.05504321246829915,0.07030467283405528,0.07536419687064343,0.0720660003977302,0.07333662728958248,0.08210254633217741,0.06711769585108932,0.07599608168665523,0.05672863496036139,0.07155452271480944,0.08170399320224292,0.07327301172637501,0.06409272053677828,0.07559298049255328,0.06116857332722965,0.08347176761728196,0.07825912422358342,0.07156130171444994,0.07676849396821983,0.06705531994114741,0.09497081788665689,0.07517231686392159,0.07454769308910551,0.0732109595767067,0.06693713931990874,0.08894511014344136,0.08170771216805292,0.06522007871242444,0.08453384287018981,0.08023642265182826,0.08043099296991302,0.07172020500071416,0.08199922605247949,0.06618463716923326,0.08549625905149476,0.06645013666158356,0.08128549249258149,0.05915530554130605,0.08458254018504917,0.08260484676002436,0.08179844163501539,0.06977887317018597,0.0816404291443318,0.07628109595614929,0.08655233517283481,0.07976757571814366,0.06266759834986094,0.08519043529162185,0.07640443328006832,0.075807642012743,0.08650825763199968,0.08819054754467057,0.08229007234849584,0.07091827565077703,0.0782568697657161,0.07545936548863755,0.08544337565821866,0.070982303829436,0.06324732643235617,0.058586858232642675,0.07948602220713197,0.07882226291453573,0.06968132012710508,0.08668675031559625,0.07991984896149718,0.0652180801299603,0.06905075361603816,0.06133478413645378,0.06488204192550087,0.060970181406334256,0.079107440634725,0.07904579642132052,0.06891140063509421,0.07795709810183106,0.08056695933550986,0.07604777790578092,0.07274130244374649,0.08369407357612904,0.07366834754401522,0.06851150255060956,0.08444132856482434,0.07330105538230774,0.07949075820496954,0.06716833612052828,0.07807286708788563,0.07208567766846223,0.0728197192549124,0.08431601539327702,0.07181379519435,0.060686247931755095,0.07580489633215279,0.08404592288745771,0.07494765970424139,0.07436148633375639,0.08065174186587408,0.07639660140637144,0.07867733365521774,0.08822829543168692,0.08344105040558707,0.07811786250125011,0.08429259088411384,0.0756060106513233,0.07964465144500323,0.07845381898743752,0.07484210036320477,0.08250094496721821,0.06724832019269089,0.06730753036647649,0.06785922820894214,0.07435766025574894,0.07486631122250616,0.0778720358777922,0.07119456407782263,0.07347932571089316,0.07414869008001987,0.07331170735908812,0.08013896803992285,0.08482851289112801,0.0730874623438012,0.09133504920117998,0.06890233693707347,0.07720130478705836,0.0878941663981786,0.07504802697308433,0.08732067870170057,0.07459155803602238,0.06623484858406674,0.0836852729536616,0.08723456863003605,0.06623593799229488,0.08760452908395874,0.06906171585067208,0.08159916289640673,0.08606627156608837,0.07940616228208869,0.06519242533040631,0.0813185852002756,0.07602538085494057,0.07921123107580288,0.08746044476323195,0.08399812235412016,0.07793239716755801,0.08912524140979015,0.07669147726512295,0.08568709438782807,0.08989606343017542,0.07860351286053607,0.07002326192021857,0.08224883722159107,0.07880583559676974,0.07124794130151645,0.07651274334192779,0.07865845104240668,0.07418366862844272,0.07823132732547189,0.07921756284793911,0.07392013801531663,0.08803694174133361,0.08832575471693702,0.07977067283548189,0.08425332817102538,0.0911278519137169,0.07501753889577956,0.07286055625671466,0.06422004640068935,0.07932491634439726,0.0774143195975831,0.08006512185360885,0.07364590194389498,0.0553765641778919,0.06850705886664624,0.0807132976156199,0.06842932972080062,0.0752710662850133,0.08521357897593888,0.0798719114680345,0.07797418269062872,0.07494674581494691,0.07764064551857569,0.08566740886278582,0.07069268069326246,0.08319718957978772,0.08009461577787796,0.0800145624995397,0.07150205922691139,0.08171457164616219,0.08245361546288082,0.08264386512370019,0.06777639950654504,0.08066687018587104,0.08207544942404149,0.07209126456752107,0.07717247666652167,0.07162957501264948,0.08168921877283178,0.07757360759105959,0.08139045007249052,0.06891492185810755,0.07943146180626963,0.07320602716330514,0.07009294254609297,0.06951283367780314,0.07921298626720523,0.0786887930152208,0.06792117373280832,0.08063032107627081,0.09085648441068675,0.08534351174190868,0.07623757241163773,0.08533031203673788,0.09126559073936993,0.09099438054729611,0.07351579168591757,0.078896197611518,0.0717886027176228,0.06886117098621669,0.07063278385615204,0.07691159717072056,0.0816420799437039,0.07461810143927916,0.06806433833267576,0.08973510144457314,0.07477998715312509,0.0655997155288744,0.08867878652431302,0.09545946408103606,0.07671906666306735,0.06450338660917387,0.07808220640366129,0.06479947465332118,0.0830404892823091,0.0829267948172356,0.0741923892575101,0.08115216391223465,0.06540840375478393,0.08495700381996213,0.07999701261324937,0.07359273888194824,0.07017867443602793,0.07091526816947213,0.08659433285662888],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Investment Rate"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [18]:
simulated_home_growth_rates = np.random.beta(10, 5, n) * (0.08 - 0.03) + 0.03
render_plotly_html(px.histogram(simulated_home_growth_rates, title="Home Growth Rate"))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="628b3c66-f5c8-49c4-a3d3-3b147988c123" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("628b3c66-f5c8-49c4-a3d3-3b147988c123")) {                    Plotly.newPlot(                        "628b3c66-f5c8-49c4-a3d3-3b147988c123",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[0.06732670930594567,0.06982496692309828,0.06352749718412018,0.07161088410364005,0.05892829751275826,0.060046461321327585,0.057308464663040154,0.06333697372001221,0.056042965992588054,0.06584418082452242,0.061272525238067296,0.065920522739976,0.06670524308960984,0.059815301141895386,0.07384353244045183,0.06243099366858306,0.062365647806754,0.06386276917313885,0.06273570477109107,0.0681937041288666,0.062162219876037976,0.05236966058605498,0.06006379759443046,0.06718073090398646,0.05981065670233543,0.07048627641771588,0.07095376928206996,0.056963366963210665,0.05631292121544749,0.06471092262532666,0.06655051589228579,0.07099829782392961,0.07621825277518934,0.06645380982681456,0.06319485935120064,0.06159544956058063,0.07066752155281134,0.05069208519590074,0.06456908767126429,0.05274133784800891,0.060914338388194544,0.06435934115685332,0.06997196811559953,0.05657673558667885,0.06391962818622009,0.06035290734058425,0.06340465610901541,0.06825289636900278,0.05561805470004008,0.06621560654400246,0.06537644549263708,0.06879600725106169,0.0520685645425481,0.07305363613069163,0.06488160203852184,0.06713758163985709,0.06371670780194114,0.05963767903426405,0.05421654137411101,0.06653299093769696,0.06433280023594055,0.060199263178224036,0.06468856915353854,0.05804849710069171,0.0714195719500752,0.06497518924581486,0.06611265699081134,0.06484757745473005,0.05870629424722876,0.07085013317742729,0.065313988448752,0.07244543152438604,0.07010544655974259,0.06674369992411405,0.06579019373937237,0.054554681202405794,0.06699251424078254,0.058250555665526185,0.06786506140780438,0.06080627674525667,0.059729459958969416,0.06942116999889598,0.06408844466677208,0.06636522298130466,0.05974236791206661,0.059219688990406597,0.054744864951898564,0.061710740548379855,0.07189333855955482,0.04580980616290138,0.060205445927670215,0.0611649757112797,0.06788926481181488,0.06306438826552202,0.056984395321965656,0.06292169093115231,0.06405794883021768,0.06468422193355039,0.054092319218548915,0.06652473715783105,0.06088618896312274,0.06493894604198586,0.06623321887843085,0.06738909875261606,0.06750120467206644,0.07167865583032426,0.06214974727393119,0.0624398809436146,0.0539240664932405,0.04555211389026019,0.07170032174859306,0.0633563737444196,0.064626422063391,0.06957571744176676,0.05855284738297968,0.06504312780357199,0.06524417870203005,0.0667959442620131,0.07016781144251022,0.07069053351179941,0.05988199616611182,0.06794645176581068,0.0741987706020143,0.05045147198008975,0.06156976614190359,0.07284506983071085,0.06558063373643636,0.05491783746345345,0.055945856694284736,0.051432079432157024,0.06748735769814854,0.0650577783659157,0.06069556659160677,0.055060978653904274,0.0660541925127398,0.06751683065491831,0.050330511169826654,0.06836031714414646,0.06065358762584451,0.061826222802183375,0.06295739422087226,0.06399350508381985,0.06702729952648034,0.05944981213284643,0.06234951471202415,0.06524149161803382,0.06549283023610189,0.06365516761494427,0.0643431260678703,0.060460888239143036,0.0556339580461513,0.06478169192863523,0.06321579550048606,0.07305699222802176,0.07553462259380483,0.05976273606923892,0.06339840157976606,0.0697332192619072,0.06461695425037361,0.07182845327793938,0.06761928519943675,0.06535970597493548,0.05698531015664393,0.06753113574483285,0.06475113255549754,0.07043789318069044,0.06817392623301949,0.05322144631988826,0.062500246781974,0.058901351121210546,0.07043620289899896,0.053254879215113536,0.06791756684487574,0.05824208263207366,0.06429959567019197,0.06513144318422762,0.056414159262376286,0.062055473588024,0.06235298130335245,0.06525064367471387,0.06111657401116459,0.07381172275995487,0.06075986089691436,0.07125646392604379,0.06318077843721331,0.06235220528707413,0.05572249234538683,0.06745668912594918,0.06675634821826952,0.06983866794765445,0.052543101794060076,0.07428035987593079,0.053556649347671695,0.06797365033287445,0.05143975182785537,0.06172370135039719,0.06429566589136514,0.0631428678204887,0.07047842151742448,0.0696542866813133,0.07392455249318378,0.06682127623847212,0.07019256504832475,0.06698989824157542,0.0627871342594098,0.06656359860456784,0.06396990586319262,0.053244084756004735,0.06383581603444835,0.07149715536543125,0.06181254623110839,0.06340989042830932,0.06894717359685909,0.06058139650877856,0.05842043700176204,0.06154882933942515,0.05595970653237714,0.055438698725506205,0.06303790732839679,0.06425700130312095,0.05882316048895201,0.06397098920645461,0.05507671541239154,0.06178258980619948,0.06971405498350938,0.0633476861319269,0.07164946394977445,0.05686536998755164,0.05874341476796094,0.06911017596180108,0.0631966513445511,0.061859256774085276,0.06685655152903394,0.07301785927153291,0.062020359634469724,0.06381437891063145,0.06868258441175118,0.06309641157075586,0.06866896971989259,0.061859764743726516,0.05380408497572872,0.06297216699023339,0.0663630996057599,0.061024287870686374,0.056284857610263396,0.06414623486154361,0.06017032434978224,0.066071170030077,0.06045227640861476,0.06410218067896892,0.06860564254659884,0.06220889950264496,0.061550576086612956,0.05897137464561632,0.05924569310124844,0.06832433011310243,0.07439882809552494,0.0641686750764141,0.06025014126648759,0.05615962868364083,0.05944664924748745,0.07277297496736809,0.06505984469556764,0.06062972361707855,0.06436160555585646,0.05698310847477829,0.053897071749175254,0.06479356716357235,0.06446205254162338,0.06115547998290667,0.048254009546726095,0.059006699132611765,0.06616764149337984,0.06535292157711431,0.06095801621691124,0.05968573854587085,0.0689379790766991,0.07290224633625042,0.06665109786691709,0.06753224523387028,0.05446481310416017,0.07054778900305257,0.07243087460226452,0.06580023233150135,0.06615504985040224,0.07276983952012625,0.06594404313148594,0.0641354778238706,0.05875639390951663,0.06925954864250192,0.07047324709369025,0.06036621093492105,0.05848534554870261,0.050844562361081,0.06536858090520294,0.05530527744060931,0.07168712116718849,0.06576030419238535,0.05939658386625753,0.06209573147511624,0.05495698690027491,0.06901372941513859,0.061723421626614365,0.060124075645507744,0.06800770676924045,0.07261456233399138,0.05364217400831907,0.050774981159297336,0.059034252363959594,0.07219138015668058,0.06549533731955348,0.07008406101698884,0.06060153724539375,0.06636727075107686,0.06629947026744568,0.06360794072317938,0.06796163747392653,0.06596288741333514,0.06671342106233207,0.06752754255291651,0.05843687325437883,0.0703400515874962,0.06136110660924046,0.06244926216005663,0.057515101504204726,0.057468005050723775,0.06946623930444248,0.051744646190860125,0.06462476455085792,0.05996210061406216,0.07376373283304923,0.05225646154247854,0.07108364752043278,0.06011760735664261,0.06194432062421484,0.06002383170090002,0.061910954589882404,0.06163991600331109,0.06353368338493322,0.06751569090314873,0.07270620315889942,0.051070801550406886,0.06809322660166303,0.05754943830544302,0.06176099942108773,0.07210352003165946,0.0707337835970106,0.06436522218704108,0.06369846459422374,0.0634526643863774,0.0627865192985744,0.0657075551423911,0.05984723948362117,0.06611644913133094,0.06488585665837009,0.07264386401684947,0.06430957491150927,0.049947192577436295,0.060707785838867684,0.06574568945764404,0.06148860652932882,0.07028732815019123,0.06344913829541446,0.06950921806919766,0.059082138327580275,0.058551868735700605,0.061093046097462717,0.06683809610300571,0.06535708512117286,0.0608968479281663,0.0700330014728132,0.06572061533831049,0.05261564735638166,0.06716202542375789,0.06722937676384017,0.06796015526984786,0.073241127898265,0.06654956550047114,0.05664954209275948,0.06689991745104326,0.07001490493600415,0.06718811243215525,0.06206031331391814,0.0576027453888636,0.06823123380240734,0.06645862870847916,0.06408063762745161,0.05313211426477121,0.06042348338290805,0.06276261716437462,0.064005343012714,0.06630524750707917,0.07459963284840047,0.061543771568243734,0.062053812714433824,0.0690351528127996,0.05190683339659102,0.06255066892031168,0.04728701186115993,0.06392544137129813,0.05890299370241796,0.07364041951124636,0.06702299434260389,0.05764486864386918,0.06371575339789678,0.07043448527339245,0.062281856553775994,0.06857677595408712,0.06355075167372135,0.07203000682350758,0.060336914120337815,0.06327660969863161,0.0558055823457264,0.06464879253482517,0.06786061938888899,0.07178960077823254,0.05837993240916545,0.06471042682977737,0.06547278086082112,0.06854929867233661,0.06652718800760708,0.060293816138298276,0.05950286456770071,0.06547111329100092,0.07322290247496771,0.054434876308876404,0.06465475607638246,0.07283832696093165,0.06807193690721967,0.061555602327840624,0.062379786499657947,0.06831482533860676,0.06338160461901057,0.07157381093520693,0.06712069407638985,0.05328697065675468,0.060754222169522656,0.057130481639335706,0.07458240329565421,0.07576476911147051,0.07516812535747985,0.0636533741220526,0.06076601121482711,0.06037616179980833,0.07351984098496969,0.06552231392299623,0.05669422426072008,0.06261442792327829,0.05634043002984291,0.062136772002549794,0.06489527384187838,0.07302456835193835,0.07083206557170217,0.06175617734797585,0.05029607593397012,0.06950386088925972,0.07441481963910482,0.07339959943888874,0.06607060897559769,0.058633493378539236,0.0677124074448231,0.06111992746582826,0.0716312917747699,0.05945692610974995,0.05504193856654782,0.06912008004549154,0.06991547025039493,0.056621915935659885,0.05758643864385325,0.07095185649471658,0.06661626230023385,0.06399261945694665,0.05709629333665761,0.05674463211484168,0.06146090603630991,0.060248845592231655,0.07025911443194974,0.0715984714931823,0.0656165314809023,0.05862112938143696,0.06716855050083317,0.056790617088579465,0.0687986738673722,0.07251514914828072,0.06355847549376087,0.06155215271251659,0.0636850161470873,0.07373745949889782,0.05772327418981749,0.053251407982516145,0.05411152127318315,0.0647773338511329,0.06132562900888189,0.058830941388866034,0.06708952692354592,0.06655753901225094,0.06642177750251653,0.07449085824663829,0.07050237134833282,0.06147701714346971,0.06570042574283677,0.06612585063160376,0.05840827102617305,0.07710986556472021,0.06587140767561574,0.06364347000676895,0.05288454982824342,0.062182679915848596,0.04851085531582662,0.05966451521054361,0.06621826307073707,0.05500883356249403,0.07146526504979642,0.06324093647457532,0.057908718693667295,0.05933980209763072,0.06377346007045456,0.05739997929121904,0.07095024762857807,0.060945602545004446,0.0686544867980618,0.06522874646891083,0.06875694915684732,0.0662850441022122,0.059174250764329875,0.06118622143736162,0.0664817115799893,0.07022903595597205,0.06515387064930511,0.06253888188373896,0.06826980247776604,0.06017869078963438,0.06522599670839013,0.06715669082142692,0.06488046869580194,0.05700807904987562,0.07158393544381808,0.062318625737671336,0.06636855726416754,0.06013686344414908,0.05930532133940772,0.06333431396057382,0.06518830006760892,0.07190065228930748,0.05238977002066515,0.05962716882372763,0.06344864401576904,0.06105394747356249,0.07214723150964023,0.05942168372356707,0.057157781824612744,0.057636355503754774,0.05324313217908622,0.05795704496730448,0.0649437988648455,0.05710453726316654,0.06196499532041672,0.0685648853062686,0.061367504861045724,0.058832962848250216,0.06481379464057196,0.06517101960554053,0.07307241137658105,0.06633728029324643,0.04549848821271579,0.05718123658016278,0.06546073533282096,0.05321814270443906,0.056925375553375956,0.05907462112497368,0.07001579332704264,0.06726765031511703,0.06141052571792701,0.06057064184535477,0.05831052215802758,0.058999823276273844,0.06475239575257588,0.07157332698957311,0.06855895084559681,0.06366895618806709,0.05752976494376982,0.06012907331855611,0.06413187668991345,0.0692907570722946,0.0683759697005227,0.06149848473727525,0.06376644579952082,0.0601431427710899,0.07042541334962013,0.06376003919006765,0.07044658130764098,0.05252525360278838,0.06597549019091535,0.07338748312822363,0.06689013158407767,0.06376959375411324,0.06960969624286316,0.06650684298712392,0.07216198006811195,0.06650472770225233,0.06627476780037131,0.070160308765821,0.0651016932328462,0.06498328028021204,0.05750330393300146,0.05754214687142606,0.06741358211474244,0.0693897665635805,0.05799098747895222,0.0643169601725739,0.0629036648111754,0.05847995536722225,0.06938034821459296,0.06888584205058038,0.05508338706907216,0.062016443253346,0.06885164234119773,0.06014048566383951,0.0740159304081364,0.050520079976584836,0.0628168377221135,0.05956594521757379,0.07302780634301151,0.05763724447175162,0.06662028901884673,0.06717183214835053,0.062413474341529905,0.05260458161394209,0.0717702874122332,0.06311518969473566,0.06830435101858828,0.06796678827050753,0.05885481183709036,0.07175214503711465,0.06924301526877008,0.0584171051503557,0.06718542450627646,0.052162862325941,0.05907464372077567,0.06489716309520821,0.06703269864812925,0.07405097130502233,0.07068840873890261,0.061385843679387035,0.05108673449746669,0.052955631638065284,0.07067445649836315,0.06487326437845008,0.0721250525109216,0.05390273432587972,0.07302725397465093,0.05631387247657703,0.07190885294519386,0.06286571570722232,0.07163573821200164,0.06117391092906903,0.061821314216349385,0.059841219372638615,0.07014473492840684,0.06493479916411263,0.06944470941580477,0.07707165164766511,0.06988847097672135,0.06190306361319236,0.06026529858248781,0.06045746289616472,0.06379705418826234,0.06022935195520197,0.06099825967716719,0.05114106548894716,0.05892032541580989,0.05424644952271126,0.06213541139730886,0.06998779543434624,0.06240526325569379,0.06113442365390576,0.0664645532202747,0.05802114373439735,0.06512569317973638,0.06458315010860358,0.056172210621990445,0.06552141445651588,0.0631630248897103,0.057237482416015534,0.05748074928781538,0.06712784145941575,0.06580078453157451,0.060809200326937735,0.059494292320686024,0.06668894697918984,0.059250441326451866,0.06715699281487345,0.05728936361277617,0.06454088454446924,0.06582281770028933,0.06904092984010586,0.06719447318854935,0.05198673095704273,0.050476254887042785,0.05640240377904428,0.05698705548683254,0.07209279778989194,0.06567580449505883,0.06278628862242482,0.057454533245674774,0.06251762471726391,0.06338986196658863,0.0648425296348066,0.06306062102546262,0.06129272170212404,0.07028358842393242,0.05986713613710276,0.059048459258164265,0.059990007606660636,0.06605676823045381,0.05467582591545117,0.07641461367994595,0.0654712825977342,0.05413457949461789,0.06584201193381195,0.0703052415602867,0.057253035260924054,0.07499502347943644,0.06638810616310878,0.05846497711992936,0.06172126949186665,0.07142092640037762,0.07087891187155201,0.0623765647571669,0.06409625430771776,0.0574505853808821,0.06269985899998021,0.06601853292893659,0.06067256519988338,0.06303981334404848,0.07096572397503625,0.06532324933306466,0.061428220279532916,0.057909422974853325,0.056427036707505485,0.06739860843964739,0.06296910861608468,0.06440295148607914,0.04961129185437156,0.06254561242640379,0.0627576730929581,0.064398643539331,0.06579381355753877,0.06858787456901552,0.06762216848335556,0.06387950212134365,0.06462264869918515,0.0651605223090238,0.07141914984925826,0.07057307245491395,0.06695203377474807,0.06238555504550889,0.06772646625197018,0.06325324971621159,0.07048644079706298,0.0683698171407594,0.07451619329500159,0.0540985806727618,0.06940936021327178,0.0689947653887853,0.060761801826735476,0.06690035047829809,0.07229784636945949,0.05959872216986912,0.06724744492160341,0.054312954909147226,0.07054627620914031,0.0651985491556919,0.05494117727236737,0.06048490729203174,0.061661183653421874,0.06669056402776463,0.054641940716970205,0.06807391603890176,0.053225770491016205,0.06323988770523198,0.06458462209325189,0.06076156640936746,0.0605968156893921,0.06747960907902811,0.07278949991218689,0.06543814259773559,0.05892909713739927,0.061287246293885295,0.05407370947806754,0.06639450221141208,0.05776945402069453,0.062147722221308536,0.07009462050945237,0.06895017082303737,0.06848262643964545,0.06969832472642742,0.06253786237969844,0.06190271894798866,0.05764829388961555,0.06470464659220146,0.06374543304410732,0.06086430485164186,0.0650368064302893,0.0694741056587854,0.0606158987379531,0.06534992672082761,0.0650213290453655,0.06741954850191242,0.06930351078739663,0.06352690388785069,0.05229436139104497,0.0679944064795556,0.059499757688577265,0.06524411752523557,0.06512038213307847,0.06899781660547366,0.05378145512289837,0.06442617796485348,0.06708350375565852,0.06374496189676826,0.06659244132389572,0.06044684425257894,0.07530299425233763,0.06337143495298911,0.06986187134698713,0.06516002810192528,0.06503705063700349,0.0591588446741331,0.04899817071200089,0.06966268234072656,0.06761605387932348,0.06412966529534223,0.06655507995962526,0.0597520726335663,0.06544315457178515,0.06089216601863745,0.06273791468245699,0.05691254465376484,0.056108784513063545,0.06332718483514857,0.07632355275651076,0.07054304817719395,0.05362946154353465,0.06262406617777802,0.05917037825822156,0.06442748979230914,0.05596741528966634,0.062084844195800376,0.0660872543044953,0.07007849061388183,0.05761780391084702,0.07179809669420512,0.06041469985766133,0.07086482190303563,0.055191450326241634,0.06152263847259053,0.06152614472381242,0.06810250838416237,0.062377767451082554,0.0622391579214486,0.06787792844781015,0.06477394019454852,0.06326969541689964,0.06205416432687868,0.0737280026663162,0.06317500861697731,0.07240237133529875,0.060159335402168904,0.05943477269186419,0.06089776919463797,0.0575119082641361,0.06144667991434401,0.0660116023613731,0.06303897864898403,0.06458112525954715,0.05603254225060558,0.07150499094047091,0.05289428432677115,0.06541188601866695,0.06575245780005592,0.06022582565598411,0.07395774916991402,0.06926326689945064,0.054162952432418074,0.06288145645647256,0.06671789090273979,0.06977731475932122,0.06526594435309534,0.060502975806018794,0.05111548440800755,0.06800421908570684,0.06223656119392536,0.0646085786237161,0.06831899764702416,0.05542282628377783,0.05607508665924306,0.044164572447497656,0.06132051232274169,0.07520044262403991,0.056792062826254326,0.06588697315241876,0.06513938979490033,0.06964471896440749,0.06419252636383282,0.05966318867301515,0.07036740613577985,0.06826014692989463,0.0683884949170318,0.05275289081506275,0.05669711774964452,0.062281714006561954,0.07052148221527642,0.06296696946265351,0.06130467390370298,0.06260904509839377,0.06670633424111153,0.07080362779370722,0.06533119576563726,0.06669025743666374,0.0687396518939889,0.060551288109595265,0.06573934597326184,0.07015033056262907,0.07113900139283916,0.05897456110321808,0.06588453828105972,0.07210311274454834,0.05217941427844178,0.06551421929876525,0.07399803767983126,0.05731249313305409,0.05629641711248832,0.07269437623605554,0.07002890407085469,0.07108044005075777,0.06027304754208912,0.0660454046127534,0.05748705151832002,0.06922850570293375,0.06368196094105752,0.06151419278373593,0.07173140407233661,0.07290179301106933,0.06999600460787245,0.0634004545146695,0.06664485688583108,0.06358083896660563,0.06687793103423004,0.06572904561534004,0.07430328609152598,0.06049594612217951,0.06802262623576127,0.06763780303664096,0.06176631830012584,0.052929306850894586,0.07318737709509968,0.06129224778666835,0.06441447234251213,0.06916099836420346,0.05942341965513753,0.06453350308388621,0.0605567229995376,0.06967459234452067,0.05355183145769765,0.06894886754139572,0.07388274090783925,0.06702968455081962,0.06460353225442833,0.05959982253475185,0.05693992523379266,0.06262803213084253,0.07121258897156008,0.07367767568225203,0.055012963796260896,0.0709474015727132,0.06120874225167937,0.062071700751327896,0.06502022936430252,0.06314884586156823,0.05807078798558127,0.06544365949641978,0.06576379994805698,0.05886030953888596,0.06151497711795281,0.06321649720819666,0.06177551559545831,0.06694675720154387,0.0647095374854636,0.06339706616613575,0.060621025578236155,0.062481196515243774,0.06733228883479903,0.05978167984510111,0.07087678854360466,0.07360176551673908,0.057776089485861,0.05234548251707485,0.06602006055503407,0.058697589241966136,0.0603844112031995,0.06288105883940392,0.06381143891937915,0.05375477861863664,0.05895900683046102,0.05736444569321268,0.06584865458280427,0.06117325256379508,0.05801284382294637,0.07135077515795288,0.06370885900796112,0.05818240264372754,0.06914633308508226,0.057926628295808115,0.06927033281090955,0.05843824110043029,0.06948560401951567,0.07186248709149012,0.06166148674198429,0.057259955943710875,0.07021545239074586,0.06900234893884918,0.06644570732697502,0.05470936725035234,0.06133031497795545,0.05530427272120697,0.062499290459117354,0.07203957008615575,0.06938313432100429,0.06357050045631599,0.06510091891367883,0.06729078047338097,0.060362797249559313,0.06887738616812443,0.06651195385743915,0.06700596454436972,0.06370054875050865,0.0586292005864419,0.06407901864592591,0.07300785135145024,0.06986828023482583,0.06385624049632652,0.06798358597918573,0.058839848287261756,0.0695510819493258,0.06244510723435573,0.060289813162880235,0.06933627440584753,0.06596060345961743,0.061929713789686494,0.06826270004751644,0.06568310610235467,0.06753135540010699,0.06875785875527154,0.058761719701858714,0.06784791027011033,0.06919923001125503,0.07003668002884328,0.07123635060619571,0.06586902895956061,0.07464912521319518,0.07356652992977875,0.048912012606892825,0.06993584506125848,0.06522931679221389,0.07280526572442264,0.06591307951591632,0.05907917633314687,0.06742956330043043,0.05471483505368743,0.05994010150071205,0.06575924444250011,0.059613256485368846,0.06952209862592479,0.06641427014209178,0.06996339554203207,0.06699475849063727,0.06523248966363593,0.06532168591993193,0.06290892168968623,0.06505051937126949,0.0650694015163299,0.06704810487438365,0.06108830860358466,0.06824460694465434,0.06453472458624901,0.06862316782616575,0.05990170540598323,0.06331018544807464,0.0561711887693979,0.06424294950157239,0.07010137192213442,0.06177097969208699,0.06303798860781712,0.06210798195436096,0.06422791041756737,0.06755394406615714,0.07586055113375606,0.0642408892792807,0.0548704540522916,0.06458927968156625,0.0541369597566219,0.07118810596235343,0.06850967252412338,0.06057726019982365,0.05743352977261609,0.06677226537263385,0.07331136937070337,0.0676482901590644,0.06121456282389368,0.06580641798355853,0.05792467462517677,0.0648609437528641,0.06280218672030707,0.06522801758799757,0.06746388441327171,0.07094767359868873,0.0641046455274708,0.062159563022772445,0.058773011067333764,0.07066139913959159,0.07193553956229198,0.04723037833937244,0.06228674490456198,0.0551231596300268,0.06935120314168308,0.06679422766920257,0.06381558563859865,0.06847012692620127,0.0695071637458421,0.06523193670364193,0.06487567847962467,0.067479928214708,0.06781762356832238,0.07313351333414991,0.06224604436991051,0.06817787854659466,0.06847785805180283,0.06462315643035293,0.06305576572239709,0.05879741426525417,0.061790653309325684,0.06727716668282005,0.06722741553881836,0.06659312647810592,0.07608480561021716,0.05962637317177542,0.054146597834496275,0.07382320238898699,0.06805712236702151,0.062445267116750705,0.05498287597339503,0.06993357201987405,0.05351545295409302,0.07667739349570643,0.062333944890204544,0.05599861843267197,0.05592856892283174,0.05814899638387494,0.06509484956592163,0.06901851272999515,0.06276349713007609,0.06542163034376758,0.061895730261391266,0.05827403540794293,0.07117511699774842,0.07113202053529388,0.04815351791302238,0.06583794912432092,0.06303648304180949,0.06711440492795695,0.05842669262475048,0.07319093523863911,0.06821908953962219,0.0705271990087568,0.06627250666471304,0.0722833049355866,0.048963977811757015,0.07428036346941873,0.0669979133425164,0.05493696994260658,0.05160190475524357,0.061853291951056405,0.059387967894243596,0.06807175406691973,0.06982025217774623,0.07138993620767692,0.05268373627000818,0.06770537827521297,0.07172395718561697,0.0695578473452568,0.06762446768176739,0.05882362342213372,0.06820060299891806,0.06310926471218102,0.05530909363551157,0.07377349568664071,0.0696532835526869,0.06996640406956912,0.06404576062156901,0.06279761116280741,0.0714069453931043,0.06795041074682392,0.06566368707833381,0.06629803140321003,0.07197806781230992,0.0489673824632495,0.05712105877304602,0.06391351058807807,0.07330193001956137,0.0662803305695408,0.07210116135448391,0.05577142830295374,0.05086451859243078,0.06553120234387802,0.05429514262086237,0.07343755553794999,0.06764985854466207,0.06525662113704711,0.049329588691100476,0.0633001102488352,0.06203462480005858,0.0723605661864043,0.07206677423184543,0.06687924411189183,0.06086905326097723,0.0726024266645463,0.04679414213854831,0.0627226300535671,0.06493824882004681,0.058747424067768034,0.061633197024657464,0.056417627304484165,0.05717955057759764,0.06602312217007883,0.059594951896856554,0.07235979824056843,0.05522538467682593,0.06010329660771322,0.0669277340581351,0.06038647193133588,0.06810590684068782,0.06868386677482155,0.06421032971309425,0.0660982157195055,0.057646420191215145,0.0729137215788221,0.054876423317817696,0.057454158179959816,0.050326596607945806,0.06554480332925286,0.06475577541739129,0.06749654558600993,0.0543077993316841,0.06304435327857427,0.05896299716953213,0.05519266592393397,0.05540938230674643,0.06505114172904927,0.04452994845555893,0.056248918964838995,0.06500906619205038,0.06596307331047463,0.06017641009947469,0.0762569629743404,0.06149886200265896,0.06199246972095394,0.061483581729363544,0.06930754430727545,0.06136561706919742,0.05849714511554416,0.06046226839099878,0.059168575989932706,0.06593655860849823,0.054375636333029886,0.06699365223573012,0.060102651557061126,0.06358309283434696,0.05413600817238613,0.06501928694778578,0.06000264659338492,0.06899848066819016,0.0640310016767414,0.04652278544432731,0.06089666590831329,0.07176153990465094,0.0712987782710425,0.06421206301516322,0.063398429904572,0.062077345164962196,0.06939062381394608,0.062279081678501554,0.07286632157847428,0.056056529218890414,0.06677777081318481,0.0636734153317896,0.06237998791700966,0.07001665832661116,0.07361301886483923,0.06319298335087381,0.06800424098849145,0.06057559481630431,0.06003465952680384,0.05689933816006737,0.06749953478240386,0.061144543701577736,0.06643595507255162,0.06467135117825557,0.05366508948924678,0.06308682783572123,0.06877144902031553,0.058360905979787145,0.04762341778880343,0.06562374254849523,0.05806083999109647,0.06965697298507292,0.06796289340173914,0.06866075414252443,0.05110610681672543,0.05095474717330736,0.06340808703007678,0.07036031387208903,0.06907400910004484,0.05423200669476102,0.06590920894655608,0.06176639600151864,0.056140872567713176,0.06749340279240951,0.07080942524369682,0.07380241587399192,0.06267980067192294,0.06841037263132244,0.06967837658695585,0.05810925951272708,0.05939580834425805,0.07004726319110578,0.055436873308048654,0.06058500447761461,0.06443489867382957,0.06309729776709522,0.06426679755610423,0.06207784020955065,0.050772859983020446,0.0712318589345492,0.07347203241948852,0.06770833366220458,0.07504702727947772,0.05881463545239567,0.06464284705279019,0.04648802722421712,0.06844585041734624,0.05819865892812388,0.07611976441782725,0.06198415311926705,0.05688774208535892,0.07069087396630959,0.07050925745660344,0.0691833145936628,0.0676827184011239,0.05719074823365467,0.06774780272915171,0.0673672081658468,0.05577821069093786,0.055677444766816186,0.07124905100988732,0.057550804482928056,0.056674514780807636,0.05672764777109319,0.06992347480708083,0.07389074060721444,0.07236695918224106,0.06381399740261455,0.05703478315611124,0.05799026811544353,0.06353578167410534,0.05383181389324017,0.06883242106471496,0.0667382082264735,0.07134578578981551,0.06903766577288077,0.0687436389674386,0.06445729464745674,0.06023509359108922,0.06985358588386158,0.06489846015179679,0.06883838746659154,0.05320603636968523,0.054354284368404976,0.05591802649625774,0.06440062246530623,0.06073773656438087,0.06860605819583539,0.06528506154620711,0.06303586769181774,0.06207036723566197,0.06460096636514559,0.055552437018893244,0.06136361835930222,0.059885554168107244,0.06722091863190938,0.06371206234223528,0.06661009606448873,0.05642968578192778,0.07595916074911378,0.06837062963095242,0.0632985151506815,0.05693279116087753,0.06295459051190894,0.055117488379911074,0.05875548791255511,0.06404137611825245,0.06570074342878945,0.06363647368837069,0.06961309808380711,0.0661097795133373,0.06896744662734461,0.0658957336090942,0.060802986845243163,0.07069579778185436,0.06468056589680035,0.0677090508705537,0.07500180236437845,0.06487307072758536,0.06537561444538217,0.06547353250391086,0.0651298955669101,0.05982924009404312,0.0661205800259872,0.06226992565791372,0.06517102470782243,0.05962926355167259,0.0674445791145481,0.0712806903644011,0.06638926643655821,0.05729661783181775,0.06276044246490414,0.06918143080250921,0.0653037963370969,0.06547469321313192,0.06330914189605497,0.07360756388774599,0.061503035167821594,0.0657576719262313,0.06450783226026757,0.07101533610559638,0.06520999386234001,0.06317749394079042,0.06609005105672451,0.06816871933850475,0.07341884925620101,0.06742469792571221,0.06421839535819784,0.06542604362919116,0.05986463147320317,0.05903018132300111,0.06067204510689976,0.06704961681721308,0.0640511941665228,0.06469183629427153,0.06374628450764236,0.06938334442247995,0.05858950152056351,0.06465944810492355,0.062490908922325436,0.07491404982327604,0.06816647786852228,0.06301712317019822,0.060276758664451716,0.06929658321676932,0.06079493166326756,0.06402088274521053,0.0625396607663393,0.07071257771812736,0.0649891463252853,0.05802803763448658,0.06589742566456094,0.06248718555138002,0.0662101581307109,0.05760318726164121,0.06266689628124528,0.06062216641889384,0.062101017002871683,0.06587166000940045,0.07423259607506164,0.060267394173519596,0.05862011262116215,0.05687887815501325,0.053366157079896545,0.05886344648716932,0.05370852929798925,0.06684890701278523,0.06771763838659946,0.06791147560130506,0.07162011836006366,0.06424272632154177,0.06977010374370218,0.07219417550367864,0.06411471991027172,0.06722105369419852,0.07006420802157659,0.06358845215774923,0.06319615737123069,0.07458753468896989,0.060716168668401294,0.07192351180972952,0.05772448794341317,0.06837817433503482,0.06588090057710852,0.07706338121538217,0.07338694176484567,0.06330966739728777,0.06802178687965835,0.05793542120758805,0.06361441189275763,0.06826771618512942,0.06767087179568243,0.059951534358114164,0.06000526624296672,0.06613696498552432,0.056032620022011995,0.06885015684335524,0.06673171847040196,0.0682657597318981,0.0656664590764727,0.0629937797962843,0.06250141495685724,0.06406004939955953,0.06828508885649309,0.06205072721562765,0.053311218391576876,0.07375414587744582,0.0641169772135331,0.05601885593895338,0.061322419547241803,0.06379022125126863,0.06591957122078723,0.05511685625253736,0.06398301149917215,0.05493082319348562,0.06908112792495938,0.06165688650368695,0.06230301153173818,0.04851690571158496,0.06774805371764439,0.06712318753942105,0.0594571756483601,0.06998383954857196,0.06880651891237241,0.061583823248246045,0.06787353290322695,0.07138295617912835,0.06400813782317502,0.06012498958885984,0.06565824314317528,0.06786700616756083,0.06216227841956797,0.06195198470509224,0.06909920481138859,0.06588594980430243,0.07141102296127957,0.05907856105922191,0.05808647262929269,0.057052515449070654,0.0683406058889979,0.06962198631806943,0.054717821676186655,0.0579656043498927,0.059578250412126596,0.05332762825762022,0.06990485566873891,0.060316551252188634,0.05618788682548109,0.06965843591251547,0.06480125765247234,0.06204865720853861,0.057749073196570945,0.05778567219019619,0.06195051498957251,0.0685971293942299,0.05577809128371859,0.06505544050580112,0.06019442281122556,0.0637514819578518,0.06375223539584021,0.06684268741833055,0.06586684272583178,0.05042717173448759,0.06065946150944718,0.06706681994553959,0.06699062415651183,0.06302559589639495,0.06039776225455019,0.04991586322507621,0.06577357439696685,0.06995285525929573,0.07217989015613147,0.06262453184372663,0.06339803277204425,0.059055134449374005,0.061231586242092725,0.06640465059430609,0.06670608465018907,0.06324501359162134,0.06821498049883609,0.06535421922096535,0.06104904507432366,0.058973311277921166,0.06981515154091401,0.04721944305209126,0.05874769995502191,0.05565806717538935,0.07107950063682736,0.06788502532949425,0.06487106962717586,0.06360647064749037,0.04890041652162669,0.05897179565048833,0.06991721846593875,0.071585638921736,0.06039552521313022,0.04943716233143928,0.07041949984364348,0.0548546295759586,0.07191097427612986,0.056690088731252586,0.07124720022865352,0.0747622173837682,0.056437621684968166,0.06855911067694917,0.06382803834816589,0.06107724176655652,0.06627835412153527,0.0588288257070934,0.056914938563236386,0.060429561708545056,0.0628078467755186,0.05792402015129181,0.06311312128197824,0.07547979154243425,0.06005499915498958,0.06198088128330397,0.05538416503715071,0.06411559417782467,0.06599310099149841,0.05288595039801797,0.058629898191407626,0.06556867408803138,0.05312646460903611,0.056297765636665165,0.05748939708524765,0.05997878499575253,0.06253813380688851,0.06633652245478186,0.07053559458077135,0.060753229528710026,0.06577815849225482,0.06799816048254986,0.05240459405153809,0.06070097191895682,0.07226765368931695,0.06353520414965197,0.06870089460550641,0.061768219155374726,0.06424276823448576,0.06501306954085141,0.06235022926074268,0.057302169245478526,0.06916164699208505,0.050050904193445676,0.059840486199053786,0.06815761623845867,0.060160036541867946,0.07078755563373938,0.06701535306318349,0.05868342503607072,0.06509123132191168,0.06100771753892055,0.054418812819143667,0.06562699359147353,0.06841447458463333,0.0721630500732437,0.06705926696042583,0.05246400534759295,0.05635894063369867,0.05146195050138473,0.06414555886455675,0.05789371502281491,0.06924672437098689,0.0636220856592215,0.06514079576557849,0.05224414497827318,0.06915183794025284,0.06077882295354079,0.06494493767455564,0.05411565521327008,0.054396055520502695,0.0607254335790517,0.07076998968397924,0.06294952068953852,0.06129997848443812,0.06072075862980189,0.06421918571473836,0.06028990970704258,0.07241076518735728,0.07347273530073128,0.057430013789597806,0.06603043519889043,0.060695886121142476,0.06071126129934848,0.07135031769879288,0.060118913541839215,0.0654086643773396,0.059530155399661125,0.07139704762564075,0.06311988607103966,0.06253376340930614,0.05976552442998668,0.06634311297597609,0.07092354998058348,0.06438915579941833,0.05910764127508536,0.06707616663340826,0.06539449957269816,0.06701277781185903,0.06820532861999527,0.062367166092150456,0.06003062241405356,0.06584000717130481,0.06400121696640113,0.06733943848910695,0.0684554630203191,0.0749747908400856,0.06862945678585491,0.05961678481560519,0.04607628463128065,0.07276486075949562,0.066524845698872,0.07051433722138373,0.06753859919143018,0.0670666124267805,0.06575348372035711,0.06298459255988373,0.05424594842266674,0.05891501814255791,0.06109032231393911,0.05965577960313338,0.06469996788900238,0.06583218523907544,0.05374548596712288,0.06654071218828517,0.06835901080636958,0.05156428131044522,0.06446243164770041,0.0531795019171286,0.07342399631207694,0.05996917704701185,0.06388411952536818,0.06776573374021935,0.06527685408289663,0.0678040469180895,0.05443300610952877,0.06172147462822821,0.06820718610791565,0.06724087188400915,0.0663762954551092,0.06280332997465177,0.06719838979600465,0.06659607665882579,0.06437178734268599,0.057913268436316154,0.06138980696027987,0.06388377976006145,0.06895999630016296,0.0672292334834607,0.06704746805591547,0.059316316106140896,0.055251645639297006,0.0587145964880193,0.0722917282521219,0.06386046168734177,0.06261896405995444,0.056760795755428625,0.06467827087492345,0.0633860340521403,0.06286891727955503,0.07387081369381873,0.06029773432225081,0.0633319435632666,0.05919869879830174,0.061224311249316044,0.061578200485645176,0.07036311891620095,0.06071374550217809,0.0677180222917842,0.06211712901255622,0.07535866870490746,0.06935671347629353,0.06204680593091291,0.060199719932979734,0.052264313634451234,0.07074971231659008,0.05907923963533386,0.06668677146830707,0.07134439681809616,0.06701218256849029,0.06846991968154058,0.05685527150027001,0.060460988958359246,0.049895454147455526,0.06397173323386765,0.06577142508983902,0.06640433040165747,0.07462802713810107,0.06371782644778154,0.06169273439418278,0.07019111156399448,0.06678115560830174,0.054054954079274395,0.06924475766063518,0.06639100659011335,0.055069492142117496,0.05136801877199257,0.0632405905949391,0.07192054372337021,0.06584959857316153,0.05877681760106791,0.06444679894409935,0.06072489992475583,0.05070170563911791,0.06805564922277235,0.06320913472048201,0.052472986870164835,0.060392589208387804,0.054858149596478674,0.06510483528061449,0.06532905710243937,0.06009488813556703,0.0664281523002507,0.06578354290674834,0.06529780050987039,0.05899935321765866,0.05403049568912954,0.05355624053107917,0.06409098670441174,0.06813382046616576,0.06446765584443387,0.05358604309067,0.05615764356881649,0.06441824613984592,0.06615587684450934,0.06171856969893285,0.06031224572483944,0.05446359316701348,0.06517425529019116,0.06582813192298531,0.058745796924475985,0.06985309367016596,0.06279071833943625,0.06051878955308493,0.06792420844310669,0.06613042250485116,0.06495873300802577,0.06590253543437555,0.061139663636145994,0.06866671375460989,0.05852752835777077,0.06694738109787887,0.05333932654680232,0.06157168409658544,0.06186537237541608,0.0635874172609083,0.04978513233754601,0.052351019429711196,0.06330318072699848,0.05558037012464965,0.06697917537125998,0.06345813511411538,0.06655077348524438,0.06076069606661409,0.05953393540837719,0.07230507355091274,0.07095410777805095,0.06097366869598299,0.07466949647099766,0.07018397571720493,0.06184030226433617,0.062424941027329005,0.06922004361149443,0.06425416816445156,0.057251612969541066,0.07558243543087112,0.06335991205711772,0.06830781547325229,0.05845456267938055,0.061799484319105646,0.0673706930722898,0.05885350231769199,0.06598528059289435,0.052594539126785064,0.0677003040956291,0.060716677103992124,0.053875542274554905,0.05999568437621286,0.06741547555691188,0.062482964672498635,0.05066319006423082,0.049902943731550714,0.061342070860537466,0.06663743791788754,0.06647437326125585,0.0712659632854615,0.06080867077246109,0.06394740989173886,0.05851602687826482,0.06428570598588704,0.06630857421331482,0.07278398356665376,0.06659436159540158,0.06005263574000301,0.06265919996267387,0.06919264693581362,0.06806471138794931,0.06917950732466321,0.06467452113335788,0.06350965431475233,0.06977013551591367,0.0589073680540699,0.05775558868677336,0.05632016572704965,0.06864348472371821,0.06440836657757242,0.06003977445241002,0.05995502022839687,0.06973244502021404,0.05673148178249078,0.06290902392735964,0.0615744629572826,0.06850608486080542,0.06505517260012282,0.06312040509640371,0.06189427795944096,0.05106400673785018,0.060363845915263156,0.05809933785465905,0.07034130860974008,0.06353792993005891,0.050627243965793295,0.06656272062771751,0.05135133007082615,0.06161407116481517,0.04719802027387092,0.0631666452291407,0.06870180492500269,0.062319553734722526,0.06621206322275988,0.07142420731153147,0.07399384031942209,0.0601989139106877,0.06101482964827376,0.061441867098529994,0.06879905402934486,0.06474971442401034,0.059636781930011734,0.05595192177043637,0.06997088312654837,0.06399199128457048,0.06677783295750883,0.05750678252540484,0.05891735751520017,0.058427079648529515,0.05324173541730612,0.05723882001532527,0.05516968671210393,0.06596628095445847,0.05894899909718291,0.05559134881262373,0.05642012297933703,0.06115298714640602,0.06311153618565063,0.04681535623707624,0.05817140914542744,0.06409197160987268,0.06522899584607039,0.06024316333872982,0.06362444942853213,0.07056214071183652,0.057793344723943735,0.06509638180804232,0.06653135177708239,0.06772500096692484,0.0666668039436528,0.05940416926951152,0.06901939515803687,0.06303291122274517,0.0658925244481761,0.06148222273859727,0.05393632051929842,0.04642788129124237,0.06641267403249013,0.05383576760679368,0.05054099567113911,0.06187431398879483,0.06540430893290156,0.06829679367520519,0.05782515956013257,0.05622435454633876,0.05796271966666046,0.06543434112510907,0.05206312225110048,0.07092638964309353,0.06531498918557077,0.06506010596410561,0.06652459517594418,0.0618987728616495,0.06422523111170012,0.05822433602805366,0.06420700322192448,0.06301322656326563,0.07119033909734995,0.07205440722019749,0.06396520088727811,0.06942900170879987,0.07535902936930719,0.07229406086968154,0.06357429034966394,0.06584861308836143,0.05827132941841419,0.0644613647530915,0.06038213239749746,0.06270918444444268,0.06779257627643526,0.06141061601130279,0.05804638254982389,0.06072215931633544,0.07240132285641845,0.05642256969413245,0.07231862532014055,0.06184855588683774,0.051689348766221103,0.07117449179439728,0.04783218956995951,0.0727499932190078,0.06052431654129255,0.0717462941603452,0.06957251901471856,0.058634764250672586,0.06050200482782368,0.07250396669610304,0.07052694981454635,0.05966937656789904,0.061859998827137296,0.06581778267526986,0.07155697430546576,0.066256073646563,0.05776230821211653,0.06810886598874435,0.0625262892998637,0.060988847643394,0.0660628449773622,0.0678441798597364,0.06465890075026987,0.06490194157063628,0.06336583928214931,0.06512916837385051,0.061062155062994034,0.06643615585650942,0.06554139737497058,0.06046544906001333,0.07085701983921325,0.06465470019541335,0.067689930703973,0.06198845149530013,0.06392579206521742,0.0629888108961656,0.07553954357895963,0.06839019405869248,0.06953208133931701,0.05920987737950723,0.0632819145895918,0.0609332064119162,0.05948278249111688,0.06960446491456482,0.05216104425997039,0.06703334933786692,0.05815018597179371,0.06307765568998625,0.05785322904805472,0.05877656971006185,0.0658059692682966,0.055018194323912986,0.06762119512403927,0.06252880377208009,0.06265466266753192,0.07033053277663634,0.06881797413911947,0.055781386075398044,0.061897097705019416,0.05700398709118493,0.06380519992475134,0.06354072983565243,0.07312014467378358,0.05627020969401844,0.06406391181370394,0.05853939453691705,0.06469845629688747,0.0725804990456187,0.06639598842623093,0.07295907696942329,0.07505138202398115,0.06030159669273674,0.06127250938136627,0.06951076134993706,0.06254761587979174,0.06770184908809448,0.06998053967993859,0.05485159650976505,0.05697361250463316,0.06667597049037413,0.06459398455963242,0.05606028598565372,0.06401545077867447,0.05649320214344442,0.072156436063078,0.050893949218915505,0.06794832478743544,0.07191572643814625,0.07144692618664727,0.058169689373336914,0.05785004537711716,0.05766707110308139,0.06924222474129894,0.0575605148640313,0.0706926332580119,0.06361826127645724,0.057054217716363737,0.07371909814231364,0.06696636022858518,0.06993695382916756,0.06956590037780828,0.06591442589905999,0.07234045892215946,0.06350817551919397,0.06896529528540782,0.062048334983015395,0.06268752427381753,0.05698823589944016,0.062266603341944414,0.05689528865671008,0.06460038125426298,0.05546548491590286,0.06725687536699229,0.05484496572674752,0.06914386380816197,0.06657579534938363,0.05364823654784785,0.06111314971938485,0.061922567720939965,0.06652046319057935,0.06875837554238812,0.06842929975737572,0.06676365347907337,0.0631541485842306,0.06411333058200114,0.06190855107611752,0.06404071186091151,0.05709515890319328,0.06709984177613815,0.0686833560979061,0.06062319714688325,0.055069488616918044,0.06922063857149496,0.052067559861331644,0.06368161657446672,0.06406512831279232,0.0695998201502459,0.06822909116503756,0.05528616739599568,0.07145288499909638,0.05965037649876002,0.0660151343446469,0.06541793798722934,0.060988215181215236,0.06361198317802173,0.056689979509920455,0.058528163899869035,0.0613337328335144,0.07019776270140873,0.06621923435735644,0.053711014673368286,0.06115471692131047,0.06672833022086137,0.06818365530452238,0.07026832610922662,0.07240306820009262,0.0708508674062452,0.044091228018018236,0.05721017897599212,0.06949887066714022,0.060984698067177406,0.06263316218465742,0.05666767286068753,0.0655814233806894,0.05113605260419251,0.06503264092034891,0.056697954565878506,0.06452445857975805,0.06072262009770677,0.06175049716038543,0.06896163394258892,0.06448742490912965,0.06633042217990984,0.06796022186705392,0.06370077736925245,0.06520183515876961,0.055632610220470044,0.06233543014758358,0.06650191048681309,0.06798150963941116,0.05556896711568308,0.06826500878589162,0.06559014024220683,0.05968096684349529,0.06821969982804485,0.06263977566407464,0.06840265082725142,0.06873227262928491,0.06478601977714607,0.06535735939567469,0.05988589247156474,0.055659399422386985,0.0662315396930537,0.0763857146193239,0.060018564764749135,0.058248638654974316,0.06933171659730908,0.06550893030413364,0.04996874750764077,0.06245131698782546,0.06239619040848609,0.06439261605185365,0.04680825394478995,0.06417686556175528,0.053582388875051706,0.06796546074584647,0.06885905024570661,0.06566957563806239,0.05899180553879514,0.06133857989211206,0.06994675282394022,0.06279655724885166,0.06508613542025847,0.05724778705268069,0.059688067837876446,0.06083454366746671,0.058812968868664946,0.05401232511902615,0.05978299320394875,0.061902670134262365,0.06747302815973036,0.06546640342358893,0.05447677214781627,0.04958779421453735,0.07019430750488939,0.056442806637609644,0.06557755750876224,0.06281986072123742,0.06432665461706183,0.06783290752714749,0.05894119624248399,0.056272426350851626,0.06337184626660398,0.059537955335672035,0.06042928833012021,0.06290961756136126,0.0670766406813156,0.0575857277602643,0.05953930073896905,0.05954924578160925,0.06841581314912742,0.058861767347173036,0.06280679708928819,0.05946633768734608,0.06900581164384263,0.07248958351448523,0.05874635306831866,0.06522083941293114,0.0643157394394053,0.060459147051448295,0.05742057147930703,0.0648450907703256,0.06333834938104185,0.06807989802364345,0.056875693383479864,0.07078218480544496,0.06196608489403115,0.05739889516078253,0.06668930249313995,0.06474111923814024,0.06658810841782155,0.06101324796621788,0.06842041894007268,0.063230652154349,0.0626189143110524,0.07268532105204137,0.05970098754428347,0.049798896655807195,0.05847784832239411,0.05735534037151935,0.056808468502518424,0.06042711351965008,0.06681507942078588,0.07258257663713685,0.06790734118510114,0.060789821363875826,0.06601359146249801,0.0684089544945233,0.06918895804533007,0.07290470541097507,0.07061478928570498,0.06967001500879852,0.06735152116904841,0.0599025715508638,0.059412697852098226,0.06425898906303723,0.06652204124203948,0.05977790771034314,0.0510015685569115,0.056245885731678213,0.052464773732061265,0.06418024023308545,0.04998591335399237,0.06617909766989508,0.06534233062937753,0.06450819953040897,0.07628672741702197,0.05885755333363022,0.06501330376578246,0.05954338740353955,0.06653572373070538,0.06466999009172084,0.07040191884417038,0.07153510854073844,0.05762325737893549,0.06216407203565924,0.053670060301943684,0.07323200048871452,0.05471558475280164,0.06645037139222507,0.06179538739183256,0.06687134645865472,0.07021114717742397,0.0682674430608893,0.06585313152240654,0.06316562921329301,0.07033482776748953,0.06645391918114522,0.06587620149965948,0.06211432123680878,0.062385552159836676,0.06406577002697299,0.06568459612114658,0.07241645791684079,0.0610871407073543,0.06801977871121323,0.07307100331635716,0.05996664417499543,0.060366036924748935,0.06461944409273684,0.06618995200353064,0.059498227405944654,0.06389612963170416,0.058537404155056594,0.07213064568052155,0.06446983983679289,0.0656816835681007,0.058598181155752924,0.06357699798584862,0.06466872929401919,0.06596079154096152,0.06435631177386761,0.05098678085006286,0.06726441789138873,0.05863650588056997,0.0671221156420247,0.05372353361097991,0.06568008603318126,0.06309932418650774,0.07146640038285801,0.0585878732281242,0.06677420423378214,0.05083349831861275,0.06522766373442584,0.06533640334438726,0.06264040422373754,0.06974823843935443,0.06667086383103092,0.050876246222130384,0.04596355522640394,0.06653044238997925,0.06431635530555438,0.05073162764440377,0.05566401572409484,0.06062505286405904,0.066020061659871,0.06754857142423137,0.0632756094582512,0.06666528841251496,0.06874482137029458,0.0656270248507543,0.05174866619184222,0.06137541553714244,0.06251189284589197,0.0522537432058747,0.07080127009245225,0.062465466326202715,0.05303496378190811,0.06208888561452982,0.06460408402107112,0.0679593089742507,0.06762336563023064,0.05082794640727026,0.06045555243024254,0.07019499058624909,0.060945446396876976,0.0581186699892364,0.057989661387670706,0.06394335895472075,0.06257132102130067,0.05779552639210443,0.06301074452406383,0.07130121143188992,0.06806189238802712,0.07455320323737122,0.07115925725160634,0.06551697238029103,0.06454196452633024,0.06209107402452707,0.06765092207225912,0.06397504886705127,0.06723051846262154,0.07127329384073454,0.06123793241733365,0.06028039735609385,0.06879322301984016,0.06271067949708983,0.06350193283337352,0.07021682677425517,0.061543911380535526,0.06973614282842136,0.0575761498485487,0.0651976427773305,0.05790368823766596,0.06743758009061998,0.06546226417229442,0.06001945826779126,0.06384071887027917,0.07320527296616305,0.054127128388941514,0.06005200411159198,0.06068977662273446,0.07261753319370537,0.07036661479477488,0.06394363473547127,0.06982936476526477,0.07180493901115992,0.07094930577977045,0.05967048581014063,0.06221105862674889,0.06434295208244811,0.06205003303764433,0.06942826805176781,0.058519272959357944,0.058515613632505126,0.06406685023763611,0.05913998157497619,0.05758339637064434,0.0549328705711556,0.05995176127303696,0.05905385386246784,0.05583564168175205,0.05791099100931023,0.05859029328211615,0.06262713351380278,0.06277488557526778,0.07356888534111347,0.059975620604324806,0.06230902510032867,0.06895476567893148,0.05387350182390965,0.05908264009274265,0.06386484401722481,0.06776320144198403,0.04752588635136415,0.055801976065510295,0.062303958708723614,0.06671071231152709,0.06861428535852292,0.06088039824345004,0.06407043146807058,0.06481320022981926,0.05762964071232339,0.06408237079650644,0.05924681159451031,0.06310309742068013,0.073506079156749,0.07063287438144727,0.06838025196567349,0.0674852066851788,0.06092000976272824,0.06540826911254224,0.05334088075451147,0.06239637736758434,0.05938630004366769,0.06999339849613323,0.06766099209538351,0.06023301557825049,0.06240788793351109,0.049031471054438044,0.05992383566073049,0.062226856045848794,0.05538013391719622,0.055438730628273876,0.05926295210354765,0.05914264021432745,0.07066849870858832,0.05953890291912091,0.07240597335963511,0.07350901837143331,0.05832972010836318,0.0704031862924682,0.07301799461118277,0.06635432879482386,0.062049019056983214,0.06392545920222528,0.0694444398540649,0.06965957135425216,0.05941874714898983,0.05495019279099409,0.06759667925122326,0.07227950531832891,0.06972810795239737,0.05425037338635384,0.0670822354882521,0.06869162005203053,0.061090873580899874,0.060818937049940805,0.07126103872098077,0.05842058955734118,0.05785569482761991,0.05859926648656794,0.06701406690503639,0.07139346774581493,0.0635093711650881,0.05419611242317049,0.06239716937472149,0.06514251729635967,0.0674541832070206,0.05978522077041719,0.06599057669320293,0.055043613192293284,0.06819872758288267,0.05242880781032079,0.05704800224182785,0.061920681460041926,0.06254084334743089,0.06942235748200545,0.06461141517363463,0.06360636339499362,0.06640840018800909,0.07097397500792926,0.057307877654545564,0.058068460556803043,0.05972186163338479,0.07237997896608386,0.06976245397940321,0.055675461654239075,0.06084885306167774,0.07459327361993229,0.06094208409081607,0.0643583291105225,0.06484905214751487,0.06976899907771719,0.05892661315590629,0.04869666593252339,0.06310546627113875,0.06977877394286908,0.06914425015058356,0.05341581782354378,0.07316910229661414,0.07092538052659256,0.06356334182927009,0.06090471443293832,0.05956109479122698,0.06373063025813516,0.06991733965207805,0.07222757608599042,0.061597180454467396,0.07090262178345767,0.05700098465076149,0.05765933780016319,0.07187461826425581,0.054217946979753635,0.06738033161784293,0.059400599645267796,0.07756206688527514,0.061233165520137106,0.06903623255141997,0.056873433214737534,0.05828054343176096,0.06198240065119034,0.05127924788073852,0.06663965672332386,0.07050951481271805,0.0570127645757497,0.053510045814743154,0.0708462672362653,0.06479352431689452,0.06501343809517363,0.0705796025711671,0.06781957058173325,0.05124718926147126,0.06341706894168161,0.07190138823465098,0.07208905629433565,0.0675984616832197,0.06200991974674044,0.06489589014919751,0.06389645568696158,0.06787578744440581,0.06348084794674104,0.06737500193669044,0.06538827507042633,0.05293214468399643,0.06064541586883049,0.04588647521790725,0.06256722301303194,0.06709152581234923,0.06313446530237445,0.05838951748581657,0.06365087510408655,0.058354155206282954,0.07120276409475665,0.06413274085212264,0.05270146663795028,0.06693148619237144,0.06829769393365931,0.05649657142005987,0.05761504960798151,0.0606100223775813,0.06127855931252471,0.07192797975050241,0.07195784647024459,0.06769190228158209,0.06525898537574115,0.0663310391059218,0.06620970653330857,0.062187834995855285,0.06721382027077027,0.06834758751803247,0.064163232261925,0.04871007626115294,0.06399887604602507,0.06007292512939463,0.06108061206697615,0.06835096811381391,0.06347480094070566,0.06753576888861187,0.04974403853241194,0.07115624323239983,0.06981412447343613,0.05888990031005435,0.06978271161172017,0.05701257073259512,0.06646454171976758,0.054805176813638695,0.05114418253007147,0.05799385389776922,0.0712609931874368,0.07253665356510003,0.06209034705647394,0.061581292613939374,0.06002760445894233,0.07415104557056706,0.06522419014257409,0.06068386511763374,0.05793950744734669,0.06103272951260078,0.06529331606719876,0.05973480245202885,0.06953727206575996,0.05650942162791421,0.06768398964754563,0.061442937628535324,0.06575389044899373,0.05948120928297669,0.06048205951136404,0.07344741174938477,0.07035723455494666,0.053750334004896747,0.06528715342425606,0.06356150059665147,0.05710730164271072,0.06846926442234927,0.06180106401658897,0.05898413105510296,0.060930757638544025,0.06547841134380339,0.06815810455722965,0.06825644790827728,0.057933630104098445,0.05019047557069185,0.06211098686732519,0.05401281877845185,0.06583997713350973,0.060874597729095495,0.059626995694054837,0.046038544577070455,0.06240823014475918,0.05398090587390948,0.06633627880517043,0.060567049798502345,0.053801787093332026,0.06734115429103599,0.06660956265568632,0.059305478745198745,0.06786520328101694,0.07542870823547351,0.06724961441574763,0.0658902056962902,0.06520334139850564,0.06333337411909157,0.06332653341672367,0.07132113323182745,0.06279055191244967,0.07042688130230965,0.059754519877476625,0.0678161211451128,0.059692309813661734,0.06337092995362245,0.06162024325588692,0.055864951751590265,0.05899889152574071,0.05001779018084478,0.06883221786347288,0.06391446836214904,0.05343503311831161,0.061017037827458195,0.0601340158572881,0.06917365664083594,0.07019429268933614,0.0683855713484862,0.06261070346030609,0.06423001762500395,0.04934631684347261,0.057947503057040856,0.055254726850337985,0.06732544142273564,0.060631090380968596,0.05845036254329129,0.0584686089851474,0.06166076903088399,0.05807694281597289,0.07066988205287794,0.06543803156603004,0.06062833991194061,0.06539604872025974,0.04973758828926889,0.06407971627857761,0.05706503077990694,0.06630745743711985,0.06455378128473009,0.06516421436153756,0.05141788670277201,0.053817299467252076,0.05534338346886495,0.06778482593158833,0.064988352645308,0.05661173919125116,0.06653854022818825,0.0748164592268159,0.06559798509568628,0.06574002382065598,0.05315993414561639,0.0707726440658345,0.0700143962869082,0.06583831355497605,0.05501136943583361,0.05380641183856983,0.06521217523876335,0.06171166763243887,0.058696913504760015,0.06191926300907572,0.07051821751232618,0.06118126155297999,0.056744336774713564,0.0766167066119979,0.0646879089414778,0.061404176964558806,0.06384087133678494,0.07079172775733028,0.05307927342405823,0.07604337874367231,0.05341980229595607,0.06930596407372713,0.06430549417378988,0.07248279345211847,0.07159279639951266,0.06383705505401727,0.06864404839785354,0.06001380771982335,0.06383229717971657,0.05592745653818089,0.07025400190199613,0.06418262962896915,0.06442553149267455,0.06422269246035098,0.06442624985535,0.06557403687128399,0.06485639208663332,0.0707109682742744,0.05088155595144927,0.05891850584999747,0.06052361748896557,0.048876349044958065,0.07091589064029874,0.06984505206646896,0.051594426129406276,0.05657522699696427,0.06110440756917346,0.06520314928807877,0.06032073310484804,0.04999947403742941,0.06781126320616712,0.06509575970975076,0.06623619381243717,0.060840690636396036,0.06779424323289014,0.060014227000992605,0.05499486472631983,0.0632990010302123,0.06264366208720448,0.06209029857045186,0.07051271370944187,0.06118352351366532,0.06134391705677637,0.06236450644191156,0.0704535788892861,0.05719096123540286,0.06729371689405321,0.06733567615236549,0.06235919459287045,0.060550940537780956,0.06279572905552444,0.06145294120769839,0.06761888729124568,0.06373062735600564,0.0569104605111229,0.06891742265625925,0.07037296432656996,0.057408661289074014,0.07574724971110913,0.06297660527194268,0.05897550059893104,0.06822554405444303,0.06541233656772055,0.06343909477008643,0.06923504285887427,0.07050514465608258,0.06016386237170318,0.06059830637340367,0.0708513440720048,0.06890092316463267,0.06501305237760782,0.07368685989071593,0.06472383713058252,0.06325370322516957,0.07090401474432007,0.06962894681125459,0.06185353010731689,0.06727254126222182,0.06708807799219457,0.05961303707349025,0.054435112025563476,0.06532001606422,0.05992535963799246,0.06882497916440067,0.06511713019355962,0.0707138042778341,0.06277375770860327,0.06568255913833002,0.04707681840340147,0.07533108207684655,0.06676550479225263,0.0585570918042226,0.05745305798920697,0.056668660535409955,0.07337685165785618,0.07196471352486492,0.06147601113214784,0.0600823685434137,0.06214317233002977,0.06688431287001301,0.06503716388412867,0.0727450208478036,0.07007430646539686,0.07313277502482277,0.07128772350543924,0.062474159535505995,0.06706849413456697,0.06165605336204297,0.05918370588625166,0.06275883934732912,0.06652111203086569,0.0677965551876934,0.06157318065684218,0.05275812780388328,0.062313969754184684,0.06852490833792013,0.06318236196153015,0.058289216338023184,0.05233413344960945,0.0602387962725834,0.07041426955102029,0.06510491462984883,0.06765804308860329,0.0657386921541249,0.06976076828290104,0.06827195163086644,0.06421107189496816,0.05619843893315421,0.06969767111206107,0.06410019616735912,0.06492003782309363,0.06725648573773291,0.06927569953050453,0.06708593922105492,0.05568884633751274,0.05824531288039943,0.06899866435994068,0.05692848845211154,0.06674952441002634,0.06428046970592755,0.06499847709452539,0.06682113931724154,0.05942862423605537,0.06414323461814833,0.06762562769176622,0.06743661207051413,0.06565746368237094,0.05395761637833353,0.06294100198270504,0.06793824910380447,0.05768293702633502,0.05764681893929623,0.06339998280909381,0.06287769811421007,0.06908183048022044,0.06247014143201219,0.05799895476430335,0.05763855234148946,0.04706433762200364,0.06427657197115466,0.06029546109483344,0.06555424668470358,0.055412853488991415,0.06637152490139028,0.06084915321631415,0.06775732925048744,0.05583797057629894,0.07074102266451765,0.07273351712326875,0.05663819200000109,0.06157781893231131,0.06486036273353882,0.06682290723470008,0.06713752955034293,0.07097585492610747,0.06727226190470156,0.06716753101812117,0.06553574842797094,0.06767524600290098,0.05452303607098522,0.0654637522169986,0.06014244893563547,0.06489537411073987,0.06010292236364768,0.0627855239398722,0.07144954116255245,0.060118174432293114,0.051828279020599104,0.05904555308272408,0.061854754921561485,0.05531745483619334,0.06766596891381584,0.0639521344564668,0.06230502685307932,0.060113482840609675,0.07217064108304028,0.05945342440644953,0.0667153848839715,0.06670787246503361,0.0729789168923391,0.055598611776883536,0.05952951302351242,0.061847839756887964,0.0523775442854614,0.06274785970050797,0.054613754616741034,0.058667257198507045,0.06580510584484,0.06536763022376443,0.06512238916466725,0.06422356419765396,0.06779062360033772,0.06001807569578234,0.06505249879927535,0.06879853034737454,0.0667858171240228,0.07044782171360425,0.057695121148324216,0.06607394524259194,0.06363373261013555,0.05498866140453443,0.06616960171127466,0.06716779024761864,0.05442475636847263,0.05805866031140634,0.06970016723842873,0.0581491957062904,0.057891121711981144,0.05985739925778477,0.07295149171911186,0.07249293577113658,0.0637175384318864,0.07153518214723609,0.060001871277357954,0.05875640503716337,0.06433738190091427,0.07138388418575425,0.06149571494085261,0.058439201047781386,0.05151490729428271,0.05939771407083652,0.05680020169934839,0.0730718373074205,0.0728751225731665,0.06936798266916742,0.0676925451283101,0.05806913020810009,0.05665408817608085,0.07432583495332187,0.0732801990374305,0.06585963972106093,0.06375065843720948,0.0669099636528849,0.05353845873695799,0.05983667227199489,0.055256243201096025,0.051746126216346354,0.06543241261642968,0.056144719888434905,0.0644340710172793,0.06559307401957909,0.04479985078819883,0.04674298851659164,0.05927718631107634,0.06572886201363087,0.06648480082113156,0.0622354382987243,0.06922042483261756,0.06082065939946727,0.0661654645733039,0.06710826169921594,0.056417612136696674,0.058868354040402346,0.06762273652756999,0.06331417542820164,0.06606277638285565,0.061740849766376366,0.05292946853624744,0.050402316086681435,0.06351806888226533,0.07177979765944605,0.053069500150482216,0.06549146817048308,0.05986963054349205,0.05945505519839166,0.0544216577499427,0.06429461616249171,0.0671300781888978,0.06031233859160101,0.04994507555084143,0.06565251891921572,0.05906932625746723,0.06393413329498328,0.06155513283566882,0.05486012822080616,0.056233406803398875,0.06108098958685486,0.06459389773344301,0.05457819087220445,0.06226073381056133,0.056506159527780805,0.06782616881075049,0.06637773369186838,0.061570444585876204,0.06934132047391234,0.05802136709371192,0.052514079137861144,0.056452543513808295,0.06627584120989582,0.06319322912336289,0.060242290606530316,0.061619079821787714,0.06555973144160379,0.06489938738674145,0.05493435429719447,0.06278120827898127,0.05886300602489892,0.06966944351995635,0.06007603595420234,0.06610172372907513,0.06477809977817932,0.07248027610244495,0.060255530514096274,0.05430578334419364,0.07518191910276364,0.06084283859623844,0.06262329688460846,0.06553268766429667,0.060697968163722665,0.06611379110125011,0.06394792086544034,0.0663623606364997,0.06569093174431789,0.07368585329630414,0.06994711808075144,0.07179354481834962,0.055179325029282814,0.07071060334406377,0.06523897983852825,0.0709171433855986,0.06808989596759503,0.06054658270150182,0.07191380090380063,0.07201491427085771,0.07139010404101939,0.06852630264999772,0.06623249441959148,0.051970150498109574,0.06283896899475577,0.06546129592990874,0.060342958726146104,0.07087868373579526,0.05093667584977536,0.06052627735806616,0.05892614697912596,0.05772118823087258,0.06780539586987344,0.07095151019193655,0.053359108431179886,0.05837576867740754,0.06727502485586091,0.06495612311981092,0.07387569942499572,0.04954274452089735,0.06812547878781933,0.05970879040624388,0.062018342103976346,0.06882853289829197,0.07101771505188015,0.0618771903714884,0.06262792463999568,0.07187772548149662,0.06334124163925611,0.06523456230484934,0.06289885931549963,0.07237066698669015,0.06030891080875221,0.05927004039344809,0.07110600024510905,0.06508202435753549,0.052176651705360955,0.06329635375301684,0.06716682856761455,0.06301299167213473,0.05515869541863461,0.06428224260025787,0.07099663636172897,0.05757563792672542,0.061337668016686324,0.04823109731693118,0.06907450438151594,0.06382830338233844,0.0616551755486974,0.07161025120988164,0.06620397920125319,0.05994450680713975,0.07035472066759806,0.06423888139877187,0.06818475591238132,0.07325334028582925,0.05403013912846048,0.06628788157290935,0.07093495641235323,0.07159582904497305,0.06741814391406353,0.068321181124207,0.053602925656318315,0.06855032925270052,0.06992632339121321,0.058211824426499345,0.0701359957338121,0.06001083253629789,0.07005359204417588,0.06042205839770457,0.056729726232910574,0.07271494023033322,0.06371783747886903,0.058423115079564894,0.06325210367616582,0.056945468193714,0.06300427358257057,0.069898568774824,0.0650699728081878,0.06393037148553277,0.0577548422452559,0.07388413837094732,0.07095283631122296,0.05662371813591173,0.06030654446945105,0.06679434578680327,0.06599772462145653,0.06774651606190404,0.05733853654077005,0.06723422458913926,0.06947683417062409,0.06231481955486319,0.056086259522524134,0.050286710051086425,0.07176008879941047,0.05452001137377051,0.07164828222736674,0.05509459008941532,0.06558280592056578,0.05461098623588223,0.07023905687000226,0.06077067295643783,0.06242523525922417,0.06197115274732439,0.0671286799713141,0.06329102528342545,0.06430361351036626,0.062024285185028526,0.059308081453843434,0.06027680777156634,0.0654246329691739,0.06934706230409926,0.0599962451650702,0.060055221116944724,0.052297801423517046,0.06346013864744907,0.06939079842074189,0.05376983524726709,0.058227485990498574,0.06855466845889392,0.06511256520132656,0.05064925290840326,0.06669352115130356,0.05610837387357388,0.062323493442229476,0.0676318454200448,0.052840490412428415,0.06149377686978055,0.051690819146261835,0.04867594526355913,0.0673128514829794,0.06839795546651972,0.06524736117938801,0.06126272472878017,0.07366585831853725,0.06906933126044401,0.06310842477960053,0.06666548145102064,0.060386595129695866,0.06306177066815738,0.05960505267431519,0.06666523207664632,0.06919637651804371,0.06448854815869182,0.07442289201688206,0.058278798265890275,0.05973220203706868,0.05746052908832386,0.06375338240843526,0.07177055407599783,0.05449833259352911,0.06636855606358441,0.0665032551615625,0.06667685638225124,0.06114603325942822,0.06872717759156054,0.06918757726836175,0.06940376962261233,0.06485716955042362,0.06385444711315871,0.05918803802904146,0.069762306090012,0.06825727760311281,0.07251786440121188,0.06242384738086526,0.06987764644613204,0.0653835920761245,0.07214910986535429,0.06047474302470231,0.06748965420328015,0.06253010150298832,0.06553120466471249,0.06498869901558947,0.07384409737920156,0.06703322867409428,0.06958020247611466,0.05821381757845644,0.0737532296949357,0.05881053417397143,0.0653470551314935,0.0661345408588484,0.0685595485189921,0.04209409371729231,0.07026528076232703,0.053858343739836195,0.06707940687885344,0.06620061154452325,0.06830308109649247,0.06416585241268044,0.06398907701307335,0.06762373363896644,0.052767399664831634,0.05297693280985494,0.06778168067475346,0.06405903718098263,0.05987974871892971,0.06739534129855683,0.05472426714250973,0.0664614285087672,0.06508873527447659,0.06740972178007001,0.06508907616680645,0.0626480364629545,0.06997003343659591,0.0668780090416925,0.05292152347803823,0.06158368938933913,0.05155488378887363,0.06791441607609697,0.05842217146472515,0.05663794824128744,0.06830711624794542,0.06523409553381779,0.06710648072337363,0.06363170278844017,0.06946662601399248,0.062448667419914596,0.06324369733653581,0.054535125887697905,0.0625894836051572,0.06695684568525684,0.06822515282205929,0.06880008929317483,0.06577571099118305,0.06525931571951138,0.06509822013874417,0.05498063439201978,0.06232788199340928,0.07421650753782236,0.061118392133253166,0.05776236506076661,0.07074512044586881,0.06360496898221421,0.06517315018036526,0.060630607240137746,0.06336288599919282,0.06776315574041672,0.058627323845989995,0.06685924559795267,0.07219318716654131,0.07006715774589042,0.059507554559755156,0.05786042037207266,0.05127087683166871,0.06293418439317591,0.062255969007664724,0.052806366516147105,0.05145377693614128,0.07336670147151941,0.0627470415987284,0.06955499442675239,0.06778352321462011,0.06928010281394048,0.06789122594878842,0.05534147605008497,0.06361304780611055,0.052578752408415225,0.05483781250283194,0.0540963041495739,0.06503852554518169,0.05649037002056157,0.05806288873695567,0.06220122662852975,0.057508106964326136,0.07346496017380588,0.06860103561994886,0.059917683054483625,0.06453132897490291,0.06683422425257463,0.05527685833489751,0.059192516225034464,0.05812774590212415,0.06762084852430952,0.06990035295356326,0.07050065776292871,0.06212329117541809,0.053504948738323885,0.0711947446231899,0.06258383624377191,0.07109637827340559,0.06792755212790963,0.06453067452162399,0.06307914276756993,0.07228229383657771,0.06458722411922932,0.05254799989089606,0.06348594921362995,0.06696363774195037,0.04911388981771372,0.05599232065465237,0.061789789329635685,0.06266198139874685,0.05429159576979534,0.06345112657936461,0.06997757223156689,0.05725082209824886,0.0535928281688616,0.06232863716152576,0.0712124789603501,0.07259589185024617,0.06743823524941128,0.05980304634501888,0.06632959211727439,0.06849799702082601,0.07183229943619662,0.06409714739271842,0.06552565594319701,0.05612205451423999,0.07692929652580888,0.06024640011793229,0.07022871137790355,0.0690979570093626,0.07397306114805915,0.06053790741655135,0.0655761676406234,0.06421114630929972,0.07466786317554733,0.04663558681309876,0.05584763974577761,0.056086192990530165,0.06189828858851553,0.06610306890045678,0.0683626614605213,0.061110680993547675,0.05849052891520882,0.062383585301616244,0.05461245983682535,0.06791593299833354,0.06036408452598331,0.07043148937098677,0.05072548688605438,0.0587295957804906,0.07419804714660128,0.060802488525387985,0.06242823576633792,0.05608335223337248,0.07224634102058361,0.06984476865845789,0.06782265778158081,0.06388046635045216,0.07074682037743207,0.07328806477865366,0.06991941178193291,0.07215750629068277,0.06540487853115012,0.07280524378824219,0.04788992594910533,0.0650546150194117,0.062338628238315506,0.06879607574877614,0.059026049446224006,0.06493130422909887,0.0611355630718229,0.05969918956368561,0.06813742997103496,0.060754640119048076,0.06944611907545675,0.06670071295890695,0.06020906569880603,0.06256808502414743,0.06692607293310374,0.0597244917283421,0.0659636691960967,0.06458104289048303,0.0662560572208355,0.0655273925514676,0.057231222272343854,0.06494100347524426,0.05987347568709512,0.06954007955098943,0.07042589035361249,0.06960916929261726,0.06325791597333552,0.06985585970536504,0.061343519456477764,0.06569303914975705,0.05910189092366977,0.0575675057624692,0.0639858935836298,0.06340166380021989,0.05749986027849234,0.06192729593178762,0.06569937599896819,0.060996638726012714,0.06361387970090407,0.06056763430882191,0.06321852415036272,0.06714779868254131,0.055762262090206016,0.05544054220703108,0.06852260218507367,0.05899764095304785,0.05771466759557772,0.06145568455969921,0.06419621629420938,0.06743624112687366,0.06258662098236802,0.07098034828507041,0.07375132898463695,0.06257407765745929,0.061799138337207786,0.05496087766260804,0.058375060587548745,0.056884903469313075,0.06131147994124747,0.05425503086987321,0.07216131538391127,0.07393001076146578,0.0656098110794783,0.06767246999566784,0.06704938556266131,0.05733328441014658,0.07129879228822522,0.05648882875400553,0.07272363930654446,0.06604901033103908,0.05423578334768918,0.059473537199336304,0.07659521612429337,0.057920391281820485,0.05540059557327269,0.06648049576894359,0.06387832100802143,0.06750468856190084,0.07240441523890749,0.062498778600200346,0.0585664593873318,0.057996533117745136,0.06399893302790921,0.0671646413340618,0.05554753479660983,0.07369269110812046,0.06670421306899464,0.0686718846788488,0.056379379119165754,0.07188659721611977,0.06872013305475622,0.051005087048415415,0.058864716851561805,0.0706016356875515,0.06671073304486969,0.06618308908992263,0.07179303599551567,0.06146366893177776,0.07308446352174383,0.06574255584734087,0.06544819049123599,0.06668464259507105,0.07253011339863806,0.0614123189287836,0.07249398433273249,0.0643096251588178,0.06994183806727722,0.055857624401849416,0.062321849498600895,0.06716237345065734,0.07213760755625252,0.0611795969203425,0.0632709411047845,0.058769081446513695,0.061164932623927123,0.06477221871210764,0.0707667686413197,0.060633766624383745,0.06270233522149478,0.05545924969710292,0.0644098239399315,0.07339408453347684,0.0623875918074368,0.062403720195839826,0.05796894540010389,0.05920644988489593,0.0719206954318176,0.06512233570299857,0.06367937589140785,0.06926947430148864,0.061388157562882066,0.07034908553257072,0.07553847897993354,0.055158391824538536,0.05399030311684813,0.06665767553215858,0.07602935532257071,0.0650289452743203,0.06667580965939779,0.06952891570936856,0.05792993929799753,0.06255101944243181,0.073108040815699,0.062289422451063566,0.07410353992404696,0.0656383133782657,0.05842530108728143,0.07435966196636426,0.06684713942710069,0.051706624360533776,0.06719610819566407,0.054194840359110805,0.06170763165747733,0.07080030544301404,0.061684904273526166,0.07243207920864964,0.060095685723814546,0.061596580059346165,0.06182337155693703,0.06215581462599668,0.06929745136417542,0.06278077587921674,0.06106441300379241,0.05513280445434127,0.07167031930669016,0.0618852758592757,0.060750375626151265,0.059737533368058954,0.05689597821151701,0.059008570754236614,0.0784035384421056,0.05444303361786069,0.06099876796331255,0.056461656318317066,0.07239355725525046,0.05460302826919833,0.06366699427908977,0.06088176778131227,0.06140285288130012,0.060218531604734296,0.06843344710916702,0.049773962089062596,0.0678171480033386,0.062139167977756864,0.06338942253132496,0.05785501887275167,0.0673746232146446,0.06645855204856371,0.07046630098541656,0.06382005583506017,0.07165918404833779,0.07139782094679364,0.07010364529842994,0.06420646871950408,0.052957381496394965,0.06674169944384777,0.05579737659842958,0.06497058558760173,0.05992890623026234,0.06781403170324118,0.07233310899351952,0.06279560380469315,0.0645208583395637,0.05558994954258087,0.06045541048045937,0.06501117686486299,0.05386577495032522,0.06447063724255281,0.06191964048280696,0.06935322174429889,0.06395345234145308,0.05863872117987638,0.0491211562473874,0.056915385354842564,0.06079230516009293,0.06661118808497699,0.058872095089122944,0.06757745409522126,0.06077733882414866,0.061051740541397725,0.061625685565943576,0.06698347681072325,0.07020494289087356,0.06634236042720436,0.0597448825027351,0.07084523538954357,0.05656253030329764,0.06842306427711033,0.051087944826856135,0.06768330627619745,0.07107099269850423,0.059834002443997156,0.058560821746215394,0.0668330697744314,0.06301801593251252,0.06494051206631725,0.06883378752422278,0.05796678805050012,0.06703831743444752,0.05613107484718211,0.06232800199803775,0.052301781161197505,0.05774863027531055,0.05537245300513151,0.06437423060621265,0.059337209236462185,0.07246369525807272,0.06942859372492792,0.06198227999966329,0.06980827724675379,0.05059344025000438,0.06191939121010834,0.06585022108089136,0.06447674378681795,0.06008828224273002,0.06905094548748376,0.05048460355695583,0.07441305809203891,0.05414411770605236,0.06662986415973118,0.05911839969199552,0.05692837500013706,0.06518487068041455,0.06446101085171217,0.06620946627665378,0.06664489727513295,0.05835547180406252,0.048585729395487276,0.0607435485389461,0.06124855211304832,0.06804465794288223,0.06877257975472431,0.06370438273041515,0.05501011199684645,0.058374276987988014,0.049952496157556966,0.06401352293310611,0.05892619178268621,0.05300027362608414,0.060874741529799006,0.06376410798234887,0.06542570688865645,0.05967799434540836,0.055274251718728934,0.06490523702501039,0.06805754978965042,0.06843443448882769,0.073616411082431,0.06495873664418315,0.06659001705651657,0.06563599055455147,0.06305496806881503,0.0580575475540495,0.055488060748559886,0.06649931779666998,0.056957667472999005,0.06686237590133237,0.0604865627284112,0.057117324061952274,0.0605395235323985,0.07050996572332707,0.059519306886303,0.067336221391366,0.05587728805885273,0.062368733994983154,0.05965405295776236,0.056328150449000955,0.06985646788203603,0.054568189307893,0.06945501114475887,0.06125169131668554,0.07097825638186264,0.061161541965058214,0.06523829876434178,0.05685349758827564,0.057253079740133245,0.06732443497356147,0.06084679673181918,0.07146074536297473,0.058545252700295855,0.06228979293772536,0.0659869867934138,0.05513556275608193,0.07249971071361938,0.07514936143136713,0.06039378827447714,0.061721912665564795,0.06062656508474085,0.07126295556370704,0.046384015936490455,0.06251970432509976,0.0685574008394923,0.07479972238987287,0.07098349910817801,0.06860247116646515,0.07097130798842993,0.07395179964407156,0.061760735737558414,0.06648623623262723,0.06142087273298826,0.062499763534027974,0.06780194008737328,0.06737465705012173,0.06157932278277761,0.071262557544016,0.06814979538358726,0.07316780720054294,0.054789848220796473,0.055017906587492935,0.05980506667910063,0.0689838182466595,0.0592714584693384,0.07354349507428123,0.056728827053334804,0.07073327530287019,0.06440704788007542,0.06710195985638881,0.06399602221574621,0.06903363015515901,0.061883673539896406,0.06671983458258052,0.07347461825997274,0.07335269519119259,0.07168080788994163,0.0581940383190665,0.06627873966694653,0.06215336077604804,0.061941763349391214,0.0570122734180792,0.07013241950219545,0.07513991321868813,0.05775944159743848,0.05465083088343052,0.05729000575597063,0.06355876822695132,0.05632683602732649,0.05865269799002096,0.0594345054824213,0.061207267166246256,0.05708587857457183,0.07239915779095274,0.0673920747762272,0.06533733366274955,0.061114570697545084,0.0726423842217149,0.06342828228207972,0.06618928790058784,0.06843258174554656,0.05919001787944343,0.05942732050513278,0.05431809894316152,0.06635693176394995,0.06505715628038014,0.06322333355022733,0.0644022062646877,0.07116639529438501,0.06168275713776512,0.0594756912891217,0.06403746238066113,0.06403651426996848,0.07014332981175792,0.06719219514876684,0.06738027843532665,0.06414147231484126,0.06335837856067282,0.06152601014450058,0.06062685293941348,0.06450494779928526,0.06528551601931774,0.048873963234050084,0.06756162165193119,0.07206572657779266,0.06272108210194845,0.0479101916007707,0.06800682460032925,0.06416843534174185,0.06574897443845487,0.06522497309425361,0.06475506695666783,0.0681725361792668,0.06062405659860201,0.06423836157749876,0.05784909657967001,0.07303226452718525,0.06200311729381104,0.05223918984197025,0.05981094312782961,0.06244027821642659,0.07098123063167125,0.05625642679638217,0.07249876093504798,0.07250453884190464,0.0630932763782435,0.0700074236628872,0.06705184888236859,0.06730782431298943,0.059363335382170465,0.055802483776800395,0.07036295045290156,0.06545136238388667,0.061355484048940985,0.06762892117702042,0.06730631926026137,0.06741356388569522,0.06500624460555765,0.06516303245794115,0.06663617331121094,0.057800246113941106,0.07502332959641783,0.04923257750903917,0.06892645672690495,0.06060102609345206,0.06773957684080519,0.07121414621373606,0.06694190936312369,0.0684816220617768,0.06535674216656788,0.057637467826940486,0.05130946504099229,0.07121615564732522,0.055967487053004916,0.07183180478515705,0.07209785726240137,0.06623335758319096,0.05639705222219461,0.061667830961640004,0.06432847164721638,0.0653058079546722,0.05022678858011913,0.061794515749810756,0.057497767011931857,0.0588997828434262,0.06873873457517907,0.05255371804245352,0.05813601802410602,0.06720807564248993,0.06857952058794646,0.06619408875234012,0.05957527011644151,0.05985060060627091,0.06000478637928373,0.06169566543613024,0.0721872471779538,0.05846037976444313,0.07468311188021462,0.06584511826202252,0.06839204308502467,0.05985489083819043,0.06909741816951044,0.07216006128605339,0.05481683078551042,0.06951440825583752,0.06839808276688931,0.06533176986535282,0.07169825254371855,0.06187946725257505,0.07259805720242848,0.06740794672460867,0.0676656346311995,0.06538927574573346,0.0669602618684342,0.0638644434388039,0.04873127805743721,0.051416616857803306,0.060823073655554685,0.06949328672097621,0.06913354840897581,0.07251160023890471,0.06442843674247847,0.07262403734386319,0.06060671103376778,0.06933163399709569,0.04798297074110018,0.04745516517996258,0.06543889955994028,0.0651801304480195,0.06261543703384895,0.054406827412645006,0.05923498233293479,0.06281406384446975,0.061213532505699726,0.06464251114161122,0.0684985133010686,0.06459436313105675,0.06512229713489658,0.06309948777226299,0.05607823953114003,0.060132254148699646,0.06010469255627629,0.059578808222206375,0.060325273264174015,0.06974908281760664,0.07075532201367699,0.06851101420329389,0.056348179909004065,0.0619177465705077,0.05549348932152407,0.06426255519826293,0.06625315719780496,0.06547913557484913,0.07034976900447856,0.05876636362041727,0.06136696441668033,0.06714841353665899,0.07072475871253361,0.056541657490183674,0.05115457745016372,0.05619185421041309,0.060597312954883686,0.059880974709029436,0.06834763801774897,0.05768290435909143,0.06045670290920083,0.06395921366164395,0.05802012220872857,0.06721293952210138,0.0661312584806589,0.06495971214200369,0.05282130120236865,0.06153198196142589,0.07129071197827697,0.05606209080046194,0.07559170709436533,0.05795927623930669,0.0670038519079865,0.06186916737763781,0.07313302238290598,0.06999771652777304,0.0697866351203332,0.06653248416638835,0.06964883344152148,0.0702161714538835,0.062154569617110626,0.060145275254347215,0.06941902507960396,0.06820949421469631,0.060916585527281145,0.06202345753623364,0.0650263148445889,0.06834350287479601,0.0664199639766728,0.0589635264937655,0.06326335383574375,0.06793409132728906,0.06851297771582725,0.07353059590550509,0.07289139421206159,0.07267776121358409,0.06606936361967845,0.06516801477119252,0.07093403481067795,0.0647628353201298,0.06009779133598708,0.052221534900056335,0.06899229285854325,0.0646238844025604,0.05716430853555488,0.0731046050869171,0.06250359765754039,0.06560916700173686,0.07445849858865941,0.060264849772469056,0.05250711311067639,0.05271563688822618,0.048686738562893773,0.0706879345189809,0.06433504132207254,0.05255760221033655,0.07277726920957184,0.058208801960913936,0.060087956673121876,0.06276438250599287,0.06503075499788769,0.06171912322171059,0.0653515351184776,0.06838087766478021,0.06618864303020053,0.06026041162648669,0.06522098294376669,0.05956053802712961,0.06976534501701992,0.07065475440525883,0.07251507329940657,0.05952614901561035,0.056088948901399885,0.060017745384238,0.06496530678398668,0.05400750914694086,0.06452073916209883,0.06840797639343052,0.07180399653666461,0.07333900646755004,0.051492254714901584,0.0671366011726493,0.06536793475193153,0.06925019112833525,0.058613673678815495,0.07002217242619382,0.057941186683875975,0.05426783452575211,0.07020250118964749,0.07127153895917972,0.06320075948417364,0.07069319156683715,0.06932129206486778,0.04987050736455833,0.056271374920160835,0.06064207172390196,0.06836213092901136,0.06357523342655924,0.06076373288715044,0.05147469562961629,0.05270463277035483,0.05823844361378708,0.07015062835938271,0.05830076793697842,0.07443963736630249,0.066870115166628,0.04742650728276762,0.06580922827126215,0.07203922631197704,0.04562093624449356,0.07105616731870101,0.06575350386695106,0.06699957380264697,0.06340967432784471,0.06087967276179014,0.06044136386250146,0.06678927654538036,0.06844441942871017,0.050901359967499546,0.058332403945096395,0.06797829930775673,0.06952218610057252,0.05758595455456514,0.07290028544830121,0.05841369572661021,0.06460778747014553,0.06392667286538666,0.06743721728385794,0.06924724382604533,0.06895409778224229,0.05156978837443352,0.06215942294405657,0.06696933667770463,0.05655880991049972,0.07111597734477194,0.056046932260649204,0.06421523085675414,0.05634249787124569,0.06226198508266315,0.06122002359790717,0.06383473509899454,0.05962854278004026,0.07298864808733246,0.0662586794170947,0.0643040473117586,0.07106732721865833,0.05547212781621742,0.0647412839077377,0.055467423480275876,0.0712047664454517,0.057702796845705714,0.0684970241986281,0.058927996541133176,0.06799108916135936,0.07071236040960574,0.07082255724908312,0.06944107314319214,0.06697171102650357,0.057385584841083914,0.05351912812726503,0.07004347075283252,0.06853166540046013,0.06363098217716176,0.06699094374932063,0.06795032516715682,0.06197330913625024,0.062074291222642686,0.06732537891248916,0.06374312778931626,0.06899807123534511,0.06050933948887996,0.06915253487574907,0.06413969411567434,0.06935645646078742,0.07265770845682995,0.05584043125118256,0.06008955863417101,0.05575664327117098,0.06368565263694326,0.06288058334270633,0.06134895680751884,0.0654145229878675,0.06463749099729835,0.05875596784767426,0.04989765785731441,0.06180268339066869,0.06453207977399084,0.06621815589251012,0.06680229919022966,0.060302251934750535,0.0629259460229826,0.0619351330571653,0.0558679139798897,0.06243331715297239,0.048463173447589006,0.05650000122997938,0.061922758937002366,0.04963853056333235,0.05702915788192796,0.06449824007606896,0.060930732454618025,0.06623265263499374,0.04528638692508424,0.06636937541199792,0.05676852747814658,0.05273519003765456,0.07288833598573878,0.06813255441344657,0.05649027952090188,0.05595735486481188,0.06703811587181449,0.06316051170014186,0.07257142748694997,0.06640167004258057,0.059058730841719406,0.05557047731848894,0.06809578793847153,0.06692023908789337,0.0632975293062463,0.06994803796196089,0.05818798436452181,0.06693845863704789,0.06099735160332892,0.061668106423798504,0.06504757226147453,0.07285898567665665,0.07257335313823929,0.06327146518811794,0.07062346194589189,0.06837945783520913,0.05703245443009539,0.053618480491637144,0.05184170108287714,0.07390367895917196,0.06265045881015022,0.06395029715918038,0.06721198597922556,0.059061134404787,0.05809121228732174,0.0646771254458399,0.06709041420734815,0.05445814794610124,0.05592254227057199,0.06176677548927136,0.05704219880560161,0.05566140492816979,0.06484412926858771,0.05398227204692653,0.06464209589647521,0.07254552986335713,0.0608463890323724,0.061970592103853216,0.0661604200129223,0.05232519824125453,0.056919474719183266,0.06209736349638514,0.07306552184863109,0.07154479726284771,0.05399782551783608,0.061787115291688365,0.058218880807528184,0.06436681275398103,0.06668047388616669,0.06115650516448838,0.06318285042675342,0.068260271055289,0.05928490946751851,0.06281216137867279,0.07269556995252408,0.062443794638930496,0.06673067069261683,0.06966739357212301,0.0638214719964024,0.06079344820830353,0.06618483450691784,0.06118061684737308,0.06817796697328951,0.06865265139291427,0.070575337399086,0.06730160056971339,0.05591268488129422,0.06495782455374613,0.06866230900301978,0.05917701187801897,0.06958976475167673,0.06552822006166215,0.07369706564608743,0.055959907511670084,0.06720332122325093,0.06931820565909985,0.06470130651944811,0.061728422991425363,0.06214625868740003,0.07391477221345219,0.05534902472172044,0.05407555166766709,0.05890760039530722,0.06051345266070829,0.06347255319925849,0.056715312927727496,0.06376595787667308,0.06318069983715313,0.04931176623217824,0.06253889073430668,0.06537046774173866,0.055139361557660804,0.061100164180337486,0.07336040770865118,0.05968266800088601,0.06801369839970878,0.05818746299595504,0.0670116927960368,0.06766777258576225,0.06474384470845232,0.06835394072714099,0.07155557735008303,0.06535133217973052,0.05964689988051834,0.0559822444382438,0.07241700484545285,0.06114666753750918,0.06531680507380963,0.06352997561592683,0.05865609428028981,0.06947287944340806,0.05296815158870312,0.07412702345076005,0.06419748200203304,0.0736148191594673,0.06917316604905896,0.058556233355141526,0.057599016338170314,0.06987560690067035,0.0685687981058174,0.05400020223916145,0.05767717559671172,0.06956226150450298,0.05947971519157445,0.06115950836506008,0.0640066668287338,0.07161091485516019,0.07255288042254668,0.06724976656071019,0.06749787852382128,0.06100079285427793,0.049487988548944165,0.055125653849862025,0.06127891342985647,0.058317335692554234,0.05428472950979865,0.053473959383027056,0.07344262735236787,0.04854965882523767,0.06152053311288013,0.06576139448001564,0.06895991875670524,0.06318568887354328,0.06043232284070871,0.05594132036692654,0.059980684072330676,0.06913575941848799,0.07238682974624505,0.07159441419903698,0.06811165723708926,0.05906216714796694,0.0536460746859384,0.07212174255992325,0.06658454130959487,0.06361510800493211,0.06965389653195642,0.058082388071277025,0.05453977978399662,0.05575064842279309,0.058949929248671304,0.0703933111332051,0.06727930549980696,0.05258167657364482,0.06889354957138916,0.06461716014050012,0.061945691930045815,0.053717675715335506,0.06924556826440051,0.061490621551742175,0.07009816712831024,0.06808273684969562,0.0724938642264511,0.06734532759014161,0.060466539167874936,0.061626466781883736,0.07035671561191845,0.04589261589650917,0.048608147655946704,0.06319518536569713,0.0570229968294114,0.051781721659483436,0.060227031454409316,0.056721476641598684,0.06169917687008889,0.06777431284735777,0.06372166936755998,0.06980248372027784,0.05844799684586077,0.060284143504727106,0.05993667183923329,0.07534399056824909,0.05179091523718261,0.054038189512955984,0.06451587270519224,0.07604735514288838,0.06936857177441844,0.06768827659486146,0.06626694395852445,0.06291349040941303,0.06712862205304575,0.06384197994913926,0.07224215414684967,0.06483694631293545,0.05854632369049685,0.06634041574608315,0.06319539897288996,0.07228016321662128,0.06726116658787268,0.0735161198127823,0.06517820679939942,0.058756693420829516,0.05782800152607642,0.06067562332829597,0.0694052095948091,0.07056369521377709,0.056600868674053145,0.06162171602896252,0.06583476484538281,0.06573932873275015,0.06741313633718593,0.059067803144074005,0.06465892662923436,0.05519431959906326,0.06648010828661934,0.05813971629595889,0.05966541950547702,0.053387266348521326,0.06925651950642854,0.06449442915071321,0.06380592946960925,0.06355512337832088,0.07137116275204,0.06497435000881849,0.06437566419972582,0.05886822842782655,0.07498090828513228,0.05516514055306945,0.06815611539024913,0.06843327456519041,0.07112919209828883,0.06409821965704995,0.06399835358073624,0.05668925916666877,0.06368743563240226,0.06284941267671071,0.06052835196400938,0.06498636160706736,0.06888668310325036,0.0667437070545811,0.06273419341787166,0.06464012188684001,0.06426032334948724,0.05528502427668758,0.0527853676143102,0.056172956175025,0.06314633878474035,0.04462152486044163,0.06332639797615681,0.05873271804407612,0.05660421018712146,0.05653770573726061,0.06384536212592423,0.07417006584970884,0.062283980449409145,0.07266266968266528,0.07374747824860564,0.07270313948956475,0.061146604715074715,0.059272922586985785,0.06855542431528563,0.06419563988673108,0.06460592068753132,0.06706492689596921,0.06156940570548978,0.061054421672377984,0.06536294292646333,0.06076867790874119,0.06580334633872842,0.04569378692647667,0.055534577276654375,0.060088591174622646,0.07015099279194462,0.06499357889016807,0.06905230388101488,0.05286817479321841,0.06231373960126251,0.05751775930056158,0.06485699285994345,0.06056052858458333,0.07182461465442434,0.05751080497009343,0.06342020867031381,0.06347609951177671,0.0671620934192095,0.06736377881905953,0.05598856973006003,0.07214691178467211,0.05510819500551037,0.07503695855299813,0.07115949307334563,0.06842040281929676,0.06595530081006641,0.05561435294568609,0.05772285116354642,0.06143709121734981,0.061403789845661964,0.06679806939873267,0.07123540940164358,0.06244652864567945,0.06158400901223497,0.06547147241556434,0.07244117462388158,0.06871703014649513,0.05719018732651207,0.06478737194675482,0.06956242660751094,0.07366182445987297,0.06493489887417,0.06552287318306446,0.05973257187263241,0.07528627941964652,0.059436442858566266,0.06819048994832982,0.06633280114854057,0.06953794294762566,0.0677764242478651,0.069645575522026,0.06467555344341948,0.07127619717442003,0.07277618467990701,0.06030565719262132,0.05576928473001147,0.06881451718969593,0.07375214776649072,0.057520476641573934,0.05761597461401627,0.058932255292244745,0.06780788563224499,0.06097930952856293,0.06059530982501861,0.06294970332188947,0.06218702585178914,0.055854116662493965,0.05806946455482643,0.05566084321144271,0.06898576195691344,0.05339837894214351,0.06471633750946088,0.07423706936303559,0.055938546750998686,0.06485275925106057,0.06094591103897745,0.06172376701920095,0.05300632830255958,0.06903732885194308,0.06369638597406163,0.07319000564552045,0.05571557089217261,0.057300130772646396,0.05981652116597813,0.06221079727161258,0.06265743847846239,0.05709594704313216,0.053922913624273136,0.061001055249116906,0.06156624599318031,0.06851281043974786,0.07111660073330987,0.07000375131847417,0.06494393363997833,0.06288545013941496,0.06366535078367846,0.056688581799362384,0.06578555843431427,0.06271953546246849,0.06559672196996366,0.05905997187100111,0.06771372638041209,0.0546821100669931,0.05701117619402689,0.05384104788636789,0.06537304715067219,0.06733259999694394,0.06806509929710333,0.05690752136578525,0.06609062211455669,0.06728308972035302,0.05336573104633085,0.06977998728360754,0.05982466331445714,0.06751421206216515,0.06843023699614884,0.06481336431379855,0.058035102346407186,0.06252604958754027,0.06130031150218152,0.07190580377084492,0.06464012505188599,0.05056428061628308,0.06397778639808427,0.05738961720651775,0.06391659033750184,0.06793986411799374,0.060079279644193975,0.07446894662876812,0.06925203937894597,0.06356822310941221,0.05697246646236431,0.056325433249748924,0.07163734164356134,0.060470752760060115,0.060954944640921854,0.06649027258470802,0.06880500294651073,0.06586650431795794,0.065552823123673,0.06127789279460481,0.057608477772800315,0.07454134858516295,0.06359932130463511,0.06492523703817571,0.07106730898718794,0.06960941488972183,0.05848985334198859,0.06182028191029972,0.062355906351208155,0.0672602311371511,0.05597906686323423,0.06491253252603904,0.0683766186191349,0.06616985689740376,0.06661425747660867,0.05982027975513175,0.06336975170029502,0.06954504061277174,0.07041599663066433,0.0633874110352846,0.07009128351860497,0.07058302072533676,0.0645840645761373,0.05408115268480122,0.06463459350988737,0.05737875123052964,0.06354878456220958,0.06049769609120054,0.06472784523885439,0.07322099582727024,0.05986172202511739,0.05691838381549459,0.06353216840484666,0.06556870348843921,0.05293230926616653,0.0543507534767466,0.06934634572656609,0.06320758303790164,0.06424944446345615,0.06647940808326148,0.059898157178477615,0.057960246248579164,0.062187546725469264,0.0652712493807598,0.06080656968912724,0.0617298225094653,0.06580961823014603,0.05661602359032811,0.0715814552056642,0.0659021858618678,0.06268554569587853,0.06110231304698967,0.05568329558828773,0.05380388099994511,0.07398226220257559,0.06774918338446081,0.07047044635241857,0.05916159471173613,0.0637878247016121,0.06941094737835658,0.057356078236070925,0.04870480647282445,0.053354186026603266,0.07073243249479094,0.05805604070885859,0.07327087878630836,0.07356918294006962,0.0628627164554578,0.07205309120684719,0.06657577195695898,0.0664911776795934,0.056668265668578195,0.06458948152339702,0.05800894624579522,0.05591810207189582,0.06219444341990319,0.06149521588887045,0.06513946773684207,0.06746503162332143,0.07718874496269568,0.06425099752182628,0.062250833869424746,0.06524506552581422,0.06301440407536421,0.06950396448608093,0.07040370795439246,0.0644518530332776,0.06173999189294454,0.0681246250183134,0.06224280491968426,0.06872272207662448,0.06257861157795541,0.05576340424023269,0.06447895234039536,0.06742683548883155,0.058237979135100426,0.05945466731975477,0.06594566330399076,0.0535573997050767,0.06249372503064233,0.06599543697517181,0.05992451040009623,0.06937860380138292,0.0530453227313175,0.05959474611892341,0.06409004651935968,0.06171924535534134,0.06720854989160227,0.07272299553289506,0.06705259208500758,0.06985459383483325,0.07059502339263421,0.05963749322305757,0.06686316677467971,0.056098033400399185,0.06049838438737244,0.061477297094683366,0.06451132652393507,0.06245164391227208,0.054406858466650304,0.07227981385786064,0.06984826170280214,0.05938447437840545,0.07314102845357033,0.06609124063130929,0.07258752190512344,0.06417854589927607,0.060357861563578405,0.06960372574619,0.06706918665709757,0.06894597690946387,0.06147536276134745,0.07118621534673152,0.0655295252223525,0.06852450778907682,0.07105735325076576,0.055494346747375245,0.07288800567415069,0.06992694700656457,0.057681402883183866,0.0526222938208757,0.05820785387219124,0.07616876700633074,0.06199150406430604,0.0632067559969631,0.06871816993591784,0.057030329143306004,0.06333032565795892,0.06259969475438074,0.06224970801782445,0.06264312530584107,0.06810334704257313,0.06582849141419175,0.07327704347423743,0.06617226458063605,0.06288541644966933,0.0649590655672137,0.061344015203745786,0.06654300893839418,0.06164305985861915,0.06069025708616467,0.07048248081885203,0.06895099373227229,0.06902965635871111,0.06307477512259074,0.06493428185049319,0.06032317191654926,0.06645691727205211,0.05765359275249954,0.06617055072174344,0.06669101543790346,0.05714071480477807,0.07156686609591695,0.06104289145291126,0.05869854191522485,0.07708565562760647,0.0516432890120394,0.05246610102165758,0.0682346336455766,0.06931105540067717,0.069113919660376,0.06871927108599427,0.06690942329686098,0.06372323077537126,0.07548118458959904,0.05246407152831538,0.07465224883886143,0.05692214860274991,0.058518089174324606,0.05743624033682991,0.0590809993181559,0.054638836425379284,0.07155682324638107,0.05878536461658258,0.06975307073805728,0.07001195276501844,0.06135366062621531,0.0721248234992818,0.05743296992306225,0.06990109336450188,0.06430633857735976,0.058650891721151385,0.06445061221922324,0.06578551871203471,0.06460072231301547,0.06707988801250454,0.06868678531836109,0.05328913911746165,0.055369288904403964,0.06362285015861824,0.0706470442150471,0.07160221692608593,0.06474687264168212,0.05231721928400363,0.06285670697922045,0.0679792602582235,0.05359494640610124,0.06293060302921935,0.06964174546814741,0.06290447100528768,0.06684957673098374,0.06383426448344727,0.07264159795227759,0.06849090050949416,0.06532872850126459,0.06776081458473543,0.057285452927184566,0.06764553471353232,0.0680430637601795,0.06130482472709101,0.06726587887098126,0.06699584649946996,0.06551985644331226,0.06254589120014784,0.07319309599454363,0.06716293563004821,0.06371157598947046,0.054636566687125965,0.05664561723918732,0.06238199378019134,0.06053473197969468,0.06484250704555132,0.06881343652376751,0.06066504039611379,0.06507549440260499,0.06104260862789895,0.06315584006486705,0.058609991956444674,0.058856265040282074,0.06570214532025492,0.06788801855883247,0.05658728405740993,0.06584757122817189,0.0567384394746201,0.0688599336184762,0.069985729447753,0.06275917370408574,0.06980718805376898,0.06623973712251144,0.06155662821185683,0.06955778538962862,0.053144301946406194,0.0636067001517492,0.06538223091653914,0.06698393780166209,0.06288013476807935,0.06619667551688146,0.05955469801122435,0.0667500613264149,0.0648093935425263,0.05593903925397983,0.05398138992796156,0.07005617312457507,0.06044301888601472,0.068897741310527,0.06739560375277001,0.061485134292732994,0.06621750439205111,0.0684962262562272,0.06956718783017149,0.06782762667876938,0.06854971797218859,0.05901555303352474,0.05841624086498541,0.0708911490337161,0.06827254454806053,0.05774330406184555,0.055868609493077884,0.07348527725939921,0.05792579156711625,0.0709646532615153,0.06675800763845623,0.06848753629981533,0.06248541650016607,0.05645633538387897,0.06642827236460641,0.06068409018522593,0.058129535437512814,0.059029015124544205,0.06441776752277056,0.06568430345640489,0.0691408357910144,0.06652146135791785,0.06062325119060613,0.0577817487843046,0.06253868627001533,0.061910993573010775,0.05673218493736104,0.0580826675650347,0.06480937766226255,0.06118566588087304,0.05796953662914625,0.06737813178835479,0.05762546926988035,0.05904333095444163,0.05693718557238931,0.06353137162166224,0.06103231820307187,0.06886096017055734,0.05668970546804539,0.06626305544538649,0.06664129972192531,0.06220146309399398,0.04657495719992356,0.06326119206494912,0.053831762235482764,0.05415563265380058,0.06307324992044899,0.0669463790628693,0.05786313192621208,0.06081194375051184,0.053881390167717924,0.05454565847311098,0.07126928586095835,0.06517318513846194,0.059940424449349945,0.050653525370051924,0.0645142389516726,0.07067951388898805,0.06303569283219237,0.06148074996246664,0.07147518263881517,0.058391303943087045,0.05712624490155746,0.057976249135603154,0.05727653881556338,0.05619467057298226,0.06907215367897541,0.06173754692750246,0.07786255115942117,0.061132258956782544,0.059407642196608164,0.07081790520286466,0.07502286211044762,0.0747187133590234,0.06994891662792486,0.06292506825318953,0.06707408401032215,0.0661371089978455,0.06415755126302564,0.0537221031712245,0.0698597564962225,0.06256112546239453,0.06791805198710485,0.06665517465162187,0.055807374358211564,0.0683400853644636,0.06797530470039387,0.06765888430439151,0.06924381824500078,0.061322183508631375,0.07028563550515787,0.06408378021360714,0.06734383972996835,0.05824713067378546,0.06890284698006816,0.06026154007425677,0.060815318491756656,0.05313191308677591,0.048021467778885905,0.07045718601563403,0.06196478558728874,0.06565294847489672,0.07210795086142735,0.07099651171739826,0.06517892864347205,0.06651882827806518,0.05592888147868301,0.06732328175077718,0.06801786436244306,0.06213301733963974,0.06933053446691712,0.06180276791669513,0.0697098916577668,0.06499873802093437,0.06754663127458488,0.05351090778995751,0.06114941269081853,0.0651718238277294,0.06809242854150896,0.057350837110504926,0.06174906256735149,0.0699771605673661,0.07511065948052437,0.06816900299490841,0.0656892882239386,0.05993417298919816,0.06448363244645389,0.07267284308027522,0.06807602311558628,0.06538765301052205,0.06694785168174391,0.04781649845239916,0.060591001875397954,0.061951645818114624,0.06611384074116686,0.0645412934559946,0.0615308310113502,0.0650634452819463,0.06914903128810657,0.06100380524809705,0.06737055202480047,0.07008649543349274,0.06572324318083356,0.05618726255286275,0.06702154979784071,0.06576856149315786,0.046208415843920075,0.07091338726642943,0.07102493649225092,0.06764411898087197,0.07144350997978063,0.06689237307916554,0.07315252171482878,0.06204470466841152,0.05379649632052118,0.06135966411259273,0.061053709115790236,0.06174749431544166,0.06265767052614177,0.05922440586829819,0.06162786987720616,0.058862961320086955,0.06993410994215302,0.05927988778901648,0.06850498591797359,0.0668064805219239,0.05536844698159264,0.06169779377330935,0.051316309918600395,0.07076733753890842,0.05583335304100122,0.07089647925728343,0.06345807739106449,0.05376759753330517,0.06704072830886501,0.06682289189252713,0.05993242091552463,0.057309191533171794,0.0745147716537294,0.06728634005359602,0.07038183454311381,0.05447568153043764,0.06877849936481534,0.05996927427585097,0.05535154117807507,0.05685170239310931,0.06145261145394288,0.06884909035269457,0.05932924048640002,0.06593182934436381,0.06027843696993215,0.06589578407066862,0.06817223731828825,0.06793116971515126,0.061749294029188474,0.06411730354969222,0.05489821394728207,0.059999272560666506,0.0660276016695126,0.06446998739321479,0.07330371123306051,0.06800288498594126,0.055544928843081834,0.06630835907516128,0.059750191635680294,0.05410181247616344,0.048166621389819184,0.06479502959088067,0.07414576005924338,0.06849519023151537,0.07179993161850626,0.054909100641944444,0.06828720801669211,0.06253329275079661,0.06133131388724849,0.060502379069131425,0.05370518111731165,0.05466139357108586,0.05995211715411647,0.07150476016531282,0.05175062068256921,0.05548698355406094,0.05908649775618402,0.06354324636532793,0.05882447381606918,0.05587488234384702,0.0537432444593783,0.06624487528435216,0.05451117942380052,0.06093139750342659,0.06144561745355241,0.07448883343380719,0.06011960729711874,0.07189768078533518,0.07172747296305655,0.06840394657178955,0.05381716481933462,0.06936786559731124,0.06100613092878139,0.057292189344793396,0.06319518119243409,0.07280385187238206,0.05588673130983092,0.06471558977352353,0.07256347444236916,0.064872173845237,0.05585809789810109,0.06428523382901259,0.06236476834185743,0.051580519515932954,0.06959379732550748,0.0628284619170015,0.06425842258988013,0.07136310592561843,0.06585360145521191,0.060083313655654426,0.06324571716454297,0.05931305613244823,0.06651207444445296,0.05849820518150393,0.06976178358466151,0.06499646601887925,0.06885912509736648,0.06659768426354293,0.07278696182373774,0.07168766887432151,0.06707821306865971,0.05796928363900698,0.04933877713468426,0.07163135019506972,0.06784925932748014,0.06210519957062552,0.05400944500211436,0.06015258625411941,0.05524884285371898,0.060206928292891954,0.05207844952162298,0.06722055333479168,0.05989942070682887,0.06713627865701371,0.07025653096965834,0.06960621031740383,0.062427490953010444,0.05941825353320837,0.060517071558992094,0.07264721448310804,0.06437850099449602,0.06998704790189074,0.06015025604756931,0.05836448697607686,0.06404489199385138,0.07027744212414136,0.06001017845924003,0.07194892594473898,0.06164479045095508,0.0619905715873774,0.05600820233726445,0.05231384658605821,0.07317423843665169,0.06392671728720156,0.05602684799928112,0.05860602356730314,0.05495225436437022,0.06358017862883873,0.0691605680392853,0.055851766380983534,0.051841577210916825,0.06065430063163435,0.07223911264015409,0.05847136815479553,0.06780083404967445,0.06605600895138702,0.06890093069328276,0.068678440516211,0.07426930405059878,0.0496972825377254,0.05906362868093133,0.06852084411139572,0.059571522212834856,0.06089237997077566,0.05606913708474071,0.050663234631118964,0.06716922243017408,0.05646932010825144,0.06745676881519057,0.06831643450911704,0.07605637654585246,0.07126296616830011,0.06779949545336347,0.06196420524139462,0.0663863774639725,0.05393240573589579,0.0560409414139586,0.06854982077219093,0.0700757137336434,0.06739397362399487,0.06398513968024358,0.05697322929064681,0.06142345871008196,0.060985237878840984,0.05949727616548664,0.063597326982794,0.06244947004310965,0.06260381367056994,0.06329150524155197,0.05928312129100575,0.06995167837596564,0.06697944498142623,0.06115649427358714,0.05808069266230702,0.05662448691708759,0.07101705068655947,0.06676021797581619,0.06455441023706149,0.060939611136897595,0.05974134052321982,0.06455663581953217,0.0677824322380415,0.05799460134360675,0.06357617451987185,0.07024741830730857,0.05792475295851027,0.0662864929872041,0.06911150710324779,0.07246081148378061,0.06674409542254017,0.06614512806727654,0.06667451415510059,0.06435049668311682,0.05763221639057393,0.0533828641188252,0.053217180456890764,0.06238441458472037,0.06938395997564764,0.06088907079883006,0.06793712550005364,0.06302421052781318,0.0549242311730494,0.04733278539966621,0.06806031377477988,0.07056012366709344,0.06247046287312542,0.05724161234421349,0.05852925880230206,0.06869998682355842,0.059527899001510405,0.06494881361023125,0.05659468172376615,0.06385930449208349,0.05626366982682416,0.06041132451393605,0.05491463501430992,0.05962337711737989,0.06519559263733041,0.06215887508415422,0.07266962672978745,0.06682791790204701,0.0663161693803879,0.06087167250975987,0.05963489249191504,0.06719209425141177,0.0638847537852534,0.06677061923848499,0.056566710687474944,0.07511741854661858,0.06615197646203591,0.06464698012853501,0.0639777230187283,0.04800058771292916,0.07139318564975197,0.05438598951965528,0.05518227521942831,0.06126566504409801,0.058850957571294216,0.057927184965894836,0.07404479462264445,0.06698068036357666,0.06346747036607332,0.06472478096260986,0.05757904741574128,0.055915243179887636,0.06989473470950604,0.05637621116726605,0.05796795256092842,0.06478831565793616,0.0610935067853938,0.07027021449222351,0.06605950813664474,0.06029749591360312,0.06951867564770359,0.06437503708931948,0.06024027233950004,0.0663578922371969,0.05293831159308403,0.0681944704586891,0.06598001444927115,0.06371129605002296,0.054255632993566186,0.06652687389175005,0.07802511638164975,0.064441212456442,0.06561107816421965,0.07154974818586864,0.06993433319083325,0.07275250136255708,0.06227412141373729,0.060371036148071607,0.0734001287518759,0.05875745896505831,0.06979913308696836,0.06851100503284113,0.0626907273444702,0.060081969992059454,0.0642507731681745,0.05985426456737636,0.0727410247605229,0.0714997628491055,0.05916002049155332,0.06549981642009606,0.05881196560921412,0.0626772108273739,0.06866794973817739,0.059801259928266084,0.06347990893768399,0.06264438940332956,0.058713022428704714,0.06040307880551668,0.06046623588310776,0.05546973374835544,0.06970413375666903,0.06874397489241407,0.05727637234895103,0.06307182367741186,0.06279222870087889,0.06539688413677905,0.06450536490019315,0.06853247052640439,0.0672651515974354,0.0725527847401552,0.06302288809246495,0.05712762113326922,0.06690408463561287,0.07248454372245895,0.06742904415610923,0.061520614833862584,0.0715764541531595,0.06452451503621612,0.06655527836849692,0.05982727903254004,0.06031393844299897,0.06299865224638401,0.061337349820334446,0.05638449093842277,0.05729830978810567,0.06818585333810527,0.06981750689992897,0.05796647432383521,0.05699335468121414,0.0565839508744056,0.06353628893990718,0.06920976912954033,0.0651917122275116,0.0716856276996142,0.07270270722013414,0.049194877758444305,0.057217744734999826,0.05731433145085364,0.062011590509109364,0.06087383621892532,0.05665588674374295,0.06036426396634699,0.06943042955344117,0.05873887698581204,0.06477588639116774,0.059225999037958485,0.06673046391151996,0.0679739372540924,0.06105342831017908,0.06481071406157246,0.07049112771555399,0.06071726586667679,0.05680085283157282,0.06374222064106566,0.06235313718992406,0.05979071285706178,0.06627644912282146,0.0582526589455135,0.058138246520924086,0.06680593459981077,0.06942095589250141,0.05716348053220162,0.0734361404737818,0.06851273567726364,0.06510074464317729,0.0635578687537979,0.0745745400780845,0.0680526940458941,0.06196888866986759,0.06762467764493037,0.06500314226635749,0.06441437184029422,0.0679600826987797,0.06013305855217636,0.06887506549474537,0.04610480682422763,0.06671019328866343,0.06710979104155185,0.06526956850949805,0.0732392286097635,0.06417089495402353,0.05485360191389774,0.06763851282002936,0.05996890906114331,0.06520478280017633,0.0657696741416903,0.06745803552966156,0.06743064279175613,0.060736609521283416,0.06078781580725588,0.062325117017161334,0.062340170849542076,0.06714099159345951,0.057911603910042145,0.06833872086794754,0.05828529768924252,0.060349724843296655,0.056844408156237786,0.06256464336075693,0.05964940089906827,0.06022419790200716,0.065698509015256,0.06813985036285908,0.06993672598364424,0.06896005530289036,0.06397456411012119,0.0591727904817356,0.05402979866519905,0.07443249644554827,0.06109097713336392,0.06859430922366963,0.06730814779452457,0.05829510174443418,0.06940852132713385,0.057533831416814715,0.06434709101696659,0.0648043392597221,0.05760961334794926,0.062306129956203625,0.05749240198141537,0.05923209709699756,0.07288700043510582,0.07323570112004396,0.0714739240699937,0.0542295372439688,0.0652743150137717,0.055541309529541394,0.0519985478250129,0.055374111421966216,0.05886890335235319,0.06712316823556944,0.06595461017989757,0.05953909768126252,0.06510666306364656,0.06132937949280768,0.06337954103997576,0.06206558149172742,0.06014953322442183,0.0571494324850412,0.06619059210311654,0.06424953459931901,0.05546804766832275,0.06650733857195507,0.05927026970634145,0.05610864952000669,0.05820213039967807,0.06498723612723462,0.06436322792508238,0.0692514604999342,0.0636670640437388,0.059899157103045525,0.06510275352517567,0.07054704429003769,0.06515072720361173,0.060214391128892666,0.062178092327244285,0.058210716249196534,0.046006100494592965,0.05786702928056567,0.06420965183465158,0.06566942440668935,0.06235543384919605,0.06394552719976118,0.0603864650078344,0.07008889688515993,0.05290003635736272,0.0673352736625216,0.06245716092661454,0.06706170138308618,0.062316038453310046,0.05603870879711865,0.06727802082907085,0.07064812333475151,0.0593443070452768,0.06572624659171834,0.06195966267693503,0.061533869125650946,0.05854209983646445,0.06965717811827683,0.06627564658420441,0.05410423787818485,0.05936210715875209,0.06899150020377505,0.06914198273404375,0.06640667591431285,0.07247629602030128,0.05068782097013798,0.06026244445277874,0.07051957291478553,0.05476768283098536,0.06918788589338574,0.06668508720304014,0.06139417386091462,0.05098504647838044,0.0601393873603189,0.05789754965113199,0.07002643240628548,0.07297165655294498,0.07139468148402905,0.06457095172928809,0.06428288598548373,0.0656206197525058,0.06516285785343087,0.06833408124086093,0.06640803052564345,0.05995017250580832,0.06833398226712495,0.06654951918615887,0.05663324952771108,0.06733815229935289,0.06520310670205737,0.061398595955950784,0.06051688709232457,0.07298695821590642,0.07344186702382727,0.06320612041921761,0.05840036447380888,0.056542898392917935,0.07086015496492032,0.06840136680889633,0.06899459203334324,0.05693707225005011,0.06424936940405446,0.07011969548861052,0.055926305135278256,0.06527870643544709,0.06754178239715015,0.0607648736123174,0.057704112556758086,0.06868680496405832,0.06682020077570197,0.06478231362381198,0.06344169575810049,0.059643945710857404,0.0598620063464602,0.06843197063789204,0.06196806084290212,0.0692872081102673,0.06687908762248282,0.05675174265818583,0.05628805409148334,0.055039710495035504,0.07194007224969942,0.055566193359395805,0.055824882970775994,0.0703935580242599,0.0630442653837689,0.06699839028528123,0.06678675281915994,0.06941612320536644,0.05800339447790985,0.06472349028766908,0.057579073756032786,0.06588109934457123,0.06610582880607796,0.053449012632789145,0.06096266871615401,0.06544636931735606,0.05561596273448328,0.06623360535795694,0.06727671177142194,0.053741429007232056,0.05426234301128059,0.07222374644554541,0.06165439515477934,0.05599147644076982,0.06963891823559246,0.07323119170358681,0.05863082670177845,0.06623543612850977,0.06863555829559286,0.07164992033079391,0.06804072483700069,0.054196262212484955,0.0644232074782482,0.06039756533785707,0.06787725327257298,0.07177119542025702,0.05976813247273195,0.06308009122843708,0.06045221601660068,0.060042912292117556,0.053202530715055305,0.06115182503549136,0.0610067300792481,0.06692368906195435,0.0632701620564031,0.06808339631194066,0.05869337061407236,0.0662289304692974,0.0573120865479809,0.05617694980554834,0.0670391904014406,0.06726093829196994,0.06291344959493915,0.055856178251991656,0.06127263427454936,0.05480855761758811,0.05989918775647689,0.05653938046965218,0.06449020935532558,0.06077128534340566,0.06383524763429252,0.06103505956820328,0.07342072047615665,0.07384180713274957,0.06954144542877366,0.05035185322358589,0.06842289908538131,0.06032478625373282,0.06483390959032553,0.06427289866339284,0.07003814096889664,0.05960650007909041,0.07010745918731576,0.0720318280045979,0.06532681396530785,0.07034029917016424,0.0648825728218749,0.062211769826784315,0.05646302742013277,0.06466939532524012,0.05901959869930238,0.06382754425416157,0.06842417627014769,0.060963279069838464,0.06606638100287685,0.06946292098684954,0.06289252621293613,0.06901504344253684,0.07453760901265935,0.07449686794307987,0.06837906109298547,0.0603531000342634,0.06449263039282035,0.057412500230015914,0.06265129694397248,0.06798096657094385,0.061834746173794995,0.060330007507381794,0.06226010715124271,0.04917308228655188,0.06626646746225298,0.06326336778163101,0.05376111126793916,0.06809927177989346,0.06426447642186667,0.06350826097373705,0.07238316553506419,0.06991556496083826,0.06962207415374508,0.0643689298695127,0.056703881582133554,0.06959066736628544,0.061434130160721155,0.05710385770572429,0.05084469331383938,0.05758193092916126,0.060124372511814225,0.06963014810240549,0.0573384506454056,0.0652017040292096,0.05916302566252683,0.05995292135607974,0.05239730180629023,0.05473320504274265,0.062158568273116194,0.05930678112700272,0.06778643749707342,0.05157957552906842,0.0667015934878089,0.05614268092202974,0.06534410069883566,0.07197149073055459,0.06988349563153606,0.06394484977328996,0.05960358022991509,0.06237552701758674,0.07094778407026732,0.059521493879774304,0.05753915833028486,0.0665065792167776,0.061872230487550625,0.05362170761825766,0.06640432878976402,0.06602433256827824,0.07058118160529081,0.056034883811221264,0.06636280260944466,0.06187889349614502,0.06643795154527116,0.06479000310746857,0.062027096722045406,0.05918111577237241,0.06768883333487809,0.06596114561422353,0.06389052831920193,0.061679722300713724,0.06942084329526056,0.06799510089419092,0.0662555532772108,0.06509926311653691,0.06347604133142296,0.06391074871623317,0.04756766005847973,0.06133635691515051,0.06308955487504188,0.060103484237586054,0.06357766868880654,0.07083491888828734,0.06346371294716553,0.06293923816469628,0.059206661642660155,0.0655250994283309,0.06993820809570608,0.055563121677242644,0.057381053620568764,0.06860313997090853,0.07305879735370639,0.06126144841030343,0.06476920548846238,0.06384737809733808,0.06256030807940788,0.06996734928067752,0.06464586794994218,0.0744254445673532,0.06805016042424217,0.06132164079359912,0.062283129364640304,0.05384314579349252,0.06669305351018165,0.06456519631193586,0.06560502378848426,0.06737181248604701,0.06555501160844868,0.0677840522752696,0.06598623424206146,0.05747566093539708,0.06759203931915625,0.06287920027473932,0.07080739756474816,0.07092766525456898,0.0701497169630656,0.06816519870527638,0.05733669626956549,0.06841734608852632,0.06984922276949704,0.06028272596521865,0.06733853968822877,0.054366777246345394,0.06189818762114379,0.058690280261560104,0.06630589067309961,0.07365638913960634,0.07308801101773003,0.060028986755638836,0.049258724449994566,0.06525066156597795,0.0626279101420216,0.05856808345593731,0.06743932193076932,0.06851641453649437,0.06911408055529517,0.06266717003393428,0.051166435680047376,0.058811771987064373,0.07565820712409313,0.06328734044516758,0.060530312304157466,0.06412834481088683,0.06289025714921465,0.07018833366720437,0.06278479638113468,0.06257147715819009,0.06474039997073688,0.061834592889675126,0.07441287194813473,0.06106354107081821,0.06431520998090884,0.06803430535298491,0.06093555610235693,0.06400741122474204,0.062086714535848256,0.06990272590701987,0.06794982982754633,0.06755403041693109,0.04950548202598391,0.0546748265978918,0.06449397460028125,0.06739698647510235,0.058803218351742466,0.06428390231971223,0.05817751989801651,0.07714479502750612,0.06026284661629214,0.070190003067711,0.0686843234153224,0.07283957741966782,0.06320103568009552,0.06389726880713818,0.06690131039238738,0.05724296911987068,0.065173202011919,0.054839821560351845,0.07235144298472022,0.058998439246775114,0.05805796860792689,0.06133857694703188,0.053292250063858446,0.07012383020453669,0.05281835750263321,0.06323095364460489,0.06949427098204483,0.06441537896329241,0.07025771407790442,0.05370364049847198,0.05050355922572142,0.05705484051890877,0.0668572853999528,0.06377167379819756,0.05482034626824586,0.0642280583893054,0.060204091117882955,0.062140440988460326,0.06530630884207703,0.07041228017210298,0.048963435431950725,0.06140197523314161,0.05782280793274842,0.05773663048967252,0.06497708940285515,0.06403231306891133,0.06090492310410449,0.05426604320267292,0.07045517242314717,0.0679231619855617,0.05968238164926369,0.06452332157344681,0.061066615598448254,0.06685878165931701,0.06462977290501998,0.06020413594761782,0.060738921278204135,0.06906399474516262,0.07195474408336794,0.054614127497525075,0.06631668198366479,0.07486905582882314,0.058769588116526335,0.06421963486949248,0.059984197279760315,0.05013284967344853,0.06051452270898698,0.07172916431756657,0.05577911128875428,0.06896170352173649,0.06216016085916853,0.05240571877340838,0.06289893712086408,0.060663091917386835,0.06267087903526292,0.0642421367006372,0.06959971659083608,0.06423978824315883,0.0618484966626997,0.06125277570884631,0.06502689577797671,0.06997495723079457,0.0661306991210577,0.04942551934024454,0.06445938318409533,0.06305209699107428,0.06602366588870241,0.06542562659094359,0.07036408151100124,0.059840970511492377,0.058288311489325215,0.0584319096846202,0.06870589422511916,0.051650880630465495,0.07510316417151122,0.06474234883386312,0.0630824823060938,0.05836870810817446,0.06492967235617225,0.06269474369676979,0.07148769369877737,0.06390087257547059,0.06299563287144835,0.06892825302666555,0.059323250830317795,0.06642953897674839,0.060000768902643775,0.055388120688673756,0.06047274608100931,0.058356112060055934,0.059150134608592655,0.05814479329754964,0.059504106988837596,0.06482795029333842,0.06235217643993506,0.06335397235894494,0.06552700034044877,0.06827041931282535,0.059248469021128965,0.07022839587718108,0.06333857154556796,0.060378183996595725,0.051245782337464144,0.062328550451422375,0.06258944180616074,0.052850571333954245,0.06596126023287918,0.05548158964267325,0.0631945144326735,0.05692739911563436,0.05033637148831838,0.05927129939332501,0.0643044861309888,0.06929478570904966,0.05739405925803018,0.0655212228948692,0.07165568327288169,0.058035689026679466,0.05997426615022819,0.05623367669313792,0.05773395033596344,0.0638450153960394,0.07082483258064246,0.06632109308443979,0.06790685593994286,0.05854673717379297,0.048829288704720544,0.0596415904314388,0.06220677078637212,0.0633965357237272,0.06803035517032253,0.06735091298476707,0.06563571923843592,0.07403244924809815,0.07046088555367827,0.06608183877690033,0.06506038249104858,0.05681466392878813,0.06313895036519346,0.06108216523641241,0.0634254454391057,0.053606659495287076,0.07189652748378736,0.0664174907855022,0.05070388746924344,0.0646384267339035,0.06693496781721585,0.05839634895762044,0.05733153980792663,0.061303134508154646,0.06068954646650304,0.06296273485331358,0.05794106977422661,0.06617925557776841,0.07306333564230248,0.06188730149511144,0.0685234353858105,0.07427907943114857,0.04794637242770443,0.07082370457966325,0.06891903273264147,0.06156588690823172,0.05797789184926276,0.05712020814332233,0.06332584274590591,0.0631373378423358,0.05516006550630921,0.0645583973513572,0.06744352130566844,0.07188900052228031,0.06779040214860693,0.0678942714651983,0.06711747874413226,0.0733975641505327,0.06568929842027388,0.047714709664581884,0.07027614945792171,0.05947100978137687,0.061784600880602385,0.053953401126582076,0.060067705855505166,0.06992176487513133,0.06333301371126761,0.05645880264605903,0.05750608911790879,0.06509547998757625,0.06231906899732318,0.06672320143489857,0.05575780667469703,0.05637754982607569,0.06895171171746547,0.061312106674669874,0.06639386626538253,0.062019533225051354,0.051174233419399695,0.0634006630125403,0.06772742844917848,0.06875346920473094,0.06222830672189543,0.0633139856331926,0.06377891633142546,0.062304449124853924,0.058375397179756185,0.05771854595543244,0.05595141370909768,0.07293424962880893,0.05862949670755975,0.06739943303491522,0.07232401904913116,0.05578874068944924,0.07066919444434479,0.06655446223676605,0.0619215768461134,0.07216528475267237,0.07372131052092727,0.06945153176554822,0.06881034655269302,0.06603640374863828,0.07114003314398615,0.06474398571934332,0.06474575750740225,0.057772312350321126,0.069245307326547,0.05908415792207303,0.06182355208210022,0.06732159044192233,0.06033557065830322,0.06599960184908907,0.06830289433728667,0.06832306557640529,0.06628589207795577,0.06283945423209125,0.05743733330559156,0.07048342146834519,0.06565498008731435,0.06360652831319155,0.06633419058130594,0.06409164736685719,0.05667200684013952,0.05246278970221742,0.05797673876105977,0.06898718471409643,0.06481864984247217,0.0616639138402116,0.06104907492844067,0.06611659964591704,0.07143428121543335,0.05766542902403443,0.058838404108729106,0.05558432985323387,0.05896610649982326,0.0688888509540921,0.06510651426386212,0.04864592589482359,0.06109697120013671,0.07455483086547599,0.06034109286223843,0.06887064261905554,0.06301200868826948,0.05829327944943316,0.06634949201441451,0.06009489451020995,0.0651913809940518,0.06423304525900186,0.06555212951449567,0.06392090177680539,0.06320201840021367,0.05249035373845323,0.06055838758397365,0.05967400591856341,0.06198257367102474,0.07093610902565509,0.0587410067077368,0.056824156117315935,0.05853462490746661,0.0717304462007145,0.05536873012951499,0.06894972280771577,0.053035096577131124,0.06336434930863818,0.05722434953344794,0.07136609598349107,0.06780959028343246,0.06806444895094305,0.06684065804953558,0.05246266548601104,0.0676921728953919,0.06251149421449842,0.05450337860883652,0.059949804159254716,0.06811197871181357,0.0646367813074323,0.06623843102573887,0.06492727072120032,0.07452721760244899,0.054860727190229455,0.061295512785234194,0.05815129677886474,0.0634058853784848,0.06206280496896158,0.06893731267381281,0.07130619690301718,0.07218430026339312,0.05088623121295548,0.06280879616803609,0.05893879735958396,0.06307308146199461,0.0541592725172641,0.06960271437444038,0.06538921365359784,0.06076524532762605,0.06013919433251719,0.06666186224620152,0.051417690956238694,0.06249756603384196,0.06957783452436182,0.05913011591891795,0.06827157463748873,0.06368003482739172,0.06241155614533226,0.05365889894115712,0.06345978348810197,0.06956419578750783,0.07128822549497704,0.06800788082208487,0.06267249550621574,0.04851870620434261,0.073534607177668,0.06258019751436797,0.06950510290574882,0.07201837720783602,0.05873446426004329,0.06923489856046863,0.06661038117211698,0.05523319532633076,0.07438945421419275,0.06550701031905021,0.06586370528971683,0.06319921214268248,0.0676973574616967,0.07180045120804025,0.07157066308993743,0.06041047732620264,0.06770491682159217,0.06492205218627481,0.06237222517373739,0.06212227242744702,0.06167987546121713,0.06546486971268063,0.06856592006921237,0.06729228192771788,0.06028854165395403,0.0664388353661369,0.07004711041194728,0.06343090869028273,0.07025033590498689,0.06352104286286048,0.06034911383992282,0.07027382118073358,0.05608274230329413,0.058807466631482785,0.06269338241755326,0.06008699295436125,0.06889234578220929,0.05595850827415136,0.06522252204209492,0.05811247091877584,0.060564655848198146,0.06294845315773352,0.06550074506664982,0.06409799445662134,0.06417744082029316,0.06426089021925105,0.07114250731032992,0.052540255126963364,0.0714858892939022,0.06239401555554915,0.05242162662866626,0.059776457093423864,0.07255431727088829,0.06379559486716069,0.06840016083722848,0.0646152361923876,0.0653002585743625,0.05999993912868001,0.05344693994095334,0.06716795804161138,0.07213914541768535,0.05072786916795834,0.06287297201573264,0.07188138862837262,0.06736726338577383,0.061340766777297995,0.0617601454404926,0.04760371799114364,0.050822358198960034,0.06318638346246884,0.0512022896763448,0.06540777114736901,0.06948942741150999,0.06580672347489597,0.06944905950055386,0.05246634037067838,0.05529180563215,0.04915913197032616,0.060877146542845276,0.05882412140014552,0.062249199399507446,0.07117302954464153,0.06318808968187364,0.05627978517440478,0.07513100799233724,0.07406529449892604,0.06118999014794081,0.06488730119147793,0.07490449026705734,0.05143875758857547,0.06804711998769768,0.06468779235809773,0.06999533916479347,0.06020914004588085,0.06085920114004467,0.06962643707824075,0.06668002059906436,0.07123106859265886,0.05981832120489074,0.05346879355152742,0.06747192914667234,0.05635328238171185,0.07087331190274285,0.0614585703062162,0.07103776400610326,0.07335439575031219,0.06444707124410826,0.060729396262011966,0.06316639490958237,0.06462569301428167,0.07517570416941004,0.0644027130696945,0.06911109000955445,0.06578101098505243,0.05761916817967401,0.06637570204509413,0.07015628537210855,0.05633043995714302,0.05853853007026075,0.054202205143683506,0.05286964282923868,0.06244706081491544,0.048921539315624304,0.06956971906594042,0.051389980265033455,0.05998335481781656,0.06302690960471483,0.060669645303611776,0.06348462379780934,0.06511289392763106,0.056707972687404976,0.05644111442604369,0.06977855088292173,0.05249098953541457,0.06648341193653215,0.0675127232292343,0.06847486011964846,0.05898067540874888,0.06183947857423635,0.06137805326929659,0.04928284077433097,0.05498829212709239,0.06366024995228031,0.05302387611311121,0.05845096896910296,0.06795529174906634,0.04963769300467376,0.05927741260452135,0.06375008752036213,0.06768461298997905,0.07485190288086138,0.06919788339367762,0.06400663156084102,0.052074301746937,0.06566711919401846,0.06021153761624546,0.052267480275932345,0.06265206906331917,0.061033105938751206,0.06350930091941615,0.06243241643430258,0.06123914705657882,0.0396969724189327,0.05503114472461533,0.058682232860673575,0.05922283972469097,0.05940634472977486,0.07129199451794732,0.05883289130173856,0.0631916214340345,0.062280060612398644,0.06681679904898677,0.06329386616046453,0.06338869121501423,0.05848637056622169,0.05455531515860085,0.06994822496681473,0.06387012841019493,0.06467687067387977,0.060467480006524084,0.06898642083560189,0.05885082485161095,0.05534828417358323,0.06755696000975883,0.05293433368453099,0.0572017000317815,0.06834079806115187,0.06255160676473268,0.0615864785543726,0.0647910902418257,0.06815905171254305,0.06854201722205014,0.05544172733195764,0.05755783062472224,0.06314449366314262,0.05759602115337038,0.06917730273020267,0.066567578024077,0.06344423088616691,0.053166426289001686,0.06612904557176069,0.06706403404470652,0.06852347116038439,0.07315690351803139,0.06585564134989288,0.07445318651748506,0.06587133205042087,0.05106255554206863,0.06979295243122267,0.061257817518496835,0.0618651775928355,0.0689329339049918,0.0601019842932322,0.06333515315473395,0.06468853438208819,0.06248140375786565,0.0674018305866293,0.04525030627738891,0.0601181653569874,0.06618506966516544,0.05697510887803063,0.060501927356692645,0.0670564031210713,0.065102733514294,0.06745198798076685,0.05765749163513825,0.06636492644655304,0.05858000062465253,0.058964732049118396,0.053435657811015856,0.06064623514066457,0.06668959872671935,0.07172092845501789,0.07077894470608675,0.07338932474239682,0.07261456623102339,0.060857325547275555,0.06966498315666886,0.054036301688697885,0.04821693624658316,0.07074156326817158,0.06858284208653524,0.06249712164269965,0.06531516951885885,0.06756901024192882,0.06299804956117658,0.06735302419712766,0.05947408282359251,0.062386251861937456,0.054038760474563866,0.0709701216930951,0.0643018046309342,0.06405587668797988,0.05178821962752449,0.07005616017462485,0.06604258157417228,0.061731205798819154,0.06748362032387675,0.06285037400092645,0.05412047404952969,0.06204407942805641,0.060632919708350594,0.06283001916114697,0.050571674287123294,0.06230305558756405,0.06890542933302371,0.07224153037979826,0.06898838416133052,0.06578302171591197,0.06703832614116514,0.05950372654726302,0.05978760582579584,0.06196477665939467,0.05394583014211947,0.07198203471043449,0.06776961377848784,0.06418561838468423,0.06631943809020277,0.0673852180891472,0.0694341203547156,0.06274560461749087,0.06213787226058466,0.07460028135236697,0.05861724092138757,0.05769274729253665,0.06839536379540137,0.0676186588816442,0.06854406976542918,0.0563082222061099,0.05832869635711958,0.05559072900121416,0.0672352947388934,0.06343754030751136,0.05818533148912226,0.06356354188110815,0.06933387429319404,0.05614674858525419,0.0687859549000231,0.061106422895100806,0.07254494318234572,0.053734915188143204,0.0689050798653013,0.06256558789253341,0.0727393719455186,0.06973464171554616,0.05828219381292804,0.07236982858458091,0.06007785043070619,0.06520552224024569,0.06876780799593557,0.06652468796282746,0.07311078255643536,0.07584882005667519,0.06803932645216768,0.05918842747683151,0.06482100912009378,0.06695880600146589,0.06086556498231001,0.0667229287306843,0.07251043197965301,0.059697525510499505,0.0578837332620536,0.04434330967847957,0.05546734026024169,0.07164771197608694,0.06793246960327362,0.06089682364992727,0.05802265998816597,0.06475211229548472,0.060893298859251994,0.06853265831803071,0.05399120870545155,0.06842061010051675,0.07362914363740264,0.05783792784255197,0.06466884804504215,0.06455359704849764,0.06785532935036438,0.07049189356000124,0.06289595615597893,0.06804756789596632,0.06934875075637517,0.06890771673110302,0.05305229202668408,0.0591217653190947,0.06401171595773075,0.06979148615209146,0.054868066334627624,0.06631996100287287,0.0611964404181919,0.0656631358353733,0.06585597658582804,0.061991977098846046,0.0643182883152538,0.07335970569437883,0.07428443503766531,0.05808931970445072,0.06343215232242039,0.06672781037676478,0.054761397258301744,0.07127986528957897,0.07157155187592959,0.054716777792763946,0.0495680995668348,0.07187826307062363,0.06552496217740039,0.07006825017107277,0.07286825824145426,0.06057772109258264,0.07385797342710917,0.06368637875075281,0.05676992311953208,0.07082042572697912,0.06534560148560474,0.06861703418518636,0.07227728559099633,0.058106722457414856,0.06353937935989867,0.07246318172141536,0.06580901031334221,0.05858511040303657,0.0545870544635999,0.06976499055054661,0.0637925052507032,0.06180266921887203,0.049743907648586834,0.058766382934951567,0.06888590376769221,0.06390928143801633,0.062005178841791186,0.06482608524374073,0.05865085864341667,0.056682007267174844,0.06882384326058552,0.06633264704828315,0.06420704452264975,0.061684888788307156,0.0639139162639743,0.061601896474727386,0.06696446403075185,0.06632563825321326,0.0581653096153555,0.06329597969167847,0.0661985671957834,0.06574180825478736,0.06311453520818255,0.06399775020960681,0.06943811412838605,0.07052199923061363,0.06661611575935561,0.049214618125190734,0.060895542595745075,0.06211357040164816,0.06802490149587803,0.06485007712494115,0.06972504480913094,0.04906485347697825,0.05668976550519336,0.06588345195105484,0.06365461432405789,0.06876066306674444,0.062077884564285084,0.06689506660112071,0.061073718651817405,0.0763969757202963,0.06840649019997283,0.06403595394624947,0.05334250322770251,0.05154384293018746,0.06991512463172489,0.05896665959220826,0.061210941856331386,0.05869174452077494,0.05670650239049714,0.06536434380563677,0.06060503783537431,0.049005473593642435,0.06018479319931387,0.07081208929547783,0.06902021181950613,0.06906235043791037,0.05922154719822484,0.06843891636865829,0.055721828263256015,0.06023274444917394,0.06687567236790959,0.06791940490878612,0.06502256921017288,0.06062767314950338,0.05811974370532523,0.04559767243686225,0.0657987513979705,0.06618074956389539,0.07096434979070283,0.07062872675843596,0.061195348282793174,0.06217255136001004,0.05749505873465356,0.05931467280088288,0.07184537926891,0.055633394495527885,0.060900094925367534,0.06949311686831028,0.06770823139803646,0.06991900738475841,0.06014947454236212,0.06672039497258725,0.06606842185914921,0.06564333607076567,0.05578115252448275,0.05777705416771056,0.0576547275776593,0.06085686576176237,0.0550293910546828,0.0611340773353877,0.07510083534831477,0.06165046167298627,0.06809499971857758,0.054445500572473424,0.06147689021432915,0.06125168374976367,0.06435592064327988,0.062308567004566995,0.054378439709449694,0.06816663777600841,0.06079069765081854,0.049955366560768426,0.06645454912157636,0.06637898823367577,0.065960598380127,0.06778221114677582,0.04616337682350104,0.05339057574877974,0.05743266572928246,0.0647805958083004,0.06046742261460558,0.05974443447776159,0.06733744901570954,0.06502717797676541,0.06622835481945315,0.06969422896538086,0.041798292570756235,0.06063200433501861,0.07322430549498367,0.07698102807731994,0.07473568252087814,0.07256465431569369,0.06312492668112396,0.06513658893427143,0.050448083615378204,0.05742374047369234,0.06366449296304187,0.06490289851525471,0.07089445114181347,0.058013597866627395,0.05976234070551502,0.07217464451216664,0.0712902291905525,0.06633940398444238,0.05973842988097183,0.06128345128010673,0.06175318510188669,0.06285862585024102,0.06438445152355732,0.061254178317914125,0.06623695641778635,0.062355270286215306,0.0635423973186561,0.07062395365451629,0.0549431169581132,0.059562701340558244,0.06266530081823474,0.06989398724801157,0.05331280551940731,0.05209716393308478,0.06595150781101208,0.07092866191735633,0.06075148690966077,0.05206595926530845,0.06533550131595936,0.06425602946822451,0.06923358616671368,0.0613588568661896,0.057148658748849376,0.059176932795595,0.05630451052499441,0.07113175076921682,0.05981155931261618,0.06657279002755961,0.058746405973868716,0.06939497371239295,0.07069758762813633,0.065833846774042,0.06504483588639678,0.06252879989493562,0.06747869208796453,0.06147256530987846,0.06408725936769188,0.06762246612906211,0.06019485240707277,0.057161709295050524,0.06825247219163941,0.06883068264215172,0.07132447361209432,0.0630382239990967,0.05733008309255526,0.07207635801555129,0.0657700441382168,0.06055923060584986,0.05586571412116585,0.06162176873854779,0.06279596729876373,0.06600051529424125,0.053782278307898876,0.06801214976538296,0.06192791191934192,0.06433850628969687,0.06239316068414614,0.061864879140794325,0.0584377121437261,0.06473100798910253,0.07035064552124604,0.06384060013336718,0.07118454067034308,0.059675112154435175,0.06285169370317475,0.06728843371344348,0.06432601483107867,0.06714836090078471,0.06388486648275865,0.06424745675269858,0.0713815772925761,0.06812324642537615,0.062081521777547694,0.06390120116788653,0.06553292048173615,0.06727485980853223,0.06484660998932633,0.05222348650384585,0.06319872925017567,0.0714693661530345,0.06189762814389576,0.06039776112428315,0.0642882427023456,0.0538623729684761,0.05980167747108514,0.06270034757017455,0.07171940821203152,0.07527743709132134,0.0606141111274624,0.05560613385466509,0.05731842970463472,0.06384297922832766,0.07493864980339807,0.06895607606844184,0.06545049162758074,0.06246858651313695,0.0652807810218198,0.05561793146575052,0.0579811859463573,0.05970440416628582,0.06449951750195937,0.06545706425046746,0.06199633857370981,0.06469985418791543,0.06297562810675711,0.06764951411330491,0.07302993395569916,0.06716112220504525,0.0687067527505901,0.06568385581739339,0.06305938226060935,0.05735490503151703,0.05973720602173122,0.07315595111737165,0.06208350433231765,0.055991769492529764,0.07012914099280984,0.07111481486891003,0.05506336410531158,0.05907071943591206,0.07649895426398182,0.06796865373779554,0.0636031535737599,0.06410042740011117,0.06130444412529212,0.07468138646502469,0.05843307605364207,0.06266160872058324,0.06493902411844771,0.0570719031176104,0.062117746720715177,0.06550964673627466,0.062224426836461434,0.04840938712679256,0.06217736517724228,0.06959994812235899,0.06943619800199077,0.06019709167508631,0.06744105174306542,0.06744796101445591,0.06335734141005359,0.059325919755267145,0.06449757343930168,0.07353535505217544,0.06368001156537748,0.05408689858764998,0.05938682224910061,0.06370125596593715,0.06760822525144061,0.06751134170236814,0.06550212766098659,0.0563773237436182,0.07195575805828025,0.0605204280759793,0.06189913637036352,0.0699786645113554,0.060551980578094286,0.05558989534439,0.07447587988750687,0.05606776834149186,0.06812412177925858,0.05707920680462828,0.059479859006328475,0.06327312602487306,0.06693420657164402,0.0678581481203324,0.06393240769676109,0.06867743859408501,0.0642560099775257,0.06316810366225231,0.061257577106018854,0.06999524258275368,0.0662852287873541,0.07059553020419079,0.054954515915505625,0.06548118676419393,0.06474372448432048,0.06612844482238721,0.06467999854961709,0.0635745328512479,0.056817944459131284,0.05046423525239738,0.06695640951686402,0.05589067083892533,0.05773001642167445,0.0642980387673561,0.06779163340025675,0.06357483525475813,0.05324126483973007,0.06595158127801712,0.05486024541702157,0.06282845078463242,0.06066506046539645,0.06459222119851535,0.06323198575417241,0.06695031348603582,0.05573474173601995,0.05303486110240589,0.061298323317628974,0.05976133966124797,0.06453659530040719,0.05191210023865773,0.06094499302277408,0.06166495717180532,0.06925514595448237,0.06688718608996565,0.06144857234672862,0.0645007575358465,0.0576952630088432,0.07270080162994258,0.06623635783012358,0.057282022895163084,0.06020835164444911,0.05009133993434667,0.06427367884930074,0.05878333792139204,0.05772634320642078,0.0645050801878922,0.0665090698659635,0.05578842654709082,0.06308185328307385,0.06694723422664528,0.05288361625725748,0.07029191384787291,0.06790176113941691,0.062472701852191316,0.06462106685455005,0.05819701582320429,0.06690895460489622,0.0581038514722807,0.06231084313221435,0.05989751609079322,0.06831668150437195,0.06230124047499682,0.0567598921879873,0.0661604206055092,0.06660112569892723,0.0647821038162863,0.05451427692180405,0.06329865438284929,0.06059509943417919,0.05237273121680015,0.06125650202975422,0.07072637082338393,0.05715650267277324,0.06015406387965154,0.0660353393800946,0.05761365778553382,0.05956483987166473,0.05310768937378854,0.05767453025121139,0.058883589624864854,0.06894077533705666,0.05662142361420895,0.06825959276145478,0.055607983757287746,0.059587303368421565,0.06714282935182093,0.056730743517220156,0.056405216896912254,0.06678840188440567,0.06966460581492785,0.06894388206477498,0.06889713488899192,0.06701075174479657,0.05304027066873303,0.06753715705333083,0.06818619447050536,0.07228212664880243,0.06287804472968148,0.06510523460024406,0.07037299973448688,0.06207084634791634,0.05182543811819014,0.07061627070453072,0.06731287276732809,0.06375781859526142,0.06371948143389997,0.06356031100438517,0.05870338926903083,0.07188561882166891,0.06243052587836517,0.0606436677182754,0.05802602747085492,0.07176269518017672,0.06439290834470579,0.07107953638241035,0.06717100177035779,0.06737851814559111,0.06258270279323501,0.06762868158398185,0.06796325113891663,0.0552689405015486,0.07401842107674969,0.06352945084815417,0.0640986417350013,0.06192081979331376,0.060227799029816384,0.060797098029338116,0.06582503766890194,0.07028668174907744,0.062245134640626636,0.05996859028292513,0.05331610221342918,0.06472011365991859,0.06914363063511061,0.06346701971227423,0.06379552377262745,0.06911503441038166,0.06433262209673514,0.05951489598351574,0.0637687912049507,0.06013919676294974,0.05452417261943822,0.07056612864419969,0.05977039146181635,0.06312681809086687,0.06314215777234848,0.05360541431234135,0.056885888529971215,0.06760446435262246,0.06127601518000869,0.06081903747673782,0.060442343573937324,0.06155788371430887,0.05756599688736655,0.0627898118427764,0.0577045249592396,0.06826930032299175,0.07172049891476413,0.07210015668538916,0.05920420926574964,0.07146217137998719,0.061853255326277105,0.07259623218521692,0.057703225252616674,0.061779334667521106,0.05556524864964272,0.06073420829261929,0.06256901204775288,0.06481988117422774,0.0568674789678872,0.056526902176762914,0.0676760501894603,0.06350087890742646,0.05823381553470925,0.05906180115829147,0.06398415001098773,0.06895223096728859,0.06137240939438878,0.055800171952460925,0.06445593904415245,0.056887156815538395,0.05399154726855141,0.06633833429675606,0.062001855063499714,0.07151337103704114,0.06742338915872645,0.05712746295638557,0.06285484440043985,0.05570824031029126,0.06379156032149362,0.06409996807316205,0.06424980077153242,0.06606272268680606,0.06294213420884179,0.05577010130716849,0.07364961105755075,0.07337643071372216,0.06200784792018478,0.0643058604528966,0.06525029184566813,0.05972160015998491,0.05468025560234911,0.06540005096936896,0.06749212179988856,0.05274656941116167,0.06602606850231382,0.06990360256379508,0.0545743470321765,0.058192938487525034,0.06818863979829448,0.06726009959858331,0.05612433752907837,0.055707190013185956,0.06299158913848367,0.0626682553618443,0.06689646169977517,0.0655796736148008,0.06125164080173259,0.05718702914184731,0.07223303386074906,0.06757216366042568,0.06858766292778601,0.06316908103970426,0.062008735756864185,0.06677000370749261,0.06977060757751552,0.051649774086980786,0.0584593864828835,0.0585218395292208,0.05999925561227093,0.06674879930324512,0.0561622521436235,0.06182136205659826,0.06626356654743507,0.06268918772190005,0.05849487828068145,0.058442930365643256,0.06722964534388547,0.05988180573701125,0.06966448745013343,0.05497367665671324,0.06610320364642536,0.06661404370449946,0.05840481676175087,0.06988951398442916,0.05580002776231646,0.0695134452291532,0.06187529219041589,0.06646789207725141,0.05965213897628786,0.06981693038862671,0.0574472947409317,0.05958664244223877,0.06511765888712191,0.06573882540846482,0.059915209189847884,0.05686105651938461,0.05953229393462206,0.05775811085827388,0.06133747573704361,0.05948164700978746,0.06068580007427478,0.045826782831172745,0.05649708774400124,0.0681541361587562,0.06523455302344383,0.05814623262410296,0.07125907551642616,0.062003587340093556,0.05652938936754891,0.06024735611766466,0.057988294059868974,0.0714411769826868,0.06980792460721078,0.05745121652919034,0.06694263035525991,0.05066328985506599,0.0649072354171462,0.062073936757327725,0.059582515695146665,0.061478887006412956,0.062133383815076264,0.06410470111409408,0.06650852697202561,0.06951541437082429,0.07017863704646202,0.04286296456773608,0.05912100374979519,0.061619964859274644,0.06748442232455475,0.06441622267079347,0.06387484394831877,0.061694684283157594,0.06337489533915744,0.05845003182154483,0.068548929713527,0.0667132958064143,0.06554276827797588,0.07153119671453168,0.06494492248939282,0.06251643229047522,0.05990819005267234,0.060531677478645726,0.05989203214531728,0.06181227773497829,0.07025718612405116,0.06066013989274989,0.05732489471249059,0.05780547850633619,0.05694450459279782,0.06906920975450007,0.05708338217846265,0.06169831804111819,0.062045058533877255,0.068717611405535,0.06241864734280701,0.07090002179809043,0.07012280610576369,0.0536921700114539,0.06074795209360044,0.07620987628114004,0.06473935516809334,0.07094504229264498,0.06096029875763429,0.06701316334122112,0.0588903564891525,0.06365706832882921,0.0553906754821958,0.06745722065431871,0.05841177136126297,0.06811392077903047,0.05491887214251814,0.05726048675803997,0.07363282461922616,0.06613736139980778,0.07085700042542581,0.05716517446866723,0.06587153529729237,0.06411982139978312,0.059219051918051976,0.06400408918067416,0.06489776870120648,0.06838423272938779,0.054327045969457304,0.05866710400308725,0.06281543031826553,0.0652956467497132,0.05883815121710492,0.07464952846566607,0.0643426909500701,0.06584744248677266,0.06627362850027634,0.058959058646975926,0.06174427930809357,0.06330154026512577,0.06049525380468533,0.06652071050324719,0.06596126185294689,0.07260304290600647,0.06112578523965693,0.06124976389246049,0.06086990324680218,0.07055900787464464,0.06449853842499764,0.05531315041232249,0.07405313954367365,0.07386785462449451,0.07116376728039323,0.05076064337867392,0.060726679230557745,0.061566838969528415,0.06626654324966846,0.06748720529392951,0.054291344435543246,0.061117412097009716,0.0641491870122137,0.0511020691790674,0.057319356484226244,0.06289267403334489,0.06040146454675269,0.05965219577098087,0.05823104498680412,0.06456359112294002,0.07356111554455044,0.06446259185893845,0.07239120214136632,0.058193964270633544,0.07184452462501967,0.06770464834641343,0.06814900146078112,0.060855867453478936,0.06977335378454272,0.05840765825417302,0.06454417153154106,0.06760219761431863,0.06279208256494292,0.057075739402376646,0.06783359578951312,0.0589222500059606,0.07774189871690657,0.07228651384543834,0.061312143985163305,0.07013885936012394,0.07062048830774442,0.0703278405591935,0.06733828261612242,0.05982205669602966,0.061122759638044075,0.06791635542618543,0.06593697609740816,0.07023382861202582,0.061182541484732136,0.05942776104018049,0.06466815155880296,0.05921770265511943,0.06667239564987339,0.06660421830453438,0.06362517235143589,0.06098203025616017,0.06773004178847145,0.06232902453709185,0.06402892964865145,0.06087469274655781,0.0559389586542081,0.06858870847976006,0.06286740631330069,0.06714918291065514,0.06812328623145456,0.0666199039548177,0.06529045751766616,0.06343765044302432,0.06277116602316989,0.0647913778886165,0.06555244323425846,0.06926377104546964,0.06142385928120993,0.05662099828871553,0.06622396766273173,0.07345296557152625,0.0567911074016387,0.06363335135583832,0.06437662194485766,0.06984556495286551,0.06443831866906019,0.06553019549238011,0.051582640403075655,0.0725260605171816,0.05255049376090143,0.06371910625475931,0.06543712842546595,0.06766582245378133,0.059026930971198405,0.07062026670481042,0.05991439185945942,0.06454155446911955,0.0693744599652185,0.06324561801551937,0.05225501412112539,0.05898273178415292,0.06860951826107597,0.06079593286238433,0.05780187894903491,0.05729755253766168,0.07621864034275805,0.07174691985614129,0.07104366788754365,0.055647736346409515,0.07335819438156263,0.06287625450199782,0.05488600641809405,0.06576130369565397,0.06875307743674575,0.05674207188355873,0.06495131388026892,0.06044678805053714,0.06029447009724895,0.06355066561221637,0.06552084942140235,0.06522937704085605,0.06973183260288154,0.06926929865032805,0.05107807601553416,0.0689514621625151,0.06596670358167278,0.06795541377431102,0.06548486146648994,0.0718142030588137,0.0602592206192195,0.06147292132426573,0.0682068032494367,0.0639874391261735,0.05896122272623132,0.05442833529928654,0.058730217261744766,0.061831390986190504,0.06575607315187391,0.07162150507110099,0.06689818659834132,0.056526382873243824,0.062087516741327085,0.06396393826293532,0.06373616748527494,0.05503322082299414,0.06529172818273737,0.0574110389800544,0.06637530662080708,0.06809634415843091,0.06893394575796051,0.06443633526355058,0.06801687127328718,0.044576552059240014,0.05899907657326082,0.06532965207555733,0.07113551716009048,0.06906903346095296,0.06281435565641068,0.05911827960421437,0.06138659444510635,0.07424344863153184,0.06770228445297444,0.06307656444033907,0.06183250261648705,0.06820930388785998,0.07293760911892977,0.06690031477106687,0.06059958077732548,0.0661450491801279,0.06487004178190636,0.05962259462892559,0.05894342003540558,0.06265800970922952,0.06001661621487103,0.055865148182366695,0.06447989810609497,0.0692517040105877,0.06043100730118155,0.061253009802369515,0.05674943395342451,0.07353574312547172,0.05771448299413779,0.058381022138953166,0.0699273303172391,0.06778622209581225,0.060297434748285035,0.0647776018887011,0.04877228715974044,0.06820253796887618,0.06708941528152156,0.06771168112964247,0.06726976500906126,0.068056205654527,0.06599259089409087,0.07087038005824184,0.06730314184115085,0.06109545812956518,0.067740084838792,0.0645780283186206,0.0527193090002505,0.056785705816912974,0.06217757810413262,0.05834096140129109,0.0723335135924024,0.0677722135175558,0.0606986959999406,0.0624065266059168,0.06985785962639843,0.061849434997608255,0.06318272330314947,0.055814308507906286,0.05874238726933996,0.0681526054619154,0.0707335101871254,0.06534141793574028,0.055149911801995286,0.06996218769091578,0.05782715777367376,0.07415065816890332,0.06226016631855683,0.06865870199325996,0.0700485286784486,0.06954659013737349,0.05963422352731094,0.07040173762954871,0.0598739519785485,0.05165172305580348,0.048913219497088906,0.060906743421780804,0.06299201886246938,0.061358352935537076,0.05124923478284956,0.06190580648504777,0.07205095258608533,0.05359391610367753,0.06696263369445057,0.07035935698270702,0.05008537447182393,0.06014681385679008,0.07169298886988712,0.054326305820992155,0.05878256383285692,0.05421575520874321,0.05630340425361019,0.06407116085637546,0.06825586568325516,0.07026446532891387,0.06833942369912413,0.05712801231971438,0.05817182082408792,0.06360101368722179,0.056873669038541866,0.055976938322063144,0.06671002683041052,0.06593819075398269,0.06991781030187738,0.056579950583172986,0.0567775423155745,0.06552178364781755,0.07098520306860573,0.06296258147580841,0.054177746679888894,0.06565903576231824,0.06617495013225613,0.06895239874848028,0.05604557783702596,0.07224022376813535,0.07025059040229441,0.06664807961106414,0.05947926466536978,0.05752883487097611,0.05739549479551037,0.07545099674345072,0.06812371570092815,0.0674407135940206,0.06483182800756922,0.06907529469381982,0.06622868765055957,0.069926578192159,0.06900253501786886,0.06574033117543493,0.06287518138781717,0.07280829910864277,0.06620439608035743,0.06686822135955656,0.06257015286205594,0.06862605333564936,0.06293321588546952,0.06171267507276003,0.06035267135231483,0.06590967293289851,0.05681880978729629,0.05514508615166889,0.0581111808167831,0.06911673695785703,0.07138070362272766,0.06516036690057644,0.06103901932174566,0.06663230735005024,0.0723589555587188,0.0659044762228389,0.06956884296145968,0.0702213932426651,0.05679329456503736,0.059018706039610985,0.06648227861527352,0.06633247900860711,0.07472966065986303,0.06943978917428417,0.0605440393770421,0.06780907159391844,0.0713383071413913,0.05937347131909238,0.0626282184374379,0.06003329049359571,0.06930611608973833,0.05835023838355553,0.06276307412578508,0.07083116455084809,0.06555705282099797,0.06561247342506388,0.06638261385775635,0.0537057374771884,0.06301606158825043,0.0683250112116611,0.06334588054097026,0.06675173113494039,0.05460446998140013,0.0702490324069025,0.06344223080232839,0.06119079335930178,0.0664398549656186,0.06317929585575738,0.06523212239062204,0.06924860270065444,0.06852380048455683,0.06181857770674485,0.0652469645461169,0.06630602000664665,0.05603699606131622,0.05623957899039127,0.0650718594086438,0.05933602141813877,0.06649720793168476,0.06773036899680719,0.06293971333074411,0.0618531425487519,0.06012174440385918,0.06302681189138562,0.056879604539796874,0.06266922885821977,0.061534057657117336,0.05622836722282282,0.0557330631641819,0.06439199725952205,0.06456143445340742,0.0734709971236953,0.05010965099856454,0.06712705200889447,0.05714795270953084,0.059495973256648745,0.06575887773658948,0.05640436741838428,0.06733781334538924,0.0636434655545656,0.06628837894092451,0.06294757083646652,0.06579764402493335,0.06510156445352,0.06544656921081005,0.059776862306977846,0.06152292800039593,0.05740712957408639,0.05966924129346555,0.06770178782717387,0.059476531712549796,0.0636430660209957,0.057505972489544824,0.06252085663042065,0.060020267409982904,0.06696687805185075,0.0609395165789451,0.07132153808046542,0.059258414604946796,0.05246148885977092,0.0647484420620571,0.06851265499066232,0.062437580482753184,0.054609776951975005,0.06803034422909024,0.05914298651154727,0.055357436702826446,0.051245887427730084,0.054417597652754626,0.06013171665838064,0.058792137687884834,0.06754310771184532,0.07150809949025941,0.06203036683190443,0.07068668177619594,0.06385281637740053,0.07348069161518125,0.05797509538756736,0.05721390543812853,0.06334202057324803,0.06228900353284977,0.056842474693485336,0.06811930151359427,0.06522947455454994,0.0722320265631392,0.06408446835287332,0.06987983231189207,0.057058386049768844,0.059297857919763455,0.05832532618142533,0.06433055974539156,0.05883892004601852,0.05979964302166646,0.0691665717833902,0.06471834116260945,0.07629578162952866,0.059527136023631536,0.06994245229123419,0.06392310746756846,0.06414968433608999,0.06897698329788711,0.06507847921685014,0.05778229816233982,0.059343838743060935,0.07045669483473696,0.07247426303419938,0.06037936630235277,0.07342216885820171,0.06195986993430625,0.07038658698516026,0.0641844997346063,0.06071351156944536,0.06263532068260867,0.06887431825867016,0.05566819628861823,0.05290266825312352,0.06663391706079963,0.05628553584448075,0.060400064218652245,0.05510539391375491,0.06193135722873573,0.0533749984065625,0.058133273336355346,0.06742706878980943,0.07352321885929738,0.06356562682275635,0.06607471671249288,0.07237732028063144,0.07183064288934846,0.05521381659343633,0.06547809619273266,0.0545638105609169,0.06053931769211896,0.06548085981387286,0.0585845105660603,0.06621029428383027,0.048337320706064085,0.06293715878356798,0.06977143342712222,0.05951288267789416,0.05921212041053674,0.07135304835412509,0.05088049467697791,0.06592514925489326,0.07381287906358436,0.06772395801397832,0.06224045463942981,0.06318317997612014,0.06831400054813795,0.06091947289638455,0.053111216478765905,0.06786408645647976,0.059699261883525784,0.061326616223469105,0.07064181563359333,0.07012985466703192,0.06559583610222361,0.060019199205237725,0.07340846732777112,0.06636605661212505,0.06054073443550409,0.05868839607918509,0.06288570775727204,0.06013952655249445,0.05933739400567968,0.05388639961444251,0.06236412851318617,0.06648807562099215,0.07186093922209469,0.07299021679507756,0.06954363816219178,0.0690320038466058,0.06438377189885781,0.06245579672117864,0.06495967493489857,0.06237555944954799,0.06309939055373337,0.07075078062551618,0.06827076488607439,0.0679229908560543,0.051783255249330334,0.05251771401352833,0.06619785534124481,0.07109144004889395,0.06830366088465491,0.05517024139113201,0.06647736921547966,0.06914394629288678,0.06284187671908983,0.06790618733400819,0.06310901760776769,0.07111562792442641,0.07072707288105987,0.06877992485320553,0.061112530506542515,0.05427236089161348,0.06721490820628462,0.05767287258918574,0.054592226815133434,0.055725149025893775,0.05737931928072286,0.0670000808944631,0.059583321227115134,0.059324131778142775,0.06893782076510224,0.05678633267711258,0.06597736190007439,0.07284304506340511,0.07199915871512924,0.06296124370715928,0.060629062537187146,0.06813922158936875,0.07513442546671756,0.06391387827669194,0.0626866804894523,0.06713090557771395,0.05310855613807123,0.06519508511372903,0.06657311367347732,0.058639762472341025,0.07062061016055526,0.06186292258320079,0.06378705638208215,0.06253298194847415,0.04363090337143403,0.056334695326240004,0.0581748404474509,0.06559094527467274,0.062335882591103016,0.05435265059244454,0.06845000383747502,0.06362721404665797,0.06416044331045963,0.06847635976927113,0.06577658167341427,0.060873787438885274,0.05597210794817107,0.06490199368710356,0.06250604000400609,0.06914691235547006,0.0651724360223174,0.06975943689930983,0.07071653718814486,0.06697057882951657,0.06488776905994185,0.053755787462364577,0.07040425948592766,0.07044084190282421,0.06308402779254085,0.056140477689245905,0.07058313901333665,0.06604516839506788,0.053382612737730824,0.05277309542620427,0.059315077310242714,0.06005304589429579,0.06692303226078156,0.05529002298450553,0.07154157828710578,0.06121645386912909,0.056969567408484814,0.06330131890375926,0.0736792206337118,0.06428460978425896,0.0655202664972359,0.06860346460108019,0.06822715268668829,0.07418525268888339,0.05670510041354085,0.07010625106306535,0.07075070958355004,0.06891662676431073,0.06586106954713117,0.05419690413847332,0.06530257879740033,0.06798871840202325,0.07092564248370711,0.06100051574335111,0.061515728135650136,0.059563601100011826,0.06717276680502061,0.06123448047362884,0.0693164412373738,0.06381061513787195,0.06260760721005851,0.0700717525938549,0.07226241048115833,0.06523043143945426,0.04766378637415325,0.07027212389665499,0.06043880863499794,0.06700175499923122,0.06886096013404246,0.07058720878057467,0.07140674476001113,0.06465741581703024,0.06210662868121468,0.056600437724261085,0.05724715858735199,0.07018086315007582,0.06622242788677488,0.06917674501350252,0.05266631979284375,0.05693723322310287,0.06950352568265446,0.06218021346030736,0.05521318284693736,0.07270940508214702,0.06554901070381883,0.06357116281852895,0.06077995253618033,0.05577622783304949,0.05925274292854669,0.06894464725869276,0.06063591210488911,0.07321534925644924,0.06667924238093187,0.07120688093266841,0.06068622570900542,0.05640027569808823,0.06831638948623489,0.05162099340523668,0.0605315099755413,0.06355279452961735,0.06831756069899633,0.0685163023077838,0.057947890984105035,0.056538876430548675,0.06014245903833555,0.05845622463166484,0.052576303312752495,0.06120031729060091,0.07394340650853724,0.0668429765785804,0.06787667728726399,0.06670806781631088,0.05551037405915907,0.06539096941256056,0.06371972103322363,0.06895568882192504,0.05216444092913893,0.07067716984320294,0.0651636399224842,0.06095078387343392,0.06411472676250424,0.06764890130053017,0.0639308314751654,0.06514197498976498,0.06438689432070914,0.05955137271583065,0.06071079885413322,0.05258277017594719,0.0720811396706321,0.05694271211835676,0.06932235682568622,0.061744837491594026,0.05877837858853408,0.06756235198581712,0.06256841227701995,0.06917803202926251,0.059547389695008196,0.06219344911024479,0.046981048569894136,0.060802360602283424,0.06529122827874989,0.0665940463998512,0.0614195806760325,0.058643748065144836,0.056578094880415,0.062257934836746805,0.06888931007587207,0.058955957630598646,0.06436193143318349,0.06456549725843426,0.06617292610812206,0.07030672110672503,0.06752226295265448,0.07080161401242406,0.05700728121868771,0.06501124373736636,0.06328853248488248,0.054866156866047,0.06789342034540928,0.05832732804158651,0.05323548122332265,0.05335885107354192,0.060970948928386406,0.05430488249489486,0.05314618854655222,0.06093626882742343,0.051536820571837214,0.06248862172706871,0.06929275032776741,0.07129168201472152,0.07428584898064608,0.06279568800564456,0.07093111897781827,0.06454826123785082,0.06173239969539126,0.06506777318110452,0.06796324365142858,0.07225887349173742,0.07158560569732875,0.057688440673170346,0.06305837151666008,0.06362815295688189,0.05748169298165354,0.06626006112346802,0.055549754115769186,0.061859811681153246,0.04969668249720623,0.0682434184149123,0.06679496820081757,0.053368859984766856,0.06326157501958497,0.0725527769631637,0.0593174806708338,0.05708997899557644,0.06493400573615773,0.05192286170743694,0.05842066636875405,0.06008218427656227,0.05875018273254296,0.057778467785280174,0.0639462338740078,0.0640826681181918,0.051040278930339994,0.07089771636297859,0.05869105460348664,0.06373650645729659,0.06382520334138804,0.0669123063754111,0.049797492878700264,0.06148249353321362,0.05812572941294798,0.06660097806727255,0.05755481282865413,0.0647046548815737,0.057426008960244225,0.06272094654238505,0.06543688731824444,0.06961876812229743,0.06513675387001217,0.05344697302109283,0.06924941234994848,0.06618806632990012,0.06044276985081951,0.0732768241139679,0.06315656091430422,0.06678402606039752,0.06203165634508134,0.0674882062686343,0.058874636934373864,0.06285521757519405,0.07344558979763449,0.0503041709707959,0.060524241835173784,0.04754655653883699,0.06399411846516215,0.07010873658869463,0.06439437373406445,0.07340939407618215,0.06617478434285448,0.06300147551051119,0.06406234346876957,0.06333190311520986,0.06409398954522715,0.060684572092415304,0.07279165895276675,0.056246895549799426,0.07161673360706772,0.0718954888719236,0.07212953862435018,0.06792943735162142,0.06444001137551615,0.06729427132839255,0.05831494864579338,0.062087853011302103,0.059143999757759885,0.05872438577641016,0.06336948271224879,0.06507834022120898,0.05748060441416271,0.07191287836109095,0.06907664464018146,0.06033079090179315,0.06511617789281172,0.06344749211241835,0.06871342524817892,0.06410302796300259,0.06558725183692643,0.05599828747456677,0.05748150225664464,0.061633174876735565,0.06194303137098552,0.06449872670793534,0.05836730451509865,0.056869335166409865,0.06749740869466811,0.0613120794261503,0.06615991369642212,0.057644025849181026,0.06575471933506757,0.06540531832533514,0.0605567870283706,0.0643628257839246,0.060983486863016315,0.0616936187268266,0.07068129919024246,0.06460727609012104,0.05558320312120643,0.056426967919201196,0.06892315210881794,0.06593700090089093,0.0596269266900979,0.0560813282046604,0.05062917229432558,0.05929131932544203,0.057010963835831804,0.06187821807795389,0.0585087969317907,0.06781844217958208,0.06647807382309733,0.07126763301861834,0.056118815402961375,0.0632846702586557,0.05712395122455441,0.05789819022960428,0.06293703423823559,0.0639329289499298,0.0612673702605088,0.05750058586290681,0.07147246771103785,0.06781168365515094,0.0687444713937182,0.06336508788952286,0.06905208765918079,0.06032958279742959,0.06937698143812673,0.05253076487432977,0.060092546325861654,0.05098358015062233,0.07326133178047595,0.06147837702194701,0.06346050608152688,0.06563680740621447,0.05518781748024651,0.05513863764732618,0.06691225714344803,0.06457413477388732,0.06973179921550716,0.052376673665095325,0.06596062535308314,0.05452046496685191,0.06341383015887435,0.061729489405651934,0.05919881557899785,0.06823819812266112,0.07167828475857634,0.06537645884875212,0.0652903045310721,0.06597643688666444,0.06559479026202783,0.0627314716660138,0.051362837364195287,0.07475296402210149,0.06182689471818972,0.061034370109656545,0.06273584848739774,0.0683368022235541,0.06496153882095396,0.06735960836716078,0.05822003091423862,0.05430799196925033,0.0620759975574861,0.057647047159880144,0.05441984959720034,0.06688763650527385,0.0579796140984506,0.07046890137173988,0.06321674845606433,0.06049775906782607,0.057369808575680456,0.060379411856031,0.06050766849507359,0.05077468547640117,0.06981256473399121,0.048259026685145895,0.06921907088758417,0.05478733672160346,0.055822806189831625,0.054205507320222876,0.06682916084568881,0.05402923485661175,0.06508273087241873,0.06356859879120841,0.05732092575788787,0.0544316802580329,0.06656218668197415,0.06501346862851318,0.059728027107467135,0.06646356633873735,0.07093672562782707,0.05943291830201815,0.06959322563432291,0.061975196304862666,0.061125452428284495,0.0672057240901095,0.06596037884342215,0.063646720144787,0.06901912731366136,0.06015013747091592,0.06039062959473704,0.05627653889658084,0.05886591425173197,0.067168912900488,0.06385509641418073,0.05716496912901171,0.06949695591771282,0.06816613712746737,0.06915883095079434,0.06481794281867001,0.05992250334066883,0.058399551160658515,0.06712635458063909,0.0602302009528521,0.05452107681016312,0.06266923257614788,0.06367068065910042,0.0699187944817973,0.06604084459845272,0.0566452518450823,0.057561327758341566,0.06638880003152853,0.06375653042545132,0.06501971456888747,0.053616555294280374,0.06220025247009696,0.06683832381128299,0.06994478915247793,0.05685356196351664,0.06478794998030883,0.0734813230008994,0.06199551489807407,0.059681150861652515,0.06201869197430697,0.05911703288400319,0.059438655465102685,0.06331869584644394,0.06638984002197315,0.06345377899830598,0.06137204497142663,0.07087775760363274,0.05861343529949899,0.055200285678830965,0.0636125816919649,0.0662507903575845,0.06364276967472937,0.06299635726709636,0.05263619173309826,0.06415622022527195,0.0692577107122087,0.06156370420004034,0.06752032591739357,0.06203307287507922,0.0561686840818995,0.05310715323788298,0.06868404460148374,0.054367204333399005,0.0500694912329666,0.07025781498779365,0.0648118763923547,0.06946633476663322,0.055082305408801976,0.0661725760109734,0.06408268092658323,0.057248532871720686,0.061068797561650845,0.05944635842874661,0.0687990191871628,0.06530512478305636,0.0682298663141371,0.04953032683246282,0.06740451380319204,0.0635465390801267,0.06034188188394847,0.05454857608325728,0.06962348400282675,0.06858875567068698,0.05646524112921323,0.05876281810492387,0.06641485659223859,0.06990581141013746,0.06049602569156158,0.0657343187891401,0.05433906497788804,0.05884908437185668,0.06612836987506074,0.06685635976401186,0.06367914703399853,0.06921038091411988,0.06655397386912999,0.06711035855126538,0.0716726300399992,0.07325913398593337,0.06463228503737051,0.05307841261689879,0.0704299225572878,0.05459499316172994,0.05402077727857141,0.06068140660450158,0.0713004485357,0.051766557786744306,0.061789398925071,0.07126168896715462,0.05596088036806594,0.06069780207918772,0.05157155818190164,0.06776929397313493,0.06035790646362621,0.0567650176454354,0.05603819968024713,0.06286922338270631,0.06170781581265638,0.06100279875917704,0.0671702756653963,0.06312639142468726,0.05840707939041119,0.07231135016047507,0.06591217821802337,0.0608666491871422,0.057275941881821224,0.06065164227449832,0.06892993896505983,0.06596858089552993,0.06214068864367703,0.07044428422360491,0.06787463826633211,0.0649927281466425,0.07188165332957422,0.06311195799821995,0.05057329702201009,0.056380928212540156,0.059325969544264834,0.05944390907376393,0.056502695393201116,0.061975874058248434,0.059719220416488014,0.06508913744080475,0.05387938065279974,0.07061729681059489,0.058576823073520384,0.0636873522778153,0.06423983372024014,0.0608064514331398,0.06531221422443909,0.05117558342345388,0.064281270850695,0.06634475922471064,0.0561959474674699,0.06889058263405129,0.05149636078113799,0.07018282838659418,0.058799551414354306,0.07184191542086826,0.07294206304223545,0.06855947193014592,0.05453404333775335,0.05685108325407937,0.0576168046674707,0.06111868889120045,0.06904584858708557,0.06505956655131091,0.062170064434907746,0.06267609918694271,0.06711412043237552,0.0646668069036193,0.06091218429660839,0.0598852686771213,0.05840600762098592,0.05558711176826546,0.047129741886772165,0.056517831861973955,0.06368437514407145,0.06331280485290894,0.06154212464381575,0.06443287373738175,0.06995318662642556,0.06842546714250711,0.0755829532768745,0.06962125155858906,0.05889449964432145,0.053341078598546435,0.06129050173912515,0.05819827426587344,0.06243695254764282,0.05549763759587664,0.061326906416808956,0.05065738576579759,0.06177854808857213,0.06061654466067412,0.07319381646899561,0.06510019437504189,0.06904507378863763,0.07289632433954966,0.0637112677938477,0.05879151897376632,0.07376584566280751,0.06540111399970963,0.061480244202924916,0.07247883118436818,0.06401796834364934,0.06470982430621519,0.061733706605188,0.06280455072504183,0.06725027858976931,0.05239262231768597,0.06742800464099036,0.05260779264550888,0.06988310709024698,0.05520282348376849,0.059490647689683776,0.05227994575764104,0.06501683151994657,0.05797742559164263,0.04960363131529395,0.07042264366475787,0.061971548313104,0.0637791275699023,0.06767201146802145,0.06930037840227259,0.07465569676065051,0.06649159094182691,0.05501891831163478,0.06651990221176424,0.06292928584793084,0.06818614282429286,0.054689453626246026,0.06433964916780444,0.0670787635819443,0.05897035580587624,0.062360105820769465,0.05776059601922605,0.06488452308897993,0.06919060674205019,0.06649514046407899,0.063393961885206,0.06893630708343809,0.06266358289652849,0.05939629081005343,0.0643177574942061,0.06708503851206721,0.060696052569332526,0.07168569661777652,0.0648973034606233,0.06905549593497248,0.06871839995482434,0.06500218528453414,0.05759232798220663,0.06425458959423716,0.06735830241312421,0.0649587282548417,0.05231986633425844,0.06588200146100076,0.06356607153885914,0.05242042992476847,0.0645392791232662,0.07310428897698504,0.06118197075082277,0.06386399680797936,0.05425536964227194,0.07107025355897684,0.052402032612557506,0.05631792248841026,0.06614951544289557,0.05635733726193843,0.056238457657678514,0.06730163076405737,0.05466402062891952,0.06433568844587541,0.06716105874767606,0.0506155005178114,0.06566572930982796,0.059604501904773524,0.05653620615591651,0.061372377601148935,0.05823565326670453,0.05776334601946538,0.06481870392697006,0.06925207142133896,0.06520079579469001,0.07017607089328831,0.06875364282160465,0.07024435970524569,0.05971768179890606,0.07029502999604333,0.05520683666304109,0.05616750841222097,0.0739411472004478,0.07070992437946313,0.05923253474608643,0.06927025822275264,0.0688906869639068,0.0616000120213091,0.06510546562955598,0.05096545714502908,0.061678145617422256,0.06680120605632972,0.05570710118859473,0.05448224019840692,0.06546622261595178,0.0722539020311843,0.06636910611756047,0.069181965090579,0.06614037254772606,0.06997118289707854,0.06909154135761962,0.07051532082571643,0.06763616279136028,0.06027061525012138,0.06754011410567862,0.06320521546165478,0.06339589078403762,0.060509351986411455,0.061567445129895974,0.05675911445864299,0.06521192202927677,0.07075711768973547,0.06215740200152008,0.0683109913646453,0.05912858286837758,0.06709399464241875,0.07150865232892958,0.05904957449576498,0.06832213108511379,0.06111042029873816,0.0626848325829444,0.059513316841737385,0.060589944869668455,0.07568044258842622,0.05954130197162786,0.05981816691378784,0.07010387598986173,0.06527637366827468,0.06785486600636415,0.05491837030977163,0.06831023609297501,0.0586240736293722,0.05803015981961028,0.06936845007626183,0.07099497187485387,0.07166363721477309,0.06281370965497415,0.06811483106244695,0.0708143516689963,0.057408939841481904,0.06953749953070035,0.06300155048413192,0.05884300447159732,0.05669440698799162,0.06122755029682034,0.06732124482969479,0.06300698404787267,0.06081220285488665,0.04328789886637872,0.06480668557469532,0.050841230736547824,0.07019222359983723,0.05583483546170587,0.05594035118952537,0.061781452144114395,0.051495546834790365,0.049398103005863794,0.06679389848381823,0.0767848779308838,0.06393710677274643,0.0634909896465242,0.06760258708653852,0.057577972191325474,0.06268013946411452,0.06545735284919543,0.05429767586016712,0.05025530076428315,0.052411252830455585,0.06287848709344869,0.07156113990338595,0.06497000690086603,0.051035795951617,0.0713338244793828,0.06814176539084507,0.07393528488167009,0.06467490303330864,0.06347570182596068,0.05832485751616594,0.060350541814049925,0.055539259169430394,0.06226565281999171,0.06179532985711536,0.06034477525968035,0.06374699477182733,0.06934011939790857,0.06657942122600534,0.05433974671291508,0.06799099022628499,0.06692316109149257,0.06341382783371344,0.06010027816417429,0.07163321216182804,0.05247550408661068,0.06899931850042215,0.06612365581671689,0.04764662260546711,0.057990324233017046,0.05896352748347139,0.07029066416567487,0.06760812508990074,0.06069114838220664,0.0521771623648574,0.07353178522344253,0.06005079440939598,0.050856499201100205,0.05748951076975374,0.06763080326375306,0.06258623436216038,0.06181111962831456,0.05686628363045973,0.06727460746168826,0.061663386022863385,0.06620142178895247,0.06313671166713974,0.06021040743703279,0.05185527925023328,0.05727845677728366,0.054387799151258506,0.07383696616474164,0.06566828448743003,0.06313648710248572,0.06195904217030885,0.060126988602100404,0.07085639774330059,0.05510610132187871,0.06748228711560167,0.07017646677932196,0.06622841169903404,0.06288003353354576,0.07030534365973982,0.06565220436348179,0.06688449176140969,0.054279501729650084,0.06524550224892048,0.0534560966147112,0.07127067400804984,0.06458621900892286,0.07114254718901686,0.05565579485793612,0.0700280505003476,0.06216894749873782,0.06396394064976348,0.06675382916789943,0.05995128809213239,0.06011954206483841,0.054666242478827495,0.060902232831247016,0.054377063817829134,0.06572453561629968,0.06568086328128343,0.05914403954010504,0.05436129231036994,0.06220605719188892,0.055913532976449104,0.06306540805386526,0.06422958531807366,0.06691355640947616,0.06525250115828143,0.057610025743955934,0.07424381048109793,0.06419949454167484,0.07126133736746376,0.07406656029365741,0.06778638850777621,0.05613484816831292,0.06782138552295185,0.0631731669980678,0.05516452339714541,0.06538542416001342,0.059163398527638064,0.06933989120278343,0.06864241485667666,0.06739891463071368,0.05642205938048482,0.06680738348404347,0.06038582483677552,0.0569314483105446,0.0644812152303351,0.07128893960473359,0.054317185421616315,0.06881457007234718,0.06398826974478676,0.06994590680995158,0.06854234104739046,0.07029508376377955,0.06396677571925719,0.05999020911752244,0.06771608737402891,0.06130012147534029,0.07075920063602886,0.07497523353470727,0.06906963754698794,0.061029983216777675,0.06405654946217268,0.06723968205329361,0.06406078138803989,0.07061132805483297,0.06270092571108943,0.05631153619425888,0.06638843020072058,0.05931517530044686,0.052182936327703405,0.0610998282971346,0.05397579805364451,0.06174330394564311,0.06058856801012584,0.06517410277825594,0.07031804513604312,0.0636366041171472,0.06113359455641132,0.05939024073264934,0.06246008275182826,0.06619287171095659,0.07248541051653887,0.05638319611234874,0.06397874686279487,0.06049801374095995,0.06390599752337502,0.07066809622794998,0.0657622647432416,0.0587092141425443,0.05174578373340781,0.05902676556911616,0.05684315031463178,0.05472713203421062,0.06528562613382344,0.06812269762861631,0.06746321386095289,0.06751195084655046,0.06495952490521958,0.07524150829383816,0.06515029231581587,0.0657243767199666,0.06391174540020811,0.057973410769486325,0.072903023805653,0.07004211882154483,0.06793145201206155,0.05666130675393669,0.04985093912919573,0.06776873105055359,0.056836158495343556,0.059305891303748845,0.06698221475610676,0.06680301290700918,0.061421783925110024,0.07003706911067434,0.06910309017051924,0.06424079953430106,0.05590644143963651,0.06853233484547507,0.0697773795255821,0.06005841301374709,0.06811890357110278,0.06126528402136278,0.06682929209139311,0.06570453389334974,0.07062237985295866,0.0696361600525691,0.06305141773098935,0.053388280080009044,0.0720248443204382,0.0624985563683478,0.06273726746642838,0.060630460291390915,0.052683202356052596,0.0621167241314282,0.07154352083414314,0.057207361693872846,0.04653176632666822,0.056148784120441365,0.07608732424113888,0.07128542414773134,0.07032808147071162,0.06385261257007999,0.0728674517225779,0.0629501507534607,0.05887630203160142,0.06986998656307661,0.06224176186254827,0.06164339363180625,0.06735754429458093,0.06710238544080097,0.06802782119946028,0.055072610968560046,0.06961023599542536,0.06270015825472772,0.07138653783013141,0.06439790791564035,0.05270683104313462,0.06999319014166275,0.07393172859720121,0.06564612699882327,0.05403920542430206,0.05808889421760644,0.06589982330705715,0.05742887552599192,0.0601785112310832,0.06030667338738536,0.06640545767623851,0.06246774993841377,0.058074121631873823,0.0580952173433079,0.069928539567052,0.060179815511114204,0.06898640772724429,0.06273761183575713,0.055142054020525064,0.05331446776528524,0.06914680740521495,0.0691973934306516,0.06960309193821007,0.059833913289612545,0.07003454587422447,0.051232818532900584,0.06269626021406055,0.06176087292330662,0.07276430357181257,0.059412280178253736,0.0746637162538667,0.06596710157873734,0.06734931394924798,0.051890058440405645,0.05825865917369516,0.06424926111052076,0.06875006361253236,0.06376892573309664,0.06189611198652422,0.06272147094080525,0.07017719635356234,0.06197213026354678,0.061653090489961475,0.058890862744007934,0.0702017423992796,0.063343978391695,0.06524733264292615,0.06776226466532721,0.05941778400664571,0.07390576090783701,0.0638990533864725,0.058685822468940926,0.05519305546466287,0.05855111346024039,0.06538885206495726,0.06955301986918477,0.06554416991480325,0.06151121471287916,0.053158698431700235,0.06061552682335887,0.06327972551336916,0.07513047202974751,0.06493547047829341,0.06174881037701155,0.059423453701961296,0.06036808116213058,0.05776275639563057,0.05806384429283687,0.06192458025224831,0.06946369559821833,0.06907360610199756,0.052132980996106665,0.06808502873752892,0.053157701888893835,0.0626556558816789,0.05675747274265927,0.06350236062209191,0.060688726749855273,0.07084928153511005,0.05848243763184102,0.05979621452045496,0.061935326557777386,0.06222224903485421,0.060642299930792704,0.06046680607485003,0.053984030626706675,0.06244329963504641,0.054468378489037045,0.07584526369358036,0.06859588959179937,0.06819170393141089,0.06715663608012917,0.05995718839875051,0.0623458133239321,0.057934490180550975,0.07132099092977362,0.06359396561057468,0.062438557525806786,0.06891672170653744,0.05586001871517968,0.07018051072900462,0.05895539047923436,0.06275110069517992,0.06588936843533155,0.06903168252772074,0.06934122469584848,0.06998817903525809,0.0688036483865171,0.05764274901730494,0.07356043996337505,0.06232471365207609,0.07114236342187577,0.06482535436864459,0.060382260911177474,0.05751470670412676,0.07486188659598372,0.06240203678686585,0.07102589900723727,0.05878667317791178,0.061026202707746946,0.06974436998074772,0.06876961251862258,0.06767859111352414,0.05311258985847385,0.06187116200325764,0.05854005825415538,0.06264178407733251,0.06692874097397916,0.06059343107216987,0.05885355898150303,0.06058390038113354,0.05735899276189125,0.06436762635208254,0.0638446669411134,0.06879506328912109,0.06645251368254121,0.04750545395178767,0.05737540470463605,0.054913321241884264,0.05620676427042877,0.058281996766852126,0.06647379483545929,0.06652631657849825,0.06620952729596574,0.06141851100322677,0.06583054343282863,0.06711429464750636,0.0648715696505275,0.057604481446124955,0.059550538825228566,0.06774346954036839,0.07269079262406618,0.05052502316419416,0.062383161037562355,0.07146153753013862,0.05192476082018907,0.05892528866762779,0.053000007770379956,0.06265536191385424,0.057438662719220726,0.06902568795469832,0.06548406119949154,0.06542504834295201,0.06625813821496121,0.05604249681343024,0.06391544733120924,0.07557871162898028,0.06862260634327104,0.06871671515868732,0.05878477715766142,0.06020125612348658,0.06770777305360842,0.06238071135745518,0.06508228070425433,0.06277687445492558,0.05638170996023581,0.05663864273598626,0.06416887624300766,0.058232072167326523,0.06843958791091849,0.06389514443768907,0.06514351801724694,0.06894203245754923,0.06608497885292904,0.05759668721955659,0.05965681961454526,0.07317138906514156,0.05827835831244982,0.07094959253175523,0.06439426508717705,0.06510752716460354,0.06104372758982049,0.07539088430868146,0.06684390749398017,0.0640026137667486,0.06620494041923013,0.07014030720189102,0.07102412502483488,0.05012866326830742,0.057307259161623714,0.05830685774362583,0.0699723378639171,0.0659178518723377,0.06421658364143532,0.06850930343968942,0.06351003463604228,0.0601263532536396,0.06471249242942108,0.06731314040549119,0.0632428785902915,0.06933836112546646,0.06740385002898255,0.05579405334685715,0.06245944295683097,0.05766224721853037,0.07091854108491638,0.060072277165386684,0.0702193894759027,0.05886680616176236,0.07454847021060118,0.06753619175496198,0.06192878184500444,0.05194069519734906,0.06289998642548206,0.06737724611055024,0.06378299076306035,0.0698910345266677,0.06922191646667107,0.059299852592835656,0.05387304244857931,0.05996771739422267,0.059803926656850366,0.06462745272132481,0.053084227656229166,0.05760884068860646,0.06726995944798957,0.05858545329111965,0.06565161128751049,0.052143252525546586,0.05663911334714962,0.07201249722452988,0.055713757512937875,0.055001224377422595,0.057549084474617186,0.061461244026838675,0.06393731849901083,0.06944323124102075,0.07049738956224479,0.05091943150874517,0.0659847370706452,0.06887642704558052,0.06717993506232578,0.07165709668953245,0.058197337367605934,0.05802734798671122,0.06922082429404883,0.06427371628485826,0.055456550997586174,0.06727200673575168,0.057619528988622805,0.07249299643118162,0.05866625080004188,0.06940555216150877,0.056340940231896106,0.06310550809462498,0.06162790959336122,0.05950965595371982,0.07128776351674451,0.0627132736124886,0.06780280065546612,0.05908661331413462,0.06003581184592438,0.06135317753432548,0.06321521954504633,0.0668618267797358,0.06736436898624348,0.06481085574923183,0.06678476913338681,0.056158058098506075,0.07240530341410309,0.06366886524750515,0.05398430769023349,0.06795830691593538,0.05348684565316835,0.06485878070063028,0.06149453228292575,0.06298372354134427,0.07569890201290391,0.06342545447021551,0.07090916351025772,0.060509553421245324,0.05437041853891696,0.06744184108986889,0.06415970869614124,0.060313013049106814,0.06840503288581948,0.054150161775141176,0.07050116174305077,0.066108445170476,0.06420937081702503,0.07037047227230472,0.07167593917919485,0.06677806046833126,0.060045305928325333,0.05984948931419182,0.05744093701838871,0.068704235264028,0.059877869508819695,0.057879785188335975,0.06288805160587004,0.07068285175931574,0.06754656608945078,0.07345688538098796,0.047866090833253105,0.06263221541221059,0.06056728657560854,0.06408248791265375,0.06631728350975863,0.06283349393083279,0.06042966246745428,0.06713446893417768,0.06460375773674623,0.04428503957300676,0.06878367647282523,0.07412784926925452,0.06870990921108269,0.06873580619963912,0.05234645991187589,0.056555018113757474,0.06033704653041598,0.06288642936317446,0.06707223172219776,0.06507902133325735,0.05644007560975749,0.07076608888105258,0.059345041335781265,0.0767409843520156,0.07085111805435734,0.06448845486650037,0.07252506860445387,0.06390998743923682,0.061542347755356254,0.06958531816545924,0.06863237503584971,0.06714741502683355,0.07092067059214108,0.06558471019470756,0.06493292209117382,0.06523341045604739,0.0638520217384572,0.06279659726634841,0.06819659125634941,0.06075535421231241,0.0701576639062416,0.05916138169717576,0.06216296275563409,0.06650563477433505,0.06474085971233283,0.058909599183163554,0.06555601025473512,0.06057380396467148,0.06990186041671609,0.05936743835583656,0.056620109919831296,0.07246422133336727,0.07164191867003525,0.06769863482188859,0.06555755572163527,0.06673833873101537,0.05688085688498368,0.061031676488865386,0.07147305384730845,0.06498496871293194,0.05840663257251588,0.048715281823596635,0.06287228368834569,0.0682077455567244,0.0557601286647911,0.07145985108969591,0.06873712437152973,0.07485810892163938,0.06726849852456038,0.06535265235573459,0.05886398472480595,0.06655940821439055,0.0597481501408299,0.06031346610632439,0.06517889084332887,0.06225943198521954,0.05782489376657769,0.07528476178466788,0.05665309115638094,0.06251150361172256,0.06835105689585916,0.06686750952188752,0.06343678415416056,0.07295339724283265,0.0618373000866387,0.06481368003122734,0.07114339907294351,0.057074676529330154,0.06623086789164515,0.058691056947209455,0.06322138170652769,0.06156396202318482,0.06886772513486938,0.06336613864046955,0.0592022145168325,0.06689316539681386,0.053640803177666674,0.07065784583006696,0.07101959020686482,0.06569740560217449,0.0626698173434233,0.059001704710463745,0.0658019739769179,0.06267291472791992,0.056259528813070545,0.06877807644883471,0.06660411372328161,0.06447099534778689,0.06486135133956297,0.07553359957149644,0.06975998039173628,0.04984108779467848,0.0700475575076436,0.06997288566323853,0.07040128559499195,0.07170586227602099,0.0639856202213533,0.05747230756778544,0.0647677503369495,0.06965951190580041,0.06603526877235094,0.07307613326515164,0.06020447241273836,0.057887812848600645,0.057481150687102564,0.06279197895325962,0.06522395923677829,0.07048519595256891,0.06743824493914052,0.06762043347447463,0.07128212233910239,0.05611380648098363,0.05634580367970164,0.060203196749080326,0.06586168714882593,0.06365604563505559,0.053580795105730675,0.06438837191614313,0.06334383265601246,0.058653189224737674,0.05520196191207613,0.06829873361724809,0.06097210813548971,0.06284271887659035,0.06169677439104391,0.05980443652674325,0.06096683080748355,0.05692941779253144,0.06587462024291454,0.05449387227592705,0.062239111031077266,0.05656428654740476,0.0566794299082989,0.065270448229212,0.0537254534048698,0.0586620221477432,0.06505872096815835,0.0632700076216281,0.05954280555583938,0.06197306260624276,0.06886941253903378,0.0584531002390274,0.06416429292043957,0.05829363382445396,0.0668706204180761,0.06338279652823207,0.06034595864814857,0.06014160301969752,0.06591658220037061,0.06050230328599972,0.07323642717082669,0.06226582464679307,0.06021853849863352,0.06845733206416905,0.06128001581119296,0.06934440712815891,0.06132047399414552,0.061783761849478466,0.05765552848755374,0.05217125527264958,0.06530495515397261,0.06339613310090915,0.05551629679515789,0.06630226257190647,0.07058584124203918,0.06439300537605053,0.06763069353912274,0.0721342291512636,0.06619163042589776,0.05516947391048309,0.060968495977870366,0.06821199109149426,0.06058845474290747,0.06399559900452612,0.05694961761218817,0.04544165524585902,0.05938537013264297,0.06175778126893507,0.05696262939476515,0.06740337113808693,0.06694385659182714,0.05549532183024739,0.060384069615901544,0.06793600801856034,0.0598605403822865,0.05637732697208698,0.0722729444611703,0.07001335040515191,0.060326922314516085,0.06714651124918997,0.07252443179648205,0.07164008182801854,0.06770556849093876,0.06472742259429567,0.0642736645702017,0.06195967245846397,0.0720304363473461,0.055883505465116536,0.06726475836565939,0.058559604819444985,0.07147938662457562,0.052267657578712025,0.06642049664111709,0.06843081131237239,0.058764868011440013,0.06899349494684653,0.06022266692836628,0.056212660417942845,0.07140702011072142,0.052013795242818836,0.06559455315883311,0.060681260306886545,0.05870726776312405,0.0694413492071493,0.0612446580288456,0.05988320073915063,0.06127354645809038,0.07094428794186028,0.06150417384109396,0.06235908624632271,0.0643534264159282,0.06783289270133945,0.05781557873645846,0.06774976070934557,0.06388641580812743,0.06267719982317402,0.07271394678776599,0.06451598065505135,0.061288531187950614,0.053643136993052454,0.06501449020664309,0.07286553635080116,0.06557482670235254,0.06303593779433864,0.06954082687254945,0.06316352837957823,0.06744613625889806,0.05811111584813159,0.06296615040637338,0.05801542832422214,0.07299771382949351,0.05327991555074739,0.06511888522258034,0.061866556016870175,0.0670836083437358,0.07126156784578506,0.061006147713407816,0.06302767493447109,0.06565478819472961,0.06153501066949826,0.05546468353858978,0.060945070745755864,0.051303867868818534,0.06464114848170806,0.06733912482179699,0.06043386395066653,0.06468732298130345,0.059311623787961995,0.05646758087966666,0.05129258005819293,0.06328332827578162,0.0567776653363319,0.07256415969833013,0.06014294174746421,0.056859958150855025,0.056013618586864336,0.0649120383866241,0.07241126530893972,0.06724774353989285,0.06276138075853507,0.06005825433673338,0.055965236931213844,0.06577882487948758,0.0688312396009313,0.05234050473077978,0.05987567899346735,0.06538608204405086,0.07340769514196632,0.062064824273144796,0.06724365627219155,0.05841937079286931,0.06719392659689294,0.05724196356916246,0.050197809282698944,0.062053268210241215,0.06233306264309984,0.0539712381308428,0.05460283647579257,0.06474750257506023,0.05560429458051336,0.06264449847404091,0.06306650187899224,0.06959777611392762,0.055323604604379184,0.05788878945461397,0.06299742531482164,0.05903491068051721,0.06501272360150166,0.06810193899096648,0.05896992499051476,0.06871086289924624,0.06530131064139674,0.06563389933400612,0.05769829301776706,0.06930192421965137,0.060895078422032706,0.054708918640939935,0.057811621071198374,0.07114265314449122,0.06196936538739806,0.06095285242748387,0.06531903698882247,0.06621818621808245,0.05730780360614583,0.06383461388341728,0.05804097819961013,0.05911574053959276,0.05836934897835562,0.06365504690494322,0.0599428991245915,0.0665975112607001,0.06474071270239008,0.06302835965857614,0.05459257523834346,0.06404646406413331,0.06327980227899063,0.0665811828409929,0.061588326773060496,0.060657582109800275,0.0654703995230905,0.06962074269867183,0.06757924243832239,0.06977190691054352,0.06736681567423611,0.0643084246012427,0.06894123607291608,0.07208248438218431,0.06193589663684922,0.06735358818819989,0.054841258521003425,0.06130125524738586,0.06641106449703696,0.06318600643326516,0.0516064226516124,0.07211114021903847,0.0727130616989102,0.06045231477303424,0.05135613248173827,0.06417338724352113,0.05419889263226291,0.07299457649647087,0.06802569998966548,0.05427767635269694,0.06468087488462879,0.06589724435085001,0.06662746412391199,0.05857356142617824,0.06987724376133145,0.05898860825431765,0.06950405834988882,0.06294019320280418,0.0622963169489605,0.06336802055686634,0.05649734266746839,0.06821185925990035,0.06679535248458299,0.05789632413413784,0.06942912408518193,0.06730475266935393,0.06556245885069809,0.07186735866544805,0.06493033133361656,0.07027204579404157,0.06740225083144365,0.06980153408035242,0.06731767945702899,0.06303925010017808,0.07221719223177761,0.05854683808483799,0.06547268517880356,0.0720304617324237,0.06109664481152988,0.07444832209545718,0.057068685222778,0.06056972626732432,0.059648007679652795,0.06305972484531952,0.05990584818343915,0.061398208001069834,0.06299363692356971,0.07314496891875488,0.06303983955716194,0.051653515657934856,0.052994692702532445,0.06823223968543765,0.06443844754134598,0.06829255178957448,0.0680099116162928,0.06871289148823306,0.06909812690656675,0.058776360658292684,0.05914954684756236,0.06001727490806884,0.061817278105527305,0.0723212556364754,0.05950060434287542,0.07104734509873531,0.06869480352138195,0.06292364659670771,0.0599804893605449,0.06135846376197678,0.06526569325653957,0.07414947644828705,0.06719937525722125,0.058320772962882184,0.05678761262305003,0.06501823145508648,0.06687996898915904,0.06719971848953195,0.06968362714359477,0.06222797867901785,0.06483823126017838,0.06393098102460751,0.06994984255968245,0.06515582723127017,0.07061394470876203,0.07488640124119517,0.06473989558545133,0.06107085006225672,0.07034339225312042,0.056062576552900895,0.06256914819582096,0.05676098358465254,0.06962516228095256,0.07343163545201223,0.06394670274720973,0.07530318329380248,0.06118856085597321,0.06941859535129391,0.06973646603878292,0.06369627540738482,0.06548182228469643,0.068766690542373,0.06399647719686302,0.060500350919850135,0.06652183562598199,0.07274866223848284,0.06358644586469564,0.0628444305336734,0.05563798708155186,0.0688656826269636,0.06705465814049894,0.06781292083465207,0.07041673002115348,0.05577912126749142,0.06177601245013829,0.05455126548536651,0.056063782706002305,0.05521867824511542,0.069993503656966,0.06599685026629647,0.05882568651673606,0.054739233325032485,0.06483306007912751,0.07376923841470141,0.06391132764157913,0.0743892215305427,0.06910942030076844,0.06442104018120429,0.06421998297171518,0.0737654095533995,0.06174556048919072,0.06242980535691925,0.06500353174404049,0.06892979005064134,0.06263130000850095,0.054871168641236985,0.060179586116300254,0.06544781098387145,0.06369024461666412,0.06281828534659242,0.06837569702039162,0.06599369150047735,0.04377590865362661,0.06864760316352976,0.05205666696256864,0.05226065029752423,0.06340863673979076,0.0735153580957443,0.06067290557242462,0.06465952539787623,0.06744317012073839,0.06074347587963477,0.06889227139049202,0.06446399020816132,0.06362251963477603,0.059232920165770184,0.05521957264201634,0.05999120120897192,0.0698742152532087,0.0683485854989408,0.06609844850968388,0.059756245730136145,0.06823990348164756,0.06495556738901434,0.06236661165176669,0.06177216649052644,0.06528508964685878,0.06711905941777604,0.0692721810842273,0.06767774207683236,0.06663866665506292,0.0689163311591923,0.06434148494380004,0.06596802308988861,0.06258575530392158,0.0705887535566885,0.06569441800992748,0.06735446368293924,0.055997061619850165,0.06843248050329756,0.0635612566175619,0.0684641581944041,0.06200945554004194,0.05888910694057537,0.06278104672697836,0.07199234303021779,0.06293656556325813,0.06347716367463502,0.06629669599882196,0.05875352540077711,0.04776449685004342,0.061140090657926596,0.058324894867317935,0.06647784264082426,0.06524267923966223,0.07284684216311034,0.06841543345044407,0.06611918930716394,0.07050239067120512,0.07400479081527318,0.061560615941061485,0.06752337447928873,0.06471892099581783,0.06585128291866689,0.05224094548146818,0.0689799778844793,0.07197100169425563,0.05104312763048301,0.059436969031296694,0.06837026344866258,0.06195464548950248,0.07005843601123207,0.06996072394380354,0.06288550165844617,0.058388945132427386,0.06173397114374978,0.06834161798878646,0.05630903984931809,0.0672014864893787,0.06841502110905376,0.05506609704356439,0.06253641564205328,0.07187861702172893,0.06872344851864243,0.06821842820535122,0.06386993026625432,0.05968692970171843,0.07020304785302948,0.06442687110858542,0.07308335629310755,0.05946476930952575,0.06326034385623445,0.07144614993475262,0.05361524849445954,0.06460326966491746,0.06589160803047919,0.06048337995829396,0.06388364597709312,0.06590828523266046,0.07124357875542345,0.06824580819659909,0.06334628841626887,0.06944251780210142,0.06118930783578029,0.05358340417229023,0.05478556306451404,0.06815645875515824,0.06120266617579044,0.0586502891129656,0.06361213085045411,0.06226780230639619,0.060134606247619626,0.06307903693342051,0.058052234901577966,0.06265355222170249,0.07159523572903079,0.05947862360693599,0.06332574244003511,0.06539178038245072,0.05467600960274527,0.06329181585548792,0.06628564401463433,0.07633691171226396,0.06910247512359957,0.06307425973751121,0.05855663061550963,0.06834431317485765,0.06874761619288755,0.06335721721761199,0.061166413321608853,0.050876344613362476,0.06369043065494986,0.07465067254901428,0.05460590780798005,0.06847358849679591,0.07399535775024807,0.06220503588108164,0.06282799418488583,0.06731538605733667,0.07044773469262072,0.06344654982091585,0.06225735443722929,0.05832754846208715,0.06246997130837522,0.060259403273643986,0.047375271931771365,0.06234272125820399,0.054671159474875004,0.06532595088033025,0.06513287689671968,0.05953578458647091,0.0647179185370356,0.06123211367102539,0.06077083264312251,0.06701573443774284,0.07040615606244413,0.06524410707080412,0.06654454152245218,0.05394769593104702,0.0593932155367321,0.06989296234397356,0.06376756630784927,0.05591528679584644,0.0591637886453423,0.0590142357484976,0.06487823588435473,0.06922511170963483,0.06186780061873445,0.07168825906905411,0.07150620210410527,0.05797103788132926,0.060005745128688705,0.06979081053064679,0.07401383928697763,0.06929584582190233,0.0551921012762491,0.06110147412946065,0.06489976449097776,0.05974923951182576,0.05603595383815474,0.061388980859223785,0.061371790203741165,0.05779005563866363,0.05581589478456875,0.06601082200608965,0.062234001484579846,0.06005621279515232,0.053822375495358274,0.05431105021981508,0.05011961217404631,0.05631928927766862,0.06599269742426064,0.060517503839212644,0.06362746476383775,0.06710046921810514,0.07244389803805773,0.07092311236242543,0.05765977670548539,0.06773320488925712,0.06840031799488408,0.06804848275956765,0.04985867052437836,0.06130305266906956,0.06253698369706469,0.06291675630755106,0.05645693281079253,0.0597023073397477,0.05638241382159566,0.06370479997101199,0.07328571456554445,0.06989649064606182,0.06541310197490444,0.062017332392380616,0.06765085533860578,0.05818907272202485,0.061778921413657886,0.06379789920769247,0.07192386883229665,0.06750548887890134,0.07001051290240967,0.053764297307691916,0.06384562424154894,0.06552586275167718,0.0683038921927822,0.07281594345458017,0.06331522192633335,0.06431813019744442,0.06186515218971476,0.06307742705018518,0.05971823950401947,0.06565732661242668,0.06527306896957161,0.06578272547837219,0.054974169324653765,0.058561676636269586,0.06784646351114415,0.05931833758905844,0.06003850902849998,0.06899738510049432,0.06823813790965821,0.0688246093710081,0.07139604013673786,0.06462604950257325,0.06552258831870253,0.07014799409171646,0.06427007707523155,0.07247269214314159,0.06699102213553335,0.06561598776934696,0.06131747559406493,0.06463345520602515,0.06345354913020604,0.07291111551892371,0.06633981205036533,0.06557058589099016,0.06777524522431555,0.06564652056763007,0.06679546763648261,0.06943772046375668,0.06619661099151275,0.06033729413448917,0.06972088154532832,0.055569716359326005,0.06955554456358659,0.06475981095954988,0.05866276003313624,0.0706938409845051,0.06471237964669199,0.052528270715070116,0.05914756326852577,0.05370256011331642,0.06764274594302683,0.07134608759139706,0.05964504348196127,0.07239032696166864,0.05483567784775126,0.056418644724084956,0.049764703386089376,0.07179621620386041,0.06694485141637918,0.06957741069783979,0.05343330506054379,0.06350269596680716,0.07146219519937601,0.06553630774779598,0.060522187970079966,0.06039550786376095,0.06844196558674125,0.0704918838728962,0.0584934692053546,0.07255147064880202,0.062304827152036495,0.06568967080432155,0.07337589273151465,0.07122751513410527,0.06900054379493317,0.06685917547501921,0.07052425224067141,0.05869746407060794,0.06634618686657923,0.06477418993212122,0.06065426616007492,0.0619769124320298,0.07347976491087523,0.07154515356299374,0.0659186238191913,0.07316964023305668,0.06679743025069693,0.054783048279151605,0.0589716875149935,0.053293217237243506,0.06392048862872837,0.05899910746165468,0.06430104044071026,0.06998304540560338,0.06780278230748513,0.06256753738107101,0.060435234387600574,0.07131736647553358,0.062370973458017176,0.0586672176163332,0.06692361740767269,0.0663252759445227,0.06865260539203738,0.0663619251114162,0.0647119162920322,0.06955186361772042,0.06792431947968688,0.06657885360346369,0.06674006563420518,0.06622520589969214,0.06791291368691468,0.0704103100769537,0.05585342755029393,0.053602811728600507,0.06673043484854871,0.07252947101837555,0.05979435215775543,0.06943404622728065,0.06310960801432633,0.05124079678660117,0.07013534974181025,0.061430335892477504,0.058929224330560304,0.06412718581136964,0.061846084730403705,0.05527988244667088],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Home Growth Rate"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [19]:
random_kwargs = kwargs.copy()
random_kwargs.update(
    {
        "investment_return_rate": simulated_investment_rate,
        "home_price_growth_rate": simulated_home_growth_rates,
    }
)
simulated_break_even = RentVsBuy().calculate(**random_kwargs).value
render_plotly_html(px.histogram(simulated_break_even, title="Simulated Opportunity"))
# Print the p05, mean, and p95 of the distribution.
print(
    "p05",
    np.quantile(simulated_break_even, 0.05),
    "\nMean",
    simulated_break_even.mean(),
    "\np95",
    np.quantile(simulated_break_even, 0.95),
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="275faf7a-bbf4-4ab6-90e1-5fd022966652" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("275faf7a-bbf4-4ab6-90e1-5fd022966652")) {                    Plotly.newPlot(                        "275faf7a-bbf4-4ab6-90e1-5fd022966652",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[445736.4728673254,326662.79639873654,129565.96101899864,191613.84975813376,-76217.43836053321,-404684.4453008431,-408708.7232178622,568694.418392201,-268571.6121205315,289732.4153981833,388188.559876014,96964.34171326691,390417.0709142061,-359956.0799551662,1016709.6240797695,141110.24904993223,222027.90881446796,-401003.95112611214,123526.9369969489,753862.5113079059,349593.51461731177,-245793.38516933937,299768.6006875667,399662.24214008916,-403375.33913384844,952392.3888269258,580690.2703032047,-31941.29598821327,-209619.5686749844,151014.97072054725,449593.2441614326,745035.5637116204,698223.6851199158,537931.6024368186,493847.83740105154,290359.25911973184,1103711.0602671746,102550.97655516188,222776.25634742482,-403221.4916737303,275796.6337786759,58664.83219920751,129262.53124052053,226211.57867259765,413142.7178690946,-715.1369152651168,9033.465590414591,122283.38058393961,-185014.31224621646,106059.3345089634,35724.872365907766,290018.79315744387,-362285.68351954827,685623.6473922902,671811.6220306153,274665.8486948088,121611.44162593642,-64763.99462128151,-507728.6744674635,528966.2866912168,394378.0782077601,-132651.33243514365,-130974.69866185496,-116819.47178991744,86800.05651224591,292847.1054721088,84532.04601198994,228434.04229524639,-503866.1264085169,279379.0910693612,121898.77575736726,631288.6819660761,306974.2870311956,495851.7948793126,-44816.20290396782,-266818.71711718617,172641.04228024278,-192614.89824226173,87388.49288989697,98454.95806049043,-423201.49458920676,698982.8454667772,267342.2126653292,874899.2582316967,215309.56885519857,-429063.8109709108,-98702.78817047738,474773.6081740069,942156.8778613126,-1047237.7399713674,282922.96098389104,-437574.25350190606,648270.5192516046,282827.5708880224,165398.31353864726,-121059.34455294302,566512.0206424263,801591.7316734884,-903348.6468261871,320962.206988472,162449.14708700543,189072.93997433735,786212.3801779677,219677.91325729014,658030.3393219993,409819.7798574427,381897.7039181823,-206316.18683840474,-324698.64284094656,-576259.3748335466,753557.8439489468,443562.79478490073,577728.6458819667,477258.9635829483,-247736.76819395227,765862.5298148938,-290901.14282817766,-248895.62520430982,425507.4931201562,481612.1836767928,24110.009176620282,699284.723161784,460314.06517836964,-86687.98046023864,-349996.4940985637,-126240.71854398074,10124.404040231835,-81577.37391954334,-276962.033474321,-768097.8428404387,429546.01885692915,92973.50565948058,-249221.10006941063,1023.0312871625647,200496.80488636112,504006.52262328146,-549311.0428659692,692682.9706902918,332814.96080839774,422718.66253981926,619696.5941260606,481448.2243862916,-71423.77596033784,-581799.2101425831,-331735.84667242644,-160349.85453999648,252730.2536235121,-186222.65654299827,779204.0848895451,-135032.87869588053,-253895.80472023226,275857.53304536315,297929.20877203625,318087.04862242565,1008689.5814422988,-71854.95870524691,26121.288786079735,582029.216845361,375845.9778565788,1378060.2041331853,372051.6805421845,-199299.28097222652,-65950.27164274035,722359.3026866419,495995.2931661708,348474.33551899483,812262.5000892542,-729370.4446247406,296531.54853163287,-225994.35987251112,490081.85106818844,-146859.88070811518,594916.1962900828,-38175.486296485644,586320.6127078803,488231.2952666492,130924.79510961473,452022.60615216754,-47304.214073299896,714251.8994520116,-365054.94807901,1139816.0438375608,59077.77859447617,1390116.357305991,449243.0875023487,321441.3265624596,-466848.77094111964,443109.3583067842,57654.391814991366,584951.198046824,-437455.15308974544,1181825.6297791079,136965.41249874653,294471.5806696075,-102829.78547758283,-8804.346149194054,696333.9213101035,98819.23709315388,677201.4469373706,597562.1012992794,1049271.743247081,668055.321156641,400829.5876771379,-133029.43261060398,86221.71864265855,302345.55001340527,-227780.77540806448,-146829.59676581854,-18823.619066674728,761261.0893416549,-35495.47487403173,296363.756814552,508888.0318681905,-237235.75063260552,246843.45286771283,-126860.62024128716,48100.640344162006,-470948.1449707099,193480.04307179758,-8777.611611220054,185654.97566368384,629155.3409466771,-445145.5014131679,499524.599014075,164543.78059177753,639496.9264616896,1012111.3702720338,-156704.49069756875,325279.0694585303,532682.4604074936,-40889.705063737,603742.1202145438,419354.92443513265,560618.3898952296,159409.852870767,518789.2084707888,549234.0236324538,698395.5301948492,860719.2803000351,74070.93412975082,-730744.839392479,39658.003312761895,824060.3483492709,434637.788382425,-313384.06930246344,253966.09453415498,-76749.10677592922,784780.7464714586,289178.59207859775,664636.6026026765,7433.121346979868,76635.60190081084,-15597.095849902369,-473600.2912745881,179201.9059172133,427206.9334678082,675181.1671798942,-234657.69608965376,-144414.90814976394,-116098.86828077072,217930.31651172275,865907.9007625282,492858.1492618364,118193.01231326116,393643.0481829131,-450924.7019429873,-64032.45030384278,-83433.62236711895,230663.79040783457,-174371.04049007455,-603169.9018205602,-172724.3020145269,323895.7679010057,457085.6027915855,-43647.7730754097,-345965.3460927401,450174.0455460157,513425.10475956276,525827.8320904644,83350.5132912281,-155573.6709215301,26954.13988512382,450473.33464862034,791422.0070403279,696364.8034055759,1347904.097644297,269760.0845247586,224921.20203701407,-217898.9539361475,536348.9846109496,974880.3339141486,245913.694656122,-311895.7217370686,-765840.3407624946,709830.7639730591,-214124.5255768476,246878.19979742588,696369.1039810111,269497.4960278501,34540.2853729683,-201736.79260886367,787381.9160631848,-230641.7767238398,323049.7987277652,-157841.6443292941,807073.939705336,252699.18827317725,-93218.01501263538,234587.853553297,1103141.9996974245,390721.48453322565,787648.5305477832,103714.40377539676,-201541.79558319738,240551.63951036753,-99437.10594029259,615175.6593641592,-162680.41055336688,287418.6705193445,818374.8730240455,-124082.36492253002,619974.7498927144,-32377.496835495345,550518.0762903029,-400018.98227349436,-129065.195378887,328054.87275934825,-364424.5911652427,605273.5357385012,76893.44731647568,834848.8304042364,-775620.1199555038,390911.7539969208,483371.1503215735,127836.02377461782,235495.41187326983,419591.5103425565,145789.58422122942,-12578.532838378567,706540.7963773464,509779.25443634344,-51411.57565186033,927677.8619491868,-260771.31885029608,-193620.95901621785,592558.5790243531,373846.7565710475,-79452.17557929084,553418.528096606,-535107.3939878135,266294.4774695295,-175085.07419153443,101650.96365467785,361317.4215425099,294312.12462736433,879056.5487811728,407401.2356154821,-134286.11241976987,109280.24357949942,338447.0513511966,145666.526998932,774749.1854413161,292306.8769969023,590287.4767176625,297171.4824530021,114197.71514006099,-468371.58876434853,481349.51971193496,403081.4691238422,-100938.07578410627,645287.4556866684,267548.72690678434,-492941.08379827486,580247.5994958063,150951.29300656542,-343502.38454626966,710796.6820169184,-190413.9478409728,-568671.1853711293,618214.3498293539,-24864.76173775224,264960.2976357904,161458.1764246174,-103520.39286069386,658322.605301918,838713.6446844717,385631.16233878303,-167962.14498684648,-272968.73584486265,-382234.6087375302,474206.70811644103,430765.6912943162,305266.1016571545,652761.120935912,113349.89411145821,14834.095131543,78932.50018694578,489452.2618514672,-397039.9996123279,541113.1188430996,-185853.4084547176,660381.2826429568,486250.47934610024,293261.72550094686,162506.35687417677,461425.93951628776,397055.23080984596,26287.838310597,-234223.68178731343,1042318.4824428824,-89089.08626213623,44168.8652073727,31814.921694757883,470754.7227590666,995843.4727258193,846104.800350822,-173463.55622787587,634796.481345891,171865.84348508855,493928.2736683255,126705.04037126526,123885.01745794294,298945.74641932175,742440.4431314091,621881.3077125195,-690016.6765161948,120998.60682959622,763782.1018213728,133345.96019902918,-217877.90622267686,529538.1884578806,416530.8347957749,801664.3521599702,858440.7134360541,546198.7898576241,-120009.9167206753,665216.5558600468,-315786.06856257934,511239.24886610033,786181.0123251127,445619.4731955994,-78325.73266138043,-666395.6361862659,211390.68990182038,672963.8694336223,-26829.894832507707,-539920.631971518,180122.23646812746,128194.5773921581,-20447.406074522063,-78161.01005990757,911394.4797805366,217273.07952537946,-409841.9577255463,-636412.8545363043,537276.2957919985,1056096.9428024595,1142170.3683504974,35712.71725132642,-187235.8613368217,474871.1007111189,251714.48715281067,998718.3542877841,-83975.54769425653,-494440.9966780017,203646.07192490948,676442.2234251788,-775613.1897475999,61743.422988105565,1002721.4898015647,313818.19160539145,557460.0000438741,36767.96264324756,-513767.2617790145,588904.0679858117,-520028.1535167475,96106.6764491708,614447.013579417,353928.5531305354,-662014.2758403285,538883.1079880577,-116306.95766080916,781604.8911456363,1125600.8802743074,-375692.95696943626,591730.344254341,-56759.31918433122,487300.88662405545,-571422.1105204215,-34246.987751573324,-804053.3073265245,-56619.26713420963,-288361.61689444724,26159.819372728467,434241.4972315724,413805.80490033794,520325.7876701304,713368.666568832,346682.8134829486,-12027.249352570157,98252.84776368039,601856.2900629793,-426794.32156389253,969233.9900764367,426104.7125303531,559184.6826868169,-728737.4552894752,-148748.2338920934,-956119.1877816797,-80966.13953981688,281629.50846474525,-35338.05805376684,733541.5791444494,405825.03258060757,23129.077044642996,-112238.1425035689,499024.72911068285,-469512.1865337072,904645.2186666713,678910.0377445586,597091.9485611282,873210.2872980088,-25693.57348984154,296632.63424525084,-243989.0306024761,504004.9219638072,504005.2273357855,832439.4436843479,252745.31117294356,526572.3292998364,690266.0777393449,114624.6763735502,-296189.1654071766,-260820.79891030444,823080.4915340492,330632.9345703097,319834.4935120791,-8552.758055269718,537525.0929137152,-402735.5573069318,328715.9657643954,209947.22226221114,413958.52810062654,901396.5432349402,-853972.1853765761,-198821.25996434083,-446643.9268537373,54284.43798193894,725625.8505446124,85414.17753979703,-546795.3084071348,446574.02852607635,-350146.46471782913,-20626.522106260527,-186118.38034500834,-500538.38352562254,-196298.2356797713,593286.7715553618,179573.7972066272,88923.81949628936,550257.0008063763,-584405.340321939,823031.4384197593,411634.4302062411,-961763.316504271,-55894.90336743556,150548.89398216084,-491194.82396650687,-104579.31763247726,-425782.3840358956,327849.77134537,55813.72798987711,239459.6673976709,-115910.98961937753,60307.87377907336,151791.34009080194,181535.59145892458,814517.2022416433,684680.8854863234,281268.598804574,-410157.0890552695,225655.0335473516,142940.19791837502,448211.2627062639,216235.58114008885,-61895.85166381858,358589.2326144166,585340.3415745478,420392.3615355594,292879.3465377311,567075.6369192963,-281252.5000235676,292548.2571128919,416189.5185405705,703896.3299834039,689679.9246897609,692558.2594612744,47522.9050674038,869155.8484722413,568779.2351947175,470.3835862148553,715400.2921910523,280044.1471126899,146216.80403377628,-499366.96773446165,48462.93701355718,711963.0091345212,643338.5503654326,-458195.42188928556,253572.91254261788,380975.0533145871,-300932.6033999161,798935.6202105857,725369.1314018862,44951.305278376676,250547.8056485504,-267148.7724032174,199674.325931801,365969.5136483363,-982857.6033612513,-323170.5577677917,562859.8676325614,486384.6642468381,-39699.708719768096,368549.273151346,590144.8736030324,294033.72834589705,-583908.5248527075,1066315.1307360823,97941.41703113634,688822.478612267,-230839.1288028597,-469116.2480027606,618715.4152759314,199379.7581699919,-440332.50949416356,253706.91220417758,-1008811.2772428286,-70854.16565255215,235397.81006594654,365896.47092532273,800909.2706476506,529795.5666091093,357650.75325499475,-81324.10821497673,-622131.551487505,895325.0101880501,237593.81338022463,577669.0593171138,-124963.86411160184,1148800.6402390683,49524.64527565148,1010915.2035352779,252768.20903105056,585447.8542844835,222423.89429704193,-252484.74673548667,142521.7024374106,74131.91092108656,324540.14815124497,996074.8024747157,1208422.4386365842,483787.94487495534,-246324.3450640333,58111.40374078695,-380259.8342620395,-66770.64178622421,-289751.02370845806,-301831.1845973856,-131423.8403253788,192217.54055583477,-785105.4466894818,-358443.17766019236,401336.238539875,351548.83713835105,-255610.079250996,91648.50570956431,-502735.66551095014,315239.60025207885,41779.02795155672,177333.92524513928,757744.6373886727,104045.47368406923,-335255.24171289103,-352642.47105974844,12947.098682330921,-113659.21885012137,-162175.46784865577,-134112.91350058792,715634.4436498354,-563264.5454173256,432831.332850785,-688423.2233560015,228172.74328747764,601199.0618704222,362221.72925774055,-240931.46236465126,10604.113001050893,-400273.02638451755,-296606.143231031,-434280.5207684953,488178.6806770796,-103435.22330378834,-138740.98842163198,64528.99299851386,-306147.49449709523,461148.85833757604,131597.06156728556,-95090.12934512412,-695720.6002061474,1159454.2118422333,-544927.6311934083,-501920.17757424107,52223.96498130681,349970.8026596275,-265130.54274991713,1136456.3433130188,680624.8735222127,-126580.17468662513,576307.0987378675,1202748.4960949044,-430335.153032009,1586283.1790298917,101448.95721186884,-196350.7926028003,127399.18718579877,578454.9061986683,429156.40494514443,-52111.743817726616,512893.2514197994,-320589.9484655252,557934.1441827905,424778.9101791284,403061.29769763304,-307884.05996101676,507948.17153275805,429430.88702691393,683007.6576456621,283161.9442180488,-795087.8660796848,116509.0920607606,76217.27629354317,173090.9115903452,-108919.2314970612,832139.5302301073,101642.60289353365,336209.1650414411,533759.992949801,68014.34083886072,240694.73981931666,510439.8293886804,735095.9195694476,189344.99775663624,524771.2858029217,907537.7797762742,402311.5619118321,-162147.72520252503,619441.8265280412,-265690.6258951691,223215.61271048244,1029614.4200735372,830936.8193007624,-106146.70440540649,10145.276982277632,324658.54047551425,37134.642110211775,-45660.46681472566,741410.4692345858,-240537.1763100042,372366.6774849938,-568490.7055063723,452005.3582287142,521582.26374592865,-42703.86015836056,-112381.25088776508,-192350.64516985882,-177219.07098390115,4934.6865994283,407597.6396406968,-66384.83158889506,393779.4200733835,-49071.62061445601,162565.66833723662,20294.727740969975,171224.0809923145,827947.4551249538,-229296.21894059097,44413.3090448305,-327990.6236915882,-696192.3000601628,349396.5355605646,100798.0544420979,366792.11478494527,606914.0261184517,-106052.75665013306,701493.8554097544,277036.23492821073,-28602.108097043354,-269688.33302852605,-473532.39506689087,-45309.4377182927,-259782.58322826726,195739.5671217302,661059.8714936394,914107.8971171277,-93056.38770632865,217557.35491112666,674441.3535528746,30331.34083067812,188042.3221946326,412309.67836600216,-57688.11776734749,672151.1852209652,8478.82309525134,926265.744252837,-9702.237003583461,1040453.7356796344,456884.34861457674,317538.950370091,266318.4827869716,426430.3412345024,153816.32336810092,-306591.2993822992,717606.8191601187,228399.71196868643,1296617.7577615648,584604.353585823,222058.39419886563,-378510.6609738786,-622871.4302705408,547111.6921202247,478577.5330404723,275757.3148836051,253537.55328929983,-459046.5734629738,468850.6588024348,367166.46644703066,413343.81553047197,174833.37602440407,-217687.07371353824,-522571.28485525306,1028387.0819873735,775931.9056685972,-510803.69862676645,367783.6893941569,157781.08293908974,210804.15839770203,171092.78463424463,222481.6394408974,-126696.46198936738,724909.6247898559,298780.6417938932,520478.85433577,-61833.83040512353,731012.3792598876,-198352.95952929882,403601.05140351644,-365551.7062139008,356448.52786232997,409589.19425347727,288458.21895533334,493958.56639118306,246584.39789077966,6465.227093514986,-189250.98787320452,649178.5712629529,216621.5078893439,678478.4102686518,-211671.65588246193,405959.1713002743,-121140.5486224494,56702.17276760796,384672.9901843313,703117.504055514,382829.7484609224,819976.6669098036,-696767.5589108607,547110.0303398846,-304588.2418917869,444291.13169337995,19993.869511341676,-378362.7186381263,1241681.2762460602,1153898.5645249179,-88151.34321958432,68789.1326429015,-92730.23398172064,657594.9022984486,705459.5861100629,447399.45333584305,-687601.508750305,603263.7147362204,-193387.64565833937,339137.0617452301,692350.8058835533,68883.85743815126,-470233.5411547329,-618265.357884485,728762.6573334385,1000477.849112351,-46216.520190327894,49895.7754517314,329499.1559279002,1120911.2251186976,274282.5878388635,-72391.99547980074,734816.2942803605,773719.8247218733,795264.1232186616,497685.37465270935,-404360.4296213621,178365.83231487824,697853.4672976863,307574.2906928654,126350.86856730934,166072.30653665308,214932.38933432102,155641.90360099263,897660.7373317184,342812.74405894196,478798.2101127594,8820.43658419745,-169782.59487659717,1073616.5361010786,356701.94796232646,480979.4930014708,123746.1938912007,711776.6342496565,-331682.7126477505,172746.99339054525,641256.0722759408,-68563.44375548651,-240210.67973733367,1031246.2327359752,690628.4200138333,1265065.2095445846,-349772.32609170396,253134.4106172612,-670515.706869829,127034.67426823592,91663.33694782015,154662.46302716294,1101713.179052223,1158500.5471437671,387514.1555802105,364324.98083044216,456562.0086218063,160124.03318418702,93840.36518758582,109457.42500590021,867685.0415332462,-168309.71130317263,741107.4361595735,52463.857038909104,264351.14914872544,-84800.6482463167,491790.95594649157,754587.508869465,486603.7422992126,-116819.32951605786,382094.3790543382,607516.1467445982,302052.1927790623,581600.440958634,-350208.70262213657,629490.0837970315,493824.2570202998,144298.1509365281,136468.1404573694,301737.93839043705,486504.58540203655,-512678.7013493995,609803.3867131835,926814.5284743393,-546552.3892350704,52605.92092493875,223293.0842905552,272114.7358881468,716554.8287183116,324263.92002020124,-107248.89214364253,295417.44400319643,344609.7192208292,-67673.72073087376,161163.06954409415,-84957.65426557651,265749.2885224726,648305.933675928,84734.18528227182,485414.13196519157,42440.20800664881,59634.420452213846,935627.2305598836,150056.31989731034,675580.7217585249,597284.4514919729,-478562.3397128484,-569037.7736473545,210243.3407277069,171514.40778297652,-71794.8159252666,28154.690929714125,92449.75668880949,-647727.021529106,-255092.993431787,96633.9654041687,1120637.899554246,-146370.47371504735,-190184.5019452935,531662.391907284,-250086.43658539746,-194612.3980726595,828520.7403971404,-334942.7756175939,483385.0750798569,247830.80687809084,397320.04348221235,558379.5651472812,-177649.80835803738,-222129.24785153382,235919.2246895125,295530.6688057417,367332.0040328414,-221972.8733228608,115184.45736182574,224515.76137601398,204906.9353501913,156108.88580443105,652896.7423614301,-316410.9366268562,446003.0973891453,716660.3060974134,-35004.138154895976,310611.9286878584,240138.95636423398,1469.4116307478398,382596.2481991076,-147492.16726166988,464556.3675028705,751998.6515490757,742726.7961296169,-278404.8814010313,69902.94104797486,-336688.93138250476,849903.6456483081,510263.63975771004,-53113.53664767789,249628.255838125,321395.629903364,50573.56399297062,813251.7832106082,50654.71163072251,969788.3815698689,770840.4662853396,232663.11682423484,481730.66563074663,433498.6282253037,-67893.14674908295,705315.6839280236,774214.1954300497,556062.0469358498,972587.5576657536,-125319.11805552524,925528.6871538842,375719.44454513397,1013199.8154414813,548922.1378481933,62736.75096258754,410906.2649847814,-549146.0827564867,-248394.50404351624,569764.5040991288,-128869.81294409372,499093.9814346186,429229.9794414947,215933.33332046308,379412.613011695,-44065.934155412484,707183.0005237423,213625.16516064247,355343.61554721836,795443.5878936914,45425.142093006056,57731.54171214625,63288.28243360203,364149.65047782613,591570.3915223554,-9387.170984339435,98663.3045259607,-569309.7376745865,122635.46574828355,828827.0211255862,115590.47583523626,297503.75836671656,642354.6406408926,-372428.4648329653,499265.5811759569,560546.044853902,792497.9768642345,-445783.19350853516,598723.6333157937,-569569.8075297265,304331.6699478915,717183.1094256304,-211383.8045341731,137988.55761101656,725487.176456288,457731.29266627505,526619.4583100155,-464649.8762785434,537785.2960000657,105606.18530133134,498215.62638285663,716544.7748460001,114649.05854811566,249801.36756581394,582213.7463483843,287495.9107677373,-88692.61006034445,145180.13052129745,941374.4947353317,514986.63376295334,-388728.8071136819,235740.16951218247,-545274.5191979543,132525.49647104926,427881.5407548668,264079.63020577235,263546.43231252814,522832.59106019884,306464.0551583995,176364.56157312496,-6077.906551329419,260980.73031353904,1231516.1480028108,219606.10990202893,506247.72068779357,305420.4969696277,324694.2790045352,466598.49118128605,-163315.63404199155,138717.20263106376,794771.4123656251,391844.39751108875,802911.8497056253,731281.8494112454,-475250.76035444206,57746.82379938848,748514.0210634875,448441.681511051,-190385.4478140925,-285.4772186083719,-226861.5272703669,-153062.95175349154,575296.3015986909,-180402.12517569074,58415.95089140907,151099.18039714266,-294800.33366319165,114701.1818032749,584880.8879500334,131033.51916814642,65341.165517805144,-436694.6730634086,-332603.0781241986,323234.13766028173,865785.676877494,-146315.18545344076,560441.544159505,78564.59243389359,504178.59804121265,295403.16186769353,1334941.2066509756,332628.9577751155,355314.22499589855,510011.78311800025,-153477.18079673965,-721431.288205839,613835.0685133561,914486.1008996302,106006.09327405086,-61126.50293980073,499550.66170254163,-135002.2234416767,-3631.013093932066,852876.885393755,679658.2307753591,-119600.56569670234,909604.2972412859,925822.0732488693,642521.9721093941,-85989.94765360141,-127952.29243798554,228483.76854975428,-165532.06993666478,-362600.9071821468,1007290.3346199719,845899.7051416715,978540.3619362493,370079.14924477506,-203841.31008674577,422487.6423773649,610844.9673649501,125404.67660544254,-120231.75192504236,308129.75816563563,-504838.7029284749,-502055.80829228926,1134423.619557269,1103530.9935072176,237342.8113944945,636056.522848683,-79851.45812527929,-796627.2480539107,221087.45189906424,-399963.2433343716,1160050.8329664338,519059.11653439933,11141.204904299695,-836648.7984869708,472473.6011172761,206379.3516201917,1070033.2764819907,206824.6131213894,840506.6532990737,-179332.91076757712,828032.1018144218,-544920.1382679401,-545351.6584581174,696955.5793191255,-184748.93664235715,113586.72595614335,-407334.73354148725,-267352.597048033,271283.0912017552,-133248.79582447372,385594.3558788975,327476.0479377082,-422958.15022567986,508890.2707444406,378038.2277870551,30803.1619794555,490943.04216574365,160233.34419702226,224336.43609027984,-435550.4006060627,449989.2909787502,106877.91469858447,-281002.41589964135,-706179.9453735529,734298.9953262694,452707.0352157545,457997.2704701815,-353351.16248200973,394945.9488125406,17136.83486587275,-379647.60164483404,-20231.16631085286,604029.7897189651,-390388.49089170876,-684533.3966338248,556432.8164908928,120148.11204481637,187645.4378753961,520745.0178164216,172021.32716133352,-111156.8388969819,116112.94789485121,739794.2699361877,537655.9319053623,-223067.80069456156,288452.46167546185,-116102.26947541861,242687.1506083617,-204527.33135513775,84193.53503901651,-215595.77230515704,151961.05099423788,51747.28488879837,356747.8960606768,366897.0194720975,815937.3336988338,-144784.26983013283,-588002.7171308782,-63073.50270998897,535078.4514237167,581935.978219742,255773.67147759115,147900.8383564353,-94876.55273253378,577668.9846895896,-315683.56186194904,1208779.7739222548,-474535.54452902125,515729.9164745528,113367.90813837992,200843.83352662856,631974.1032287404,1024590.9811672908,382477.31142496783,239807.70163797727,-79839.72041982412,-33938.497791365255,-491482.90787089197,505405.3750910242,501356.1158363577,-218621.12583750626,177324.237883545,75559.93212877167,210829.85673077963,464148.6828049789,17317.941747073084,-334214.6385326355,339569.62941556703,147206.2942659813,426788.5266838423,440426.72718333453,611255.7262650211,-71848.10691491375,-421963.88011814887,406266.677225919,937715.2793136062,484761.28555242904,-182598.58702906873,518202.6625125059,469806.3786723465,-373017.92141680047,55670.766980789136,481210.33275175793,526487.5525411,320163.052769633,155424.73927172273,812039.7227533394,226079.26064588362,-332820.13106084755,-181.66947107529268,-362622.1514020427,-70325.20488259848,-519741.0658106366,283108.3614703519,106158.12282318948,516211.48929907475,-472134.0564907063,679908.6918578423,716154.4974896861,180894.81260544714,1096875.1354942326,-143398.9452238297,397279.40621668426,-299760.85570051917,299973.5467806207,-709671.8944013081,1506268.248860267,296714.54588096077,-24049.296971501783,644871.0921208258,640322.9020959968,703696.9297756003,1114195.9306393394,-348871.01346773,195880.33388814516,703540.9086350789,-82264.50962661207,-1003321.5155680566,907876.718422405,-338818.372813656,-238672.36512153083,21812.208689231426,651080.4806472058,803586.8671322446,722013.1362671047,545102.1691514323,-606831.7596784299,-298117.44792544004,812731.248499986,-766681.0688680755,515531.8144407957,87670.92168219853,412444.8881342183,279716.48653933825,800814.108552828,601104.3956322083,-340248.98072778434,634711.1539266859,463770.7940804581,429992.3879024512,-114758.83443173021,156895.68857548153,-109560.95080410782,45449.98421228444,-243296.89525855193,350716.356201021,239083.76325930655,-150222.43867217004,66426.21486524679,675305.3333328622,588825.9769843209,68642.29113907227,298233.8185343733,496533.75958223175,266827.0663618515,301527.1744544869,-218285.27853811998,908575.8651382178,554399.2119637374,107531.13507175865,-350851.67713922495,171309.1754820305,18090.296529003885,-364867.7533637681,19756.91148109082,968959.1504818341,204339.12079980178,430886.16880223574,-157024.41559355427,521409.13537558634,-283431.80712592555,-326363.269411847,976825.0914012948,419054.90591949644,475639.8138177325,928420.775981775,225378.4099971638,-31222.469280433375,435107.2688054908,436360.31061591674,-474446.2863192721,-40727.25868544867,600526.7522690049,650004.9097938128,-323593.0035473057,628782.6860805103,517011.4444053401,-145748.60473461868,-9628.496488778852,501491.63119338313,112937.64000184368,330857.34136154875,894450.2406590581,305843.11772456113,1190869.5797342295,122690.13921383489,824187.776306313,104623.40093979845,547189.8188617053,-95352.46225606976,-182938.93550483882,392829.61372716445,238597.19888708415,472896.4846169371,-118285.88734122273,-336825.923148619,604541.9043477932,54827.87500335369,-184931.73687025625,-114096.2290346073,585720.2993275793,-99540.36253874702,-193940.4944242183,79319.22272841586,929449.0019714418,-301569.24216264207,226117.9332041582,-105327.48191767372,733233.9141065842,721424.2817222457,621302.8710559029,599840.0657107944,557794.1863837941,703223.4905316704,231438.60297986818,361567.79265980795,119236.67595052533,-132133.69596798904,-278098.10478503397,9611.96595447464,-269874.27486359375,125490.23849862115,-362170.7265340844,336120.6351654297,-591958.2644338696,-332926.4407073781,-44961.09461715445,538718.5630181273,411080.3169020787,-83904.80811157031,-202240.93910222687,-621109.8468149411,153393.14423560817,-688741.7108864705,-212367.1247752905,271187.440571242,531934.7733115698,590784.8761787093,375619.80195688317,983767.4032164384,246939.9677408794,841322.9186619145,391783.6768199452,380313.84621181386,142075.68581719976,270187.2256481871,722988.5162204164,-398040.8445791602,1087044.5685637733,-349280.88259579614,662976.2083083293,659256.5401103655,1220205.2316805832,1134800.197069713,129393.63348594541,-126391.03590630321,-38360.75054484932,-23036.49326945562,558073.3682665508,218735.06108316872,-354739.9647500552,-104607.94196029054,-444982.06804923806,-257690.35636451468,785541.5843925932,697542.170182847,593580.7690111957,-116152.87035224913,225884.01956729824,-149206.94518439425,400714.9221661505,893205.5031582932,-9072.119260008447,-780295.1376916436,960832.750935568,891727.2344218148,50654.46369396616,206843.4981966624,88520.78891466884,-62516.61252488755,23063.406303995755,295921.58215305954,-63617.88742510555,675945.1653690287,-411876.95781077305,475001.7918444886,-369011.6814319079,409921.46237398917,488920.8376481044,507104.5053146342,171274.1311693131,68926.1999206962,166388.78267403832,167775.8628320922,310855.0229252004,273202.77186655346,20539.297307901084,152025.76584584685,333359.0807086602,7137.807003428694,131495.0551829934,511861.73394035967,-278105.1490518898,639185.7911903095,-339520.8354597683,368786.0419658972,-373977.5224817237,74731.59104397101,579721.6061440385,-89176.47176216263,-169272.36194551736,258304.3508721334,-408877.368175162,384655.07954023825,-142484.07661458012,-162033.98432655353,840269.6620724238,450535.6858853088,408688.5735587287,-130091.25834410777,-388013.3634373895,-299272.01433180785,300214.5369049092,40591.22045697924,368591.1549492269,508391.8686243789,288290.2669972773,46140.42220018944,339422.692881207,167684.36310275504,-152324.82997669932,-480811.5816280716,659377.1726845833,811953.7739486401,512554.6056207544,-126086.14616601309,-385470.9850455329,950924.8287398273,807258.499585398,800546.0889345878,377713.17069097655,281888.4996356233,13199.821791368071,-303105.67390225176,220387.7185665965,250836.72977216728,273936.72573276516,226585.19818018982,-30284.67144576693,124595.84582785144,-335752.5768122813,541844.2981177135,-269593.9901866282,-71707.52054236922,-230604.7492761365,577902.6017366182,19565.090941098984,387543.8554101321,-242073.7928708545,-32498.947594113415,264604.3180983574,136551.8680820805,1427923.8040250188,483702.2390851765,-950821.5734358465,153173.1964407675,-93751.14761585137,670355.1550689745,78176.68552911282,648403.7108833813,887034.9403631431,19288.85886333976,266689.2230851911,387721.33188899513,-4411.267322867177,398550.02907050075,-328328.0243815575,-721253.9274859014,-437981.6242630752,-257905.21785181994,-222227.17294653924,-71012.90893639624,955842.6084916331,-227384.8591458914,-30521.961409972515,64399.325241273735,-139906.27680441877,383815.21844121953,-27236.57367475843,-202982.48010545224,737987.5330544868,-389918.4857091112,235080.89002060518,191349.80674297363,-80832.64433215,-434034.5101277516,-53232.59226436075,254553.19457532512,-564253.1283043083,253349.66696352419,718324.3991482556,-804740.325469018,131309.37063275138,390546.42947100196,366482.57689457526,466128.4298561774,427632.9267791938,388861.7974461983,24777.33599220449,171684.0612933347,-133385.62858768506,83337.89421292767,-782817.7638659594,-204737.2418795037,492463.50065811165,-105883.23119617347,580340.8232480274,674808.2236104687,-182052.07161570713,592910.6065636864,-44564.755907030776,-376962.2780477428,1061477.8748253116,801348.6594149638,738512.7273165775,165579.53598766122,-732577.7503382782,-207179.47106528655,-610646.6143007306,-110959.14203711366,-309566.2569486429,638370.9140550666,40687.13745816238,-94699.87098848028,-537944.9474403793,862487.7303377939,281212.4333870206,51061.35885583935,-377266.9494375582,47689.18178988015,41648.92326340964,1068158.2425024468,307123.1210767869,-349116.87913248176,-31033.006617306266,-253223.6472940147,273946.7507044128,1017279.5664711944,878364.0331879705,-237579.7283973815,805839.4626554647,-233491.89646888757,28993.64611204574,309672.6517329188,185291.42832499696,646951.8116379217,-106291.75399141246,795195.2706674053,209265.54809244676,-81615.12720121117,215484.75036616903,-63527.30163784744,457241.23800907796,44466.291794652585,46819.94209254952,519630.68104543583,640892.9990140563,47169.50694605289,762285.1913029724,-387162.22624135343,-31659.462321848143,35376.44500639336,338214.0674947747,483920.89499637485,894645.0367403207,1145285.2956139515,12019.77436578786,-284558.4555464713,-1169105.66783308,812184.5518053947,385675.9553245078,776174.9190940424,626105.7846738021,492603.82265662076,276831.05821205257,-1758.5088409753516,-63713.40559902787,-48905.51902108127,-171788.30069630593,346622.80956087075,-239432.4307313473,-27081.03344953293,-349107.94481910905,765273.2188436221,690669.8012757115,-363811.3695728993,278160.5126368152,-86188.1656545219,855244.5079639503,111576.03218071535,-465520.3249627766,524801.5420795674,189694.71285225218,729617.496231507,-843022.9960147757,241185.4745637644,255515.59573233174,196403.5545755187,658998.6828118637,-61281.91564667644,780073.39886055,397867.4518331522,681629.0201936793,-282909.2546461001,-77909.95894146618,-266142.2442349964,709146.507035885,730841.1669229316,188202.5248269923,-247817.42652876023,504659.21398284496,-318353.1889164853,230898.15792035265,463590.58921100525,522223.339468264,-478690.62143293256,-177515.44460497284,-34565.80091784103,688235.8955994737,1250687.021492748,557170.5873410711,-493587.5235034609,-71067.65080232453,-259231.2354245158,-285099.0399522525,1026463.6556923883,185089.95859511523,386788.63396983175,295848.5707943151,458309.36324171163,887440.6534954179,-198727.49946658965,441866.2278991495,134841.95077927224,1139008.750835387,-9250.14519869117,297464.9958551382,774432.8369148583,781527.5942382519,488572.95023723505,-322835.3450494148,-797.7134091947228,-634578.6528594338,485771.50875784736,263541.3910374092,317678.8562509464,843635.6582342726,44195.35161760589,281813.2147794054,-47486.732652892824,435971.4464391805,295193.1589092065,753874.1926166206,231895.65000551473,45263.25951576559,-1001069.9322809675,158161.7479600208,1093394.9723779694,297264.88266595034,205677.3219515006,429716.94227998145,507066.33725588256,-948363.2919778284,438424.5582199795,271644.12886171136,-253519.92167821713,362554.05900929123,49527.202768459916,69256.61759266397,-234450.09823153773,231627.52919058735,124898.37181106023,452767.68001000304,-54.019888784270734,151264.69706302183,-925964.7608457012,-625346.6118564587,398865.15330691123,614282.1464713537,472396.74111016,-342445.97834162647,-455565.9875369654,230778.75604058197,180790.2152990112,362477.8849107032,180386.92221227428,-231997.22606525663,262328.8203614061,223368.47090333467,-530965.3452509302,841461.6014437987,495449.95522827236,-186590.06342999823,478344.48746930715,109058.96883236105,756250.2620359967,508563.9542907565,282337.8967062314,-90030.35399101349,-241798.40199228982,117477.42605381506,-487307.19800554775,72779.53398154769,417303.46999685233,343447.8234711676,-725483.4400890355,-187200.81724726874,587568.368674933,-279778.628508782,755754.2328475313,262723.64134783065,392027.51106864447,-278796.28646050347,-360394.59107395075,816985.2554567573,842191.8860008251,211798.5483328947,716105.7313511358,244566.842370491,-545487.9406554541,18254.375879243948,415379.70806479454,207468.06532035023,-134886.25084518362,1354150.9530570558,460669.53680374986,228783.29703743244,-551883.7685753535,289082.0729720448,113019.03851624578,-22336.47813167423,1009967.0161759683,-384070.8268223307,619284.8221956985,519866.0115236663,-181629.76801725058,-71874.86217903439,66444.36374163581,357143.45838136645,-632936.2663899036,-142409.60101623577,-454506.33614883013,793483.6305815573,-350499.82262220467,776181.1045815856,215601.81786829839,17691.46248036716,-209539.30334837735,636300.8748491402,400538.0586037724,986014.317719881,603784.317981862,-343880.5534209241,310945.31325328024,349931.20896583656,382073.1971931895,425159.0082512363,428290.6166806407,423644.47003217135,1072947.8159136786,-658282.7519599041,-70953.52765057655,149274.04758987296,334056.80418911297,-140323.37474755477,58994.41621134477,-668452.6699826666,828729.4509116821,463610.334733624,271893.24622293795,415210.3995808726,820425.3171144868,317260.91153986985,315736.87538430747,-309102.31801582826,-41296.613551039714,-424922.1795769511,118214.02974248677,998477.7138358145,339148.53757532034,-588705.0742519782,56795.16787539469,-290141.67273377907,-454454.11764063826,-394894.50911572925,232040.75672603818,98859.37463169359,425032.4272525497,316331.26025163196,593547.8421191461,1226488.2465197053,29561.93343289476,-137200.84276789892,-243019.2340581026,377632.4018606832,453483.4785410329,145080.35001333244,278301.0278115347,513318.7810543943,259493.43532506702,191752.2630033209,-40728.290816706605,17230.094694045838,336853.7214293545,225966.13082173467,12569.16406831285,-409319.7548834621,194761.30797145749,-434659.1626003678,-524271.62899958435,-117760.06309201987,-264926.81945042685,74231.82087517297,-564920.405498523,78236.09695030795,-117403.59228692157,69716.00764871575,127755.81308518117,-267241.07096038386,935061.4634079742,-24095.549029214773,253893.10549843824,861040.2265851242,718170.9184304914,741984.9699497744,54106.260324006435,159982.28341777995,301552.5568455532,65259.336154632736,307707.34562220145,-261377.18090976588,-702344.1769624266,-12730.65413061576,-341879.96460989397,-642891.3931282344,-338803.35259853536,447051.74783264706,603461.5974592245,-125988.16342300177,-277480.22037886083,-695005.4278695174,268741.87601117184,18425.96297587501,1234901.146908816,42368.57126434473,54607.70516239572,722196.7836042787,-444468.915826187,333119.5237378548,-377825.74954387126,269243.1207686141,163837.67155877547,770074.6311135227,954557.5587224839,6485.982017850503,233621.4592978512,1098408.8504310884,452384.58332916256,465547.1512304847,322878.03222219646,-213279.67312401207,153370.63578911824,657122.2367198488,334443.41678272793,835252.6009664438,258165.3142103497,-294682.61165218335,58215.615858356,943215.3607270373,-236090.85903029516,343006.11142670596,662660.7979765488,-400994.5533992369,335761.7455382808,-54736.70900210971,397853.8059259658,48775.257189503405,1208220.659577609,487921.2562052775,445984.2188076591,-71339.79468053253,584733.7417254471,723046.3112153807,80177.00167646352,-280894.2632173733,976413.5734991466,220542.6045936565,131425.6632708083,-372835.63552435813,-101260.55432347255,116096.48247974273,132095.60150536336,528479.6307660076,574633.6919489456,76551.01128455065,580449.6010443573,127669.24226502562,273398.9084325684,322814.1627214961,333171.21787857125,146847.51104987552,-414524.3533221907,952009.4776351769,540572.3028602931,597459.4195455424,282593.1563247279,630840.7022618186,-8690.80826706998,563800.0352274291,-35052.04447887279,88682.02541603381,28691.99714822555,-373612.3284175806,-382063.610164369,-79663.86017643102,470118.5441268203,-4367.596914699767,231063.8339389977,232891.81819734536,695891.6708552204,-111375.99449098716,351148.01447172184,420903.9849844142,-589216.3718712674,-143446.43060241872,428200.9514101236,369018.6658408316,149172.0887388955,200280.76527064852,-337988.8849123963,-400138.77275443077,-80562.43673189031,579946.2074024761,238.40174179105088,429176.48293575924,145527.78846136946,260883.5159555953,332625.7523116148,327107.799605201,989872.4196047275,410471.33335170336,806899.0232669925,1364541.375976231,-21314.082989589777,-61891.598283214495,835380.9638134534,223451.45983757498,729773.0846337224,379393.3378013596,66736.20892941393,-20607.01538628852,219938.36390560865,276139.04645381495,-277298.31110251043,635132.2930034474,-470787.9928155639,317809.0968365357,-86104.68963777833,345167.3013757407,760031.7229498983,658508.6376579278,-138279.26824387582,-756985.1885501775,-19076.063568585087,301431.73946729256,-538309.7053319085,421739.06542149466,121281.82317256741,-475404.5461304118,1316808.816825037,452245.64864797285,57788.28815352963,759142.2492360454,316178.9194633332,1129166.174382859,716561.2055350915,-35354.778312725946,237086.09927658457,195835.60690598283,-160782.92794890422,194298.50005666586,489446.9051291004,596819.3007745729,-254604.9584822082,479779.0570637742,-784716.6650915775,748817.0174997267,572274.8109428738,-635427.7591963648,84640.63720022561,239789.38934586197,934652.3185353209,360240.36708052456,722701.04709533,523826.00377678964,-178380.00921858707,656757.8844100414,-574742.3121742127,143289.83410379197,-222667.71403491078,579813.3134434996,410293.55014154175,352993.9380919868,112016.9668419999,971988.7418374112,-268363.84434674913,629076.1083548646,509707.4627340636,374243.8114521587,272649.30981468037,-768032.2561736559,722657.829919483,-411412.8462183005,504609.239894337,-342143.9899384519,96511.08745166892,-158689.6374380961,-333904.782929271,-102994.08569583762,223308.99676606804,1241894.4422978126,-110394.4308280698,-910241.7397208568,631081.2705797297,479072.9359169095,61753.26201781584,634471.1420795773,568352.967404108,515453.7133484455,-511330.69823065517,45566.180782103445,697565.4912848664,243077.57135660434,541398.349102254,-33226.095234585926,469725.25501565216,-15089.187786869239,825867.4827544084,-695950.8716510767,955826.6777122188,191546.17717088433,612885.301872639,231087.19763369346,160574.65658808593,-60715.01917589083,-59590.90439864388,486438.0500338902,108334.74638150586,-62339.7846815628,176098.4480205723,758026.1612774408,350717.4879775406,-182437.00128229428,625568.7736546746,231097.70701427525,305206.06870948477,433833.7832617955,-78894.63416622486,684642.9534757431,383226.9630587017,435251.00684568984,515335.1726040086,-354623.7383943284,50071.50724171102,153059.1141935764,1548511.6169368918,-233719.95560281118,-501.333113997709,571796.4764869967,-250200.46051576035,11277.693841672735,-129455.37531817704,146639.04656952992,-356913.5414053425,-472287.27920907526,250005.89550826326,-537979.5206141253,913130.8822196792,477735.93435998634,-4119.171044610441,372930.50283005135,240314.381229267,240784.91765375994,-282356.62595081935,-59982.27211779496,-542291.9176986353,285766.7561889915,-97638.97248241818,89455.13519594353,-715751.1426032679,-300567.003925961,-5349.434560344089,438205.7838841225,419278.51212361874,-398242.8999500121,-291964.4380678525,296709.1144432295,-880701.7570167053,111636.49984324956,-1189.7017433582805,-212165.83030870184,817854.2368056998,-366859.10260443995,-118244.93504458945,-172825.50744239613,364651.78705924656,181257.8016905971,147866.7551755975,300257.6721166554,-262780.8757817014,146641.81695535826,-345327.9741168446,175756.7046491159,205648.9172405866,-228328.6547034029,150656.36081491224,734723.6721493839,729070.2985492283,-580133.7421250567,264783.4878926473,348824.55484344997,-91729.10479124356,-607392.8909643185,924644.5488547885,443926.37152193254,481048.723332705,47143.65078413999,1051878.9874621192,-223794.58876478113,-162112.7863555667,139731.09644656908,327487.1783146891,248315.87331149774,-530884.9694333468,303068.79127365816,-381432.2654565382,-156654.63162978133,946198.1893301387,-283936.44375842484,-771299.8219904378,417153.65908625256,-558412.6847739215,256553.6480152323,236599.53079784522,837442.8568959371,702866.7550516231,434736.46143396385,315469.81508234795,-57676.79971707007,866490.056456516,53704.624610035215,556384.1205188115,189876.6266127173,97836.96837045113,-301612.6761268135,105797.60512017878,-100278.1908708401,64050.83089206973,248855.48899155948,-270828.3848223649,-705043.3067363212,160329.59222386917,-649825.8436976471,-428174.45653213887,-445793.8557274102,164129.47379623028,413235.75703249313,566152.65792723,816677.6250912454,-325699.8226141576,63891.16993596777,-116663.43404640397,1103836.7657341396,406172.81662617996,936525.7642393205,752446.8658972238,-490086.74264257494,-96612.51928483462,61884.05394042563,421759.26456673257,-70243.2297763424,729141.0241330802,454035.90081871813,177630.22914269892,398790.0614279872,333084.4026065441,965124.8073036671,-73369.4607113325,514264.509546048,631193.8579411344,474309.40794199845,399836.6656883424,554253.8470790675,-117739.99185982766,486092.7830341989,261749.0602459563,426966.68243036745,554916.3704925566,326792.62894412456,56005.725554640405,-98327.97205017973,408682.2682657875,146149.39723973675,472739.933211023,371842.214425595,-56152.26945701707,880744.0144038615,424145.3596205902,267559.0481802006,393622.3142455467,57382.026758432854,364122.006650832,622914.576305402,-89260.09925412573,-178035.6430802294,943218.4912963924,142852.96259865118,892760.4560415824,207431.29271312896,159269.1733860476,103889.79215134867,1149716.460769231,-23107.890750367194,700129.3418080988,-642955.9836986745,-193192.0375193078,801016.2938919384,-95883.17040792946,405252.6672466346,577672.3241866641,-49157.4829860786,-996365.3263886927,-271195.5375775364,592797.7327437443,-748607.4470216259,-704662.8182544368,-22060.13261221815,441469.963588309,134765.6730923094,117166.66826340742,-206269.83848652896,695996.7687695823,-97774.5145399645,-510249.35310061835,249925.16805211315,365425.40285525983,-12802.731365097687,541765.8010384971,359530.70507070003,-31205.05666812323,491129.54030255694,642224.7198993536,455737.5676703793,326049.00760649564,-845149.6421952769,-218525.82729681302,793012.8772851396,10191.452337692957,-420306.95904957457,-566916.112233615,136489.891261423,196742.61409620242,-152920.24116197228,537128.7859413815,482038.0617534085,-210572.535840631,1170425.1404529624,929597.3262185385,440102.7967251362,29700.24928149581,220195.94985239813,332482.12224554317,451497.68316480936,670636.1812083502,615501.7908157804,-103626.90109320357,528723.825781412,490284.03591655474,530011.4294980611,-241774.62689442374,669456.3440765305,-240984.46237010928,957794.9667690368,-355568.91403555823,55688.99570163386,-64209.99833772192,443200.42704430176,87234.78248340124,99857.09445479792,510114.02001257706,1387613.5555011015,-423213.9825079907,308976.5344355097,-84448.01732539712,674775.8707262855,859698.1783868377,-257060.4398407028,324807.1688931915,581493.7262380943,724714.7616700446,-377242.5313440971,-53544.03792315209,490413.610145085,-226932.42348718736,534429.297165141,-149399.61289928947,43087.50633484963,506268.0885376469,-150557.400869044,365962.3714266019,-515559.65768432803,-482486.29810877005,450676.47745581856,-252769.34062435478,-373178.67840939295,-111025.10959483124,379032.27662726305,149554.26292039733,665748.235248792,278650.1161346319,703367.2070528921,696285.6839913381,-498048.22552176705,468515.53215785045,500890.2019227557,554647.0650047464,-876056.5895788912,173913.2715680725,316105.41799125075,405959.79393819114,1118574.2134424127,-798.778444998432,14030.92736044759,738215.4334796802,-113716.35331912152,167292.46264605923,-73498.68636350939,190862.77022544108,744626.5647906251,528284.1906173169,282571.69325457886,472268.86666316073,-175601.01974242972,606537.321784595,-450210.65436038934,333135.12928316183,-12609.394352055155,528086.243077619,305405.2243294781,26967.778432418127,410684.33301406447,-476360.8440915686,-352000.0085604219,90156.2988497275,-524634.5454352731,-311641.3433429762,-112623.42872447288,-613086.3948935815,429850.4765035957,-611623.1926471866,642855.9329591799,571352.5062118317,-537973.262646562,1021515.2375265402,1035525.4662852683,924408.0674000797,184147.5649796161,555499.1706309766,338323.1341816988,987607.5871162526,-247780.97705595195,-641677.0768533498,-71347.910671602,1174312.1557591413,519007.33763629524,-375353.84613174386,508767.92149258405,1155760.179027427,77156.40305383317,29465.16762491269,300929.0692641651,-47836.60460900795,-58844.84052311443,-351224.7424209812,701525.6840956807,670025.5932366112,-259128.25954471854,251304.12959459587,314662.90358418133,309736.7753158659,802730.1744405958,-619891.235723115,593181.8893928924,-668745.4553774288,80877.23802152276,-673406.934364628,-507818.56265037414,593476.967852185,64679.74424785562,486058.72283528326,675846.9651152287,634433.6880403804,272733.0354784634,1017046.3434791253,72152.23147565126,-211260.9480599109,379955.1159699806,938989.5495590712,1128521.3182955338,-145678.03421641886,-300697.63184900535,240678.1736143995,-52631.358488198835,572006.7142953612,-147169.65798996063,391422.5122565804,-485954.8270630925,15021.134573207004,-179789.3910364369,672248.6225640858,611338.4445337392,-241042.0414306107,741690.3564033047,934167.84097614,-195207.2271692413,36054.21827238891,-145155.34112125775,12844.39593011234,1134917.0999932722,757317.4146538232,555109.5601682612,25486.459461049177,65911.35053522745,345737.05664065806,509960.0407204428,-638090.2081079273,523730.2977406746,-488737.6509989756,1025589.2893127752,201471.64722176362,749577.5348388106,525457.6343922345,343466.7336459062,117625.14870269177,-627402.0217223107,666812.9923401647,775198.196788834,-114005.31466020644,-454389.26291814307,757411.2158910627,926082.2603006158,536490.5937409736,406039.15833419096,302824.7978091431,-845018.0579452408,-18465.477206763346,610625.0960503877,861754.6330050821,773959.2659169519,223299.56000690442,-315742.1831241236,521122.55994846486,387932.162570572,66026.93247962976,163614.58585618204,60711.41874418501,-425909.03259458207,143020.8871861389,-695531.5844365039,-5864.83397777332,291780.81409834325,402640.2405630229,-133492.3423809898,371518.1754882075,-638260.3262041048,800621.7100227266,482575.8366725985,-799208.334407771,274770.4986080718,559029.9023689493,-442198.3031203472,-558903.6153800823,372364.0492654601,223991.15990803624,960061.619421633,805823.4610136338,307992.118972857,-77925.20665720385,1039918.574204728,-20541.32417430496,-531656.9117441806,805821.8455451322,878133.9854709301,-243055.01586614037,-731390.4319833885,187916.57504995214,178738.14377802797,115017.82344227703,650857.3945524893,183739.36031815875,1224846.8370920809,58810.68573874049,839159.0694670645,311413.7328935391,374196.7331798007,441508.91084486386,-162080.6161880782,467027.16539181396,-180774.26994026406,-831489.3165939967,-291707.4231466218,231464.33851073775,702586.5883066761,285499.77901926124,-261891.395556313,-687472.3119215816,898023.9078950998,283986.8748604376,104927.14561186964,-434154.8048485657,-527182.2687464347,-226404.38543872163,146925.40082749026,426666.9435881539,-355175.1882275706,639583.5063781585,137592.92008110834,-156369.75010096654,-300990.5542963068,-98109.68153115688,1112759.6389105911,670793.5665825219,-405268.1714904364,130007.54011704959,514980.88303686073,-244931.97561653517,637159.4603505968,121276.4396575503,314721.28096533613,-380668.03798476234,512303.6178208515,524359.0820177943,334417.77823485713,241388.46735461568,-245002.89434391772,441712.65471704444,-411043.3894240926,258688.96490141982,308690.56350694597,-266707.3626047028,-993294.4498391899,-71126.28849672433,-392029.6647678013,716099.1713936478,317014.1316051828,-182029.6170710139,325993.72856744286,497405.02692686906,-206899.77146173595,747978.8306030948,927302.2184062228,244656.04416806856,470430.9187456812,253073.2293093251,47482.00871614134,126474.91719642375,327764.71774564683,503358.8981074048,334278.38772246847,-166679.2681417237,352320.1327472138,87583.90031202137,-36099.47629692033,495692.59753827006,-273990.9834997039,-556692.9914949904,-545857.0272880003,882901.0762364939,173205.19079548074,106904.0917673423,-28062.963594216853,-120823.78746330598,210020.83130106423,464864.6753651039,661154.2396282079,104873.58503277274,234380.3474197602,-1117999.5152106679,-155296.624124886,-192470.96309432387,506329.69548313646,-179334.7033183584,-339780.5678683375,364477.64258101396,-278057.606175276,-372966.18174051214,227797.44200325524,364958.899076045,117549.23599589616,318418.83628437296,-506702.1694759114,-106894.70472023915,-80073.89099523472,528954.3674162589,-104127.56123075075,291544.33175839996,-1184133.9490743084,-675868.4420031342,-897176.8908008044,527349.9133224459,851848.9984189747,-407911.7799162613,256695.2314794967,1159782.7634076513,139687.0454919315,303778.6888643885,-551410.1448016949,428781.35965525033,1077600.8892725022,645059.6504672314,-604421.3204395338,-313676.0852447683,202154.0377051877,346770.91844000854,-264500.6029462456,288157.70352482004,918655.7076355368,254805.29062569886,-833160.2308608484,1240587.8998444583,161692.86320314696,-2692.645552904345,42379.479170323815,249437.3472490781,-437249.3044513492,350830.78134069033,-224433.19552839175,357151.46328908624,-146474.5254581673,761940.99370356,941920.2287943261,-50155.94955270039,552992.7647750131,510970.2119345907,310409.7692737337,231893.1399068227,985806.2354639969,160872.69835167937,426944.4059173949,-330472.6595841064,-338792.293213828,-21620.889244501945,596174.5316559179,696140.5194400442,-44281.50212564226,-366401.9648765153,-379483.9523150637,-839682.7017249204,806122.2797209271,464635.8006109712,-602228.4767907397,-373478.460061464,-140875.26585490117,543773.1832919656,-477340.3982751514,-845305.1841096568,480732.4941472574,322369.83680529566,-97413.29775705514,-45649.363397818524,342472.2592860493,-535376.9125137376,-260376.15436775144,-171492.65294924006,607461.7990659885,-386510.8310800232,732747.909990597,613762.0455818586,52938.59936343925,-302620.9090232551,638235.9314326248,-290887.93732293183,699896.5447562658,65383.83169780485,487024.27183397766,161530.15608066088,434789.1683115084,191573.8094306793,1011930.9760153373,509719.79708799534,210798.00478459382,-12251.024902009405,617011.1671198341,-352866.3967539533,997702.183027829,270179.3140182481,-145479.00839735474,714860.0406318395,36902.80270346999,105138.4910122091,517215.9595259568,52093.71775024105,99487.10385789536,282211.662794122,477600.1022812519,659809.4311059727,993315.3237800403,675092.9431013684,-445616.47168981936,219954.17378936103,-16521.337096467614,916197.9858661857,355973.0881663789,690455.1307059051,-207644.7946190117,-351878.7328805006,-303297.91342042945,73792.7851994494,-278837.1091227615,208426.37613947922,-162715.1258828109,452730.04340734985,-385347.7626007418,435822.09098794684,-335805.0451435789,694920.1355024604,862979.3368543726,-359129.6627676566,-479611.1909084101,48479.151373314206,632622.8641984588,54940.600775388535,487996.5837839553,144418.91364576807,-210320.14988012426,331185.518019333,190849.7446329347,333913.41201913124,593828.9633749775,810783.6906977524,1062864.7362468736,114105.03317383351,-251377.8256248776,-353367.5852887188,-156267.3456602986,-154900.17014028085,565054.7276306693,953153.2049221289,184580.02880851552,-389984.3686164743,-153557.26135707088,671432.7399976831,52353.90832206141,-216217.66091164574,-210055.40111432178,-16600.919630311895,1338913.3484175114,474779.89088249346,191024.6816722951,242893.92197823338,143580.46915546292,595220.992383658,317594.7487045941,-330441.40873830765,912739.0135782077,238353.28165121144,566274.4935839106,19932.273562344722,607169.7748404113,130206.84989306284,-175145.3769037146,-330333.6416968778,745101.5232315231,-729298.4523088587,469053.96648559626,168328.6557886498,-450726.40052025253,658273.6115551689,144538.67272040574,280527.15617666906,471975.4040781916,862001.1032716599,182813.92509682057,-201005.55326931737,197682.00001798943,389798.06318694865,347691.31822860055,-306252.1088346909,-428927.23087154096,429135.4867856852,415818.0827247463,658965.0592738413,192382.18810860906,-12914.320921281353,-1020094.8321634063,380891.1033322867,-218443.9460856351,161584.3608981031,-633185.1131317806,520607.36164650414,-344493.43830018165,327141.5857437742,22355.97962910263,565878.7662848704,1125294.1062681568,-135847.5542565398,363725.4817976528,117005.22879236285,281698.7034301134,644532.2423617016,813104.630914351,172929.9414784643,544504.0926198894,161094.87164854165,548243.4011039012,130145.86639678711,23589.63534239214,-71892.15235651005,-7601.870885901153,179302.13351406436,95097.1882831431,849704.7545490409,107881.64510112535,-171477.62988550123,-328296.01487434376,-255566.92882369738,-445312.16405070387,118147.7958747726,-86366.4270534846,11013.677531946916,189675.9336694004,446087.2800968867,-191048.3196951053,232951.47132486617,358882.8637332218,59333.577523071785,93832.74027692014,-50352.25661392789,-159450.45361158298,-760608.1231756001,-54184.50329916738,-200791.29964565346,-23076.049105547834,232808.19274119334,549320.3313301657,502156.7838837579,669699.0112893102,508798.72518437775,422322.00931571703,304218.94230749365,-123390.22626706352,165129.3746887166,299873.0969771645,32531.46942835301,346019.0099579138,-110533.31409633951,95638.00865092175,737331.0521998894,225479.0342898625,-707137.0477956021,383147.5174515364,458677.7472717981,-527917.7714452944,-482137.17596617853,-25986.976557769813,836080.3787075127,625234.2828333876,-182117.87377685728,1012569.0392928286,-142560.23624868412,-56142.55606567161,702495.1318437541,1145009.85341615,11003.865999754984,-223024.58357594907,-712010.9101167116,-100456.54668368492,-175885.0887375297,915286.069617759,489845.85852968786,230404.66262783576,426913.7433066438,230110.162575264,-301631.28234435664,826813.7350588306,1115938.1768940198,814204.0153511544,502346.3720782576,353684.15926902415,-703395.8838590197,321644.21767297434,201123.74598094262,-139488.11199225858,123558.98811821966,-91999.78233568277,119277.0254970286,-201478.02478143247,-959093.8946118115,-468434.50154523156,262399.4432090055,778968.1880343673,361673.13812324544,-319309.0697600795,848118.4867330873,-82028.62905414682,224078.64653949952,328790.0309294821,249207.52475668257,63890.306977428496,-65940.29467612645,-198903.1751583726,586566.6397349616,-602979.2089481894,-308543.997329623,-201336.86658813944,32153.662639397196,913096.2620653454,-398797.76186961355,650144.2910675742,519104.9849169217,-77783.82231434248,-513489.87105595693,317987.2488870518,746121.0638480769,435500.2188788201,-996898.9240313284,139453.1132926657,-35073.32347008493,-28909.954435663298,599853.2722447785,61294.28825922124,115818.6221777671,-389609.2859688783,815761.106153423,-138215.8273336445,186183.45961780846,145807.40702064382,1004590.3987197233,94273.02981683519,-228944.92379406327,928803.6577251954,-65437.15990873426,-225030.714709301,-326645.35683515016,60986.16081118863,-34595.9583797208,-280748.03023095615,892208.6677710682,195638.19048214797,-235783.3688357263,-180745.74501592806,-115402.9761037617,-257866.60364622576,515187.88312069885,306195.79234714527,139918.2363476348,203952.55429736618,1157393.45807336,11999.554890378844,150487.03433890687,1362317.8910787571,94247.06872508954,-56200.29314556206,55974.75222817203,-107107.01601290423,408648.645503724,-472883.6286291075,710750.3808818869,403405.0948282783,1104302.8364829444,429832.7678636261,892765.9384541884,-480895.18953242665,908962.1140191113,-147855.10611904226,1084524.3471886525,318444.2947794958,-61514.42819697177,388957.8645442161,671998.1293072393,1048169.3210745417,623108.4607587173,527366.3304942306,-586639.0589251127,-384657.4997944203,-99179.85856992193,-1886.7355686118826,391520.2745744516,-662247.6275903708,-54887.0746781053,19034.21281504724,-104592.19693128997,397680.16440951405,347569.91480498016,-276197.60317676,176896.44973195158,658110.9952447833,-257648.3573106355,1492749.2246020287,-788673.8878363769,592915.4450065242,-617631.0210645455,46405.01539203012,447485.1448237486,488542.2015556828,426230.53574015014,257837.80527353706,731602.9606922893,162849.5252091228,409038.4639451504,385966.9911613497,279458.6238085418,-222516.45637177723,258174.6152107329,489113.52385594975,175989.03315312928,-563503.3465385777,336133.1166050243,590098.7281936291,68413.00203246204,-202473.4632760454,-232120.0375599824,495952.39076793613,-486540.35403825715,189332.39019623026,-297153.27196318214,129996.04948254395,29236.591148334555,-12051.900480943266,452157.7756357966,306060.4721579831,21295.32614025986,324182.23181283986,18494.87181969639,62903.806665482465,1010559.4244258543,-522136.00750077376,819648.031141872,593608.8492796039,346450.8606747566,388238.60961174686,455963.13524353085,-991723.4084075633,226649.38867414184,464696.7326551359,-584897.7966811829,187905.92742529558,-383273.8174744821,637474.773925181,-249662.09893475985,-278418.5615262748,934296.4888164154,-136767.1165469233,-730565.8556474,-340487.0236635152,51507.24220668711,267771.36867968366,418925.006003933,190176.2578021465,39521.33145731222,-569846.8999780798,386895.3937420482,997178.2605573353,-213282.16969866538,275903.43374349223,439559.2357327556,608406.5644425876,-71666.03497398645,198844.73235713923,399747.6819337229,822067.0017533246,80280.53076265194,-137196.8334645722,-237010.92086336715,421772.07072168356,-653308.9305400825,290624.36665093526,-292820.20255770674,332334.250496909,-243698.37272782205,72283.67206938332,-89908.35653908458,503652.40464193886,427505.5920794667,252163.9706241102,-184800.34651258634,305064.8741501104,-27744.270893314388,299853.4930203967,-480385.5643770825,-46662.39573698165,436467.94211467635,230692.6533690095,-198372.11080339737,28474.135097406805,-13250.612130470574,990206.3593971874,-443101.2378765987,-12384.26386979036,-185158.47138134437,345971.2603005371,-230158.41407610662,225129.6343522882,-64274.35766054597,618924.5847605946,366915.6795939035,145134.48438583058,-60328.99524102127,-395135.3748037019,-314579.5043618558,603827.569559304,450563.56038558204,309741.11434030207,-59441.54810730275,938780.0924117663,18262.443140541203,298907.9683439769,161765.16372472886,-522050.07722549234,247701.16138686426,74381.3023864124,-149956.33663045615,256733.82379596867,-50946.882442967035,792697.8578325985,-313251.9894758733,153560.37868217845,-23954.55676328065,-208381.55004316056,621100.9771301649,-636175.5892087831,18469.366480085533,-235838.5656913207,115534.847182584,93846.89425887587,387979.4794637854,638442.5558903627,285969.16959888255,393003.84088824363,753962.8920803508,-386739.0997644244,221397.49626673805,502791.24297666317,875081.9976775567,-233701.21375691332,304896.23962677177,167609.1593275359,740470.3664133283,-470889.86807910074,1121152.8821242999,-314740.7637722972,-68460.36005578516,379055.61572690075,353631.3394495612,490614.74859936675,504106.9169733124,-412384.2851733421,785091.2425008495,235179.0293235178,383308.88088597497,549276.9252076051,492180.1774292658,-1228119.327970831,504214.727356181,-311637.626196519,810650.1392938108,776500.9982144586,273592.56323381513,-410891.6525016837,333959.6296737958,-52647.57174443919,-312283.3601256632,-316485.63215218065,699007.0219948268,257700.9838170167,-550278.1945200483,481399.96702858014,-337340.03637054935,360876.91726879356,580218.9319674536,302600.5413179416,241491.94053360494,727738.1947260965,550490.1151417582,-111537.8680033707,-296676.0033398983,-318709.6781111248,1952.1220228364691,727712.4677506266,-185062.948741714,-214150.51021757815,986532.5541824573,74644.54641740583,289898.15550011536,178014.46071112063,501315.50562096573,-42368.34318956081,-438667.8025503135,-781652.7665890646,-175731.68497547973,75020.74657432223,-104948.38258987432,330994.5214293618,-208017.8865175238,66796.63080067514,248998.31417538272,-377835.02031295514,373042.26154629793,1258940.4295292185,-376985.4209765252,-152394.9014329319,576732.8348334674,556227.762012824,233631.65299142106,-227060.34733002633,394502.72846808657,563239.5545238745,379444.2552667018,211668.71307569975,369923.0059216218,594044.5189131382,-168453.72152759042,-280213.10015232954,-763222.3579059057,-605536.032385489,616094.4984800825,91030.32491167868,-519994.5943658608,796204.8352098605,-86767.8341105287,454627.4189391108,126058.4760934487,585235.1477396018,-236947.73422067193,-365818.53806663165,87810.17037685309,-278057.7679466903,-388618.594913532,-449358.656933703,288931.6035362035,67775.10878932336,69058.61601658631,300729.5973929791,-329577.2064149589,1094684.9779981226,639844.8197611701,-394038.4125093124,-159826.49466346577,182970.59339874238,-379377.30461381515,-567669.5958665237,264978.1532428982,180260.1865577018,568316.6677199979,728006.8527710433,144918.65357265435,55329.265878872946,925072.6942377109,302258.69059137534,389508.2232304099,956818.9586776244,314999.8694599578,175884.49321123306,775458.9598277677,89354.7501364965,-863395.751811943,76590.41692890134,626152.9577590623,-802493.1856965583,-158303.79486269504,-58620.702417551074,214096.740237684,75017.27398054162,191822.82848362532,215487.7058611503,338902.4177655927,-113151.38262577355,-178808.14271706948,468206.19227671856,726901.3662237842,292798.5308922408,-216018.09699733928,173062.1964314133,313029.01953184465,141132.7995769605,-647369.851728416,596564.2756080544,133292.53219577298,977943.3632278852,153482.53576935455,611969.3269894323,981524.5135511211,415818.6591676348,39044.107745760586,-72849.48077426618,-148247.68137901137,858250.4894599956,-844817.4860704935,-113746.86047738232,-33175.23833762342,157243.34112150548,98235.9025830985,597188.3780752677,-12141.020208358765,209571.3985448312,139042.7845253246,178654.4459487009,154827.2829560209,-77129.2649872452,728479.9176598014,-763093.0819230685,72551.56673006993,1194984.4946871423,499082.09809904965,-432943.1625758447,-51234.51085710013,246588.9806640749,908473.2943922905,-144585.44464975083,-346498.58468745695,1035825.8433364546,835060.8631685958,696193.2567316308,211673.79008217715,273680.5264228573,851579.4248836217,-354877.6239423114,-135009.366187213,201329.02703665756,401504.1544635077,32619.16493702773,219264.93542833067,393980.8015715806,59605.84147510771,722559.0994887832,-267196.56984571647,217345.58012515958,-400034.35806525685,-396001.0768416277,-203450.0910998555,303069.3986782953,507755.7105995817,289229.3004334406,87524.06786893308,339232.03113174625,643124.5607704143,-84690.87911604764,260767.1714494126,-500562.4516738993,405095.7188663399,708280.2344015376,-91196.35438876133,-157936.9700034163,353393.6686232807,109711.87470825994,339564.0729376953,-133961.14876159607,-128941.01783012133,322459.8524372759,455446.2500481177,-173339.00996975927,442131.571263331,423566.68248568056,184930.4083594815,878216.5057135464,-313170.438024546,288933.69658759143,209027.48696862673,338729.55276405835,-316890.7673387993,365073.2330953372,292873.89812726714,56512.1926201242,613817.4887382798,72730.82920963038,239266.72882351652,359049.6576982378,312010.7012074678,666536.9601717182,335580.72286698315,91167.0069592637,-491448.0720325331,-594789.8610225888,331219.63260277966,269735.62364616897,16944.954772166908,911025.64297863,687796.6519636568,799701.2224236359,420076.0524649131,542535.0883635622,-656767.9175667032,792777.0887351804,13521.834068522323,523766.8970810678,-5633.398231414612,-348312.67035362823,-477228.8465510011,1758241.6711834362,331442.30271231337,-677351.3087662533,711464.797419908,1053539.1993488362,919065.316292366,1020468.8520814944,207843.61785083264,-220319.6684487462,-265198.40759312594,188129.88527664682,747794.8793874453,4203.334788219538,1090276.6080460818,495949.20999460807,31996.27278777538,-166086.52231611684,527849.1110218451,555019.7777747717,-659915.441112875,-30574.56958079245,694173.1781137371,289555.92313381797,102201.81967691751,805900.7993118372,-102508.75322669744,1164785.4422631469,202325.64749489632,14144.52144516958,233393.27204667777,708177.1471915725,345634.62316968245,507227.6992511521,467894.24275419815,960928.396764298,-151051.96651934832,238593.777779466,802688.5272734095,716392.271015584,-380500.6437149518,863575.8119751536,95521.65382665396,322389.92894691695,-324407.18210350303,110785.44796512695,254275.02679514838,145954.3844382083,48216.7624910716,89613.26234255079,839808.8110625967,221755.94446366653,-42822.32000932749,-89931.54350259155,77774.35577108804,639523.1151479669,427258.60988611076,609421.9045893508,343298.1104843314,111924.09815336345,696647.6746403426,707869.6238122182,416663.28189215274,-443921.3121775659,186406.7731611943,903516.74900254,28162.533934514504,544044.2696472206,720803.2998950928,-401506.97612678586,27444.623552449513,1018685.6104146736,487096.0309866681,872044.3950778884,439764.36987001635,336853.05727354344,734277.447091646,662076.1047462774,-376938.9310043552,696662.4814910819,-50489.88711030362,-84793.89945932664,735233.4145080489,444746.25562452665,1305123.4784803204,-60639.42057871958,143265.71560166217,-539606.9337410294,833921.4005819256,88114.06856040563,190020.30293346616,-136463.68380554114,200175.13056006376,373913.7539297412,-21608.351885108743,-189298.27297856845,-454546.48760028975,29109.265864596236,-35261.68575716438,1241441.5404273635,128625.93471533153,351870.880833677,-35024.2688125032,774038.4386813189,-136302.99452024978,417323.7367788241,-290777.4794993885,-16111.329528298229,31914.598485208116,314432.9009262165,-612081.2415028089,874364.02849977,443569.35855311714,-283278.89273817325,-568884.9848273923,528133.1412846982,422870.2514506085,752436.877221399,296131.4356108587,889174.6487193098,610213.4593015681,475670.72424123855,278539.8861342706,-170581.50453056628,534785.5294275163,-369744.7445912366,293969.19695384195,-407606.7383893505,843362.259508878,889448.993721934,594756.1714238687,327800.92048574425,-133396.16713805124,114850.71752477344,432416.40072595095,-566889.7647092077,95334.15906063747,8245.922031366266,492499.54235923477,-88286.94882233161,-338462.25443947315,96412.96011268953,-132944.06236104295,290698.5266318745,442960.76617247844,-86565.84866405651,1133683.4947963625,-379953.7259794008,-253868.8728847201,-57899.30888539879,47552.89850435639,436118.33583930135,970159.7231244431,109852.15920041408,944213.9051978542,-103433.8156039198,706171.4807913327,-594549.8086263561,-212293.1243473026,857969.0944380406,419309.8272463335,243294.25396019453,77314.86276599253,260854.1773234685,6901.459423741326,149483.84517594846,-36943.64836238185,605030.2665175633,-430238.8103928631,-118296.81376789045,-122226.20658176346,-469479.0681912317,51352.63538396126,550162.7420249283,-138653.35204187594,756836.2387232035,774495.7188309641,420764.1802669666,426399.6260410524,-497212.07801291347,-10531.19545623986,476376.46075953124,-195842.02705436014,-397073.7842091508,136021.00813413784,-481297.4136966523,553766.4336654712,34318.701299294364,-20040.086471363436,-210384.79802294588,-458380.40123922145,-58802.34198144544,-10288.276293645613,146917.4593214402,-7920.142089574132,-500533.3041605288,-752250.3925359861,-376987.28977389727,359927.5643637115,491424.90933273314,382490.8361707791,-394832.8689246257,86371.25402561482,134581.56151023274,-567938.2207937345,-332878.70042861346,185314.7037403006,-464262.39139450714,-475273.87737507746,-132089.86148102488,654193.8594371537,-103159.11258025281,-57402.409616929945,-328369.31161986385,-115300.15881446283,505586.0825753445,866120.5031929184,337491.34326812904,509424.8070557327,411853.4043340259,181514.2733297455,-673136.5621335097,-147993.04686893336,26977.356432447676,-826783.9402564857,462573.2317835111,-426384.91817171127,165949.63951468887,-701002.360646714,800665.752580076,409698.5737287393,468830.44561826997,-87383.84466946544,-59987.623722952325,-264493.3023326625,-380538.32742038835,452082.93249435537,-240405.7528981259,-85018.08970100805,153371.33190097287,1433133.4463997602,51034.975991431624,519101.5464656297,-338887.9147718786,-421823.51494238386,195263.10618167045,97107.26261368394,756744.1315135933,-84917.72055903636,654383.0341564934,256277.57278489554,-697539.0693506096,329511.80837092875,1542272.0209529423,-223722.1789569538,-48897.99271177454,79618.6679711435,16026.870078690816,-376263.5209715462,580678.2984760194,370843.4116143938,1019804.8969029277,815371.6642445531,436810.03519932926,246068.8813198409,696106.2126902156,-53582.036950809415,682526.5429717158,406402.6494392962,644936.3731366876,513932.29032897577,688912.7190321642,294250.96599047305,941426.3440103442,462314.8254551622,176235.26710327296,-478222.2784882607,-224903.67022821587,-421925.6073748674,520743.44694911875,-232282.73559964262,1039033.3680240717,-916622.3174721403,665300.135610078,573045.4944745144,550579.1856259666,280676.5284520807,478371.87823192636,531492.5923639531,388185.5669064913,815494.2398630334,744012.6224319125,751731.7769953101,354058.159790738,462611.5298450319,445806.3978635422,-65663.505244751,148027.3138926914,1290799.6102728082,863952.6995593221,-296708.9695567917,-216455.67662264686,-664628.4096546625,-67870.98122301465,254489.12402296858,-67838.63358717179,-181440.8741090987,-129785.86785196699,-224466.7835509805,1008109.7153704823,757523.4072722774,467085.2687484403,-95565.84787156899,459882.2580596674,151736.33508893894,277665.2018741979,638357.1451531593,-503819.84377819765,169284.6949222777,-47741.60561387893,220489.3186753802,-405732.9397025928,-211655.6185732768,399101.4207292395,382121.00177382445,196618.04431701778,-96311.04763178388,560061.5950113698,353048.77230863785,564233.7433977951,136566.86349835992,842027.6971074776,487656.87328221835,364396.6635575392,-370792.94221287407,51860.928949703,-2379.1894914824516,3667.91171185486,-227950.0988887453,542356.0177743924,1181175.7890764954,-445860.25374040985,-724677.1739162311,563686.3101424365,457051.40813557524,355571.3619676102,-123293.0110689043,595420.0949627017,685983.0607444337,122027.327178068,103320.24953149026,-542967.3107483815,833353.2185357059,224899.73693566257,-610392.3742811345,-389361.50317734573,134258.4722078489,202974.409874266,-558477.748206012,863025.9193147863,747351.2385788304,415809.39148256416,700735.8507949002,644908.0191539964,321233.3496425417,-161864.74091405934,325804.4777908998,911815.079564373,839795.0698924679,-475699.60333388066,337148.0235546939,355898.5087384195,529004.9260647395,169466.95471634762,179762.87638272112,208119.10751491785,-703349.5738499267,791253.9682288882,-651202.3748046614,561260.722273543,-280191.55063786404,852400.4694868834,999865.7939754268,809462.2294482272,331981.48250211775,-165387.72840060946,-110744.24286526628,-352290.317679503,420801.42259031115,-24014.21080625942,302369.87812548317,738700.4064272433,608091.7543284446,-476795.3779137451,392289.3517072168,427527.3727588076,162525.24896235112,-331246.1423763237,338972.0604745196,-178657.15069948183,-412771.2209441783,38019.66044709692,-18889.772852447815,263971.1618949268,711050.4617920839,811374.8811813509,670946.5655663023,-276445.1184973642,-30734.443159683608,-102303.43111435557,-413522.969448111,922307.7856785478,-550547.7164229378,891265.5818934725,622849.3187308237,613316.2073546448,-819251.885641518,325663.9317673915,727123.6330512669,-332821.2391467495,568731.4790976215,-286385.6226086598,291965.30468680663,962427.9414341464,277368.104147905,1097777.4462214862,570047.7617590972,662367.2046908732,284151.6296383389,701913.1847411403,438266.4521335275,-1012408.158949795,-643999.4324184419,315945.61164820055,615658.3281690977,591224.4697349365,357222.2128882846,669741.7222018321,1076314.9644104186,-42868.79221007181,-87148.12546104472,-836336.4226609706,-666292.8791333756,782766.542379925,304433.7717352314,-10510.401202937588,-544285.7479438093,102183.13302305201,-174764.07891171006,229040.5227233339,550253.1682510208,236286.1544855754,24422.33917231206,-217807.80932700634,-73535.09745401004,136923.19963903958,271444.58823539736,58946.6684439946,260428.69692207687,-317498.510774754,139400.70224368805,637916.5936016729,416594.5156091773,125564.8017593259,-386659.53789958684,-180567.7170057278,290775.0688064075,292590.2933888668,-431623.2255575992,852008.0908657159,354975.1451329035,634124.6351326345,40933.014201367274,145080.13024138054,16490.725458630826,-656023.9638855685,-145650.38543807343,255557.66395273712,-677520.315641881,458968.435265325,-252975.22857129388,65853.37559205852,-70551.89445246616,-577562.4076340962,805451.0539252209,458737.95824698266,146284.46938553778,-57844.20765444962,-422305.2433528742,558916.3092703777,-131497.49033787986,949822.8889394025,326766.2922008387,567440.8360470734,239638.53227423504,531304.8605704494,643862.6346365176,444658.3802890084,441529.30411744677,788874.7353949021,566255.5139610884,-228409.98785487423,-542.1987678548321,819295.1541056046,716313.1986269462,9730.480176448356,296826.2418694813,748149.2749160859,272868.10402538953,12519.550555430353,-475805.4472780372,-278847.262515537,431946.95521604316,74530.66952346731,1094322.7393378317,722799.2478367849,439873.09376579477,584039.7918271241,-388764.206557794,817149.3713611439,279145.1157487179,-340356.21087899944,-380599.7278920086,717823.7027603327,77280.18760929862,-574567.9376699179,1058333.8644442013,-226690.34080414055,577139.589015488,765485.3803666476,151771.9698951547,-794053.905676093,-491606.6594923986,-640043.5295555892,499557.8143660221,203722.2163305264,-741711.1689021811,358370.67190892156,161173.08339793282,147473.08328752732,-82742.15217822976,142479.18720634328,-388918.85548675247,529579.4603367345,907807.7415188723,102263.86303631635,423857.5568455579,-23412.865716592874,-67843.22172898147,836065.4594665575,281545.2607843471,653639.1119301529,219729.96749484167,-706813.1385184368,-235279.29047866957,393813.48235671036,-195550.62353512552,729900.4848595215,589184.6586536281,554465.906833176,927240.9403744149,-668156.4694272024,795888.0097466004,-499956.0734683885,467240.7509352383,-500193.14443116914,884019.5236769752,48427.265870204195,-458500.986521482,829895.0485311123,680395.1331717311,-6978.901151823346,402385.90185511624,-19726.286957015283,-323162.86981284176,-260663.53358931467,-65035.48324215878,361206.739703191,38565.78093979973,228418.41758675687,-606725.0608278052,-260572.8523137006,-269139.50417036284,812373.1890044813,-362862.66130013764,386286.9414846874,516153.8529614983,-619443.2416892457,70570.76882739319,700097.4883524952,-390573.0447405791,917369.5995839476,610056.2262153341,530586.7308325013,18737.49250163557,-449428.7101360271,-542942.7951245112,-146369.44112753775,656491.0362552749,-258818.04602977354,-123492.88767090486,628717.2144887312,989845.1037509288,-742970.6930987472,287879.36137248715,-621935.349211989,195167.1570131476,167252.3374012257,100803.64991299668,819998.5666863383,481093.6455144305,-540600.0125108664,177684.13389742328,643332.4877333143,-98134.88599630725,521772.6248418512,1240.8289978057146,219274.89022124792,-264253.4037892278,-211623.54977940628,342575.54930090765,52314.94339706935,80953.21573094558,1181401.0168825304,49309.93154445011,141259.25002340833,357173.43384321826,-213624.42563445354,540426.2250710237,-386508.50274006696,198867.33392641693,-352893.0851951926,536605.7651043441,-17578.477713530418,110771.55395133421,1008499.9202724085,256543.3085872787,133858.74818145763,614982.0712258401,199505.45923910802,-498257.1035174588,299666.3022201406,840459.097446145,406577.416135828,517463.3299977798,518534.41836524615,41421.18154475093,19937.503095155582,-49797.44808250386,36977.23847908713,596645.5671205241,-430155.0976515156,392484.0085187629,314288.87330764346,824936.517356629,1293447.347086125,-507432.15449694684,517966.8454131987,-220757.20662000263,332909.11137073254,116393.85000650445,238848.66105252597,681414.9216836607,649469.6825469262,262027.7507600505,-457982.17546294467,406063.51637070905,31909.43616080517,448225.88342249114,144727.43555705342,337887.3292115894,149046.4354424174,-249376.86198195908,-446321.4764151047,119618.48486065492,-335710.24347198615,-651693.0775180813,24541.57571174437,-452309.03231774527,-355186.0246353154,987175.0892622736,-376785.2164265793,-171388.06111259945,-649366.9537378864,778517.7823725026,-340801.42464610934,-443615.1280481932,944233.5678811688,-17526.077585505787,-232943.44798192615,-213193.93399656797,615839.40585797,-253396.98616497638,796216.8766680076,342118.58971994463,-188685.65146361152,-531412.2722426294,825515.3869179995,805605.852425837,300791.00897036586,321031.4789515431,122814.96893248288,427282.3949402296,62741.13231108058,27640.650735612027,419437.239149387,997291.7368428498,255992.86870837072,431783.46075126436,92082.99730764702,832937.5306873727,-215533.7807977777,-133706.96861189976,-746962.4793351553,989366.8934759619,265217.9825713895,227207.5368242846,909964.0821863343,429963.2694619838,-715257.8670606003,282526.22964802943,378618.1558770365,-1106837.7324337326,-551819.2905044472,-305010.64413781324,334959.91208682815,-502889.2941177287,241325.79354595952,-311566.07849304704,158559.70822659973,496523.1455267011,295902.2483698395,55782.81617661333,937085.518133332,61761.0983259147,-420415.2249146532,-92251.02033522772,504729.45288368873,758030.563651219,-211868.28287171386,-150716.70357543975,263814.2974343607,32581.90118585946,-130491.62445935654,334556.149170727,-134550.1685229526,513443.21793519007,-29279.73213119153,167441.69179176167,711898.9940186827,262410.51207699627,522213.8943941961,1032286.4500620458,289309.7415943071,-234241.51297034044,581699.286832714,139889.99732988002,524098.8230057694,667485.9879303081,485989.9812876014,688279.6400342956,-590139.0182875465,398476.93862593593,684990.5947860335,108909.22356800921,490878.6571725863,874872.8205688563,856815.0219557113,-470917.8180536586,569633.0760498624,450352.0599663928,285433.85709930724,-61749.74122506939,-168098.33290940942,1067428.9604184353,-207080.66433195816,22644.40007636929,230776.63285082253,-68124.62196742604,239851.42912212852,-643284.3862290285,46695.596080534626,411094.1773381443,-764055.990552102,29930.0656668921,489760.25725380844,-128649.00629714178,514529.67383389035,677744.1094755004,-452075.4699408384,367787.6275755111,-654095.3752001459,480284.72524653375,708091.5406859461,-68581.73898747051,493582.14336889656,715419.9711851017,547497.1374462103,200820.9539361433,-180632.560587015,1203232.3669715086,514561.22808321984,358196.9388111639,204185.48387144506,-168218.8815554683,-320045.3981422656,-292692.78770765243,728101.7836432294,599856.7089692801,587529.5947267083,212155.58072052058,-5651.619008305017,-447958.5564278406,484664.0481474367,712500.3335789558,-73526.93310371088,-437903.1623513326,589394.6704467204,64814.978313467,-93264.4435861092,-179533.7172887465,760655.4005171643,536685.6812730758,665519.4704280379,519672.44058995415,-268480.5752734258,-346278.69141371245,389557.50559594715,213702.61479567736,-369949.6354264668,488424.22547357646,-330590.92169298185,762505.7021487192,-98034.38302209042,-215566.87473415677,-28437.77849614201,-77720.5260359738,376452.96738227597,-572244.2588721616,-431755.77886015736,114429.1470210934,554102.1209981632,744353.6886150595,940882.095631165,190377.49209195934,-153089.42686346453,-979234.2352843494,264022.8279422461,435427.79639677564,-158822.45545065217,1045794.3135762671,12292.498278884683,-764843.3007070329,335.8102016672492,-315942.96653433423,168301.5843029837,-155410.2665844066,-318064.3622163981,738267.151941746,31881.536822847556,119707.53945444804,-334767.7897565854,941889.8092060797,286141.10864194855,397576.3413167475,954389.8581432197,580077.2584539959,471203.8961536791,32218.81231869897,-393025.11341170873,565121.9645894775,-280573.7401649493,-30555.589126832085,21318.184171185363,-166465.5275564338,-154415.22058757162,57445.382379267365,-766815.6032010755,-17313.79445054615,68738.7293932666,-131297.14413325442,829855.6457694415,326563.53008885216,-304644.73169750394,-213641.26619561994,1174078.7927224254,-378321.9361488875,-581493.4180876156,426447.99256978417,982104.7568288464,726319.178540655,287526.6624409659,15558.434762202669,-628268.3408780945,786469.2447945871,402888.888240085,370724.044029403,285564.5460189297,17785.16741218185,15611.546132270247,473010.7672853796,234122.7638330995,648792.7448100233,485821.106572662,639566.1650758251,102116.94910469884,-65077.042033582926,300381.1983581339,642475.5333749293,601243.573938496,364144.16706861905,567342.5657711718,-1501.1617938429117,13943.654026377015,511893.67444979167,-632919.8997848048,3538.7177625880577,-730625.9717196357,-156319.09045390924,-448870.88988639927,89746.65061797248,-426935.8667587796,353888.5414596349,-166011.409804462,360998.22058935557,-346780.2683237945,660307.705895626,-288578.37860809267,-373566.9338535187,-84708.28550834954,1171210.4474810637,-519869.9071122655,-37370.5056132013,86008.84080901695,886218.8639563206,312219.92662882805,228140.04616163624,-184097.81177499797,377606.4205784453,359379.8761174353,-428895.5180359762,225517.7476605815,349771.20737991855,-83675.96560482541,-105406.23774839938,-51846.717403492425,322012.4500627802,188336.78385514067,-635758.9698488,-493918.1444024574,371032.77026116103,-852747.8935380033,-435578.4480199707,-322203.0780587881,-122322.93819588842,82421.7476045913,-177215.79304157896,753128.1740341485,72527.00218502292,592797.713297212,511557.8856876432,1081159.6285331845,203188.304603694,122358.81937781489,385400.65168595547,624638.5080738231,-59349.726183292456,562857.8691433333,-97215.6443735431,-415382.3309582728,467807.3287450457,-295332.8428325667,451206.6177245667,-778630.8588988236,-150177.0288124946,151006.4446308338,1003687.2899571075,-19903.939736846834,892078.568948112,114930.92043062858,8287.094291594345,-623396.9410809865,122384.3170289523,-68847.26630153647,1060842.1895225523,-63778.275269020814,138843.83750273194,721758.0372403497,502925.31417057477,961057.3614327195,-538516.6557242847,944137.5185313271,176518.14330264414,641980.8518110523,57517.98511299165,168765.73510074615,361919.6822443595,-251351.6599385552,-136535.60689921817,-57515.983767807484,106050.3046870213,122989.31411207002,396517.1160189123,186998.1852525347,-585130.1181301884,382063.8287177724,472315.8479719884,798639.9821403637,-271876.57611639705,-134238.89447678346,918298.0334347584,810346.2530430942,-125882.57816675864,427262.3584164763,-513264.8709143712,893177.7442554114,531956.3009593587,-235470.6492392337,130864.36607604334,95155.292289373,189795.16119135544,865490.809225867,593232.7392837261,641687.1930751204,708708.4376466912,260498.8331929436,-226244.4828248066,934564.866397595,873036.6261970955,-577247.6859288379,-109752.92631916516,644854.8910768384,308602.6539351875,-179185.04778483277,-202865.94089710433,1892.8961720298976,294990.1808119137,-96032.96183897043,-573215.6649978962,224056.48593090987,9459.022930317558,-251006.92005752167,1597.5271283350885,811873.5949141751,-386618.669032854,115020.47037673322,377851.7952353307,-244560.27902012225,62933.613977661356,-151272.32133100554,-93374.47563064937,388227.3594798343,-470032.7754383702,-521926.4552349765,43017.67871578364,413630.7401180845,-569458.368105297,-145892.96381667815,386.60180258518085,-229918.84781141067,155850.56673485273,815731.965620806,844014.3455931228,680060.7874002336,5821.185373231303,205176.85617537284,-610645.9232863416,-643435.5059632137,202006.85610703798,30781.98694359744,383971.37832654826,-243913.51961740293,792366.4583830452,-275065.8558425633,149039.173872686,-108331.63156428142,503450.734332473,-56340.007502766326,535399.5094376137,-881931.8035184597,740607.1462958953,288834.5089161941,-201332.89730834775,595357.7382294829,389701.8299294291,693218.172905555,616808.2908088136,490766.3581164088,-61767.45115242759,200804.26248565037,49252.149470723234,629933.4927913998,605030.5389613551,-83617.46167199127,363511.9353298149,241883.87097713258,103033.36025047675,26786.63691259781,-302426.64114361024,1540819.468896973,666444.2365565524,386733.4926667856,-491570.04176699556,-264939.50471258955,867255.4878028212,-649069.2330566454,137114.1190327527,287788.81816491904,85560.51944662817,773965.3437385121,280863.705847871,283254.53047718806,57706.06228464143,605836.6460610195,737018.4324819259,61096.18459935021,-114665.49863113463,327619.7254043147,350694.1907233298,187071.59399243956,302476.0433945884,893787.5130491313,-235589.81756227557,-102543.42588826502,541559.5607089591,185660.61806956073,409418.9102368229,-8301.67702227598,143708.09870553156,503837.55508380337,686581.9730516267,646508.7749637282,909460.5397625794,340119.6682731756,72222.98719574325,-25588.659093104303,398855.7213964253,-442552.2660422395,48664.16845617071,-221020.69330645772,233967.55762008764,481097.4972024937,398868.146649315,381309.2417840387,329289.4236516906,748035.9399847924,-815769.6160213351,-172575.04423674988,701117.3434694842,68334.51654044911,217938.6577473632,646117.4191849958,336477.40578128444,-257237.7179686213,680573.1818473195,498785.7867968562,-95730.86344626127,220478.78374788584,80987.08736995421,-282175.90266261017,508348.44450584985,-95673.4574718182,19050.36209106818,285041.96014890634,140997.60700442689,-187793.56768399477,836168.4657260831,466989.3224099395,417099.498845465,-150592.99757465813,-476324.39416633314,620947.0865926542,379530.2584084552,-781929.8183214685,-267169.4398846328,1212764.923579746,471436.1143792209,126222.27663183119,1154299.3453612328,613039.8485244154,853081.4005953344,416907.6986171827,325749.6689715828,-843864.2516428838,-113047.94806235889,-70891.42995188897,-627291.3103994024,215856.38952253107,-287340.3421409265,415478.44695388246,72808.75813648151,1051049.0224821893,78259.91142201703,-211247.18799506547,692432.0322823543,17761.046081055887,22711.56335333269,599408.0078145466,777682.947193156,117396.7714634738,690495.5081647942,-181063.1133291861,603438.0299067502,418625.9361308832,-333004.6779646417,192025.8299497948,436071.38390233414,-369036.26234452287,223507.89299298497,144871.17954446655,108312.57407867862,-153291.6972331917,6651.912421253044,-296400.0052488581,-39913.5417665164,-849256.7319092941,266157.9596840604,-388508.2801330122,-170966.44195794733,612474.4739304534,917920.0536964634,80184.02085331455,530082.9883033824,814551.7372724069,271408.0847566151,410512.8952721786,-436559.7822001125,422903.5060295267,467296.47255641455,270960.09324362315,-247497.40509537607,-362789.7902051904,999681.6313578552,324018.6665188717,-299323.1179282479,634624.8694282817,242453.82705602096,664055.2861553817,-106158.69609919889,40381.91468465142,251200.46884644637,-341231.2840964673,193268.61563611915,-146728.15484394878,501479.4301828467,290251.1664720746,427057.1286445148,879935.1172682913,-708246.4526317287,499778.36963444157,300440.9333830518,-9119.583838158287,-699213.6565988334,-478651.73394806497,1021766.2592701805,-15667.902955076657,-420567.3215012662,610068.9433604777,-393344.0383305084,563280.6108478257,634548.708782367,348657.6586900032,61561.082251750864,606414.9680340658,486079.8621597127,996673.9329598891,-183746.81015530694,523062.03363388777,71178.88328871224,469414.80168932397,250423.67195214145,-254767.19290236942,-95749.95658711903,1001980.753051409,109574.36607218813,612042.1630302612,177878.72798122885,-455379.8707220969,400660.0886244476,242628.39512686897,126016.08314629877,218525.36393538816,507740.4640584127,-539442.82975253,292648.07433382375,-31645.173602610826,-178936.6214131941,1413451.6825892315,-107976.45258098748,-461706.433186383,-178033.80826230394,642116.5251830784,957737.9266638425,141379.32633051416,328963.0705515733,133502.5674603223,1343795.4187316378,163654.48760218197,827785.8058706359,-644980.1635246454,10790.879682451487,44435.44050241867,157641.7747769812,-213368.92695405753,652764.0552268522,174329.38127638772,852981.3062481107,1311220.1649256444,674063.0980464234,1139821.908687769,-396892.12183811655,474099.7200622256,881841.8742721523,20130.19050944643,625234.665970719,213373.32830013055,351912.8020231286,466865.8053426165,437852.84763012547,-739006.4869631757,-500093.78969905945,107363.53454234777,656017.520901877,1200098.4040149692,-100702.44792815577,73252.045655753,671350.4519016072,310028.3852449381,-470107.69446367724,-219723.39139388502,430796.0630883821,-494349.62021549605,1013755.2772713292,460101.7278057076,618040.2316859113,811115.2993766181,266105.0192172243,107478.09315878898,-386844.23525113845,713044.7553500533,557537.4015765502,147909.81805317942,80971.81165983854,176072.73147826828,36343.35601853905,359324.4761939887,802019.5621719612,418812.2618836933,-230168.90190669242,-492849.3379804408,-778371.5631017275,382401.5531443218,-107281.13028312521,748941.5287035778,974355.7381809624,72912.41022285167,211913.95680929208,-293334.2411355595,196048.5689597479,355257.57935896236,41179.59234386077,565160.4756961376,-172199.57806007052,-605836.9059414617,197880.36688151583,-422000.8137606359,-25112.357303597033,349317.40638873447,91102.14269502927,339332.5565912025,10896.151347529143,44546.406566035934,448098.2669016919,-274224.631266179,-295690.3002210972,297452.555085022,567288.2911353591,-169452.7806672803,71113.25659165438,-120327.43441653671,-50375.772478598636,423942.8486207882,250327.0987330312,284982.35239092866,628624.8921060516,153180.23255725112,780281.5264772517,-155306.13537574885,134946.43208343908,-67353.14596328791,819782.9546047519,91908.00233167224,585535.3091486879,972794.5399145186,-179517.5265690689,-405699.3107093228,935005.912190157,405461.7391491397,-82665.35609098803,-468601.5531997299,664475.8098932533,2773.199334261939,373235.8258612384,43132.873037470505,60454.809707154054,80948.83612662274,-261484.37940419558,231816.72132738028,-268083.27499020286,-433514.69515087083,108043.54367535515,194825.87107758503,130136.07179235714,709575.675142732,537920.9950829009,184187.70001214743,-139618.1461890936,-114864.3978099199,76378.51497929031,-207288.91199636878,-122298.32291777618,239703.41419093544,145821.45503681852,-837950.2931445497,762441.8922343436,-700781.0938184997,294026.424924416,226235.4766958924,127090.65646878723,92821.27478062036,193920.89054433256,207092.58709266223,434660.60284884786,351074.3222405473,-225303.9583769869,-712135.0957253405,-100388.52570569655,-554966.9290082874,-153422.88496082043,-458528.1059274506,791060.7498298967,63142.86256918032,525967.8518979135,-118876.97369085485,153369.83966035768,695752.9510914264,253595.10469754506,-293752.48288213694,-443445.5167901269,221268.25530571584,709745.6272516488,102460.66031534178,-567799.331283831,726488.3177888114,77445.9330794895,-196246.04502664367,262953.72549549397,165680.56024280563,310970.00753586227,418077.09785793675,579502.6909609395,709170.9702031193,145327.46940163616,-345218.27633952256,469389.5257539209,874357.6763303825,1212421.8814131217,497626.30867528915,-9174.064985958394,72346.28963355208,882.243029284291,-148673.43795294594,-408400.4906737618,544107.9083436844,393253.99546693685,529891.2704428402,158628.4213986653,-956750.3153739693,199399.7664077012,666064.0178858135,944502.3688415061,569497.0706701749,-101445.51067218743,395914.78567100177,373050.26390689425,837859.519343989,-234834.296758499,788179.9925267603,-99143.22355407476,478184.26894294145,-136010.6484190058,-261907.580705869,1238525.3255537087,-227660.97662717663,17122.604314469267,1213533.4767961255,677110.2253102772,6234.113259609323,67160.61575811356,-503967.7503498937,313116.15654151374,723464.8304888336,-56238.448533938266,264999.08101413585,-517611.39903087355,319577.564315164,117401.24383839313,542748.6024955423,-23591.188910050783,264832.606064701,394066.9102063789,464739.87559395796,-201680.24610019382,-129858.40443056542,314777.8295032652,1282876.336320768,47330.982390758116,309323.33828502614,355131.45740397833,512117.2486289041,977706.6800615853,418373.37324141525,822599.1068498446,478520.5247194837,-524361.8425584869,-88499.29273400782,223382.05439548846,70812.60758978734,110786.83901335578,277441.59361823415,116170.66788949957,199547.19300660025,-308446.7861767281,596442.2149920589,466300.33632748947,302011.7088220604,-288135.83688911377,605236.524975373,267042.18604251416,-516929.761469851,967172.5645541889,615026.2997665959,208433.21234927792,191552.015252091,-82115.30320253735,944522.3143420336,462083.6762683792,-635999.0219476325,-296429.65917744534,394263.3870686763,-10522.890107187442,91789.94409220945,-202119.385333878,-77867.76745874248,-337781.99066427257,451747.0337555893,-526487.2870503133,265328.38372492045,67109.97590241535,-283054.7268522,281316.15288492804,25438.955681707244,573003.0408297111,-108060.76730169449,1107875.522252963,232689.43313980429,-246812.03483037325,257295.92381105945,121683.21528531704,-57631.79878640594,-421637.2821788653,1257324.2315735975,483974.2313647014,443864.5264587272,-801689.4597052187,660848.1872491264,55760.29648500681,-198581.64187721815,-625162.0241208035,309616.03792725364,687267.6749964887,-96954.03201338323,433022.94730910053,-48971.018244744744,508884.22786530387,274478.47483876627,459477.41345067276,-181473.96593713248,118201.45095050707,-59538.286942116916,-305385.5623507225,603216.6477102651,320427.42225805763,467360.9133996917,134952.66333396593,-277137.15095382696,981254.1681363196,247785.58756803535,-744253.5538869286,-1007197.7308315989,354656.25005171355,1142245.1472713347,262103.09818922728,1105936.677430177,-821330.2090871432,568424.6467934842,-255747.03862936283,-293347.99189088447,397885.8800041736,-255923.45587636624,8889.392426339444,-217285.19709569355,383961.20855135843,-383965.03565378534,-463180.22644658666,-399024.02980395313,97707.65612239018,272212.11114196014,-47297.692206413485,-126869.4141095602,-158343.43151296722,-700303.7905528252,-465618.76328060636,282219.8287199829,1153656.4432790782,51194.27952811122,421375.1452707192,107931.16436889302,114913.45349971391,-457675.54715149244,650347.8192327823,30480.920068617444,-94718.74248778261,497108.2892431519,803693.8431635401,-601881.9020690848,459379.4290989442,554071.7187035428,-79206.29169723857,6808.892975401133,6099.408081852365,244576.0936254277,-874226.2225805637,283516.4288714556,205176.85208762297,577513.7944521895,787854.8174307966,592000.7884651544,81059.32270123344,304230.0488720699,401.08825771091506,169485.47823294625,88422.80986119527,245137.63723273156,660707.2737509678,506574.3981598513,772686.539046749,1065225.4390560752,825336.263965379,592521.740875558,39831.31712527992,-546619.7025677492,103826.8153870618,-5793.680706846993,-426835.84314191295,-224393.89652600186,-496834.3400312448,-369571.3900548271,466030.6289456589,-762246.8824365772,937278.5877372301,-89459.22477101488,595928.0100160902,691729.5881234542,648954.7642953298,-127195.98005512403,-57502.87380234245,187805.93654629262,240584.88680536486,-102653.85263420176,495915.12604459794,-39530.62735449243,68139.88204489369,92830.88085541362,154817.03511164756,-60892.18364768615,1231858.4266631752,119934.65819453262,-632775.0985095031,-421047.52039446216,326896.97620877926,1119980.1348470394,350475.8880524002,-290622.97847303236,-139173.84438808635,-105524.7959731957,14.74552352540195,519212.32639910653,79380.40318092238,-105474.20442680363,-134962.54489190876,362267.08887265343,-365456.5335779842,522747.8964727442,621043.6370963464,149983.08748298557,433162.6669628825,516989.07067582244,-374837.9804816353,267966.4313074867,888826.6007287013,-255357.15245471662,259812.93161716405,-53519.4379254682,-310189.87379712006,299218.2110737371,-261352.84593633562,465938.80163676664,57567.34477431979,1689151.4880231787,260224.24881561985,344977.9743521488,309790.16746648215,237182.59788669273,-867177.3946440783,-319612.5786136212,-78425.29567398038,360938.51773152733,479145.4666914074,327563.0091977315,-577971.9954887764,-353727.582440007,-439228.6984908711,159692.0581936976,-178118.26645236928,-38746.80396375619,349769.32707412355,182511.76759950025,-476537.47813634155,891169.8272412103,320407.1166096111,-335075.5845507984,275804.5366308824,-235975.8124330882,425998.5970930895,524384.0781651409,-108633.22444442846,-579860.7338732546,-392077.279061866,-515989.7846521619,44059.69166979985,-126975.43908554967,-7894.384552994277,968441.967378791,-62288.57110084873,702319.1488299728,474067.39470120287,428053.3088451247,445815.4274296975,118488.72673508199,292626.32242078846,317735.92888945853,-578445.5479661543,381305.579406864,-214385.91338987136,262304.40095078433,135237.719744049,130296.78997828951,717221.0371566801,-5131.317017823458,-658526.5795301478,-417824.96283510583,296593.737831288,864663.8230412365,-314109.1075311508,-268867.2002211623,-257740.414935756,545366.1302265339,-73850.18305933708,720007.6526104584,-14667.746606357396,123709.06507368013,-458526.272236838,626404.6775382613,-292577.532066762,42117.263172009494,489885.2412453382,94645.78743568389,799034.3536439524,970351.1758209039,115224.81500577275,131860.93265822483,481761.85527284164,384172.68240417354,235047.57656936347,906173.8949370338,59898.12947564293,604419.8729633363,247381.97791987704,-24452.00472882809,535156.225260538,-90845.40343035269,1095349.9543310916,-124017.56805860111,-915730.1120348452,-103857.39123752108,452440.23855092656,-528787.8467313428,729080.7293496397,118066.37343082437,-181046.74404224195,454852.1057609571,116550.2114124042,-391321.37753685657,917835.6367253601,-459342.63740776526,-352637.0363473445,766394.8629183299,171993.2299627182,1143006.031348145,-9898.659267062321,206950.0628099679,405005.0614463845,-302188.570768401,-350423.29294235585,100661.69981639134,-734577.0193139804,557129.7421744782,507450.079898946,325611.1077869879,-227396.44023618195,467191.16870151646,866806.805824995,512998.1722726724,122592.95223304862,590356.3608077457,678279.0023623067,631780.0236998582,220432.2918759049,100896.76299757557,971239.7765138899,-47443.577515960205,473306.44871409703,425865.7469964358,-328503.50004557194,377392.868810921,-590084.9614878618,-324683.05468874006,914878.5433815089,20732.607228282373,38326.62985319132,-89005.85294326721,-49572.74885617662,-74828.32027472882,885108.5303702438,-536061.1926755803,69421.56690096622,89857.06050935853,-627125.7588472767,-137617.71644205414,-44393.99964384455,-271007.1709722229,571668.870150758,430405.0005975948,-174381.65479216585,182182.5036031236,-52112.704279953614,586414.6949733188,526068.3237121222,119433.29766678857,737894.4953876943,961215.5695929686,209679.22182003828,-425391.6175971725,-6613.965996508487,401223.7886797385,597808.4008536339,611094.7578785876,1461461.4336866087,396021.94033456454,575911.8117377781,-33723.03282961622,141441.85536812712,-175649.26398246503,-93070.84487218643,163782.95680646272,-616769.386434955,491531.53670424037,1287213.7114017953,201443.75814208202,182032.30139132915,18999.809043838643,-436317.21239248384,568649.2934023784,524265.42675407464,39771.87559461547,739128.44449066,-20853.30524614593,-135090.3557216958,-544107.3299108907,-356845.15191741893,624761.9160739607,27982.352266333066,37478.29583484726,631046.1553053102,-26154.77097218577,654936.335484176,-146146.2908100104,645285.3394929375,983008.4928246546,72522.21474847244,275891.20688592456,484766.4326779484,418213.0974671976,162491.00124705257,-425058.4540752061,211453.740211959,31577.122287774924,600108.0955178197,109571.40110699879,-257420.7634482314,688557.8652556674,572693.000377058,99230.36201129993,953693.7127027437,-122771.37522858754,477298.51260535466,116269.20681635197,1651906.7277547086,4255.893704489339,-204833.5139617785,804141.4081472084,-52722.48825754132,617722.5544525972,697054.3593641347,535901.4638334867,743249.0023081936,-725931.6581664733,123362.01858447026,856457.9821193484,219184.4458387331,645436.4476309456,412993.4623519359,-798670.6579961628,279560.5733990832,-175216.83019649982,565957.9355127672,130704.53400438838,921457.263917245,654171.0002743443,150122.7602196238,-247104.73863778496,332159.71816717787,815348.8210313527,156062.9069007798,37509.51380575448,370606.14915140346,110779.99453321937,-116723.00094993226,225827.1494062012,412802.08775377227,358779.5746703809,-153639.8182379203,309862.348760989,339230.3450204581,422361.6091987365,497836.36480848957,296721.38716316083,-748266.0094992067,-265044.44973505195,767785.7438750472,66616.13989264006,443036.488727625,109513.47695063706,25680.013831729535,525117.3386441832,-488600.48780448316,-3029.255783472676,607586.870116476,-725740.861834723,175749.73449767986,257641.67116031563,-182434.19693366066,449400.7762903399,854131.6873528585,466266.81441169744,-72119.56988833658,399936.6792265144,-81050.07018004311,-196177.3484315779,-358798.20730108116,-89731.23686935985,715890.5197223206,746155.3220613156,6919.868267986458,338870.228823395,7722.815071112476,424588.6820843932,290185.64597391896,-134720.62100722408,-152995.8135603359,-15032.698028338607,21821.732170677744,-328655.41233369336,277460.93753662193,82899.31072003348,447366.66997401696,200353.80837978562,711104.7588570835,448243.7556014117,734887.2669524639,349071.7634393149,151237.58262139792,254338.80417293962,972070.2436098102,507777.5687354957,-447455.6044553388,-49662.8638128601,229162.07638652157,-795098.778767386,228342.50781850377,-234925.29875964578,969146.8670231907,527376.4818734513,315653.82747556455,126243.76318306662,917132.6393139516,-479215.1834472921,321030.6307100002,267669.7379774852,105605.19213665416,69185.48548278026,-683590.1963403281,413843.00451700203,1193306.149640346,-107742.46108813165,260947.496574637,-155845.81345915142,-182143.45839262241,158877.97151144594,589471.932042243,446619.45003701467,-371643.10431788396,284470.1628102814,348039.73523147684,475138.23422574345,-380429.58448483096,943989.0099638584,-762150.1451834235,-90040.00129825296,776778.1883839131,-132382.0364978048,260680.27646115888,572413.3411656716,-163543.8613188523,-327782.473803998,-528996.2841951731,278186.55541328527,573133.6752320658,797048.6920933523,670962.2444056286,406673.1093391995,225497.08328126557,457162.3482555053,111479.73581821378,652471.8760615652,38726.889818414114,295246.2004674487,1084069.8658442185,-111630.41246518726,42081.91717182193,617655.4340148545,-351823.3497783574,-16911.106563434005,199789.74612905504,782363.1564144855,822660.3541694912,-271116.1078522764,-157026.4457093915,115729.50013368158,-59598.62955618324,564898.7164263963,112274.19355268823,-657823.450930954,575507.591077961,1008600.0004713424,-774120.5264103385,-39501.33121736627,425109.595775987,-296775.68410135154,230125.8871475435,753301.6651180196,147481.78282557568,-477633.6891521448,777080.1898462276,-216436.10373711772,193895.3645873363,444538.3839487112,70750.00564034842,773405.5441583134,745655.7593242843,-129859.25018117344,346936.10478353663,-188795.04844209086,508310.92312879954,-16210.981773898937,-69437.62707115104,985363.4998858324,-91697.96805285523,108080.0351492432,399796.9426538036,184327.18125603395,140660.9110223921,-312242.90623674495,70711.33648573328,239554.29409313202,497850.17905040644,-1129003.3684428297,217199.09174639452,-353893.8246779521,-169748.79509160202,164087.83756510308,362463.4842659612,68308.28584631626,-454598.72715088446,576913.6795062642,108217.94041489856,-510597.38940948714,777946.7623189283,1165392.1610626327,-342525.6482518995,705698.0947464826,936782.2086697277,844415.1806228277,669173.2005401831,-297518.9082362647,434508.6303297635,-201159.51898148935,799524.7065578802,686510.8554756264,172426.87398813618,-42971.55293145869,-235284.80183714535,267721.49973972095,-434434.592968476,117654.30831937026,784853.0129196276,475022.84434052184,220840.77642505057,-293440.9966531941,-316579.8745911252,-52892.45469474327,333635.6541806841,160125.17192007508,758436.2974119759,-67325.68825423485,284678.53666192945,-505014.51544555556,449651.47550188005,-765571.906728928,38376.74682303332,-203437.82602582593,404576.76143079577,34093.01668679016,-30959.235141124576,-124237.36337399809,1375475.3035914842,1187622.9055624562,896215.1076200409,-1015373.8711332446,935944.0006038295,-115387.2482835562,199039.61948242737,-117535.88201960502,551938.8563774144,64624.15825507464,-180459.63815539936,566756.1874330053,-172307.61976709543,196668.07825739682,530883.7421487044,-130339.30412570247,-297683.13012020756,430772.58883145684,213782.38095520483,93313.01932602841,457627.217835994,-184431.5472660889,216032.91192090837,377721.4814967057,205307.13719715597,712684.8184299394,1322547.2644511024,1004948.4983805949,344982.0896776221,95235.19158511655,-245593.0476509421,-44433.05731051229,-290373.5209753127,598454.4208305264,-765182.7027527425,-409360.69267142704,511817.5556792249,-10192.11760527012,320039.91536280885,665415.7474625856,294796.47143018385,794749.6862938702,240544.11138725188,418841.20987223135,842412.1737456247,905069.012275313,270299.97930502193,380449.64198129997,-349535.65378709417,609957.4528713753,119542.13280509133,-374619.53118485864,-234263.66172249848,-170717.63493018365,510844.9138834127,660873.2561675706,47351.01977695152,361712.4029068877,655.5342575614341,-240733.37983771274,-344065.0308033973,-497608.45051238546,339961.2534178444,-317997.5751318862,866493.8597767181,-499726.52921149647,-149264.17427772982,-148456.94838547846,537896.7495584036,1099461.430940698,901672.9011354763,262542.37675377587,139861.14536666917,79961.87451795721,711139.3285125392,-297821.7965695248,-257992.58819134487,-19740.394902033266,370788.1888304702,-360121.5765861897,503654.1832768875,162471.6644102335,808027.1503813053,-90014.87644905364,478992.6168559715,-82866.43909137044,93613.1331978133,492960.82849652227,-396540.66691122344,-341888.5246956367,713866.0435029911,-35013.05352038238,232983.18963955715,483387.8229271155,331699.01820028573,630443.6481994791,-199378.65422657924,323523.6733424957,244118.6181978886,208758.0606697239,239772.27057543513,-294913.57046767464,526188.4883710528,-385263.462534423,-170235.62426225562,-155832.89258348476,306890.0799510586,434040.8761469093,-126494.61330445623,448634.5772204697,275612.7723413878,-360494.7391184103,-522286.55463160854,160962.32201643987,1310361.7983171963,172508.68958565453,-21127.86633241456,-80029.38979642792,310750.47691324074,654871.168928484,545721.3167645317,677553.7789416858,361204.209382514,-46922.96279938705,83604.0973631274,-557343.7300434713,285433.3913032273,655497.459659101,-2756.778541917447,238743.0389285935,372277.6595160393,651225.8330907742,673137.2019226002,-158084.58998704702,564449.4870656049,-14275.383383935783,469382.30063072406,853962.1499813446,809092.1957384874,90021.16420508549,72629.75921547599,562000.6595508493,-4005.766844446305,-57911.238707658835,-78303.01586637367,-944237.9083720353,769601.2161814487,-270673.1846509259,63748.41710884916,750723.708217687,544873.1782129533,421632.3069371297,103934.71359010064,-102243.6717745387,-157594.90810838947,-596310.738502155,194104.12804056518,640129.5546628311,504697.6108288048,303436.04088075133,-182166.08603168512,33258.738007928245,762523.810171992,-77012.17923581414,228976.65235178592,403399.05296465056,579531.9283953579,285023.6332200207,224061.26469912706,207341.87160631968,426575.8024680214,185269.60754019767,287018.46181709366,-272038.57360812044,-123786.78977969382,615413.5803811159,-244966.80872275727,1019428.9205466369,72885.39838651568,329421.7042222745,125892.51670038654,243635.5735621918,-561480.7899937739,190349.31479239976,76971.36533416156,26071.997034577653,26975.75628678873,275335.2375915628,-266970.79034722364,1299795.685826249,15719.883858301677,365738.54271059437,-314639.1543348371,283585.109168652,20404.962480484974,243533.26594199426,823334.7279666904,-572135.130870543,186594.26151277218,80074.97006952344,385943.9486800884,-84392.17266625166,-17851.706161015667,326756.8681013556,-422712.1771572386,479281.74548420403,-691924.0164449732,338350.383301476,216701.09129553195,383707.7215446946,145213.3523238958,271545.66805168125,-487291.10209813993,-430766.3430252969,398297.2114890637,427594.38516956335,-756126.1521707973,-302514.25618362706,40185.76029817341,175879.02672541142,-386552.52543704305,794099.0106717353,-200386.23140932876,-51787.56874330016,-531725.3960867492,-6131.975590077229,389583.5333011267,-160154.8507916783,398049.42687137425,-355111.46360133914,792660.304155577,277494.4533187542,-272658.10632714024,200235.60291313846,59780.93818975473,155977.12982486794,675671.7467482714,-16755.347282141447,138722.82256551972,1165612.9103048032,1029635.7193604168,-477678.9317104062,44641.40387008479,1086832.724446185,43872.71303567663,384974.94512051484,140527.92254806682,-977208.6518057035,-549864.6149936728,610792.844095746,-80798.92482430767,503100.2143259188,-263214.6480076951,-836179.5321437339,774539.2766981216,-451550.6740231891,-265299.8714917181,598047.6258550463,94203.70638663089,139743.65846210392,283233.0469577876,88850.87589153508,441294.6006188621,592486.4213216198,402983.84857135173,-12794.329679137794,363233.5703604007,-30810.32580096135,499427.86399439955,869744.1265359744,634013.1285228049,-410171.28512561554,-98446.10576332081,-39293.51946737431,-215872.79507797118,-310302.89977961266,602069.92639287,-246624.5811556033,10566.111640527844,140098.86232062196,50744.74701270461,-260566.113112967,798028.4014339773,-320936.09815635066,87156.20693753194,320571.48189270124,-164100.00239278516,771188.3233034918,-93040.47374841524,-894125.6791978213,-6156.478503292892,-430972.22360444255,307995.30996747874,-480530.3654189096,-473546.79113546433,310805.8446396934,-369738.966485037,128272.07236944232,390328.9975580303,137999.98596484168,183274.4523337069,1193285.63741699,71238.99061883986,-22918.5548144849,-473472.168938824,-170512.46136440895,163941.8220808832,-225562.00118148373,429271.51792609645,-203677.04528328823,71115.81927874219,-42438.66239624424,-96575.36399876326,385760.0354975816,-299542.88162875734,739188.8550179568,-907330.1638623197,573091.1639094241,1276556.9591363627,-309272.85738679906,-113237.84485655231,-170430.359192878,-155413.59504322102,86113.98402786255,410111.35511230724,24225.720722242724,831923.2597139762,110131.75651568547,-264224.5897624551,69812.15038797818,113589.66972058965,93684.86625029845,317094.6458005947,753276.9836817654,504420.268418822,550502.3942129561,516480.18096902594,296803.95300801285,262282.77431229316,-238540.03177618235,41043.91706157243,77249.15323887579,86423.91867301194,-85664.30047889566,505376.43087398866,258868.1885119574,-67233.27565278858,699757.8106304868,833212.6729954695,-397269.14516063686,-288140.178269499,437906.081415968,19462.080340424553,73936.38286038721,-202034.82686200645,69250.4348647329,863071.9924006439,466200.1796727991,552027.570805341,941923.796570533,-796517.5145958476,867339.559352011,70461.52106672991,61342.5503783077,36816.02195724938,-189055.37545302417,61702.077027089894,72831.67174850404,309592.5951161445,417938.3353502094,-41643.361217299476,1046009.9414360635,279807.87072267523,623916.1021848358,224994.41533689015,1240172.8858200712,288562.4689321467,-1008133.87603936,548814.1378630786,-351046.6931938147,3718.4342969856225,-414960.22547863703,-474283.5291796867,620161.3880345812,-119384.84418028453,-72101.8833760214,225721.78681931878,498388.2292249608,288836.5838627615,331512.1960450141,25862.75969021069,-300201.28504663985,1140205.1664063553,138618.74726642156,803291.5680688191,160067.92552597495,-255435.79169861786,308632.19594323775,1016900.2375986674,615832.5331755318,370883.8820994799,463751.1989204646,-170268.61290919874,449939.3074101028,78687.46850168053,-172187.18621357623,66569.13578465115,358904.657527789,-145860.8205739637,371974.53020132845,914977.1645443775,103815.10951276077,55953.40363971889,415223.0745082204,380345.25013415096,759287.498570466,1169103.9321728004,359396.26987793995,596412.5443615131,629693.280402469,400105.87586368853,223055.25980522577,46604.21491615288,33559.05250737304,461192.199024444,126485.98732470302,-488644.5667986255,379232.77916822117,-250746.0933937896,872419.9155396768,616624.5029664268,657599.3223879361,498359.8618977531,-28139.521126598585,-101571.03234602232,574386.8231201051,217308.43709235592,776383.5546621457,509056.5854216032,797084.0633603693,-275292.7730811401,-240523.66858687578,-536159.90739568,211456.4530158867,225523.5101208333,277664.5284886048,250013.42826873856,738995.7961607352,459729.4402485639,-114405.00687622931,-373242.44870312,-151158.53243466886,282978.8527281899,477169.05940918997,313068.9326231759,-154346.09725245484,231599.62020478863,965213.3241557172,-425658.2999580852,759514.7261651042,126811.24723415496,350942.1779311588,570942.8019584566,-131956.44937264314,681860.9342960785,-51119.22607127018,281090.7515939656,381101.47157548275,830245.3607040481,-230668.69515867578,-141016.9120414965,537954.2537229257,354779.1490332871,800171.001365867,-641487.2786034322,-161171.15576756652,318378.9624781045,950567.6651964765,-319142.6873569493,922149.2856224799,-764875.1286400249,-190265.80928227073,-605784.1606211332,408413.95565322274,477539.2893877276,-4107.5567816784605,609988.8407488214,-867877.617406413,588263.6304432508,335254.14819210256,-98217.0941641652,656509.2789004594,320807.9364475119,-11968.31136953691,544963.5052754548,187574.7815738013,1136923.6989394831,-80659.11855313834,131884.63258797536,456992.1789211945,453468.46666233987,-16769.656989406794,385508.88444103906,569365.9952742709,1113342.4630587264,-558428.6123839314,279709.80188976275,255768.9418429942,330278.5133040333,-164470.43282467173,944559.3082171842,96506.01321386965,239511.71781041566,-230829.4450789336,606501.554622103,-898706.6242483384,199092.40307895513,908098.5639689704,-166272.9963047197,412326.6963352435,-300416.44954846986,13869.38040557364,-360433.2440364603,332941.9370872886,624681.164260901,877586.9692322309,915777.4266750803,20886.45344759617,-631968.002336717,514034.0237835734,162068.6835027109,190956.6497391872,699351.9560046578,-486132.4628815893,828985.191109085,343638.8779260721,452628.49719578447,1091758.2895046226,495057.0639535999,161765.1482577431,-358843.7459710003,345764.01492877677,565364.752499965,750796.0039597512,-323133.4103814657,892830.4325846257,171234.51454260573,-395896.36449844344,198353.455431798,662208.1586186951,19293.731895287056,323971.0007848982,659983.3466478679,-330653.07114521414,414690.15693618776,662120.0010374272,21548.16232209606,435144.44660441624,532872.4155647089,-282098.74962118873,648207.6990123931,-254805.72272045258,-357742.79173658043,385911.99247683864,73786.1159609775,295632.94784781197,-265603.24657332525,329835.00600425573,58866.60078033898,195639.00912201544,112759.51448793896,543199.9450365785,471258.1145315729,260630.4788266872,99408.98130272143,462600.1748944782,55585.310752206016,876852.1957125962,556223.9773047082,-229771.8621350536,-188787.76496452978,1398275.1104005352,1602.94076696923,222135.69383071223,379330.9138230374,126816.07673438964,-123457.81646113703,-390144.7314559384,302129.4916775711,1122495.4102987247,-386954.0156441722,104110.82205995172,95519.22458562301,526886.4974192688,505807.8187684347,-221749.08377430215,-429672.1455402954,-929671.8157354277,189123.4885255699,-24858.72763993824,553094.7064842489,779300.6150103384,41664.878358324524,766578.7127938503,-204810.94310276257,-373347.59376307484,-1013291.4766982214,101699.108041937,-229596.0384690808,-283257.56959026493,1082884.4893761715,7.952813897747546,-553881.8784876359,1316124.7909838627,583150.2468412998,102804.84842340834,323086.3747332576,1292332.937968167,-290546.53359675873,1135347.4000278416,478028.68422712805,867051.2238831497,50180.970054250676,-487318.33492671745,408870.4467327595,242312.07554934314,296393.8190546185,-243130.42836047383,59743.862127246335,680867.0706023327,-277216.63112888904,359465.96700693714,-626249.7925327928,805086.9799023871,238896.86564136855,-371911.63064435916,-488088.3125992026,648317.2729168171,351154.7461999105,670217.6150132087,313475.4400527659,444260.9726328007,245905.64559591655,445263.9302148456,58551.73299723631,742686.7374327178,253710.32436356414,-125491.2581201545,-261192.84279421205,-312784.4984926195,49434.68431732291,-278903.36215986544,1011254.1225208999,9118.735705728177,-316169.8352501299,-404148.8422598075,-56149.25961283175,-217935.47162097506,27234.897247442976,-471454.8301429418,-250648.0258941925,152814.47025809437,-519065.6076225429,-22961.548217849806,301335.23672323953,-78227.80710424483,129342.14447260695,-52952.44407379953,201108.32880674629,-426756.7107613224,-389357.21169204684,304132.51768792747,-186221.4304601173,-281643.38143807184,504180.5896374667,-334656.9760590603,-509423.70221160585,455660.50124196894,668822.5456466693,754488.9159523663,631072.4923956613,58263.83895409573,-496989.497665382,647890.9378443141,382730.08734275214,-411090.7984036999,-22120.964884362184,-431216.66536411317,30911.599893260747,203388.18182151113,535368.645527489,-552764.0323291088,-543645.6156877303,7323.765194260981,641650.3920413794,42514.28577870922,131465.71726680454,-428270.2123213089,336750.01820613304,-28467.484470033552,994567.146676945,427889.58016923117,211794.23505787225,-118920.59215700347,-851104.144218937,94410.58742573671,305216.8616410806,481093.89636493195,-18167.550358036533,850823.7984398785,58159.85169919953,213844.66991916113,643900.9413090586,-308598.0366704762,116839.48692319822,732954.9593737596,-356312.3222666858,280151.4796184995,528042.5709278984,60652.301382674836,463432.97362308064,-405729.4626620449,351535.4634894533,-64132.847826656885,333203.86027984787,846307.4932882651,195533.44780438067,212108.1188405659,-770542.0037994068,127339.83128408156,392520.1086025047,366080.3667453495,867442.8173168995,390324.0111294994,1011674.9113112022,121628.97575306753,-615711.3426563367,510586.9452681779,638677.8414196344,-332451.11895280797,784681.3795472966,50120.462156246416,123945.02722115861,383698.558248356,-313081.5106342188,1080450.8148543937,-127700.60357090831,-201109.424111492,276367.58057825733,96447.9202795499,248627.3277808451,524418.6350581264,-42153.41223890381,536496.8084777477,235340.15617956733,428864.26309281494,-110388.46233864967,-259384.50613212073,-458119.7403498576,-51275.57444215752,401995.3514815704,947731.5515443818,733219.2633886365,1049326.6217317297,148970.69525615824,282513.51070860587,558005.4983112193,-676984.4941800218,-845859.2835737609,-68268.04103094526,525949.1492405282,-15085.605296981987,386594.3471781183,1289334.9493654594,215058.16796592204,-282753.84723146446,-267182.3334768871,-30745.88362326566,-151232.5316923717,478338.86260491656,-151820.9976668572,-183997.40926627116,-787918.6493048226,1211581.1461710883,553960.282090195,-217837.09681271715,31521.728719333652,222956.31154585816,105692.85177204432,-391658.64373890636,210616.5993071366,606880.0792284333,-110830.65597857162,-314278.4002741757,697549.0501177125,530675.8192439475,1106809.8617203594,366101.15072731907,526339.9254506542,298585.85537309526,-7107.895027937368,153264.82478631753,-223709.3745311806,704464.2594571421,247669.4701652294,938974.0090106868,337017.1606366718,-102180.18829549616,377745.4154450735,-175128.3268149849,-166946.16493043164,767610.39651328,-190758.94730233029,63282.39403075073,954171.3138522753,116172.28017814411,397027.7995865606,-567209.4201061623,-535297.7410972645,-656842.5426355368,143321.61998329172,331825.5444700415,-155549.50134083396,168288.33261597902,11126.643501979765,-249557.847277835,613213.1644542865,566503.106849716,980603.3619399653,-152931.43778735027,873726.6528707715,-338549.3091616295,604252.903770926,668126.3310297402,206164.13506691344,575894.1483505061,56011.11260673497,1051853.708743382,584766.4100294998,757791.9249553639,822405.7599912039,931884.003761176,980628.3542373423,132137.46288546082,321494.0200626976,106050.96054965397,-271506.88909236947,407724.74063035054,880876.217157485,-36747.97404716816,108117.06535119144,-594914.4174386528,-42898.70303965034,184481.544565951,248001.0823228364,5652.9043031199835,-363659.6122972523,266058.4056480103,58939.789534335956,316240.80401112884,-619583.831796891,239765.89985312335,389172.3273200891,259341.57447298337,-26421.646307812072,67512.17109883064,467814.7699185461,181655.57105211262,58796.6048714784,928504.9457171289,337683.447707342,529178.9359925413,-323741.2649456193,-519424.76273588557,450187.66844621534,498830.158120065,-209098.13786514243,652745.2735228906,35072.759222304914,796425.6601205412,284118.0294567859,-113256.99032905139,106539.53287843987,334657.18179561803,671915.1960690385,434376.9842129119,-88326.95618468011,820041.6912969602,-143627.11185568292,744991.3788671726,577448.4961000881,-315298.94617775176,-435574.42239949806,391782.612778421,242654.4834139133,290177.89674725337,858256.675613129,-407574.2241180111,582891.0381051153,-39864.916338526644,-375573.0734900348,494876.38031258155,74092.92088272236,738654.2048881077,525642.599251952,54469.71608984517,654778.3178413641,963064.760402801,171905.71126636444,-52840.787549315486,-199703.63086593803,589544.397703629,552273.017278282,130331.68676253362,-697923.7932632186,98974.31171683082,539591.0226475457,243128.94162153406,-481223.452741242,-9367.89428562671,-122909.06224643346,-482412.1588468817,656093.017323406,471090.14860037947,502486.49914485356,94399.18906320399,677002.5343944351,-165596.58696832508,152867.2480408596,154573.88899161387,-271278.29186936375,378474.4966415949,366590.1054502586,118790.31917490484,-552282.7763416534,512841.00387737807,247782.20472907275,54746.95496111363,-192380.1568149021,-607307.4870164928,-20418.340579256415,392506.3448526887,177792.97456462448,548253.8537290031,350681.63984975126,-226148.32640932268,-447802.3701961753,142225.5525946645,-203451.2362420871,19771.364235497545,-528198.7258634437,702266.5069383704,408326.87987437285,837323.6575434464,773568.4641051665,-135413.88036389975,-378521.9568919623,-646285.1130878106,153437.77646301594,-59992.61544819828,-30495.78468907345,74759.81965788314,291494.95547780534,422340.5257495865,131000.1168711707,-984203.7992972012,-148685.73949862737,282965.92664924264,395366.6193221132,693179.9173200396,-74589.27371448232,614620.9240834718,-825309.5810558419,-397531.1290507098,541907.4768675044,432053.01364309853,341505.7100571175,54293.71415584162,8292.986265650019,-591342.0186431615,814747.4866648731,392364.6192514966,861225.6683423053,837885.8985460196,686656.5639424564,-159918.15109404922,-20258.0128673343,235756.20616134815,1036198.5486246133,-703585.0135927736,304393.29549952736,484351.23007851467,566363.8495438211,804422.546858733,251492.48095511878,82285.9184398572,787570.3474452137,-158956.88265329786,-223203.58941657422,-143324.14466397697,228022.4558181432,-373211.9619128094,-815314.1114739305,120017.34752328554,352250.1390309492,18733.418921969365,163521.90095173102,182545.68772582198,-230465.10304744449,37649.8365986296,142613.46520916093,-438382.00947551243,-917434.9689567201,363298.27532880334,61606.99248500308,-675156.6357130618,203680.56668149028,281394.6314649703,-170466.46183557156,557862.565659503,-242196.92385044415,-256028.46298034722,-233586.8707974716,470792.8876969032,-262197.6775970822,-460415.7055340018,32169.59946345864,619402.2891538846,624344.718302391,131491.386745743,-670259.0781288191,-9382.07378173992,838695.819925135,1169876.8885668735,1054588.6627487568,387918.1417214335,254638.16768500116,774688.802164482,-833739.6368554016,-180243.42866762122,-1936.9460326191038,-277529.17729993165,904236.5515357074,-388509.9480935931,408269.09723316506,940873.6856136154,593951.0852815285,-162263.76836057892,181804.35737655265,-400838.20390894497,23003.482782335486,-41629.20220802212,166244.23837858997,38776.21358207241,489106.9025443103,-353966.8176985909,35187.53762075445,533571.5279830401,-316535.9707065369,82840.57227483578,-103998.65077318903,876024.3967321026,-287229.3142743916,-494112.022619294,549163.1668232637,96635.50211263727,122248.97787801549,51119.36855458096,405654.98349530017,-249926.4106358639,299102.1622065068,144654.61907852674,-116385.54585406044,187746.320304499,-40571.40811795555,305855.9453636282,-381027.7843140261,364647.6497152066,-485365.8872019863,546783.462182364,561983.1487459596,237008.143872309,340559.6598151834,139909.91816471005,659944.8381699291,208150.33150992868,406811.35788537515,821748.8017058056,-37723.561287596356,9932.206251109019,908387.969229728,193423.06242633378,243955.50764128193,413270.6971301567,-142860.08588237688,983852.6199673777,-34011.1824189038,-49999.4753396553,-453136.86106383475,186436.20069894567,-391025.011072014,118253.87687194208,-470591.5807668902,446537.4598955563,-147569.2144825398,303553.1799585917,-14943.559942948632,-165993.91230309475,-204012.31727860915,548541.5935198334,78588.14450412896,299566.46044834005,219402.66410203697,-639824.7171827792,184249.42953033932,300780.08353819605,23837.49278380489,687254.7895644442,-278424.88772650156,515816.6266365503,1068545.4428660148,58775.69139343267,-303464.4805463953,-167412.58528047847,498457.0956077152,368845.6253743493,324924.20180181833,-828074.4240637375,-335679.47880806867,571058.889052026,-71297.6113067544,192707.7223595623,-256052.8089759848,-100823.10991907865,-282083.03894632636,50148.164214429446,147142.44812493306,1669609.2087561612,-465718.0166734415,-920663.639676468,65100.181497586425,-226667.1291486742,1342538.2657950278,530614.0857838774,500100.5814854894,-204066.301560272,231339.27316684462,-474873.6307142149,-129044.12028709939,328146.76340555074,166726.78302577836,134603.4803402503,216786.95988830412,444097.7588927308,528104.1840227777,578894.1504501756,1155563.2541369516,406534.62912478996,637965.3957411149,-253344.03473614436,323838.7513456177,166127.2639155551,230969.5580949923,1019848.9532936849,345375.9089848022,76586.41741882451,808081.0790951024,528079.7514283927,-655135.3279690845,-177161.3347476297,677859.6811211719,658219.0610668613,852633.67962456,415730.2722952161,-91810.14636226185,1218136.295985302,-58725.48873442132,-49943.62531771138,-2258.879771771375,-663032.846152409,-709144.0232553892,496922.5824306803,-265945.633330483,-309493.1073644385,-237645.2417171267,223146.41980984993,806860.2895936943,148074.20864945324,474391.05996291246,417560.5204824116,190659.063374429,-137981.8586966605,555529.5768428692,455433.5437375908,33729.66626671469,-646207.8862907193,102228.6156863058,341121.4538898412,410933.9428208503,650069.3863533847,724680.4904175568,189673.46196963638,957355.8844838506,75822.62039605575,-379741.51989589166,-229018.05710222526,265800.7419011891,-436680.3815005794,1198066.6862194985,208773.14817293268,684637.8075851919,251260.14909421513,357027.46619680384,434874.0137294163,276828.2481157831,257788.5531964437,-71027.60004200554,516463.29190149345,-418694.33531510644,-157364.85202752193,593919.1806014567,489068.6772108865,464905.03580404585,886400.0426442008,-694978.6633926658,974514.4263109206,458439.5128736929,588398.8875369779,145612.14269021153,-37968.63065662328,-192151.14120771177,-923710.3640362313,-29233.70000516856,-45608.87134273909,248369.25327050546,117212.31190429907,504430.4796348568,210105.59706982737,-649686.6625748803,26387.659284662455,25666.83181332983,251624.59884137008,-15930.501468592323,306112.1902344981,458812.0075488072,201802.2537019709,279098.8817856582,-477431.17433043616,151581.97279547155,559249.7244931965,202665.2570156739,-468709.4613720053,357801.9196517649,366579.9820828163,74294.68023484666,169466.5368854683,-205569.92458943836,229690.58615774196,29492.562214670237,502373.186171636,231691.63111972203,-101798.8448655922,157358.6154554775,-383802.01849711174,243468.1436065426,-181570.47339011217,-125243.67997807963,374260.6250122213,-110983.31581554702,-161034.6506194002,53951.30481614359,642668.2809176133,-12590.490333052352,293739.56988174794,421893.2594823963,593509.9172003968,-235414.67948887497,-163328.36305190017,683031.7058491013,-267930.08867955767,314759.66914863046,147049.93233289756,271275.665007378,371091.26666224934,193540.66971826134,559731.3290503859,-284197.1673792065,853035.4030752489,-865136.512421736,34465.90164897172,84509.64790824056,-462815.1733623096,86137.04261986492,383544.2645049086,-221985.05037086876,-187762.48056165315,-409076.47609866224,8731.946158317383,-81966.51599603286,-268460.7446475392,-4773.178177716676,290756.9737404897,595307.1685341275,63299.000353185926,320734.4820625456,-79687.58432396967,-72661.66827959754,-18832.576073199976,-478262.67853556434,-589550.8584590061,-205763.1317810975,663565.2882788312,268295.42409640923,563126.4273453285,-192628.98861262575,-438613.525530444,652476.396553685,347388.5946870018,600980.4702123441,-310012.79400037276,98025.95364173548,54341.779151343275,-24268.844515389763,-382516.4716063752,1070361.8638199614,515835.1791955759,-701.36616572598,-77539.72431007493,143996.1799619412,-299995.216848074,112652.51493526297,-53149.93408398982,-144841.51529360795,-436488.0543769328,609846.7779399105,420172.99744420545,1007544.0634046579,484338.1248253812,706520.960826851,185889.7744026552,391039.34017684264,174167.82191567076,-136098.50052535348,990405.926953922,243663.62122860644,-65198.35521714855,-399057.13239592314,501796.38566746376,334826.35247890046,429933.68566188496,874122.6123288875,473557.5323233735,171771.26002613595,-504410.49135468295,-179863.29041258385,866305.8441734281,-169658.0176778389,206488.65731802396,558067.989002517,3894.2900883043185,-521711.367347986,-12356.393588401377,75851.5671332539,178723.463292012,774821.5620055059,-71688.43977257097,256096.7351627131,650632.2947978526,-567337.633098986,-106942.78987557767,1012789.2609640155,307197.40768119413,123545.96783554927,53634.172306866385,-2761.611286104191,-209587.29740028037,53390.101339994,61426.80886822846,647752.9618043057,889722.7246696311,705922.6835939321,57634.997932875995,267532.3791608177,309564.2061716947,848412.9489094885,-693699.8081316319,417149.8901484846,-642262.276852529,-86760.30692892382,119303.98908251524,523920.58943555737,127279.4585703318,78680.31690856954,-84664.87653283076,-27459.047782555688,-336285.28752783826,-266727.19421648094,-161416.44804252405,888632.0708182394,334279.40087035485,176194.58588463534,-95328.96794828074,-662441.4664633321,-19908.214169768617,320329.3810082893,-138837.92151058186,503855.49243239686,796154.2277881717,-32455.890607231762,38606.761853075586,-681567.411877281,453802.35756333405,220706.02700847387,157792.7721424671,-89590.80092881573,436650.97075472353,-230960.52758142212,778639.7357759988,766492.4890255108,-4372.294903687667,-157941.0739916088,613113.5818966026,338285.2605969757,-305656.479002676,434653.92419409147,536531.9473404903,-258082.96305773268,491573.0655549392,740334.8737417976,105925.02268545656,-86719.18195613706,265389.99966026284,310718.4538154332,-141755.9806068018,-676718.2268410549,-437346.09165446134,180444.91472956957,-86425.96783715393,241281.45671798429,208819.95677428134,264394.10568210855,801690.4747054907,801421.8438592996,837879.9313961151,743657.6946706194,33506.70076251635,723952.429031367,1069575.425140366,-131698.53007501038,-191380.45472352626,-43392.1796414773,381887.3379871524,553061.5385773308,-4784.666423977353,258037.1859258907,-140233.95792774204,205404.3801564062,-574546.7140532527,-89012.11218857067,533297.1947390842,-15903.379924530629,188517.9532918781,-182296.57147990307,277615.48680216493,19444.349541463424,-475046.3907642942,779199.7570807678,-279377.1550067491,188634.36351006385,368701.63212385355,543139.1361957695,-141589.82955895085,420269.1479410222,-359847.7643171628,-168436.49320582813,287213.58941391343,2377.015439565759,207396.9081120207,-101141.5245250347,-129916.93250640854,-94737.98703260068,284011.5688988045,71123.5080688009,83421.85602776287,-723697.2009045931,-308115.72906021867,840229.6520975088,-423041.95389576117,-226202.38745959662,499552.9441047427,-350268.7769746785,124834.51407393813,194519.79667546554,278259.11413175566,569655.7234400958,684144.5297330664,-114357.27762555424,-7058.433134004939,-220607.5624688398,174056.159809642,215345.94986536028,-253528.40493259253,306347.3049505702,-176933.32345908182,362912.8835567739,295341.7088246411,866617.8974882918,335201.4817313864,-817547.2825448515,19133.7227131892,153131.5869370494,435085.2181311045,945833.1736711357,391362.2722271462,-255829.72122828895,-185589.55875953566,-613854.185814044,671347.6987645687,148113.98745827517,176153.65069757635,299706.1911019166,297167.2801884832,185639.37188475113,294199.48632906564,-214362.43711144757,332851.5323768044,134488.1324295574,155481.900641907,389526.0365444957,-165791.81157136336,-609807.6460384275,317351.0942277475,657653.4659941583,65330.7143080309,627395.7452587355,646187.2525406457,398611.6395949307,709319.9356503403,-43240.48005408142,1110100.7366701225,-229482.0946498518,194607.75901423115,839676.1473359959,-270765.1664958196,581299.8673271686,215215.56786997523,562025.5363027127,-512984.91446980834,-308007.48945233086,199086.4327856321,494015.59871556936,13757.555005834438,516767.31969457306,-352210.61907339515,-12067.734469715506,319751.0462084175,891431.7318174108,465355.82735493593,-284497.41136519285,525230.4398528975,229437.35256458353,-481379.099578192,-62798.260458848905,391144.32229492255,517909.12278116774,114774.55011658324,-211829.76103694038,636872.1707782946,103101.77456765715,96899.96448885882,858325.7692343355,668528.7622063407,265114.8009039578,-90519.95352938352,-94996.47408445366,-525207.2560602976,386820.3629552671,219949.14347594324,25064.23010886414,44975.24621883128,941016.6887534624,60716.60714783147,-188602.7089996892,629571.726159944,-14731.408497363795,196117.31532426132,-197975.429392396,366084.0429228954,633588.0258115949,1150024.5908026085,-2068.0296385362744,68969.97487521544,-23693.331990122795,-20211.823193543125,97392.97821096517,-575422.1655245642,-90794.08355887746,-12243.345096396748,-58291.61636591796,-620819.4541348424,34680.00833875872,-583616.815251641,-415894.6776511404,-404470.0797246406,-18991.197372694965,547148.1199897756,-324103.2767611416,532998.7943893299,380796.7493476276,341344.0500731026,-47516.79038339015,-89.17490084189922,747821.2730789178,155101.97776240855,-309824.91049102554,660895.8417428299,746800.7831586045,476101.2345749731,-204710.44241809705,-112557.75768160308,-161172.10452571232,771273.3791843853,1132213.871411554,186370.9140721867,366099.2350640269,721285.5595635981,-76827.3322411282,-226195.74012368033,85293.77935918001,876317.9940089695,374631.5535537158,175032.74453638867,435001.17329796916,-240270.17913468182,-411776.01059675217,813767.3708747784,134152.6312053413,726079.6862097848,246466.7283114656,3050.2096499092877,179040.7795500001,288162.2028994737,384889.3853296996,636007.973687984,-145105.091391752,-12959.095178728458,196727.36843772465,-86950.42050113017,429307.69394617435,138848.10935500683,-5319.893907929305,452552.4024498956,581072.5564274173,465596.8579677944,-57274.927917903755,657728.5137809669,429123.64136234485,-663084.1488858489,-631074.4020082154,495256.17655222304,1045816.3297839202,18793.873486317694,453734.9090962042,-511793.2951802388,798063.0240627397,4877.197952003684,337790.8045556359,-858725.7596987109,549838.7565083336,-173758.5377389714,-171861.02475833707,521395.612932615,262547.4489680021,-220804.09727004496,851979.6881903643,-66955.9140093145,118429.40598408459,906881.1285332022,377675.35974969156,-425618.1370906825,346576.72276765015,58309.748357858974,164503.6501830998,-49610.3023456973,-98151.49963468639,1223689.2009882797,854020.0710507086,152844.36740368,50635.15161297936,712159.0415479289,-389427.06652213726,-63650.92681032792,414293.63508977834,909962.8292868035,-361666.80576769216,606717.0236597448,-138508.03599173762,-172977.3606161424,304331.9911352745,666752.3737180238,342375.1601988389,403613.5090987701,1147474.1420617695,-488402.94332469534,1128930.7968899524,575865.5599709908,-82763.4598063305,526436.8212564443,490595.3485068367,-420761.359994513,110922.30928295432,191626.69649815094,230391.91198310768,-187122.456857495,-453255.58275324805,-127454.1823537196,43286.05633850023,458925.97331991047,167337.878896208,-91719.49647071306,-75286.12842329172,-32642.25240060268,264209.4995639031,70207.54792822525,-679711.3685106225,483793.2650774927,28115.964728945866,527221.7957833186,304675.9619849529,425950.77362347255,-162641.0711701573,517117.50553537766,-26771.765576662496,207791.66768312128,381131.4626514367,124414.31335692666,545146.0007748636,11361.938200305682,-312722.7597591607,268105.28758742334,604067.8507816126,211403.88604117464,723458.8392357621,390515.1342568053,495328.39020387083,997954.7512766747,607381.1528208447,364759.3216641983,785791.8902485087,388935.2738727727,-85346.88755164482,-704897.6712990887,387930.19041291345,-93513.19649656396,490213.62558853533,128210.03300111368,108763.44805850973,81752.93433601037,291066.5137408115,3581.2021722099744,1068833.5038903444,332028.43802227126,-488557.23888558056,326133.2655090629,319452.23394931294,172971.85838453798,13784.12073194189,-478327.398186242,144216.69009167794,667193.3603083696,692667.49756695,482662.60364725906,613733.9157879921,214295.31053264998,1035687.2734360364,603913.3177143601,13155.980556044262,240643.9017570573,256881.33737548068,5962.729345281608,-333267.35190183064,142074.0346027431,183545.2315206146,751992.7665567412,290967.19795542955,-202639.06605973328,-71164.11583929881,586303.7178806318,381278.7253118665,185989.24817767087,-618906.5730984295,193938.61943687592,569389.5808797264,982870.814404224,424902.0612115301,137132.16142460518,690403.3807383687,222339.71953102015,-27915.06785432715,269269.69491842436,973499.0592787378,188630.2090522591,10607.90093048988,-405664.7402478205,1014180.9741279609,706152.8281827932,-40715.820236491505,-1127935.412296219,-157748.20411314862,36470.60619373107,308839.43623878155,-795757.3664386845,-510585.766544858,66312.8658307353,-256751.68052252103,121960.63253771886,1081353.8372980505,203571.36729977326,-188850.53067371715,1224775.7761393827,-221982.84676293377,-470440.354147173,-116249.53571619513,267106.0666006529,180940.46725776372,596694.0929546217,427190.5822779229,780404.0321261813,-334103.1982444115,-623811.4318936998,727132.0587010747,-170668.08764191344,-232425.05318447622,320755.5002379571,289428.18972737575,44506.7397589148,-369793.91670899605,-360157.12464369414,511628.2576391725,781758.0136468676,-576175.5589571097,-217885.44262369536,516989.7330645835,706116.5742352176,781830.443608969,186054.69915085426,546505.9498662003,248105.23470533686,386511.8215687503,105238.87102265703,-62239.198294997215,-279371.8173799934,934237.7156171207,131074.6049455083,540233.1976926564,-63993.54799977457,338203.7076711878,521622.23267930653,276114.9731283807,436372.9405406099,-72742.42408906063,-100923.2252191375,492556.9226361825,363493.2272887728,165020.4663497205,-132054.382900659,384193.50676999334,-387071.6585326884,-201971.25677299965,388591.6248522368,57891.793188395444,46711.22771929158,-224550.593907482,495661.35737672634,576925.1739447005,618107.1754805627,275482.039227691,396503.9408031986,380891.2238291092,348530.24724885914,753982.6631143759,84936.40220133588,578800.9432365499,87481.76411316078,291726.58717891714,242046.15372148016,261595.6583626056,494621.0224087215,580828.9030979341,71655.03646915033,467162.9568542512,-103793.6585839577,-641177.2311461619,182730.00467624934,-203274.14308622386,973273.0290929838,-108762.21133708023,-144123.71917569265,1249004.126559442,-36115.690131762996,583411.1241135816,127649.43609131686,-665080.7589852554,61428.84528726479,247305.05126404762,227127.1445881161,211512.11973357433,-449340.68444401026,1035732.9598952071,334877.095833682,-570397.6724205622,115261.34205773985,283934.7451777174,190916.72826761054,717449.1368644726,400291.0468500019,189713.58677282138,56183.74066938786,292037.9930594438,-31759.980662042275,-223561.0221159379,-119657.89229646791,-27715.092536657117,636745.433994744,881375.1245943019,-141970.98089784943,-265719.1374539784,186041.2536596423,-295399.2653314965,-68691.15441831434,422052.8497264944,43190.1691867169,-221124.3180610519,-52316.75228040805,369422.1791664129,566676.192238288,771499.5417597662,-382688.854478264,131171.52275800426,-135210.4512788197,46286.95582455024,55578.46907378873,-246206.70223543048,759683.8126233774,576736.8044042932,377565.9320687102,95572.76811103243,437194.3712707474,315396.5096223061,228641.32929125335,126784.97759791091,-299542.96353878547,-779172.4113500626,332098.9892066377,495953.1891237376,230975.0971025899,223676.4701463855,-213127.05172820063,-166822.18026994402,332994.9766309657,295880.7167538502,659284.9153063935,766402.5620811619,65742.94542527478,-497636.3105755383,415484.49499564525,76062.88161216537,18110.9402748188,-442094.559889507,267020.86592297256,48579.513487841,131203.84440382756,-273021.909709665,-395252.6038424731,139921.97037804453,287752.12845849246,-39275.48558865907,681643.6903182971,411792.65242088586,832033.197572221,99244.93186792871,284741.6826865487,59602.85259695724,71558.50610196497,267490.48704555305,365702.4889832712,-435674.03499149904,-6082.054445840884,776306.6664995337,1123485.7083460004,660446.2163802534,57946.32763436437,-468578.3246054943,-197521.1443951577,-102343.84617610322,304609.1049899808,8567.841485075653,525944.6972821956,438943.27616058895,-157944.85216651857,1619191.2960261,62992.173216391355,393741.6534061469,698353.2890186966,245532.25838662684,482746.5088577103,-91224.71749356901,-215352.6338046454,106030.62488339329,488613.1262384909,849409.2082446511,-366456.3784606932,954629.7922166805,-191353.72082151333,485738.67508728895,57600.219759472646,190392.0350995129,128977.00316056609,753218.260726152,-380574.07578949444,-45290.77202827856,341436.59558763076,148408.9970681928,-151106.80521420483,36511.63448613463,-76164.85191786196,-163725.40422986215,-4911.984061517753,71511.79439301044,890178.9792546146,674554.5563010848,197662.50016612327,1094053.144622088,478791.46007526945,-261182.88624680927,114019.89446154563,-457275.2519009649,98195.48339174408,133983.46229137247,233884.90287575964,318786.80233271094,-376061.7261944534,9130.360936979763,755505.6827091686,-418076.62109645363,16573.025341739412,821970.2283561635,-599965.3173692678,274904.2470634831,616192.9987382009,87632.13616361422,-260357.1820006282,-275619.15019593854,1005284.0658673602,30378.559799876995,-966122.9101877213,635991.6701650196,-183715.22960908478,106283.88741882984,744745.8446759209,1162342.1452763155,283301.27720200643,-546781.210790094,1016804.9217612278,576900.0621178336,390592.65600022674,-82749.7360144197,48854.77998889098,-414493.8795514959,-55625.846370554995,-986688.9581973436,170455.7965037115,258929.60641851695,873281.1264629625,1023846.6762994663,536773.2514778874,455756.1417668066,-262911.1521202838,224983.74573878478,363556.5880245636,-413430.16403429164,363415.1729541002,1040046.961890921,111812.3530642814,879251.1339306342,-307569.68835819047,-587938.1881245039,348686.9690990504,373797.3412172599,-197853.2698674393,-969038.2640753915,-27387.01300400123,349294.6592297349,-173380.31422761548,971984.3170368052,-153931.35401748307,521612.37698518997,246795.99029536033,92623.99292506091,-164513.59187570587,178805.19742828468,-102438.35092408676,328456.07531285984,-238066.4653906189,-511789.8168226746,205823.78775688354,618259.2809840534,574518.2472775364,54197.07979471935,954326.1904339497,38146.75777467759,322840.8728899718,590534.1980519746,1036956.5687893666,4417.6329551427625,259935.50175945507,714494.9959355085,643932.4111834806,655041.6042584497,332652.16680288967,-67427.85033654794,-169792.73463667557,-85025.29977064626,-231452.26367124962,-506.2230881648138,575910.1018798328,-117596.86428886559,529322.2313080211,567293.2143057557,-605314.5415986571,-113984.22130347975,-427132.7782042399,199925.72615928808,82604.5034435736,-97573.93436241569,214250.50880624074,746343.5821755147,191877.5337244952,606822.108464722,32489.766542884987,132158.23408058612,-602398.8515505781,249934.7358040926,539335.5467597586,353485.22185971495,1146011.331544809,230698.65307637537,997816.3038887721,549668.5077254102,936459.7202852962,-197631.71614448447,1137295.4499349198,974715.4051766559,39897.01039868314,354792.6968696865,750904.4405088085,-53460.24413308455,-345604.0207845131,-372999.0828599804,-210325.22672200808,-116547.80012130644,72384.21886859834,205333.0469419323,659081.8933936292,10554.213270406239,-168906.50496807555,-121288.37364128744,1622293.5837488403,-108646.32285096776,803695.6241661645,982866.0376570625,627718.7785879723,496021.19919927465,-111363.98193147127,561898.681079255,606146.025562481,679762.5945731918,386539.8171984153,212028.0299863466,525193.070267552,-4198.847398372367,983614.1284279474,251083.4802381238,194734.9038022193,55082.98568034079,561795.8187680752,-123511.12557332544,760805.2713260553,83836.02916820673,143696.4509931123,704124.2949383571,1051548.2143149064,408841.2134760106,-667528.326364686,305816.58108094987,-629905.5041097361,310744.91305604903,618147.9261611858,157538.39456929686,623364.9651793237,275747.45136211393,123582.2246366404,-215952.81980284397,-712405.1016019587,629250.7815983472,254461.8578978409,832262.7155735665,-555312.535552097,-101836.43005553912,404032.9560638997,43766.94634133624,-391541.59113593027,366236.009298699,308733.4046539366,463410.4660460227,38161.254818732385,-293918.57203681394,-307315.1372895688,140726.42522094678,-331580.80375529826,262369.9888288644,306012.29426764185,346054.19701217767,126418.60081986524,-71626.5192230558,612551.2129312675,-681173.6013527,423335.22110492364,-62044.48872270761,559743.898375581,914228.9613161236,-167244.18955028197,-114118.86848252267,48237.38043829892,-568579.4927344262,-206035.01791369775,28342.083766239695,1038787.7106260126,331895.7120025428,132636.2874189294,472433.15508423885,-334444.1640298348,-52908.47628701292,430510.3537079282,393370.32236368954,-49787.110700822435,170231.25012113433,302589.080042209,147961.61950484337,495062.96863822825,-318537.2305696225,-1805.1405144766904,128627.87260105927,914677.1118781359,419746.0522016082,368777.89217212796,-371796.4727142821,430654.3956340654,-538597.0053865048,576113.838058853,123794.0691778739,186279.26528962655,543798.8191465619,168032.68588613672,155458.80945563503,86889.92773599783,-77402.41627801582,-535590.5964315385,-114239.72203970794,-86925.30097983778,334070.4646434258,-127195.75064741774,185714.9061435014,31175.374204561114,560846.6741917944,252973.48602976883,267890.8306185077,160588.38637270732,-216376.76599815954,383435.3367209849,605428.3893940556,-156523.27939961338,911478.6261835904,-501301.8599140374,114686.55966037931,549785.0608199611,-370835.1155467783,440939.57356002834,-48578.95048858784,-231983.3817156963,79943.08511028066,-219699.66368352855,21659.641823841725,-867796.5662534018,207554.78619863046,-284386.1575988792,172151.18357662205,60243.614768244326,857408.0117560183,174911.5709427842,427976.1821140689,809644.6379648792,-34822.27080483874,371012.54159120517,819740.1374663189,260330.56447961228,231393.35283541633,-7555.505107597448,-134580.05079633463,404730.1533770859,204065.00355359167,-336604.7447928651,486296.1104476759,212308.87461351138,-235378.3167570443,-442731.4033164494,278996.440368555,502999.5676699667,-224315.89274499286,200036.63106463663,608176.2922558044,-110705.39350267965,-347830.29355903855,562018.3590636738,-356725.7718444057,244974.4017328797,-743.7435862137936,39613.34841589304,-160230.3412871943,303698.44790076744,273850.405259056,-597034.0705486462,903250.2995663872,-72174.62431008974,453682.4510352672,-222728.863196515,467235.1111795744,71540.32082394161,-48964.396023620386,-741615.5239420151,514057.0200327807,-225837.00972756092,99133.01055726921,114735.77550337743,232859.88620490115,402785.44331528293,729661.359787277,-40140.47720868187,-901281.737619325,938652.7782495478,341329.94446460344,-101461.23406595783,527097.8393139159,467729.7908781427,197054.779678904,-28592.798374392092,720501.4620801816,250347.40714724455,-367223.1256277971,1020487.7596171075,-375386.16262591444,30467.57362164883,-479911.8700166235,407435.24678047653,652078.9408599036,228295.6605431391,458146.86900288146,216122.86213368922,-406361.0220027589,-339285.9065932124,451892.3346451833,500797.53345585475,261361.34437837405,515876.9302507262,-34447.278844890185,575578.5740830274,52786.26971964119,612747.4532320746,270535.3893373483,189923.9115125644,241700.19363295054,96148.9491059822,72348.92316667456,121695.20311694779,-16058.10175303137,646592.6415855167,518613.48204342835,-536844.5955401547,362841.9930017977,433444.4348998335,-125606.01624163566,589996.7229849859,680183.837913434,363622.1146808858,612748.8781562936,-238819.11976882396,-279215.63130858866,4428.407857737504,67824.93006834667,551044.2609758573,367716.3871007301,-409042.5329771703,-584996.1411651303,738736.4528530631,212830.6997711379,541963.549756322,462657.82701593544,443059.06954375096,-362170.2683606222,-356578.3055418576,677610.358444036,21731.676621573977,201200.3819996924,508859.54312115395,59254.706763315946,133799.18437126232,-120361.83197491709,316057.2744889776,110783.28958420828,-248341.47125880187,-244503.3019699622,-889739.4184982618,26337.94626874244,-66213.12455709884,-77426.66781133926,-99328.32287245197,170139.91496312292,648219.9411926507,43297.14554506261,-155132.94493247848,235843.9296959252,-571712.7724682097,-789676.9454628592,571342.3043381637,345601.468671944,99930.9018332418,-65189.07168221148,134504.07931643678,124508.67719997838,867054.4902499793,349436.0136523191,437919.3634808427,300392.16550137894,252261.54960324895,-102403.72454480315,68060.88908614777,-302929.92550543277,929306.633572509,-172134.7432860704,324255.7704138695,291537.6016431707,-547999.9757472393,-437494.19869276695,-75481.69057318103,205608.67557921307,693097.0063668452,-147845.94614129374,69587.4751078249,-817614.839852009,213711.8640192449,316476.2039361219,51663.03730514366,310782.87348727696,-180031.5974379941,374932.0142209702,-20111.82882115757,887609.0427463059,606740.9913202175,300750.2855638885,59155.03798758751,1427154.2244360363,353095.8574153106,-56627.94000425143,-288120.1889388147,250833.28415631317,831820.317237061,567208.570379993,-148602.85967673548,-380463.269683653,155856.9555031117,60217.98177339928,-52379.28825613903,626489.1042754143,-218287.74108975194,79537.45612718398,116260.88664361415,-449787.4429243924,-248446.07866352051,-40076.66234895121,-379784.19999627443,-267964.92174837226,690663.3549683266,-353418.18073466024,272898.09026117064,-359204.87075079605,-284478.37960533565,-397502.40296302317,-241571.39511854015,-457164.41722699115,322104.1821618583,255379.6955429325,-55144.25990774296,39928.03009578213,349293.2017980665,340723.0798181966,-507351.4879677361,-33132.73393106321,633684.8360797698,-365530.0384248998,579462.7127581467,602765.9403961473,328008.4785370184,42395.74695173977,224729.3268236057,15200.964497621171,688654.2869786932,-51011.77939006407,-121725.73686968954,114934.23708083946,-269100.0394174345,221900.28027960518,292042.00080189854,-259791.98274791427,391328.56802800065,-7565.91290207766,338385.7572950432,319783.2044016174,-411108.53682241123,-77047.02666771179,-159633.10824642563,-162891.98792555882,-703726.9672139161,-179063.09700939571,-317305.6035822062,1206930.5075692777,236665.32712915493,-124884.98041136842,-534756.553545515,206303.71902063768,-245168.63948741462,591995.7667382681,263345.32393553667,234974.18263665028,593554.9203727623,350659.54783612443,-64737.81104215514,511000.8492912515,883640.1902452693,272941.279823381,345644.8842690787,14804.443076808937,-464931.41124401754,29320.955185577273,216093.9830552712,-349444.267389636,442013.393231574,-535524.6177620436,298750.97516287165,-383106.14915735833,-170844.9556513494,329045.6695604655,199141.75504750665,47894.402419409715,287176.2133959392,-389408.5336425882,505336.7138560293,340746.4589397372,7140.916760024615,275135.4929505321,781056.9924622786,-235483.87099641422,-415184.45449432405,425678.9020579271,-245177.46804555226,-665033.8104420591,201365.06008343818,118182.41685408447,226788.30234488146,117405.85209800815,148847.83263862366,-185170.83278397564,-272611.687297469,-468947.86548047094,316605.0587892607,154422.30669763405,358087.8736993242,941589.545143574,-111468.2899490369,594690.8723440538,829500.7120210114,-282373.63795263786,-827763.5770396651,653526.1265690681,248541.2660419764,-174010.95605593966,-368539.52991366666,662218.956831445,781469.8408518038,-245164.27656158805,625207.0934450044,90640.60316063603,-510762.85687854374,570756.8725422965,713345.1338938521,-358779.31754857535,544330.4058000245,691251.2468413068,345029.25932637043,728809.5934058563,768261.0058114524,162227.1053014486,-606813.49686991,333358.9516911423,-326956.0956154219,-381431.1263770838,-79139.06552774459,214949.03372177994,-845031.7152631683,546170.5580734457,321100.03916146327,-495820.36890374776,157313.6445608223,-521958.45253650984,-153285.57132204995,-507883.0599364983,-230168.09474432375,-171102.67088466138,-259065.45849508792,-117098.81737216283,-143912.8031356642,283101.03609488206,-7292.726183968596,-440950.6663067774,721939.4742064909,74101.7639178941,66461.3943842426,183105.1439172253,-638261.8897331273,773886.9265656858,622819.5485782186,163705.35003658477,964485.5619440307,547158.1107200328,640234.030938827,523702.68576812604,-306818.1426572255,-662304.2972103334,61247.24640106782,-186485.15163211245,177312.08085532766,-314610.7115277946,346851.48519595293,-46491.63670952525,-25018.895314423367,-431197.9844921916,407348.33853222243,-862278.5153458621,743315.6561568133,828494.9549760511,38684.52637852542,727779.2210388202,-502163.77948681125,209941.63850635942,383341.26004744787,-351911.66343659954,343060.4816366015,106325.7266621862,1036725.1428597919,-124590.57714532269,522118.86125859665,1022672.6804317441,705496.8606837206,-129543.81486962177,-210192.4748647958,-508313.8228839589,71465.72007885017,544570.3316289159,488371.72688769456,-155226.55699404515,317305.64420259744,316011.48648696346,318465.54992672894,73406.04396293592,-232363.45141874533,-370521.3562296806,-171119.8985881526,-861906.3132369376,-232003.87859519152,-579675.191091272,58.977694855537266,88087.7888471554,197580.97555919923,126931.64178822469,111069.03006321611,1112539.9577820622,898334.3622699743,-183108.60218272498,-371609.65469568083,-392461.3483679537,-500290.51628979435,350077.65946924826,-316478.194059066,-82148.91051367251,-212348.08309076726,204504.29603829933,376189.1479237396,1013657.744361436,632140.9209745033,504771.3443592051,1405030.2778379857,-29962.39012659667,206344.96880875994,1050337.8143428396,999225.0054468187,737064.2043726014,1029276.732417847,474947.13335233694,464013.4099423182,-428431.8472250886,525886.072612809,307041.80404837755,62251.895089050755,432672.24922444066,-751351.8001483278,808934.7484004935,-833873.3899899661,-75584.78188455943,-12841.543083941098,282396.4256284642,-264303.8602972417,-246751.89669571188,445944.68301659916,44405.126075516455,614556.2975665419,421745.2177526164,345905.70014956454,573220.0989212077,209259.19672719995,-322763.90066017024,236286.2006295137,-232285.26354659116,403470.1027273326,-85580.70211215178,-129334.9201708287,-150885.069058178,333410.6758758249,-86163.71513078595,312554.6498755128,267325.709266196,225620.92656400753,48664.66363405716,31463.774205981288,769219.282027754,259062.6552770147,69482.7959754304,241060.49719664687,123639.7015234381,48498.769972791895,36122.19684691075,-147096.8400796,804726.2735477695,505110.5880796267,418750.3471566993,80118.56002166867,665862.3132734639,-105676.99994916609,106694.24187629158,19025.9641235373,240112.19617154356,122567.90577099752,-628164.0714025162,225295.90818509692,408420.227925634,-277013.97362195514,215780.02086805087,246201.55091780866,612766.0739254202,-648773.1068729069,-309170.84333860315,412104.2461615419,-6435.9499624772,113343.57246245677,656092.224670196,-473487.9227695763,169919.10171134956,232667.86083570635,-355907.5921482453,-202644.16823505284,463566.8717660387,-515056.50618275534,146060.64861515444,487777.6724694548,-266472.1781269233,391371.5568861081,415571.08588416455,-92139.69347208412,1173281.9287442705,966923.3761655311,236918.82135438453,-416664.42965087295,384719.4509696737,-423777.08910588035,496232.0589727515,666232.8159738146,315175.87859050045,-496525.66009596875,474541.7142600408,541017.4500797847,165287.5754249529,-379340.7165975659,-326487.6303527076,-151594.81947993534,612721.4403766976,-374439.9016124052,-519291.3006232702,549987.3320107856,382617.0455227196,276605.4817638127,628218.6413559364,-150607.6693783896,915391.3084354214,772346.6158427517,1072951.5786638749,442433.87125862297,-335040.5924455933,567314.9611737104,88355.37658586027,-28564.88967361441,428697.5186488698,264126.3708882639,-287176.10714025376,557835.1024244493,950953.0977891521,941311.8411774989,188794.05225491337,107241.66903830552,586937.635351277,1330842.370072118,-144371.45283317566,391208.3955625077,99870.30954989558,184116.3469829266,-200458.27430448402,162991.94981180876,1553947.3474975466,201743.56982287206,-193858.46659990167,420669.300620351,471185.2419263809,454517.3589076032,-165370.94890556578,800213.0882878196,-601133.7225366384,-206212.02249802742,90315.52766981395,265428.55808098055,908920.4619229212,640119.3378102398,-62883.70034033898,1210798.1088524107,30299.02342053037,278395.3773302641,250592.90847409097,81707.34085235465,-7287.450671500061,68578.70285394182,349064.01899593696,386704.59088254254,189975.143522589,-1007496.248202618,268901.9880461083,-239944.80436600884,251382.56441947632,-128919.11715260567,177720.73091880325,11444.00936077442,-549392.4005613467,-889979.159271806,585836.1151524405,1419385.8588642045,199729.19498276897,-561065.8197028749,273903.32586663356,-561823.3465218125,810862.529030693,465149.12728245463,-238880.4947355115,-24488.959234315902,-544825.853058131,228529.76362202875,360242.87939241854,340311.24427433684,107153.85541072092,802305.9482988939,291385.76807577675,987064.3621784523,270870.41859737225,647381.0757650007,-295006.1073523299,224060.2561435611,-399927.7394366353,132608.6423483668,471790.3414003118,-165333.9266301426,538864.2291433169,305116.3436368434,249988.23470815085,-533616.5511959912,611174.366193194,557739.0816629175,-241197.98232145887,-85712.64547679806,1305439.6960775894,-198779.68779687118,99213.77402215684,209753.3553535561,-600706.6108155281,453064.70523765427,109283.10503460467,606749.0593231828,757977.1719188769,-372996.8830867801,-1023544.934355191,552581.6168474071,418057.5447794418,-890993.0817818982,36215.573433072306,664947.7883328986,386296.63331612386,-163634.22605755087,50048.850436799694,202746.02616230585,166192.32740336843,550311.3650049604,23277.610035460908,144930.83335740212,-3303.095892345067,-152002.82919156458,-54393.067041188944,1163625.745594385,497124.405239725,324033.14055933664,426362.5916964896,-436017.99155782023,722176.4474605778,-168710.23138610693,386839.02912168857,752851.171471653,323146.8258575746,254539.66242782306,583943.9583953307,655400.6028540004,291405.5656921463,-59752.03292310098,48387.22024553269,-245600.5024369047,681059.1094610835,558351.9127793056,496119.4984904155,-320945.0448851986,448622.3271850897,-278552.210508673,-396973.30053813616,-101250.13608262921,385272.56847084546,92603.69821254909,-552336.4991893396,-389866.0102744787,-359608.81239271583,684354.6744608385,403723.4030055809,-364630.41605349537,350165.791706905,71562.50094211427,-274427.1333826124,333471.12751714606,-85225.5097659002,149404.3002571729,546142.686350835,161655.5230897176,1075017.6500374004,-201624.90999404015,179421.81278234953,863864.0948700453,293677.2233574316,-176167.77699269308,132397.33458062774,37192.906573012006,-784201.3274440919,-6873.764356112573,252229.0223887586,97172.00560407853,39370.810540614184,439271.453964734,-524968.2821638784,791954.2540373472,4610.7175732264295,-379808.03536911216,-178396.465624962,741160.5604855143,-234481.66034599068,614698.297439774,-102495.52137564728,-124621.26688109292,539950.2954300116,579548.021235955,368051.2071157382,-138744.69790367363,448372.31997714285,383675.54141333234,148851.6730826241,740060.1377707529,279598.8379984936,241623.65043767123,335562.766258202,-175006.6118472875,-251003.6473433124,527740.5604637004,180596.31218236266,-168678.29128320515,734734.9213230852,-85147.31315599615,-83941.51561763929,-557479.5662965705,105706.19657276338,-374418.6828727033,-102325.0919575966,206329.82171785925,351335.1894288687,-101080.91666091047,317340.81028056145,181660.61786915828,650798.7015361753,474830.493998711,1096170.8964166204,83993.6325216163,506778.99094022997,367458.80531059904,402715.2289959807,532893.4700253098,104027.80512546841,148817.7319236528,-391518.9589229049,-144801.65636857925,-283382.3653848814,269875.5525907744,361225.0131789055,-55269.77471525315,605719.0181599697,448084.46655528527,-95626.39378931327,762473.8421702608,121104.17705377052,385201.48829867365,180962.74946155073,222534.31285674218,987603.0597846471,611475.7736794283,56220.011010751594,-496891.0048400499,115586.55453089182,951723.3317857324,432067.6208839505,85808.10517404182,443640.07527640695,692596.0887838402,413131.5973859513,594714.6960502397,552887.2418507761,93471.47686069505,-294977.45271409536,959501.5169993411,1025213.7422997234,33853.04852355365,185347.40514013497,28313.534607470036,874928.5418484146,340495.38161129877,1092447.0914305155,198260.20179643203,146999.73575087497,-650855.4780348022,856781.7531132116,329170.45107711386,377245.3958702185,314346.6251592892,-310442.7255603536,435593.0947476807,886196.2107463707,223422.3729858799,-368691.69709994225,-550181.2644093619,1227049.011945474,1086426.8041570825,709301.7073185192,463096.5263293488,814812.4649085766,419854.41139290575,256802.1190354582,724437.5870092725,-189214.1207624157,517755.0136349099,465429.429236806,502800.24899498513,-92549.22979584616,-106120.84391360125,1380055.9240373687,-89837.30377990566,386886.5384968342,-83770.43856317177,-48491.43646972347,370023.0387407183,623535.19667516,-24979.255582759622,132798.24260084378,-101232.08856903063,443724.8577953731,-447923.8431634577,571883.0048026333,382033.5047913627,-145743.531196916,375254.2846909552,-317600.4773287317,328185.38858498586,633182.4452550872,117324.11920597916,17318.206783811096,-88289.17522582551,-11533.547966700047,-353232.5165161942,620339.1700290167,152012.1619348768,457347.1858207821,206523.29553425452,413212.598560967,-493540.1674436331,96696.34997068951,-270848.68957269564,1085938.9431970534,-183622.51378973294,700577.357007931,471670.9500190276,63965.78124725446,-477866.8058213135,-504151.65115646506,765242.4127330906,527313.2291927054,64499.621371183544,-33515.32476834487,-7732.59291596571,206695.6326427851,-191552.9844727046,23360.971572999377,-364462.40916965157,711163.188830378,129069.25808412768,-24678.22368015861,185697.7260185741,-64341.8987468346,426235.8866037503,7607.450627746992,228335.0453314269,-488413.09374591336,41511.83043725323,477875.2256713123,390893.67341793654,-4826.288273461163,137720.87451528152,-556680.431433789,519842.05563288974,306224.36060085753,1054608.3602324259,556119.8315352187,6033.189514248166,-315858.8670298909,-76437.97780287964,232071.65970432805,367147.3441144256,547413.922366072,533941.1118873591,921895.2075400762,-469041.427352258,103954.30436228449,42989.70653858129,309716.4167572232,-374289.3244761997,50333.22116517043,246744.95567193488,682451.6612845017,-308926.80821824865,393078.24584726896,146034.16740558483,-91613.31038506376,158854.08848082088,-119677.9614356067,-38122.96292464109,310450.6697839857,-148377.08707117056,821285.0710849613,266903.80268582655,570021.2321045836,438492.0064976518,133589.4547171509,11604.516179394443,-20192.319833163172,956377.3575571119,167796.45234838827,715378.3093544329,548404.6360255023,-726752.4242328894,600266.2768358369,259624.29227703298,-426020.3014696371,813809.1383097656,371648.79488946963,712397.8452514773,88387.68523582118,379528.0824446273,502662.8219190354,482566.44282413274,939680.2341858996,600673.5469541824,38999.75152464816,-454835.03968674876,-225518.3527900786,975959.241582315,244043.43329725647,1206905.3933279682,141424.38937819703,132403.37617606064,826262.4308520923,581496.4205429815,88732.97339250566,-114853.42034392711,655347.7310980526,63628.09968129452,394981.5735346498,133349.40902270842,25324.276224075817,-180723.45631511183,118540.78389873682,545421.2932928947,505639.72650637105,-140111.98306525266,80732.60120887682,-24501.999612623826,-494970.9426383178,-312946.5067438674,126950.79983030865,-188942.23227800755,185375.97238843516,670465.312872624,866076.5024300958,550084.8659666032,-22815.07970895106,715819.7807718404,342644.36522858264,-324926.73077270435,416030.72978423117,-236707.93743438134,855323.9640766848,924214.8051919411,-473820.26453778334,-74668.66318746516,722535.7550308793,103637.79073147802,-282900.82065685093,-154257.7693753494,-8505.465641487855,31185.633709015325,187582.47559292288,22133.190373172984,520734.6484511872,400906.4597686152,-264455.7963142344,-164234.93547481624,699244.0004561795,545119.6843153387,724496.8991557681,253803.41571672168,14816.07425167691,110275.06035067886,58875.01353226323,383565.63956142357,156261.51751882164,-289074.81656489894,55285.20488273678,97379.6874411651,-473715.71761481976,1097875.0346427145,347050.9019314316,610711.7783116861,760507.6106652995,843675.3881618511,411800.052538689,-11237.730740594212,655216.2219444592,132570.3824501289,358563.19177105976,435879.1256856513,360092.651651904,-30583.40436986182,1185890.3795957193,926160.8340356201,337728.96408604505,776863.3804460047,317664.0766776074,985501.8314356357,-402595.80427850084,-18924.426322195213,265966.71016621916,447785.9572792277,140624.98590587545,149188.17298331158,469276.8068836266,349946.50481048785,561992.4327995961,474573.41342448164,265520.4383288836,-78692.18478271086,841258.0057857418,193168.69006113568,18874.405843606684,-317228.58764009085,-424948.6211729031,587141.7076378171,-16779.710226806346,1012815.3208059501,-278342.9076425154,926637.5083254036,365929.14632154536,37348.47821932612,-14916.707448767498,-109630.22875655675,683040.281618801,-333066.6212354116,227962.6866008439,585877.0178901735,-129087.5067567071,-335962.4262963692,411221.01827545464,105513.353359784,-144034.6537643331,30592.15363495797,-522660.5331454328,223447.94613083545,-184687.47739369003,780680.0860009096,-407045.6549198753,97431.20783841377,497564.22252574936,-433003.6742278272,-1049520.8717022855,-109974.75368890772,42032.89536901796,8755.283490662929,628948.303629891,1153924.3342192364,-785210.4315049318,83988.78935387265,744100.8719346887,170241.10738176527,860452.0693372125,468596.85403533606,-392664.0180852036,769922.239372652,-260095.4280227935,-485160.71364914067,640390.0015586321,-4061.912566799205,992266.7976475218,-582010.031528831,877061.5868872898,-226632.64241467137,7374.929376437329,590321.528420581,67266.5588232819,811656.7778546028,19622.00100572128,279923.04244646477,174675.1190789519,210193.72895332938,155838.9324099617,-21229.461625264492,387009.3195195468,843526.68517263,214866.7177868723,54589.519644417334,-283580.6328684795,628737.8011006834,184111.83746321918,-306541.8885951601,323673.5097312415,30407.89983477909,447444.2415886596,-109181.0960627892,395170.5062599252,819849.2259623692,186100.80677324766,611436.5237701936,-499148.7552301651,74464.70173718175,616950.1004288169,134230.0381300198,-25694.970769473817,222562.30553898867,-675365.5681689638,215689.4058174477,664822.9536586441,289119.87773668347,-39932.01585521642,390960.0584180169,-106927.56157984445,552447.3096080215,-448646.4717787043,-304151.1595802959,535756.3077859371,-362996.5425072517,-127477.6534419111,261984.8792703906,500987.4386196621,-125321.4758043699,195060.6643465357,-941540.9600765815,-55545.53880964173,14731.454756827094,257319.99676775327,892160.5204579979,-14630.97123171715,-105198.20470739622,809033.5633797199,160883.23944871267,-674402.2807655097,503992.6930953213,919125.1240950446,695875.788873327,126805.52674818691,-152008.5888534491,-396096.34506209753,-303042.90765064256,-413368.2142310543,40400.98946262989,-285859.4349162914,-287268.42877223063,529732.7410414908,-289841.1271435325,1506683.2362183803,690325.4552004407,240577.52711307583,696726.8811044786,-115778.87316070264,-616814.7454690686,1019148.1703072852,271696.04010992823,626131.8261178262,495812.4288729904,32256.771164200734,399226.393043702,-79897.9757728437,221309.93545939913,48748.08602299355,1096683.642993129,32031.008513041306,199746.6912119193,-248451.21962405276,637261.04953556,-4587.28652892774,193044.5494812606,221255.1318106884,489833.4289284176,-258196.55362414056,395711.7975531155,-33440.971513540484,114973.47569403192,934348.1400285498,66949.38516182639,614769.3678365531,-260912.0264084502,646927.490909094,-644905.2920907908,97022.52903383132,1176463.8933336595,431715.3887770758,251625.34452526039,-550502.9832581624,84873.76628576778,716659.4400119935,-86433.78788324399,762139.6584009137,586370.2266898346,1230541.9277698924,231436.0248998059,104735.26568868896,-116337.86264734482,756925.5226240321,353894.87708117627,-557683.4423486162,-146956.72312380606,-278658.55987531226,-206639.33839406446,1301727.5061956942,134328.67707414506,527743.437766315,832867.7767301109,418799.5620634714,539272.7036316157,696906.2117589908,-80971.3851333526,772599.2235057577,517603.5992852696,-549235.7751267729,246036.55267850822,-140200.75637658592,449590.9959866055,268161.3486640388,801203.8969666613,667510.674575652,361757.3536379002,605086.9369004951,-32803.53561689099,264006.142749941,152939.52226217464,468497.6294890228,297501.74917311454,-247144.64224167494,666860.911676377,102291.21250975737,-24461.641294357367,80931.04123943392,-554241.5294203786,-258845.87732878,791930.1641117879,1134282.393971065,487262.39025247656,64632.82016623486,1050187.9431858775,979468.1558905961,245770.1205068538,83406.33334162878,204201.89414662216,-668256.443806468,247417.61400018493,533906.9735615049,-31723.368861543946,1258969.639607829,169823.0341491648,-370206.71829743264,-726464.3245992833,139368.16001222795,-87624.24434449803,640603.563527883,656136.167885392,837820.4545785212,861968.0757433008,-631021.6097139041,-136478.7459762618,-3211.3109601070173,-30213.846243643202,-122656.81637036428,-470677.90200343635,228129.41364340857,539282.2555158543,4052.186926940456,-47485.08278267179,299683.25637712004,61457.42765621934,-305756.8696153583,71185.38510904973,-571162.4466356114,-157400.83986763703,-252202.6834646226,294262.8027155474,-236210.7833951097,-290231.92778061004,-189217.57688771468,-450195.65717685595,276698.25814079726,-81478.02562065609,55292.79467663169,560935.1029756404,609309.6195404846,-536996.0465130317,-108551.32726754202,286335.28957638796,-334689.1058404492,436214.1432495224,-412844.00057405885,685055.5206999481,179258.75015932182,122331.41540097678,-582402.1150262798,816729.640599967,396767.9165033321,483495.8581168321,-388907.5856061587,-15044.019150966778,504867.0230364115,540283.5174905681,1105891.570929869,223820.67370557878,89835.92082201317,37976.602876908146,77699.6158152027,516870.35713976296,192008.08602944668,-82306.3359463131,941976.9651850238,981592.8522806652,-118304.96528686816,720228.5681593562,566188.0577218826,869516.6091353348,-825714.1495613484,-114284.32524422836,886284.1348820971,-242727.56622980768,629950.0846179337,286875.80208986485,-1096842.7193296677,-538827.0890319739,-120610.77148084622,-10005.502963856794,-247109.60294442484,351242.5360926548,-455479.59442352597,125042.64645574475,1100848.218780462,164226.68757916754,-583948.754688194,1384898.2108315807,288651.7381828814,-414406.45549055515,804056.3600348635,668051.2216974096,799145.9293192071,-174861.3240556256,434752.248202831,329470.5968423318,302436.1540766158,509879.5052592228,-836125.8158840924,424078.42071781494,-74989.58593638055,1147126.3296694933,-640611.7185503002,103216.48395874305,-3931.5329912449233,-569535.507729731,209315.730163523,33513.440396778286,-482660.81235179584,924687.7628949117,-827723.0232431106,273541.1698943069,173156.7332509458,-388817.8758587665,1044103.9472318334,220553.22609686106,-71209.11620872281,538240.3068177556,249193.69626288908,603409.1363964528,-278820.4129413245,113522.90913711907,-37777.18025999889,340505.92110722465,462633.6301500276,533757.7859593728,-246635.8937339317,466647.6553379884,180913.54467909224,-201689.0887011108,-267335.3463365822,618013.4074962148,587261.5100313765,-312675.4258943368,54243.721783507615,800880.3304808126,119140.73072153283,467762.90057391627,186976.4922224786,-404379.84720236855,-366810.49511632277,914651.4929361008,-113975.7924250518,382058.8287605187,140079.9422682044,755249.3141450542,960773.851257951,-355686.7287555705,-264313.1860375367,-444005.8490246739,-22632.191368090454,-576868.8833375587,553429.8586542322,-588765.1049595159,107051.43575437646,425184.7060165494,-8133.4052182813175,-46517.54148526862,-420507.06627007667,-237931.2287589768,-83073.89068118064,219387.1435395456,-771900.2820892208,274268.52004490746,414897.8650670708,93908.74956174474,368873.9640846392,-52607.73485649424,187962.56624859618,822009.4144628053,-143888.9382799561,266975.3824483999,-171651.71294398326,670499.0493549597,230383.73596549733,-966221.5938080274,308208.31634886004,77858.55615433678,340060.5909046023,266616.9647864299,162539.67600428872,-181522.0546432496,574857.2311853087,-623492.4369927091,-424989.3010397847,373973.88634745916,439976.85998923844,207357.0625390329,-57514.354444507975,209898.1347954739,-292528.2299312679,494218.4337897147,-231844.88303132728,958897.9980847226,-234080.2612302578,-344197.4492921359,242037.70999782346,-79740.12812809553,626227.4427841147,605480.1889639851,-15589.869954444468,419469.7616918897,-223985.59152158536,454774.9728963701,-452633.24480874836,702144.8546822593,566501.9827365298,290741.9420116963,-81707.09969765972,1161508.3661592733,258290.8647665847,-32158.39912588382,347102.1671586833,528662.7840792988,-66953.31877999753,262813.2993899039,75984.78180120233,-325590.1793608391,-444526.3686036067,230238.32154390123,-490404.0736415107,-97240.53541065473,60996.86104566697,141372.9759961809,-247012.7256235811,169173.72607306298,-19970.795029097702,130310.94671202498,-1199.3637788114138,119889.73082109913,-283946.61949708546,1082681.0629336643,530246.5427559665,202689.02644122578,734382.1471188413,60946.806226912886,419495.0818999107,734420.1539650913,333685.05048886966,372492.7172052632,127116.37364923675,29603.452989334706,513446.49969590595,32557.094501260202,-497480.73564944323,544079.3148439508,757483.158556439,394021.7469858816,-835274.8124443134,9386.392482043244,-143999.0001391396,297298.17664069217,351826.0919187907,68789.9137139651,205796.40960524138,-315801.8257705034,111463.13577311346,-129364.1883304012,987133.157222067,647069.2223423931,487143.9834487885,28530.29244096577,681930.0504068397,625111.4164183382,-499086.2601496731,223744.37567027658,530919.1027510287,49995.08361257566,659625.91589869,745231.9845551867,341684.480071987,986135.1035947907,111435.3378300597,871886.6911643948,315675.1293334882,219947.95254505984,919873.2211940316,194170.25570381293,1027579.3497615983,-187592.59893174656,603530.877402592,278785.3366920417,-270623.87864519376,915290.2939077085,-98040.59372979216,-217939.725582222,-67203.80854407931,274187.5028266688,384406.7736159037,252166.9539564862,341372.67663407,711920.6344597894,467373.2491763127,-636996.6012696614,-470188.45018127887,922261.9915191433,-322010.8605033476,573120.3220810452,338777.0608246564,407817.55712010106,966775.7131393752,253508.7674585525,247140.36734144296,-3553.135368465446,-377694.27624000236,182919.35618165182,-243061.44543434493,491774.0450830064,412123.9921165649,-158061.06116818963,-250141.2244495498,290433.0384991029,338783.5494696018,847023.6286969306,276287.2545617479,-412443.88082800014,-760605.1797132404,372750.32334260456,103473.77680288116,722078.1080394033,-18493.913562277332,-332279.2365322332,-34715.943021170795,-74146.60780830588,934440.7567849495,599297.6986864959,391163.0820749365,340637.7901024157,148612.245294407,70224.26413612068,754426.924451849,-336009.8516916218,239779.8237642371,-439795.38880318124,494430.38914820505,710212.8183909361,-190783.89518447174,1405559.003116175,-657048.823001293,430161.27199337026,377764.40160367964,510557.9412119952,576389.359128267,1056986.5205929219,140408.47162012476,-253200.94259718154,22789.012317343615,399670.8462384185,74153.90045572771,295984.9382456965,-452304.20164667675,691037.0405212352,595166.7139068539,-182663.7370275068,655605.1716236458,-585428.6679660208,63415.84810075071,-619345.3969768481,-242508.03347445047,-43460.572729710024,558607.3934139749,3262.6165346233174,-298811.10174907837,-114015.51926402841,-113355.6192121678,571912.763780456,332430.2094903006,813461.6568286782,1192715.4326448168,335648.4106677277,294510.978538875,942108.9380084341,-252053.88174064364,432067.93250841927,42846.81571435835,217574.2940882584,505274.6056293114,-722985.3476372948,-193395.40550824022,483266.3235944924,-257339.60052404413,-258544.52287548082,679384.6093306239,427485.125614251,-1048325.3838076594,268072.6905447133,-471559.29556367313,-198034.33270844584,4548.762457480654,739225.1970191961,283052.1690126066,454121.1980553325,505122.3025886505,-135676.37623763178,305318.73498586146,763824.2269895244,467950.9298082753,223496.31894858554,143332.38760425057,-41990.44814000558,490669.56295507913,217010.66552621173,820567.4493152606,-137622.21800571214,1089966.504504892,150096.53740499076,-9232.400961911771,132125.01682735607,-273302.28905415814,236383.2360500209,307212.5604794738,-190550.74612351554,387515.86147236405,320414.16114442376,180466.48593078647,-473062.75157551095,350390.19795227796,850533.2234808081,-69457.3423001431,401393.2626113687,-168820.74524715124,1038317.529063324,-96475.74109851103,165446.75784333423,-346788.63752501085,-358893.0747925951,186161.45042799227,919422.0168845695,-196384.49583044928,449194.1089423876,322520.22339524236,99237.03015016858,-826309.9334683605,51136.7711040061,-93094.60495446017,609443.1818087185,159084.3260044721,727919.3073898638,856994.8530042744,-78891.40604374046,400682.5541344066,1358506.2106442326,395255.18083726894,648293.4175947937,-170632.40588506358,357689.148729641,-325615.4047986716,476247.56067202194,1123632.3746526218,-312146.301841639,152270.98051919183,483699.8523071124,59518.68428285094,686984.3184537101,791278.003006273,-130796.07543652365,-132709.41431320366,104364.72458490543,414984.19057754707,-436159.67413543584,945850.5186789744,806457.1740375836,-521155.3722466654,-86983.58574869484,785594.3217493566,560501.7684008814,401282.644890985,16614.347530813888,-511438.37577210926,865276.6913230093,-127833.445237556,1172011.756949257,-206233.23112057522,746530.2762196986,612400.1139522637,-577002.5271771764,678755.7681628326,248389.2832343541,-14652.650133881252,247746.70054829074,557399.7997622034,1308393.042597136,663253.0350335287,469011.5903814747,1009948.3163035414,97305.43510919111,-155527.37572617037,-297449.95174051914,1140616.923307183,290768.7790181921,-15139.11257581925,362702.12757688155,244920.39353680843,-195913.51964113116,495519.9517347696,-67627.34271330014,772227.9980149302,865131.7945586415,-213312.5195437679,305073.39670271846,720260.6191639705,-214101.88709208462,685785.001968564,84162.4647936942,947313.9469505767,704674.8332417072,165644.14871440083,258140.06316861184,-322596.658829961,555032.0025371113,261858.8033876242,190331.4634797396,-96029.9627376739,-300149.15758842044,690056.4405623758,122842.32649715198,169786.21917130705,704307.9454524545,-21019.553071195725,331097.8481923598,205366.99706756743,966956.6851813812,-161464.1789036165,471202.1179115721,-254157.5475513148,696518.0328965769,-291367.0072503458,-787931.0355452218,-67918.0566126206,-16402.287319815252,103737.09103563474,299066.360511024,-411986.6482330938,143350.16518512927,529624.5515180714,-290695.56896712165,404588.0447331867,636208.0584376906,-105109.49876521388,-105002.48861799855,-500255.0084323585,176081.21080411877,511413.6203252841,251893.13130951487,-539785.9779847823,162247.8285497413,396522.8725045007,848543.9471384301,421903.3995831013,23218.329103146214,934831.7152829231,266787.131798625,-218323.505494087,388965.3622889612,832516.254483813,1354307.6590948943,933697.694446018,273641.2441899306,-28230.209422244225,181637.09107895987,260144.41704071406,-236979.58674899256,-69730.4771254086,100316.98195284558,34007.9804308014,-471615.45082310075,444117.51523586316,404730.6136259809,-295829.9000563347,-172810.56917301286,-374639.35870777117,-135355.4749297127,-227883.83454326168,498197.22396298824,169750.887062212,-101511.6593009322,573260.9918263196,1264918.0237739459,669602.6055551008,-398474.9501134474,501031.92045871075,562885.5803577206,303633.4220910119,-453444.3002800818,-1205.9974816846661,-330214.363551585,-103870.9543814431,-223020.1750495215,-307588.5409921687,-134629.76931253914,91412.9636403136,726010.2720113783,639027.6885226117,73818.27034706715,433171.38051160565,755583.31380127,214440.99075411586,182710.82570246467,274958.8403772055,657306.5197194451,618824.9157406176,695137.8642057329,-204744.90884740464,332667.0324404342,174912.4816209874,146925.38226332702,893072.7729523429,-431188.8551465636,507978.59404466534,84131.57990896888,-286247.4549389528,50244.54154728167,-117317.27683538152,368453.7003503479,677544.0551027395,-510138.7646171972,-490086.20869058976,801397.5679491372,-468825.4169142335,270268.56341801723,323672.21608700184,89552.35749868164,400204.48082605563,1059985.6215089583,76918.99179532146,330894.8747367603,490637.6845627357,-201645.31521002343,446639.4112477582,345842.1751838713,-200841.9722148124,73874.10044590663,-103942.50116731739,-356560.6595374346,694894.0516423597,588384.3657362601,93126.19461800391,359427.6822637818,506554.8842615718,387475.0546943825,467329.79320197506,436742.99535049545,-34724.044182079844,463404.45031241747,-113574.34499574266,82814.98676920356,-212972.13740385324,-178382.12668418745,320113.89874937525,-344697.82566986326,-290306.4892605236,97821.50559924822,110941.7753863018,331327.5577867152,636850.7370579196,-140372.17857998284,844590.1917024856,406313.67100544344,107002.45327792782,-618141.5062769435,983388.1368721845,441919.1713576196,208674.16677939938,-428681.32101833355,144193.78598411102,736194.7191382241,271637.76887134556,-323809.31422149576,235377.63421474677,224026.7533432264,477807.4651644351,-196219.85335967643,930650.607528863,-66585.20533980662,91624.34095798014,563970.1216179514,967054.5715031554,361332.22013430716,175208.2891858113,776737.7537238528,-78570.14841282181,534634.2505645836,70407.35899108695,6386.206363186706,-71158.02127294941,1081228.8710250333,571993.9353554947,454979.2787996982,1020613.754638717,632517.9334234525,-342526.7084253933,-121569.56523423968,-20192.81791717466,64781.315136676654,-631873.0612180186,-107775.45116520254,593324.2975612776,95202.87486550538,-468035.9905904648,-566315.1008518008,778217.5521694925,47374.46789077949,110083.41208653618,661463.9273317717,567154.0479700533,484966.37262309575,163558.11985611496,335372.59057436,849545.5849960744,-94163.23094330076,437933.32574026193,754094.7575422693,-145576.9642677009,-373127.1974076717,602602.817642781,203100.77847679053,-353561.28318970324,778097.5756945005,490710.40747220954,-246631.35541086644,633239.9094749135,-570.0874299132265,-33949.42168990942,254655.40662233764,-45270.692608084995,61438.150981036946,455232.2055136943,304862.1761082243,-618232.6947707632],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Simulated Opportunity"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

p05 -537973.5755449401 
Mean 188798.3910280675 
p95 926086.1889873659


In [20]:
# A little magic to automatically write my blog :)
import subprocess

subprocess.run(
    [
        "jupyter",
        "nbconvert",
        "--to",
        "markdown",
        "--output",
        "~/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md",
        "rent_vs_buy_blog_post.ipynb",
    ]
)

[NbConvertApp] Converting notebook rent_vs_buy_blog_post.ipynb to markdown
/opt/anaconda3/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'text/vnd.mermaid' -%}
[NbConvertApp] Writing 1158386 bytes to /Users/jakeetaylor/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md


CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'markdown', '--output', '~/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md', 'rent_vs_buy_blog_post.ipynb'], returncode=0)